In [7]:
import sys
sys.path.append("/anaconda3/lib/python3.7/site-packages")
import numpy as np
import numpy
import nltk
import pickle
import pandas as pd
import glob
import os
import re
import operator
from collections import Counter
#from read_xml_files import *
##from s_stemmer import *
from nltk.stem.porter import PorterStemmer
from nltk.stem import LancasterStemmer, WordNetLemmatizer
from itertools import groupby
from nltk.collocations import *
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.cluster import KMeans
from scipy.cluster.vq import whiten
from nltk.tokenize.toktok import ToktokTokenizer
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2
import xml.etree.ElementTree as ET
import pandas as pd
#Import scikit-learn metrics module for accuracy calculation
from sklearn import metrics
##from pattern.text.en import singularize
##nltk.download('vader_lexicon')
##nltk.download('punkt')
#import the relevant modules from the NLTK library
from nltk.sentiment.vader import SentimentIntensityAnalyzer
sentence_tokenizer = nltk.data.load('tokenizers/punkt/english.pickle')
word_tokenizer = nltk.tokenize.RegexpTokenizer(r'\w+')
##tt = ToktokTokenizer()#keeps the url as is...................................................
from nltk.tokenize import TweetTokenizer
tt = TweetTokenizer()
Porter_stemmer = PorterStemmer()
Lancaster_stemmer = LancasterStemmer()
WordNet_lemmatizer = WordNetLemmatizer()
#nltk.download('averaged_perceptron_tagger')
##Train_path="D:/Data/PAN2015-Profiling/pan15-author-profiling-training-dataset-english-2015-04-23"#path for train data
##Test_path="D:/Data/PAN2015-Profiling/pan-ap2015-test/en" #path for test data

##Train_path="D:/NLP/PAN2014/pan14-author-profiling-training-corpus-english-twitter-2014-04-16"#path for train data D:\Data\PAN2013-Profiling\Training\en  D:\NLP\PAN2014
##train_truth_path="D:/NLP/PAN2014/truth_train"
##
##test_truth_path="D:/NLP/PAN2014/pan14_test"
##Test_path="D:/NLP/PAN2014/pan14_test/en"
##Content = open('results_of_clssification_all.txt', 'w')

M_train_file= "/Users/catherine/Desktop/NLP/PAN_Datasets/PAN2014blog/2014-blog-txtfiles/PAN2014_blog_male_train.txt"
F_train_file= "/Users/catherine/Desktop/NLP/PAN_Datasets/PAN2014blog/2014-blog-txtfiles/PAN2014_blog_female_train.txt"

M_test_file= "/Users/catherine/Desktop/NLP/PAN_Datasets/PAN2014blog/2014-blog-txtfiles/PAN2014_blog_male_test.txt"
F_test_file= "/Users/catherine/Desktop/NLP/PAN_Datasets/PAN2014blog/2014-blog-txtfiles/PAN2014_blog_female_test.txt"

def all_txt(M_txt_file,  F_txt_file):
    with open(M_txt_file, mode="r", encoding="utf-8") as f:
            txt_list = [(line[:-1]).split("\t") for line in f]
            male_dict = dict()
            content=[]
            for line in txt_list:
                if line[1] in male_dict:
                # append the new number to the existing array at this slot
                    male_dict[line[1]].append(line[3])
                else:
                # create a new array in this slot
                    male_dict[line[1]] = [line[3]]
    male_content= [("M", (' '.join(a)).split()) for a in list(male_dict.values())]
  
    with open(F_txt_file, mode="r", encoding="utf-8") as f:
            txt_list = [(line[:-1]).split("\t") for line in f]
            female_dict = dict()
            content=[]
            for line in txt_list:
                if line[1] in female_dict:
                # append the new number to the existing array at this slot
                    female_dict[line[1]].append(line[3])
                else:
                # create a new array in this slot
                    female_dict[line[1]] = [line[3]]
    female_content= [("F", (' '.join(a)).split()) for a in list(female_dict.values())]
    print("len_male_content =", len(male_content), "len_female_content = ", len(female_content))
    all_txt_per_person = male_content + female_content
    #print(len([x for item in all_txt_per_person for x in item]))
    return all_txt_per_person

##print( [((all_txt(M_txt_file,  F_txt_file))[i][0])  for i in range(len((all_txt(M_txt_file,  F_txt_file))))]) helps to determine the gender....

      
"""
create feature vector from the training set.....................................................................................
"""

def WordFeatures(word_list, all_training_text):

    fvs_words = np.array([[author.count(word) for word in word_list] for author in all_training_text]).astype(np.float64)

    # normalise by dividing each row by number of tokens for each author........
    fvs_words /= np.c_[np.array([len(author) for author in all_training_text])]

    return fvs_words



"""
From the saved csv file, recover the saved features to be used...............................................................
"""
import csv
word_list=[]
##txt_files =[ "/Users/catherine/Desktop/NLP/PAN Datasets/PAN_chosen_tokens/2014/output2014OR.1M.txt", "/Users/catherine/Desktop/NLP/PAN Datasets/PAN_chosen_tokens/2014/output2014OR.2F.txt"]
txt_files =[ "/Users/catherine/Desktop/NLP/PAN_Datasets/PAN2014blog/2014-blog-txtfiles/PAN2014_blog_female_chosen_set.csv", "/Users/catherine/Desktop/NLP/PAN_Datasets/PAN2014blog/2014-blog-txtfiles/PAN2014_blog_male_chosen_set.csv"]
print("txt_files of features used",txt_files)
##for txt_file in txt_files:
##    with open(txt_file, mode="r", encoding="utf-8") as f:
##        reader = csv.reader(f, delimiter=",")
##        next(reader) # skip header
##        word_list1 =  ([(r.split(":"))[1] for r in f])
##        print(txt_file, len(word_list1))
##        word_list = word_list + word_list1
##print("length of features used =", len(word_list))

for txt_file in txt_files:
    with open(txt_file, mode="r", encoding="utf-8") as f:
        reader = csv.reader(f, delimiter=",")
        next(reader) # skip header
        word_list1 =  [r[0] for r in reader]
        print(txt_file, len(word_list1))
        word_list = word_list + word_list1
print("length of features used =", len(word_list))


"""
Prepare the training and test sets to be parsed to the classifies............................................................
"""
A=(all_txt(M_train_file,  F_train_file))
all_training_text = ( [(A[i][1])  for i in range(len(A))])
print("len(all_training_text)",len(all_training_text))
print("mean length = ",(len([x for item in all_training_text for x in item]))/len(all_training_text))
X_train=WordFeatures(word_list, all_training_text)
y_train=np.array( [(A[i][0])  for i in range(len(A))])
print("len(X_train)=",len(X_train), "len(y_train)=", len(y_train))

B=(all_txt(M_test_file,  F_test_file))
all_test_text = ( [(B[i][1])  for i in range(len(B))])
print("len(all_test_text)",len(all_test_text))
X_test = WordFeatures(word_list, all_test_text)
y_test = np.array( [(B[i][0])  for i in range(len(B))])
print("len(X_test=",len(X_test), "len(y_test)=",len(y_test))


X_train = np.nan_to_num(X_train)
X_test = np.nan_to_num(X_test)

"""
Perform feature selection using chi2............................................................
"""
from sklearn.feature_selection import *
##
####def select_Bestfeatures(X, y):
##print("selector = SelectKBest(score_func=chi2, k=1000)")
##selector = SelectKBest(score_func=chi2, k=1000)
##fit = selector.fit(X_train,y_train)
##train_selected_features = fit.transform(X_train)
##test_selected_features = fit.transform(X_test)



"""
x_train and x_test based on the new features using chi2............................................................
"""
##X_train = select_Bestfeatures(X_train, y_train)
##X_test = select_Bestfeatures(X_test, y_test)


"""
Perform feature selection using f_classif............................................................
"""
from sklearn.feature_selection import *
##def f_classif_select_Bestfeatures(X, y):
##selector=SelectKBest(score_func=f_classif,k=500)
##fit = selector.fit(X_train,y_train)
##train_selected_features = fit.transform(X_train)
##test_selected_features = fit.transform(X_test)
##    selected_features = fit.transform(X)
##    return selected_features

"""
x_train and x_test based on the new features using f_classif............................................................
"""
##X_train = f_classif_select_Bestfeatures(X_train, y_train)
##X_test = f_classif_select_Bestfeatures(X_test, y_test)



"""
Perform feature selection using mutual_info_classif............................................................
"""
from sklearn.feature_selection import *
##def mutual_info_classif_select_Bestfeatures(X_tr, X_te, y):
##selector=SelectKBest(score_func=mutual_info_classif,k=100)
##fit = selector.fit(X_train,y_train)
##train_selected_features = fit.transform(X_train)
##test_selected_features = fit.transform(X_test)
##    return train_selected_features, test_selected_features

"""
x_train and x_test based on the new features using f_classif............................................................
"""
##X_train =train_selected_features 
##X_test = test_selected_features


"""
Perform feature selection using pca............................................................
"""

##from sklearn.decomposition import PCA
##def PCA_select_Bestfeatures(X):
##    # feature extraction
##    pca = PCA(n_components=200)
##    fit = pca.fit(X, y)
##    Train_selected_features = fit.transform(X)
##    return selected_features

"""
x_train and x_test based on the new features using f_classif............................................................
"""
##X_train = PCA_select_Bestfeatures(X_train)
##X_test = PCA_select_Bestfeatures(X_test)

"""
Below are a list of Classification Algorithms with different distance measres ..............................................................................
"""
"""
K-Nearest Neighbours (average aa over all the nearest neighbour selectiopn)..............................................................................
"""

k=5

print(k)
"""
8. manhattan K-Nearest Neighbours..............................................................................
"""
from sklearn.neighbors import KNeighborsClassifier
knn = KNeighborsClassifier(n_neighbors = k, metric= "manhattan")
knn.fit(X_train, y_train)
y_pred = knn.predict(X_test)
print("manhattan K-Nearest Neighbours")
print(metrics.accuracy_score(y_test, y_pred))

    
"""
8. cosine K-Nearest Neighbours..............................................................................
"""
from sklearn.neighbors import KNeighborsClassifier
knn = KNeighborsClassifier(n_neighbors = k, metric= "cosine")
knn.fit(X_train, y_train)
y_pred = knn.predict(X_test)
print("cosine K-Nearest Neighbours")
print(metrics.accuracy_score(y_test, y_pred))


"""
8. euclidean K-Nearest Neighbours..............................................................................
"""
from sklearn.neighbors import KNeighborsClassifier
knn = KNeighborsClassifier(n_neighbors = k, metric= "euclidean")
knn.fit(X_train, y_train)
y_pred = knn.predict(X_test)
print("euclidean K-Nearest Neighbours")
print(metrics.accuracy_score(y_test, y_pred))

        
"""
9. scipy.spatial.distance K-Nearest Neighbours..............................................................................
"""
from scipy.spatial.distance import *
knn = KNeighborsClassifier(n_neighbors = k, metric= "braycurtis")
knn.fit(X_train, y_train)
y_pred = knn.predict(X_test)
print("braycurtis K-Nearest Neighbours")
print(metrics.accuracy_score(y_test, y_pred))
       
        
knn = KNeighborsClassifier(n_neighbors = k, metric= "canberra")
knn.fit(X_train, y_train)
y_pred = knn.predict(X_test)
print("canberra K-Nearest Neighbours")
print(metrics.accuracy_score(y_test, y_pred))
  
        
knn = KNeighborsClassifier(n_neighbors = k, metric= "jaccard")
knn.fit(X_train, y_train)
y_pred = knn.predict(X_test)
print("jaccard K-Nearest Neighbours")
print(metrics.accuracy_score(y_test, y_pred))
        
knn = KNeighborsClassifier(n_neighbors = k, metric= "yule")
knn.fit(X_train, y_train)
y_pred = knn.predict(X_test)
print("yule K-Nearest Neighbours")
print(metrics.accuracy_score(y_test, y_pred))
print("************************************ ****************************")



txt_files of features used ['/Users/catherine/Desktop/NLP/PAN_Datasets/PAN2014blog/2014-blog-txtfiles/PAN2014_blog_female_chosen_set.csv', '/Users/catherine/Desktop/NLP/PAN_Datasets/PAN2014blog/2014-blog-txtfiles/PAN2014_blog_male_chosen_set.csv']
/Users/catherine/Desktop/NLP/PAN_Datasets/PAN2014blog/2014-blog-txtfiles/PAN2014_blog_female_chosen_set.csv 1196
/Users/catherine/Desktop/NLP/PAN_Datasets/PAN2014blog/2014-blog-txtfiles/PAN2014_blog_male_chosen_set.csv 1636
length of features used = 2832
len_male_content = 74 len_female_content =  73
len(all_training_text) 147
mean length =  2787.598639455782
len(X_train)= 147 len(y_train)= 147
len_male_content = 39 len_female_content =  39
len(all_test_text) 78
len(X_test= 78 len(y_test)= 78
5
manhattan K-Nearest Neighbours
0.6153846153846154
cosine K-Nearest Neighbours
0.6538461538461539
euclidean K-Nearest Neighbours
0.6153846153846154
braycurtis K-Nearest Neighbours
0.6410256410256411
canberra K-Nearest Neighbours
0.5897435897435898
jacca

/anaconda3/lib/python3.7/site-packages/sklearn/utils/validation.py:475: DataConversionWarning: Data with input dtype float64 was converted to bool by check_pairwise_arrays.
  warnings.warn(msg, DataConversionWarning)


In [5]:

import sys
sys.path.append("/anaconda3/lib/python3.7/site-packages")
import numpy as np
import numpy
import nltk
import pickle
import pandas as pd
import glob
import os
import re
import operator
from collections import Counter
#from read_xml_files import *
##from s_stemmer import *
from nltk.stem.porter import PorterStemmer
from nltk.stem import LancasterStemmer, WordNetLemmatizer
from itertools import groupby
from nltk.collocations import *
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.cluster import KMeans
from scipy.cluster.vq import whiten
from nltk.tokenize.toktok import ToktokTokenizer
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2
import xml.etree.ElementTree as ET
import pandas as pd
#Import scikit-learn metrics module for accuracy calculation
from sklearn import metrics
##from pattern.text.en import singularize
##nltk.download('vader_lexicon')
##nltk.download('punkt')
#import the relevant modules from the NLTK library
from nltk.sentiment.vader import SentimentIntensityAnalyzer
sentence_tokenizer = nltk.data.load('tokenizers/punkt/english.pickle')
word_tokenizer = nltk.tokenize.RegexpTokenizer(r'\w+')
##tt = ToktokTokenizer()#keeps the url as is...................................................
from nltk.tokenize import TweetTokenizer
tt = TweetTokenizer()
Porter_stemmer = PorterStemmer()
Lancaster_stemmer = LancasterStemmer()
WordNet_lemmatizer = WordNetLemmatizer()
#nltk.download('averaged_perceptron_tagger')
##Train_path="D:/Data/PAN2015-Profiling/pan15-author-profiling-training-dataset-english-2015-04-23"#path for train data
##Test_path="D:/Data/PAN2015-Profiling/pan-ap2015-test/en" #path for test data

##Train_path="D:/NLP/PAN2014/pan14-author-profiling-training-corpus-english-twitter-2014-04-16"#path for train data D:\Data\PAN2013-Profiling\Training\en  D:\NLP\PAN2014
##train_truth_path="D:/NLP/PAN2014/truth_train"
##
##test_truth_path="D:/NLP/PAN2014/pan14_test"
##Test_path="D:/NLP/PAN2014/pan14_test/en"
##Content = open('results_of_clssification_all.txt', 'w')

M_train_file= "/Users/catherine/Desktop/NLP/PAN Datasets/PAN_tokens/2014_blog_Tokens/PAN2014_blog_male_train.txt"
F_train_file= "/Users/catherine/Desktop/NLP/PAN Datasets/PAN_tokens/2014_blog_Tokens/PAN2014_blog_female_train.txt"

M_test_file= "/Users/catherine/Desktop/NLP/PAN Datasets/PAN_tokens/2014_blog_Tokens/PAN2014_blog_male_test.txt"
F_test_file= "/Users/catherine/Desktop/NLP/PAN Datasets/PAN_tokens/2014_blog_Tokens/PAN2014_blog_female_test.txt"

def all_txt(M_txt_file,  F_txt_file):
    with open(M_txt_file, mode="r", encoding="utf-8") as f:
            txt_list = [(line[:-1]).split("\t") for line in f]
            male_dict = dict()
            content=[]
            for line in txt_list:
                if line[1] in male_dict:
                # append the new number to the existing array at this slot
                    male_dict[line[1]].append(line[3])
                else:
                # create a new array in this slot
                    male_dict[line[1]] = [line[3]]
    male_content= [("M", (' '.join(a)).split()) for a in list(male_dict.values())]
  
    with open(F_txt_file, mode="r", encoding="utf-8") as f:
            txt_list = [(line[:-1]).split("\t") for line in f]
            female_dict = dict()
            content=[]
            for line in txt_list:
                if line[1] in female_dict:
                # append the new number to the existing array at this slot
                    female_dict[line[1]].append(line[3])
                else:
                # create a new array in this slot
                    female_dict[line[1]] = [line[3]]
    female_content= [("F", (' '.join(a)).split()) for a in list(female_dict.values())]
    all_txt_per_person = male_content + female_content
    return all_txt_per_person

##print( [((all_txt(M_txt_file,  F_txt_file))[i][0])  for i in range(len((all_txt(M_txt_file,  F_txt_file))))]) helps to determine the gender....

      
"""
create feature vector from the training set.....................................................................................
"""

def WordFeatures(word_list, all_training_text):

    fvs_words = np.array([[author.count(word) for word in word_list] for author in all_training_text]).astype(np.float64)

    # normalise by dividing each row by number of tokens for each author........
    fvs_words /= np.c_[np.array([len(author) for author in all_training_text])]

    return fvs_words



"""
From the saved csv file, recover the saved features to be used...............................................................
"""
import csv
word_list=[]
txt_files =[ "/Users/catherine/Desktop/NLP/PAN Datasets/EB2014Selection/output2014OR.1M.txt", "/Users/catherine/Desktop/NLP/PAN Datasets/EB2014Selection/output2014OR.2F.txt"]
##txt_files =[ "/Users/catherine/Desktop/NLP/PAN Datasets/PAN_tokens/2014_blog_Tokens/PAN2014_blog_female_chosen_set.csv", "/Users/catherine/Desktop/NLP/PAN Datasets/PAN_tokens/2014_blog_Tokens/PAN2014_blog_male_chosen_set.csv"]
print("txt_files of features used",txt_files)
for txt_file in txt_files:
    with open(txt_file, mode="r", encoding="utf-8") as f:
##        reader = csv.reader(f, delimiter=",")
##        next(reader) # skip header
        word_list1 =  ([(r.split(":"))[1] for r in f])[0:100]
        print(txt_file, len(word_list1))
        word_list = word_list + word_list1
print("length of features used =", len(word_list))

#for txt_file in txt_files:
 #   with open(txt_file, mode="r", encoding="utf-8") as f:
 #       reader = csv.reader(f, delimiter=",")
 #       next(reader) # skip header
 #       word_list1 =  [r[0] for r in reader]
 #       print(txt_file, len(word_list1))
 #       word_list = word_list + word_list1
#print("length of features used =", len(word_list))


"""
Prepare the training and test sets to be parsed to the classifies............................................................
"""
A=(all_txt(M_train_file,  F_train_file))
all_training_text = ( [(A[i][1])  for i in range(len(A))])
print("len(all_training_text)",len(all_training_text))
X_train=WordFeatures(word_list, all_training_text)
y_train=np.array( [(A[i][0])  for i in range(len(A))])
print("len(X_train)=",len(X_train), "len(y_train)=", len(y_train))

B=(all_txt(M_test_file,  F_test_file))
all_test_text = ( [(B[i][1])  for i in range(len(B))])
print("len(all_test_text)",len(all_test_text))
X_test = WordFeatures(word_list, all_test_text)
y_test = np.array( [(B[i][0])  for i in range(len(B))])
print("len(X_test=",len(X_test), "len(y_test)=",len(y_test))


X_train = np.nan_to_num(X_train)
X_test = np.nan_to_num(X_test)

"""
Perform feature selection using chi2............................................................
"""
from sklearn.feature_selection import *
##
####def select_Bestfeatures(X, y):
##print("selector = SelectKBest(score_func=chi2, k=1000)")
##selector = SelectKBest(score_func=chi2, k=1000)
##fit = selector.fit(X_train,y_train)
##train_selected_features = fit.transform(X_train)
##test_selected_features = fit.transform(X_test)



"""
x_train and x_test based on the new features using chi2............................................................
"""
##X_train = select_Bestfeatures(X_train, y_train)
##X_test = select_Bestfeatures(X_test, y_test)


"""
Perform feature selection using f_classif............................................................
"""
from sklearn.feature_selection import *
##def f_classif_select_Bestfeatures(X, y):
##selector=SelectKBest(score_func=f_classif,k=500)
##fit = selector.fit(X_train,y_train)
##train_selected_features = fit.transform(X_train)
##test_selected_features = fit.transform(X_test)
##    selected_features = fit.transform(X)
##    return selected_features

"""
x_train and x_test based on the new features using f_classif............................................................
"""
##X_train = f_classif_select_Bestfeatures(X_train, y_train)
##X_test = f_classif_select_Bestfeatures(X_test, y_test)



"""
Perform feature selection using mutual_info_classif............................................................
"""
from sklearn.feature_selection import *
##def mutual_info_classif_select_Bestfeatures(X_tr, X_te, y):
##selector=SelectKBest(score_func=mutual_info_classif,k=100)
##fit = selector.fit(X_train,y_train)
##train_selected_features = fit.transform(X_train)
##test_selected_features = fit.transform(X_test)
##    return train_selected_features, test_selected_features

"""
x_train and x_test based on the new features using f_classif............................................................
"""
##X_train =train_selected_features 
##X_test = test_selected_features


"""
Perform feature selection using pca............................................................
"""

##from sklearn.decomposition import PCA
##def PCA_select_Bestfeatures(X):
##    # feature extraction
##    pca = PCA(n_components=200)
##    fit = pca.fit(X, y)
##    Train_selected_features = fit.transform(X)
##    return selected_features

"""
x_train and x_test based on the new features using f_classif............................................................
"""
##X_train = PCA_select_Bestfeatures(X_train)
##X_test = PCA_select_Bestfeatures(X_test)

"""
Below are a list of Classification Algorithms with different distance measres ..............................................................................
"""
"""
K-Nearest Neighbours (average aa over all the nearest neighbour selectiopn)..............................................................................
"""

k=5

print(k)
"""
8. manhattan K-Nearest Neighbours..............................................................................
"""
from sklearn.neighbors import KNeighborsClassifier
knn = KNeighborsClassifier(n_neighbors = k, metric= "manhattan")
knn.fit(X_train, y_train)
y_pred = knn.predict(X_test)
print("manhattan K-Nearest Neighbours")
print(metrics.accuracy_score(y_test, y_pred))

    
"""
8. cosine K-Nearest Neighbours..............................................................................
"""
from sklearn.neighbors import KNeighborsClassifier
knn = KNeighborsClassifier(n_neighbors = k, metric= "cosine")
knn.fit(X_train, y_train)
y_pred = knn.predict(X_test)
print("cosine K-Nearest Neighbours")
print(metrics.accuracy_score(y_test, y_pred))


"""
8. euclidean K-Nearest Neighbours..............................................................................
"""
from sklearn.neighbors import KNeighborsClassifier
knn = KNeighborsClassifier(n_neighbors = k, metric= "euclidean")
knn.fit(X_train, y_train)
y_pred = knn.predict(X_test)
print("euclidean K-Nearest Neighbours")
print(metrics.accuracy_score(y_test, y_pred))

        
"""
9. scipy.spatial.distance K-Nearest Neighbours..............................................................................
"""
from scipy.spatial.distance import *
knn = KNeighborsClassifier(n_neighbors = k, metric= "braycurtis")
knn.fit(X_train, y_train)
y_pred = knn.predict(X_test)
print("braycurtis K-Nearest Neighbours")
print(metrics.accuracy_score(y_test, y_pred))
       
        
knn = KNeighborsClassifier(n_neighbors = k, metric= "canberra")
knn.fit(X_train, y_train)
y_pred = knn.predict(X_test)
print("canberra K-Nearest Neighbours")
print(metrics.accuracy_score(y_test, y_pred))
  
        
knn = KNeighborsClassifier(n_neighbors = k, metric= "jaccard")
knn.fit(X_train, y_train)
y_pred = knn.predict(X_test)
print("jaccard K-Nearest Neighbours")
print(metrics.accuracy_score(y_test, y_pred))
        
knn = KNeighborsClassifier(n_neighbors = k, metric= "yule")
knn.fit(X_train, y_train)
y_pred = knn.predict(X_test)
print("yule K-Nearest Neighbours")
print(metrics.accuracy_score(y_test, y_pred))
print("************************************ ****************************")



txt_files of features used ['/Users/catherine/Desktop/NLP/PAN Datasets/EB2014Selection/output2014OR.1M.txt', '/Users/catherine/Desktop/NLP/PAN Datasets/EB2014Selection/output2014OR.2F.txt']
/Users/catherine/Desktop/NLP/PAN Datasets/EB2014Selection/output2014OR.1M.txt 100
/Users/catherine/Desktop/NLP/PAN Datasets/EB2014Selection/output2014OR.2F.txt 100
length of features used = 200
len(all_training_text) 147
len(X_train)= 147 len(y_train)= 147
len(all_test_text) 78
len(X_test= 78 len(y_test)= 78
5
manhattan K-Nearest Neighbours
0.5
cosine K-Nearest Neighbours
0.5
euclidean K-Nearest Neighbours
0.5
braycurtis K-Nearest Neighbours
0.5
canberra K-Nearest Neighbours
0.5
jaccard K-Nearest Neighbours
0.5
yule K-Nearest Neighbours
0.5
************************************ ****************************


/anaconda3/lib/python3.7/site-packages/sklearn/utils/validation.py:475: DataConversionWarning: Data with input dtype float64 was converted to bool by check_pairwise_arrays.
  warnings.warn(msg, DataConversionWarning)


In [6]:

import sys
sys.path.append("/anaconda3/lib/python3.7/site-packages")
import numpy as np
import numpy
import nltk
import pickle
import pandas as pd
import glob
import os
import re
import operator
from collections import Counter
#from read_xml_files import *
##from s_stemmer import *
from nltk.stem.porter import PorterStemmer
from nltk.stem import LancasterStemmer, WordNetLemmatizer
from itertools import groupby
from nltk.collocations import *
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.cluster import KMeans
from scipy.cluster.vq import whiten
from nltk.tokenize.toktok import ToktokTokenizer
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2
import xml.etree.ElementTree as ET
import pandas as pd
#Import scikit-learn metrics module for accuracy calculation
from sklearn import metrics
##from pattern.text.en import singularize
##nltk.download('vader_lexicon')
##nltk.download('punkt')
#import the relevant modules from the NLTK library
from nltk.sentiment.vader import SentimentIntensityAnalyzer
sentence_tokenizer = nltk.data.load('tokenizers/punkt/english.pickle')
word_tokenizer = nltk.tokenize.RegexpTokenizer(r'\w+')
##tt = ToktokTokenizer()#keeps the url as is...................................................
from nltk.tokenize import TweetTokenizer
tt = TweetTokenizer()
Porter_stemmer = PorterStemmer()
Lancaster_stemmer = LancasterStemmer()
WordNet_lemmatizer = WordNetLemmatizer()
#nltk.download('averaged_perceptron_tagger')
##Train_path="D:/Data/PAN2015-Profiling/pan15-author-profiling-training-dataset-english-2015-04-23"#path for train data
##Test_path="D:/Data/PAN2015-Profiling/pan-ap2015-test/en" #path for test data

##Train_path="D:/NLP/PAN2014/pan14-author-profiling-training-corpus-english-twitter-2014-04-16"#path for train data D:\Data\PAN2013-Profiling\Training\en  D:\NLP\PAN2014
##train_truth_path="D:/NLP/PAN2014/truth_train"
##
##test_truth_path="D:/NLP/PAN2014/pan14_test"
##Test_path="D:/NLP/PAN2014/pan14_test/en"
##Content = open('results_of_clssification_all.txt', 'w')

M_train_file= "/Users/catherine/Desktop/NLP/PAN Datasets/PAN_tokens/2014_blog_Tokens/PAN2014_blog_male_train.txt"
F_train_file= "/Users/catherine/Desktop/NLP/PAN Datasets/PAN_tokens/2014_blog_Tokens/PAN2014_blog_female_train.txt"

M_test_file= "/Users/catherine/Desktop/NLP/PAN Datasets/PAN_tokens/2014_blog_Tokens/PAN2014_blog_male_test.txt"
F_test_file= "/Users/catherine/Desktop/NLP/PAN Datasets/PAN_tokens/2014_blog_Tokens/PAN2014_blog_female_test.txt"

def all_txt(M_txt_file,  F_txt_file):
    with open(M_txt_file, mode="r", encoding="utf-8") as f:
            txt_list = [(line[:-1]).split("\t") for line in f]
            male_dict = dict()
            content=[]
            for line in txt_list:
                if line[1] in male_dict:
                # append the new number to the existing array at this slot
                    male_dict[line[1]].append(line[3])
                else:
                # create a new array in this slot
                    male_dict[line[1]] = [line[3]]
    male_content= [("M", (' '.join(a)).split()) for a in list(male_dict.values())]
  
    with open(F_txt_file, mode="r", encoding="utf-8") as f:
            txt_list = [(line[:-1]).split("\t") for line in f]
            female_dict = dict()
            content=[]
            for line in txt_list:
                if line[1] in female_dict:
                # append the new number to the existing array at this slot
                    female_dict[line[1]].append(line[3])
                else:
                # create a new array in this slot
                    female_dict[line[1]] = [line[3]]
    female_content= [("F", (' '.join(a)).split()) for a in list(female_dict.values())]
    all_txt_per_person = male_content + female_content
    return all_txt_per_person

##print( [((all_txt(M_txt_file,  F_txt_file))[i][0])  for i in range(len((all_txt(M_txt_file,  F_txt_file))))]) helps to determine the gender....

      
"""
create feature vector from the training set.....................................................................................
"""

def WordFeatures(word_list, all_training_text):

    fvs_words = np.array([[author.count(word) for word in word_list] for author in all_training_text]).astype(np.float64)

    # normalise by dividing each row by number of tokens for each author........
    fvs_words /= np.c_[np.array([len(author) for author in all_training_text])]

    return fvs_words



"""
From the saved csv file, recover the saved features to be used...............................................................
"""
import csv
word_list=[]
txt_files =[ "/Users/catherine/Desktop/NLP/PAN Datasets/EB2014Selection/output2014OR.1M.txt", "/Users/catherine/Desktop/NLP/PAN Datasets/EB2014Selection/output2014OR.2F.txt"]
##txt_files =[ "/Users/catherine/Desktop/NLP/PAN Datasets/PAN_tokens/2014_blog_Tokens/PAN2014_blog_female_chosen_set.csv", "/Users/catherine/Desktop/NLP/PAN Datasets/PAN_tokens/2014_blog_Tokens/PAN2014_blog_male_chosen_set.csv"]
print("txt_files of features used",txt_files)
for txt_file in txt_files:
    with open(txt_file, mode="r", encoding="utf-8") as f:
##        reader = csv.reader(f, delimiter=",")
##        next(reader) # skip header
        word_list1 =  ([(r.split(":"))[1] for r in f])[0:150]
        print(txt_file, len(word_list1))
        word_list = word_list + word_list1
print("length of features used =", len(word_list))

#for txt_file in txt_files:
 #   with open(txt_file, mode="r", encoding="utf-8") as f:
 #       reader = csv.reader(f, delimiter=",")
 #       next(reader) # skip header
 #       word_list1 =  [r[0] for r in reader]
 #       print(txt_file, len(word_list1))
 #       word_list = word_list + word_list1
#print("length of features used =", len(word_list))


"""
Prepare the training and test sets to be parsed to the classifies............................................................
"""
A=(all_txt(M_train_file,  F_train_file))
all_training_text = ( [(A[i][1])  for i in range(len(A))])
print("len(all_training_text)",len(all_training_text))
X_train=WordFeatures(word_list, all_training_text)
y_train=np.array( [(A[i][0])  for i in range(len(A))])
print("len(X_train)=",len(X_train), "len(y_train)=", len(y_train))

B=(all_txt(M_test_file,  F_test_file))
all_test_text = ( [(B[i][1])  for i in range(len(B))])
print("len(all_test_text)",len(all_test_text))
X_test = WordFeatures(word_list, all_test_text)
y_test = np.array( [(B[i][0])  for i in range(len(B))])
print("len(X_test=",len(X_test), "len(y_test)=",len(y_test))


X_train = np.nan_to_num(X_train)
X_test = np.nan_to_num(X_test)

"""
Perform feature selection using chi2............................................................
"""
from sklearn.feature_selection import *
##
####def select_Bestfeatures(X, y):
##print("selector = SelectKBest(score_func=chi2, k=1000)")
##selector = SelectKBest(score_func=chi2, k=1000)
##fit = selector.fit(X_train,y_train)
##train_selected_features = fit.transform(X_train)
##test_selected_features = fit.transform(X_test)



"""
x_train and x_test based on the new features using chi2............................................................
"""
##X_train = select_Bestfeatures(X_train, y_train)
##X_test = select_Bestfeatures(X_test, y_test)


"""
Perform feature selection using f_classif............................................................
"""
from sklearn.feature_selection import *
##def f_classif_select_Bestfeatures(X, y):
##selector=SelectKBest(score_func=f_classif,k=500)
##fit = selector.fit(X_train,y_train)
##train_selected_features = fit.transform(X_train)
##test_selected_features = fit.transform(X_test)
##    selected_features = fit.transform(X)
##    return selected_features

"""
x_train and x_test based on the new features using f_classif............................................................
"""
##X_train = f_classif_select_Bestfeatures(X_train, y_train)
##X_test = f_classif_select_Bestfeatures(X_test, y_test)



"""
Perform feature selection using mutual_info_classif............................................................
"""
from sklearn.feature_selection import *
##def mutual_info_classif_select_Bestfeatures(X_tr, X_te, y):
##selector=SelectKBest(score_func=mutual_info_classif,k=100)
##fit = selector.fit(X_train,y_train)
##train_selected_features = fit.transform(X_train)
##test_selected_features = fit.transform(X_test)
##    return train_selected_features, test_selected_features

"""
x_train and x_test based on the new features using f_classif............................................................
"""
##X_train =train_selected_features 
##X_test = test_selected_features


"""
Perform feature selection using pca............................................................
"""

##from sklearn.decomposition import PCA
##def PCA_select_Bestfeatures(X):
##    # feature extraction
##    pca = PCA(n_components=200)
##    fit = pca.fit(X, y)
##    Train_selected_features = fit.transform(X)
##    return selected_features

"""
x_train and x_test based on the new features using f_classif............................................................
"""
##X_train = PCA_select_Bestfeatures(X_train)
##X_test = PCA_select_Bestfeatures(X_test)

"""
Below are a list of Classification Algorithms with different distance measres ..............................................................................
"""
"""
K-Nearest Neighbours (average aa over all the nearest neighbour selectiopn)..............................................................................
"""

k=5

print(k)
"""
8. manhattan K-Nearest Neighbours..............................................................................
"""
from sklearn.neighbors import KNeighborsClassifier
knn = KNeighborsClassifier(n_neighbors = k, metric= "manhattan")
knn.fit(X_train, y_train)
y_pred = knn.predict(X_test)
print("manhattan K-Nearest Neighbours")
print(metrics.accuracy_score(y_test, y_pred))

    
"""
8. cosine K-Nearest Neighbours..............................................................................
"""
from sklearn.neighbors import KNeighborsClassifier
knn = KNeighborsClassifier(n_neighbors = k, metric= "cosine")
knn.fit(X_train, y_train)
y_pred = knn.predict(X_test)
print("cosine K-Nearest Neighbours")
print(metrics.accuracy_score(y_test, y_pred))


"""
8. euclidean K-Nearest Neighbours..............................................................................
"""
from sklearn.neighbors import KNeighborsClassifier
knn = KNeighborsClassifier(n_neighbors = k, metric= "euclidean")
knn.fit(X_train, y_train)
y_pred = knn.predict(X_test)
print("euclidean K-Nearest Neighbours")
print(metrics.accuracy_score(y_test, y_pred))

        
"""
9. scipy.spatial.distance K-Nearest Neighbours..............................................................................
"""
from scipy.spatial.distance import *
knn = KNeighborsClassifier(n_neighbors = k, metric= "braycurtis")
knn.fit(X_train, y_train)
y_pred = knn.predict(X_test)
print("braycurtis K-Nearest Neighbours")
print(metrics.accuracy_score(y_test, y_pred))
       
        
knn = KNeighborsClassifier(n_neighbors = k, metric= "canberra")
knn.fit(X_train, y_train)
y_pred = knn.predict(X_test)
print("canberra K-Nearest Neighbours")
print(metrics.accuracy_score(y_test, y_pred))
  
        
knn = KNeighborsClassifier(n_neighbors = k, metric= "jaccard")
knn.fit(X_train, y_train)
y_pred = knn.predict(X_test)
print("jaccard K-Nearest Neighbours")
print(metrics.accuracy_score(y_test, y_pred))
        
knn = KNeighborsClassifier(n_neighbors = k, metric= "yule")
knn.fit(X_train, y_train)
y_pred = knn.predict(X_test)
print("yule K-Nearest Neighbours")
print(metrics.accuracy_score(y_test, y_pred))
print("************************************ ****************************")




txt_files of features used ['/Users/catherine/Desktop/NLP/PAN Datasets/EB2014Selection/output2014OR.1M.txt', '/Users/catherine/Desktop/NLP/PAN Datasets/EB2014Selection/output2014OR.2F.txt']
/Users/catherine/Desktop/NLP/PAN Datasets/EB2014Selection/output2014OR.1M.txt 150
/Users/catherine/Desktop/NLP/PAN Datasets/EB2014Selection/output2014OR.2F.txt 150
length of features used = 300
len(all_training_text) 147
len(X_train)= 147 len(y_train)= 147
len(all_test_text) 78
len(X_test= 78 len(y_test)= 78
5
manhattan K-Nearest Neighbours
0.5
cosine K-Nearest Neighbours
0.5
euclidean K-Nearest Neighbours
0.5
braycurtis K-Nearest Neighbours
0.5
canberra K-Nearest Neighbours
0.5
jaccard K-Nearest Neighbours
0.5
yule K-Nearest Neighbours
0.5
************************************ ****************************


/anaconda3/lib/python3.7/site-packages/sklearn/utils/validation.py:475: DataConversionWarning: Data with input dtype float64 was converted to bool by check_pairwise_arrays.
  warnings.warn(msg, DataConversionWarning)


In [7]:

import sys
sys.path.append("/anaconda3/lib/python3.7/site-packages")
import numpy as np
import numpy
import nltk
import pickle
import pandas as pd
import glob
import os
import re
import operator
from collections import Counter
#from read_xml_files import *
##from s_stemmer import *
from nltk.stem.porter import PorterStemmer
from nltk.stem import LancasterStemmer, WordNetLemmatizer
from itertools import groupby
from nltk.collocations import *
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.cluster import KMeans
from scipy.cluster.vq import whiten
from nltk.tokenize.toktok import ToktokTokenizer
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2
import xml.etree.ElementTree as ET
import pandas as pd
#Import scikit-learn metrics module for accuracy calculation
from sklearn import metrics
##from pattern.text.en import singularize
##nltk.download('vader_lexicon')
##nltk.download('punkt')
#import the relevant modules from the NLTK library
from nltk.sentiment.vader import SentimentIntensityAnalyzer
sentence_tokenizer = nltk.data.load('tokenizers/punkt/english.pickle')
word_tokenizer = nltk.tokenize.RegexpTokenizer(r'\w+')
##tt = ToktokTokenizer()#keeps the url as is...................................................
from nltk.tokenize import TweetTokenizer
tt = TweetTokenizer()
Porter_stemmer = PorterStemmer()
Lancaster_stemmer = LancasterStemmer()
WordNet_lemmatizer = WordNetLemmatizer()
#nltk.download('averaged_perceptron_tagger')
##Train_path="D:/Data/PAN2015-Profiling/pan15-author-profiling-training-dataset-english-2015-04-23"#path for train data
##Test_path="D:/Data/PAN2015-Profiling/pan-ap2015-test/en" #path for test data

##Train_path="D:/NLP/PAN2014/pan14-author-profiling-training-corpus-english-twitter-2014-04-16"#path for train data D:\Data\PAN2013-Profiling\Training\en  D:\NLP\PAN2014
##train_truth_path="D:/NLP/PAN2014/truth_train"
##
##test_truth_path="D:/NLP/PAN2014/pan14_test"
##Test_path="D:/NLP/PAN2014/pan14_test/en"
##Content = open('results_of_clssification_all.txt', 'w')

M_train_file= "/Users/catherine/Desktop/NLP/PAN Datasets/PAN_tokens/2014_blog_Tokens/PAN2014_blog_male_train.txt"
F_train_file= "/Users/catherine/Desktop/NLP/PAN Datasets/PAN_tokens/2014_blog_Tokens/PAN2014_blog_female_train.txt"

M_test_file= "/Users/catherine/Desktop/NLP/PAN Datasets/PAN_tokens/2014_blog_Tokens/PAN2014_blog_male_test.txt"
F_test_file= "/Users/catherine/Desktop/NLP/PAN Datasets/PAN_tokens/2014_blog_Tokens/PAN2014_blog_female_test.txt"

def all_txt(M_txt_file,  F_txt_file):
    with open(M_txt_file, mode="r", encoding="utf-8") as f:
            txt_list = [(line[:-1]).split("\t") for line in f]
            male_dict = dict()
            content=[]
            for line in txt_list:
                if line[1] in male_dict:
                # append the new number to the existing array at this slot
                    male_dict[line[1]].append(line[3])
                else:
                # create a new array in this slot
                    male_dict[line[1]] = [line[3]]
    male_content= [("M", (' '.join(a)).split()) for a in list(male_dict.values())]
  
    with open(F_txt_file, mode="r", encoding="utf-8") as f:
            txt_list = [(line[:-1]).split("\t") for line in f]
            female_dict = dict()
            content=[]
            for line in txt_list:
                if line[1] in female_dict:
                # append the new number to the existing array at this slot
                    female_dict[line[1]].append(line[3])
                else:
                # create a new array in this slot
                    female_dict[line[1]] = [line[3]]
    female_content= [("F", (' '.join(a)).split()) for a in list(female_dict.values())]
    all_txt_per_person = male_content + female_content
    return all_txt_per_person

##print( [((all_txt(M_txt_file,  F_txt_file))[i][0])  for i in range(len((all_txt(M_txt_file,  F_txt_file))))]) helps to determine the gender....

      
"""
create feature vector from the training set.....................................................................................
"""

def WordFeatures(word_list, all_training_text):

    fvs_words = np.array([[author.count(word) for word in word_list] for author in all_training_text]).astype(np.float64)

    # normalise by dividing each row by number of tokens for each author........
    fvs_words /= np.c_[np.array([len(author) for author in all_training_text])]

    return fvs_words



"""
From the saved csv file, recover the saved features to be used...............................................................
"""
import csv
word_list=[]
txt_files =[ "/Users/catherine/Desktop/NLP/PAN Datasets/EB2014Selection/output2014OR.1M.txt", "/Users/catherine/Desktop/NLP/PAN Datasets/EB2014Selection/output2014OR.2F.txt"]
##txt_files =[ "/Users/catherine/Desktop/NLP/PAN Datasets/PAN_tokens/2014_blog_Tokens/PAN2014_blog_female_chosen_set.csv", "/Users/catherine/Desktop/NLP/PAN Datasets/PAN_tokens/2014_blog_Tokens/PAN2014_blog_male_chosen_set.csv"]
print("txt_files of features used",txt_files)
for txt_file in txt_files:
    with open(txt_file, mode="r", encoding="utf-8") as f:
##        reader = csv.reader(f, delimiter=",")
##        next(reader) # skip header
        word_list1 =  ([(r.split(":"))[1] for r in f])[0:250]
        print(txt_file, len(word_list1))
        word_list = word_list + word_list1
print("length of features used =", len(word_list))

#for txt_file in txt_files:
 #   with open(txt_file, mode="r", encoding="utf-8") as f:
 #       reader = csv.reader(f, delimiter=",")
 #       next(reader) # skip header
 #       word_list1 =  [r[0] for r in reader]
 #       print(txt_file, len(word_list1))
 #       word_list = word_list + word_list1
#print("length of features used =", len(word_list))


"""
Prepare the training and test sets to be parsed to the classifies............................................................
"""
A=(all_txt(M_train_file,  F_train_file))
all_training_text = ( [(A[i][1])  for i in range(len(A))])
print("len(all_training_text)",len(all_training_text))
X_train=WordFeatures(word_list, all_training_text)
y_train=np.array( [(A[i][0])  for i in range(len(A))])
print("len(X_train)=",len(X_train), "len(y_train)=", len(y_train))

B=(all_txt(M_test_file,  F_test_file))
all_test_text = ( [(B[i][1])  for i in range(len(B))])
print("len(all_test_text)",len(all_test_text))
X_test = WordFeatures(word_list, all_test_text)
y_test = np.array( [(B[i][0])  for i in range(len(B))])
print("len(X_test=",len(X_test), "len(y_test)=",len(y_test))


X_train = np.nan_to_num(X_train)
X_test = np.nan_to_num(X_test)

"""
Perform feature selection using chi2............................................................
"""
from sklearn.feature_selection import *
##
####def select_Bestfeatures(X, y):
##print("selector = SelectKBest(score_func=chi2, k=1000)")
##selector = SelectKBest(score_func=chi2, k=1000)
##fit = selector.fit(X_train,y_train)
##train_selected_features = fit.transform(X_train)
##test_selected_features = fit.transform(X_test)



"""
x_train and x_test based on the new features using chi2............................................................
"""
##X_train = select_Bestfeatures(X_train, y_train)
##X_test = select_Bestfeatures(X_test, y_test)


"""
Perform feature selection using f_classif............................................................
"""
from sklearn.feature_selection import *
##def f_classif_select_Bestfeatures(X, y):
##selector=SelectKBest(score_func=f_classif,k=500)
##fit = selector.fit(X_train,y_train)
##train_selected_features = fit.transform(X_train)
##test_selected_features = fit.transform(X_test)
##    selected_features = fit.transform(X)
##    return selected_features

"""
x_train and x_test based on the new features using f_classif............................................................
"""
##X_train = f_classif_select_Bestfeatures(X_train, y_train)
##X_test = f_classif_select_Bestfeatures(X_test, y_test)



"""
Perform feature selection using mutual_info_classif............................................................
"""
from sklearn.feature_selection import *
##def mutual_info_classif_select_Bestfeatures(X_tr, X_te, y):
##selector=SelectKBest(score_func=mutual_info_classif,k=100)
##fit = selector.fit(X_train,y_train)
##train_selected_features = fit.transform(X_train)
##test_selected_features = fit.transform(X_test)
##    return train_selected_features, test_selected_features

"""
x_train and x_test based on the new features using f_classif............................................................
"""
##X_train =train_selected_features 
##X_test = test_selected_features


"""
Perform feature selection using pca............................................................
"""

##from sklearn.decomposition import PCA
##def PCA_select_Bestfeatures(X):
##    # feature extraction
##    pca = PCA(n_components=200)
##    fit = pca.fit(X, y)
##    Train_selected_features = fit.transform(X)
##    return selected_features

"""
x_train and x_test based on the new features using f_classif............................................................
"""
##X_train = PCA_select_Bestfeatures(X_train)
##X_test = PCA_select_Bestfeatures(X_test)

"""
Below are a list of Classification Algorithms with different distance measres ..............................................................................
"""
"""
K-Nearest Neighbours (average aa over all the nearest neighbour selectiopn)..............................................................................
"""

k=5

print(k)
"""
8. manhattan K-Nearest Neighbours..............................................................................
"""
from sklearn.neighbors import KNeighborsClassifier
knn = KNeighborsClassifier(n_neighbors = k, metric= "manhattan")
knn.fit(X_train, y_train)
y_pred = knn.predict(X_test)
print("manhattan K-Nearest Neighbours")
print(metrics.accuracy_score(y_test, y_pred))

    
"""
8. cosine K-Nearest Neighbours..............................................................................
"""
from sklearn.neighbors import KNeighborsClassifier
knn = KNeighborsClassifier(n_neighbors = k, metric= "cosine")
knn.fit(X_train, y_train)
y_pred = knn.predict(X_test)
print("cosine K-Nearest Neighbours")
print(metrics.accuracy_score(y_test, y_pred))


"""
8. euclidean K-Nearest Neighbours..............................................................................
"""
from sklearn.neighbors import KNeighborsClassifier
knn = KNeighborsClassifier(n_neighbors = k, metric= "euclidean")
knn.fit(X_train, y_train)
y_pred = knn.predict(X_test)
print("euclidean K-Nearest Neighbours")
print(metrics.accuracy_score(y_test, y_pred))

        
"""
9. scipy.spatial.distance K-Nearest Neighbours..............................................................................
"""
from scipy.spatial.distance import *
knn = KNeighborsClassifier(n_neighbors = k, metric= "braycurtis")
knn.fit(X_train, y_train)
y_pred = knn.predict(X_test)
print("braycurtis K-Nearest Neighbours")
print(metrics.accuracy_score(y_test, y_pred))
       
        
knn = KNeighborsClassifier(n_neighbors = k, metric= "canberra")
knn.fit(X_train, y_train)
y_pred = knn.predict(X_test)
print("canberra K-Nearest Neighbours")
print(metrics.accuracy_score(y_test, y_pred))
  
        
knn = KNeighborsClassifier(n_neighbors = k, metric= "jaccard")
knn.fit(X_train, y_train)
y_pred = knn.predict(X_test)
print("jaccard K-Nearest Neighbours")
print(metrics.accuracy_score(y_test, y_pred))
        
knn = KNeighborsClassifier(n_neighbors = k, metric= "yule")
knn.fit(X_train, y_train)
y_pred = knn.predict(X_test)
print("yule K-Nearest Neighbours")
print(metrics.accuracy_score(y_test, y_pred))
print("************************************ ****************************")




txt_files of features used ['/Users/catherine/Desktop/NLP/PAN Datasets/EB2014Selection/output2014OR.1M.txt', '/Users/catherine/Desktop/NLP/PAN Datasets/EB2014Selection/output2014OR.2F.txt']
/Users/catherine/Desktop/NLP/PAN Datasets/EB2014Selection/output2014OR.1M.txt 250
/Users/catherine/Desktop/NLP/PAN Datasets/EB2014Selection/output2014OR.2F.txt 250
length of features used = 500
len(all_training_text) 147
len(X_train)= 147 len(y_train)= 147
len(all_test_text) 78
len(X_test= 78 len(y_test)= 78
5
manhattan K-Nearest Neighbours
0.5
cosine K-Nearest Neighbours
0.5
euclidean K-Nearest Neighbours
0.5
braycurtis K-Nearest Neighbours
0.5
canberra K-Nearest Neighbours
0.5
jaccard K-Nearest Neighbours
0.5
yule K-Nearest Neighbours
0.5
************************************ ****************************


/anaconda3/lib/python3.7/site-packages/sklearn/utils/validation.py:475: DataConversionWarning: Data with input dtype float64 was converted to bool by check_pairwise_arrays.
  warnings.warn(msg, DataConversionWarning)


In [8]:

import sys
sys.path.append("/anaconda3/lib/python3.7/site-packages")
import numpy as np
import numpy
import nltk
import pickle
import pandas as pd
import glob
import os
import re
import operator
from collections import Counter
#from read_xml_files import *
##from s_stemmer import *
from nltk.stem.porter import PorterStemmer
from nltk.stem import LancasterStemmer, WordNetLemmatizer
from itertools import groupby
from nltk.collocations import *
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.cluster import KMeans
from scipy.cluster.vq import whiten
from nltk.tokenize.toktok import ToktokTokenizer
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2
import xml.etree.ElementTree as ET
import pandas as pd
#Import scikit-learn metrics module for accuracy calculation
from sklearn import metrics
##from pattern.text.en import singularize
##nltk.download('vader_lexicon')
##nltk.download('punkt')
#import the relevant modules from the NLTK library
from nltk.sentiment.vader import SentimentIntensityAnalyzer
sentence_tokenizer = nltk.data.load('tokenizers/punkt/english.pickle')
word_tokenizer = nltk.tokenize.RegexpTokenizer(r'\w+')
##tt = ToktokTokenizer()#keeps the url as is...................................................
from nltk.tokenize import TweetTokenizer
tt = TweetTokenizer()
Porter_stemmer = PorterStemmer()
Lancaster_stemmer = LancasterStemmer()
WordNet_lemmatizer = WordNetLemmatizer()
#nltk.download('averaged_perceptron_tagger')
##Train_path="D:/Data/PAN2015-Profiling/pan15-author-profiling-training-dataset-english-2015-04-23"#path for train data
##Test_path="D:/Data/PAN2015-Profiling/pan-ap2015-test/en" #path for test data

##Train_path="D:/NLP/PAN2014/pan14-author-profiling-training-corpus-english-twitter-2014-04-16"#path for train data D:\Data\PAN2013-Profiling\Training\en  D:\NLP\PAN2014
##train_truth_path="D:/NLP/PAN2014/truth_train"
##
##test_truth_path="D:/NLP/PAN2014/pan14_test"
##Test_path="D:/NLP/PAN2014/pan14_test/en"
##Content = open('results_of_clssification_all.txt', 'w')

M_train_file= "/Users/catherine/Desktop/NLP/PAN Datasets/PAN_tokens/2014_blog_Tokens/PAN2014_blog_male_train.txt"
F_train_file= "/Users/catherine/Desktop/NLP/PAN Datasets/PAN_tokens/2014_blog_Tokens/PAN2014_blog_female_train.txt"

M_test_file= "/Users/catherine/Desktop/NLP/PAN Datasets/PAN_tokens/2014_blog_Tokens/PAN2014_blog_male_test.txt"
F_test_file= "/Users/catherine/Desktop/NLP/PAN Datasets/PAN_tokens/2014_blog_Tokens/PAN2014_blog_female_test.txt"

def all_txt(M_txt_file,  F_txt_file):
    with open(M_txt_file, mode="r", encoding="utf-8") as f:
            txt_list = [(line[:-1]).split("\t") for line in f]
            male_dict = dict()
            content=[]
            for line in txt_list:
                if line[1] in male_dict:
                # append the new number to the existing array at this slot
                    male_dict[line[1]].append(line[3])
                else:
                # create a new array in this slot
                    male_dict[line[1]] = [line[3]]
    male_content= [("M", (' '.join(a)).split()) for a in list(male_dict.values())]
  
    with open(F_txt_file, mode="r", encoding="utf-8") as f:
            txt_list = [(line[:-1]).split("\t") for line in f]
            female_dict = dict()
            content=[]
            for line in txt_list:
                if line[1] in female_dict:
                # append the new number to the existing array at this slot
                    female_dict[line[1]].append(line[3])
                else:
                # create a new array in this slot
                    female_dict[line[1]] = [line[3]]
    female_content= [("F", (' '.join(a)).split()) for a in list(female_dict.values())]
    all_txt_per_person = male_content + female_content
    return all_txt_per_person

##print( [((all_txt(M_txt_file,  F_txt_file))[i][0])  for i in range(len((all_txt(M_txt_file,  F_txt_file))))]) helps to determine the gender....

      
"""
create feature vector from the training set.....................................................................................
"""

def WordFeatures(word_list, all_training_text):

    fvs_words = np.array([[author.count(word) for word in word_list] for author in all_training_text]).astype(np.float64)

    # normalise by dividing each row by number of tokens for each author........
    fvs_words /= np.c_[np.array([len(author) for author in all_training_text])]

    return fvs_words



"""
From the saved csv file, recover the saved features to be used...............................................................
"""
import csv
word_list=[]
txt_files =[ "/Users/catherine/Desktop/NLP/PAN Datasets/EB2014Selection/output2014PMI.1M.txt", "/Users/catherine/Desktop/NLP/PAN Datasets/EB2014Selection/output2014PMI.2F.txt"]
##txt_files =[ "/Users/catherine/Desktop/NLP/PAN Datasets/PAN_tokens/2014_blog_Tokens/PAN2014_blog_female_chosen_set.csv", "/Users/catherine/Desktop/NLP/PAN Datasets/PAN_tokens/2014_blog_Tokens/PAN2014_blog_male_chosen_set.csv"]
print("txt_files of features used",txt_files)
for txt_file in txt_files:
    with open(txt_file, mode="r", encoding="utf-8") as f:
##        reader = csv.reader(f, delimiter=",")
##        next(reader) # skip header
        word_list1 =  ([(r.split(":"))[1] for r in f])[0:100]
        print(txt_file, len(word_list1))
        word_list = word_list + word_list1
print("length of features used =", len(word_list))

#for txt_file in txt_files:
 #   with open(txt_file, mode="r", encoding="utf-8") as f:
 #       reader = csv.reader(f, delimiter=",")
 #       next(reader) # skip header
 #       word_list1 =  [r[0] for r in reader]
 #       print(txt_file, len(word_list1))
 #       word_list = word_list + word_list1
#print("length of features used =", len(word_list))


"""
Prepare the training and test sets to be parsed to the classifies............................................................
"""
A=(all_txt(M_train_file,  F_train_file))
all_training_text = ( [(A[i][1])  for i in range(len(A))])
print("len(all_training_text)",len(all_training_text))
X_train=WordFeatures(word_list, all_training_text)
y_train=np.array( [(A[i][0])  for i in range(len(A))])
print("len(X_train)=",len(X_train), "len(y_train)=", len(y_train))

B=(all_txt(M_test_file,  F_test_file))
all_test_text = ( [(B[i][1])  for i in range(len(B))])
print("len(all_test_text)",len(all_test_text))
X_test = WordFeatures(word_list, all_test_text)
y_test = np.array( [(B[i][0])  for i in range(len(B))])
print("len(X_test=",len(X_test), "len(y_test)=",len(y_test))


X_train = np.nan_to_num(X_train)
X_test = np.nan_to_num(X_test)

"""
Perform feature selection using chi2............................................................
"""
from sklearn.feature_selection import *
##
####def select_Bestfeatures(X, y):
##print("selector = SelectKBest(score_func=chi2, k=1000)")
##selector = SelectKBest(score_func=chi2, k=1000)
##fit = selector.fit(X_train,y_train)
##train_selected_features = fit.transform(X_train)
##test_selected_features = fit.transform(X_test)



"""
x_train and x_test based on the new features using chi2............................................................
"""
##X_train = select_Bestfeatures(X_train, y_train)
##X_test = select_Bestfeatures(X_test, y_test)


"""
Perform feature selection using f_classif............................................................
"""
from sklearn.feature_selection import *
##def f_classif_select_Bestfeatures(X, y):
##selector=SelectKBest(score_func=f_classif,k=500)
##fit = selector.fit(X_train,y_train)
##train_selected_features = fit.transform(X_train)
##test_selected_features = fit.transform(X_test)
##    selected_features = fit.transform(X)
##    return selected_features

"""
x_train and x_test based on the new features using f_classif............................................................
"""
##X_train = f_classif_select_Bestfeatures(X_train, y_train)
##X_test = f_classif_select_Bestfeatures(X_test, y_test)



"""
Perform feature selection using mutual_info_classif............................................................
"""
from sklearn.feature_selection import *
##def mutual_info_classif_select_Bestfeatures(X_tr, X_te, y):
##selector=SelectKBest(score_func=mutual_info_classif,k=100)
##fit = selector.fit(X_train,y_train)
##train_selected_features = fit.transform(X_train)
##test_selected_features = fit.transform(X_test)
##    return train_selected_features, test_selected_features

"""
x_train and x_test based on the new features using f_classif............................................................
"""
##X_train =train_selected_features 
##X_test = test_selected_features


"""
Perform feature selection using pca............................................................
"""

##from sklearn.decomposition import PCA
##def PCA_select_Bestfeatures(X):
##    # feature extraction
##    pca = PCA(n_components=200)
##    fit = pca.fit(X, y)
##    Train_selected_features = fit.transform(X)
##    return selected_features

"""
x_train and x_test based on the new features using f_classif............................................................
"""
##X_train = PCA_select_Bestfeatures(X_train)
##X_test = PCA_select_Bestfeatures(X_test)

"""
Below are a list of Classification Algorithms with different distance measres ..............................................................................
"""
"""
K-Nearest Neighbours (average aa over all the nearest neighbour selectiopn)..............................................................................
"""

k=5

print(k)
"""
8. manhattan K-Nearest Neighbours..............................................................................
"""
from sklearn.neighbors import KNeighborsClassifier
knn = KNeighborsClassifier(n_neighbors = k, metric= "manhattan")
knn.fit(X_train, y_train)
y_pred = knn.predict(X_test)
print("manhattan K-Nearest Neighbours")
print(metrics.accuracy_score(y_test, y_pred))

    
"""
8. cosine K-Nearest Neighbours..............................................................................
"""
from sklearn.neighbors import KNeighborsClassifier
knn = KNeighborsClassifier(n_neighbors = k, metric= "cosine")
knn.fit(X_train, y_train)
y_pred = knn.predict(X_test)
print("cosine K-Nearest Neighbours")
print(metrics.accuracy_score(y_test, y_pred))


"""
8. euclidean K-Nearest Neighbours..............................................................................
"""
from sklearn.neighbors import KNeighborsClassifier
knn = KNeighborsClassifier(n_neighbors = k, metric= "euclidean")
knn.fit(X_train, y_train)
y_pred = knn.predict(X_test)
print("euclidean K-Nearest Neighbours")
print(metrics.accuracy_score(y_test, y_pred))

        
"""
9. scipy.spatial.distance K-Nearest Neighbours..............................................................................
"""
from scipy.spatial.distance import *
knn = KNeighborsClassifier(n_neighbors = k, metric= "braycurtis")
knn.fit(X_train, y_train)
y_pred = knn.predict(X_test)
print("braycurtis K-Nearest Neighbours")
print(metrics.accuracy_score(y_test, y_pred))
       
        
knn = KNeighborsClassifier(n_neighbors = k, metric= "canberra")
knn.fit(X_train, y_train)
y_pred = knn.predict(X_test)
print("canberra K-Nearest Neighbours")
print(metrics.accuracy_score(y_test, y_pred))
  
        
knn = KNeighborsClassifier(n_neighbors = k, metric= "jaccard")
knn.fit(X_train, y_train)
y_pred = knn.predict(X_test)
print("jaccard K-Nearest Neighbours")
print(metrics.accuracy_score(y_test, y_pred))
        
knn = KNeighborsClassifier(n_neighbors = k, metric= "yule")
knn.fit(X_train, y_train)
y_pred = knn.predict(X_test)
print("yule K-Nearest Neighbours")
print(metrics.accuracy_score(y_test, y_pred))
print("************************************ ****************************")




txt_files of features used ['/Users/catherine/Desktop/NLP/PAN Datasets/EB2014Selection/output2014PMI.1M.txt', '/Users/catherine/Desktop/NLP/PAN Datasets/EB2014Selection/output2014PMI.2F.txt']
/Users/catherine/Desktop/NLP/PAN Datasets/EB2014Selection/output2014PMI.1M.txt 100
/Users/catherine/Desktop/NLP/PAN Datasets/EB2014Selection/output2014PMI.2F.txt 100
length of features used = 200
len(all_training_text) 147
len(X_train)= 147 len(y_train)= 147
len(all_test_text) 78
len(X_test= 78 len(y_test)= 78
5
manhattan K-Nearest Neighbours
0.5
cosine K-Nearest Neighbours
0.5
euclidean K-Nearest Neighbours
0.5
braycurtis K-Nearest Neighbours
0.5
canberra K-Nearest Neighbours
0.5
jaccard K-Nearest Neighbours
0.5
yule K-Nearest Neighbours
0.5
************************************ ****************************


/anaconda3/lib/python3.7/site-packages/sklearn/utils/validation.py:475: DataConversionWarning: Data with input dtype float64 was converted to bool by check_pairwise_arrays.
  warnings.warn(msg, DataConversionWarning)


In [9]:

import sys
sys.path.append("/anaconda3/lib/python3.7/site-packages")
import numpy as np
import numpy
import nltk
import pickle
import pandas as pd
import glob
import os
import re
import operator
from collections import Counter
#from read_xml_files import *
##from s_stemmer import *
from nltk.stem.porter import PorterStemmer
from nltk.stem import LancasterStemmer, WordNetLemmatizer
from itertools import groupby
from nltk.collocations import *
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.cluster import KMeans
from scipy.cluster.vq import whiten
from nltk.tokenize.toktok import ToktokTokenizer
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2
import xml.etree.ElementTree as ET
import pandas as pd
#Import scikit-learn metrics module for accuracy calculation
from sklearn import metrics
##from pattern.text.en import singularize
##nltk.download('vader_lexicon')
##nltk.download('punkt')
#import the relevant modules from the NLTK library
from nltk.sentiment.vader import SentimentIntensityAnalyzer
sentence_tokenizer = nltk.data.load('tokenizers/punkt/english.pickle')
word_tokenizer = nltk.tokenize.RegexpTokenizer(r'\w+')
##tt = ToktokTokenizer()#keeps the url as is...................................................
from nltk.tokenize import TweetTokenizer
tt = TweetTokenizer()
Porter_stemmer = PorterStemmer()
Lancaster_stemmer = LancasterStemmer()
WordNet_lemmatizer = WordNetLemmatizer()
#nltk.download('averaged_perceptron_tagger')
##Train_path="D:/Data/PAN2015-Profiling/pan15-author-profiling-training-dataset-english-2015-04-23"#path for train data
##Test_path="D:/Data/PAN2015-Profiling/pan-ap2015-test/en" #path for test data

##Train_path="D:/NLP/PAN2014/pan14-author-profiling-training-corpus-english-twitter-2014-04-16"#path for train data D:\Data\PAN2013-Profiling\Training\en  D:\NLP\PAN2014
##train_truth_path="D:/NLP/PAN2014/truth_train"
##
##test_truth_path="D:/NLP/PAN2014/pan14_test"
##Test_path="D:/NLP/PAN2014/pan14_test/en"
##Content = open('results_of_clssification_all.txt', 'w')

M_train_file= "/Users/catherine/Desktop/NLP/PAN Datasets/PAN_tokens/2014_blog_Tokens/PAN2014_blog_male_train.txt"
F_train_file= "/Users/catherine/Desktop/NLP/PAN Datasets/PAN_tokens/2014_blog_Tokens/PAN2014_blog_female_train.txt"

M_test_file= "/Users/catherine/Desktop/NLP/PAN Datasets/PAN_tokens/2014_blog_Tokens/PAN2014_blog_male_test.txt"
F_test_file= "/Users/catherine/Desktop/NLP/PAN Datasets/PAN_tokens/2014_blog_Tokens/PAN2014_blog_female_test.txt"

def all_txt(M_txt_file,  F_txt_file):
    with open(M_txt_file, mode="r", encoding="utf-8") as f:
            txt_list = [(line[:-1]).split("\t") for line in f]
            male_dict = dict()
            content=[]
            for line in txt_list:
                if line[1] in male_dict:
                # append the new number to the existing array at this slot
                    male_dict[line[1]].append(line[3])
                else:
                # create a new array in this slot
                    male_dict[line[1]] = [line[3]]
    male_content= [("M", (' '.join(a)).split()) for a in list(male_dict.values())]
  
    with open(F_txt_file, mode="r", encoding="utf-8") as f:
            txt_list = [(line[:-1]).split("\t") for line in f]
            female_dict = dict()
            content=[]
            for line in txt_list:
                if line[1] in female_dict:
                # append the new number to the existing array at this slot
                    female_dict[line[1]].append(line[3])
                else:
                # create a new array in this slot
                    female_dict[line[1]] = [line[3]]
    female_content= [("F", (' '.join(a)).split()) for a in list(female_dict.values())]
    all_txt_per_person = male_content + female_content
    return all_txt_per_person

##print( [((all_txt(M_txt_file,  F_txt_file))[i][0])  for i in range(len((all_txt(M_txt_file,  F_txt_file))))]) helps to determine the gender....

      
"""
create feature vector from the training set.....................................................................................
"""

def WordFeatures(word_list, all_training_text):

    fvs_words = np.array([[author.count(word) for word in word_list] for author in all_training_text]).astype(np.float64)

    # normalise by dividing each row by number of tokens for each author........
    fvs_words /= np.c_[np.array([len(author) for author in all_training_text])]

    return fvs_words



"""
From the saved csv file, recover the saved features to be used...............................................................
"""
import csv
word_list=[]
txt_files =[ "/Users/catherine/Desktop/NLP/PAN Datasets/EB2014Selection/output2014PMI.1M.txt", "/Users/catherine/Desktop/NLP/PAN Datasets/EB2014Selection/output2014PMI.2F.txt"]
##txt_files =[ "/Users/catherine/Desktop/NLP/PAN Datasets/PAN_tokens/2014_blog_Tokens/PAN2014_blog_female_chosen_set.csv", "/Users/catherine/Desktop/NLP/PAN Datasets/PAN_tokens/2014_blog_Tokens/PAN2014_blog_male_chosen_set.csv"]
print("txt_files of features used",txt_files)
for txt_file in txt_files:
    with open(txt_file, mode="r", encoding="utf-8") as f:
##        reader = csv.reader(f, delimiter=",")
##        next(reader) # skip header
        word_list1 =  ([(r.split(":"))[1] for r in f])[0:150]
        print(txt_file, len(word_list1))
        word_list = word_list + word_list1
print("length of features used =", len(word_list))

#for txt_file in txt_files:
 #   with open(txt_file, mode="r", encoding="utf-8") as f:
 #       reader = csv.reader(f, delimiter=",")
 #       next(reader) # skip header
 #       word_list1 =  [r[0] for r in reader]
 #       print(txt_file, len(word_list1))
 #       word_list = word_list + word_list1
#print("length of features used =", len(word_list))


"""
Prepare the training and test sets to be parsed to the classifies............................................................
"""
A=(all_txt(M_train_file,  F_train_file))
all_training_text = ( [(A[i][1])  for i in range(len(A))])
print("len(all_training_text)",len(all_training_text))
X_train=WordFeatures(word_list, all_training_text)
y_train=np.array( [(A[i][0])  for i in range(len(A))])
print("len(X_train)=",len(X_train), "len(y_train)=", len(y_train))

B=(all_txt(M_test_file,  F_test_file))
all_test_text = ( [(B[i][1])  for i in range(len(B))])
print("len(all_test_text)",len(all_test_text))
X_test = WordFeatures(word_list, all_test_text)
y_test = np.array( [(B[i][0])  for i in range(len(B))])
print("len(X_test=",len(X_test), "len(y_test)=",len(y_test))


X_train = np.nan_to_num(X_train)
X_test = np.nan_to_num(X_test)

"""
Perform feature selection using chi2............................................................
"""
from sklearn.feature_selection import *
##
####def select_Bestfeatures(X, y):
##print("selector = SelectKBest(score_func=chi2, k=1000)")
##selector = SelectKBest(score_func=chi2, k=1000)
##fit = selector.fit(X_train,y_train)
##train_selected_features = fit.transform(X_train)
##test_selected_features = fit.transform(X_test)



"""
x_train and x_test based on the new features using chi2............................................................
"""
##X_train = select_Bestfeatures(X_train, y_train)
##X_test = select_Bestfeatures(X_test, y_test)


"""
Perform feature selection using f_classif............................................................
"""
from sklearn.feature_selection import *
##def f_classif_select_Bestfeatures(X, y):
##selector=SelectKBest(score_func=f_classif,k=500)
##fit = selector.fit(X_train,y_train)
##train_selected_features = fit.transform(X_train)
##test_selected_features = fit.transform(X_test)
##    selected_features = fit.transform(X)
##    return selected_features

"""
x_train and x_test based on the new features using f_classif............................................................
"""
##X_train = f_classif_select_Bestfeatures(X_train, y_train)
##X_test = f_classif_select_Bestfeatures(X_test, y_test)



"""
Perform feature selection using mutual_info_classif............................................................
"""
from sklearn.feature_selection import *
##def mutual_info_classif_select_Bestfeatures(X_tr, X_te, y):
##selector=SelectKBest(score_func=mutual_info_classif,k=100)
##fit = selector.fit(X_train,y_train)
##train_selected_features = fit.transform(X_train)
##test_selected_features = fit.transform(X_test)
##    return train_selected_features, test_selected_features

"""
x_train and x_test based on the new features using f_classif............................................................
"""
##X_train =train_selected_features 
##X_test = test_selected_features


"""
Perform feature selection using pca............................................................
"""

##from sklearn.decomposition import PCA
##def PCA_select_Bestfeatures(X):
##    # feature extraction
##    pca = PCA(n_components=200)
##    fit = pca.fit(X, y)
##    Train_selected_features = fit.transform(X)
##    return selected_features

"""
x_train and x_test based on the new features using f_classif............................................................
"""
##X_train = PCA_select_Bestfeatures(X_train)
##X_test = PCA_select_Bestfeatures(X_test)

"""
Below are a list of Classification Algorithms with different distance measres ..............................................................................
"""
"""
K-Nearest Neighbours (average aa over all the nearest neighbour selectiopn)..............................................................................
"""

k=5

print(k)
"""
8. manhattan K-Nearest Neighbours..............................................................................
"""
from sklearn.neighbors import KNeighborsClassifier
knn = KNeighborsClassifier(n_neighbors = k, metric= "manhattan")
knn.fit(X_train, y_train)
y_pred = knn.predict(X_test)
print("manhattan K-Nearest Neighbours")
print(metrics.accuracy_score(y_test, y_pred))

    
"""
8. cosine K-Nearest Neighbours..............................................................................
"""
from sklearn.neighbors import KNeighborsClassifier
knn = KNeighborsClassifier(n_neighbors = k, metric= "cosine")
knn.fit(X_train, y_train)
y_pred = knn.predict(X_test)
print("cosine K-Nearest Neighbours")
print(metrics.accuracy_score(y_test, y_pred))


"""
8. euclidean K-Nearest Neighbours..............................................................................
"""
from sklearn.neighbors import KNeighborsClassifier
knn = KNeighborsClassifier(n_neighbors = k, metric= "euclidean")
knn.fit(X_train, y_train)
y_pred = knn.predict(X_test)
print("euclidean K-Nearest Neighbours")
print(metrics.accuracy_score(y_test, y_pred))

        
"""
9. scipy.spatial.distance K-Nearest Neighbours..............................................................................
"""
from scipy.spatial.distance import *
knn = KNeighborsClassifier(n_neighbors = k, metric= "braycurtis")
knn.fit(X_train, y_train)
y_pred = knn.predict(X_test)
print("braycurtis K-Nearest Neighbours")
print(metrics.accuracy_score(y_test, y_pred))
       
        
knn = KNeighborsClassifier(n_neighbors = k, metric= "canberra")
knn.fit(X_train, y_train)
y_pred = knn.predict(X_test)
print("canberra K-Nearest Neighbours")
print(metrics.accuracy_score(y_test, y_pred))
  
        
knn = KNeighborsClassifier(n_neighbors = k, metric= "jaccard")
knn.fit(X_train, y_train)
y_pred = knn.predict(X_test)
print("jaccard K-Nearest Neighbours")
print(metrics.accuracy_score(y_test, y_pred))
        
knn = KNeighborsClassifier(n_neighbors = k, metric= "yule")
knn.fit(X_train, y_train)
y_pred = knn.predict(X_test)
print("yule K-Nearest Neighbours")
print(metrics.accuracy_score(y_test, y_pred))
print("************************************ ****************************")





txt_files of features used ['/Users/catherine/Desktop/NLP/PAN Datasets/EB2014Selection/output2014PMI.1M.txt', '/Users/catherine/Desktop/NLP/PAN Datasets/EB2014Selection/output2014PMI.2F.txt']
/Users/catherine/Desktop/NLP/PAN Datasets/EB2014Selection/output2014PMI.1M.txt 150
/Users/catherine/Desktop/NLP/PAN Datasets/EB2014Selection/output2014PMI.2F.txt 150
length of features used = 300
len(all_training_text) 147
len(X_train)= 147 len(y_train)= 147
len(all_test_text) 78
len(X_test= 78 len(y_test)= 78
5
manhattan K-Nearest Neighbours
0.5
cosine K-Nearest Neighbours
0.5
euclidean K-Nearest Neighbours
0.5
braycurtis K-Nearest Neighbours
0.5
canberra K-Nearest Neighbours
0.5
jaccard K-Nearest Neighbours
0.5
yule K-Nearest Neighbours
0.5
************************************ ****************************


/anaconda3/lib/python3.7/site-packages/sklearn/utils/validation.py:475: DataConversionWarning: Data with input dtype float64 was converted to bool by check_pairwise_arrays.
  warnings.warn(msg, DataConversionWarning)


In [10]:

import sys
sys.path.append("/anaconda3/lib/python3.7/site-packages")
import numpy as np
import numpy
import nltk
import pickle
import pandas as pd
import glob
import os
import re
import operator
from collections import Counter
#from read_xml_files import *
##from s_stemmer import *
from nltk.stem.porter import PorterStemmer
from nltk.stem import LancasterStemmer, WordNetLemmatizer
from itertools import groupby
from nltk.collocations import *
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.cluster import KMeans
from scipy.cluster.vq import whiten
from nltk.tokenize.toktok import ToktokTokenizer
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2
import xml.etree.ElementTree as ET
import pandas as pd
#Import scikit-learn metrics module for accuracy calculation
from sklearn import metrics
##from pattern.text.en import singularize
##nltk.download('vader_lexicon')
##nltk.download('punkt')
#import the relevant modules from the NLTK library
from nltk.sentiment.vader import SentimentIntensityAnalyzer
sentence_tokenizer = nltk.data.load('tokenizers/punkt/english.pickle')
word_tokenizer = nltk.tokenize.RegexpTokenizer(r'\w+')
##tt = ToktokTokenizer()#keeps the url as is...................................................
from nltk.tokenize import TweetTokenizer
tt = TweetTokenizer()
Porter_stemmer = PorterStemmer()
Lancaster_stemmer = LancasterStemmer()
WordNet_lemmatizer = WordNetLemmatizer()
#nltk.download('averaged_perceptron_tagger')
##Train_path="D:/Data/PAN2015-Profiling/pan15-author-profiling-training-dataset-english-2015-04-23"#path for train data
##Test_path="D:/Data/PAN2015-Profiling/pan-ap2015-test/en" #path for test data

##Train_path="D:/NLP/PAN2014/pan14-author-profiling-training-corpus-english-twitter-2014-04-16"#path for train data D:\Data\PAN2013-Profiling\Training\en  D:\NLP\PAN2014
##train_truth_path="D:/NLP/PAN2014/truth_train"
##
##test_truth_path="D:/NLP/PAN2014/pan14_test"
##Test_path="D:/NLP/PAN2014/pan14_test/en"
##Content = open('results_of_clssification_all.txt', 'w')

M_train_file= "/Users/catherine/Desktop/NLP/PAN Datasets/PAN_tokens/2014_blog_Tokens/PAN2014_blog_male_train.txt"
F_train_file= "/Users/catherine/Desktop/NLP/PAN Datasets/PAN_tokens/2014_blog_Tokens/PAN2014_blog_female_train.txt"

M_test_file= "/Users/catherine/Desktop/NLP/PAN Datasets/PAN_tokens/2014_blog_Tokens/PAN2014_blog_male_test.txt"
F_test_file= "/Users/catherine/Desktop/NLP/PAN Datasets/PAN_tokens/2014_blog_Tokens/PAN2014_blog_female_test.txt"

def all_txt(M_txt_file,  F_txt_file):
    with open(M_txt_file, mode="r", encoding="utf-8") as f:
            txt_list = [(line[:-1]).split("\t") for line in f]
            male_dict = dict()
            content=[]
            for line in txt_list:
                if line[1] in male_dict:
                # append the new number to the existing array at this slot
                    male_dict[line[1]].append(line[3])
                else:
                # create a new array in this slot
                    male_dict[line[1]] = [line[3]]
    male_content= [("M", (' '.join(a)).split()) for a in list(male_dict.values())]
  
    with open(F_txt_file, mode="r", encoding="utf-8") as f:
            txt_list = [(line[:-1]).split("\t") for line in f]
            female_dict = dict()
            content=[]
            for line in txt_list:
                if line[1] in female_dict:
                # append the new number to the existing array at this slot
                    female_dict[line[1]].append(line[3])
                else:
                # create a new array in this slot
                    female_dict[line[1]] = [line[3]]
    female_content= [("F", (' '.join(a)).split()) for a in list(female_dict.values())]
    all_txt_per_person = male_content + female_content
    return all_txt_per_person

##print( [((all_txt(M_txt_file,  F_txt_file))[i][0])  for i in range(len((all_txt(M_txt_file,  F_txt_file))))]) helps to determine the gender....

      
"""
create feature vector from the training set.....................................................................................
"""

def WordFeatures(word_list, all_training_text):

    fvs_words = np.array([[author.count(word) for word in word_list] for author in all_training_text]).astype(np.float64)

    # normalise by dividing each row by number of tokens for each author........
    fvs_words /= np.c_[np.array([len(author) for author in all_training_text])]

    return fvs_words



"""
From the saved csv file, recover the saved features to be used...............................................................
"""
import csv
word_list=[]
txt_files =[ "/Users/catherine/Desktop/NLP/PAN Datasets/EB2014Selection/output2014PMI.1M.txt", "/Users/catherine/Desktop/NLP/PAN Datasets/EB2014Selection/output2014PMI.2F.txt"]
##txt_files =[ "/Users/catherine/Desktop/NLP/PAN Datasets/PAN_tokens/2014_blog_Tokens/PAN2014_blog_female_chosen_set.csv", "/Users/catherine/Desktop/NLP/PAN Datasets/PAN_tokens/2014_blog_Tokens/PAN2014_blog_male_chosen_set.csv"]
print("txt_files of features used",txt_files)
for txt_file in txt_files:
    with open(txt_file, mode="r", encoding="utf-8") as f:
##        reader = csv.reader(f, delimiter=",")
##        next(reader) # skip header
        word_list1 =  ([(r.split(":"))[1] for r in f])[0:250]
        print(txt_file, len(word_list1))
        word_list = word_list + word_list1
print("length of features used =", len(word_list))

#for txt_file in txt_files:
 #   with open(txt_file, mode="r", encoding="utf-8") as f:
 #       reader = csv.reader(f, delimiter=",")
 #       next(reader) # skip header
 #       word_list1 =  [r[0] for r in reader]
 #       print(txt_file, len(word_list1))
 #       word_list = word_list + word_list1
#print("length of features used =", len(word_list))


"""
Prepare the training and test sets to be parsed to the classifies............................................................
"""
A=(all_txt(M_train_file,  F_train_file))
all_training_text = ( [(A[i][1])  for i in range(len(A))])
print("len(all_training_text)",len(all_training_text))
X_train=WordFeatures(word_list, all_training_text)
y_train=np.array( [(A[i][0])  for i in range(len(A))])
print("len(X_train)=",len(X_train), "len(y_train)=", len(y_train))

B=(all_txt(M_test_file,  F_test_file))
all_test_text = ( [(B[i][1])  for i in range(len(B))])
print("len(all_test_text)",len(all_test_text))
X_test = WordFeatures(word_list, all_test_text)
y_test = np.array( [(B[i][0])  for i in range(len(B))])
print("len(X_test=",len(X_test), "len(y_test)=",len(y_test))


X_train = np.nan_to_num(X_train)
X_test = np.nan_to_num(X_test)

"""
Perform feature selection using chi2............................................................
"""
from sklearn.feature_selection import *
##
####def select_Bestfeatures(X, y):
##print("selector = SelectKBest(score_func=chi2, k=1000)")
##selector = SelectKBest(score_func=chi2, k=1000)
##fit = selector.fit(X_train,y_train)
##train_selected_features = fit.transform(X_train)
##test_selected_features = fit.transform(X_test)



"""
x_train and x_test based on the new features using chi2............................................................
"""
##X_train = select_Bestfeatures(X_train, y_train)
##X_test = select_Bestfeatures(X_test, y_test)


"""
Perform feature selection using f_classif............................................................
"""
from sklearn.feature_selection import *
##def f_classif_select_Bestfeatures(X, y):
##selector=SelectKBest(score_func=f_classif,k=500)
##fit = selector.fit(X_train,y_train)
##train_selected_features = fit.transform(X_train)
##test_selected_features = fit.transform(X_test)
##    selected_features = fit.transform(X)
##    return selected_features

"""
x_train and x_test based on the new features using f_classif............................................................
"""
##X_train = f_classif_select_Bestfeatures(X_train, y_train)
##X_test = f_classif_select_Bestfeatures(X_test, y_test)



"""
Perform feature selection using mutual_info_classif............................................................
"""
from sklearn.feature_selection import *
##def mutual_info_classif_select_Bestfeatures(X_tr, X_te, y):
##selector=SelectKBest(score_func=mutual_info_classif,k=100)
##fit = selector.fit(X_train,y_train)
##train_selected_features = fit.transform(X_train)
##test_selected_features = fit.transform(X_test)
##    return train_selected_features, test_selected_features

"""
x_train and x_test based on the new features using f_classif............................................................
"""
##X_train =train_selected_features 
##X_test = test_selected_features


"""
Perform feature selection using pca............................................................
"""

##from sklearn.decomposition import PCA
##def PCA_select_Bestfeatures(X):
##    # feature extraction
##    pca = PCA(n_components=200)
##    fit = pca.fit(X, y)
##    Train_selected_features = fit.transform(X)
##    return selected_features

"""
x_train and x_test based on the new features using f_classif............................................................
"""
##X_train = PCA_select_Bestfeatures(X_train)
##X_test = PCA_select_Bestfeatures(X_test)

"""
Below are a list of Classification Algorithms with different distance measres ..............................................................................
"""
"""
K-Nearest Neighbours (average aa over all the nearest neighbour selectiopn)..............................................................................
"""

k=5

print(k)
"""
8. manhattan K-Nearest Neighbours..............................................................................
"""
from sklearn.neighbors import KNeighborsClassifier
knn = KNeighborsClassifier(n_neighbors = k, metric= "manhattan")
knn.fit(X_train, y_train)
y_pred = knn.predict(X_test)
print("manhattan K-Nearest Neighbours")
print(metrics.accuracy_score(y_test, y_pred))

    
"""
8. cosine K-Nearest Neighbours..............................................................................
"""
from sklearn.neighbors import KNeighborsClassifier
knn = KNeighborsClassifier(n_neighbors = k, metric= "cosine")
knn.fit(X_train, y_train)
y_pred = knn.predict(X_test)
print("cosine K-Nearest Neighbours")
print(metrics.accuracy_score(y_test, y_pred))


"""
8. euclidean K-Nearest Neighbours..............................................................................
"""
from sklearn.neighbors import KNeighborsClassifier
knn = KNeighborsClassifier(n_neighbors = k, metric= "euclidean")
knn.fit(X_train, y_train)
y_pred = knn.predict(X_test)
print("euclidean K-Nearest Neighbours")
print(metrics.accuracy_score(y_test, y_pred))

        
"""
9. scipy.spatial.distance K-Nearest Neighbours..............................................................................
"""
from scipy.spatial.distance import *
knn = KNeighborsClassifier(n_neighbors = k, metric= "braycurtis")
knn.fit(X_train, y_train)
y_pred = knn.predict(X_test)
print("braycurtis K-Nearest Neighbours")
print(metrics.accuracy_score(y_test, y_pred))
       
        
knn = KNeighborsClassifier(n_neighbors = k, metric= "canberra")
knn.fit(X_train, y_train)
y_pred = knn.predict(X_test)
print("canberra K-Nearest Neighbours")
print(metrics.accuracy_score(y_test, y_pred))
  
        
knn = KNeighborsClassifier(n_neighbors = k, metric= "jaccard")
knn.fit(X_train, y_train)
y_pred = knn.predict(X_test)
print("jaccard K-Nearest Neighbours")
print(metrics.accuracy_score(y_test, y_pred))
        
knn = KNeighborsClassifier(n_neighbors = k, metric= "yule")
knn.fit(X_train, y_train)
y_pred = knn.predict(X_test)
print("yule K-Nearest Neighbours")
print(metrics.accuracy_score(y_test, y_pred))
print("************************************ ****************************")



txt_files of features used ['/Users/catherine/Desktop/NLP/PAN Datasets/EB2014Selection/output2014PMI.1M.txt', '/Users/catherine/Desktop/NLP/PAN Datasets/EB2014Selection/output2014PMI.2F.txt']
/Users/catherine/Desktop/NLP/PAN Datasets/EB2014Selection/output2014PMI.1M.txt 250
/Users/catherine/Desktop/NLP/PAN Datasets/EB2014Selection/output2014PMI.2F.txt 250
length of features used = 500
len(all_training_text) 147
len(X_train)= 147 len(y_train)= 147
len(all_test_text) 78
len(X_test= 78 len(y_test)= 78
5
manhattan K-Nearest Neighbours
0.5
cosine K-Nearest Neighbours
0.5
euclidean K-Nearest Neighbours
0.5
braycurtis K-Nearest Neighbours
0.5
canberra K-Nearest Neighbours
0.5
jaccard K-Nearest Neighbours
0.5
yule K-Nearest Neighbours
0.5
************************************ ****************************


/anaconda3/lib/python3.7/site-packages/sklearn/utils/validation.py:475: DataConversionWarning: Data with input dtype float64 was converted to bool by check_pairwise_arrays.
  warnings.warn(msg, DataConversionWarning)


In [11]:

import sys
sys.path.append("/anaconda3/lib/python3.7/site-packages")
import numpy as np
import numpy
import nltk
import pickle
import pandas as pd
import glob
import os
import re
import operator
from collections import Counter
#from read_xml_files import *
##from s_stemmer import *
from nltk.stem.porter import PorterStemmer
from nltk.stem import LancasterStemmer, WordNetLemmatizer
from itertools import groupby
from nltk.collocations import *
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.cluster import KMeans
from scipy.cluster.vq import whiten
from nltk.tokenize.toktok import ToktokTokenizer
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2
import xml.etree.ElementTree as ET
import pandas as pd
#Import scikit-learn metrics module for accuracy calculation
from sklearn import metrics
##from pattern.text.en import singularize
##nltk.download('vader_lexicon')
##nltk.download('punkt')
#import the relevant modules from the NLTK library
from nltk.sentiment.vader import SentimentIntensityAnalyzer
sentence_tokenizer = nltk.data.load('tokenizers/punkt/english.pickle')
word_tokenizer = nltk.tokenize.RegexpTokenizer(r'\w+')
##tt = ToktokTokenizer()#keeps the url as is...................................................
from nltk.tokenize import TweetTokenizer
tt = TweetTokenizer()
Porter_stemmer = PorterStemmer()
Lancaster_stemmer = LancasterStemmer()
WordNet_lemmatizer = WordNetLemmatizer()
#nltk.download('averaged_perceptron_tagger')
##Train_path="D:/Data/PAN2015-Profiling/pan15-author-profiling-training-dataset-english-2015-04-23"#path for train data
##Test_path="D:/Data/PAN2015-Profiling/pan-ap2015-test/en" #path for test data

##Train_path="D:/NLP/PAN2014/pan14-author-profiling-training-corpus-english-twitter-2014-04-16"#path for train data D:\Data\PAN2013-Profiling\Training\en  D:\NLP\PAN2014
##train_truth_path="D:/NLP/PAN2014/truth_train"
##
##test_truth_path="D:/NLP/PAN2014/pan14_test"
##Test_path="D:/NLP/PAN2014/pan14_test/en"
##Content = open('results_of_clssification_all.txt', 'w')

M_train_file= "/Users/catherine/Desktop/NLP/PAN Datasets/PAN_tokens/2014_blog_Tokens/PAN2014_blog_male_train.txt"
F_train_file= "/Users/catherine/Desktop/NLP/PAN Datasets/PAN_tokens/2014_blog_Tokens/PAN2014_blog_female_train.txt"

M_test_file= "/Users/catherine/Desktop/NLP/PAN Datasets/PAN_tokens/2014_blog_Tokens/PAN2014_blog_male_test.txt"
F_test_file= "/Users/catherine/Desktop/NLP/PAN Datasets/PAN_tokens/2014_blog_Tokens/PAN2014_blog_female_test.txt"

def all_txt(M_txt_file,  F_txt_file):
    with open(M_txt_file, mode="r", encoding="utf-8") as f:
            txt_list = [(line[:-1]).split("\t") for line in f]
            male_dict = dict()
            content=[]
            for line in txt_list:
                if line[1] in male_dict:
                # append the new number to the existing array at this slot
                    male_dict[line[1]].append(line[3])
                else:
                # create a new array in this slot
                    male_dict[line[1]] = [line[3]]
    male_content= [("M", (' '.join(a)).split()) for a in list(male_dict.values())]
  
    with open(F_txt_file, mode="r", encoding="utf-8") as f:
            txt_list = [(line[:-1]).split("\t") for line in f]
            female_dict = dict()
            content=[]
            for line in txt_list:
                if line[1] in female_dict:
                # append the new number to the existing array at this slot
                    female_dict[line[1]].append(line[3])
                else:
                # create a new array in this slot
                    female_dict[line[1]] = [line[3]]
    female_content= [("F", (' '.join(a)).split()) for a in list(female_dict.values())]
    all_txt_per_person = male_content + female_content
    return all_txt_per_person

##print( [((all_txt(M_txt_file,  F_txt_file))[i][0])  for i in range(len((all_txt(M_txt_file,  F_txt_file))))]) helps to determine the gender....

      
"""
create feature vector from the training set.....................................................................................
"""

def WordFeatures(word_list, all_training_text):

    fvs_words = np.array([[author.count(word) for word in word_list] for author in all_training_text]).astype(np.float64)

    # normalise by dividing each row by number of tokens for each author........
    fvs_words /= np.c_[np.array([len(author) for author in all_training_text])]

    return fvs_words



"""
From the saved csv file, recover the saved features to be used...............................................................
"""
import csv
word_list=[]
txt_files =[ "/Users/catherine/Desktop/NLP/PAN Datasets/EB2014Selection/output2014ProbD.1M.txt", "/Users/catherine/Desktop/NLP/PAN Datasets/EB2014Selection/output2014ProbD.2F.txt"]
##txt_files =[ "/Users/catherine/Desktop/NLP/PAN Datasets/PAN_tokens/2014_blog_Tokens/PAN2014_blog_female_chosen_set.csv", "/Users/catherine/Desktop/NLP/PAN Datasets/PAN_tokens/2014_blog_Tokens/PAN2014_blog_male_chosen_set.csv"]
print("txt_files of features used",txt_files)
for txt_file in txt_files:
    with open(txt_file, mode="r", encoding="utf-8") as f:
##        reader = csv.reader(f, delimiter=",")
##        next(reader) # skip header
        word_list1 =  ([(r.split(":"))[1] for r in f])[0:100]
        print(txt_file, len(word_list1))
        word_list = word_list + word_list1
print("length of features used =", len(word_list))

#for txt_file in txt_files:
 #   with open(txt_file, mode="r", encoding="utf-8") as f:
 #       reader = csv.reader(f, delimiter=",")
 #       next(reader) # skip header
 #       word_list1 =  [r[0] for r in reader]
 #       print(txt_file, len(word_list1))
 #       word_list = word_list + word_list1
#print("length of features used =", len(word_list))


"""
Prepare the training and test sets to be parsed to the classifies............................................................
"""
A=(all_txt(M_train_file,  F_train_file))
all_training_text = ( [(A[i][1])  for i in range(len(A))])
print("len(all_training_text)",len(all_training_text))
X_train=WordFeatures(word_list, all_training_text)
y_train=np.array( [(A[i][0])  for i in range(len(A))])
print("len(X_train)=",len(X_train), "len(y_train)=", len(y_train))

B=(all_txt(M_test_file,  F_test_file))
all_test_text = ( [(B[i][1])  for i in range(len(B))])
print("len(all_test_text)",len(all_test_text))
X_test = WordFeatures(word_list, all_test_text)
y_test = np.array( [(B[i][0])  for i in range(len(B))])
print("len(X_test=",len(X_test), "len(y_test)=",len(y_test))


X_train = np.nan_to_num(X_train)
X_test = np.nan_to_num(X_test)

"""
Perform feature selection using chi2............................................................
"""
from sklearn.feature_selection import *
##
####def select_Bestfeatures(X, y):
##print("selector = SelectKBest(score_func=chi2, k=1000)")
##selector = SelectKBest(score_func=chi2, k=1000)
##fit = selector.fit(X_train,y_train)
##train_selected_features = fit.transform(X_train)
##test_selected_features = fit.transform(X_test)



"""
x_train and x_test based on the new features using chi2............................................................
"""
##X_train = select_Bestfeatures(X_train, y_train)
##X_test = select_Bestfeatures(X_test, y_test)


"""
Perform feature selection using f_classif............................................................
"""
from sklearn.feature_selection import *
##def f_classif_select_Bestfeatures(X, y):
##selector=SelectKBest(score_func=f_classif,k=500)
##fit = selector.fit(X_train,y_train)
##train_selected_features = fit.transform(X_train)
##test_selected_features = fit.transform(X_test)
##    selected_features = fit.transform(X)
##    return selected_features

"""
x_train and x_test based on the new features using f_classif............................................................
"""
##X_train = f_classif_select_Bestfeatures(X_train, y_train)
##X_test = f_classif_select_Bestfeatures(X_test, y_test)



"""
Perform feature selection using mutual_info_classif............................................................
"""
from sklearn.feature_selection import *
##def mutual_info_classif_select_Bestfeatures(X_tr, X_te, y):
##selector=SelectKBest(score_func=mutual_info_classif,k=100)
##fit = selector.fit(X_train,y_train)
##train_selected_features = fit.transform(X_train)
##test_selected_features = fit.transform(X_test)
##    return train_selected_features, test_selected_features

"""
x_train and x_test based on the new features using f_classif............................................................
"""
##X_train =train_selected_features 
##X_test = test_selected_features


"""
Perform feature selection using pca............................................................
"""

##from sklearn.decomposition import PCA
##def PCA_select_Bestfeatures(X):
##    # feature extraction
##    pca = PCA(n_components=200)
##    fit = pca.fit(X, y)
##    Train_selected_features = fit.transform(X)
##    return selected_features

"""
x_train and x_test based on the new features using f_classif............................................................
"""
##X_train = PCA_select_Bestfeatures(X_train)
##X_test = PCA_select_Bestfeatures(X_test)

"""
Below are a list of Classification Algorithms with different distance measres ..............................................................................
"""
"""
K-Nearest Neighbours (average aa over all the nearest neighbour selectiopn)..............................................................................
"""

k=5

print(k)
"""
8. manhattan K-Nearest Neighbours..............................................................................
"""
from sklearn.neighbors import KNeighborsClassifier
knn = KNeighborsClassifier(n_neighbors = k, metric= "manhattan")
knn.fit(X_train, y_train)
y_pred = knn.predict(X_test)
print("manhattan K-Nearest Neighbours")
print(metrics.accuracy_score(y_test, y_pred))

    
"""
8. cosine K-Nearest Neighbours..............................................................................
"""
from sklearn.neighbors import KNeighborsClassifier
knn = KNeighborsClassifier(n_neighbors = k, metric= "cosine")
knn.fit(X_train, y_train)
y_pred = knn.predict(X_test)
print("cosine K-Nearest Neighbours")
print(metrics.accuracy_score(y_test, y_pred))


"""
8. euclidean K-Nearest Neighbours..............................................................................
"""
from sklearn.neighbors import KNeighborsClassifier
knn = KNeighborsClassifier(n_neighbors = k, metric= "euclidean")
knn.fit(X_train, y_train)
y_pred = knn.predict(X_test)
print("euclidean K-Nearest Neighbours")
print(metrics.accuracy_score(y_test, y_pred))

        
"""
9. scipy.spatial.distance K-Nearest Neighbours..............................................................................
"""
from scipy.spatial.distance import *
knn = KNeighborsClassifier(n_neighbors = k, metric= "braycurtis")
knn.fit(X_train, y_train)
y_pred = knn.predict(X_test)
print("braycurtis K-Nearest Neighbours")
print(metrics.accuracy_score(y_test, y_pred))
       
        
knn = KNeighborsClassifier(n_neighbors = k, metric= "canberra")
knn.fit(X_train, y_train)
y_pred = knn.predict(X_test)
print("canberra K-Nearest Neighbours")
print(metrics.accuracy_score(y_test, y_pred))
  
        
knn = KNeighborsClassifier(n_neighbors = k, metric= "jaccard")
knn.fit(X_train, y_train)
y_pred = knn.predict(X_test)
print("jaccard K-Nearest Neighbours")
print(metrics.accuracy_score(y_test, y_pred))
        
knn = KNeighborsClassifier(n_neighbors = k, metric= "yule")
knn.fit(X_train, y_train)
y_pred = knn.predict(X_test)
print("yule K-Nearest Neighbours")
print(metrics.accuracy_score(y_test, y_pred))
print("************************************ ****************************")





txt_files of features used ['/Users/catherine/Desktop/NLP/PAN Datasets/EB2014Selection/output2014ProbD.1M.txt', '/Users/catherine/Desktop/NLP/PAN Datasets/EB2014Selection/output2014ProbD.2F.txt']
/Users/catherine/Desktop/NLP/PAN Datasets/EB2014Selection/output2014ProbD.1M.txt 100
/Users/catherine/Desktop/NLP/PAN Datasets/EB2014Selection/output2014ProbD.2F.txt 100
length of features used = 200
len(all_training_text) 147
len(X_train)= 147 len(y_train)= 147
len(all_test_text) 78
len(X_test= 78 len(y_test)= 78
5
manhattan K-Nearest Neighbours
0.5
cosine K-Nearest Neighbours
0.5
euclidean K-Nearest Neighbours
0.5
braycurtis K-Nearest Neighbours
0.5
canberra K-Nearest Neighbours
0.5
jaccard K-Nearest Neighbours
0.5
yule K-Nearest Neighbours
0.5
************************************ ****************************


/anaconda3/lib/python3.7/site-packages/sklearn/utils/validation.py:475: DataConversionWarning: Data with input dtype float64 was converted to bool by check_pairwise_arrays.
  warnings.warn(msg, DataConversionWarning)


In [12]:

import sys
sys.path.append("/anaconda3/lib/python3.7/site-packages")
import numpy as np
import numpy
import nltk
import pickle
import pandas as pd
import glob
import os
import re
import operator
from collections import Counter
#from read_xml_files import *
##from s_stemmer import *
from nltk.stem.porter import PorterStemmer
from nltk.stem import LancasterStemmer, WordNetLemmatizer
from itertools import groupby
from nltk.collocations import *
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.cluster import KMeans
from scipy.cluster.vq import whiten
from nltk.tokenize.toktok import ToktokTokenizer
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2
import xml.etree.ElementTree as ET
import pandas as pd
#Import scikit-learn metrics module for accuracy calculation
from sklearn import metrics
##from pattern.text.en import singularize
##nltk.download('vader_lexicon')
##nltk.download('punkt')
#import the relevant modules from the NLTK library
from nltk.sentiment.vader import SentimentIntensityAnalyzer
sentence_tokenizer = nltk.data.load('tokenizers/punkt/english.pickle')
word_tokenizer = nltk.tokenize.RegexpTokenizer(r'\w+')
##tt = ToktokTokenizer()#keeps the url as is...................................................
from nltk.tokenize import TweetTokenizer
tt = TweetTokenizer()
Porter_stemmer = PorterStemmer()
Lancaster_stemmer = LancasterStemmer()
WordNet_lemmatizer = WordNetLemmatizer()
#nltk.download('averaged_perceptron_tagger')
##Train_path="D:/Data/PAN2015-Profiling/pan15-author-profiling-training-dataset-english-2015-04-23"#path for train data
##Test_path="D:/Data/PAN2015-Profiling/pan-ap2015-test/en" #path for test data

##Train_path="D:/NLP/PAN2014/pan14-author-profiling-training-corpus-english-twitter-2014-04-16"#path for train data D:\Data\PAN2013-Profiling\Training\en  D:\NLP\PAN2014
##train_truth_path="D:/NLP/PAN2014/truth_train"
##
##test_truth_path="D:/NLP/PAN2014/pan14_test"
##Test_path="D:/NLP/PAN2014/pan14_test/en"
##Content = open('results_of_clssification_all.txt', 'w')

M_train_file= "/Users/catherine/Desktop/NLP/PAN Datasets/PAN_tokens/2014_blog_Tokens/PAN2014_blog_male_train.txt"
F_train_file= "/Users/catherine/Desktop/NLP/PAN Datasets/PAN_tokens/2014_blog_Tokens/PAN2014_blog_female_train.txt"

M_test_file= "/Users/catherine/Desktop/NLP/PAN Datasets/PAN_tokens/2014_blog_Tokens/PAN2014_blog_male_test.txt"
F_test_file= "/Users/catherine/Desktop/NLP/PAN Datasets/PAN_tokens/2014_blog_Tokens/PAN2014_blog_female_test.txt"

def all_txt(M_txt_file,  F_txt_file):
    with open(M_txt_file, mode="r", encoding="utf-8") as f:
            txt_list = [(line[:-1]).split("\t") for line in f]
            male_dict = dict()
            content=[]
            for line in txt_list:
                if line[1] in male_dict:
                # append the new number to the existing array at this slot
                    male_dict[line[1]].append(line[3])
                else:
                # create a new array in this slot
                    male_dict[line[1]] = [line[3]]
    male_content= [("M", (' '.join(a)).split()) for a in list(male_dict.values())]
  
    with open(F_txt_file, mode="r", encoding="utf-8") as f:
            txt_list = [(line[:-1]).split("\t") for line in f]
            female_dict = dict()
            content=[]
            for line in txt_list:
                if line[1] in female_dict:
                # append the new number to the existing array at this slot
                    female_dict[line[1]].append(line[3])
                else:
                # create a new array in this slot
                    female_dict[line[1]] = [line[3]]
    female_content= [("F", (' '.join(a)).split()) for a in list(female_dict.values())]
    all_txt_per_person = male_content + female_content
    return all_txt_per_person

##print( [((all_txt(M_txt_file,  F_txt_file))[i][0])  for i in range(len((all_txt(M_txt_file,  F_txt_file))))]) helps to determine the gender....

      
"""
create feature vector from the training set.....................................................................................
"""

def WordFeatures(word_list, all_training_text):

    fvs_words = np.array([[author.count(word) for word in word_list] for author in all_training_text]).astype(np.float64)

    # normalise by dividing each row by number of tokens for each author........
    fvs_words /= np.c_[np.array([len(author) for author in all_training_text])]

    return fvs_words



"""
From the saved csv file, recover the saved features to be used...............................................................
"""
import csv
word_list=[]
txt_files =[ "/Users/catherine/Desktop/NLP/PAN Datasets/EB2014Selection/output2014ProbD.1M.txt", "/Users/catherine/Desktop/NLP/PAN Datasets/EB2014Selection/output2014ProbD.2F.txt"]
##txt_files =[ "/Users/catherine/Desktop/NLP/PAN Datasets/PAN_tokens/2014_blog_Tokens/PAN2014_blog_female_chosen_set.csv", "/Users/catherine/Desktop/NLP/PAN Datasets/PAN_tokens/2014_blog_Tokens/PAN2014_blog_male_chosen_set.csv"]
print("txt_files of features used",txt_files)
for txt_file in txt_files:
    with open(txt_file, mode="r", encoding="utf-8") as f:
##        reader = csv.reader(f, delimiter=",")
##        next(reader) # skip header
        word_list1 =  ([(r.split(":"))[1] for r in f])[0:150]
        print(txt_file, len(word_list1))
        word_list = word_list + word_list1
print("length of features used =", len(word_list))

#for txt_file in txt_files:
 #   with open(txt_file, mode="r", encoding="utf-8") as f:
 #       reader = csv.reader(f, delimiter=",")
 #       next(reader) # skip header
 #       word_list1 =  [r[0] for r in reader]
 #       print(txt_file, len(word_list1))
 #       word_list = word_list + word_list1
#print("length of features used =", len(word_list))


"""
Prepare the training and test sets to be parsed to the classifies............................................................
"""
A=(all_txt(M_train_file,  F_train_file))
all_training_text = ( [(A[i][1])  for i in range(len(A))])
print("len(all_training_text)",len(all_training_text))
X_train=WordFeatures(word_list, all_training_text)
y_train=np.array( [(A[i][0])  for i in range(len(A))])
print("len(X_train)=",len(X_train), "len(y_train)=", len(y_train))

B=(all_txt(M_test_file,  F_test_file))
all_test_text = ( [(B[i][1])  for i in range(len(B))])
print("len(all_test_text)",len(all_test_text))
X_test = WordFeatures(word_list, all_test_text)
y_test = np.array( [(B[i][0])  for i in range(len(B))])
print("len(X_test=",len(X_test), "len(y_test)=",len(y_test))


X_train = np.nan_to_num(X_train)
X_test = np.nan_to_num(X_test)

"""
Perform feature selection using chi2............................................................
"""
from sklearn.feature_selection import *
##
####def select_Bestfeatures(X, y):
##print("selector = SelectKBest(score_func=chi2, k=1000)")
##selector = SelectKBest(score_func=chi2, k=1000)
##fit = selector.fit(X_train,y_train)
##train_selected_features = fit.transform(X_train)
##test_selected_features = fit.transform(X_test)



"""
x_train and x_test based on the new features using chi2............................................................
"""
##X_train = select_Bestfeatures(X_train, y_train)
##X_test = select_Bestfeatures(X_test, y_test)


"""
Perform feature selection using f_classif............................................................
"""
from sklearn.feature_selection import *
##def f_classif_select_Bestfeatures(X, y):
##selector=SelectKBest(score_func=f_classif,k=500)
##fit = selector.fit(X_train,y_train)
##train_selected_features = fit.transform(X_train)
##test_selected_features = fit.transform(X_test)
##    selected_features = fit.transform(X)
##    return selected_features

"""
x_train and x_test based on the new features using f_classif............................................................
"""
##X_train = f_classif_select_Bestfeatures(X_train, y_train)
##X_test = f_classif_select_Bestfeatures(X_test, y_test)



"""
Perform feature selection using mutual_info_classif............................................................
"""
from sklearn.feature_selection import *
##def mutual_info_classif_select_Bestfeatures(X_tr, X_te, y):
##selector=SelectKBest(score_func=mutual_info_classif,k=100)
##fit = selector.fit(X_train,y_train)
##train_selected_features = fit.transform(X_train)
##test_selected_features = fit.transform(X_test)
##    return train_selected_features, test_selected_features

"""
x_train and x_test based on the new features using f_classif............................................................
"""
##X_train =train_selected_features 
##X_test = test_selected_features


"""
Perform feature selection using pca............................................................
"""

##from sklearn.decomposition import PCA
##def PCA_select_Bestfeatures(X):
##    # feature extraction
##    pca = PCA(n_components=200)
##    fit = pca.fit(X, y)
##    Train_selected_features = fit.transform(X)
##    return selected_features

"""
x_train and x_test based on the new features using f_classif............................................................
"""
##X_train = PCA_select_Bestfeatures(X_train)
##X_test = PCA_select_Bestfeatures(X_test)

"""
Below are a list of Classification Algorithms with different distance measres ..............................................................................
"""
"""
K-Nearest Neighbours (average aa over all the nearest neighbour selectiopn)..............................................................................
"""

k=5

print(k)
"""
8. manhattan K-Nearest Neighbours..............................................................................
"""
from sklearn.neighbors import KNeighborsClassifier
knn = KNeighborsClassifier(n_neighbors = k, metric= "manhattan")
knn.fit(X_train, y_train)
y_pred = knn.predict(X_test)
print("manhattan K-Nearest Neighbours")
print(metrics.accuracy_score(y_test, y_pred))

    
"""
8. cosine K-Nearest Neighbours..............................................................................
"""
from sklearn.neighbors import KNeighborsClassifier
knn = KNeighborsClassifier(n_neighbors = k, metric= "cosine")
knn.fit(X_train, y_train)
y_pred = knn.predict(X_test)
print("cosine K-Nearest Neighbours")
print(metrics.accuracy_score(y_test, y_pred))


"""
8. euclidean K-Nearest Neighbours..............................................................................
"""
from sklearn.neighbors import KNeighborsClassifier
knn = KNeighborsClassifier(n_neighbors = k, metric= "euclidean")
knn.fit(X_train, y_train)
y_pred = knn.predict(X_test)
print("euclidean K-Nearest Neighbours")
print(metrics.accuracy_score(y_test, y_pred))

        
"""
9. scipy.spatial.distance K-Nearest Neighbours..............................................................................
"""
from scipy.spatial.distance import *
knn = KNeighborsClassifier(n_neighbors = k, metric= "braycurtis")
knn.fit(X_train, y_train)
y_pred = knn.predict(X_test)
print("braycurtis K-Nearest Neighbours")
print(metrics.accuracy_score(y_test, y_pred))
       
        
knn = KNeighborsClassifier(n_neighbors = k, metric= "canberra")
knn.fit(X_train, y_train)
y_pred = knn.predict(X_test)
print("canberra K-Nearest Neighbours")
print(metrics.accuracy_score(y_test, y_pred))
  
        
knn = KNeighborsClassifier(n_neighbors = k, metric= "jaccard")
knn.fit(X_train, y_train)
y_pred = knn.predict(X_test)
print("jaccard K-Nearest Neighbours")
print(metrics.accuracy_score(y_test, y_pred))
        
knn = KNeighborsClassifier(n_neighbors = k, metric= "yule")
knn.fit(X_train, y_train)
y_pred = knn.predict(X_test)
print("yule K-Nearest Neighbours")
print(metrics.accuracy_score(y_test, y_pred))
print("************************************ ****************************")






txt_files of features used ['/Users/catherine/Desktop/NLP/PAN Datasets/EB2014Selection/output2014ProbD.1M.txt', '/Users/catherine/Desktop/NLP/PAN Datasets/EB2014Selection/output2014ProbD.2F.txt']
/Users/catherine/Desktop/NLP/PAN Datasets/EB2014Selection/output2014ProbD.1M.txt 150
/Users/catherine/Desktop/NLP/PAN Datasets/EB2014Selection/output2014ProbD.2F.txt 150
length of features used = 300
len(all_training_text) 147
len(X_train)= 147 len(y_train)= 147
len(all_test_text) 78
len(X_test= 78 len(y_test)= 78
5
manhattan K-Nearest Neighbours
0.5
cosine K-Nearest Neighbours
0.5
euclidean K-Nearest Neighbours
0.5
braycurtis K-Nearest Neighbours
0.5
canberra K-Nearest Neighbours
0.5
jaccard K-Nearest Neighbours
0.5
yule K-Nearest Neighbours
0.5128205128205128
************************************ ****************************


/anaconda3/lib/python3.7/site-packages/sklearn/utils/validation.py:475: DataConversionWarning: Data with input dtype float64 was converted to bool by check_pairwise_arrays.
  warnings.warn(msg, DataConversionWarning)


In [13]:

import sys
sys.path.append("/anaconda3/lib/python3.7/site-packages")
import numpy as np
import numpy
import nltk
import pickle
import pandas as pd
import glob
import os
import re
import operator
from collections import Counter
#from read_xml_files import *
##from s_stemmer import *
from nltk.stem.porter import PorterStemmer
from nltk.stem import LancasterStemmer, WordNetLemmatizer
from itertools import groupby
from nltk.collocations import *
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.cluster import KMeans
from scipy.cluster.vq import whiten
from nltk.tokenize.toktok import ToktokTokenizer
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2
import xml.etree.ElementTree as ET
import pandas as pd
#Import scikit-learn metrics module for accuracy calculation
from sklearn import metrics
##from pattern.text.en import singularize
##nltk.download('vader_lexicon')
##nltk.download('punkt')
#import the relevant modules from the NLTK library
from nltk.sentiment.vader import SentimentIntensityAnalyzer
sentence_tokenizer = nltk.data.load('tokenizers/punkt/english.pickle')
word_tokenizer = nltk.tokenize.RegexpTokenizer(r'\w+')
##tt = ToktokTokenizer()#keeps the url as is...................................................
from nltk.tokenize import TweetTokenizer
tt = TweetTokenizer()
Porter_stemmer = PorterStemmer()
Lancaster_stemmer = LancasterStemmer()
WordNet_lemmatizer = WordNetLemmatizer()
#nltk.download('averaged_perceptron_tagger')
##Train_path="D:/Data/PAN2015-Profiling/pan15-author-profiling-training-dataset-english-2015-04-23"#path for train data
##Test_path="D:/Data/PAN2015-Profiling/pan-ap2015-test/en" #path for test data

##Train_path="D:/NLP/PAN2014/pan14-author-profiling-training-corpus-english-twitter-2014-04-16"#path for train data D:\Data\PAN2013-Profiling\Training\en  D:\NLP\PAN2014
##train_truth_path="D:/NLP/PAN2014/truth_train"
##
##test_truth_path="D:/NLP/PAN2014/pan14_test"
##Test_path="D:/NLP/PAN2014/pan14_test/en"
##Content = open('results_of_clssification_all.txt', 'w')

M_train_file= "/Users/catherine/Desktop/NLP/PAN Datasets/PAN_tokens/2014_blog_Tokens/PAN2014_blog_male_train.txt"
F_train_file= "/Users/catherine/Desktop/NLP/PAN Datasets/PAN_tokens/2014_blog_Tokens/PAN2014_blog_female_train.txt"

M_test_file= "/Users/catherine/Desktop/NLP/PAN Datasets/PAN_tokens/2014_blog_Tokens/PAN2014_blog_male_test.txt"
F_test_file= "/Users/catherine/Desktop/NLP/PAN Datasets/PAN_tokens/2014_blog_Tokens/PAN2014_blog_female_test.txt"

def all_txt(M_txt_file,  F_txt_file):
    with open(M_txt_file, mode="r", encoding="utf-8") as f:
            txt_list = [(line[:-1]).split("\t") for line in f]
            male_dict = dict()
            content=[]
            for line in txt_list:
                if line[1] in male_dict:
                # append the new number to the existing array at this slot
                    male_dict[line[1]].append(line[3])
                else:
                # create a new array in this slot
                    male_dict[line[1]] = [line[3]]
    male_content= [("M", (' '.join(a)).split()) for a in list(male_dict.values())]
  
    with open(F_txt_file, mode="r", encoding="utf-8") as f:
            txt_list = [(line[:-1]).split("\t") for line in f]
            female_dict = dict()
            content=[]
            for line in txt_list:
                if line[1] in female_dict:
                # append the new number to the existing array at this slot
                    female_dict[line[1]].append(line[3])
                else:
                # create a new array in this slot
                    female_dict[line[1]] = [line[3]]
    female_content= [("F", (' '.join(a)).split()) for a in list(female_dict.values())]
    all_txt_per_person = male_content + female_content
    return all_txt_per_person

##print( [((all_txt(M_txt_file,  F_txt_file))[i][0])  for i in range(len((all_txt(M_txt_file,  F_txt_file))))]) helps to determine the gender....

      
"""
create feature vector from the training set.....................................................................................
"""

def WordFeatures(word_list, all_training_text):

    fvs_words = np.array([[author.count(word) for word in word_list] for author in all_training_text]).astype(np.float64)

    # normalise by dividing each row by number of tokens for each author........
    fvs_words /= np.c_[np.array([len(author) for author in all_training_text])]

    return fvs_words



"""
From the saved csv file, recover the saved features to be used...............................................................
"""
import csv
word_list=[]
txt_files =[ "/Users/catherine/Desktop/NLP/PAN Datasets/EB2014Selection/output2014ProbD.1M.txt", "/Users/catherine/Desktop/NLP/PAN Datasets/EB2014Selection/output2014ProbD.2F.txt"]
##txt_files =[ "/Users/catherine/Desktop/NLP/PAN Datasets/PAN_tokens/2014_blog_Tokens/PAN2014_blog_female_chosen_set.csv", "/Users/catherine/Desktop/NLP/PAN Datasets/PAN_tokens/2014_blog_Tokens/PAN2014_blog_male_chosen_set.csv"]
print("txt_files of features used",txt_files)
for txt_file in txt_files:
    with open(txt_file, mode="r", encoding="utf-8") as f:
##        reader = csv.reader(f, delimiter=",")
##        next(reader) # skip header
        word_list1 =  ([(r.split(":"))[1] for r in f])[0:250]
        print(txt_file, len(word_list1))
        word_list = word_list + word_list1
print("length of features used =", len(word_list))

#for txt_file in txt_files:
 #   with open(txt_file, mode="r", encoding="utf-8") as f:
 #       reader = csv.reader(f, delimiter=",")
 #       next(reader) # skip header
 #       word_list1 =  [r[0] for r in reader]
 #       print(txt_file, len(word_list1))
 #       word_list = word_list + word_list1
#print("length of features used =", len(word_list))


"""
Prepare the training and test sets to be parsed to the classifies............................................................
"""
A=(all_txt(M_train_file,  F_train_file))
all_training_text = ( [(A[i][1])  for i in range(len(A))])
print("len(all_training_text)",len(all_training_text))
X_train=WordFeatures(word_list, all_training_text)
y_train=np.array( [(A[i][0])  for i in range(len(A))])
print("len(X_train)=",len(X_train), "len(y_train)=", len(y_train))

B=(all_txt(M_test_file,  F_test_file))
all_test_text = ( [(B[i][1])  for i in range(len(B))])
print("len(all_test_text)",len(all_test_text))
X_test = WordFeatures(word_list, all_test_text)
y_test = np.array( [(B[i][0])  for i in range(len(B))])
print("len(X_test=",len(X_test), "len(y_test)=",len(y_test))


X_train = np.nan_to_num(X_train)
X_test = np.nan_to_num(X_test)

"""
Perform feature selection using chi2............................................................
"""
from sklearn.feature_selection import *
##
####def select_Bestfeatures(X, y):
##print("selector = SelectKBest(score_func=chi2, k=1000)")
##selector = SelectKBest(score_func=chi2, k=1000)
##fit = selector.fit(X_train,y_train)
##train_selected_features = fit.transform(X_train)
##test_selected_features = fit.transform(X_test)



"""
x_train and x_test based on the new features using chi2............................................................
"""
##X_train = select_Bestfeatures(X_train, y_train)
##X_test = select_Bestfeatures(X_test, y_test)


"""
Perform feature selection using f_classif............................................................
"""
from sklearn.feature_selection import *
##def f_classif_select_Bestfeatures(X, y):
##selector=SelectKBest(score_func=f_classif,k=500)
##fit = selector.fit(X_train,y_train)
##train_selected_features = fit.transform(X_train)
##test_selected_features = fit.transform(X_test)
##    selected_features = fit.transform(X)
##    return selected_features

"""
x_train and x_test based on the new features using f_classif............................................................
"""
##X_train = f_classif_select_Bestfeatures(X_train, y_train)
##X_test = f_classif_select_Bestfeatures(X_test, y_test)



"""
Perform feature selection using mutual_info_classif............................................................
"""
from sklearn.feature_selection import *
##def mutual_info_classif_select_Bestfeatures(X_tr, X_te, y):
##selector=SelectKBest(score_func=mutual_info_classif,k=100)
##fit = selector.fit(X_train,y_train)
##train_selected_features = fit.transform(X_train)
##test_selected_features = fit.transform(X_test)
##    return train_selected_features, test_selected_features

"""
x_train and x_test based on the new features using f_classif............................................................
"""
##X_train =train_selected_features 
##X_test = test_selected_features


"""
Perform feature selection using pca............................................................
"""

##from sklearn.decomposition import PCA
##def PCA_select_Bestfeatures(X):
##    # feature extraction
##    pca = PCA(n_components=200)
##    fit = pca.fit(X, y)
##    Train_selected_features = fit.transform(X)
##    return selected_features

"""
x_train and x_test based on the new features using f_classif............................................................
"""
##X_train = PCA_select_Bestfeatures(X_train)
##X_test = PCA_select_Bestfeatures(X_test)

"""
Below are a list of Classification Algorithms with different distance measres ..............................................................................
"""
"""
K-Nearest Neighbours (average aa over all the nearest neighbour selectiopn)..............................................................................
"""

k=5

print(k)
"""
8. manhattan K-Nearest Neighbours..............................................................................
"""
from sklearn.neighbors import KNeighborsClassifier
knn = KNeighborsClassifier(n_neighbors = k, metric= "manhattan")
knn.fit(X_train, y_train)
y_pred = knn.predict(X_test)
print("manhattan K-Nearest Neighbours")
print(metrics.accuracy_score(y_test, y_pred))

    
"""
8. cosine K-Nearest Neighbours..............................................................................
"""
from sklearn.neighbors import KNeighborsClassifier
knn = KNeighborsClassifier(n_neighbors = k, metric= "cosine")
knn.fit(X_train, y_train)
y_pred = knn.predict(X_test)
print("cosine K-Nearest Neighbours")
print(metrics.accuracy_score(y_test, y_pred))


"""
8. euclidean K-Nearest Neighbours..............................................................................
"""
from sklearn.neighbors import KNeighborsClassifier
knn = KNeighborsClassifier(n_neighbors = k, metric= "euclidean")
knn.fit(X_train, y_train)
y_pred = knn.predict(X_test)
print("euclidean K-Nearest Neighbours")
print(metrics.accuracy_score(y_test, y_pred))

        
"""
9. scipy.spatial.distance K-Nearest Neighbours..............................................................................
"""
from scipy.spatial.distance import *
knn = KNeighborsClassifier(n_neighbors = k, metric= "braycurtis")
knn.fit(X_train, y_train)
y_pred = knn.predict(X_test)
print("braycurtis K-Nearest Neighbours")
print(metrics.accuracy_score(y_test, y_pred))
       
        
knn = KNeighborsClassifier(n_neighbors = k, metric= "canberra")
knn.fit(X_train, y_train)
y_pred = knn.predict(X_test)
print("canberra K-Nearest Neighbours")
print(metrics.accuracy_score(y_test, y_pred))
  
        
knn = KNeighborsClassifier(n_neighbors = k, metric= "jaccard")
knn.fit(X_train, y_train)
y_pred = knn.predict(X_test)
print("jaccard K-Nearest Neighbours")
print(metrics.accuracy_score(y_test, y_pred))
        
knn = KNeighborsClassifier(n_neighbors = k, metric= "yule")
knn.fit(X_train, y_train)
y_pred = knn.predict(X_test)
print("yule K-Nearest Neighbours")
print(metrics.accuracy_score(y_test, y_pred))
print("************************************ ****************************")






txt_files of features used ['/Users/catherine/Desktop/NLP/PAN Datasets/EB2014Selection/output2014ProbD.1M.txt', '/Users/catherine/Desktop/NLP/PAN Datasets/EB2014Selection/output2014ProbD.2F.txt']
/Users/catherine/Desktop/NLP/PAN Datasets/EB2014Selection/output2014ProbD.1M.txt 250
/Users/catherine/Desktop/NLP/PAN Datasets/EB2014Selection/output2014ProbD.2F.txt 250
length of features used = 500
len(all_training_text) 147
len(X_train)= 147 len(y_train)= 147
len(all_test_text) 78
len(X_test= 78 len(y_test)= 78
5
manhattan K-Nearest Neighbours
0.5
cosine K-Nearest Neighbours
0.5
euclidean K-Nearest Neighbours
0.5
braycurtis K-Nearest Neighbours
0.5
canberra K-Nearest Neighbours
0.5
jaccard K-Nearest Neighbours
0.5
yule K-Nearest Neighbours
0.5128205128205128
************************************ ****************************


/anaconda3/lib/python3.7/site-packages/sklearn/utils/validation.py:475: DataConversionWarning: Data with input dtype float64 was converted to bool by check_pairwise_arrays.
  warnings.warn(msg, DataConversionWarning)


In [15]:

import sys
sys.path.append("/anaconda3/lib/python3.7/site-packages")
import numpy as np
import numpy
import nltk
import pickle
import pandas as pd
import glob
import os
import re
import operator
from collections import Counter
#from read_xml_files import *
##from s_stemmer import *
from nltk.stem.porter import PorterStemmer
from nltk.stem import LancasterStemmer, WordNetLemmatizer
from itertools import groupby
from nltk.collocations import *
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.cluster import KMeans
from scipy.cluster.vq import whiten
from nltk.tokenize.toktok import ToktokTokenizer
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2
import xml.etree.ElementTree as ET
import pandas as pd
#Import scikit-learn metrics module for accuracy calculation
from sklearn import metrics
##from pattern.text.en import singularize
##nltk.download('vader_lexicon')
##nltk.download('punkt')
#import the relevant modules from the NLTK library
from nltk.sentiment.vader import SentimentIntensityAnalyzer
sentence_tokenizer = nltk.data.load('tokenizers/punkt/english.pickle')
word_tokenizer = nltk.tokenize.RegexpTokenizer(r'\w+')
##tt = ToktokTokenizer()#keeps the url as is...................................................
from nltk.tokenize import TweetTokenizer
tt = TweetTokenizer()
Porter_stemmer = PorterStemmer()
Lancaster_stemmer = LancasterStemmer()
WordNet_lemmatizer = WordNetLemmatizer()
#nltk.download('averaged_perceptron_tagger')
##Train_path="D:/Data/PAN2015-Profiling/pan15-author-profiling-training-dataset-english-2015-04-23"#path for train data
##Test_path="D:/Data/PAN2015-Profiling/pan-ap2015-test/en" #path for test data

##Train_path="D:/NLP/PAN2014/pan14-author-profiling-training-corpus-english-twitter-2014-04-16"#path for train data D:\Data\PAN2013-Profiling\Training\en  D:\NLP\PAN2014
##train_truth_path="D:/NLP/PAN2014/truth_train"
##
##test_truth_path="D:/NLP/PAN2014/pan14_test"
##Test_path="D:/NLP/PAN2014/pan14_test/en"
##Content = open('results_of_clssification_all.txt', 'w')

M_train_file= "/Users/catherine/Desktop/NLP/PAN Datasets/PAN_tokens/2014_blog_Tokens/PAN2014_blog_male_train.txt"
F_train_file= "/Users/catherine/Desktop/NLP/PAN Datasets/PAN_tokens/2014_blog_Tokens/PAN2014_blog_female_train.txt"

M_test_file= "/Users/catherine/Desktop/NLP/PAN Datasets/PAN_tokens/2014_blog_Tokens/PAN2014_blog_male_test.txt"
F_test_file= "/Users/catherine/Desktop/NLP/PAN Datasets/PAN_tokens/2014_blog_Tokens/PAN2014_blog_female_test.txt"

def all_txt(M_txt_file,  F_txt_file):
    with open(M_txt_file, mode="r", encoding="utf-8") as f:
            txt_list = [(line[:-1]).split("\t") for line in f]
            male_dict = dict()
            content=[]
            for line in txt_list:
                if line[1] in male_dict:
                # append the new number to the existing array at this slot
                    male_dict[line[1]].append(line[3])
                else:
                # create a new array in this slot
                    male_dict[line[1]] = [line[3]]
    male_content= [("M", (' '.join(a)).split()) for a in list(male_dict.values())]
  
    with open(F_txt_file, mode="r", encoding="utf-8") as f:
            txt_list = [(line[:-1]).split("\t") for line in f]
            female_dict = dict()
            content=[]
            for line in txt_list:
                if line[1] in female_dict:
                # append the new number to the existing array at this slot
                    female_dict[line[1]].append(line[3])
                else:
                # create a new array in this slot
                    female_dict[line[1]] = [line[3]]
    female_content= [("F", (' '.join(a)).split()) for a in list(female_dict.values())]
    all_txt_per_person = male_content + female_content
    return all_txt_per_person

##print( [((all_txt(M_txt_file,  F_txt_file))[i][0])  for i in range(len((all_txt(M_txt_file,  F_txt_file))))]) helps to determine the gender....

      
"""
create feature vector from the training set.....................................................................................
"""

def WordFeatures(word_list, all_training_text):

    fvs_words = np.array([[author.count(word) for word in word_list] for author in all_training_text]).astype(np.float64)

    # normalise by dividing each row by number of tokens for each author........
    fvs_words /= np.c_[np.array([len(author) for author in all_training_text])]

    return fvs_words



"""
From the saved csv file, recover the saved features to be used...............................................................
"""
import csv
word_list=[]
txt_files =["/Users/catherine/Desktop/NLP/PAN Datasets/EB2014Selection/output2014CHI.txt"]
##txt_files =[ "/Users/catherine/Desktop/NLP/PAN Datasets/PAN_tokens/2014_blog_Tokens/PAN2014_blog_female_chosen_set.csv", "/Users/catherine/Desktop/NLP/PAN Datasets/PAN_tokens/2014_blog_Tokens/PAN2014_blog_male_chosen_set.csv"]
print("txt_files of features used",txt_files)
for txt_file in txt_files:
    with open(txt_file, mode="r", encoding="utf-8") as f:
##        reader = csv.reader(f, delimiter=",")
##        next(reader) # skip header
        word_list =  ([(r.split(":"))[1] for r in f])[0:200]
        print(txt_file, len(word_list))
        #word_list = word_list + word_list1
print("length of features used =", len(word_list))

#for txt_file in txt_files:
 #   with open(txt_file, mode="r", encoding="utf-8") as f:
 #       reader = csv.reader(f, delimiter=",")
 #       next(reader) # skip header
 #       word_list1 =  [r[0] for r in reader]
 #       print(txt_file, len(word_list1))
 #       word_list = word_list + word_list1
#print("length of features used =", len(word_list))


"""
Prepare the training and test sets to be parsed to the classifies............................................................
"""
A=(all_txt(M_train_file,  F_train_file))
all_training_text = ( [(A[i][1])  for i in range(len(A))])
print("len(all_training_text)",len(all_training_text))
X_train=WordFeatures(word_list, all_training_text)
y_train=np.array( [(A[i][0])  for i in range(len(A))])
print("len(X_train)=",len(X_train), "len(y_train)=", len(y_train))

B=(all_txt(M_test_file,  F_test_file))
all_test_text = ( [(B[i][1])  for i in range(len(B))])
print("len(all_test_text)",len(all_test_text))
X_test = WordFeatures(word_list, all_test_text)
y_test = np.array( [(B[i][0])  for i in range(len(B))])
print("len(X_test=",len(X_test), "len(y_test)=",len(y_test))


X_train = np.nan_to_num(X_train)
X_test = np.nan_to_num(X_test)

"""
Perform feature selection using chi2............................................................
"""
from sklearn.feature_selection import *
##
####def select_Bestfeatures(X, y):
##print("selector = SelectKBest(score_func=chi2, k=1000)")
##selector = SelectKBest(score_func=chi2, k=1000)
##fit = selector.fit(X_train,y_train)
##train_selected_features = fit.transform(X_train)
##test_selected_features = fit.transform(X_test)



"""
x_train and x_test based on the new features using chi2............................................................
"""
##X_train = select_Bestfeatures(X_train, y_train)
##X_test = select_Bestfeatures(X_test, y_test)


"""
Perform feature selection using f_classif............................................................
"""
from sklearn.feature_selection import *
##def f_classif_select_Bestfeatures(X, y):
##selector=SelectKBest(score_func=f_classif,k=500)
##fit = selector.fit(X_train,y_train)
##train_selected_features = fit.transform(X_train)
##test_selected_features = fit.transform(X_test)
##    selected_features = fit.transform(X)
##    return selected_features

"""
x_train and x_test based on the new features using f_classif............................................................
"""
##X_train = f_classif_select_Bestfeatures(X_train, y_train)
##X_test = f_classif_select_Bestfeatures(X_test, y_test)



"""
Perform feature selection using mutual_info_classif............................................................
"""
from sklearn.feature_selection import *
##def mutual_info_classif_select_Bestfeatures(X_tr, X_te, y):
##selector=SelectKBest(score_func=mutual_info_classif,k=100)
##fit = selector.fit(X_train,y_train)
##train_selected_features = fit.transform(X_train)
##test_selected_features = fit.transform(X_test)
##    return train_selected_features, test_selected_features

"""
x_train and x_test based on the new features using f_classif............................................................
"""
##X_train =train_selected_features 
##X_test = test_selected_features


"""
Perform feature selection using pca............................................................
"""

##from sklearn.decomposition import PCA
##def PCA_select_Bestfeatures(X):
##    # feature extraction
##    pca = PCA(n_components=200)
##    fit = pca.fit(X, y)
##    Train_selected_features = fit.transform(X)
##    return selected_features

"""
x_train and x_test based on the new features using f_classif............................................................
"""
##X_train = PCA_select_Bestfeatures(X_train)
##X_test = PCA_select_Bestfeatures(X_test)

"""
Below are a list of Classification Algorithms with different distance measres ..............................................................................
"""
"""
K-Nearest Neighbours (average aa over all the nearest neighbour selectiopn)..............................................................................
"""

k=5

print(k)
"""
8. manhattan K-Nearest Neighbours..............................................................................
"""
from sklearn.neighbors import KNeighborsClassifier
knn = KNeighborsClassifier(n_neighbors = k, metric= "manhattan")
knn.fit(X_train, y_train)
y_pred = knn.predict(X_test)
print("manhattan K-Nearest Neighbours")
print(metrics.accuracy_score(y_test, y_pred))

    
"""
8. cosine K-Nearest Neighbours..............................................................................
"""
from sklearn.neighbors import KNeighborsClassifier
knn = KNeighborsClassifier(n_neighbors = k, metric= "cosine")
knn.fit(X_train, y_train)
y_pred = knn.predict(X_test)
print("cosine K-Nearest Neighbours")
print(metrics.accuracy_score(y_test, y_pred))


"""
8. euclidean K-Nearest Neighbours..............................................................................
"""
from sklearn.neighbors import KNeighborsClassifier
knn = KNeighborsClassifier(n_neighbors = k, metric= "euclidean")
knn.fit(X_train, y_train)
y_pred = knn.predict(X_test)
print("euclidean K-Nearest Neighbours")
print(metrics.accuracy_score(y_test, y_pred))

        
"""
9. scipy.spatial.distance K-Nearest Neighbours..............................................................................
"""
from scipy.spatial.distance import *
knn = KNeighborsClassifier(n_neighbors = k, metric= "braycurtis")
knn.fit(X_train, y_train)
y_pred = knn.predict(X_test)
print("braycurtis K-Nearest Neighbours")
print(metrics.accuracy_score(y_test, y_pred))
       
        
knn = KNeighborsClassifier(n_neighbors = k, metric= "canberra")
knn.fit(X_train, y_train)
y_pred = knn.predict(X_test)
print("canberra K-Nearest Neighbours")
print(metrics.accuracy_score(y_test, y_pred))
  
        
knn = KNeighborsClassifier(n_neighbors = k, metric= "jaccard")
knn.fit(X_train, y_train)
y_pred = knn.predict(X_test)
print("jaccard K-Nearest Neighbours")
print(metrics.accuracy_score(y_test, y_pred))
        
knn = KNeighborsClassifier(n_neighbors = k, metric= "yule")
knn.fit(X_train, y_train)
y_pred = knn.predict(X_test)
print("yule K-Nearest Neighbours")
print(metrics.accuracy_score(y_test, y_pred))
print("************************************ ****************************")






txt_files of features used ['/Users/catherine/Desktop/NLP/PAN Datasets/EB2014Selection/output2014CHI.txt']
/Users/catherine/Desktop/NLP/PAN Datasets/EB2014Selection/output2014CHI.txt 200
length of features used = 200
len(all_training_text) 147
len(X_train)= 147 len(y_train)= 147
len(all_test_text) 78
len(X_test= 78 len(y_test)= 78
5
manhattan K-Nearest Neighbours
0.5
cosine K-Nearest Neighbours
0.5
euclidean K-Nearest Neighbours
0.5
braycurtis K-Nearest Neighbours
0.5
canberra K-Nearest Neighbours
0.5
jaccard K-Nearest Neighbours
0.5
yule K-Nearest Neighbours
0.5
************************************ ****************************


/anaconda3/lib/python3.7/site-packages/sklearn/utils/validation.py:475: DataConversionWarning: Data with input dtype float64 was converted to bool by check_pairwise_arrays.
  warnings.warn(msg, DataConversionWarning)


In [16]:

import sys
sys.path.append("/anaconda3/lib/python3.7/site-packages")
import numpy as np
import numpy
import nltk
import pickle
import pandas as pd
import glob
import os
import re
import operator
from collections import Counter
#from read_xml_files import *
##from s_stemmer import *
from nltk.stem.porter import PorterStemmer
from nltk.stem import LancasterStemmer, WordNetLemmatizer
from itertools import groupby
from nltk.collocations import *
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.cluster import KMeans
from scipy.cluster.vq import whiten
from nltk.tokenize.toktok import ToktokTokenizer
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2
import xml.etree.ElementTree as ET
import pandas as pd
#Import scikit-learn metrics module for accuracy calculation
from sklearn import metrics
##from pattern.text.en import singularize
##nltk.download('vader_lexicon')
##nltk.download('punkt')
#import the relevant modules from the NLTK library
from nltk.sentiment.vader import SentimentIntensityAnalyzer
sentence_tokenizer = nltk.data.load('tokenizers/punkt/english.pickle')
word_tokenizer = nltk.tokenize.RegexpTokenizer(r'\w+')
##tt = ToktokTokenizer()#keeps the url as is...................................................
from nltk.tokenize import TweetTokenizer
tt = TweetTokenizer()
Porter_stemmer = PorterStemmer()
Lancaster_stemmer = LancasterStemmer()
WordNet_lemmatizer = WordNetLemmatizer()
#nltk.download('averaged_perceptron_tagger')
##Train_path="D:/Data/PAN2015-Profiling/pan15-author-profiling-training-dataset-english-2015-04-23"#path for train data
##Test_path="D:/Data/PAN2015-Profiling/pan-ap2015-test/en" #path for test data

##Train_path="D:/NLP/PAN2014/pan14-author-profiling-training-corpus-english-twitter-2014-04-16"#path for train data D:\Data\PAN2013-Profiling\Training\en  D:\NLP\PAN2014
##train_truth_path="D:/NLP/PAN2014/truth_train"
##
##test_truth_path="D:/NLP/PAN2014/pan14_test"
##Test_path="D:/NLP/PAN2014/pan14_test/en"
##Content = open('results_of_clssification_all.txt', 'w')

M_train_file= "/Users/catherine/Desktop/NLP/PAN Datasets/PAN_tokens/2014_blog_Tokens/PAN2014_blog_male_train.txt"
F_train_file= "/Users/catherine/Desktop/NLP/PAN Datasets/PAN_tokens/2014_blog_Tokens/PAN2014_blog_female_train.txt"

M_test_file= "/Users/catherine/Desktop/NLP/PAN Datasets/PAN_tokens/2014_blog_Tokens/PAN2014_blog_male_test.txt"
F_test_file= "/Users/catherine/Desktop/NLP/PAN Datasets/PAN_tokens/2014_blog_Tokens/PAN2014_blog_female_test.txt"

def all_txt(M_txt_file,  F_txt_file):
    with open(M_txt_file, mode="r", encoding="utf-8") as f:
            txt_list = [(line[:-1]).split("\t") for line in f]
            male_dict = dict()
            content=[]
            for line in txt_list:
                if line[1] in male_dict:
                # append the new number to the existing array at this slot
                    male_dict[line[1]].append(line[3])
                else:
                # create a new array in this slot
                    male_dict[line[1]] = [line[3]]
    male_content= [("M", (' '.join(a)).split()) for a in list(male_dict.values())]
  
    with open(F_txt_file, mode="r", encoding="utf-8") as f:
            txt_list = [(line[:-1]).split("\t") for line in f]
            female_dict = dict()
            content=[]
            for line in txt_list:
                if line[1] in female_dict:
                # append the new number to the existing array at this slot
                    female_dict[line[1]].append(line[3])
                else:
                # create a new array in this slot
                    female_dict[line[1]] = [line[3]]
    female_content= [("F", (' '.join(a)).split()) for a in list(female_dict.values())]
    all_txt_per_person = male_content + female_content
    return all_txt_per_person

##print( [((all_txt(M_txt_file,  F_txt_file))[i][0])  for i in range(len((all_txt(M_txt_file,  F_txt_file))))]) helps to determine the gender....

      
"""
create feature vector from the training set.....................................................................................
"""

def WordFeatures(word_list, all_training_text):

    fvs_words = np.array([[author.count(word) for word in word_list] for author in all_training_text]).astype(np.float64)

    # normalise by dividing each row by number of tokens for each author........
    fvs_words /= np.c_[np.array([len(author) for author in all_training_text])]

    return fvs_words



"""
From the saved csv file, recover the saved features to be used...............................................................
"""
import csv
word_list=[]
txt_files =["/Users/catherine/Desktop/NLP/PAN Datasets/EB2014Selection/output2014CHI.txt"]
##txt_files =[ "/Users/catherine/Desktop/NLP/PAN Datasets/PAN_tokens/2014_blog_Tokens/PAN2014_blog_female_chosen_set.csv", "/Users/catherine/Desktop/NLP/PAN Datasets/PAN_tokens/2014_blog_Tokens/PAN2014_blog_male_chosen_set.csv"]
print("txt_files of features used",txt_files)
for txt_file in txt_files:
    with open(txt_file, mode="r", encoding="utf-8") as f:
##        reader = csv.reader(f, delimiter=",")
##        next(reader) # skip header
        word_list =  ([(r.split(":"))[1] for r in f])[0:300]
        print(txt_file, len(word_list))
        #word_list = word_list + word_list1
print("length of features used =", len(word_list))

#for txt_file in txt_files:
 #   with open(txt_file, mode="r", encoding="utf-8") as f:
 #       reader = csv.reader(f, delimiter=",")
 #       next(reader) # skip header
 #       word_list1 =  [r[0] for r in reader]
 #       print(txt_file, len(word_list1))
 #       word_list = word_list + word_list1
#print("length of features used =", len(word_list))


"""
Prepare the training and test sets to be parsed to the classifies............................................................
"""
A=(all_txt(M_train_file,  F_train_file))
all_training_text = ( [(A[i][1])  for i in range(len(A))])
print("len(all_training_text)",len(all_training_text))
X_train=WordFeatures(word_list, all_training_text)
y_train=np.array( [(A[i][0])  for i in range(len(A))])
print("len(X_train)=",len(X_train), "len(y_train)=", len(y_train))

B=(all_txt(M_test_file,  F_test_file))
all_test_text = ( [(B[i][1])  for i in range(len(B))])
print("len(all_test_text)",len(all_test_text))
X_test = WordFeatures(word_list, all_test_text)
y_test = np.array( [(B[i][0])  for i in range(len(B))])
print("len(X_test=",len(X_test), "len(y_test)=",len(y_test))


X_train = np.nan_to_num(X_train)
X_test = np.nan_to_num(X_test)

"""
Perform feature selection using chi2............................................................
"""
from sklearn.feature_selection import *
##
####def select_Bestfeatures(X, y):
##print("selector = SelectKBest(score_func=chi2, k=1000)")
##selector = SelectKBest(score_func=chi2, k=1000)
##fit = selector.fit(X_train,y_train)
##train_selected_features = fit.transform(X_train)
##test_selected_features = fit.transform(X_test)



"""
x_train and x_test based on the new features using chi2............................................................
"""
##X_train = select_Bestfeatures(X_train, y_train)
##X_test = select_Bestfeatures(X_test, y_test)


"""
Perform feature selection using f_classif............................................................
"""
from sklearn.feature_selection import *
##def f_classif_select_Bestfeatures(X, y):
##selector=SelectKBest(score_func=f_classif,k=500)
##fit = selector.fit(X_train,y_train)
##train_selected_features = fit.transform(X_train)
##test_selected_features = fit.transform(X_test)
##    selected_features = fit.transform(X)
##    return selected_features

"""
x_train and x_test based on the new features using f_classif............................................................
"""
##X_train = f_classif_select_Bestfeatures(X_train, y_train)
##X_test = f_classif_select_Bestfeatures(X_test, y_test)



"""
Perform feature selection using mutual_info_classif............................................................
"""
from sklearn.feature_selection import *
##def mutual_info_classif_select_Bestfeatures(X_tr, X_te, y):
##selector=SelectKBest(score_func=mutual_info_classif,k=100)
##fit = selector.fit(X_train,y_train)
##train_selected_features = fit.transform(X_train)
##test_selected_features = fit.transform(X_test)
##    return train_selected_features, test_selected_features

"""
x_train and x_test based on the new features using f_classif............................................................
"""
##X_train =train_selected_features 
##X_test = test_selected_features


"""
Perform feature selection using pca............................................................
"""

##from sklearn.decomposition import PCA
##def PCA_select_Bestfeatures(X):
##    # feature extraction
##    pca = PCA(n_components=200)
##    fit = pca.fit(X, y)
##    Train_selected_features = fit.transform(X)
##    return selected_features

"""
x_train and x_test based on the new features using f_classif............................................................
"""
##X_train = PCA_select_Bestfeatures(X_train)
##X_test = PCA_select_Bestfeatures(X_test)

"""
Below are a list of Classification Algorithms with different distance measres ..............................................................................
"""
"""
K-Nearest Neighbours (average aa over all the nearest neighbour selectiopn)..............................................................................
"""

k=5

print(k)
"""
8. manhattan K-Nearest Neighbours..............................................................................
"""
from sklearn.neighbors import KNeighborsClassifier
knn = KNeighborsClassifier(n_neighbors = k, metric= "manhattan")
knn.fit(X_train, y_train)
y_pred = knn.predict(X_test)
print("manhattan K-Nearest Neighbours")
print(metrics.accuracy_score(y_test, y_pred))

    
"""
8. cosine K-Nearest Neighbours..............................................................................
"""
from sklearn.neighbors import KNeighborsClassifier
knn = KNeighborsClassifier(n_neighbors = k, metric= "cosine")
knn.fit(X_train, y_train)
y_pred = knn.predict(X_test)
print("cosine K-Nearest Neighbours")
print(metrics.accuracy_score(y_test, y_pred))


"""
8. euclidean K-Nearest Neighbours..............................................................................
"""
from sklearn.neighbors import KNeighborsClassifier
knn = KNeighborsClassifier(n_neighbors = k, metric= "euclidean")
knn.fit(X_train, y_train)
y_pred = knn.predict(X_test)
print("euclidean K-Nearest Neighbours")
print(metrics.accuracy_score(y_test, y_pred))

        
"""
9. scipy.spatial.distance K-Nearest Neighbours..............................................................................
"""
from scipy.spatial.distance import *
knn = KNeighborsClassifier(n_neighbors = k, metric= "braycurtis")
knn.fit(X_train, y_train)
y_pred = knn.predict(X_test)
print("braycurtis K-Nearest Neighbours")
print(metrics.accuracy_score(y_test, y_pred))
       
        
knn = KNeighborsClassifier(n_neighbors = k, metric= "canberra")
knn.fit(X_train, y_train)
y_pred = knn.predict(X_test)
print("canberra K-Nearest Neighbours")
print(metrics.accuracy_score(y_test, y_pred))
  
        
knn = KNeighborsClassifier(n_neighbors = k, metric= "jaccard")
knn.fit(X_train, y_train)
y_pred = knn.predict(X_test)
print("jaccard K-Nearest Neighbours")
print(metrics.accuracy_score(y_test, y_pred))
        
knn = KNeighborsClassifier(n_neighbors = k, metric= "yule")
knn.fit(X_train, y_train)
y_pred = knn.predict(X_test)
print("yule K-Nearest Neighbours")
print(metrics.accuracy_score(y_test, y_pred))
print("************************************ ****************************")







txt_files of features used ['/Users/catherine/Desktop/NLP/PAN Datasets/EB2014Selection/output2014CHI.txt']
/Users/catherine/Desktop/NLP/PAN Datasets/EB2014Selection/output2014CHI.txt 300
length of features used = 300
len(all_training_text) 147
len(X_train)= 147 len(y_train)= 147
len(all_test_text) 78
len(X_test= 78 len(y_test)= 78
5
manhattan K-Nearest Neighbours
0.5
cosine K-Nearest Neighbours
0.5
euclidean K-Nearest Neighbours
0.5
braycurtis K-Nearest Neighbours
0.5
canberra K-Nearest Neighbours
0.5
jaccard K-Nearest Neighbours
0.5
yule K-Nearest Neighbours
0.5
************************************ ****************************


/anaconda3/lib/python3.7/site-packages/sklearn/utils/validation.py:475: DataConversionWarning: Data with input dtype float64 was converted to bool by check_pairwise_arrays.
  warnings.warn(msg, DataConversionWarning)


In [17]:

import sys
sys.path.append("/anaconda3/lib/python3.7/site-packages")
import numpy as np
import numpy
import nltk
import pickle
import pandas as pd
import glob
import os
import re
import operator
from collections import Counter
#from read_xml_files import *
##from s_stemmer import *
from nltk.stem.porter import PorterStemmer
from nltk.stem import LancasterStemmer, WordNetLemmatizer
from itertools import groupby
from nltk.collocations import *
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.cluster import KMeans
from scipy.cluster.vq import whiten
from nltk.tokenize.toktok import ToktokTokenizer
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2
import xml.etree.ElementTree as ET
import pandas as pd
#Import scikit-learn metrics module for accuracy calculation
from sklearn import metrics
##from pattern.text.en import singularize
##nltk.download('vader_lexicon')
##nltk.download('punkt')
#import the relevant modules from the NLTK library
from nltk.sentiment.vader import SentimentIntensityAnalyzer
sentence_tokenizer = nltk.data.load('tokenizers/punkt/english.pickle')
word_tokenizer = nltk.tokenize.RegexpTokenizer(r'\w+')
##tt = ToktokTokenizer()#keeps the url as is...................................................
from nltk.tokenize import TweetTokenizer
tt = TweetTokenizer()
Porter_stemmer = PorterStemmer()
Lancaster_stemmer = LancasterStemmer()
WordNet_lemmatizer = WordNetLemmatizer()
#nltk.download('averaged_perceptron_tagger')
##Train_path="D:/Data/PAN2015-Profiling/pan15-author-profiling-training-dataset-english-2015-04-23"#path for train data
##Test_path="D:/Data/PAN2015-Profiling/pan-ap2015-test/en" #path for test data

##Train_path="D:/NLP/PAN2014/pan14-author-profiling-training-corpus-english-twitter-2014-04-16"#path for train data D:\Data\PAN2013-Profiling\Training\en  D:\NLP\PAN2014
##train_truth_path="D:/NLP/PAN2014/truth_train"
##
##test_truth_path="D:/NLP/PAN2014/pan14_test"
##Test_path="D:/NLP/PAN2014/pan14_test/en"
##Content = open('results_of_clssification_all.txt', 'w')

M_train_file= "/Users/catherine/Desktop/NLP/PAN Datasets/PAN_tokens/2014_blog_Tokens/PAN2014_blog_male_train.txt"
F_train_file= "/Users/catherine/Desktop/NLP/PAN Datasets/PAN_tokens/2014_blog_Tokens/PAN2014_blog_female_train.txt"

M_test_file= "/Users/catherine/Desktop/NLP/PAN Datasets/PAN_tokens/2014_blog_Tokens/PAN2014_blog_male_test.txt"
F_test_file= "/Users/catherine/Desktop/NLP/PAN Datasets/PAN_tokens/2014_blog_Tokens/PAN2014_blog_female_test.txt"

def all_txt(M_txt_file,  F_txt_file):
    with open(M_txt_file, mode="r", encoding="utf-8") as f:
            txt_list = [(line[:-1]).split("\t") for line in f]
            male_dict = dict()
            content=[]
            for line in txt_list:
                if line[1] in male_dict:
                # append the new number to the existing array at this slot
                    male_dict[line[1]].append(line[3])
                else:
                # create a new array in this slot
                    male_dict[line[1]] = [line[3]]
    male_content= [("M", (' '.join(a)).split()) for a in list(male_dict.values())]
  
    with open(F_txt_file, mode="r", encoding="utf-8") as f:
            txt_list = [(line[:-1]).split("\t") for line in f]
            female_dict = dict()
            content=[]
            for line in txt_list:
                if line[1] in female_dict:
                # append the new number to the existing array at this slot
                    female_dict[line[1]].append(line[3])
                else:
                # create a new array in this slot
                    female_dict[line[1]] = [line[3]]
    female_content= [("F", (' '.join(a)).split()) for a in list(female_dict.values())]
    all_txt_per_person = male_content + female_content
    return all_txt_per_person

##print( [((all_txt(M_txt_file,  F_txt_file))[i][0])  for i in range(len((all_txt(M_txt_file,  F_txt_file))))]) helps to determine the gender....

      
"""
create feature vector from the training set.....................................................................................
"""

def WordFeatures(word_list, all_training_text):

    fvs_words = np.array([[author.count(word) for word in word_list] for author in all_training_text]).astype(np.float64)

    # normalise by dividing each row by number of tokens for each author........
    fvs_words /= np.c_[np.array([len(author) for author in all_training_text])]

    return fvs_words



"""
From the saved csv file, recover the saved features to be used...............................................................
"""
import csv
word_list=[]
txt_files =["/Users/catherine/Desktop/NLP/PAN Datasets/EB2014Selection/output2014CHI.txt"]
##txt_files =[ "/Users/catherine/Desktop/NLP/PAN Datasets/PAN_tokens/2014_blog_Tokens/PAN2014_blog_female_chosen_set.csv", "/Users/catherine/Desktop/NLP/PAN Datasets/PAN_tokens/2014_blog_Tokens/PAN2014_blog_male_chosen_set.csv"]
print("txt_files of features used",txt_files)
for txt_file in txt_files:
    with open(txt_file, mode="r", encoding="utf-8") as f:
##        reader = csv.reader(f, delimiter=",")
##        next(reader) # skip header
        word_list =  ([(r.split(":"))[1] for r in f])[0:500]
        print(txt_file, len(word_list))
        #word_list = word_list + word_list1
print("length of features used =", len(word_list))

#for txt_file in txt_files:
 #   with open(txt_file, mode="r", encoding="utf-8") as f:
 #       reader = csv.reader(f, delimiter=",")
 #       next(reader) # skip header
 #       word_list1 =  [r[0] for r in reader]
 #       print(txt_file, len(word_list1))
 #       word_list = word_list + word_list1
#print("length of features used =", len(word_list))


"""
Prepare the training and test sets to be parsed to the classifies............................................................
"""
A=(all_txt(M_train_file,  F_train_file))
all_training_text = ( [(A[i][1])  for i in range(len(A))])
print("len(all_training_text)",len(all_training_text))
X_train=WordFeatures(word_list, all_training_text)
y_train=np.array( [(A[i][0])  for i in range(len(A))])
print("len(X_train)=",len(X_train), "len(y_train)=", len(y_train))

B=(all_txt(M_test_file,  F_test_file))
all_test_text = ( [(B[i][1])  for i in range(len(B))])
print("len(all_test_text)",len(all_test_text))
X_test = WordFeatures(word_list, all_test_text)
y_test = np.array( [(B[i][0])  for i in range(len(B))])
print("len(X_test=",len(X_test), "len(y_test)=",len(y_test))


X_train = np.nan_to_num(X_train)
X_test = np.nan_to_num(X_test)

"""
Perform feature selection using chi2............................................................
"""
from sklearn.feature_selection import *
##
####def select_Bestfeatures(X, y):
##print("selector = SelectKBest(score_func=chi2, k=1000)")
##selector = SelectKBest(score_func=chi2, k=1000)
##fit = selector.fit(X_train,y_train)
##train_selected_features = fit.transform(X_train)
##test_selected_features = fit.transform(X_test)



"""
x_train and x_test based on the new features using chi2............................................................
"""
##X_train = select_Bestfeatures(X_train, y_train)
##X_test = select_Bestfeatures(X_test, y_test)


"""
Perform feature selection using f_classif............................................................
"""
from sklearn.feature_selection import *
##def f_classif_select_Bestfeatures(X, y):
##selector=SelectKBest(score_func=f_classif,k=500)
##fit = selector.fit(X_train,y_train)
##train_selected_features = fit.transform(X_train)
##test_selected_features = fit.transform(X_test)
##    selected_features = fit.transform(X)
##    return selected_features

"""
x_train and x_test based on the new features using f_classif............................................................
"""
##X_train = f_classif_select_Bestfeatures(X_train, y_train)
##X_test = f_classif_select_Bestfeatures(X_test, y_test)



"""
Perform feature selection using mutual_info_classif............................................................
"""
from sklearn.feature_selection import *
##def mutual_info_classif_select_Bestfeatures(X_tr, X_te, y):
##selector=SelectKBest(score_func=mutual_info_classif,k=100)
##fit = selector.fit(X_train,y_train)
##train_selected_features = fit.transform(X_train)
##test_selected_features = fit.transform(X_test)
##    return train_selected_features, test_selected_features

"""
x_train and x_test based on the new features using f_classif............................................................
"""
##X_train =train_selected_features 
##X_test = test_selected_features


"""
Perform feature selection using pca............................................................
"""

##from sklearn.decomposition import PCA
##def PCA_select_Bestfeatures(X):
##    # feature extraction
##    pca = PCA(n_components=200)
##    fit = pca.fit(X, y)
##    Train_selected_features = fit.transform(X)
##    return selected_features

"""
x_train and x_test based on the new features using f_classif............................................................
"""
##X_train = PCA_select_Bestfeatures(X_train)
##X_test = PCA_select_Bestfeatures(X_test)

"""
Below are a list of Classification Algorithms with different distance measres ..............................................................................
"""
"""
K-Nearest Neighbours (average aa over all the nearest neighbour selectiopn)..............................................................................
"""

k=5

print(k)
"""
8. manhattan K-Nearest Neighbours..............................................................................
"""
from sklearn.neighbors import KNeighborsClassifier
knn = KNeighborsClassifier(n_neighbors = k, metric= "manhattan")
knn.fit(X_train, y_train)
y_pred = knn.predict(X_test)
print("manhattan K-Nearest Neighbours")
print(metrics.accuracy_score(y_test, y_pred))

    
"""
8. cosine K-Nearest Neighbours..............................................................................
"""
from sklearn.neighbors import KNeighborsClassifier
knn = KNeighborsClassifier(n_neighbors = k, metric= "cosine")
knn.fit(X_train, y_train)
y_pred = knn.predict(X_test)
print("cosine K-Nearest Neighbours")
print(metrics.accuracy_score(y_test, y_pred))


"""
8. euclidean K-Nearest Neighbours..............................................................................
"""
from sklearn.neighbors import KNeighborsClassifier
knn = KNeighborsClassifier(n_neighbors = k, metric= "euclidean")
knn.fit(X_train, y_train)
y_pred = knn.predict(X_test)
print("euclidean K-Nearest Neighbours")
print(metrics.accuracy_score(y_test, y_pred))

        
"""
9. scipy.spatial.distance K-Nearest Neighbours..............................................................................
"""
from scipy.spatial.distance import *
knn = KNeighborsClassifier(n_neighbors = k, metric= "braycurtis")
knn.fit(X_train, y_train)
y_pred = knn.predict(X_test)
print("braycurtis K-Nearest Neighbours")
print(metrics.accuracy_score(y_test, y_pred))
       
        
knn = KNeighborsClassifier(n_neighbors = k, metric= "canberra")
knn.fit(X_train, y_train)
y_pred = knn.predict(X_test)
print("canberra K-Nearest Neighbours")
print(metrics.accuracy_score(y_test, y_pred))
  
        
knn = KNeighborsClassifier(n_neighbors = k, metric= "jaccard")
knn.fit(X_train, y_train)
y_pred = knn.predict(X_test)
print("jaccard K-Nearest Neighbours")
print(metrics.accuracy_score(y_test, y_pred))
        
knn = KNeighborsClassifier(n_neighbors = k, metric= "yule")
knn.fit(X_train, y_train)
y_pred = knn.predict(X_test)
print("yule K-Nearest Neighbours")
print(metrics.accuracy_score(y_test, y_pred))
print("************************************ ****************************")







txt_files of features used ['/Users/catherine/Desktop/NLP/PAN Datasets/EB2014Selection/output2014CHI.txt']
/Users/catherine/Desktop/NLP/PAN Datasets/EB2014Selection/output2014CHI.txt 500
length of features used = 500
len(all_training_text) 147
len(X_train)= 147 len(y_train)= 147
len(all_test_text) 78
len(X_test= 78 len(y_test)= 78
5
manhattan K-Nearest Neighbours
0.5
cosine K-Nearest Neighbours
0.5
euclidean K-Nearest Neighbours
0.5
braycurtis K-Nearest Neighbours
0.5
canberra K-Nearest Neighbours
0.5
jaccard K-Nearest Neighbours
0.5
yule K-Nearest Neighbours
0.5
************************************ ****************************


/anaconda3/lib/python3.7/site-packages/sklearn/utils/validation.py:475: DataConversionWarning: Data with input dtype float64 was converted to bool by check_pairwise_arrays.
  warnings.warn(msg, DataConversionWarning)


In [18]:

import sys
sys.path.append("/anaconda3/lib/python3.7/site-packages")
import numpy as np
import numpy
import nltk
import pickle
import pandas as pd
import glob
import os
import re
import operator
from collections import Counter
#from read_xml_files import *
##from s_stemmer import *
from nltk.stem.porter import PorterStemmer
from nltk.stem import LancasterStemmer, WordNetLemmatizer
from itertools import groupby
from nltk.collocations import *
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.cluster import KMeans
from scipy.cluster.vq import whiten
from nltk.tokenize.toktok import ToktokTokenizer
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2
import xml.etree.ElementTree as ET
import pandas as pd
#Import scikit-learn metrics module for accuracy calculation
from sklearn import metrics
##from pattern.text.en import singularize
##nltk.download('vader_lexicon')
##nltk.download('punkt')
#import the relevant modules from the NLTK library
from nltk.sentiment.vader import SentimentIntensityAnalyzer
sentence_tokenizer = nltk.data.load('tokenizers/punkt/english.pickle')
word_tokenizer = nltk.tokenize.RegexpTokenizer(r'\w+')
##tt = ToktokTokenizer()#keeps the url as is...................................................
from nltk.tokenize import TweetTokenizer
tt = TweetTokenizer()
Porter_stemmer = PorterStemmer()
Lancaster_stemmer = LancasterStemmer()
WordNet_lemmatizer = WordNetLemmatizer()
#nltk.download('averaged_perceptron_tagger')
##Train_path="D:/Data/PAN2015-Profiling/pan15-author-profiling-training-dataset-english-2015-04-23"#path for train data
##Test_path="D:/Data/PAN2015-Profiling/pan-ap2015-test/en" #path for test data

##Train_path="D:/NLP/PAN2014/pan14-author-profiling-training-corpus-english-twitter-2014-04-16"#path for train data D:\Data\PAN2013-Profiling\Training\en  D:\NLP\PAN2014
##train_truth_path="D:/NLP/PAN2014/truth_train"
##
##test_truth_path="D:/NLP/PAN2014/pan14_test"
##Test_path="D:/NLP/PAN2014/pan14_test/en"
##Content = open('results_of_clssification_all.txt', 'w')

M_train_file= "/Users/catherine/Desktop/NLP/PAN Datasets/PAN_tokens/2014_blog_Tokens/PAN2014_blog_male_train.txt"
F_train_file= "/Users/catherine/Desktop/NLP/PAN Datasets/PAN_tokens/2014_blog_Tokens/PAN2014_blog_female_train.txt"

M_test_file= "/Users/catherine/Desktop/NLP/PAN Datasets/PAN_tokens/2014_blog_Tokens/PAN2014_blog_male_test.txt"
F_test_file= "/Users/catherine/Desktop/NLP/PAN Datasets/PAN_tokens/2014_blog_Tokens/PAN2014_blog_female_test.txt"

def all_txt(M_txt_file,  F_txt_file):
    with open(M_txt_file, mode="r", encoding="utf-8") as f:
            txt_list = [(line[:-1]).split("\t") for line in f]
            male_dict = dict()
            content=[]
            for line in txt_list:
                if line[1] in male_dict:
                # append the new number to the existing array at this slot
                    male_dict[line[1]].append(line[3])
                else:
                # create a new array in this slot
                    male_dict[line[1]] = [line[3]]
    male_content= [("M", (' '.join(a)).split()) for a in list(male_dict.values())]
  
    with open(F_txt_file, mode="r", encoding="utf-8") as f:
            txt_list = [(line[:-1]).split("\t") for line in f]
            female_dict = dict()
            content=[]
            for line in txt_list:
                if line[1] in female_dict:
                # append the new number to the existing array at this slot
                    female_dict[line[1]].append(line[3])
                else:
                # create a new array in this slot
                    female_dict[line[1]] = [line[3]]
    female_content= [("F", (' '.join(a)).split()) for a in list(female_dict.values())]
    all_txt_per_person = male_content + female_content
    return all_txt_per_person

##print( [((all_txt(M_txt_file,  F_txt_file))[i][0])  for i in range(len((all_txt(M_txt_file,  F_txt_file))))]) helps to determine the gender....

      
"""
create feature vector from the training set.....................................................................................
"""

def WordFeatures(word_list, all_training_text):

    fvs_words = np.array([[author.count(word) for word in word_list] for author in all_training_text]).astype(np.float64)

    # normalise by dividing each row by number of tokens for each author........
    fvs_words /= np.c_[np.array([len(author) for author in all_training_text])]

    return fvs_words



"""
From the saved csv file, recover the saved features to be used...............................................................
"""
import csv
word_list=[]
txt_files =["/Users/catherine/Desktop/NLP/PAN Datasets/EB2014Selection/output2014GSS.txt"]
##txt_files =[ "/Users/catherine/Desktop/NLP/PAN Datasets/PAN_tokens/2014_blog_Tokens/PAN2014_blog_female_chosen_set.csv", "/Users/catherine/Desktop/NLP/PAN Datasets/PAN_tokens/2014_blog_Tokens/PAN2014_blog_male_chosen_set.csv"]
print("txt_files of features used",txt_files)
for txt_file in txt_files:
    with open(txt_file, mode="r", encoding="utf-8") as f:
##        reader = csv.reader(f, delimiter=",")
##        next(reader) # skip header
        word_list =  ([(r.split(":"))[1] for r in f])[0:200]
        print(txt_file, len(word_list))
        #word_list = word_list + word_list1
print("length of features used =", len(word_list))

#for txt_file in txt_files:
 #   with open(txt_file, mode="r", encoding="utf-8") as f:
 #       reader = csv.reader(f, delimiter=",")
 #       next(reader) # skip header
 #       word_list1 =  [r[0] for r in reader]
 #       print(txt_file, len(word_list1))
 #       word_list = word_list + word_list1
#print("length of features used =", len(word_list))


"""
Prepare the training and test sets to be parsed to the classifies............................................................
"""
A=(all_txt(M_train_file,  F_train_file))
all_training_text = ( [(A[i][1])  for i in range(len(A))])
print("len(all_training_text)",len(all_training_text))
X_train=WordFeatures(word_list, all_training_text)
y_train=np.array( [(A[i][0])  for i in range(len(A))])
print("len(X_train)=",len(X_train), "len(y_train)=", len(y_train))

B=(all_txt(M_test_file,  F_test_file))
all_test_text = ( [(B[i][1])  for i in range(len(B))])
print("len(all_test_text)",len(all_test_text))
X_test = WordFeatures(word_list, all_test_text)
y_test = np.array( [(B[i][0])  for i in range(len(B))])
print("len(X_test=",len(X_test), "len(y_test)=",len(y_test))


X_train = np.nan_to_num(X_train)
X_test = np.nan_to_num(X_test)

"""
Perform feature selection using chi2............................................................
"""
from sklearn.feature_selection import *
##
####def select_Bestfeatures(X, y):
##print("selector = SelectKBest(score_func=chi2, k=1000)")
##selector = SelectKBest(score_func=chi2, k=1000)
##fit = selector.fit(X_train,y_train)
##train_selected_features = fit.transform(X_train)
##test_selected_features = fit.transform(X_test)



"""
x_train and x_test based on the new features using chi2............................................................
"""
##X_train = select_Bestfeatures(X_train, y_train)
##X_test = select_Bestfeatures(X_test, y_test)


"""
Perform feature selection using f_classif............................................................
"""
from sklearn.feature_selection import *
##def f_classif_select_Bestfeatures(X, y):
##selector=SelectKBest(score_func=f_classif,k=500)
##fit = selector.fit(X_train,y_train)
##train_selected_features = fit.transform(X_train)
##test_selected_features = fit.transform(X_test)
##    selected_features = fit.transform(X)
##    return selected_features

"""
x_train and x_test based on the new features using f_classif............................................................
"""
##X_train = f_classif_select_Bestfeatures(X_train, y_train)
##X_test = f_classif_select_Bestfeatures(X_test, y_test)



"""
Perform feature selection using mutual_info_classif............................................................
"""
from sklearn.feature_selection import *
##def mutual_info_classif_select_Bestfeatures(X_tr, X_te, y):
##selector=SelectKBest(score_func=mutual_info_classif,k=100)
##fit = selector.fit(X_train,y_train)
##train_selected_features = fit.transform(X_train)
##test_selected_features = fit.transform(X_test)
##    return train_selected_features, test_selected_features

"""
x_train and x_test based on the new features using f_classif............................................................
"""
##X_train =train_selected_features 
##X_test = test_selected_features


"""
Perform feature selection using pca............................................................
"""

##from sklearn.decomposition import PCA
##def PCA_select_Bestfeatures(X):
##    # feature extraction
##    pca = PCA(n_components=200)
##    fit = pca.fit(X, y)
##    Train_selected_features = fit.transform(X)
##    return selected_features

"""
x_train and x_test based on the new features using f_classif............................................................
"""
##X_train = PCA_select_Bestfeatures(X_train)
##X_test = PCA_select_Bestfeatures(X_test)

"""
Below are a list of Classification Algorithms with different distance measres ..............................................................................
"""
"""
K-Nearest Neighbours (average aa over all the nearest neighbour selectiopn)..............................................................................
"""

k=5

print(k)
"""
8. manhattan K-Nearest Neighbours..............................................................................
"""
from sklearn.neighbors import KNeighborsClassifier
knn = KNeighborsClassifier(n_neighbors = k, metric= "manhattan")
knn.fit(X_train, y_train)
y_pred = knn.predict(X_test)
print("manhattan K-Nearest Neighbours")
print(metrics.accuracy_score(y_test, y_pred))

    
"""
8. cosine K-Nearest Neighbours..............................................................................
"""
from sklearn.neighbors import KNeighborsClassifier
knn = KNeighborsClassifier(n_neighbors = k, metric= "cosine")
knn.fit(X_train, y_train)
y_pred = knn.predict(X_test)
print("cosine K-Nearest Neighbours")
print(metrics.accuracy_score(y_test, y_pred))


"""
8. euclidean K-Nearest Neighbours..............................................................................
"""
from sklearn.neighbors import KNeighborsClassifier
knn = KNeighborsClassifier(n_neighbors = k, metric= "euclidean")
knn.fit(X_train, y_train)
y_pred = knn.predict(X_test)
print("euclidean K-Nearest Neighbours")
print(metrics.accuracy_score(y_test, y_pred))

        
"""
9. scipy.spatial.distance K-Nearest Neighbours..............................................................................
"""
from scipy.spatial.distance import *
knn = KNeighborsClassifier(n_neighbors = k, metric= "braycurtis")
knn.fit(X_train, y_train)
y_pred = knn.predict(X_test)
print("braycurtis K-Nearest Neighbours")
print(metrics.accuracy_score(y_test, y_pred))
       
        
knn = KNeighborsClassifier(n_neighbors = k, metric= "canberra")
knn.fit(X_train, y_train)
y_pred = knn.predict(X_test)
print("canberra K-Nearest Neighbours")
print(metrics.accuracy_score(y_test, y_pred))
  
        
knn = KNeighborsClassifier(n_neighbors = k, metric= "jaccard")
knn.fit(X_train, y_train)
y_pred = knn.predict(X_test)
print("jaccard K-Nearest Neighbours")
print(metrics.accuracy_score(y_test, y_pred))
        
knn = KNeighborsClassifier(n_neighbors = k, metric= "yule")
knn.fit(X_train, y_train)
y_pred = knn.predict(X_test)
print("yule K-Nearest Neighbours")
print(metrics.accuracy_score(y_test, y_pred))
print("************************************ ****************************")







txt_files of features used ['/Users/catherine/Desktop/NLP/PAN Datasets/EB2014Selection/output2014GSS.txt']
/Users/catherine/Desktop/NLP/PAN Datasets/EB2014Selection/output2014GSS.txt 200
length of features used = 200
len(all_training_text) 147
len(X_train)= 147 len(y_train)= 147
len(all_test_text) 78
len(X_test= 78 len(y_test)= 78
5
manhattan K-Nearest Neighbours
0.5
cosine K-Nearest Neighbours
0.5
euclidean K-Nearest Neighbours
0.5
braycurtis K-Nearest Neighbours
0.5
canberra K-Nearest Neighbours
0.5
jaccard K-Nearest Neighbours
0.5
yule K-Nearest Neighbours
0.5
************************************ ****************************


/anaconda3/lib/python3.7/site-packages/sklearn/utils/validation.py:475: DataConversionWarning: Data with input dtype float64 was converted to bool by check_pairwise_arrays.
  warnings.warn(msg, DataConversionWarning)


In [19]:

import sys
sys.path.append("/anaconda3/lib/python3.7/site-packages")
import numpy as np
import numpy
import nltk
import pickle
import pandas as pd
import glob
import os
import re
import operator
from collections import Counter
#from read_xml_files import *
##from s_stemmer import *
from nltk.stem.porter import PorterStemmer
from nltk.stem import LancasterStemmer, WordNetLemmatizer
from itertools import groupby
from nltk.collocations import *
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.cluster import KMeans
from scipy.cluster.vq import whiten
from nltk.tokenize.toktok import ToktokTokenizer
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2
import xml.etree.ElementTree as ET
import pandas as pd
#Import scikit-learn metrics module for accuracy calculation
from sklearn import metrics
##from pattern.text.en import singularize
##nltk.download('vader_lexicon')
##nltk.download('punkt')
#import the relevant modules from the NLTK library
from nltk.sentiment.vader import SentimentIntensityAnalyzer
sentence_tokenizer = nltk.data.load('tokenizers/punkt/english.pickle')
word_tokenizer = nltk.tokenize.RegexpTokenizer(r'\w+')
##tt = ToktokTokenizer()#keeps the url as is...................................................
from nltk.tokenize import TweetTokenizer
tt = TweetTokenizer()
Porter_stemmer = PorterStemmer()
Lancaster_stemmer = LancasterStemmer()
WordNet_lemmatizer = WordNetLemmatizer()
#nltk.download('averaged_perceptron_tagger')
##Train_path="D:/Data/PAN2015-Profiling/pan15-author-profiling-training-dataset-english-2015-04-23"#path for train data
##Test_path="D:/Data/PAN2015-Profiling/pan-ap2015-test/en" #path for test data

##Train_path="D:/NLP/PAN2014/pan14-author-profiling-training-corpus-english-twitter-2014-04-16"#path for train data D:\Data\PAN2013-Profiling\Training\en  D:\NLP\PAN2014
##train_truth_path="D:/NLP/PAN2014/truth_train"
##
##test_truth_path="D:/NLP/PAN2014/pan14_test"
##Test_path="D:/NLP/PAN2014/pan14_test/en"
##Content = open('results_of_clssification_all.txt', 'w')

M_train_file= "/Users/catherine/Desktop/NLP/PAN Datasets/PAN_tokens/2014_blog_Tokens/PAN2014_blog_male_train.txt"
F_train_file= "/Users/catherine/Desktop/NLP/PAN Datasets/PAN_tokens/2014_blog_Tokens/PAN2014_blog_female_train.txt"

M_test_file= "/Users/catherine/Desktop/NLP/PAN Datasets/PAN_tokens/2014_blog_Tokens/PAN2014_blog_male_test.txt"
F_test_file= "/Users/catherine/Desktop/NLP/PAN Datasets/PAN_tokens/2014_blog_Tokens/PAN2014_blog_female_test.txt"

def all_txt(M_txt_file,  F_txt_file):
    with open(M_txt_file, mode="r", encoding="utf-8") as f:
            txt_list = [(line[:-1]).split("\t") for line in f]
            male_dict = dict()
            content=[]
            for line in txt_list:
                if line[1] in male_dict:
                # append the new number to the existing array at this slot
                    male_dict[line[1]].append(line[3])
                else:
                # create a new array in this slot
                    male_dict[line[1]] = [line[3]]
    male_content= [("M", (' '.join(a)).split()) for a in list(male_dict.values())]
  
    with open(F_txt_file, mode="r", encoding="utf-8") as f:
            txt_list = [(line[:-1]).split("\t") for line in f]
            female_dict = dict()
            content=[]
            for line in txt_list:
                if line[1] in female_dict:
                # append the new number to the existing array at this slot
                    female_dict[line[1]].append(line[3])
                else:
                # create a new array in this slot
                    female_dict[line[1]] = [line[3]]
    female_content= [("F", (' '.join(a)).split()) for a in list(female_dict.values())]
    all_txt_per_person = male_content + female_content
    return all_txt_per_person

##print( [((all_txt(M_txt_file,  F_txt_file))[i][0])  for i in range(len((all_txt(M_txt_file,  F_txt_file))))]) helps to determine the gender....

      
"""
create feature vector from the training set.....................................................................................
"""

def WordFeatures(word_list, all_training_text):

    fvs_words = np.array([[author.count(word) for word in word_list] for author in all_training_text]).astype(np.float64)

    # normalise by dividing each row by number of tokens for each author........
    fvs_words /= np.c_[np.array([len(author) for author in all_training_text])]

    return fvs_words



"""
From the saved csv file, recover the saved features to be used...............................................................
"""
import csv
word_list=[]
txt_files =["/Users/catherine/Desktop/NLP/PAN Datasets/EB2014Selection/output2014GSS.txt"]
##txt_files =[ "/Users/catherine/Desktop/NLP/PAN Datasets/PAN_tokens/2014_blog_Tokens/PAN2014_blog_female_chosen_set.csv", "/Users/catherine/Desktop/NLP/PAN Datasets/PAN_tokens/2014_blog_Tokens/PAN2014_blog_male_chosen_set.csv"]
print("txt_files of features used",txt_files)
for txt_file in txt_files:
    with open(txt_file, mode="r", encoding="utf-8") as f:
##        reader = csv.reader(f, delimiter=",")
##        next(reader) # skip header
        word_list =  ([(r.split(":"))[1] for r in f])[0:300]
        print(txt_file, len(word_list))
        #word_list = word_list + word_list1
print("length of features used =", len(word_list))

#for txt_file in txt_files:
 #   with open(txt_file, mode="r", encoding="utf-8") as f:
 #       reader = csv.reader(f, delimiter=",")
 #       next(reader) # skip header
 #       word_list1 =  [r[0] for r in reader]
 #       print(txt_file, len(word_list1))
 #       word_list = word_list + word_list1
#print("length of features used =", len(word_list))


"""
Prepare the training and test sets to be parsed to the classifies............................................................
"""
A=(all_txt(M_train_file,  F_train_file))
all_training_text = ( [(A[i][1])  for i in range(len(A))])
print("len(all_training_text)",len(all_training_text))
X_train=WordFeatures(word_list, all_training_text)
y_train=np.array( [(A[i][0])  for i in range(len(A))])
print("len(X_train)=",len(X_train), "len(y_train)=", len(y_train))

B=(all_txt(M_test_file,  F_test_file))
all_test_text = ( [(B[i][1])  for i in range(len(B))])
print("len(all_test_text)",len(all_test_text))
X_test = WordFeatures(word_list, all_test_text)
y_test = np.array( [(B[i][0])  for i in range(len(B))])
print("len(X_test=",len(X_test), "len(y_test)=",len(y_test))


X_train = np.nan_to_num(X_train)
X_test = np.nan_to_num(X_test)

"""
Perform feature selection using chi2............................................................
"""
from sklearn.feature_selection import *
##
####def select_Bestfeatures(X, y):
##print("selector = SelectKBest(score_func=chi2, k=1000)")
##selector = SelectKBest(score_func=chi2, k=1000)
##fit = selector.fit(X_train,y_train)
##train_selected_features = fit.transform(X_train)
##test_selected_features = fit.transform(X_test)



"""
x_train and x_test based on the new features using chi2............................................................
"""
##X_train = select_Bestfeatures(X_train, y_train)
##X_test = select_Bestfeatures(X_test, y_test)


"""
Perform feature selection using f_classif............................................................
"""
from sklearn.feature_selection import *
##def f_classif_select_Bestfeatures(X, y):
##selector=SelectKBest(score_func=f_classif,k=500)
##fit = selector.fit(X_train,y_train)
##train_selected_features = fit.transform(X_train)
##test_selected_features = fit.transform(X_test)
##    selected_features = fit.transform(X)
##    return selected_features

"""
x_train and x_test based on the new features using f_classif............................................................
"""
##X_train = f_classif_select_Bestfeatures(X_train, y_train)
##X_test = f_classif_select_Bestfeatures(X_test, y_test)



"""
Perform feature selection using mutual_info_classif............................................................
"""
from sklearn.feature_selection import *
##def mutual_info_classif_select_Bestfeatures(X_tr, X_te, y):
##selector=SelectKBest(score_func=mutual_info_classif,k=100)
##fit = selector.fit(X_train,y_train)
##train_selected_features = fit.transform(X_train)
##test_selected_features = fit.transform(X_test)
##    return train_selected_features, test_selected_features

"""
x_train and x_test based on the new features using f_classif............................................................
"""
##X_train =train_selected_features 
##X_test = test_selected_features


"""
Perform feature selection using pca............................................................
"""

##from sklearn.decomposition import PCA
##def PCA_select_Bestfeatures(X):
##    # feature extraction
##    pca = PCA(n_components=200)
##    fit = pca.fit(X, y)
##    Train_selected_features = fit.transform(X)
##    return selected_features

"""
x_train and x_test based on the new features using f_classif............................................................
"""
##X_train = PCA_select_Bestfeatures(X_train)
##X_test = PCA_select_Bestfeatures(X_test)

"""
Below are a list of Classification Algorithms with different distance measres ..............................................................................
"""
"""
K-Nearest Neighbours (average aa over all the nearest neighbour selectiopn)..............................................................................
"""

k=5

print(k)
"""
8. manhattan K-Nearest Neighbours..............................................................................
"""
from sklearn.neighbors import KNeighborsClassifier
knn = KNeighborsClassifier(n_neighbors = k, metric= "manhattan")
knn.fit(X_train, y_train)
y_pred = knn.predict(X_test)
print("manhattan K-Nearest Neighbours")
print(metrics.accuracy_score(y_test, y_pred))

    
"""
8. cosine K-Nearest Neighbours..............................................................................
"""
from sklearn.neighbors import KNeighborsClassifier
knn = KNeighborsClassifier(n_neighbors = k, metric= "cosine")
knn.fit(X_train, y_train)
y_pred = knn.predict(X_test)
print("cosine K-Nearest Neighbours")
print(metrics.accuracy_score(y_test, y_pred))


"""
8. euclidean K-Nearest Neighbours..............................................................................
"""
from sklearn.neighbors import KNeighborsClassifier
knn = KNeighborsClassifier(n_neighbors = k, metric= "euclidean")
knn.fit(X_train, y_train)
y_pred = knn.predict(X_test)
print("euclidean K-Nearest Neighbours")
print(metrics.accuracy_score(y_test, y_pred))

        
"""
9. scipy.spatial.distance K-Nearest Neighbours..............................................................................
"""
from scipy.spatial.distance import *
knn = KNeighborsClassifier(n_neighbors = k, metric= "braycurtis")
knn.fit(X_train, y_train)
y_pred = knn.predict(X_test)
print("braycurtis K-Nearest Neighbours")
print(metrics.accuracy_score(y_test, y_pred))
       
        
knn = KNeighborsClassifier(n_neighbors = k, metric= "canberra")
knn.fit(X_train, y_train)
y_pred = knn.predict(X_test)
print("canberra K-Nearest Neighbours")
print(metrics.accuracy_score(y_test, y_pred))
  
        
knn = KNeighborsClassifier(n_neighbors = k, metric= "jaccard")
knn.fit(X_train, y_train)
y_pred = knn.predict(X_test)
print("jaccard K-Nearest Neighbours")
print(metrics.accuracy_score(y_test, y_pred))
        
knn = KNeighborsClassifier(n_neighbors = k, metric= "yule")
knn.fit(X_train, y_train)
y_pred = knn.predict(X_test)
print("yule K-Nearest Neighbours")
print(metrics.accuracy_score(y_test, y_pred))
print("************************************ ****************************")








txt_files of features used ['/Users/catherine/Desktop/NLP/PAN Datasets/EB2014Selection/output2014GSS.txt']
/Users/catherine/Desktop/NLP/PAN Datasets/EB2014Selection/output2014GSS.txt 300
length of features used = 300
len(all_training_text) 147
len(X_train)= 147 len(y_train)= 147
len(all_test_text) 78
len(X_test= 78 len(y_test)= 78
5
manhattan K-Nearest Neighbours
0.5
cosine K-Nearest Neighbours
0.5
euclidean K-Nearest Neighbours
0.5
braycurtis K-Nearest Neighbours
0.5
canberra K-Nearest Neighbours
0.5
jaccard K-Nearest Neighbours
0.5
yule K-Nearest Neighbours
0.5
************************************ ****************************


/anaconda3/lib/python3.7/site-packages/sklearn/utils/validation.py:475: DataConversionWarning: Data with input dtype float64 was converted to bool by check_pairwise_arrays.
  warnings.warn(msg, DataConversionWarning)


In [20]:

import sys
sys.path.append("/anaconda3/lib/python3.7/site-packages")
import numpy as np
import numpy
import nltk
import pickle
import pandas as pd
import glob
import os
import re
import operator
from collections import Counter
#from read_xml_files import *
##from s_stemmer import *
from nltk.stem.porter import PorterStemmer
from nltk.stem import LancasterStemmer, WordNetLemmatizer
from itertools import groupby
from nltk.collocations import *
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.cluster import KMeans
from scipy.cluster.vq import whiten
from nltk.tokenize.toktok import ToktokTokenizer
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2
import xml.etree.ElementTree as ET
import pandas as pd
#Import scikit-learn metrics module for accuracy calculation
from sklearn import metrics
##from pattern.text.en import singularize
##nltk.download('vader_lexicon')
##nltk.download('punkt')
#import the relevant modules from the NLTK library
from nltk.sentiment.vader import SentimentIntensityAnalyzer
sentence_tokenizer = nltk.data.load('tokenizers/punkt/english.pickle')
word_tokenizer = nltk.tokenize.RegexpTokenizer(r'\w+')
##tt = ToktokTokenizer()#keeps the url as is...................................................
from nltk.tokenize import TweetTokenizer
tt = TweetTokenizer()
Porter_stemmer = PorterStemmer()
Lancaster_stemmer = LancasterStemmer()
WordNet_lemmatizer = WordNetLemmatizer()
#nltk.download('averaged_perceptron_tagger')
##Train_path="D:/Data/PAN2015-Profiling/pan15-author-profiling-training-dataset-english-2015-04-23"#path for train data
##Test_path="D:/Data/PAN2015-Profiling/pan-ap2015-test/en" #path for test data

##Train_path="D:/NLP/PAN2014/pan14-author-profiling-training-corpus-english-twitter-2014-04-16"#path for train data D:\Data\PAN2013-Profiling\Training\en  D:\NLP\PAN2014
##train_truth_path="D:/NLP/PAN2014/truth_train"
##
##test_truth_path="D:/NLP/PAN2014/pan14_test"
##Test_path="D:/NLP/PAN2014/pan14_test/en"
##Content = open('results_of_clssification_all.txt', 'w')

M_train_file= "/Users/catherine/Desktop/NLP/PAN Datasets/PAN_tokens/2014_blog_Tokens/PAN2014_blog_male_train.txt"
F_train_file= "/Users/catherine/Desktop/NLP/PAN Datasets/PAN_tokens/2014_blog_Tokens/PAN2014_blog_female_train.txt"

M_test_file= "/Users/catherine/Desktop/NLP/PAN Datasets/PAN_tokens/2014_blog_Tokens/PAN2014_blog_male_test.txt"
F_test_file= "/Users/catherine/Desktop/NLP/PAN Datasets/PAN_tokens/2014_blog_Tokens/PAN2014_blog_female_test.txt"

def all_txt(M_txt_file,  F_txt_file):
    with open(M_txt_file, mode="r", encoding="utf-8") as f:
            txt_list = [(line[:-1]).split("\t") for line in f]
            male_dict = dict()
            content=[]
            for line in txt_list:
                if line[1] in male_dict:
                # append the new number to the existing array at this slot
                    male_dict[line[1]].append(line[3])
                else:
                # create a new array in this slot
                    male_dict[line[1]] = [line[3]]
    male_content= [("M", (' '.join(a)).split()) for a in list(male_dict.values())]
  
    with open(F_txt_file, mode="r", encoding="utf-8") as f:
            txt_list = [(line[:-1]).split("\t") for line in f]
            female_dict = dict()
            content=[]
            for line in txt_list:
                if line[1] in female_dict:
                # append the new number to the existing array at this slot
                    female_dict[line[1]].append(line[3])
                else:
                # create a new array in this slot
                    female_dict[line[1]] = [line[3]]
    female_content= [("F", (' '.join(a)).split()) for a in list(female_dict.values())]
    all_txt_per_person = male_content + female_content
    return all_txt_per_person

##print( [((all_txt(M_txt_file,  F_txt_file))[i][0])  for i in range(len((all_txt(M_txt_file,  F_txt_file))))]) helps to determine the gender....

      
"""
create feature vector from the training set.....................................................................................
"""

def WordFeatures(word_list, all_training_text):

    fvs_words = np.array([[author.count(word) for word in word_list] for author in all_training_text]).astype(np.float64)

    # normalise by dividing each row by number of tokens for each author........
    fvs_words /= np.c_[np.array([len(author) for author in all_training_text])]

    return fvs_words



"""
From the saved csv file, recover the saved features to be used...............................................................
"""
import csv
word_list=[]
txt_files =["/Users/catherine/Desktop/NLP/PAN Datasets/EB2014Selection/output2014GSS.txt"]
##txt_files =[ "/Users/catherine/Desktop/NLP/PAN Datasets/PAN_tokens/2014_blog_Tokens/PAN2014_blog_female_chosen_set.csv", "/Users/catherine/Desktop/NLP/PAN Datasets/PAN_tokens/2014_blog_Tokens/PAN2014_blog_male_chosen_set.csv"]
print("txt_files of features used",txt_files)
for txt_file in txt_files:
    with open(txt_file, mode="r", encoding="utf-8") as f:
##        reader = csv.reader(f, delimiter=",")
##        next(reader) # skip header
        word_list =  ([(r.split(":"))[1] for r in f])[0:500]
        print(txt_file, len(word_list))
        #word_list = word_list + word_list1
print("length of features used =", len(word_list))

#for txt_file in txt_files:
 #   with open(txt_file, mode="r", encoding="utf-8") as f:
 #       reader = csv.reader(f, delimiter=",")
 #       next(reader) # skip header
 #       word_list1 =  [r[0] for r in reader]
 #       print(txt_file, len(word_list1))
 #       word_list = word_list + word_list1
#print("length of features used =", len(word_list))


"""
Prepare the training and test sets to be parsed to the classifies............................................................
"""
A=(all_txt(M_train_file,  F_train_file))
all_training_text = ( [(A[i][1])  for i in range(len(A))])
print("len(all_training_text)",len(all_training_text))
X_train=WordFeatures(word_list, all_training_text)
y_train=np.array( [(A[i][0])  for i in range(len(A))])
print("len(X_train)=",len(X_train), "len(y_train)=", len(y_train))

B=(all_txt(M_test_file,  F_test_file))
all_test_text = ( [(B[i][1])  for i in range(len(B))])
print("len(all_test_text)",len(all_test_text))
X_test = WordFeatures(word_list, all_test_text)
y_test = np.array( [(B[i][0])  for i in range(len(B))])
print("len(X_test=",len(X_test), "len(y_test)=",len(y_test))


X_train = np.nan_to_num(X_train)
X_test = np.nan_to_num(X_test)

"""
Perform feature selection using chi2............................................................
"""
from sklearn.feature_selection import *
##
####def select_Bestfeatures(X, y):
##print("selector = SelectKBest(score_func=chi2, k=1000)")
##selector = SelectKBest(score_func=chi2, k=1000)
##fit = selector.fit(X_train,y_train)
##train_selected_features = fit.transform(X_train)
##test_selected_features = fit.transform(X_test)



"""
x_train and x_test based on the new features using chi2............................................................
"""
##X_train = select_Bestfeatures(X_train, y_train)
##X_test = select_Bestfeatures(X_test, y_test)


"""
Perform feature selection using f_classif............................................................
"""
from sklearn.feature_selection import *
##def f_classif_select_Bestfeatures(X, y):
##selector=SelectKBest(score_func=f_classif,k=500)
##fit = selector.fit(X_train,y_train)
##train_selected_features = fit.transform(X_train)
##test_selected_features = fit.transform(X_test)
##    selected_features = fit.transform(X)
##    return selected_features

"""
x_train and x_test based on the new features using f_classif............................................................
"""
##X_train = f_classif_select_Bestfeatures(X_train, y_train)
##X_test = f_classif_select_Bestfeatures(X_test, y_test)



"""
Perform feature selection using mutual_info_classif............................................................
"""
from sklearn.feature_selection import *
##def mutual_info_classif_select_Bestfeatures(X_tr, X_te, y):
##selector=SelectKBest(score_func=mutual_info_classif,k=100)
##fit = selector.fit(X_train,y_train)
##train_selected_features = fit.transform(X_train)
##test_selected_features = fit.transform(X_test)
##    return train_selected_features, test_selected_features

"""
x_train and x_test based on the new features using f_classif............................................................
"""
##X_train =train_selected_features 
##X_test = test_selected_features


"""
Perform feature selection using pca............................................................
"""

##from sklearn.decomposition import PCA
##def PCA_select_Bestfeatures(X):
##    # feature extraction
##    pca = PCA(n_components=200)
##    fit = pca.fit(X, y)
##    Train_selected_features = fit.transform(X)
##    return selected_features

"""
x_train and x_test based on the new features using f_classif............................................................
"""
##X_train = PCA_select_Bestfeatures(X_train)
##X_test = PCA_select_Bestfeatures(X_test)

"""
Below are a list of Classification Algorithms with different distance measres ..............................................................................
"""
"""
K-Nearest Neighbours (average aa over all the nearest neighbour selectiopn)..............................................................................
"""

k=5

print(k)
"""
8. manhattan K-Nearest Neighbours..............................................................................
"""
from sklearn.neighbors import KNeighborsClassifier
knn = KNeighborsClassifier(n_neighbors = k, metric= "manhattan")
knn.fit(X_train, y_train)
y_pred = knn.predict(X_test)
print("manhattan K-Nearest Neighbours")
print(metrics.accuracy_score(y_test, y_pred))

    
"""
8. cosine K-Nearest Neighbours..............................................................................
"""
from sklearn.neighbors import KNeighborsClassifier
knn = KNeighborsClassifier(n_neighbors = k, metric= "cosine")
knn.fit(X_train, y_train)
y_pred = knn.predict(X_test)
print("cosine K-Nearest Neighbours")
print(metrics.accuracy_score(y_test, y_pred))


"""
8. euclidean K-Nearest Neighbours..............................................................................
"""
from sklearn.neighbors import KNeighborsClassifier
knn = KNeighborsClassifier(n_neighbors = k, metric= "euclidean")
knn.fit(X_train, y_train)
y_pred = knn.predict(X_test)
print("euclidean K-Nearest Neighbours")
print(metrics.accuracy_score(y_test, y_pred))

        
"""
9. scipy.spatial.distance K-Nearest Neighbours..............................................................................
"""
from scipy.spatial.distance import *
knn = KNeighborsClassifier(n_neighbors = k, metric= "braycurtis")
knn.fit(X_train, y_train)
y_pred = knn.predict(X_test)
print("braycurtis K-Nearest Neighbours")
print(metrics.accuracy_score(y_test, y_pred))
       
        
knn = KNeighborsClassifier(n_neighbors = k, metric= "canberra")
knn.fit(X_train, y_train)
y_pred = knn.predict(X_test)
print("canberra K-Nearest Neighbours")
print(metrics.accuracy_score(y_test, y_pred))
  
        
knn = KNeighborsClassifier(n_neighbors = k, metric= "jaccard")
knn.fit(X_train, y_train)
y_pred = knn.predict(X_test)
print("jaccard K-Nearest Neighbours")
print(metrics.accuracy_score(y_test, y_pred))
        
knn = KNeighborsClassifier(n_neighbors = k, metric= "yule")
knn.fit(X_train, y_train)
y_pred = knn.predict(X_test)
print("yule K-Nearest Neighbours")
print(metrics.accuracy_score(y_test, y_pred))
print("************************************ ****************************")








txt_files of features used ['/Users/catherine/Desktop/NLP/PAN Datasets/EB2014Selection/output2014GSS.txt']
/Users/catherine/Desktop/NLP/PAN Datasets/EB2014Selection/output2014GSS.txt 500
length of features used = 500
len(all_training_text) 147
len(X_train)= 147 len(y_train)= 147
len(all_test_text) 78
len(X_test= 78 len(y_test)= 78
5
manhattan K-Nearest Neighbours
0.5
cosine K-Nearest Neighbours
0.5
euclidean K-Nearest Neighbours
0.5
braycurtis K-Nearest Neighbours
0.5
canberra K-Nearest Neighbours
0.5
jaccard K-Nearest Neighbours
0.5
yule K-Nearest Neighbours
0.5
************************************ ****************************


/anaconda3/lib/python3.7/site-packages/sklearn/utils/validation.py:475: DataConversionWarning: Data with input dtype float64 was converted to bool by check_pairwise_arrays.
  warnings.warn(msg, DataConversionWarning)


In [21]:
import sys
sys.path.append("/anaconda3/lib/python3.7/site-packages")
import numpy as np
import numpy
import nltk
import pickle
import pandas as pd
import glob
import os
import re
import operator
from collections import Counter
#from read_xml_files import *
##from s_stemmer import *
from nltk.stem.porter import PorterStemmer
from nltk.stem import LancasterStemmer, WordNetLemmatizer
from itertools import groupby
from nltk.collocations import *
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.cluster import KMeans
from scipy.cluster.vq import whiten
from nltk.tokenize.toktok import ToktokTokenizer
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2
import xml.etree.ElementTree as ET
import pandas as pd
#Import scikit-learn metrics module for accuracy calculation
from sklearn import metrics
##from pattern.text.en import singularize
##nltk.download('vader_lexicon')
##nltk.download('punkt')
#import the relevant modules from the NLTK library
from nltk.sentiment.vader import SentimentIntensityAnalyzer
sentence_tokenizer = nltk.data.load('tokenizers/punkt/english.pickle')
word_tokenizer = nltk.tokenize.RegexpTokenizer(r'\w+')
##tt = ToktokTokenizer()#keeps the url as is...................................................
from nltk.tokenize import TweetTokenizer
tt = TweetTokenizer()
Porter_stemmer = PorterStemmer()
Lancaster_stemmer = LancasterStemmer()
WordNet_lemmatizer = WordNetLemmatizer()
#nltk.download('averaged_perceptron_tagger')
##Train_path="D:/Data/PAN2015-Profiling/pan15-author-profiling-training-dataset-english-2015-04-23"#path for train data
##Test_path="D:/Data/PAN2015-Profiling/pan-ap2015-test/en" #path for test data

##Train_path="D:/NLP/PAN2014/pan14-author-profiling-training-corpus-english-twitter-2014-04-16"#path for train data D:\Data\PAN2013-Profiling\Training\en  D:\NLP\PAN2014
##train_truth_path="D:/NLP/PAN2014/truth_train"
##
##test_truth_path="D:/NLP/PAN2014/pan14_test"
##Test_path="D:/NLP/PAN2014/pan14_test/en"
##Content = open('results_of_clssification_all.txt', 'w')

M_train_file= "/Users/catherine/Desktop/NLP/PAN Datasets/PAN_tokens/2014_blog_Tokens/PAN2014_blog_male_train.txt"
F_train_file= "/Users/catherine/Desktop/NLP/PAN Datasets/PAN_tokens/2014_blog_Tokens/PAN2014_blog_female_train.txt"

M_test_file= "/Users/catherine/Desktop/NLP/PAN Datasets/PAN_tokens/2014_blog_Tokens/PAN2014_blog_male_test.txt"
F_test_file= "/Users/catherine/Desktop/NLP/PAN Datasets/PAN_tokens/2014_blog_Tokens/PAN2014_blog_female_test.txt"

def all_txt(M_txt_file,  F_txt_file):
    with open(M_txt_file, mode="r", encoding="utf-8") as f:
            txt_list = [(line[:-1]).split("\t") for line in f]
            male_dict = dict()
            content=[]
            for line in txt_list:
                if line[1] in male_dict:
                # append the new number to the existing array at this slot
                    male_dict[line[1]].append(line[3])
                else:
                # create a new array in this slot
                    male_dict[line[1]] = [line[3]]
    male_content= [("M", (' '.join(a)).split()) for a in list(male_dict.values())]
  
    with open(F_txt_file, mode="r", encoding="utf-8") as f:
            txt_list = [(line[:-1]).split("\t") for line in f]
            female_dict = dict()
            content=[]
            for line in txt_list:
                if line[1] in female_dict:
                # append the new number to the existing array at this slot
                    female_dict[line[1]].append(line[3])
                else:
                # create a new array in this slot
                    female_dict[line[1]] = [line[3]]
    female_content= [("F", (' '.join(a)).split()) for a in list(female_dict.values())]
    all_txt_per_person = male_content + female_content
    return all_txt_per_person

##print( [((all_txt(M_txt_file,  F_txt_file))[i][0])  for i in range(len((all_txt(M_txt_file,  F_txt_file))))]) helps to determine the gender....

      
"""
create feature vector from the training set.....................................................................................
"""

def WordFeatures(word_list, all_training_text):

    fvs_words = np.array([[author.count(word) for word in word_list] for author in all_training_text]).astype(np.float64)

    # normalise by dividing each row by number of tokens for each author........
    fvs_words /= np.c_[np.array([len(author) for author in all_training_text])]

    return fvs_words



"""
From the saved csv file, recover the saved features to be used...............................................................
"""
import csv
word_list=[]
##txt_files =[ "/Users/catherine/Desktop/NLP/PAN Datasets/PAN_chosen_tokens/2014/output2014OR.1M.txt", "/Users/catherine/Desktop/NLP/PAN Datasets/PAN_chosen_tokens/2014/output2014OR.2F.txt"]
txt_files =[ "/Users/catherine/Desktop/NLP/PAN Datasets/PAN_tokens/2014_blog_Tokens/PAN2014_blog_female_chosen_set.csv", "/Users/catherine/Desktop/NLP/PAN Datasets/PAN_tokens/2014_blog_Tokens/PAN2014_blog_male_chosen_set.csv"]
print("txt_files of features used",txt_files)
##for txt_file in txt_files:
##    with open(txt_file, mode="r", encoding="utf-8") as f:
##        reader = csv.reader(f, delimiter=",")
##        next(reader) # skip header
##        word_list1 =  ([(r.split(":"))[1] for r in f])
##        print(txt_file, len(word_list1))
##        word_list = word_list + word_list1
##print("length of features used =", len(word_list))

for txt_file in txt_files:
    with open(txt_file, mode="r", encoding="utf-8") as f:
        reader = csv.reader(f, delimiter=",")
        next(reader) # skip header
        word_list1 =  [r[0] for r in reader]
        print(txt_file, len(word_list1))
        word_list = word_list + word_list1
print("length of features used =", len(word_list))


"""
Prepare the training and test sets to be parsed to the classifies............................................................
"""
A=(all_txt(M_train_file,  F_train_file))
all_training_text = ( [(A[i][1])  for i in range(len(A))])
print("len(all_training_text)",len(all_training_text))
X_train=WordFeatures(word_list, all_training_text)
y_train=np.array( [(A[i][0])  for i in range(len(A))])
print("len(X_train)=",len(X_train), "len(y_train)=", len(y_train))

B=(all_txt(M_test_file,  F_test_file))
all_test_text = ( [(B[i][1])  for i in range(len(B))])
print("len(all_test_text)",len(all_test_text))
X_test = WordFeatures(word_list, all_test_text)
y_test = np.array( [(B[i][0])  for i in range(len(B))])
print("len(X_test=",len(X_test), "len(y_test)=",len(y_test))


X_train = np.nan_to_num(X_train)
X_test = np.nan_to_num(X_test)

"""
Perform feature selection using chi2............................................................
"""
from sklearn.feature_selection import *
##
####def select_Bestfeatures(X, y):
print("selector = SelectKBest(score_func=chi2, k=200)")
selector = SelectKBest(score_func=chi2, k=200)
fit = selector.fit(X_train,y_train)
train_selected_features = fit.transform(X_train)
test_selected_features = fit.transform(X_test)



"""
x_train and x_test based on the new features using chi2............................................................
"""
##X_train = select_Bestfeatures(X_train, y_train)
##X_test = select_Bestfeatures(X_test, y_test)


"""
Perform feature selection using f_classif............................................................
"""
from sklearn.feature_selection import *
##def f_classif_select_Bestfeatures(X, y):
##selector=SelectKBest(score_func=f_classif,k=500)
##fit = selector.fit(X_train,y_train)
##train_selected_features = fit.transform(X_train)
##test_selected_features = fit.transform(X_test)
##    selected_features = fit.transform(X)
##    return selected_features

"""
x_train and x_test based on the new features using f_classif............................................................
"""
##X_train = f_classif_select_Bestfeatures(X_train, y_train)
##X_test = f_classif_select_Bestfeatures(X_test, y_test)



"""
Perform feature selection using mutual_info_classif............................................................
"""
from sklearn.feature_selection import *
##def mutual_info_classif_select_Bestfeatures(X_tr, X_te, y):
##selector=SelectKBest(score_func=mutual_info_classif,k=100)
##fit = selector.fit(X_train,y_train)
##train_selected_features = fit.transform(X_train)
##test_selected_features = fit.transform(X_test)
##    return train_selected_features, test_selected_features

"""
x_train and x_test based on the new features using f_classif............................................................
"""
X_train =train_selected_features 
X_test = test_selected_features


"""
Perform feature selection using pca............................................................
"""

##from sklearn.decomposition import PCA
##def PCA_select_Bestfeatures(X):
##    # feature extraction
##    pca = PCA(n_components=200)
##    fit = pca.fit(X, y)
##    Train_selected_features = fit.transform(X)
##    return selected_features

"""
x_train and x_test based on the new features using f_classif............................................................
"""
##X_train = PCA_select_Bestfeatures(X_train)
##X_test = PCA_select_Bestfeatures(X_test)

"""
Below are a list of Classification Algorithms with different distance measres ..............................................................................
"""
"""
K-Nearest Neighbours (average aa over all the nearest neighbour selectiopn)..............................................................................
"""

k=5

print(k)
"""
8. manhattan K-Nearest Neighbours..............................................................................
"""
from sklearn.neighbors import KNeighborsClassifier
knn = KNeighborsClassifier(n_neighbors = k, metric= "manhattan")
knn.fit(X_train, y_train)
y_pred = knn.predict(X_test)
print("manhattan K-Nearest Neighbours")
print(metrics.accuracy_score(y_test, y_pred))

    
"""
8. cosine K-Nearest Neighbours..............................................................................
"""
from sklearn.neighbors import KNeighborsClassifier
knn = KNeighborsClassifier(n_neighbors = k, metric= "cosine")
knn.fit(X_train, y_train)
y_pred = knn.predict(X_test)
print("cosine K-Nearest Neighbours")
print(metrics.accuracy_score(y_test, y_pred))


"""
8. euclidean K-Nearest Neighbours..............................................................................
"""
from sklearn.neighbors import KNeighborsClassifier
knn = KNeighborsClassifier(n_neighbors = k, metric= "euclidean")
knn.fit(X_train, y_train)
y_pred = knn.predict(X_test)
print("euclidean K-Nearest Neighbours")
print(metrics.accuracy_score(y_test, y_pred))

        
"""
9. scipy.spatial.distance K-Nearest Neighbours..............................................................................
"""
from scipy.spatial.distance import *
knn = KNeighborsClassifier(n_neighbors = k, metric= "braycurtis")
knn.fit(X_train, y_train)
y_pred = knn.predict(X_test)
print("braycurtis K-Nearest Neighbours")
print(metrics.accuracy_score(y_test, y_pred))
       
        
knn = KNeighborsClassifier(n_neighbors = k, metric= "canberra")
knn.fit(X_train, y_train)
y_pred = knn.predict(X_test)
print("canberra K-Nearest Neighbours")
print(metrics.accuracy_score(y_test, y_pred))
  
        
knn = KNeighborsClassifier(n_neighbors = k, metric= "jaccard")
knn.fit(X_train, y_train)
y_pred = knn.predict(X_test)
print("jaccard K-Nearest Neighbours")
print(metrics.accuracy_score(y_test, y_pred))
        
knn = KNeighborsClassifier(n_neighbors = k, metric= "yule")
knn.fit(X_train, y_train)
y_pred = knn.predict(X_test)
print("yule K-Nearest Neighbours")
print(metrics.accuracy_score(y_test, y_pred))
print("************************************ ****************************")


txt_files of features used ['/Users/catherine/Desktop/NLP/PAN Datasets/PAN_tokens/2014_blog_Tokens/PAN2014_blog_female_chosen_set.csv', '/Users/catherine/Desktop/NLP/PAN Datasets/PAN_tokens/2014_blog_Tokens/PAN2014_blog_male_chosen_set.csv']
/Users/catherine/Desktop/NLP/PAN Datasets/PAN_tokens/2014_blog_Tokens/PAN2014_blog_female_chosen_set.csv 1196
/Users/catherine/Desktop/NLP/PAN Datasets/PAN_tokens/2014_blog_Tokens/PAN2014_blog_male_chosen_set.csv 1636
length of features used = 2832
len(all_training_text) 147
len(X_train)= 147 len(y_train)= 147
len(all_test_text) 78
len(X_test= 78 len(y_test)= 78
selector = SelectKBest(score_func=chi2, k=200)
5
manhattan K-Nearest Neighbours
0.5769230769230769
cosine K-Nearest Neighbours
0.6025641025641025
euclidean K-Nearest Neighbours
0.5384615384615384
braycurtis K-Nearest Neighbours
0.6538461538461539
canberra K-Nearest Neighbours
0.47435897435897434
jaccard K-Nearest Neighbours
0.5769230769230769
yule K-Nearest Neighbours
0.5384615384615384
***

/anaconda3/lib/python3.7/site-packages/sklearn/utils/validation.py:475: DataConversionWarning: Data with input dtype float64 was converted to bool by check_pairwise_arrays.
  warnings.warn(msg, DataConversionWarning)


In [22]:
import sys
sys.path.append("/anaconda3/lib/python3.7/site-packages")
import numpy as np
import numpy
import nltk
import pickle
import pandas as pd
import glob
import os
import re
import operator
from collections import Counter
#from read_xml_files import *
##from s_stemmer import *
from nltk.stem.porter import PorterStemmer
from nltk.stem import LancasterStemmer, WordNetLemmatizer
from itertools import groupby
from nltk.collocations import *
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.cluster import KMeans
from scipy.cluster.vq import whiten
from nltk.tokenize.toktok import ToktokTokenizer
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2
import xml.etree.ElementTree as ET
import pandas as pd
#Import scikit-learn metrics module for accuracy calculation
from sklearn import metrics
##from pattern.text.en import singularize
##nltk.download('vader_lexicon')
##nltk.download('punkt')
#import the relevant modules from the NLTK library
from nltk.sentiment.vader import SentimentIntensityAnalyzer
sentence_tokenizer = nltk.data.load('tokenizers/punkt/english.pickle')
word_tokenizer = nltk.tokenize.RegexpTokenizer(r'\w+')
##tt = ToktokTokenizer()#keeps the url as is...................................................
from nltk.tokenize import TweetTokenizer
tt = TweetTokenizer()
Porter_stemmer = PorterStemmer()
Lancaster_stemmer = LancasterStemmer()
WordNet_lemmatizer = WordNetLemmatizer()
#nltk.download('averaged_perceptron_tagger')
##Train_path="D:/Data/PAN2015-Profiling/pan15-author-profiling-training-dataset-english-2015-04-23"#path for train data
##Test_path="D:/Data/PAN2015-Profiling/pan-ap2015-test/en" #path for test data

##Train_path="D:/NLP/PAN2014/pan14-author-profiling-training-corpus-english-twitter-2014-04-16"#path for train data D:\Data\PAN2013-Profiling\Training\en  D:\NLP\PAN2014
##train_truth_path="D:/NLP/PAN2014/truth_train"
##
##test_truth_path="D:/NLP/PAN2014/pan14_test"
##Test_path="D:/NLP/PAN2014/pan14_test/en"
##Content = open('results_of_clssification_all.txt', 'w')

M_train_file= "/Users/catherine/Desktop/NLP/PAN Datasets/PAN_tokens/2014_blog_Tokens/PAN2014_blog_male_train.txt"
F_train_file= "/Users/catherine/Desktop/NLP/PAN Datasets/PAN_tokens/2014_blog_Tokens/PAN2014_blog_female_train.txt"

M_test_file= "/Users/catherine/Desktop/NLP/PAN Datasets/PAN_tokens/2014_blog_Tokens/PAN2014_blog_male_test.txt"
F_test_file= "/Users/catherine/Desktop/NLP/PAN Datasets/PAN_tokens/2014_blog_Tokens/PAN2014_blog_female_test.txt"

def all_txt(M_txt_file,  F_txt_file):
    with open(M_txt_file, mode="r", encoding="utf-8") as f:
            txt_list = [(line[:-1]).split("\t") for line in f]
            male_dict = dict()
            content=[]
            for line in txt_list:
                if line[1] in male_dict:
                # append the new number to the existing array at this slot
                    male_dict[line[1]].append(line[3])
                else:
                # create a new array in this slot
                    male_dict[line[1]] = [line[3]]
    male_content= [("M", (' '.join(a)).split()) for a in list(male_dict.values())]
  
    with open(F_txt_file, mode="r", encoding="utf-8") as f:
            txt_list = [(line[:-1]).split("\t") for line in f]
            female_dict = dict()
            content=[]
            for line in txt_list:
                if line[1] in female_dict:
                # append the new number to the existing array at this slot
                    female_dict[line[1]].append(line[3])
                else:
                # create a new array in this slot
                    female_dict[line[1]] = [line[3]]
    female_content= [("F", (' '.join(a)).split()) for a in list(female_dict.values())]
    all_txt_per_person = male_content + female_content
    return all_txt_per_person

##print( [((all_txt(M_txt_file,  F_txt_file))[i][0])  for i in range(len((all_txt(M_txt_file,  F_txt_file))))]) helps to determine the gender....

      
"""
create feature vector from the training set.....................................................................................
"""

def WordFeatures(word_list, all_training_text):

    fvs_words = np.array([[author.count(word) for word in word_list] for author in all_training_text]).astype(np.float64)

    # normalise by dividing each row by number of tokens for each author........
    fvs_words /= np.c_[np.array([len(author) for author in all_training_text])]

    return fvs_words



"""
From the saved csv file, recover the saved features to be used...............................................................
"""
import csv
word_list=[]
##txt_files =[ "/Users/catherine/Desktop/NLP/PAN Datasets/PAN_chosen_tokens/2014/output2014OR.1M.txt", "/Users/catherine/Desktop/NLP/PAN Datasets/PAN_chosen_tokens/2014/output2014OR.2F.txt"]
txt_files =[ "/Users/catherine/Desktop/NLP/PAN Datasets/PAN_tokens/2014_blog_Tokens/PAN2014_blog_female_chosen_set.csv", "/Users/catherine/Desktop/NLP/PAN Datasets/PAN_tokens/2014_blog_Tokens/PAN2014_blog_male_chosen_set.csv"]
print("txt_files of features used",txt_files)
##for txt_file in txt_files:
##    with open(txt_file, mode="r", encoding="utf-8") as f:
##        reader = csv.reader(f, delimiter=",")
##        next(reader) # skip header
##        word_list1 =  ([(r.split(":"))[1] for r in f])
##        print(txt_file, len(word_list1))
##        word_list = word_list + word_list1
##print("length of features used =", len(word_list))

for txt_file in txt_files:
    with open(txt_file, mode="r", encoding="utf-8") as f:
        reader = csv.reader(f, delimiter=",")
        next(reader) # skip header
        word_list1 =  [r[0] for r in reader]
        print(txt_file, len(word_list1))
        word_list = word_list + word_list1
print("length of features used =", len(word_list))


"""
Prepare the training and test sets to be parsed to the classifies............................................................
"""
A=(all_txt(M_train_file,  F_train_file))
all_training_text = ( [(A[i][1])  for i in range(len(A))])
print("len(all_training_text)",len(all_training_text))
X_train=WordFeatures(word_list, all_training_text)
y_train=np.array( [(A[i][0])  for i in range(len(A))])
print("len(X_train)=",len(X_train), "len(y_train)=", len(y_train))

B=(all_txt(M_test_file,  F_test_file))
all_test_text = ( [(B[i][1])  for i in range(len(B))])
print("len(all_test_text)",len(all_test_text))
X_test = WordFeatures(word_list, all_test_text)
y_test = np.array( [(B[i][0])  for i in range(len(B))])
print("len(X_test=",len(X_test), "len(y_test)=",len(y_test))


X_train = np.nan_to_num(X_train)
X_test = np.nan_to_num(X_test)

"""
Perform feature selection using chi2............................................................
"""
from sklearn.feature_selection import *
##
####def select_Bestfeatures(X, y):
print("selector = SelectKBest(score_func=chi2, k=200)")
selector = SelectKBest(score_func=chi2, k=300)
fit = selector.fit(X_train,y_train)
train_selected_features = fit.transform(X_train)
test_selected_features = fit.transform(X_test)



"""
x_train and x_test based on the new features using chi2............................................................
"""
##X_train = select_Bestfeatures(X_train, y_train)
##X_test = select_Bestfeatures(X_test, y_test)


"""
Perform feature selection using f_classif............................................................
"""
from sklearn.feature_selection import *
##def f_classif_select_Bestfeatures(X, y):
##selector=SelectKBest(score_func=f_classif,k=500)
##fit = selector.fit(X_train,y_train)
##train_selected_features = fit.transform(X_train)
##test_selected_features = fit.transform(X_test)
##    selected_features = fit.transform(X)
##    return selected_features

"""
x_train and x_test based on the new features using f_classif............................................................
"""
##X_train = f_classif_select_Bestfeatures(X_train, y_train)
##X_test = f_classif_select_Bestfeatures(X_test, y_test)



"""
Perform feature selection using mutual_info_classif............................................................
"""
from sklearn.feature_selection import *
##def mutual_info_classif_select_Bestfeatures(X_tr, X_te, y):
##selector=SelectKBest(score_func=mutual_info_classif,k=100)
##fit = selector.fit(X_train,y_train)
##train_selected_features = fit.transform(X_train)
##test_selected_features = fit.transform(X_test)
##    return train_selected_features, test_selected_features

"""
x_train and x_test based on the new features using f_classif............................................................
"""
X_train =train_selected_features 
X_test = test_selected_features


"""
Perform feature selection using pca............................................................
"""

##from sklearn.decomposition import PCA
##def PCA_select_Bestfeatures(X):
##    # feature extraction
##    pca = PCA(n_components=200)
##    fit = pca.fit(X, y)
##    Train_selected_features = fit.transform(X)
##    return selected_features

"""
x_train and x_test based on the new features using f_classif............................................................
"""
##X_train = PCA_select_Bestfeatures(X_train)
##X_test = PCA_select_Bestfeatures(X_test)

"""
Below are a list of Classification Algorithms with different distance measres ..............................................................................
"""
"""
K-Nearest Neighbours (average aa over all the nearest neighbour selectiopn)..............................................................................
"""

k=5

print(k)
"""
8. manhattan K-Nearest Neighbours..............................................................................
"""
from sklearn.neighbors import KNeighborsClassifier
knn = KNeighborsClassifier(n_neighbors = k, metric= "manhattan")
knn.fit(X_train, y_train)
y_pred = knn.predict(X_test)
print("manhattan K-Nearest Neighbours")
print(metrics.accuracy_score(y_test, y_pred))

    
"""
8. cosine K-Nearest Neighbours..............................................................................
"""
from sklearn.neighbors import KNeighborsClassifier
knn = KNeighborsClassifier(n_neighbors = k, metric= "cosine")
knn.fit(X_train, y_train)
y_pred = knn.predict(X_test)
print("cosine K-Nearest Neighbours")
print(metrics.accuracy_score(y_test, y_pred))


"""
8. euclidean K-Nearest Neighbours..............................................................................
"""
from sklearn.neighbors import KNeighborsClassifier
knn = KNeighborsClassifier(n_neighbors = k, metric= "euclidean")
knn.fit(X_train, y_train)
y_pred = knn.predict(X_test)
print("euclidean K-Nearest Neighbours")
print(metrics.accuracy_score(y_test, y_pred))

        
"""
9. scipy.spatial.distance K-Nearest Neighbours..............................................................................
"""
from scipy.spatial.distance import *
knn = KNeighborsClassifier(n_neighbors = k, metric= "braycurtis")
knn.fit(X_train, y_train)
y_pred = knn.predict(X_test)
print("braycurtis K-Nearest Neighbours")
print(metrics.accuracy_score(y_test, y_pred))
       
        
knn = KNeighborsClassifier(n_neighbors = k, metric= "canberra")
knn.fit(X_train, y_train)
y_pred = knn.predict(X_test)
print("canberra K-Nearest Neighbours")
print(metrics.accuracy_score(y_test, y_pred))
  
        
knn = KNeighborsClassifier(n_neighbors = k, metric= "jaccard")
knn.fit(X_train, y_train)
y_pred = knn.predict(X_test)
print("jaccard K-Nearest Neighbours")
print(metrics.accuracy_score(y_test, y_pred))
        
knn = KNeighborsClassifier(n_neighbors = k, metric= "yule")
knn.fit(X_train, y_train)
y_pred = knn.predict(X_test)
print("yule K-Nearest Neighbours")
print(metrics.accuracy_score(y_test, y_pred))
print("************************************ ****************************")



txt_files of features used ['/Users/catherine/Desktop/NLP/PAN Datasets/PAN_tokens/2014_blog_Tokens/PAN2014_blog_female_chosen_set.csv', '/Users/catherine/Desktop/NLP/PAN Datasets/PAN_tokens/2014_blog_Tokens/PAN2014_blog_male_chosen_set.csv']
/Users/catherine/Desktop/NLP/PAN Datasets/PAN_tokens/2014_blog_Tokens/PAN2014_blog_female_chosen_set.csv 1196
/Users/catherine/Desktop/NLP/PAN Datasets/PAN_tokens/2014_blog_Tokens/PAN2014_blog_male_chosen_set.csv 1636
length of features used = 2832
len(all_training_text) 147
len(X_train)= 147 len(y_train)= 147
len(all_test_text) 78
len(X_test= 78 len(y_test)= 78
selector = SelectKBest(score_func=chi2, k=200)
5
manhattan K-Nearest Neighbours
0.5769230769230769
cosine K-Nearest Neighbours
0.5769230769230769
euclidean K-Nearest Neighbours
0.6025641025641025
braycurtis K-Nearest Neighbours
0.6025641025641025
canberra K-Nearest Neighbours
0.5512820512820513
jaccard K-Nearest Neighbours
0.5641025641025641
yule K-Nearest Neighbours
0.5128205128205128
****

/anaconda3/lib/python3.7/site-packages/sklearn/utils/validation.py:475: DataConversionWarning: Data with input dtype float64 was converted to bool by check_pairwise_arrays.
  warnings.warn(msg, DataConversionWarning)


In [23]:
import sys
sys.path.append("/anaconda3/lib/python3.7/site-packages")
import numpy as np
import numpy
import nltk
import pickle
import pandas as pd
import glob
import os
import re
import operator
from collections import Counter
#from read_xml_files import *
##from s_stemmer import *
from nltk.stem.porter import PorterStemmer
from nltk.stem import LancasterStemmer, WordNetLemmatizer
from itertools import groupby
from nltk.collocations import *
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.cluster import KMeans
from scipy.cluster.vq import whiten
from nltk.tokenize.toktok import ToktokTokenizer
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2
import xml.etree.ElementTree as ET
import pandas as pd
#Import scikit-learn metrics module for accuracy calculation
from sklearn import metrics
##from pattern.text.en import singularize
##nltk.download('vader_lexicon')
##nltk.download('punkt')
#import the relevant modules from the NLTK library
from nltk.sentiment.vader import SentimentIntensityAnalyzer
sentence_tokenizer = nltk.data.load('tokenizers/punkt/english.pickle')
word_tokenizer = nltk.tokenize.RegexpTokenizer(r'\w+')
##tt = ToktokTokenizer()#keeps the url as is...................................................
from nltk.tokenize import TweetTokenizer
tt = TweetTokenizer()
Porter_stemmer = PorterStemmer()
Lancaster_stemmer = LancasterStemmer()
WordNet_lemmatizer = WordNetLemmatizer()
#nltk.download('averaged_perceptron_tagger')
##Train_path="D:/Data/PAN2015-Profiling/pan15-author-profiling-training-dataset-english-2015-04-23"#path for train data
##Test_path="D:/Data/PAN2015-Profiling/pan-ap2015-test/en" #path for test data

##Train_path="D:/NLP/PAN2014/pan14-author-profiling-training-corpus-english-twitter-2014-04-16"#path for train data D:\Data\PAN2013-Profiling\Training\en  D:\NLP\PAN2014
##train_truth_path="D:/NLP/PAN2014/truth_train"
##
##test_truth_path="D:/NLP/PAN2014/pan14_test"
##Test_path="D:/NLP/PAN2014/pan14_test/en"
##Content = open('results_of_clssification_all.txt', 'w')

M_train_file= "/Users/catherine/Desktop/NLP/PAN Datasets/PAN_tokens/2014_blog_Tokens/PAN2014_blog_male_train.txt"
F_train_file= "/Users/catherine/Desktop/NLP/PAN Datasets/PAN_tokens/2014_blog_Tokens/PAN2014_blog_female_train.txt"

M_test_file= "/Users/catherine/Desktop/NLP/PAN Datasets/PAN_tokens/2014_blog_Tokens/PAN2014_blog_male_test.txt"
F_test_file= "/Users/catherine/Desktop/NLP/PAN Datasets/PAN_tokens/2014_blog_Tokens/PAN2014_blog_female_test.txt"

def all_txt(M_txt_file,  F_txt_file):
    with open(M_txt_file, mode="r", encoding="utf-8") as f:
            txt_list = [(line[:-1]).split("\t") for line in f]
            male_dict = dict()
            content=[]
            for line in txt_list:
                if line[1] in male_dict:
                # append the new number to the existing array at this slot
                    male_dict[line[1]].append(line[3])
                else:
                # create a new array in this slot
                    male_dict[line[1]] = [line[3]]
    male_content= [("M", (' '.join(a)).split()) for a in list(male_dict.values())]
  
    with open(F_txt_file, mode="r", encoding="utf-8") as f:
            txt_list = [(line[:-1]).split("\t") for line in f]
            female_dict = dict()
            content=[]
            for line in txt_list:
                if line[1] in female_dict:
                # append the new number to the existing array at this slot
                    female_dict[line[1]].append(line[3])
                else:
                # create a new array in this slot
                    female_dict[line[1]] = [line[3]]
    female_content= [("F", (' '.join(a)).split()) for a in list(female_dict.values())]
    all_txt_per_person = male_content + female_content
    return all_txt_per_person

##print( [((all_txt(M_txt_file,  F_txt_file))[i][0])  for i in range(len((all_txt(M_txt_file,  F_txt_file))))]) helps to determine the gender....

      
"""
create feature vector from the training set.....................................................................................
"""

def WordFeatures(word_list, all_training_text):

    fvs_words = np.array([[author.count(word) for word in word_list] for author in all_training_text]).astype(np.float64)

    # normalise by dividing each row by number of tokens for each author........
    fvs_words /= np.c_[np.array([len(author) for author in all_training_text])]

    return fvs_words



"""
From the saved csv file, recover the saved features to be used...............................................................
"""
import csv
word_list=[]
##txt_files =[ "/Users/catherine/Desktop/NLP/PAN Datasets/PAN_chosen_tokens/2014/output2014OR.1M.txt", "/Users/catherine/Desktop/NLP/PAN Datasets/PAN_chosen_tokens/2014/output2014OR.2F.txt"]
txt_files =[ "/Users/catherine/Desktop/NLP/PAN Datasets/PAN_tokens/2014_blog_Tokens/PAN2014_blog_female_chosen_set.csv", "/Users/catherine/Desktop/NLP/PAN Datasets/PAN_tokens/2014_blog_Tokens/PAN2014_blog_male_chosen_set.csv"]
print("txt_files of features used",txt_files)
##for txt_file in txt_files:
##    with open(txt_file, mode="r", encoding="utf-8") as f:
##        reader = csv.reader(f, delimiter=",")
##        next(reader) # skip header
##        word_list1 =  ([(r.split(":"))[1] for r in f])
##        print(txt_file, len(word_list1))
##        word_list = word_list + word_list1
##print("length of features used =", len(word_list))

for txt_file in txt_files:
    with open(txt_file, mode="r", encoding="utf-8") as f:
        reader = csv.reader(f, delimiter=",")
        next(reader) # skip header
        word_list1 =  [r[0] for r in reader]
        print(txt_file, len(word_list1))
        word_list = word_list + word_list1
print("length of features used =", len(word_list))


"""
Prepare the training and test sets to be parsed to the classifies............................................................
"""
A=(all_txt(M_train_file,  F_train_file))
all_training_text = ( [(A[i][1])  for i in range(len(A))])
print("len(all_training_text)",len(all_training_text))
X_train=WordFeatures(word_list, all_training_text)
y_train=np.array( [(A[i][0])  for i in range(len(A))])
print("len(X_train)=",len(X_train), "len(y_train)=", len(y_train))

B=(all_txt(M_test_file,  F_test_file))
all_test_text = ( [(B[i][1])  for i in range(len(B))])
print("len(all_test_text)",len(all_test_text))
X_test = WordFeatures(word_list, all_test_text)
y_test = np.array( [(B[i][0])  for i in range(len(B))])
print("len(X_test=",len(X_test), "len(y_test)=",len(y_test))


X_train = np.nan_to_num(X_train)
X_test = np.nan_to_num(X_test)

"""
Perform feature selection using chi2............................................................
"""
from sklearn.feature_selection import *
##
####def select_Bestfeatures(X, y):
print("selector = SelectKBest(score_func=chi2, k=500)")
selector = SelectKBest(score_func=chi2, k=500)
fit = selector.fit(X_train,y_train)
train_selected_features = fit.transform(X_train)
test_selected_features = fit.transform(X_test)



"""
x_train and x_test based on the new features using chi2............................................................
"""
##X_train = select_Bestfeatures(X_train, y_train)
##X_test = select_Bestfeatures(X_test, y_test)


"""
Perform feature selection using f_classif............................................................
"""
from sklearn.feature_selection import *
##def f_classif_select_Bestfeatures(X, y):
##selector=SelectKBest(score_func=f_classif,k=500)
##fit = selector.fit(X_train,y_train)
##train_selected_features = fit.transform(X_train)
##test_selected_features = fit.transform(X_test)
##    selected_features = fit.transform(X)
##    return selected_features

"""
x_train and x_test based on the new features using f_classif............................................................
"""
##X_train = f_classif_select_Bestfeatures(X_train, y_train)
##X_test = f_classif_select_Bestfeatures(X_test, y_test)



"""
Perform feature selection using mutual_info_classif............................................................
"""
from sklearn.feature_selection import *
##def mutual_info_classif_select_Bestfeatures(X_tr, X_te, y):
##selector=SelectKBest(score_func=mutual_info_classif,k=100)
##fit = selector.fit(X_train,y_train)
##train_selected_features = fit.transform(X_train)
##test_selected_features = fit.transform(X_test)
##    return train_selected_features, test_selected_features

"""
x_train and x_test based on the new features using f_classif............................................................
"""
X_train =train_selected_features 
X_test = test_selected_features


"""
Perform feature selection using pca............................................................
"""

##from sklearn.decomposition import PCA
##def PCA_select_Bestfeatures(X):
##    # feature extraction
##    pca = PCA(n_components=200)
##    fit = pca.fit(X, y)
##    Train_selected_features = fit.transform(X)
##    return selected_features

"""
x_train and x_test based on the new features using f_classif............................................................
"""
##X_train = PCA_select_Bestfeatures(X_train)
##X_test = PCA_select_Bestfeatures(X_test)

"""
Below are a list of Classification Algorithms with different distance measres ..............................................................................
"""
"""
K-Nearest Neighbours (average aa over all the nearest neighbour selectiopn)..............................................................................
"""

k=5

print(k)
"""
8. manhattan K-Nearest Neighbours..............................................................................
"""
from sklearn.neighbors import KNeighborsClassifier
knn = KNeighborsClassifier(n_neighbors = k, metric= "manhattan")
knn.fit(X_train, y_train)
y_pred = knn.predict(X_test)
print("manhattan K-Nearest Neighbours")
print(metrics.accuracy_score(y_test, y_pred))

    
"""
8. cosine K-Nearest Neighbours..............................................................................
"""
from sklearn.neighbors import KNeighborsClassifier
knn = KNeighborsClassifier(n_neighbors = k, metric= "cosine")
knn.fit(X_train, y_train)
y_pred = knn.predict(X_test)
print("cosine K-Nearest Neighbours")
print(metrics.accuracy_score(y_test, y_pred))


"""
8. euclidean K-Nearest Neighbours..............................................................................
"""
from sklearn.neighbors import KNeighborsClassifier
knn = KNeighborsClassifier(n_neighbors = k, metric= "euclidean")
knn.fit(X_train, y_train)
y_pred = knn.predict(X_test)
print("euclidean K-Nearest Neighbours")
print(metrics.accuracy_score(y_test, y_pred))

        
"""
9. scipy.spatial.distance K-Nearest Neighbours..............................................................................
"""
from scipy.spatial.distance import *
knn = KNeighborsClassifier(n_neighbors = k, metric= "braycurtis")
knn.fit(X_train, y_train)
y_pred = knn.predict(X_test)
print("braycurtis K-Nearest Neighbours")
print(metrics.accuracy_score(y_test, y_pred))
       
        
knn = KNeighborsClassifier(n_neighbors = k, metric= "canberra")
knn.fit(X_train, y_train)
y_pred = knn.predict(X_test)
print("canberra K-Nearest Neighbours")
print(metrics.accuracy_score(y_test, y_pred))
  
        
knn = KNeighborsClassifier(n_neighbors = k, metric= "jaccard")
knn.fit(X_train, y_train)
y_pred = knn.predict(X_test)
print("jaccard K-Nearest Neighbours")
print(metrics.accuracy_score(y_test, y_pred))
        
knn = KNeighborsClassifier(n_neighbors = k, metric= "yule")
knn.fit(X_train, y_train)
y_pred = knn.predict(X_test)
print("yule K-Nearest Neighbours")
print(metrics.accuracy_score(y_test, y_pred))
print("************************************ ****************************")



txt_files of features used ['/Users/catherine/Desktop/NLP/PAN Datasets/PAN_tokens/2014_blog_Tokens/PAN2014_blog_female_chosen_set.csv', '/Users/catherine/Desktop/NLP/PAN Datasets/PAN_tokens/2014_blog_Tokens/PAN2014_blog_male_chosen_set.csv']
/Users/catherine/Desktop/NLP/PAN Datasets/PAN_tokens/2014_blog_Tokens/PAN2014_blog_female_chosen_set.csv 1196
/Users/catherine/Desktop/NLP/PAN Datasets/PAN_tokens/2014_blog_Tokens/PAN2014_blog_male_chosen_set.csv 1636
length of features used = 2832
len(all_training_text) 147
len(X_train)= 147 len(y_train)= 147
len(all_test_text) 78
len(X_test= 78 len(y_test)= 78
selector = SelectKBest(score_func=chi2, k=500)
5
manhattan K-Nearest Neighbours
0.6153846153846154
cosine K-Nearest Neighbours
0.5897435897435898
euclidean K-Nearest Neighbours
0.6153846153846154
braycurtis K-Nearest Neighbours
0.5897435897435898
canberra K-Nearest Neighbours
0.5384615384615384
jaccard K-Nearest Neighbours
0.5897435897435898
yule K-Nearest Neighbours
0.5256410256410257
****

/anaconda3/lib/python3.7/site-packages/sklearn/utils/validation.py:475: DataConversionWarning: Data with input dtype float64 was converted to bool by check_pairwise_arrays.
  warnings.warn(msg, DataConversionWarning)


In [24]:
import sys
sys.path.append("/anaconda3/lib/python3.7/site-packages")
import numpy as np
import numpy
import nltk
import pickle
import pandas as pd
import glob
import os
import re
import operator
from collections import Counter
#from read_xml_files import *
##from s_stemmer import *
from nltk.stem.porter import PorterStemmer
from nltk.stem import LancasterStemmer, WordNetLemmatizer
from itertools import groupby
from nltk.collocations import *
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.cluster import KMeans
from scipy.cluster.vq import whiten
from nltk.tokenize.toktok import ToktokTokenizer
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2
import xml.etree.ElementTree as ET
import pandas as pd
#Import scikit-learn metrics module for accuracy calculation
from sklearn import metrics
##from pattern.text.en import singularize
##nltk.download('vader_lexicon')
##nltk.download('punkt')
#import the relevant modules from the NLTK library
from nltk.sentiment.vader import SentimentIntensityAnalyzer
sentence_tokenizer = nltk.data.load('tokenizers/punkt/english.pickle')
word_tokenizer = nltk.tokenize.RegexpTokenizer(r'\w+')
##tt = ToktokTokenizer()#keeps the url as is...................................................
from nltk.tokenize import TweetTokenizer
tt = TweetTokenizer()
Porter_stemmer = PorterStemmer()
Lancaster_stemmer = LancasterStemmer()
WordNet_lemmatizer = WordNetLemmatizer()
#nltk.download('averaged_perceptron_tagger')
##Train_path="D:/Data/PAN2015-Profiling/pan15-author-profiling-training-dataset-english-2015-04-23"#path for train data
##Test_path="D:/Data/PAN2015-Profiling/pan-ap2015-test/en" #path for test data

##Train_path="D:/NLP/PAN2014/pan14-author-profiling-training-corpus-english-twitter-2014-04-16"#path for train data D:\Data\PAN2013-Profiling\Training\en  D:\NLP\PAN2014
##train_truth_path="D:/NLP/PAN2014/truth_train"
##
##test_truth_path="D:/NLP/PAN2014/pan14_test"
##Test_path="D:/NLP/PAN2014/pan14_test/en"
##Content = open('results_of_clssification_all.txt', 'w')

M_train_file= "/Users/catherine/Desktop/NLP/PAN Datasets/PAN_tokens/2014_blog_Tokens/PAN2014_blog_male_train.txt"
F_train_file= "/Users/catherine/Desktop/NLP/PAN Datasets/PAN_tokens/2014_blog_Tokens/PAN2014_blog_female_train.txt"

M_test_file= "/Users/catherine/Desktop/NLP/PAN Datasets/PAN_tokens/2014_blog_Tokens/PAN2014_blog_male_test.txt"
F_test_file= "/Users/catherine/Desktop/NLP/PAN Datasets/PAN_tokens/2014_blog_Tokens/PAN2014_blog_female_test.txt"

def all_txt(M_txt_file,  F_txt_file):
    with open(M_txt_file, mode="r", encoding="utf-8") as f:
            txt_list = [(line[:-1]).split("\t") for line in f]
            male_dict = dict()
            content=[]
            for line in txt_list:
                if line[1] in male_dict:
                # append the new number to the existing array at this slot
                    male_dict[line[1]].append(line[3])
                else:
                # create a new array in this slot
                    male_dict[line[1]] = [line[3]]
    male_content= [("M", (' '.join(a)).split()) for a in list(male_dict.values())]
  
    with open(F_txt_file, mode="r", encoding="utf-8") as f:
            txt_list = [(line[:-1]).split("\t") for line in f]
            female_dict = dict()
            content=[]
            for line in txt_list:
                if line[1] in female_dict:
                # append the new number to the existing array at this slot
                    female_dict[line[1]].append(line[3])
                else:
                # create a new array in this slot
                    female_dict[line[1]] = [line[3]]
    female_content= [("F", (' '.join(a)).split()) for a in list(female_dict.values())]
    all_txt_per_person = male_content + female_content
    return all_txt_per_person

##print( [((all_txt(M_txt_file,  F_txt_file))[i][0])  for i in range(len((all_txt(M_txt_file,  F_txt_file))))]) helps to determine the gender....

      
"""
create feature vector from the training set.....................................................................................
"""

def WordFeatures(word_list, all_training_text):

    fvs_words = np.array([[author.count(word) for word in word_list] for author in all_training_text]).astype(np.float64)

    # normalise by dividing each row by number of tokens for each author........
    fvs_words /= np.c_[np.array([len(author) for author in all_training_text])]

    return fvs_words



"""
From the saved csv file, recover the saved features to be used...............................................................
"""
import csv
word_list=[]
##txt_files =[ "/Users/catherine/Desktop/NLP/PAN Datasets/PAN_chosen_tokens/2014/output2014OR.1M.txt", "/Users/catherine/Desktop/NLP/PAN Datasets/PAN_chosen_tokens/2014/output2014OR.2F.txt"]
txt_files =[ "/Users/catherine/Desktop/NLP/PAN Datasets/PAN_tokens/2014_blog_Tokens/PAN2014_blog_female_chosen_set.csv", "/Users/catherine/Desktop/NLP/PAN Datasets/PAN_tokens/2014_blog_Tokens/PAN2014_blog_male_chosen_set.csv"]
print("txt_files of features used",txt_files)
##for txt_file in txt_files:
##    with open(txt_file, mode="r", encoding="utf-8") as f:
##        reader = csv.reader(f, delimiter=",")
##        next(reader) # skip header
##        word_list1 =  ([(r.split(":"))[1] for r in f])
##        print(txt_file, len(word_list1))
##        word_list = word_list + word_list1
##print("length of features used =", len(word_list))

for txt_file in txt_files:
    with open(txt_file, mode="r", encoding="utf-8") as f:
        reader = csv.reader(f, delimiter=",")
        next(reader) # skip header
        word_list1 =  [r[0] for r in reader]
        print(txt_file, len(word_list1))
        word_list = word_list + word_list1
print("length of features used =", len(word_list))


"""
Prepare the training and test sets to be parsed to the classifies............................................................
"""
A=(all_txt(M_train_file,  F_train_file))
all_training_text = ( [(A[i][1])  for i in range(len(A))])
print("len(all_training_text)",len(all_training_text))
X_train=WordFeatures(word_list, all_training_text)
y_train=np.array( [(A[i][0])  for i in range(len(A))])
print("len(X_train)=",len(X_train), "len(y_train)=", len(y_train))

B=(all_txt(M_test_file,  F_test_file))
all_test_text = ( [(B[i][1])  for i in range(len(B))])
print("len(all_test_text)",len(all_test_text))
X_test = WordFeatures(word_list, all_test_text)
y_test = np.array( [(B[i][0])  for i in range(len(B))])
print("len(X_test=",len(X_test), "len(y_test)=",len(y_test))


X_train = np.nan_to_num(X_train)
X_test = np.nan_to_num(X_test)

"""
Perform feature selection using chi2............................................................
"""
from sklearn.feature_selection import *
##
####def select_Bestfeatures(X, y):
#print("selector = SelectKBest(score_func=chi2, k=200)")
#selector = SelectKBest(score_func=chi2, k=200)
#fit = selector.fit(X_train,y_train)
#train_selected_features = fit.transform(X_train)
#test_selected_features = fit.transform(X_test)



"""
x_train and x_test based on the new features using chi2............................................................
"""
##X_train = select_Bestfeatures(X_train, y_train)
##X_test = select_Bestfeatures(X_test, y_test)


"""
Perform feature selection using f_classif............................................................
"""
from sklearn.feature_selection import *
##def f_classif_select_Bestfeatures(X, y):
##selector=SelectKBest(score_func=f_classif,k=500)
##fit = selector.fit(X_train,y_train)
##train_selected_features = fit.transform(X_train)
##test_selected_features = fit.transform(X_test)
##    selected_features = fit.transform(X)
##    return selected_features

"""
x_train and x_test based on the new features using f_classif............................................................
"""
##X_train = f_classif_select_Bestfeatures(X_train, y_train)
##X_test = f_classif_select_Bestfeatures(X_test, y_test)



"""
Perform feature selection using mutual_info_classif............................................................
"""
from sklearn.feature_selection import *
##def mutual_info_classif_select_Bestfeatures(X_tr, X_te, y):
print("selector=SelectKBest(score_func=mutual_info_classif,k=200)")
selector=SelectKBest(score_func=mutual_info_classif,k=200)
fit = selector.fit(X_train,y_train)
train_selected_features = fit.transform(X_train)
test_selected_features = fit.transform(X_test)
##    return train_selected_features, test_selected_features

"""
x_train and x_test based on the new features using f_classif............................................................
"""
X_train =train_selected_features 
X_test = test_selected_features


"""
Perform feature selection using pca............................................................
"""

##from sklearn.decomposition import PCA
##def PCA_select_Bestfeatures(X):
##    # feature extraction
##    pca = PCA(n_components=200)
##    fit = pca.fit(X, y)
##    Train_selected_features = fit.transform(X)
##    return selected_features

"""
x_train and x_test based on the new features using f_classif............................................................
"""
##X_train = PCA_select_Bestfeatures(X_train)
##X_test = PCA_select_Bestfeatures(X_test)

"""
Below are a list of Classification Algorithms with different distance measres ..............................................................................
"""
"""
K-Nearest Neighbours (average aa over all the nearest neighbour selectiopn)..............................................................................
"""

k=5

print(k)
"""
8. manhattan K-Nearest Neighbours..............................................................................
"""
from sklearn.neighbors import KNeighborsClassifier
knn = KNeighborsClassifier(n_neighbors = k, metric= "manhattan")
knn.fit(X_train, y_train)
y_pred = knn.predict(X_test)
print("manhattan K-Nearest Neighbours")
print(metrics.accuracy_score(y_test, y_pred))

    
"""
8. cosine K-Nearest Neighbours..............................................................................
"""
from sklearn.neighbors import KNeighborsClassifier
knn = KNeighborsClassifier(n_neighbors = k, metric= "cosine")
knn.fit(X_train, y_train)
y_pred = knn.predict(X_test)
print("cosine K-Nearest Neighbours")
print(metrics.accuracy_score(y_test, y_pred))


"""
8. euclidean K-Nearest Neighbours..............................................................................
"""
from sklearn.neighbors import KNeighborsClassifier
knn = KNeighborsClassifier(n_neighbors = k, metric= "euclidean")
knn.fit(X_train, y_train)
y_pred = knn.predict(X_test)
print("euclidean K-Nearest Neighbours")
print(metrics.accuracy_score(y_test, y_pred))

        
"""
9. scipy.spatial.distance K-Nearest Neighbours..............................................................................
"""
from scipy.spatial.distance import *
knn = KNeighborsClassifier(n_neighbors = k, metric= "braycurtis")
knn.fit(X_train, y_train)
y_pred = knn.predict(X_test)
print("braycurtis K-Nearest Neighbours")
print(metrics.accuracy_score(y_test, y_pred))
       
        
knn = KNeighborsClassifier(n_neighbors = k, metric= "canberra")
knn.fit(X_train, y_train)
y_pred = knn.predict(X_test)
print("canberra K-Nearest Neighbours")
print(metrics.accuracy_score(y_test, y_pred))
  
        
knn = KNeighborsClassifier(n_neighbors = k, metric= "jaccard")
knn.fit(X_train, y_train)
y_pred = knn.predict(X_test)
print("jaccard K-Nearest Neighbours")
print(metrics.accuracy_score(y_test, y_pred))
        
knn = KNeighborsClassifier(n_neighbors = k, metric= "yule")
knn.fit(X_train, y_train)
y_pred = knn.predict(X_test)
print("yule K-Nearest Neighbours")
print(metrics.accuracy_score(y_test, y_pred))
print("************************************ ****************************")



txt_files of features used ['/Users/catherine/Desktop/NLP/PAN Datasets/PAN_tokens/2014_blog_Tokens/PAN2014_blog_female_chosen_set.csv', '/Users/catherine/Desktop/NLP/PAN Datasets/PAN_tokens/2014_blog_Tokens/PAN2014_blog_male_chosen_set.csv']
/Users/catherine/Desktop/NLP/PAN Datasets/PAN_tokens/2014_blog_Tokens/PAN2014_blog_female_chosen_set.csv 1196
/Users/catherine/Desktop/NLP/PAN Datasets/PAN_tokens/2014_blog_Tokens/PAN2014_blog_male_chosen_set.csv 1636
length of features used = 2832
len(all_training_text) 147
len(X_train)= 147 len(y_train)= 147
len(all_test_text) 78
len(X_test= 78 len(y_test)= 78
selector=SelectKBest(score_func=mutual_info_classif,k=200)
5
manhattan K-Nearest Neighbours
0.5897435897435898
cosine K-Nearest Neighbours
0.6153846153846154
euclidean K-Nearest Neighbours
0.6025641025641025
braycurtis K-Nearest Neighbours
0.5897435897435898
canberra K-Nearest Neighbours
0.6153846153846154
jaccard K-Nearest Neighbours
0.5256410256410257
yule K-Nearest Neighbours
0.551282051

/anaconda3/lib/python3.7/site-packages/sklearn/utils/validation.py:475: DataConversionWarning: Data with input dtype float64 was converted to bool by check_pairwise_arrays.
  warnings.warn(msg, DataConversionWarning)


In [25]:
import sys
sys.path.append("/anaconda3/lib/python3.7/site-packages")
import numpy as np
import numpy
import nltk
import pickle
import pandas as pd
import glob
import os
import re
import operator
from collections import Counter
#from read_xml_files import *
##from s_stemmer import *
from nltk.stem.porter import PorterStemmer
from nltk.stem import LancasterStemmer, WordNetLemmatizer
from itertools import groupby
from nltk.collocations import *
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.cluster import KMeans
from scipy.cluster.vq import whiten
from nltk.tokenize.toktok import ToktokTokenizer
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2
import xml.etree.ElementTree as ET
import pandas as pd
#Import scikit-learn metrics module for accuracy calculation
from sklearn import metrics
##from pattern.text.en import singularize
##nltk.download('vader_lexicon')
##nltk.download('punkt')
#import the relevant modules from the NLTK library
from nltk.sentiment.vader import SentimentIntensityAnalyzer
sentence_tokenizer = nltk.data.load('tokenizers/punkt/english.pickle')
word_tokenizer = nltk.tokenize.RegexpTokenizer(r'\w+')
##tt = ToktokTokenizer()#keeps the url as is...................................................
from nltk.tokenize import TweetTokenizer
tt = TweetTokenizer()
Porter_stemmer = PorterStemmer()
Lancaster_stemmer = LancasterStemmer()
WordNet_lemmatizer = WordNetLemmatizer()
#nltk.download('averaged_perceptron_tagger')
##Train_path="D:/Data/PAN2015-Profiling/pan15-author-profiling-training-dataset-english-2015-04-23"#path for train data
##Test_path="D:/Data/PAN2015-Profiling/pan-ap2015-test/en" #path for test data

##Train_path="D:/NLP/PAN2014/pan14-author-profiling-training-corpus-english-twitter-2014-04-16"#path for train data D:\Data\PAN2013-Profiling\Training\en  D:\NLP\PAN2014
##train_truth_path="D:/NLP/PAN2014/truth_train"
##
##test_truth_path="D:/NLP/PAN2014/pan14_test"
##Test_path="D:/NLP/PAN2014/pan14_test/en"
##Content = open('results_of_clssification_all.txt', 'w')

M_train_file= "/Users/catherine/Desktop/NLP/PAN Datasets/PAN_tokens/2014_blog_Tokens/PAN2014_blog_male_train.txt"
F_train_file= "/Users/catherine/Desktop/NLP/PAN Datasets/PAN_tokens/2014_blog_Tokens/PAN2014_blog_female_train.txt"

M_test_file= "/Users/catherine/Desktop/NLP/PAN Datasets/PAN_tokens/2014_blog_Tokens/PAN2014_blog_male_test.txt"
F_test_file= "/Users/catherine/Desktop/NLP/PAN Datasets/PAN_tokens/2014_blog_Tokens/PAN2014_blog_female_test.txt"

def all_txt(M_txt_file,  F_txt_file):
    with open(M_txt_file, mode="r", encoding="utf-8") as f:
            txt_list = [(line[:-1]).split("\t") for line in f]
            male_dict = dict()
            content=[]
            for line in txt_list:
                if line[1] in male_dict:
                # append the new number to the existing array at this slot
                    male_dict[line[1]].append(line[3])
                else:
                # create a new array in this slot
                    male_dict[line[1]] = [line[3]]
    male_content= [("M", (' '.join(a)).split()) for a in list(male_dict.values())]
  
    with open(F_txt_file, mode="r", encoding="utf-8") as f:
            txt_list = [(line[:-1]).split("\t") for line in f]
            female_dict = dict()
            content=[]
            for line in txt_list:
                if line[1] in female_dict:
                # append the new number to the existing array at this slot
                    female_dict[line[1]].append(line[3])
                else:
                # create a new array in this slot
                    female_dict[line[1]] = [line[3]]
    female_content= [("F", (' '.join(a)).split()) for a in list(female_dict.values())]
    all_txt_per_person = male_content + female_content
    return all_txt_per_person

##print( [((all_txt(M_txt_file,  F_txt_file))[i][0])  for i in range(len((all_txt(M_txt_file,  F_txt_file))))]) helps to determine the gender....

      
"""
create feature vector from the training set.....................................................................................
"""

def WordFeatures(word_list, all_training_text):

    fvs_words = np.array([[author.count(word) for word in word_list] for author in all_training_text]).astype(np.float64)

    # normalise by dividing each row by number of tokens for each author........
    fvs_words /= np.c_[np.array([len(author) for author in all_training_text])]

    return fvs_words



"""
From the saved csv file, recover the saved features to be used...............................................................
"""
import csv
word_list=[]
##txt_files =[ "/Users/catherine/Desktop/NLP/PAN Datasets/PAN_chosen_tokens/2014/output2014OR.1M.txt", "/Users/catherine/Desktop/NLP/PAN Datasets/PAN_chosen_tokens/2014/output2014OR.2F.txt"]
txt_files =[ "/Users/catherine/Desktop/NLP/PAN Datasets/PAN_tokens/2014_blog_Tokens/PAN2014_blog_female_chosen_set.csv", "/Users/catherine/Desktop/NLP/PAN Datasets/PAN_tokens/2014_blog_Tokens/PAN2014_blog_male_chosen_set.csv"]
print("txt_files of features used",txt_files)
##for txt_file in txt_files:
##    with open(txt_file, mode="r", encoding="utf-8") as f:
##        reader = csv.reader(f, delimiter=",")
##        next(reader) # skip header
##        word_list1 =  ([(r.split(":"))[1] for r in f])
##        print(txt_file, len(word_list1))
##        word_list = word_list + word_list1
##print("length of features used =", len(word_list))

for txt_file in txt_files:
    with open(txt_file, mode="r", encoding="utf-8") as f:
        reader = csv.reader(f, delimiter=",")
        next(reader) # skip header
        word_list1 =  [r[0] for r in reader]
        print(txt_file, len(word_list1))
        word_list = word_list + word_list1
print("length of features used =", len(word_list))


"""
Prepare the training and test sets to be parsed to the classifies............................................................
"""
A=(all_txt(M_train_file,  F_train_file))
all_training_text = ( [(A[i][1])  for i in range(len(A))])
print("len(all_training_text)",len(all_training_text))
X_train=WordFeatures(word_list, all_training_text)
y_train=np.array( [(A[i][0])  for i in range(len(A))])
print("len(X_train)=",len(X_train), "len(y_train)=", len(y_train))

B=(all_txt(M_test_file,  F_test_file))
all_test_text = ( [(B[i][1])  for i in range(len(B))])
print("len(all_test_text)",len(all_test_text))
X_test = WordFeatures(word_list, all_test_text)
y_test = np.array( [(B[i][0])  for i in range(len(B))])
print("len(X_test=",len(X_test), "len(y_test)=",len(y_test))


X_train = np.nan_to_num(X_train)
X_test = np.nan_to_num(X_test)

"""
Perform feature selection using chi2............................................................
"""
from sklearn.feature_selection import *
##
####def select_Bestfeatures(X, y):
#print("selector = SelectKBest(score_func=chi2, k=200)")
#selector = SelectKBest(score_func=chi2, k=200)
#fit = selector.fit(X_train,y_train)
#train_selected_features = fit.transform(X_train)
#test_selected_features = fit.transform(X_test)



"""
x_train and x_test based on the new features using chi2............................................................
"""
##X_train = select_Bestfeatures(X_train, y_train)
##X_test = select_Bestfeatures(X_test, y_test)


"""
Perform feature selection using f_classif............................................................
"""
from sklearn.feature_selection import *
##def f_classif_select_Bestfeatures(X, y):
##selector=SelectKBest(score_func=f_classif,k=500)
##fit = selector.fit(X_train,y_train)
##train_selected_features = fit.transform(X_train)
##test_selected_features = fit.transform(X_test)
##    selected_features = fit.transform(X)
##    return selected_features

"""
x_train and x_test based on the new features using f_classif............................................................
"""
##X_train = f_classif_select_Bestfeatures(X_train, y_train)
##X_test = f_classif_select_Bestfeatures(X_test, y_test)



"""
Perform feature selection using mutual_info_classif............................................................
"""
from sklearn.feature_selection import *
##def mutual_info_classif_select_Bestfeatures(X_tr, X_te, y):
print("selector=SelectKBest(score_func=mutual_info_classif,k=300)")
selector=SelectKBest(score_func=mutual_info_classif,k=300)
fit = selector.fit(X_train,y_train)
train_selected_features = fit.transform(X_train)
test_selected_features = fit.transform(X_test)
##    return train_selected_features, test_selected_features

"""
x_train and x_test based on the new features using f_classif............................................................
"""
X_train =train_selected_features 
X_test = test_selected_features


"""
Perform feature selection using pca............................................................
"""

##from sklearn.decomposition import PCA
##def PCA_select_Bestfeatures(X):
##    # feature extraction
##    pca = PCA(n_components=200)
##    fit = pca.fit(X, y)
##    Train_selected_features = fit.transform(X)
##    return selected_features

"""
x_train and x_test based on the new features using f_classif............................................................
"""
##X_train = PCA_select_Bestfeatures(X_train)
##X_test = PCA_select_Bestfeatures(X_test)

"""
Below are a list of Classification Algorithms with different distance measres ..............................................................................
"""
"""
K-Nearest Neighbours (average aa over all the nearest neighbour selectiopn)..............................................................................
"""

k=5

print(k)
"""
8. manhattan K-Nearest Neighbours..............................................................................
"""
from sklearn.neighbors import KNeighborsClassifier
knn = KNeighborsClassifier(n_neighbors = k, metric= "manhattan")
knn.fit(X_train, y_train)
y_pred = knn.predict(X_test)
print("manhattan K-Nearest Neighbours")
print(metrics.accuracy_score(y_test, y_pred))

    
"""
8. cosine K-Nearest Neighbours..............................................................................
"""
from sklearn.neighbors import KNeighborsClassifier
knn = KNeighborsClassifier(n_neighbors = k, metric= "cosine")
knn.fit(X_train, y_train)
y_pred = knn.predict(X_test)
print("cosine K-Nearest Neighbours")
print(metrics.accuracy_score(y_test, y_pred))


"""
8. euclidean K-Nearest Neighbours..............................................................................
"""
from sklearn.neighbors import KNeighborsClassifier
knn = KNeighborsClassifier(n_neighbors = k, metric= "euclidean")
knn.fit(X_train, y_train)
y_pred = knn.predict(X_test)
print("euclidean K-Nearest Neighbours")
print(metrics.accuracy_score(y_test, y_pred))

        
"""
9. scipy.spatial.distance K-Nearest Neighbours..............................................................................
"""
from scipy.spatial.distance import *
knn = KNeighborsClassifier(n_neighbors = k, metric= "braycurtis")
knn.fit(X_train, y_train)
y_pred = knn.predict(X_test)
print("braycurtis K-Nearest Neighbours")
print(metrics.accuracy_score(y_test, y_pred))
       
        
knn = KNeighborsClassifier(n_neighbors = k, metric= "canberra")
knn.fit(X_train, y_train)
y_pred = knn.predict(X_test)
print("canberra K-Nearest Neighbours")
print(metrics.accuracy_score(y_test, y_pred))
  
        
knn = KNeighborsClassifier(n_neighbors = k, metric= "jaccard")
knn.fit(X_train, y_train)
y_pred = knn.predict(X_test)
print("jaccard K-Nearest Neighbours")
print(metrics.accuracy_score(y_test, y_pred))
        
knn = KNeighborsClassifier(n_neighbors = k, metric= "yule")
knn.fit(X_train, y_train)
y_pred = knn.predict(X_test)
print("yule K-Nearest Neighbours")
print(metrics.accuracy_score(y_test, y_pred))
print("************************************ ****************************")




txt_files of features used ['/Users/catherine/Desktop/NLP/PAN Datasets/PAN_tokens/2014_blog_Tokens/PAN2014_blog_female_chosen_set.csv', '/Users/catherine/Desktop/NLP/PAN Datasets/PAN_tokens/2014_blog_Tokens/PAN2014_blog_male_chosen_set.csv']
/Users/catherine/Desktop/NLP/PAN Datasets/PAN_tokens/2014_blog_Tokens/PAN2014_blog_female_chosen_set.csv 1196
/Users/catherine/Desktop/NLP/PAN Datasets/PAN_tokens/2014_blog_Tokens/PAN2014_blog_male_chosen_set.csv 1636
length of features used = 2832
len(all_training_text) 147
len(X_train)= 147 len(y_train)= 147
len(all_test_text) 78
len(X_test= 78 len(y_test)= 78
selector=SelectKBest(score_func=mutual_info_classif,k=300)
5
manhattan K-Nearest Neighbours
0.5256410256410257
cosine K-Nearest Neighbours
0.5641025641025641
euclidean K-Nearest Neighbours
0.5769230769230769
braycurtis K-Nearest Neighbours
0.5384615384615384
canberra K-Nearest Neighbours
0.47435897435897434
jaccard K-Nearest Neighbours
0.5897435897435898
yule K-Nearest Neighbours
0.51282051

/anaconda3/lib/python3.7/site-packages/sklearn/utils/validation.py:475: DataConversionWarning: Data with input dtype float64 was converted to bool by check_pairwise_arrays.
  warnings.warn(msg, DataConversionWarning)


In [26]:
import sys
sys.path.append("/anaconda3/lib/python3.7/site-packages")
import numpy as np
import numpy
import nltk
import pickle
import pandas as pd
import glob
import os
import re
import operator
from collections import Counter
#from read_xml_files import *
##from s_stemmer import *
from nltk.stem.porter import PorterStemmer
from nltk.stem import LancasterStemmer, WordNetLemmatizer
from itertools import groupby
from nltk.collocations import *
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.cluster import KMeans
from scipy.cluster.vq import whiten
from nltk.tokenize.toktok import ToktokTokenizer
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2
import xml.etree.ElementTree as ET
import pandas as pd
#Import scikit-learn metrics module for accuracy calculation
from sklearn import metrics
##from pattern.text.en import singularize
##nltk.download('vader_lexicon')
##nltk.download('punkt')
#import the relevant modules from the NLTK library
from nltk.sentiment.vader import SentimentIntensityAnalyzer
sentence_tokenizer = nltk.data.load('tokenizers/punkt/english.pickle')
word_tokenizer = nltk.tokenize.RegexpTokenizer(r'\w+')
##tt = ToktokTokenizer()#keeps the url as is...................................................
from nltk.tokenize import TweetTokenizer
tt = TweetTokenizer()
Porter_stemmer = PorterStemmer()
Lancaster_stemmer = LancasterStemmer()
WordNet_lemmatizer = WordNetLemmatizer()
#nltk.download('averaged_perceptron_tagger')
##Train_path="D:/Data/PAN2015-Profiling/pan15-author-profiling-training-dataset-english-2015-04-23"#path for train data
##Test_path="D:/Data/PAN2015-Profiling/pan-ap2015-test/en" #path for test data

##Train_path="D:/NLP/PAN2014/pan14-author-profiling-training-corpus-english-twitter-2014-04-16"#path for train data D:\Data\PAN2013-Profiling\Training\en  D:\NLP\PAN2014
##train_truth_path="D:/NLP/PAN2014/truth_train"
##
##test_truth_path="D:/NLP/PAN2014/pan14_test"
##Test_path="D:/NLP/PAN2014/pan14_test/en"
##Content = open('results_of_clssification_all.txt', 'w')

M_train_file= "/Users/catherine/Desktop/NLP/PAN Datasets/PAN_tokens/2014_blog_Tokens/PAN2014_blog_male_train.txt"
F_train_file= "/Users/catherine/Desktop/NLP/PAN Datasets/PAN_tokens/2014_blog_Tokens/PAN2014_blog_female_train.txt"

M_test_file= "/Users/catherine/Desktop/NLP/PAN Datasets/PAN_tokens/2014_blog_Tokens/PAN2014_blog_male_test.txt"
F_test_file= "/Users/catherine/Desktop/NLP/PAN Datasets/PAN_tokens/2014_blog_Tokens/PAN2014_blog_female_test.txt"

def all_txt(M_txt_file,  F_txt_file):
    with open(M_txt_file, mode="r", encoding="utf-8") as f:
            txt_list = [(line[:-1]).split("\t") for line in f]
            male_dict = dict()
            content=[]
            for line in txt_list:
                if line[1] in male_dict:
                # append the new number to the existing array at this slot
                    male_dict[line[1]].append(line[3])
                else:
                # create a new array in this slot
                    male_dict[line[1]] = [line[3]]
    male_content= [("M", (' '.join(a)).split()) for a in list(male_dict.values())]
  
    with open(F_txt_file, mode="r", encoding="utf-8") as f:
            txt_list = [(line[:-1]).split("\t") for line in f]
            female_dict = dict()
            content=[]
            for line in txt_list:
                if line[1] in female_dict:
                # append the new number to the existing array at this slot
                    female_dict[line[1]].append(line[3])
                else:
                # create a new array in this slot
                    female_dict[line[1]] = [line[3]]
    female_content= [("F", (' '.join(a)).split()) for a in list(female_dict.values())]
    all_txt_per_person = male_content + female_content
    return all_txt_per_person

##print( [((all_txt(M_txt_file,  F_txt_file))[i][0])  for i in range(len((all_txt(M_txt_file,  F_txt_file))))]) helps to determine the gender....

      
"""
create feature vector from the training set.....................................................................................
"""

def WordFeatures(word_list, all_training_text):

    fvs_words = np.array([[author.count(word) for word in word_list] for author in all_training_text]).astype(np.float64)

    # normalise by dividing each row by number of tokens for each author........
    fvs_words /= np.c_[np.array([len(author) for author in all_training_text])]

    return fvs_words



"""
From the saved csv file, recover the saved features to be used...............................................................
"""
import csv
word_list=[]
##txt_files =[ "/Users/catherine/Desktop/NLP/PAN Datasets/PAN_chosen_tokens/2014/output2014OR.1M.txt", "/Users/catherine/Desktop/NLP/PAN Datasets/PAN_chosen_tokens/2014/output2014OR.2F.txt"]
txt_files =[ "/Users/catherine/Desktop/NLP/PAN Datasets/PAN_tokens/2014_blog_Tokens/PAN2014_blog_female_chosen_set.csv", "/Users/catherine/Desktop/NLP/PAN Datasets/PAN_tokens/2014_blog_Tokens/PAN2014_blog_male_chosen_set.csv"]
print("txt_files of features used",txt_files)
##for txt_file in txt_files:
##    with open(txt_file, mode="r", encoding="utf-8") as f:
##        reader = csv.reader(f, delimiter=",")
##        next(reader) # skip header
##        word_list1 =  ([(r.split(":"))[1] for r in f])
##        print(txt_file, len(word_list1))
##        word_list = word_list + word_list1
##print("length of features used =", len(word_list))

for txt_file in txt_files:
    with open(txt_file, mode="r", encoding="utf-8") as f:
        reader = csv.reader(f, delimiter=",")
        next(reader) # skip header
        word_list1 =  [r[0] for r in reader]
        print(txt_file, len(word_list1))
        word_list = word_list + word_list1
print("length of features used =", len(word_list))


"""
Prepare the training and test sets to be parsed to the classifies............................................................
"""
A=(all_txt(M_train_file,  F_train_file))
all_training_text = ( [(A[i][1])  for i in range(len(A))])
print("len(all_training_text)",len(all_training_text))
X_train=WordFeatures(word_list, all_training_text)
y_train=np.array( [(A[i][0])  for i in range(len(A))])
print("len(X_train)=",len(X_train), "len(y_train)=", len(y_train))

B=(all_txt(M_test_file,  F_test_file))
all_test_text = ( [(B[i][1])  for i in range(len(B))])
print("len(all_test_text)",len(all_test_text))
X_test = WordFeatures(word_list, all_test_text)
y_test = np.array( [(B[i][0])  for i in range(len(B))])
print("len(X_test=",len(X_test), "len(y_test)=",len(y_test))


X_train = np.nan_to_num(X_train)
X_test = np.nan_to_num(X_test)

"""
Perform feature selection using chi2............................................................
"""
from sklearn.feature_selection import *
##
####def select_Bestfeatures(X, y):
#print("selector = SelectKBest(score_func=chi2, k=200)")
#selector = SelectKBest(score_func=chi2, k=200)
#fit = selector.fit(X_train,y_train)
#train_selected_features = fit.transform(X_train)
#test_selected_features = fit.transform(X_test)



"""
x_train and x_test based on the new features using chi2............................................................
"""
##X_train = select_Bestfeatures(X_train, y_train)
##X_test = select_Bestfeatures(X_test, y_test)


"""
Perform feature selection using f_classif............................................................
"""
from sklearn.feature_selection import *
##def f_classif_select_Bestfeatures(X, y):
##selector=SelectKBest(score_func=f_classif,k=500)
##fit = selector.fit(X_train,y_train)
##train_selected_features = fit.transform(X_train)
##test_selected_features = fit.transform(X_test)
##    selected_features = fit.transform(X)
##    return selected_features

"""
x_train and x_test based on the new features using f_classif............................................................
"""
##X_train = f_classif_select_Bestfeatures(X_train, y_train)
##X_test = f_classif_select_Bestfeatures(X_test, y_test)



"""
Perform feature selection using mutual_info_classif............................................................
"""
from sklearn.feature_selection import *
##def mutual_info_classif_select_Bestfeatures(X_tr, X_te, y):
print("selector=SelectKBest(score_func=mutual_info_classif,k=500)")
selector=SelectKBest(score_func=mutual_info_classif,k=500)
fit = selector.fit(X_train,y_train)
train_selected_features = fit.transform(X_train)
test_selected_features = fit.transform(X_test)
##    return train_selected_features, test_selected_features

"""
x_train and x_test based on the new features using f_classif............................................................
"""
X_train =train_selected_features 
X_test = test_selected_features


"""
Perform feature selection using pca............................................................
"""

##from sklearn.decomposition import PCA
##def PCA_select_Bestfeatures(X):
##    # feature extraction
##    pca = PCA(n_components=200)
##    fit = pca.fit(X, y)
##    Train_selected_features = fit.transform(X)
##    return selected_features

"""
x_train and x_test based on the new features using f_classif............................................................
"""
##X_train = PCA_select_Bestfeatures(X_train)
##X_test = PCA_select_Bestfeatures(X_test)

"""
Below are a list of Classification Algorithms with different distance measres ..............................................................................
"""
"""
K-Nearest Neighbours (average aa over all the nearest neighbour selectiopn)..............................................................................
"""

k=5

print(k)
"""
8. manhattan K-Nearest Neighbours..............................................................................
"""
from sklearn.neighbors import KNeighborsClassifier
knn = KNeighborsClassifier(n_neighbors = k, metric= "manhattan")
knn.fit(X_train, y_train)
y_pred = knn.predict(X_test)
print("manhattan K-Nearest Neighbours")
print(metrics.accuracy_score(y_test, y_pred))

    
"""
8. cosine K-Nearest Neighbours..............................................................................
"""
from sklearn.neighbors import KNeighborsClassifier
knn = KNeighborsClassifier(n_neighbors = k, metric= "cosine")
knn.fit(X_train, y_train)
y_pred = knn.predict(X_test)
print("cosine K-Nearest Neighbours")
print(metrics.accuracy_score(y_test, y_pred))


"""
8. euclidean K-Nearest Neighbours..............................................................................
"""
from sklearn.neighbors import KNeighborsClassifier
knn = KNeighborsClassifier(n_neighbors = k, metric= "euclidean")
knn.fit(X_train, y_train)
y_pred = knn.predict(X_test)
print("euclidean K-Nearest Neighbours")
print(metrics.accuracy_score(y_test, y_pred))

        
"""
9. scipy.spatial.distance K-Nearest Neighbours..............................................................................
"""
from scipy.spatial.distance import *
knn = KNeighborsClassifier(n_neighbors = k, metric= "braycurtis")
knn.fit(X_train, y_train)
y_pred = knn.predict(X_test)
print("braycurtis K-Nearest Neighbours")
print(metrics.accuracy_score(y_test, y_pred))
       
        
knn = KNeighborsClassifier(n_neighbors = k, metric= "canberra")
knn.fit(X_train, y_train)
y_pred = knn.predict(X_test)
print("canberra K-Nearest Neighbours")
print(metrics.accuracy_score(y_test, y_pred))
  
        
knn = KNeighborsClassifier(n_neighbors = k, metric= "jaccard")
knn.fit(X_train, y_train)
y_pred = knn.predict(X_test)
print("jaccard K-Nearest Neighbours")
print(metrics.accuracy_score(y_test, y_pred))
        
knn = KNeighborsClassifier(n_neighbors = k, metric= "yule")
knn.fit(X_train, y_train)
y_pred = knn.predict(X_test)
print("yule K-Nearest Neighbours")
print(metrics.accuracy_score(y_test, y_pred))
print("************************************ ****************************")




txt_files of features used ['/Users/catherine/Desktop/NLP/PAN Datasets/PAN_tokens/2014_blog_Tokens/PAN2014_blog_female_chosen_set.csv', '/Users/catherine/Desktop/NLP/PAN Datasets/PAN_tokens/2014_blog_Tokens/PAN2014_blog_male_chosen_set.csv']
/Users/catherine/Desktop/NLP/PAN Datasets/PAN_tokens/2014_blog_Tokens/PAN2014_blog_female_chosen_set.csv 1196
/Users/catherine/Desktop/NLP/PAN Datasets/PAN_tokens/2014_blog_Tokens/PAN2014_blog_male_chosen_set.csv 1636
length of features used = 2832
len(all_training_text) 147
len(X_train)= 147 len(y_train)= 147
len(all_test_text) 78
len(X_test= 78 len(y_test)= 78
selector=SelectKBest(score_func=mutual_info_classif,k=500)
5
manhattan K-Nearest Neighbours
0.6666666666666666
cosine K-Nearest Neighbours
0.6025641025641025
euclidean K-Nearest Neighbours
0.6282051282051282
braycurtis K-Nearest Neighbours
0.5769230769230769
canberra K-Nearest Neighbours
0.5641025641025641
jaccard K-Nearest Neighbours
0.5128205128205128
yule K-Nearest Neighbours
0.525641025

/anaconda3/lib/python3.7/site-packages/sklearn/utils/validation.py:475: DataConversionWarning: Data with input dtype float64 was converted to bool by check_pairwise_arrays.
  warnings.warn(msg, DataConversionWarning)


In [27]:
import sys
sys.path.append("/anaconda3/lib/python3.7/site-packages")
import numpy as np
import numpy
import nltk
import pickle
import pandas as pd
import glob
import os
import re
import operator
from collections import Counter
#from read_xml_files import *
##from s_stemmer import *
from nltk.stem.porter import PorterStemmer
from nltk.stem import LancasterStemmer, WordNetLemmatizer
from itertools import groupby
from nltk.collocations import *
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.cluster import KMeans
from scipy.cluster.vq import whiten
from nltk.tokenize.toktok import ToktokTokenizer
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2
import xml.etree.ElementTree as ET
import pandas as pd
#Import scikit-learn metrics module for accuracy calculation
from sklearn import metrics
##from pattern.text.en import singularize
##nltk.download('vader_lexicon')
##nltk.download('punkt')
#import the relevant modules from the NLTK library
from nltk.sentiment.vader import SentimentIntensityAnalyzer
sentence_tokenizer = nltk.data.load('tokenizers/punkt/english.pickle')
word_tokenizer = nltk.tokenize.RegexpTokenizer(r'\w+')
##tt = ToktokTokenizer()#keeps the url as is...................................................
from nltk.tokenize import TweetTokenizer
tt = TweetTokenizer()
Porter_stemmer = PorterStemmer()
Lancaster_stemmer = LancasterStemmer()
WordNet_lemmatizer = WordNetLemmatizer()
#nltk.download('averaged_perceptron_tagger')
##Train_path="D:/Data/PAN2015-Profiling/pan15-author-profiling-training-dataset-english-2015-04-23"#path for train data
##Test_path="D:/Data/PAN2015-Profiling/pan-ap2015-test/en" #path for test data

##Train_path="D:/NLP/PAN2014/pan14-author-profiling-training-corpus-english-twitter-2014-04-16"#path for train data D:\Data\PAN2013-Profiling\Training\en  D:\NLP\PAN2014
##train_truth_path="D:/NLP/PAN2014/truth_train"
##
##test_truth_path="D:/NLP/PAN2014/pan14_test"
##Test_path="D:/NLP/PAN2014/pan14_test/en"
##Content = open('results_of_clssification_all.txt', 'w')

M_train_file= "/Users/catherine/Desktop/NLP/PAN Datasets/PAN2014blog/2014.1-blog-txtfiles/PAN2014_blog_male_train.1.txt"
F_train_file= "/Users/catherine/Desktop/NLP/PAN Datasets/PAN2014blog/2014.1-blog-txtfiles/PAN2014_blog_female_train.1.txt"

M_test_file= "/Users/catherine/Desktop/NLP/PAN Datasets/PAN2014blog/2014.1-blog-txtfiles/PAN2014_blog_male_test.1.txt"
F_test_file= "/Users/catherine/Desktop/NLP/PAN Datasets/PAN2014blog/2014.1-blog-txtfiles/PAN2014_blog_female_test.1.txt"

def all_txt(M_txt_file,  F_txt_file):
    with open(M_txt_file, mode="r", encoding="utf-8") as f:
            txt_list = [(line[:-1]).split("\t") for line in f]
            male_dict = dict()
            content=[]
            for line in txt_list:
                if line[1] in male_dict:
                # append the new number to the existing array at this slot
                    male_dict[line[1]].append(line[3])
                else:
                # create a new array in this slot
                    male_dict[line[1]] = [line[3]]
    male_content= [("M", (' '.join(a)).split()) for a in list(male_dict.values())]
  
    with open(F_txt_file, mode="r", encoding="utf-8") as f:
            txt_list = [(line[:-1]).split("\t") for line in f]
            female_dict = dict()
            content=[]
            for line in txt_list:
                if line[1] in female_dict:
                # append the new number to the existing array at this slot
                    female_dict[line[1]].append(line[3])
                else:
                # create a new array in this slot
                    female_dict[line[1]] = [line[3]]
    female_content= [("F", (' '.join(a)).split()) for a in list(female_dict.values())]
    all_txt_per_person = male_content + female_content
    return all_txt_per_person

##print( [((all_txt(M_txt_file,  F_txt_file))[i][0])  for i in range(len((all_txt(M_txt_file,  F_txt_file))))]) helps to determine the gender....

      
"""
create feature vector from the training set.....................................................................................
"""

def WordFeatures(word_list, all_training_text):

    fvs_words = np.array([[author.count(word) for word in word_list] for author in all_training_text]).astype(np.float64)

    # normalise by dividing each row by number of tokens for each author........
    fvs_words /= np.c_[np.array([len(author) for author in all_training_text])]

    return fvs_words



"""
From the saved csv file, recover the saved features to be used...............................................................
"""
import csv
word_list=[]
##txt_files =[ "/Users/catherine/Desktop/NLP/PAN Datasets/PAN_chosen_tokens/2014/output2014OR.1M.txt", "/Users/catherine/Desktop/NLP/PAN Datasets/PAN_chosen_tokens/2014/output2014OR.2F.txt"]
txt_files =[ "/Users/catherine/Desktop/NLP/PAN Datasets/PAN2014blog/2014.1-blog-txtfiles/PAN2014_blog_female_chosen_set.1.csv", "/Users/catherine/Desktop/NLP/PAN Datasets/PAN2014blog/2014.1-blog-txtfiles/PAN2014_blog_male_chosen_set.1.csv"]
print("txt_files of features used",txt_files)
##for txt_file in txt_files:
##    with open(txt_file, mode="r", encoding="utf-8") as f:
##        reader = csv.reader(f, delimiter=",")
##        next(reader) # skip header
##        word_list1 =  ([(r.split(":"))[1] for r in f])
##        print(txt_file, len(word_list1))
##        word_list = word_list + word_list1
##print("length of features used =", len(word_list))

for txt_file in txt_files:
    with open(txt_file, mode="r", encoding="utf-8") as f:
        reader = csv.reader(f, delimiter=",")
        next(reader) # skip header
        word_list1 =  [r[0] for r in reader]
        print(txt_file, len(word_list1))
        word_list = word_list + word_list1
print("length of features used =", len(word_list))


"""
Prepare the training and test sets to be parsed to the classifies............................................................
"""
A=(all_txt(M_train_file,  F_train_file))
all_training_text = ( [(A[i][1])  for i in range(len(A))])
print("len(all_training_text)",len(all_training_text))
X_train=WordFeatures(word_list, all_training_text)
y_train=np.array( [(A[i][0])  for i in range(len(A))])
print("len(X_train)=",len(X_train), "len(y_train)=", len(y_train))

B=(all_txt(M_test_file,  F_test_file))
all_test_text = ( [(B[i][1])  for i in range(len(B))])
print("len(all_test_text)",len(all_test_text))
X_test = WordFeatures(word_list, all_test_text)
y_test = np.array( [(B[i][0])  for i in range(len(B))])
print("len(X_test=",len(X_test), "len(y_test)=",len(y_test))


X_train = np.nan_to_num(X_train)
X_test = np.nan_to_num(X_test)

"""
Perform feature selection using chi2............................................................
"""
from sklearn.feature_selection import *
##
####def select_Bestfeatures(X, y):
##print("selector = SelectKBest(score_func=chi2, k=1000)")
##selector = SelectKBest(score_func=chi2, k=1000)
##fit = selector.fit(X_train,y_train)
##train_selected_features = fit.transform(X_train)
##test_selected_features = fit.transform(X_test)



"""
x_train and x_test based on the new features using chi2............................................................
"""
##X_train = select_Bestfeatures(X_train, y_train)
##X_test = select_Bestfeatures(X_test, y_test)


"""
Perform feature selection using f_classif............................................................
"""
from sklearn.feature_selection import *
##def f_classif_select_Bestfeatures(X, y):
##selector=SelectKBest(score_func=f_classif,k=500)
##fit = selector.fit(X_train,y_train)
##train_selected_features = fit.transform(X_train)
##test_selected_features = fit.transform(X_test)
##    selected_features = fit.transform(X)
##    return selected_features

"""
x_train and x_test based on the new features using f_classif............................................................
"""
##X_train = f_classif_select_Bestfeatures(X_train, y_train)
##X_test = f_classif_select_Bestfeatures(X_test, y_test)



"""
Perform feature selection using mutual_info_classif............................................................
"""
from sklearn.feature_selection import *
##def mutual_info_classif_select_Bestfeatures(X_tr, X_te, y):
##selector=SelectKBest(score_func=mutual_info_classif,k=100)
##fit = selector.fit(X_train,y_train)
##train_selected_features = fit.transform(X_train)
##test_selected_features = fit.transform(X_test)
##    return train_selected_features, test_selected_features

"""
x_train and x_test based on the new features using f_classif............................................................
"""
##X_train =train_selected_features 
##X_test = test_selected_features


"""
Perform feature selection using pca............................................................
"""

##from sklearn.decomposition import PCA
##def PCA_select_Bestfeatures(X):
##    # feature extraction
##    pca = PCA(n_components=200)
##    fit = pca.fit(X, y)
##    Train_selected_features = fit.transform(X)
##    return selected_features

"""
x_train and x_test based on the new features using f_classif............................................................
"""
##X_train = PCA_select_Bestfeatures(X_train)
##X_test = PCA_select_Bestfeatures(X_test)

"""
Below are a list of Classification Algorithms with different distance measres ..............................................................................
"""
"""
K-Nearest Neighbours (average aa over all the nearest neighbour selectiopn)..............................................................................
"""

k=5

print(k)
"""
8. manhattan K-Nearest Neighbours..............................................................................
"""
from sklearn.neighbors import KNeighborsClassifier
knn = KNeighborsClassifier(n_neighbors = k, metric= "manhattan")
knn.fit(X_train, y_train)
y_pred = knn.predict(X_test)
print("manhattan K-Nearest Neighbours")
print(metrics.accuracy_score(y_test, y_pred))

    
"""
8. cosine K-Nearest Neighbours..............................................................................
"""
from sklearn.neighbors import KNeighborsClassifier
knn = KNeighborsClassifier(n_neighbors = k, metric= "cosine")
knn.fit(X_train, y_train)
y_pred = knn.predict(X_test)
print("cosine K-Nearest Neighbours")
print(metrics.accuracy_score(y_test, y_pred))


"""
8. euclidean K-Nearest Neighbours..............................................................................
"""
from sklearn.neighbors import KNeighborsClassifier
knn = KNeighborsClassifier(n_neighbors = k, metric= "euclidean")
knn.fit(X_train, y_train)
y_pred = knn.predict(X_test)
print("euclidean K-Nearest Neighbours")
print(metrics.accuracy_score(y_test, y_pred))

        
"""
9. scipy.spatial.distance K-Nearest Neighbours..............................................................................
"""
from scipy.spatial.distance import *
knn = KNeighborsClassifier(n_neighbors = k, metric= "braycurtis")
knn.fit(X_train, y_train)
y_pred = knn.predict(X_test)
print("braycurtis K-Nearest Neighbours")
print(metrics.accuracy_score(y_test, y_pred))
       
        
knn = KNeighborsClassifier(n_neighbors = k, metric= "canberra")
knn.fit(X_train, y_train)
y_pred = knn.predict(X_test)
print("canberra K-Nearest Neighbours")
print(metrics.accuracy_score(y_test, y_pred))
  
        
knn = KNeighborsClassifier(n_neighbors = k, metric= "jaccard")
knn.fit(X_train, y_train)
y_pred = knn.predict(X_test)
print("jaccard K-Nearest Neighbours")
print(metrics.accuracy_score(y_test, y_pred))
        
knn = KNeighborsClassifier(n_neighbors = k, metric= "yule")
knn.fit(X_train, y_train)
y_pred = knn.predict(X_test)
print("yule K-Nearest Neighbours")
print(metrics.accuracy_score(y_test, y_pred))
print("************************************ ****************************")


txt_files of features used ['/Users/catherine/Desktop/NLP/PAN Datasets/PAN2014blog/2014.1-blog-txtfiles/PAN2014_blog_female_chosen_set.1.csv', '/Users/catherine/Desktop/NLP/PAN Datasets/PAN2014blog/2014.1-blog-txtfiles/PAN2014_blog_male_chosen_set.1.csv']
/Users/catherine/Desktop/NLP/PAN Datasets/PAN2014blog/2014.1-blog-txtfiles/PAN2014_blog_female_chosen_set.1.csv 1526
/Users/catherine/Desktop/NLP/PAN Datasets/PAN2014blog/2014.1-blog-txtfiles/PAN2014_blog_male_chosen_set.1.csv 2063
length of features used = 3589
len(all_training_text) 147
len(X_train)= 147 len(y_train)= 147
len(all_test_text) 78
len(X_test= 78 len(y_test)= 78
5
manhattan K-Nearest Neighbours
0.6794871794871795
cosine K-Nearest Neighbours
0.6538461538461539
euclidean K-Nearest Neighbours
0.6410256410256411
braycurtis K-Nearest Neighbours
0.6025641025641025
canberra K-Nearest Neighbours
0.5
jaccard K-Nearest Neighbours
0.5769230769230769
yule K-Nearest Neighbours
0.5512820512820513
************************************ *

/anaconda3/lib/python3.7/site-packages/sklearn/utils/validation.py:475: DataConversionWarning: Data with input dtype float64 was converted to bool by check_pairwise_arrays.
  warnings.warn(msg, DataConversionWarning)


In [28]:
import sys
sys.path.append("/anaconda3/lib/python3.7/site-packages")
import numpy as np
import numpy
import nltk
import pickle
import pandas as pd
import glob
import os
import re
import operator
from collections import Counter
#from read_xml_files import *
##from s_stemmer import *
from nltk.stem.porter import PorterStemmer
from nltk.stem import LancasterStemmer, WordNetLemmatizer
from itertools import groupby
from nltk.collocations import *
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.cluster import KMeans
from scipy.cluster.vq import whiten
from nltk.tokenize.toktok import ToktokTokenizer
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2
import xml.etree.ElementTree as ET
import pandas as pd
#Import scikit-learn metrics module for accuracy calculation
from sklearn import metrics
##from pattern.text.en import singularize
##nltk.download('vader_lexicon')
##nltk.download('punkt')
#import the relevant modules from the NLTK library
from nltk.sentiment.vader import SentimentIntensityAnalyzer
sentence_tokenizer = nltk.data.load('tokenizers/punkt/english.pickle')
word_tokenizer = nltk.tokenize.RegexpTokenizer(r'\w+')
##tt = ToktokTokenizer()#keeps the url as is...................................................
from nltk.tokenize import TweetTokenizer
tt = TweetTokenizer()
Porter_stemmer = PorterStemmer()
Lancaster_stemmer = LancasterStemmer()
WordNet_lemmatizer = WordNetLemmatizer()
#nltk.download('averaged_perceptron_tagger')
##Train_path="D:/Data/PAN2015-Profiling/pan15-author-profiling-training-dataset-english-2015-04-23"#path for train data
##Test_path="D:/Data/PAN2015-Profiling/pan-ap2015-test/en" #path for test data

##Train_path="D:/NLP/PAN2014/pan14-author-profiling-training-corpus-english-twitter-2014-04-16"#path for train data D:\Data\PAN2013-Profiling\Training\en  D:\NLP\PAN2014
##train_truth_path="D:/NLP/PAN2014/truth_train"
##
##test_truth_path="D:/NLP/PAN2014/pan14_test"
##Test_path="D:/NLP/PAN2014/pan14_test/en"
##Content = open('results_of_clssification_all.txt', 'w')

M_train_file= "/Users/catherine/Desktop/NLP/PAN Datasets/PAN2014blog/2014.1-blog-txtfiles/PAN2014_blog_male_train.1.txt"
F_train_file= "/Users/catherine/Desktop/NLP/PAN Datasets/PAN2014blog/2014.1-blog-txtfiles/PAN2014_blog_female_train.1.txt"

M_test_file= "/Users/catherine/Desktop/NLP/PAN Datasets/PAN2014blog/2014.1-blog-txtfiles/PAN2014_blog_male_test.1.txt"
F_test_file= "/Users/catherine/Desktop/NLP/PAN Datasets/PAN2014blog/2014.1-blog-txtfiles/PAN2014_blog_female_test.1.txt"

def all_txt(M_txt_file,  F_txt_file):
    with open(M_txt_file, mode="r", encoding="utf-8") as f:
            txt_list = [(line[:-1]).split("\t") for line in f]
            male_dict = dict()
            content=[]
            for line in txt_list:
                if line[1] in male_dict:
                # append the new number to the existing array at this slot
                    male_dict[line[1]].append(line[3])
                else:
                # create a new array in this slot
                    male_dict[line[1]] = [line[3]]
    male_content= [("M", (' '.join(a)).split()) for a in list(male_dict.values())]
  
    with open(F_txt_file, mode="r", encoding="utf-8") as f:
            txt_list = [(line[:-1]).split("\t") for line in f]
            female_dict = dict()
            content=[]
            for line in txt_list:
                if line[1] in female_dict:
                # append the new number to the existing array at this slot
                    female_dict[line[1]].append(line[3])
                else:
                # create a new array in this slot
                    female_dict[line[1]] = [line[3]]
    female_content= [("F", (' '.join(a)).split()) for a in list(female_dict.values())]
    all_txt_per_person = male_content + female_content
    return all_txt_per_person

##print( [((all_txt(M_txt_file,  F_txt_file))[i][0])  for i in range(len((all_txt(M_txt_file,  F_txt_file))))]) helps to determine the gender....

      
"""
create feature vector from the training set.....................................................................................
"""

def WordFeatures(word_list, all_training_text):

    fvs_words = np.array([[author.count(word) for word in word_list] for author in all_training_text]).astype(np.float64)

    # normalise by dividing each row by number of tokens for each author........
    fvs_words /= np.c_[np.array([len(author) for author in all_training_text])]

    return fvs_words



"""
From the saved csv file, recover the saved features to be used...............................................................
"""
import csv
word_list=[]
##txt_files =[ "/Users/catherine/Desktop/NLP/PAN Datasets/PAN_chosen_tokens/2014/output2014OR.1M.txt", "/Users/catherine/Desktop/NLP/PAN Datasets/PAN_chosen_tokens/2014/output2014OR.2F.txt"]
txt_files =[ "/Users/catherine/Desktop/NLP/PAN Datasets/PAN2014blog/2014.1-blog-txtfiles/PAN2014_blog_female_chosen_set.1.csv", "/Users/catherine/Desktop/NLP/PAN Datasets/PAN2014blog/2014.1-blog-txtfiles/PAN2014_blog_male_chosen_set.1.csv"]
print("txt_files of features used",txt_files)
##for txt_file in txt_files:
##    with open(txt_file, mode="r", encoding="utf-8") as f:
##        reader = csv.reader(f, delimiter=",")
##        next(reader) # skip header
##        word_list1 =  ([(r.split(":"))[1] for r in f])
##        print(txt_file, len(word_list1))
##        word_list = word_list + word_list1
##print("length of features used =", len(word_list))

for txt_file in txt_files:
    with open(txt_file, mode="r", encoding="utf-8") as f:
        reader = csv.reader(f, delimiter=",")
        next(reader) # skip header
        word_list1 =  [r[0] for r in reader]
        print(txt_file, len(word_list1))
        word_list = word_list + word_list1
print("length of features used =", len(word_list))


"""
Prepare the training and test sets to be parsed to the classifies............................................................
"""
A=(all_txt(M_train_file,  F_train_file))
all_training_text = ( [(A[i][1])  for i in range(len(A))])
print("len(all_training_text)",len(all_training_text))
X_train=WordFeatures(word_list, all_training_text)
y_train=np.array( [(A[i][0])  for i in range(len(A))])
print("len(X_train)=",len(X_train), "len(y_train)=", len(y_train))

B=(all_txt(M_test_file,  F_test_file))
all_test_text = ( [(B[i][1])  for i in range(len(B))])
print("len(all_test_text)",len(all_test_text))
X_test = WordFeatures(word_list, all_test_text)
y_test = np.array( [(B[i][0])  for i in range(len(B))])
print("len(X_test=",len(X_test), "len(y_test)=",len(y_test))


X_train = np.nan_to_num(X_train)
X_test = np.nan_to_num(X_test)

"""
Perform feature selection using chi2............................................................
"""
from sklearn.feature_selection import *
##
####def select_Bestfeatures(X, y):
print("selector = SelectKBest(score_func=chi2, k=200)")
selector = SelectKBest(score_func=chi2, k=200)
fit = selector.fit(X_train,y_train)
train_selected_features = fit.transform(X_train)
test_selected_features = fit.transform(X_test)



"""
x_train and x_test based on the new features using chi2............................................................
"""
##X_train = select_Bestfeatures(X_train, y_train)
##X_test = select_Bestfeatures(X_test, y_test)


"""
Perform feature selection using f_classif............................................................
"""
from sklearn.feature_selection import *
##def f_classif_select_Bestfeatures(X, y):
##selector=SelectKBest(score_func=f_classif,k=500)
##fit = selector.fit(X_train,y_train)
##train_selected_features = fit.transform(X_train)
##test_selected_features = fit.transform(X_test)
##    selected_features = fit.transform(X)
##    return selected_features

"""
x_train and x_test based on the new features using f_classif............................................................
"""
##X_train = f_classif_select_Bestfeatures(X_train, y_train)
##X_test = f_classif_select_Bestfeatures(X_test, y_test)



"""
Perform feature selection using mutual_info_classif............................................................
"""
from sklearn.feature_selection import *
##def mutual_info_classif_select_Bestfeatures(X_tr, X_te, y):
##selector=SelectKBest(score_func=mutual_info_classif,k=100)
##fit = selector.fit(X_train,y_train)
##train_selected_features = fit.transform(X_train)
##test_selected_features = fit.transform(X_test)
##    return train_selected_features, test_selected_features

"""
x_train and x_test based on the new features using f_classif............................................................
"""
X_train =train_selected_features 
X_test = test_selected_features


"""
Perform feature selection using pca............................................................
"""

##from sklearn.decomposition import PCA
##def PCA_select_Bestfeatures(X):
##    # feature extraction
##    pca = PCA(n_components=200)
##    fit = pca.fit(X, y)
##    Train_selected_features = fit.transform(X)
##    return selected_features

"""
x_train and x_test based on the new features using f_classif............................................................
"""
##X_train = PCA_select_Bestfeatures(X_train)
##X_test = PCA_select_Bestfeatures(X_test)

"""
Below are a list of Classification Algorithms with different distance measres ..............................................................................
"""
"""
K-Nearest Neighbours (average aa over all the nearest neighbour selectiopn)..............................................................................
"""

k=5

print(k)
"""
8. manhattan K-Nearest Neighbours..............................................................................
"""
from sklearn.neighbors import KNeighborsClassifier
knn = KNeighborsClassifier(n_neighbors = k, metric= "manhattan")
knn.fit(X_train, y_train)
y_pred = knn.predict(X_test)
print("manhattan K-Nearest Neighbours")
print(metrics.accuracy_score(y_test, y_pred))

    
"""
8. cosine K-Nearest Neighbours..............................................................................
"""
from sklearn.neighbors import KNeighborsClassifier
knn = KNeighborsClassifier(n_neighbors = k, metric= "cosine")
knn.fit(X_train, y_train)
y_pred = knn.predict(X_test)
print("cosine K-Nearest Neighbours")
print(metrics.accuracy_score(y_test, y_pred))


"""
8. euclidean K-Nearest Neighbours..............................................................................
"""
from sklearn.neighbors import KNeighborsClassifier
knn = KNeighborsClassifier(n_neighbors = k, metric= "euclidean")
knn.fit(X_train, y_train)
y_pred = knn.predict(X_test)
print("euclidean K-Nearest Neighbours")
print(metrics.accuracy_score(y_test, y_pred))

        
"""
9. scipy.spatial.distance K-Nearest Neighbours..............................................................................
"""
from scipy.spatial.distance import *
knn = KNeighborsClassifier(n_neighbors = k, metric= "braycurtis")
knn.fit(X_train, y_train)
y_pred = knn.predict(X_test)
print("braycurtis K-Nearest Neighbours")
print(metrics.accuracy_score(y_test, y_pred))
       
        
knn = KNeighborsClassifier(n_neighbors = k, metric= "canberra")
knn.fit(X_train, y_train)
y_pred = knn.predict(X_test)
print("canberra K-Nearest Neighbours")
print(metrics.accuracy_score(y_test, y_pred))
  
        
knn = KNeighborsClassifier(n_neighbors = k, metric= "jaccard")
knn.fit(X_train, y_train)
y_pred = knn.predict(X_test)
print("jaccard K-Nearest Neighbours")
print(metrics.accuracy_score(y_test, y_pred))
        
knn = KNeighborsClassifier(n_neighbors = k, metric= "yule")
knn.fit(X_train, y_train)
y_pred = knn.predict(X_test)
print("yule K-Nearest Neighbours")
print(metrics.accuracy_score(y_test, y_pred))
print("************************************ ****************************")



txt_files of features used ['/Users/catherine/Desktop/NLP/PAN Datasets/PAN2014blog/2014.1-blog-txtfiles/PAN2014_blog_female_chosen_set.1.csv', '/Users/catherine/Desktop/NLP/PAN Datasets/PAN2014blog/2014.1-blog-txtfiles/PAN2014_blog_male_chosen_set.1.csv']
/Users/catherine/Desktop/NLP/PAN Datasets/PAN2014blog/2014.1-blog-txtfiles/PAN2014_blog_female_chosen_set.1.csv 1526
/Users/catherine/Desktop/NLP/PAN Datasets/PAN2014blog/2014.1-blog-txtfiles/PAN2014_blog_male_chosen_set.1.csv 2063
length of features used = 3589
len(all_training_text) 147
len(X_train)= 147 len(y_train)= 147
len(all_test_text) 78
len(X_test= 78 len(y_test)= 78
selector = SelectKBest(score_func=chi2, k=200)
5
manhattan K-Nearest Neighbours
0.6666666666666666
cosine K-Nearest Neighbours
0.6282051282051282
euclidean K-Nearest Neighbours
0.6410256410256411
braycurtis K-Nearest Neighbours
0.6282051282051282
canberra K-Nearest Neighbours
0.5641025641025641
jaccard K-Nearest Neighbours
0.6923076923076923
yule K-Nearest Neighb

/anaconda3/lib/python3.7/site-packages/sklearn/utils/validation.py:475: DataConversionWarning: Data with input dtype float64 was converted to bool by check_pairwise_arrays.
  warnings.warn(msg, DataConversionWarning)


In [29]:
import sys
sys.path.append("/anaconda3/lib/python3.7/site-packages")
import numpy as np
import numpy
import nltk
import pickle
import pandas as pd
import glob
import os
import re
import operator
from collections import Counter
#from read_xml_files import *
##from s_stemmer import *
from nltk.stem.porter import PorterStemmer
from nltk.stem import LancasterStemmer, WordNetLemmatizer
from itertools import groupby
from nltk.collocations import *
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.cluster import KMeans
from scipy.cluster.vq import whiten
from nltk.tokenize.toktok import ToktokTokenizer
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2
import xml.etree.ElementTree as ET
import pandas as pd
#Import scikit-learn metrics module for accuracy calculation
from sklearn import metrics
##from pattern.text.en import singularize
##nltk.download('vader_lexicon')
##nltk.download('punkt')
#import the relevant modules from the NLTK library
from nltk.sentiment.vader import SentimentIntensityAnalyzer
sentence_tokenizer = nltk.data.load('tokenizers/punkt/english.pickle')
word_tokenizer = nltk.tokenize.RegexpTokenizer(r'\w+')
##tt = ToktokTokenizer()#keeps the url as is...................................................
from nltk.tokenize import TweetTokenizer
tt = TweetTokenizer()
Porter_stemmer = PorterStemmer()
Lancaster_stemmer = LancasterStemmer()
WordNet_lemmatizer = WordNetLemmatizer()
#nltk.download('averaged_perceptron_tagger')
##Train_path="D:/Data/PAN2015-Profiling/pan15-author-profiling-training-dataset-english-2015-04-23"#path for train data
##Test_path="D:/Data/PAN2015-Profiling/pan-ap2015-test/en" #path for test data

##Train_path="D:/NLP/PAN2014/pan14-author-profiling-training-corpus-english-twitter-2014-04-16"#path for train data D:\Data\PAN2013-Profiling\Training\en  D:\NLP\PAN2014
##train_truth_path="D:/NLP/PAN2014/truth_train"
##
##test_truth_path="D:/NLP/PAN2014/pan14_test"
##Test_path="D:/NLP/PAN2014/pan14_test/en"
##Content = open('results_of_clssification_all.txt', 'w')

M_train_file= "/Users/catherine/Desktop/NLP/PAN Datasets/PAN2014blog/2014.1-blog-txtfiles/PAN2014_blog_male_train.1.txt"
F_train_file= "/Users/catherine/Desktop/NLP/PAN Datasets/PAN2014blog/2014.1-blog-txtfiles/PAN2014_blog_female_train.1.txt"

M_test_file= "/Users/catherine/Desktop/NLP/PAN Datasets/PAN2014blog/2014.1-blog-txtfiles/PAN2014_blog_male_test.1.txt"
F_test_file= "/Users/catherine/Desktop/NLP/PAN Datasets/PAN2014blog/2014.1-blog-txtfiles/PAN2014_blog_female_test.1.txt"

def all_txt(M_txt_file,  F_txt_file):
    with open(M_txt_file, mode="r", encoding="utf-8") as f:
            txt_list = [(line[:-1]).split("\t") for line in f]
            male_dict = dict()
            content=[]
            for line in txt_list:
                if line[1] in male_dict:
                # append the new number to the existing array at this slot
                    male_dict[line[1]].append(line[3])
                else:
                # create a new array in this slot
                    male_dict[line[1]] = [line[3]]
    male_content= [("M", (' '.join(a)).split()) for a in list(male_dict.values())]
  
    with open(F_txt_file, mode="r", encoding="utf-8") as f:
            txt_list = [(line[:-1]).split("\t") for line in f]
            female_dict = dict()
            content=[]
            for line in txt_list:
                if line[1] in female_dict:
                # append the new number to the existing array at this slot
                    female_dict[line[1]].append(line[3])
                else:
                # create a new array in this slot
                    female_dict[line[1]] = [line[3]]
    female_content= [("F", (' '.join(a)).split()) for a in list(female_dict.values())]
    all_txt_per_person = male_content + female_content
    return all_txt_per_person

##print( [((all_txt(M_txt_file,  F_txt_file))[i][0])  for i in range(len((all_txt(M_txt_file,  F_txt_file))))]) helps to determine the gender....

      
"""
create feature vector from the training set.....................................................................................
"""

def WordFeatures(word_list, all_training_text):

    fvs_words = np.array([[author.count(word) for word in word_list] for author in all_training_text]).astype(np.float64)

    # normalise by dividing each row by number of tokens for each author........
    fvs_words /= np.c_[np.array([len(author) for author in all_training_text])]

    return fvs_words



"""
From the saved csv file, recover the saved features to be used...............................................................
"""
import csv
word_list=[]
##txt_files =[ "/Users/catherine/Desktop/NLP/PAN Datasets/PAN_chosen_tokens/2014/output2014OR.1M.txt", "/Users/catherine/Desktop/NLP/PAN Datasets/PAN_chosen_tokens/2014/output2014OR.2F.txt"]
txt_files =[ "/Users/catherine/Desktop/NLP/PAN Datasets/PAN2014blog/2014.1-blog-txtfiles/PAN2014_blog_female_chosen_set.1.csv", "/Users/catherine/Desktop/NLP/PAN Datasets/PAN2014blog/2014.1-blog-txtfiles/PAN2014_blog_male_chosen_set.1.csv"]
print("txt_files of features used",txt_files)
##for txt_file in txt_files:
##    with open(txt_file, mode="r", encoding="utf-8") as f:
##        reader = csv.reader(f, delimiter=",")
##        next(reader) # skip header
##        word_list1 =  ([(r.split(":"))[1] for r in f])
##        print(txt_file, len(word_list1))
##        word_list = word_list + word_list1
##print("length of features used =", len(word_list))

for txt_file in txt_files:
    with open(txt_file, mode="r", encoding="utf-8") as f:
        reader = csv.reader(f, delimiter=",")
        next(reader) # skip header
        word_list1 =  [r[0] for r in reader]
        print(txt_file, len(word_list1))
        word_list = word_list + word_list1
print("length of features used =", len(word_list))


"""
Prepare the training and test sets to be parsed to the classifies............................................................
"""
A=(all_txt(M_train_file,  F_train_file))
all_training_text = ( [(A[i][1])  for i in range(len(A))])
print("len(all_training_text)",len(all_training_text))
X_train=WordFeatures(word_list, all_training_text)
y_train=np.array( [(A[i][0])  for i in range(len(A))])
print("len(X_train)=",len(X_train), "len(y_train)=", len(y_train))

B=(all_txt(M_test_file,  F_test_file))
all_test_text = ( [(B[i][1])  for i in range(len(B))])
print("len(all_test_text)",len(all_test_text))
X_test = WordFeatures(word_list, all_test_text)
y_test = np.array( [(B[i][0])  for i in range(len(B))])
print("len(X_test=",len(X_test), "len(y_test)=",len(y_test))


X_train = np.nan_to_num(X_train)
X_test = np.nan_to_num(X_test)

"""
Perform feature selection using chi2............................................................
"""
from sklearn.feature_selection import *
##
####def select_Bestfeatures(X, y):
print("selector = SelectKBest(score_func=chi2, k=300)")
selector = SelectKBest(score_func=chi2, k=300)
fit = selector.fit(X_train,y_train)
train_selected_features = fit.transform(X_train)
test_selected_features = fit.transform(X_test)



"""
x_train and x_test based on the new features using chi2............................................................
"""
##X_train = select_Bestfeatures(X_train, y_train)
##X_test = select_Bestfeatures(X_test, y_test)


"""
Perform feature selection using f_classif............................................................
"""
from sklearn.feature_selection import *
##def f_classif_select_Bestfeatures(X, y):
##selector=SelectKBest(score_func=f_classif,k=500)
##fit = selector.fit(X_train,y_train)
##train_selected_features = fit.transform(X_train)
##test_selected_features = fit.transform(X_test)
##    selected_features = fit.transform(X)
##    return selected_features

"""
x_train and x_test based on the new features using f_classif............................................................
"""
##X_train = f_classif_select_Bestfeatures(X_train, y_train)
##X_test = f_classif_select_Bestfeatures(X_test, y_test)



"""
Perform feature selection using mutual_info_classif............................................................
"""
from sklearn.feature_selection import *
##def mutual_info_classif_select_Bestfeatures(X_tr, X_te, y):
##selector=SelectKBest(score_func=mutual_info_classif,k=100)
##fit = selector.fit(X_train,y_train)
##train_selected_features = fit.transform(X_train)
##test_selected_features = fit.transform(X_test)
##    return train_selected_features, test_selected_features

"""
x_train and x_test based on the new features using f_classif............................................................
"""
X_train =train_selected_features 
X_test = test_selected_features


"""
Perform feature selection using pca............................................................
"""

##from sklearn.decomposition import PCA
##def PCA_select_Bestfeatures(X):
##    # feature extraction
##    pca = PCA(n_components=200)
##    fit = pca.fit(X, y)
##    Train_selected_features = fit.transform(X)
##    return selected_features

"""
x_train and x_test based on the new features using f_classif............................................................
"""
##X_train = PCA_select_Bestfeatures(X_train)
##X_test = PCA_select_Bestfeatures(X_test)

"""
Below are a list of Classification Algorithms with different distance measres ..............................................................................
"""
"""
K-Nearest Neighbours (average aa over all the nearest neighbour selectiopn)..............................................................................
"""

k=5

print(k)
"""
8. manhattan K-Nearest Neighbours..............................................................................
"""
from sklearn.neighbors import KNeighborsClassifier
knn = KNeighborsClassifier(n_neighbors = k, metric= "manhattan")
knn.fit(X_train, y_train)
y_pred = knn.predict(X_test)
print("manhattan K-Nearest Neighbours")
print(metrics.accuracy_score(y_test, y_pred))

    
"""
8. cosine K-Nearest Neighbours..............................................................................
"""
from sklearn.neighbors import KNeighborsClassifier
knn = KNeighborsClassifier(n_neighbors = k, metric= "cosine")
knn.fit(X_train, y_train)
y_pred = knn.predict(X_test)
print("cosine K-Nearest Neighbours")
print(metrics.accuracy_score(y_test, y_pred))


"""
8. euclidean K-Nearest Neighbours..............................................................................
"""
from sklearn.neighbors import KNeighborsClassifier
knn = KNeighborsClassifier(n_neighbors = k, metric= "euclidean")
knn.fit(X_train, y_train)
y_pred = knn.predict(X_test)
print("euclidean K-Nearest Neighbours")
print(metrics.accuracy_score(y_test, y_pred))

        
"""
9. scipy.spatial.distance K-Nearest Neighbours..............................................................................
"""
from scipy.spatial.distance import *
knn = KNeighborsClassifier(n_neighbors = k, metric= "braycurtis")
knn.fit(X_train, y_train)
y_pred = knn.predict(X_test)
print("braycurtis K-Nearest Neighbours")
print(metrics.accuracy_score(y_test, y_pred))
       
        
knn = KNeighborsClassifier(n_neighbors = k, metric= "canberra")
knn.fit(X_train, y_train)
y_pred = knn.predict(X_test)
print("canberra K-Nearest Neighbours")
print(metrics.accuracy_score(y_test, y_pred))
  
        
knn = KNeighborsClassifier(n_neighbors = k, metric= "jaccard")
knn.fit(X_train, y_train)
y_pred = knn.predict(X_test)
print("jaccard K-Nearest Neighbours")
print(metrics.accuracy_score(y_test, y_pred))
        
knn = KNeighborsClassifier(n_neighbors = k, metric= "yule")
knn.fit(X_train, y_train)
y_pred = knn.predict(X_test)
print("yule K-Nearest Neighbours")
print(metrics.accuracy_score(y_test, y_pred))
print("************************************ ****************************")




txt_files of features used ['/Users/catherine/Desktop/NLP/PAN Datasets/PAN2014blog/2014.1-blog-txtfiles/PAN2014_blog_female_chosen_set.1.csv', '/Users/catherine/Desktop/NLP/PAN Datasets/PAN2014blog/2014.1-blog-txtfiles/PAN2014_blog_male_chosen_set.1.csv']
/Users/catherine/Desktop/NLP/PAN Datasets/PAN2014blog/2014.1-blog-txtfiles/PAN2014_blog_female_chosen_set.1.csv 1526
/Users/catherine/Desktop/NLP/PAN Datasets/PAN2014blog/2014.1-blog-txtfiles/PAN2014_blog_male_chosen_set.1.csv 2063
length of features used = 3589
len(all_training_text) 147
len(X_train)= 147 len(y_train)= 147
len(all_test_text) 78
len(X_test= 78 len(y_test)= 78
selector = SelectKBest(score_func=chi2, k=300)
5
manhattan K-Nearest Neighbours
0.6538461538461539
cosine K-Nearest Neighbours
0.5897435897435898
euclidean K-Nearest Neighbours
0.6282051282051282
braycurtis K-Nearest Neighbours
0.6538461538461539
canberra K-Nearest Neighbours
0.5384615384615384
jaccard K-Nearest Neighbours
0.6153846153846154
yule K-Nearest Neighb

/anaconda3/lib/python3.7/site-packages/sklearn/utils/validation.py:475: DataConversionWarning: Data with input dtype float64 was converted to bool by check_pairwise_arrays.
  warnings.warn(msg, DataConversionWarning)


In [30]:
import sys
sys.path.append("/anaconda3/lib/python3.7/site-packages")
import numpy as np
import numpy
import nltk
import pickle
import pandas as pd
import glob
import os
import re
import operator
from collections import Counter
#from read_xml_files import *
##from s_stemmer import *
from nltk.stem.porter import PorterStemmer
from nltk.stem import LancasterStemmer, WordNetLemmatizer
from itertools import groupby
from nltk.collocations import *
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.cluster import KMeans
from scipy.cluster.vq import whiten
from nltk.tokenize.toktok import ToktokTokenizer
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2
import xml.etree.ElementTree as ET
import pandas as pd
#Import scikit-learn metrics module for accuracy calculation
from sklearn import metrics
##from pattern.text.en import singularize
##nltk.download('vader_lexicon')
##nltk.download('punkt')
#import the relevant modules from the NLTK library
from nltk.sentiment.vader import SentimentIntensityAnalyzer
sentence_tokenizer = nltk.data.load('tokenizers/punkt/english.pickle')
word_tokenizer = nltk.tokenize.RegexpTokenizer(r'\w+')
##tt = ToktokTokenizer()#keeps the url as is...................................................
from nltk.tokenize import TweetTokenizer
tt = TweetTokenizer()
Porter_stemmer = PorterStemmer()
Lancaster_stemmer = LancasterStemmer()
WordNet_lemmatizer = WordNetLemmatizer()
#nltk.download('averaged_perceptron_tagger')
##Train_path="D:/Data/PAN2015-Profiling/pan15-author-profiling-training-dataset-english-2015-04-23"#path for train data
##Test_path="D:/Data/PAN2015-Profiling/pan-ap2015-test/en" #path for test data

##Train_path="D:/NLP/PAN2014/pan14-author-profiling-training-corpus-english-twitter-2014-04-16"#path for train data D:\Data\PAN2013-Profiling\Training\en  D:\NLP\PAN2014
##train_truth_path="D:/NLP/PAN2014/truth_train"
##
##test_truth_path="D:/NLP/PAN2014/pan14_test"
##Test_path="D:/NLP/PAN2014/pan14_test/en"
##Content = open('results_of_clssification_all.txt', 'w')

M_train_file= "/Users/catherine/Desktop/NLP/PAN Datasets/PAN2014blog/2014.1-blog-txtfiles/PAN2014_blog_male_train.1.txt"
F_train_file= "/Users/catherine/Desktop/NLP/PAN Datasets/PAN2014blog/2014.1-blog-txtfiles/PAN2014_blog_female_train.1.txt"

M_test_file= "/Users/catherine/Desktop/NLP/PAN Datasets/PAN2014blog/2014.1-blog-txtfiles/PAN2014_blog_male_test.1.txt"
F_test_file= "/Users/catherine/Desktop/NLP/PAN Datasets/PAN2014blog/2014.1-blog-txtfiles/PAN2014_blog_female_test.1.txt"

def all_txt(M_txt_file,  F_txt_file):
    with open(M_txt_file, mode="r", encoding="utf-8") as f:
            txt_list = [(line[:-1]).split("\t") for line in f]
            male_dict = dict()
            content=[]
            for line in txt_list:
                if line[1] in male_dict:
                # append the new number to the existing array at this slot
                    male_dict[line[1]].append(line[3])
                else:
                # create a new array in this slot
                    male_dict[line[1]] = [line[3]]
    male_content= [("M", (' '.join(a)).split()) for a in list(male_dict.values())]
  
    with open(F_txt_file, mode="r", encoding="utf-8") as f:
            txt_list = [(line[:-1]).split("\t") for line in f]
            female_dict = dict()
            content=[]
            for line in txt_list:
                if line[1] in female_dict:
                # append the new number to the existing array at this slot
                    female_dict[line[1]].append(line[3])
                else:
                # create a new array in this slot
                    female_dict[line[1]] = [line[3]]
    female_content= [("F", (' '.join(a)).split()) for a in list(female_dict.values())]
    all_txt_per_person = male_content + female_content
    return all_txt_per_person

##print( [((all_txt(M_txt_file,  F_txt_file))[i][0])  for i in range(len((all_txt(M_txt_file,  F_txt_file))))]) helps to determine the gender....

      
"""
create feature vector from the training set.....................................................................................
"""

def WordFeatures(word_list, all_training_text):

    fvs_words = np.array([[author.count(word) for word in word_list] for author in all_training_text]).astype(np.float64)

    # normalise by dividing each row by number of tokens for each author........
    fvs_words /= np.c_[np.array([len(author) for author in all_training_text])]

    return fvs_words



"""
From the saved csv file, recover the saved features to be used...............................................................
"""
import csv
word_list=[]
##txt_files =[ "/Users/catherine/Desktop/NLP/PAN Datasets/PAN_chosen_tokens/2014/output2014OR.1M.txt", "/Users/catherine/Desktop/NLP/PAN Datasets/PAN_chosen_tokens/2014/output2014OR.2F.txt"]
txt_files =[ "/Users/catherine/Desktop/NLP/PAN Datasets/PAN2014blog/2014.1-blog-txtfiles/PAN2014_blog_female_chosen_set.1.csv", "/Users/catherine/Desktop/NLP/PAN Datasets/PAN2014blog/2014.1-blog-txtfiles/PAN2014_blog_male_chosen_set.1.csv"]
print("txt_files of features used",txt_files)
##for txt_file in txt_files:
##    with open(txt_file, mode="r", encoding="utf-8") as f:
##        reader = csv.reader(f, delimiter=",")
##        next(reader) # skip header
##        word_list1 =  ([(r.split(":"))[1] for r in f])
##        print(txt_file, len(word_list1))
##        word_list = word_list + word_list1
##print("length of features used =", len(word_list))

for txt_file in txt_files:
    with open(txt_file, mode="r", encoding="utf-8") as f:
        reader = csv.reader(f, delimiter=",")
        next(reader) # skip header
        word_list1 =  [r[0] for r in reader]
        print(txt_file, len(word_list1))
        word_list = word_list + word_list1
print("length of features used =", len(word_list))


"""
Prepare the training and test sets to be parsed to the classifies............................................................
"""
A=(all_txt(M_train_file,  F_train_file))
all_training_text = ( [(A[i][1])  for i in range(len(A))])
print("len(all_training_text)",len(all_training_text))
X_train=WordFeatures(word_list, all_training_text)
y_train=np.array( [(A[i][0])  for i in range(len(A))])
print("len(X_train)=",len(X_train), "len(y_train)=", len(y_train))

B=(all_txt(M_test_file,  F_test_file))
all_test_text = ( [(B[i][1])  for i in range(len(B))])
print("len(all_test_text)",len(all_test_text))
X_test = WordFeatures(word_list, all_test_text)
y_test = np.array( [(B[i][0])  for i in range(len(B))])
print("len(X_test=",len(X_test), "len(y_test)=",len(y_test))


X_train = np.nan_to_num(X_train)
X_test = np.nan_to_num(X_test)

"""
Perform feature selection using chi2............................................................
"""
from sklearn.feature_selection import *
##
####def select_Bestfeatures(X, y):
print("selector = SelectKBest(score_func=chi2, k=500)")
selector = SelectKBest(score_func=chi2, k=500)
fit = selector.fit(X_train,y_train)
train_selected_features = fit.transform(X_train)
test_selected_features = fit.transform(X_test)



"""
x_train and x_test based on the new features using chi2............................................................
"""
##X_train = select_Bestfeatures(X_train, y_train)
##X_test = select_Bestfeatures(X_test, y_test)


"""
Perform feature selection using f_classif............................................................
"""
from sklearn.feature_selection import *
##def f_classif_select_Bestfeatures(X, y):
##selector=SelectKBest(score_func=f_classif,k=500)
##fit = selector.fit(X_train,y_train)
##train_selected_features = fit.transform(X_train)
##test_selected_features = fit.transform(X_test)
##    selected_features = fit.transform(X)
##    return selected_features

"""
x_train and x_test based on the new features using f_classif............................................................
"""
##X_train = f_classif_select_Bestfeatures(X_train, y_train)
##X_test = f_classif_select_Bestfeatures(X_test, y_test)



"""
Perform feature selection using mutual_info_classif............................................................
"""
from sklearn.feature_selection import *
##def mutual_info_classif_select_Bestfeatures(X_tr, X_te, y):
##selector=SelectKBest(score_func=mutual_info_classif,k=100)
##fit = selector.fit(X_train,y_train)
##train_selected_features = fit.transform(X_train)
##test_selected_features = fit.transform(X_test)
##    return train_selected_features, test_selected_features

"""
x_train and x_test based on the new features using f_classif............................................................
"""
X_train =train_selected_features 
X_test = test_selected_features


"""
Perform feature selection using pca............................................................
"""

##from sklearn.decomposition import PCA
##def PCA_select_Bestfeatures(X):
##    # feature extraction
##    pca = PCA(n_components=200)
##    fit = pca.fit(X, y)
##    Train_selected_features = fit.transform(X)
##    return selected_features

"""
x_train and x_test based on the new features using f_classif............................................................
"""
##X_train = PCA_select_Bestfeatures(X_train)
##X_test = PCA_select_Bestfeatures(X_test)

"""
Below are a list of Classification Algorithms with different distance measres ..............................................................................
"""
"""
K-Nearest Neighbours (average aa over all the nearest neighbour selectiopn)..............................................................................
"""

k=5

print(k)
"""
8. manhattan K-Nearest Neighbours..............................................................................
"""
from sklearn.neighbors import KNeighborsClassifier
knn = KNeighborsClassifier(n_neighbors = k, metric= "manhattan")
knn.fit(X_train, y_train)
y_pred = knn.predict(X_test)
print("manhattan K-Nearest Neighbours")
print(metrics.accuracy_score(y_test, y_pred))

    
"""
8. cosine K-Nearest Neighbours..............................................................................
"""
from sklearn.neighbors import KNeighborsClassifier
knn = KNeighborsClassifier(n_neighbors = k, metric= "cosine")
knn.fit(X_train, y_train)
y_pred = knn.predict(X_test)
print("cosine K-Nearest Neighbours")
print(metrics.accuracy_score(y_test, y_pred))


"""
8. euclidean K-Nearest Neighbours..............................................................................
"""
from sklearn.neighbors import KNeighborsClassifier
knn = KNeighborsClassifier(n_neighbors = k, metric= "euclidean")
knn.fit(X_train, y_train)
y_pred = knn.predict(X_test)
print("euclidean K-Nearest Neighbours")
print(metrics.accuracy_score(y_test, y_pred))

        
"""
9. scipy.spatial.distance K-Nearest Neighbours..............................................................................
"""
from scipy.spatial.distance import *
knn = KNeighborsClassifier(n_neighbors = k, metric= "braycurtis")
knn.fit(X_train, y_train)
y_pred = knn.predict(X_test)
print("braycurtis K-Nearest Neighbours")
print(metrics.accuracy_score(y_test, y_pred))
       
        
knn = KNeighborsClassifier(n_neighbors = k, metric= "canberra")
knn.fit(X_train, y_train)
y_pred = knn.predict(X_test)
print("canberra K-Nearest Neighbours")
print(metrics.accuracy_score(y_test, y_pred))
  
        
knn = KNeighborsClassifier(n_neighbors = k, metric= "jaccard")
knn.fit(X_train, y_train)
y_pred = knn.predict(X_test)
print("jaccard K-Nearest Neighbours")
print(metrics.accuracy_score(y_test, y_pred))
        
knn = KNeighborsClassifier(n_neighbors = k, metric= "yule")
knn.fit(X_train, y_train)
y_pred = knn.predict(X_test)
print("yule K-Nearest Neighbours")
print(metrics.accuracy_score(y_test, y_pred))
print("************************************ ****************************")




txt_files of features used ['/Users/catherine/Desktop/NLP/PAN Datasets/PAN2014blog/2014.1-blog-txtfiles/PAN2014_blog_female_chosen_set.1.csv', '/Users/catherine/Desktop/NLP/PAN Datasets/PAN2014blog/2014.1-blog-txtfiles/PAN2014_blog_male_chosen_set.1.csv']
/Users/catherine/Desktop/NLP/PAN Datasets/PAN2014blog/2014.1-blog-txtfiles/PAN2014_blog_female_chosen_set.1.csv 1526
/Users/catherine/Desktop/NLP/PAN Datasets/PAN2014blog/2014.1-blog-txtfiles/PAN2014_blog_male_chosen_set.1.csv 2063
length of features used = 3589
len(all_training_text) 147
len(X_train)= 147 len(y_train)= 147
len(all_test_text) 78
len(X_test= 78 len(y_test)= 78
selector = SelectKBest(score_func=chi2, k=500)
5
manhattan K-Nearest Neighbours
0.6282051282051282
cosine K-Nearest Neighbours
0.5641025641025641
euclidean K-Nearest Neighbours
0.6282051282051282
braycurtis K-Nearest Neighbours
0.6025641025641025
canberra K-Nearest Neighbours
0.5384615384615384
jaccard K-Nearest Neighbours
0.6153846153846154
yule K-Nearest Neighb

/anaconda3/lib/python3.7/site-packages/sklearn/utils/validation.py:475: DataConversionWarning: Data with input dtype float64 was converted to bool by check_pairwise_arrays.
  warnings.warn(msg, DataConversionWarning)


In [31]:
import sys
sys.path.append("/anaconda3/lib/python3.7/site-packages")
import numpy as np
import numpy
import nltk
import pickle
import pandas as pd
import glob
import os
import re
import operator
from collections import Counter
#from read_xml_files import *
##from s_stemmer import *
from nltk.stem.porter import PorterStemmer
from nltk.stem import LancasterStemmer, WordNetLemmatizer
from itertools import groupby
from nltk.collocations import *
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.cluster import KMeans
from scipy.cluster.vq import whiten
from nltk.tokenize.toktok import ToktokTokenizer
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2
import xml.etree.ElementTree as ET
import pandas as pd
#Import scikit-learn metrics module for accuracy calculation
from sklearn import metrics
##from pattern.text.en import singularize
##nltk.download('vader_lexicon')
##nltk.download('punkt')
#import the relevant modules from the NLTK library
from nltk.sentiment.vader import SentimentIntensityAnalyzer
sentence_tokenizer = nltk.data.load('tokenizers/punkt/english.pickle')
word_tokenizer = nltk.tokenize.RegexpTokenizer(r'\w+')
##tt = ToktokTokenizer()#keeps the url as is...................................................
from nltk.tokenize import TweetTokenizer
tt = TweetTokenizer()
Porter_stemmer = PorterStemmer()
Lancaster_stemmer = LancasterStemmer()
WordNet_lemmatizer = WordNetLemmatizer()
#nltk.download('averaged_perceptron_tagger')
##Train_path="D:/Data/PAN2015-Profiling/pan15-author-profiling-training-dataset-english-2015-04-23"#path for train data
##Test_path="D:/Data/PAN2015-Profiling/pan-ap2015-test/en" #path for test data

##Train_path="D:/NLP/PAN2014/pan14-author-profiling-training-corpus-english-twitter-2014-04-16"#path for train data D:\Data\PAN2013-Profiling\Training\en  D:\NLP\PAN2014
##train_truth_path="D:/NLP/PAN2014/truth_train"
##
##test_truth_path="D:/NLP/PAN2014/pan14_test"
##Test_path="D:/NLP/PAN2014/pan14_test/en"
##Content = open('results_of_clssification_all.txt', 'w')

M_train_file= "/Users/catherine/Desktop/NLP/PAN Datasets/PAN2014blog/2014.1-blog-txtfiles/PAN2014_blog_male_train.1.txt"
F_train_file= "/Users/catherine/Desktop/NLP/PAN Datasets/PAN2014blog/2014.1-blog-txtfiles/PAN2014_blog_female_train.1.txt"

M_test_file= "/Users/catherine/Desktop/NLP/PAN Datasets/PAN2014blog/2014.1-blog-txtfiles/PAN2014_blog_male_test.1.txt"
F_test_file= "/Users/catherine/Desktop/NLP/PAN Datasets/PAN2014blog/2014.1-blog-txtfiles/PAN2014_blog_female_test.1.txt"

def all_txt(M_txt_file,  F_txt_file):
    with open(M_txt_file, mode="r", encoding="utf-8") as f:
            txt_list = [(line[:-1]).split("\t") for line in f]
            male_dict = dict()
            content=[]
            for line in txt_list:
                if line[1] in male_dict:
                # append the new number to the existing array at this slot
                    male_dict[line[1]].append(line[3])
                else:
                # create a new array in this slot
                    male_dict[line[1]] = [line[3]]
    male_content= [("M", (' '.join(a)).split()) for a in list(male_dict.values())]
  
    with open(F_txt_file, mode="r", encoding="utf-8") as f:
            txt_list = [(line[:-1]).split("\t") for line in f]
            female_dict = dict()
            content=[]
            for line in txt_list:
                if line[1] in female_dict:
                # append the new number to the existing array at this slot
                    female_dict[line[1]].append(line[3])
                else:
                # create a new array in this slot
                    female_dict[line[1]] = [line[3]]
    female_content= [("F", (' '.join(a)).split()) for a in list(female_dict.values())]
    all_txt_per_person = male_content + female_content
    return all_txt_per_person

##print( [((all_txt(M_txt_file,  F_txt_file))[i][0])  for i in range(len((all_txt(M_txt_file,  F_txt_file))))]) helps to determine the gender....

      
"""
create feature vector from the training set.....................................................................................
"""

def WordFeatures(word_list, all_training_text):

    fvs_words = np.array([[author.count(word) for word in word_list] for author in all_training_text]).astype(np.float64)

    # normalise by dividing each row by number of tokens for each author........
    fvs_words /= np.c_[np.array([len(author) for author in all_training_text])]

    return fvs_words



"""
From the saved csv file, recover the saved features to be used...............................................................
"""
import csv
word_list=[]
##txt_files =[ "/Users/catherine/Desktop/NLP/PAN Datasets/PAN_chosen_tokens/2014/output2014OR.1M.txt", "/Users/catherine/Desktop/NLP/PAN Datasets/PAN_chosen_tokens/2014/output2014OR.2F.txt"]
txt_files =[ "/Users/catherine/Desktop/NLP/PAN Datasets/PAN2014blog/2014.1-blog-txtfiles/PAN2014_blog_female_chosen_set.1.csv", "/Users/catherine/Desktop/NLP/PAN Datasets/PAN2014blog/2014.1-blog-txtfiles/PAN2014_blog_male_chosen_set.1.csv"]
print("txt_files of features used",txt_files)
##for txt_file in txt_files:
##    with open(txt_file, mode="r", encoding="utf-8") as f:
##        reader = csv.reader(f, delimiter=",")
##        next(reader) # skip header
##        word_list1 =  ([(r.split(":"))[1] for r in f])
##        print(txt_file, len(word_list1))
##        word_list = word_list + word_list1
##print("length of features used =", len(word_list))

for txt_file in txt_files:
    with open(txt_file, mode="r", encoding="utf-8") as f:
        reader = csv.reader(f, delimiter=",")
        next(reader) # skip header
        word_list1 =  [r[0] for r in reader]
        print(txt_file, len(word_list1))
        word_list = word_list + word_list1
print("length of features used =", len(word_list))


"""
Prepare the training and test sets to be parsed to the classifies............................................................
"""
A=(all_txt(M_train_file,  F_train_file))
all_training_text = ( [(A[i][1])  for i in range(len(A))])
print("len(all_training_text)",len(all_training_text))
X_train=WordFeatures(word_list, all_training_text)
y_train=np.array( [(A[i][0])  for i in range(len(A))])
print("len(X_train)=",len(X_train), "len(y_train)=", len(y_train))

B=(all_txt(M_test_file,  F_test_file))
all_test_text = ( [(B[i][1])  for i in range(len(B))])
print("len(all_test_text)",len(all_test_text))
X_test = WordFeatures(word_list, all_test_text)
y_test = np.array( [(B[i][0])  for i in range(len(B))])
print("len(X_test=",len(X_test), "len(y_test)=",len(y_test))


X_train = np.nan_to_num(X_train)
X_test = np.nan_to_num(X_test)

"""
Perform feature selection using chi2............................................................
"""
from sklearn.feature_selection import *
##
####def select_Bestfeatures(X, y):
print("selector = SelectKBest(score_func=mutual_info_classif, k=200)")
selector = SelectKBest(score_func=mutual_info_classif, k=200)
fit = selector.fit(X_train,y_train)
train_selected_features = fit.transform(X_train)
test_selected_features = fit.transform(X_test)



"""
x_train and x_test based on the new features using chi2............................................................
"""
##X_train = select_Bestfeatures(X_train, y_train)
##X_test = select_Bestfeatures(X_test, y_test)


"""
Perform feature selection using f_classif............................................................
"""
from sklearn.feature_selection import *
##def f_classif_select_Bestfeatures(X, y):
##selector=SelectKBest(score_func=f_classif,k=500)
##fit = selector.fit(X_train,y_train)
##train_selected_features = fit.transform(X_train)
##test_selected_features = fit.transform(X_test)
##    selected_features = fit.transform(X)
##    return selected_features

"""
x_train and x_test based on the new features using f_classif............................................................
"""
##X_train = f_classif_select_Bestfeatures(X_train, y_train)
##X_test = f_classif_select_Bestfeatures(X_test, y_test)



"""
Perform feature selection using mutual_info_classif............................................................
"""
from sklearn.feature_selection import *
##def mutual_info_classif_select_Bestfeatures(X_tr, X_te, y):
##selector=SelectKBest(score_func=mutual_info_classif,k=100)
##fit = selector.fit(X_train,y_train)
##train_selected_features = fit.transform(X_train)
##test_selected_features = fit.transform(X_test)
##    return train_selected_features, test_selected_features

"""
x_train and x_test based on the new features using f_classif............................................................
"""
X_train =train_selected_features 
X_test = test_selected_features


"""
Perform feature selection using pca............................................................
"""

##from sklearn.decomposition import PCA
##def PCA_select_Bestfeatures(X):
##    # feature extraction
##    pca = PCA(n_components=200)
##    fit = pca.fit(X, y)
##    Train_selected_features = fit.transform(X)
##    return selected_features

"""
x_train and x_test based on the new features using f_classif............................................................
"""
##X_train = PCA_select_Bestfeatures(X_train)
##X_test = PCA_select_Bestfeatures(X_test)

"""
Below are a list of Classification Algorithms with different distance measres ..............................................................................
"""
"""
K-Nearest Neighbours (average aa over all the nearest neighbour selectiopn)..............................................................................
"""

k=5

print(k)
"""
8. manhattan K-Nearest Neighbours..............................................................................
"""
from sklearn.neighbors import KNeighborsClassifier
knn = KNeighborsClassifier(n_neighbors = k, metric= "manhattan")
knn.fit(X_train, y_train)
y_pred = knn.predict(X_test)
print("manhattan K-Nearest Neighbours")
print(metrics.accuracy_score(y_test, y_pred))

    
"""
8. cosine K-Nearest Neighbours..............................................................................
"""
from sklearn.neighbors import KNeighborsClassifier
knn = KNeighborsClassifier(n_neighbors = k, metric= "cosine")
knn.fit(X_train, y_train)
y_pred = knn.predict(X_test)
print("cosine K-Nearest Neighbours")
print(metrics.accuracy_score(y_test, y_pred))


"""
8. euclidean K-Nearest Neighbours..............................................................................
"""
from sklearn.neighbors import KNeighborsClassifier
knn = KNeighborsClassifier(n_neighbors = k, metric= "euclidean")
knn.fit(X_train, y_train)
y_pred = knn.predict(X_test)
print("euclidean K-Nearest Neighbours")
print(metrics.accuracy_score(y_test, y_pred))

        
"""
9. scipy.spatial.distance K-Nearest Neighbours..............................................................................
"""
from scipy.spatial.distance import *
knn = KNeighborsClassifier(n_neighbors = k, metric= "braycurtis")
knn.fit(X_train, y_train)
y_pred = knn.predict(X_test)
print("braycurtis K-Nearest Neighbours")
print(metrics.accuracy_score(y_test, y_pred))
       
        
knn = KNeighborsClassifier(n_neighbors = k, metric= "canberra")
knn.fit(X_train, y_train)
y_pred = knn.predict(X_test)
print("canberra K-Nearest Neighbours")
print(metrics.accuracy_score(y_test, y_pred))
  
        
knn = KNeighborsClassifier(n_neighbors = k, metric= "jaccard")
knn.fit(X_train, y_train)
y_pred = knn.predict(X_test)
print("jaccard K-Nearest Neighbours")
print(metrics.accuracy_score(y_test, y_pred))
        
knn = KNeighborsClassifier(n_neighbors = k, metric= "yule")
knn.fit(X_train, y_train)
y_pred = knn.predict(X_test)
print("yule K-Nearest Neighbours")
print(metrics.accuracy_score(y_test, y_pred))
print("************************************ ****************************")




txt_files of features used ['/Users/catherine/Desktop/NLP/PAN Datasets/PAN2014blog/2014.1-blog-txtfiles/PAN2014_blog_female_chosen_set.1.csv', '/Users/catherine/Desktop/NLP/PAN Datasets/PAN2014blog/2014.1-blog-txtfiles/PAN2014_blog_male_chosen_set.1.csv']
/Users/catherine/Desktop/NLP/PAN Datasets/PAN2014blog/2014.1-blog-txtfiles/PAN2014_blog_female_chosen_set.1.csv 1526
/Users/catherine/Desktop/NLP/PAN Datasets/PAN2014blog/2014.1-blog-txtfiles/PAN2014_blog_male_chosen_set.1.csv 2063
length of features used = 3589
len(all_training_text) 147
len(X_train)= 147 len(y_train)= 147
len(all_test_text) 78
len(X_test= 78 len(y_test)= 78
selector = SelectKBest(score_func=mutual_info_classif, k=200)
5
manhattan K-Nearest Neighbours
0.5256410256410257
cosine K-Nearest Neighbours
0.6666666666666666
euclidean K-Nearest Neighbours
0.5128205128205128
braycurtis K-Nearest Neighbours
0.6282051282051282
canberra K-Nearest Neighbours
0.5
jaccard K-Nearest Neighbours
0.6282051282051282
yule K-Nearest Neighb

/anaconda3/lib/python3.7/site-packages/sklearn/utils/validation.py:475: DataConversionWarning: Data with input dtype float64 was converted to bool by check_pairwise_arrays.
  warnings.warn(msg, DataConversionWarning)


In [32]:
import sys
sys.path.append("/anaconda3/lib/python3.7/site-packages")
import numpy as np
import numpy
import nltk
import pickle
import pandas as pd
import glob
import os
import re
import operator
from collections import Counter
#from read_xml_files import *
##from s_stemmer import *
from nltk.stem.porter import PorterStemmer
from nltk.stem import LancasterStemmer, WordNetLemmatizer
from itertools import groupby
from nltk.collocations import *
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.cluster import KMeans
from scipy.cluster.vq import whiten
from nltk.tokenize.toktok import ToktokTokenizer
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2
import xml.etree.ElementTree as ET
import pandas as pd
#Import scikit-learn metrics module for accuracy calculation
from sklearn import metrics
##from pattern.text.en import singularize
##nltk.download('vader_lexicon')
##nltk.download('punkt')
#import the relevant modules from the NLTK library
from nltk.sentiment.vader import SentimentIntensityAnalyzer
sentence_tokenizer = nltk.data.load('tokenizers/punkt/english.pickle')
word_tokenizer = nltk.tokenize.RegexpTokenizer(r'\w+')
##tt = ToktokTokenizer()#keeps the url as is...................................................
from nltk.tokenize import TweetTokenizer
tt = TweetTokenizer()
Porter_stemmer = PorterStemmer()
Lancaster_stemmer = LancasterStemmer()
WordNet_lemmatizer = WordNetLemmatizer()
#nltk.download('averaged_perceptron_tagger')
##Train_path="D:/Data/PAN2015-Profiling/pan15-author-profiling-training-dataset-english-2015-04-23"#path for train data
##Test_path="D:/Data/PAN2015-Profiling/pan-ap2015-test/en" #path for test data

##Train_path="D:/NLP/PAN2014/pan14-author-profiling-training-corpus-english-twitter-2014-04-16"#path for train data D:\Data\PAN2013-Profiling\Training\en  D:\NLP\PAN2014
##train_truth_path="D:/NLP/PAN2014/truth_train"
##
##test_truth_path="D:/NLP/PAN2014/pan14_test"
##Test_path="D:/NLP/PAN2014/pan14_test/en"
##Content = open('results_of_clssification_all.txt', 'w')

M_train_file= "/Users/catherine/Desktop/NLP/PAN Datasets/PAN2014blog/2014.1-blog-txtfiles/PAN2014_blog_male_train.1.txt"
F_train_file= "/Users/catherine/Desktop/NLP/PAN Datasets/PAN2014blog/2014.1-blog-txtfiles/PAN2014_blog_female_train.1.txt"

M_test_file= "/Users/catherine/Desktop/NLP/PAN Datasets/PAN2014blog/2014.1-blog-txtfiles/PAN2014_blog_male_test.1.txt"
F_test_file= "/Users/catherine/Desktop/NLP/PAN Datasets/PAN2014blog/2014.1-blog-txtfiles/PAN2014_blog_female_test.1.txt"

def all_txt(M_txt_file,  F_txt_file):
    with open(M_txt_file, mode="r", encoding="utf-8") as f:
            txt_list = [(line[:-1]).split("\t") for line in f]
            male_dict = dict()
            content=[]
            for line in txt_list:
                if line[1] in male_dict:
                # append the new number to the existing array at this slot
                    male_dict[line[1]].append(line[3])
                else:
                # create a new array in this slot
                    male_dict[line[1]] = [line[3]]
    male_content= [("M", (' '.join(a)).split()) for a in list(male_dict.values())]
  
    with open(F_txt_file, mode="r", encoding="utf-8") as f:
            txt_list = [(line[:-1]).split("\t") for line in f]
            female_dict = dict()
            content=[]
            for line in txt_list:
                if line[1] in female_dict:
                # append the new number to the existing array at this slot
                    female_dict[line[1]].append(line[3])
                else:
                # create a new array in this slot
                    female_dict[line[1]] = [line[3]]
    female_content= [("F", (' '.join(a)).split()) for a in list(female_dict.values())]
    all_txt_per_person = male_content + female_content
    return all_txt_per_person

##print( [((all_txt(M_txt_file,  F_txt_file))[i][0])  for i in range(len((all_txt(M_txt_file,  F_txt_file))))]) helps to determine the gender....

      
"""
create feature vector from the training set.....................................................................................
"""

def WordFeatures(word_list, all_training_text):

    fvs_words = np.array([[author.count(word) for word in word_list] for author in all_training_text]).astype(np.float64)

    # normalise by dividing each row by number of tokens for each author........
    fvs_words /= np.c_[np.array([len(author) for author in all_training_text])]

    return fvs_words



"""
From the saved csv file, recover the saved features to be used...............................................................
"""
import csv
word_list=[]
##txt_files =[ "/Users/catherine/Desktop/NLP/PAN Datasets/PAN_chosen_tokens/2014/output2014OR.1M.txt", "/Users/catherine/Desktop/NLP/PAN Datasets/PAN_chosen_tokens/2014/output2014OR.2F.txt"]
txt_files =[ "/Users/catherine/Desktop/NLP/PAN Datasets/PAN2014blog/2014.1-blog-txtfiles/PAN2014_blog_female_chosen_set.1.csv", "/Users/catherine/Desktop/NLP/PAN Datasets/PAN2014blog/2014.1-blog-txtfiles/PAN2014_blog_male_chosen_set.1.csv"]
print("txt_files of features used",txt_files)
##for txt_file in txt_files:
##    with open(txt_file, mode="r", encoding="utf-8") as f:
##        reader = csv.reader(f, delimiter=",")
##        next(reader) # skip header
##        word_list1 =  ([(r.split(":"))[1] for r in f])
##        print(txt_file, len(word_list1))
##        word_list = word_list + word_list1
##print("length of features used =", len(word_list))

for txt_file in txt_files:
    with open(txt_file, mode="r", encoding="utf-8") as f:
        reader = csv.reader(f, delimiter=",")
        next(reader) # skip header
        word_list1 =  [r[0] for r in reader]
        print(txt_file, len(word_list1))
        word_list = word_list + word_list1
print("length of features used =", len(word_list))


"""
Prepare the training and test sets to be parsed to the classifies............................................................
"""
A=(all_txt(M_train_file,  F_train_file))
all_training_text = ( [(A[i][1])  for i in range(len(A))])
print("len(all_training_text)",len(all_training_text))
X_train=WordFeatures(word_list, all_training_text)
y_train=np.array( [(A[i][0])  for i in range(len(A))])
print("len(X_train)=",len(X_train), "len(y_train)=", len(y_train))

B=(all_txt(M_test_file,  F_test_file))
all_test_text = ( [(B[i][1])  for i in range(len(B))])
print("len(all_test_text)",len(all_test_text))
X_test = WordFeatures(word_list, all_test_text)
y_test = np.array( [(B[i][0])  for i in range(len(B))])
print("len(X_test=",len(X_test), "len(y_test)=",len(y_test))


X_train = np.nan_to_num(X_train)
X_test = np.nan_to_num(X_test)

"""
Perform feature selection using chi2............................................................
"""
from sklearn.feature_selection import *
##
####def select_Bestfeatures(X, y):
print("selector = SelectKBest(score_func=mutual_info_classif, k=300)")
selector = SelectKBest(score_func=mutual_info_classif, k=300)
fit = selector.fit(X_train,y_train)
train_selected_features = fit.transform(X_train)
test_selected_features = fit.transform(X_test)



"""
x_train and x_test based on the new features using chi2............................................................
"""
##X_train = select_Bestfeatures(X_train, y_train)
##X_test = select_Bestfeatures(X_test, y_test)


"""
Perform feature selection using f_classif............................................................
"""
from sklearn.feature_selection import *
##def f_classif_select_Bestfeatures(X, y):
##selector=SelectKBest(score_func=f_classif,k=500)
##fit = selector.fit(X_train,y_train)
##train_selected_features = fit.transform(X_train)
##test_selected_features = fit.transform(X_test)
##    selected_features = fit.transform(X)
##    return selected_features

"""
x_train and x_test based on the new features using f_classif............................................................
"""
##X_train = f_classif_select_Bestfeatures(X_train, y_train)
##X_test = f_classif_select_Bestfeatures(X_test, y_test)



"""
Perform feature selection using mutual_info_classif............................................................
"""
from sklearn.feature_selection import *
##def mutual_info_classif_select_Bestfeatures(X_tr, X_te, y):
##selector=SelectKBest(score_func=mutual_info_classif,k=100)
##fit = selector.fit(X_train,y_train)
##train_selected_features = fit.transform(X_train)
##test_selected_features = fit.transform(X_test)
##    return train_selected_features, test_selected_features

"""
x_train and x_test based on the new features using f_classif............................................................
"""
X_train =train_selected_features 
X_test = test_selected_features


"""
Perform feature selection using pca............................................................
"""

##from sklearn.decomposition import PCA
##def PCA_select_Bestfeatures(X):
##    # feature extraction
##    pca = PCA(n_components=200)
##    fit = pca.fit(X, y)
##    Train_selected_features = fit.transform(X)
##    return selected_features

"""
x_train and x_test based on the new features using f_classif............................................................
"""
##X_train = PCA_select_Bestfeatures(X_train)
##X_test = PCA_select_Bestfeatures(X_test)

"""
Below are a list of Classification Algorithms with different distance measres ..............................................................................
"""
"""
K-Nearest Neighbours (average aa over all the nearest neighbour selectiopn)..............................................................................
"""

k=5

print(k)
"""
8. manhattan K-Nearest Neighbours..............................................................................
"""
from sklearn.neighbors import KNeighborsClassifier
knn = KNeighborsClassifier(n_neighbors = k, metric= "manhattan")
knn.fit(X_train, y_train)
y_pred = knn.predict(X_test)
print("manhattan K-Nearest Neighbours")
print(metrics.accuracy_score(y_test, y_pred))

    
"""
8. cosine K-Nearest Neighbours..............................................................................
"""
from sklearn.neighbors import KNeighborsClassifier
knn = KNeighborsClassifier(n_neighbors = k, metric= "cosine")
knn.fit(X_train, y_train)
y_pred = knn.predict(X_test)
print("cosine K-Nearest Neighbours")
print(metrics.accuracy_score(y_test, y_pred))


"""
8. euclidean K-Nearest Neighbours..............................................................................
"""
from sklearn.neighbors import KNeighborsClassifier
knn = KNeighborsClassifier(n_neighbors = k, metric= "euclidean")
knn.fit(X_train, y_train)
y_pred = knn.predict(X_test)
print("euclidean K-Nearest Neighbours")
print(metrics.accuracy_score(y_test, y_pred))

        
"""
9. scipy.spatial.distance K-Nearest Neighbours..............................................................................
"""
from scipy.spatial.distance import *
knn = KNeighborsClassifier(n_neighbors = k, metric= "braycurtis")
knn.fit(X_train, y_train)
y_pred = knn.predict(X_test)
print("braycurtis K-Nearest Neighbours")
print(metrics.accuracy_score(y_test, y_pred))
       
        
knn = KNeighborsClassifier(n_neighbors = k, metric= "canberra")
knn.fit(X_train, y_train)
y_pred = knn.predict(X_test)
print("canberra K-Nearest Neighbours")
print(metrics.accuracy_score(y_test, y_pred))
  
        
knn = KNeighborsClassifier(n_neighbors = k, metric= "jaccard")
knn.fit(X_train, y_train)
y_pred = knn.predict(X_test)
print("jaccard K-Nearest Neighbours")
print(metrics.accuracy_score(y_test, y_pred))
        
knn = KNeighborsClassifier(n_neighbors = k, metric= "yule")
knn.fit(X_train, y_train)
y_pred = knn.predict(X_test)
print("yule K-Nearest Neighbours")
print(metrics.accuracy_score(y_test, y_pred))
print("************************************ ****************************")





txt_files of features used ['/Users/catherine/Desktop/NLP/PAN Datasets/PAN2014blog/2014.1-blog-txtfiles/PAN2014_blog_female_chosen_set.1.csv', '/Users/catherine/Desktop/NLP/PAN Datasets/PAN2014blog/2014.1-blog-txtfiles/PAN2014_blog_male_chosen_set.1.csv']
/Users/catherine/Desktop/NLP/PAN Datasets/PAN2014blog/2014.1-blog-txtfiles/PAN2014_blog_female_chosen_set.1.csv 1526
/Users/catherine/Desktop/NLP/PAN Datasets/PAN2014blog/2014.1-blog-txtfiles/PAN2014_blog_male_chosen_set.1.csv 2063
length of features used = 3589
len(all_training_text) 147
len(X_train)= 147 len(y_train)= 147
len(all_test_text) 78
len(X_test= 78 len(y_test)= 78
selector = SelectKBest(score_func=mutual_info_classif, k=300)
5
manhattan K-Nearest Neighbours
0.5384615384615384
cosine K-Nearest Neighbours
0.6025641025641025
euclidean K-Nearest Neighbours
0.5897435897435898
braycurtis K-Nearest Neighbours
0.5769230769230769
canberra K-Nearest Neighbours
0.5
jaccard K-Nearest Neighbours
0.5897435897435898
yule K-Nearest Neighb

/anaconda3/lib/python3.7/site-packages/sklearn/utils/validation.py:475: DataConversionWarning: Data with input dtype float64 was converted to bool by check_pairwise_arrays.
  warnings.warn(msg, DataConversionWarning)


In [33]:
import sys
sys.path.append("/anaconda3/lib/python3.7/site-packages")
import numpy as np
import numpy
import nltk
import pickle
import pandas as pd
import glob
import os
import re
import operator
from collections import Counter
#from read_xml_files import *
##from s_stemmer import *
from nltk.stem.porter import PorterStemmer
from nltk.stem import LancasterStemmer, WordNetLemmatizer
from itertools import groupby
from nltk.collocations import *
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.cluster import KMeans
from scipy.cluster.vq import whiten
from nltk.tokenize.toktok import ToktokTokenizer
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2
import xml.etree.ElementTree as ET
import pandas as pd
#Import scikit-learn metrics module for accuracy calculation
from sklearn import metrics
##from pattern.text.en import singularize
##nltk.download('vader_lexicon')
##nltk.download('punkt')
#import the relevant modules from the NLTK library
from nltk.sentiment.vader import SentimentIntensityAnalyzer
sentence_tokenizer = nltk.data.load('tokenizers/punkt/english.pickle')
word_tokenizer = nltk.tokenize.RegexpTokenizer(r'\w+')
##tt = ToktokTokenizer()#keeps the url as is...................................................
from nltk.tokenize import TweetTokenizer
tt = TweetTokenizer()
Porter_stemmer = PorterStemmer()
Lancaster_stemmer = LancasterStemmer()
WordNet_lemmatizer = WordNetLemmatizer()
#nltk.download('averaged_perceptron_tagger')
##Train_path="D:/Data/PAN2015-Profiling/pan15-author-profiling-training-dataset-english-2015-04-23"#path for train data
##Test_path="D:/Data/PAN2015-Profiling/pan-ap2015-test/en" #path for test data

##Train_path="D:/NLP/PAN2014/pan14-author-profiling-training-corpus-english-twitter-2014-04-16"#path for train data D:\Data\PAN2013-Profiling\Training\en  D:\NLP\PAN2014
##train_truth_path="D:/NLP/PAN2014/truth_train"
##
##test_truth_path="D:/NLP/PAN2014/pan14_test"
##Test_path="D:/NLP/PAN2014/pan14_test/en"
##Content = open('results_of_clssification_all.txt', 'w')

M_train_file= "/Users/catherine/Desktop/NLP/PAN Datasets/PAN2014blog/2014.1-blog-txtfiles/PAN2014_blog_male_train.1.txt"
F_train_file= "/Users/catherine/Desktop/NLP/PAN Datasets/PAN2014blog/2014.1-blog-txtfiles/PAN2014_blog_female_train.1.txt"

M_test_file= "/Users/catherine/Desktop/NLP/PAN Datasets/PAN2014blog/2014.1-blog-txtfiles/PAN2014_blog_male_test.1.txt"
F_test_file= "/Users/catherine/Desktop/NLP/PAN Datasets/PAN2014blog/2014.1-blog-txtfiles/PAN2014_blog_female_test.1.txt"

def all_txt(M_txt_file,  F_txt_file):
    with open(M_txt_file, mode="r", encoding="utf-8") as f:
            txt_list = [(line[:-1]).split("\t") for line in f]
            male_dict = dict()
            content=[]
            for line in txt_list:
                if line[1] in male_dict:
                # append the new number to the existing array at this slot
                    male_dict[line[1]].append(line[3])
                else:
                # create a new array in this slot
                    male_dict[line[1]] = [line[3]]
    male_content= [("M", (' '.join(a)).split()) for a in list(male_dict.values())]
  
    with open(F_txt_file, mode="r", encoding="utf-8") as f:
            txt_list = [(line[:-1]).split("\t") for line in f]
            female_dict = dict()
            content=[]
            for line in txt_list:
                if line[1] in female_dict:
                # append the new number to the existing array at this slot
                    female_dict[line[1]].append(line[3])
                else:
                # create a new array in this slot
                    female_dict[line[1]] = [line[3]]
    female_content= [("F", (' '.join(a)).split()) for a in list(female_dict.values())]
    all_txt_per_person = male_content + female_content
    return all_txt_per_person

##print( [((all_txt(M_txt_file,  F_txt_file))[i][0])  for i in range(len((all_txt(M_txt_file,  F_txt_file))))]) helps to determine the gender....

      
"""
create feature vector from the training set.....................................................................................
"""

def WordFeatures(word_list, all_training_text):

    fvs_words = np.array([[author.count(word) for word in word_list] for author in all_training_text]).astype(np.float64)

    # normalise by dividing each row by number of tokens for each author........
    fvs_words /= np.c_[np.array([len(author) for author in all_training_text])]

    return fvs_words



"""
From the saved csv file, recover the saved features to be used...............................................................
"""
import csv
word_list=[]
##txt_files =[ "/Users/catherine/Desktop/NLP/PAN Datasets/PAN_chosen_tokens/2014/output2014OR.1M.txt", "/Users/catherine/Desktop/NLP/PAN Datasets/PAN_chosen_tokens/2014/output2014OR.2F.txt"]
txt_files =[ "/Users/catherine/Desktop/NLP/PAN Datasets/PAN2014blog/2014.1-blog-txtfiles/PAN2014_blog_female_chosen_set.1.csv", "/Users/catherine/Desktop/NLP/PAN Datasets/PAN2014blog/2014.1-blog-txtfiles/PAN2014_blog_male_chosen_set.1.csv"]
print("txt_files of features used",txt_files)
##for txt_file in txt_files:
##    with open(txt_file, mode="r", encoding="utf-8") as f:
##        reader = csv.reader(f, delimiter=",")
##        next(reader) # skip header
##        word_list1 =  ([(r.split(":"))[1] for r in f])
##        print(txt_file, len(word_list1))
##        word_list = word_list + word_list1
##print("length of features used =", len(word_list))

for txt_file in txt_files:
    with open(txt_file, mode="r", encoding="utf-8") as f:
        reader = csv.reader(f, delimiter=",")
        next(reader) # skip header
        word_list1 =  [r[0] for r in reader]
        print(txt_file, len(word_list1))
        word_list = word_list + word_list1
print("length of features used =", len(word_list))


"""
Prepare the training and test sets to be parsed to the classifies............................................................
"""
A=(all_txt(M_train_file,  F_train_file))
all_training_text = ( [(A[i][1])  for i in range(len(A))])
print("len(all_training_text)",len(all_training_text))
X_train=WordFeatures(word_list, all_training_text)
y_train=np.array( [(A[i][0])  for i in range(len(A))])
print("len(X_train)=",len(X_train), "len(y_train)=", len(y_train))

B=(all_txt(M_test_file,  F_test_file))
all_test_text = ( [(B[i][1])  for i in range(len(B))])
print("len(all_test_text)",len(all_test_text))
X_test = WordFeatures(word_list, all_test_text)
y_test = np.array( [(B[i][0])  for i in range(len(B))])
print("len(X_test=",len(X_test), "len(y_test)=",len(y_test))


X_train = np.nan_to_num(X_train)
X_test = np.nan_to_num(X_test)

"""
Perform feature selection using chi2............................................................
"""
from sklearn.feature_selection import *
##
####def select_Bestfeatures(X, y):
print("selector = SelectKBest(score_func=mutual_info_classif, k=500)")
selector = SelectKBest(score_func=mutual_info_classif, k=500)
fit = selector.fit(X_train,y_train)
train_selected_features = fit.transform(X_train)
test_selected_features = fit.transform(X_test)



"""
x_train and x_test based on the new features using chi2............................................................
"""
##X_train = select_Bestfeatures(X_train, y_train)
##X_test = select_Bestfeatures(X_test, y_test)


"""
Perform feature selection using f_classif............................................................
"""
from sklearn.feature_selection import *
##def f_classif_select_Bestfeatures(X, y):
##selector=SelectKBest(score_func=f_classif,k=500)
##fit = selector.fit(X_train,y_train)
##train_selected_features = fit.transform(X_train)
##test_selected_features = fit.transform(X_test)
##    selected_features = fit.transform(X)
##    return selected_features

"""
x_train and x_test based on the new features using f_classif............................................................
"""
##X_train = f_classif_select_Bestfeatures(X_train, y_train)
##X_test = f_classif_select_Bestfeatures(X_test, y_test)



"""
Perform feature selection using mutual_info_classif............................................................
"""
from sklearn.feature_selection import *
##def mutual_info_classif_select_Bestfeatures(X_tr, X_te, y):
##selector=SelectKBest(score_func=mutual_info_classif,k=100)
##fit = selector.fit(X_train,y_train)
##train_selected_features = fit.transform(X_train)
##test_selected_features = fit.transform(X_test)
##    return train_selected_features, test_selected_features

"""
x_train and x_test based on the new features using f_classif............................................................
"""
X_train =train_selected_features 
X_test = test_selected_features


"""
Perform feature selection using pca............................................................
"""

##from sklearn.decomposition import PCA
##def PCA_select_Bestfeatures(X):
##    # feature extraction
##    pca = PCA(n_components=200)
##    fit = pca.fit(X, y)
##    Train_selected_features = fit.transform(X)
##    return selected_features

"""
x_train and x_test based on the new features using f_classif............................................................
"""
##X_train = PCA_select_Bestfeatures(X_train)
##X_test = PCA_select_Bestfeatures(X_test)

"""
Below are a list of Classification Algorithms with different distance measres ..............................................................................
"""
"""
K-Nearest Neighbours (average aa over all the nearest neighbour selectiopn)..............................................................................
"""

k=5

print(k)
"""
8. manhattan K-Nearest Neighbours..............................................................................
"""
from sklearn.neighbors import KNeighborsClassifier
knn = KNeighborsClassifier(n_neighbors = k, metric= "manhattan")
knn.fit(X_train, y_train)
y_pred = knn.predict(X_test)
print("manhattan K-Nearest Neighbours")
print(metrics.accuracy_score(y_test, y_pred))

    
"""
8. cosine K-Nearest Neighbours..............................................................................
"""
from sklearn.neighbors import KNeighborsClassifier
knn = KNeighborsClassifier(n_neighbors = k, metric= "cosine")
knn.fit(X_train, y_train)
y_pred = knn.predict(X_test)
print("cosine K-Nearest Neighbours")
print(metrics.accuracy_score(y_test, y_pred))


"""
8. euclidean K-Nearest Neighbours..............................................................................
"""
from sklearn.neighbors import KNeighborsClassifier
knn = KNeighborsClassifier(n_neighbors = k, metric= "euclidean")
knn.fit(X_train, y_train)
y_pred = knn.predict(X_test)
print("euclidean K-Nearest Neighbours")
print(metrics.accuracy_score(y_test, y_pred))

        
"""
9. scipy.spatial.distance K-Nearest Neighbours..............................................................................
"""
from scipy.spatial.distance import *
knn = KNeighborsClassifier(n_neighbors = k, metric= "braycurtis")
knn.fit(X_train, y_train)
y_pred = knn.predict(X_test)
print("braycurtis K-Nearest Neighbours")
print(metrics.accuracy_score(y_test, y_pred))
       
        
knn = KNeighborsClassifier(n_neighbors = k, metric= "canberra")
knn.fit(X_train, y_train)
y_pred = knn.predict(X_test)
print("canberra K-Nearest Neighbours")
print(metrics.accuracy_score(y_test, y_pred))
  
        
knn = KNeighborsClassifier(n_neighbors = k, metric= "jaccard")
knn.fit(X_train, y_train)
y_pred = knn.predict(X_test)
print("jaccard K-Nearest Neighbours")
print(metrics.accuracy_score(y_test, y_pred))
        
knn = KNeighborsClassifier(n_neighbors = k, metric= "yule")
knn.fit(X_train, y_train)
y_pred = knn.predict(X_test)
print("yule K-Nearest Neighbours")
print(metrics.accuracy_score(y_test, y_pred))
print("************************************ ****************************")

txt_files of features used ['/Users/catherine/Desktop/NLP/PAN Datasets/PAN2014blog/2014.1-blog-txtfiles/PAN2014_blog_female_chosen_set.1.csv', '/Users/catherine/Desktop/NLP/PAN Datasets/PAN2014blog/2014.1-blog-txtfiles/PAN2014_blog_male_chosen_set.1.csv']
/Users/catherine/Desktop/NLP/PAN Datasets/PAN2014blog/2014.1-blog-txtfiles/PAN2014_blog_female_chosen_set.1.csv 1526
/Users/catherine/Desktop/NLP/PAN Datasets/PAN2014blog/2014.1-blog-txtfiles/PAN2014_blog_male_chosen_set.1.csv 2063
length of features used = 3589
len(all_training_text) 147
len(X_train)= 147 len(y_train)= 147
len(all_test_text) 78
len(X_test= 78 len(y_test)= 78
selector = SelectKBest(score_func=mutual_info_classif, k=500)
5
manhattan K-Nearest Neighbours
0.6153846153846154
cosine K-Nearest Neighbours
0.6025641025641025
euclidean K-Nearest Neighbours
0.5641025641025641
braycurtis K-Nearest Neighbours
0.6410256410256411
canberra K-Nearest Neighbours
0.5512820512820513
jaccard K-Nearest Neighbours
0.6794871794871795
yule K

/anaconda3/lib/python3.7/site-packages/sklearn/utils/validation.py:475: DataConversionWarning: Data with input dtype float64 was converted to bool by check_pairwise_arrays.
  warnings.warn(msg, DataConversionWarning)


In [34]:
import sys
sys.path.append("/anaconda3/lib/python3.7/site-packages")
import numpy as np
import numpy
import nltk
import pickle
import pandas as pd
import glob
import os
import re
import operator
from collections import Counter
#from read_xml_files import *
##from s_stemmer import *
from nltk.stem.porter import PorterStemmer
from nltk.stem import LancasterStemmer, WordNetLemmatizer
from itertools import groupby
from nltk.collocations import *
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.cluster import KMeans
from scipy.cluster.vq import whiten
from nltk.tokenize.toktok import ToktokTokenizer
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2
import xml.etree.ElementTree as ET
import pandas as pd
#Import scikit-learn metrics module for accuracy calculation
from sklearn import metrics
##from pattern.text.en import singularize
##nltk.download('vader_lexicon')
##nltk.download('punkt')
#import the relevant modules from the NLTK library
from nltk.sentiment.vader import SentimentIntensityAnalyzer
sentence_tokenizer = nltk.data.load('tokenizers/punkt/english.pickle')
word_tokenizer = nltk.tokenize.RegexpTokenizer(r'\w+')
##tt = ToktokTokenizer()#keeps the url as is...................................................
from nltk.tokenize import TweetTokenizer
tt = TweetTokenizer()
Porter_stemmer = PorterStemmer()
Lancaster_stemmer = LancasterStemmer()
WordNet_lemmatizer = WordNetLemmatizer()
#nltk.download('averaged_perceptron_tagger')
##Train_path="D:/Data/PAN2015-Profiling/pan15-author-profiling-training-dataset-english-2015-04-23"#path for train data
##Test_path="D:/Data/PAN2015-Profiling/pan-ap2015-test/en" #path for test data

##Train_path="D:/NLP/PAN2014/pan14-author-profiling-training-corpus-english-twitter-2014-04-16"#path for train data D:\Data\PAN2013-Profiling\Training\en  D:\NLP\PAN2014
##train_truth_path="D:/NLP/PAN2014/truth_train"
##
##test_truth_path="D:/NLP/PAN2014/pan14_test"
##Test_path="D:/NLP/PAN2014/pan14_test/en"
##Content = open('results_of_clssification_all.txt', 'w')

M_train_file= "/Users/catherine/Desktop/NLP/PAN Datasets/PAN2014blog/2014.1-blog-txtfiles/PAN2014_blog_male_train.1.txt"
F_train_file= "/Users/catherine/Desktop/NLP/PAN Datasets/PAN2014blog/2014.1-blog-txtfiles/PAN2014_blog_female_train.1.txt"

M_test_file= "/Users/catherine/Desktop/NLP/PAN Datasets/PAN2014blog/2014.1-blog-txtfiles/PAN2014_blog_male_test.1.txt"
F_test_file= "/Users/catherine/Desktop/NLP/PAN Datasets/PAN2014blog/2014.1-blog-txtfiles/PAN2014_blog_female_test.1.txt"

def all_txt(M_txt_file,  F_txt_file):
    with open(M_txt_file, mode="r", encoding="utf-8") as f:
            txt_list = [(line[:-1]).split("\t") for line in f]
            male_dict = dict()
            content=[]
            for line in txt_list:
                if line[1] in male_dict:
                # append the new number to the existing array at this slot
                    male_dict[line[1]].append(line[3])
                else:
                # create a new array in this slot
                    male_dict[line[1]] = [line[3]]
    male_content= [("M", (' '.join(a)).split()) for a in list(male_dict.values())]
  
    with open(F_txt_file, mode="r", encoding="utf-8") as f:
            txt_list = [(line[:-1]).split("\t") for line in f]
            female_dict = dict()
            content=[]
            for line in txt_list:
                if line[1] in female_dict:
                # append the new number to the existing array at this slot
                    female_dict[line[1]].append(line[3])
                else:
                # create a new array in this slot
                    female_dict[line[1]] = [line[3]]
    female_content= [("F", (' '.join(a)).split()) for a in list(female_dict.values())]
    all_txt_per_person = male_content + female_content
    return all_txt_per_person

##print( [((all_txt(M_txt_file,  F_txt_file))[i][0])  for i in range(len((all_txt(M_txt_file,  F_txt_file))))]) helps to determine the gender....

      
"""
create feature vector from the training set.....................................................................................
"""

def WordFeatures(word_list, all_training_text):

    fvs_words = np.array([[author.count(word) for word in word_list] for author in all_training_text]).astype(np.float64)

    # normalise by dividing each row by number of tokens for each author........
    fvs_words /= np.c_[np.array([len(author) for author in all_training_text])]

    return fvs_words



"""
From the saved csv file, recover the saved features to be used...............................................................
"""
import csv
word_list=[]
##txt_files =[ "/Users/catherine/Desktop/NLP/PAN Datasets/PAN_chosen_tokens/2014/output2014OR.1M.txt", "/Users/catherine/Desktop/NLP/PAN Datasets/PAN_chosen_tokens/2014/output2014OR.2F.txt"]
txt_files =[ "/Users/catherine/Desktop/NLP/PAN Datasets/PAN2014blog/2014.1-blog-txtfiles/PAN2014_blog_female_vocubulary.1.csv", "/Users/catherine/Desktop/NLP/PAN Datasets/PAN2014blog/2014.1-blog-txtfiles/PAN2014_blog_male_ vocubulary_set.1.csv"]
print("txt_files of features used",txt_files)
##for txt_file in txt_files:
##    with open(txt_file, mode="r", encoding="utf-8") as f:
##        reader = csv.reader(f, delimiter=",")
##        next(reader) # skip header
##        word_list1 =  ([(r.split(":"))[1] for r in f])
##        print(txt_file, len(word_list1))
##        word_list = word_list + word_list1
##print("length of features used =", len(word_list))

for txt_file in txt_files:
    with open(txt_file, mode="r", encoding="utf-8") as f:
        reader = csv.reader(f, delimiter=",")
        next(reader) # skip header
        word_list1 =  [r[0] for r in reader]
        print(txt_file, len(word_list1))
        word_list = word_list + word_list1
print("length of features used =", len(word_list))


"""
Prepare the training and test sets to be parsed to the classifies............................................................
"""
A=(all_txt(M_train_file,  F_train_file))
all_training_text = ( [(A[i][1])  for i in range(len(A))])
print("len(all_training_text)",len(all_training_text))
X_train=WordFeatures(word_list, all_training_text)
y_train=np.array( [(A[i][0])  for i in range(len(A))])
print("len(X_train)=",len(X_train), "len(y_train)=", len(y_train))

B=(all_txt(M_test_file,  F_test_file))
all_test_text = ( [(B[i][1])  for i in range(len(B))])
print("len(all_test_text)",len(all_test_text))
X_test = WordFeatures(word_list, all_test_text)
y_test = np.array( [(B[i][0])  for i in range(len(B))])
print("len(X_test=",len(X_test), "len(y_test)=",len(y_test))


X_train = np.nan_to_num(X_train)
X_test = np.nan_to_num(X_test)

"""
Perform feature selection using chi2............................................................
"""
from sklearn.feature_selection import *
##
####def select_Bestfeatures(X, y):
#print("selector = SelectKBest(score_func=mutual_info_classif, k=500)")
#selector = SelectKBest(score_func=mutual_info_classif, k=500)
#fit = selector.fit(X_train,y_train)
#train_selected_features = fit.transform(X_train)
#test_selected_features = fit.transform(X_test)



"""
x_train and x_test based on the new features using chi2............................................................
"""
##X_train = select_Bestfeatures(X_train, y_train)
##X_test = select_Bestfeatures(X_test, y_test)


"""
Perform feature selection using f_classif............................................................
"""
from sklearn.feature_selection import *
##def f_classif_select_Bestfeatures(X, y):
##selector=SelectKBest(score_func=f_classif,k=500)
##fit = selector.fit(X_train,y_train)
##train_selected_features = fit.transform(X_train)
##test_selected_features = fit.transform(X_test)
##    selected_features = fit.transform(X)
##    return selected_features

"""
x_train and x_test based on the new features using f_classif............................................................
"""
##X_train = f_classif_select_Bestfeatures(X_train, y_train)
##X_test = f_classif_select_Bestfeatures(X_test, y_test)



"""
Perform feature selection using mutual_info_classif............................................................
"""
from sklearn.feature_selection import *
##def mutual_info_classif_select_Bestfeatures(X_tr, X_te, y):
##selector=SelectKBest(score_func=mutual_info_classif,k=100)
##fit = selector.fit(X_train,y_train)
##train_selected_features = fit.transform(X_train)
##test_selected_features = fit.transform(X_test)
##    return train_selected_features, test_selected_features

"""
x_train and x_test based on the new features using f_classif............................................................
"""
#X_train =train_selected_features 
#X_test = test_selected_features


"""
Perform feature selection using pca............................................................
"""

##from sklearn.decomposition import PCA
##def PCA_select_Bestfeatures(X):
##    # feature extraction
##    pca = PCA(n_components=200)
##    fit = pca.fit(X, y)
##    Train_selected_features = fit.transform(X)
##    return selected_features

"""
x_train and x_test based on the new features using f_classif............................................................
"""
##X_train = PCA_select_Bestfeatures(X_train)
##X_test = PCA_select_Bestfeatures(X_test)

"""
Below are a list of Classification Algorithms with different distance measres ..............................................................................
"""
"""
K-Nearest Neighbours (average aa over all the nearest neighbour selectiopn)..............................................................................
"""

k=5

print(k)
"""
8. manhattan K-Nearest Neighbours..............................................................................
"""
from sklearn.neighbors import KNeighborsClassifier
knn = KNeighborsClassifier(n_neighbors = k, metric= "manhattan")
knn.fit(X_train, y_train)
y_pred = knn.predict(X_test)
print("manhattan K-Nearest Neighbours")
print(metrics.accuracy_score(y_test, y_pred))

    
"""
8. cosine K-Nearest Neighbours..............................................................................
"""
from sklearn.neighbors import KNeighborsClassifier
knn = KNeighborsClassifier(n_neighbors = k, metric= "cosine")
knn.fit(X_train, y_train)
y_pred = knn.predict(X_test)
print("cosine K-Nearest Neighbours")
print(metrics.accuracy_score(y_test, y_pred))


"""
8. euclidean K-Nearest Neighbours..............................................................................
"""
from sklearn.neighbors import KNeighborsClassifier
knn = KNeighborsClassifier(n_neighbors = k, metric= "euclidean")
knn.fit(X_train, y_train)
y_pred = knn.predict(X_test)
print("euclidean K-Nearest Neighbours")
print(metrics.accuracy_score(y_test, y_pred))

        
"""
9. scipy.spatial.distance K-Nearest Neighbours..............................................................................
"""
from scipy.spatial.distance import *
knn = KNeighborsClassifier(n_neighbors = k, metric= "braycurtis")
knn.fit(X_train, y_train)
y_pred = knn.predict(X_test)
print("braycurtis K-Nearest Neighbours")
print(metrics.accuracy_score(y_test, y_pred))
       
        
knn = KNeighborsClassifier(n_neighbors = k, metric= "canberra")
knn.fit(X_train, y_train)
y_pred = knn.predict(X_test)
print("canberra K-Nearest Neighbours")
print(metrics.accuracy_score(y_test, y_pred))
  
        
knn = KNeighborsClassifier(n_neighbors = k, metric= "jaccard")
knn.fit(X_train, y_train)
y_pred = knn.predict(X_test)
print("jaccard K-Nearest Neighbours")
print(metrics.accuracy_score(y_test, y_pred))
        
knn = KNeighborsClassifier(n_neighbors = k, metric= "yule")
knn.fit(X_train, y_train)
y_pred = knn.predict(X_test)
print("yule K-Nearest Neighbours")
print(metrics.accuracy_score(y_test, y_pred))
print("************************************ ****************************")

txt_files of features used ['/Users/catherine/Desktop/NLP/PAN Datasets/PAN2014blog/2014.1-blog-txtfiles/PAN2014_blog_female_vocubulary.1.csv', '/Users/catherine/Desktop/NLP/PAN Datasets/PAN2014blog/2014.1-blog-txtfiles/PAN2014_blog_male_ vocubulary_set.1.csv']
/Users/catherine/Desktop/NLP/PAN Datasets/PAN2014blog/2014.1-blog-txtfiles/PAN2014_blog_female_vocubulary.1.csv 12375
/Users/catherine/Desktop/NLP/PAN Datasets/PAN2014blog/2014.1-blog-txtfiles/PAN2014_blog_male_ vocubulary_set.1.csv 18889
length of features used = 31264
len(all_training_text) 147
len(X_train)= 147 len(y_train)= 147
len(all_test_text) 78
len(X_test= 78 len(y_test)= 78
5
manhattan K-Nearest Neighbours
0.6666666666666666
cosine K-Nearest Neighbours
0.6538461538461539
euclidean K-Nearest Neighbours
0.6410256410256411
braycurtis K-Nearest Neighbours
0.6410256410256411
canberra K-Nearest Neighbours
0.5128205128205128
jaccard K-Nearest Neighbours
0.5256410256410257


/anaconda3/lib/python3.7/site-packages/sklearn/utils/validation.py:475: DataConversionWarning: Data with input dtype float64 was converted to bool by check_pairwise_arrays.
  warnings.warn(msg, DataConversionWarning)


yule K-Nearest Neighbours
0.5897435897435898
************************************ ****************************


In [35]:
import sys
sys.path.append("/anaconda3/lib/python3.7/site-packages")
import numpy as np
import numpy
import nltk
import pickle
import pandas as pd
import glob
import os
import re
import operator
from collections import Counter
#from read_xml_files import *
##from s_stemmer import *
from nltk.stem.porter import PorterStemmer
from nltk.stem import LancasterStemmer, WordNetLemmatizer
from itertools import groupby
from nltk.collocations import *
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.cluster import KMeans
from scipy.cluster.vq import whiten
from nltk.tokenize.toktok import ToktokTokenizer
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2
import xml.etree.ElementTree as ET
import pandas as pd
#Import scikit-learn metrics module for accuracy calculation
from sklearn import metrics
##from pattern.text.en import singularize
##nltk.download('vader_lexicon')
##nltk.download('punkt')
#import the relevant modules from the NLTK library
from nltk.sentiment.vader import SentimentIntensityAnalyzer
sentence_tokenizer = nltk.data.load('tokenizers/punkt/english.pickle')
word_tokenizer = nltk.tokenize.RegexpTokenizer(r'\w+')
##tt = ToktokTokenizer()#keeps the url as is...................................................
from nltk.tokenize import TweetTokenizer
tt = TweetTokenizer()
Porter_stemmer = PorterStemmer()
Lancaster_stemmer = LancasterStemmer()
WordNet_lemmatizer = WordNetLemmatizer()
#nltk.download('averaged_perceptron_tagger')
##Train_path="D:/Data/PAN2015-Profiling/pan15-author-profiling-training-dataset-english-2015-04-23"#path for train data
##Test_path="D:/Data/PAN2015-Profiling/pan-ap2015-test/en" #path for test data

##Train_path="D:/NLP/PAN2014/pan14-author-profiling-training-corpus-english-twitter-2014-04-16"#path for train data D:\Data\PAN2013-Profiling\Training\en  D:\NLP\PAN2014
##train_truth_path="D:/NLP/PAN2014/truth_train"
##
##test_truth_path="D:/NLP/PAN2014/pan14_test"
##Test_path="D:/NLP/PAN2014/pan14_test/en"
##Content = open('results_of_clssification_all.txt', 'w')

M_train_file= "/Users/catherine/Desktop/NLP/PAN Datasets/PAN2014blog/2014.1-blog-txtfiles/PAN2014_blog_male_train.1.txt"
F_train_file= "/Users/catherine/Desktop/NLP/PAN Datasets/PAN2014blog/2014.1-blog-txtfiles/PAN2014_blog_female_train.1.txt"

M_test_file= "/Users/catherine/Desktop/NLP/PAN Datasets/PAN2014blog/2014.1-blog-txtfiles/PAN2014_blog_male_test.1.txt"
F_test_file= "/Users/catherine/Desktop/NLP/PAN Datasets/PAN2014blog/2014.1-blog-txtfiles/PAN2014_blog_female_test.1.txt"

def all_txt(M_txt_file,  F_txt_file):
    with open(M_txt_file, mode="r", encoding="utf-8") as f:
            txt_list = [(line[:-1]).split("\t") for line in f]
            male_dict = dict()
            content=[]
            for line in txt_list:
                if line[1] in male_dict:
                # append the new number to the existing array at this slot
                    male_dict[line[1]].append(line[3])
                else:
                # create a new array in this slot
                    male_dict[line[1]] = [line[3]]
    male_content= [("M", (' '.join(a)).split()) for a in list(male_dict.values())]
  
    with open(F_txt_file, mode="r", encoding="utf-8") as f:
            txt_list = [(line[:-1]).split("\t") for line in f]
            female_dict = dict()
            content=[]
            for line in txt_list:
                if line[1] in female_dict:
                # append the new number to the existing array at this slot
                    female_dict[line[1]].append(line[3])
                else:
                # create a new array in this slot
                    female_dict[line[1]] = [line[3]]
    female_content= [("F", (' '.join(a)).split()) for a in list(female_dict.values())]
    all_txt_per_person = male_content + female_content
    return all_txt_per_person

##print( [((all_txt(M_txt_file,  F_txt_file))[i][0])  for i in range(len((all_txt(M_txt_file,  F_txt_file))))]) helps to determine the gender....

      
"""
create feature vector from the training set.....................................................................................
"""

def WordFeatures(word_list, all_training_text):

    fvs_words = np.array([[author.count(word) for word in word_list] for author in all_training_text]).astype(np.float64)

    # normalise by dividing each row by number of tokens for each author........
    fvs_words /= np.c_[np.array([len(author) for author in all_training_text])]

    return fvs_words



"""
From the saved csv file, recover the saved features to be used...............................................................
"""
import csv
word_list=[]
##txt_files =[ "/Users/catherine/Desktop/NLP/PAN Datasets/PAN_chosen_tokens/2014/output2014OR.1M.txt", "/Users/catherine/Desktop/NLP/PAN Datasets/PAN_chosen_tokens/2014/output2014OR.2F.txt"]
txt_files =[ "/Users/catherine/Desktop/NLP/PAN Datasets/PAN2014blog/2014.1-blog-txtfiles/PAN2014_blog_female_chosen_set.1.csv", "/Users/catherine/Desktop/NLP/PAN Datasets/PAN2014blog/2014.1-blog-txtfiles/PAN2014_blog_male_chosen_set.1.csv"]
print("txt_files of features used",txt_files)
##for txt_file in txt_files:
##    with open(txt_file, mode="r", encoding="utf-8") as f:
##        reader = csv.reader(f, delimiter=",")
##        next(reader) # skip header
##        word_list1 =  ([(r.split(":"))[1] for r in f])
##        print(txt_file, len(word_list1))
##        word_list = word_list + word_list1
##print("length of features used =", len(word_list))

for txt_file in txt_files:
    with open(txt_file, mode="r", encoding="utf-8") as f:
        reader = csv.reader(f, delimiter=",")
        next(reader) # skip header
        word_list1 =  [r[0] for r in reader]
        print(txt_file, len(word_list1))
        word_list = word_list + word_list1
print("length of features used =", len(word_list))


"""
Prepare the training and test sets to be parsed to the classifies............................................................
"""
A=(all_txt(M_train_file,  F_train_file))
all_training_text = ( [(A[i][1])  for i in range(len(A))])
print("len(all_training_text)",len(all_training_text))
X_train=WordFeatures(word_list, all_training_text)
y_train=np.array( [(A[i][0])  for i in range(len(A))])
print("len(X_train)=",len(X_train), "len(y_train)=", len(y_train))

B=(all_txt(M_test_file,  F_test_file))
all_test_text = ( [(B[i][1])  for i in range(len(B))])
print("len(all_test_text)",len(all_test_text))
X_test = WordFeatures(word_list, all_test_text)
y_test = np.array( [(B[i][0])  for i in range(len(B))])
print("len(X_test=",len(X_test), "len(y_test)=",len(y_test))


X_train = np.nan_to_num(X_train)
X_test = np.nan_to_num(X_test)

"""
Perform feature selection using chi2............................................................
"""
from sklearn.feature_selection import *
##
####def select_Bestfeatures(X, y):
print("selector = SelectKBest(score_func=f_classif, k=200)")
selector = SelectKBest(score_func=f_classif, k=200)
fit = selector.fit(X_train,y_train)
train_selected_features = fit.transform(X_train)
test_selected_features = fit.transform(X_test)



"""
x_train and x_test based on the new features using chi2............................................................
"""
##X_train = select_Bestfeatures(X_train, y_train)
##X_test = select_Bestfeatures(X_test, y_test)


"""
Perform feature selection using f_classif............................................................
"""
from sklearn.feature_selection import *
##def f_classif_select_Bestfeatures(X, y):
##selector=SelectKBest(score_func=f_classif,k=500)
##fit = selector.fit(X_train,y_train)
##train_selected_features = fit.transform(X_train)
##test_selected_features = fit.transform(X_test)
##    selected_features = fit.transform(X)
##    return selected_features

"""
x_train and x_test based on the new features using f_classif............................................................
"""
##X_train = f_classif_select_Bestfeatures(X_train, y_train)
##X_test = f_classif_select_Bestfeatures(X_test, y_test)



"""
Perform feature selection using mutual_info_classif............................................................
"""
from sklearn.feature_selection import *
##def mutual_info_classif_select_Bestfeatures(X_tr, X_te, y):
##selector=SelectKBest(score_func=mutual_info_classif,k=100)
##fit = selector.fit(X_train,y_train)
##train_selected_features = fit.transform(X_train)
##test_selected_features = fit.transform(X_test)
##    return train_selected_features, test_selected_features

"""
x_train and x_test based on the new features using f_classif............................................................
"""
X_train =train_selected_features 
X_test = test_selected_features


"""
Perform feature selection using pca............................................................
"""

##from sklearn.decomposition import PCA
##def PCA_select_Bestfeatures(X):
##    # feature extraction
##    pca = PCA(n_components=200)
##    fit = pca.fit(X, y)
##    Train_selected_features = fit.transform(X)
##    return selected_features

"""
x_train and x_test based on the new features using f_classif............................................................
"""
##X_train = PCA_select_Bestfeatures(X_train)
##X_test = PCA_select_Bestfeatures(X_test)

"""
Below are a list of Classification Algorithms with different distance measres ..............................................................................
"""
"""
K-Nearest Neighbours (average aa over all the nearest neighbour selectiopn)..............................................................................
"""

k=5

print(k)
"""
8. manhattan K-Nearest Neighbours..............................................................................
"""
from sklearn.neighbors import KNeighborsClassifier
knn = KNeighborsClassifier(n_neighbors = k, metric= "manhattan")
knn.fit(X_train, y_train)
y_pred = knn.predict(X_test)
print("manhattan K-Nearest Neighbours")
print(metrics.accuracy_score(y_test, y_pred))

    
"""
8. cosine K-Nearest Neighbours..............................................................................
"""
from sklearn.neighbors import KNeighborsClassifier
knn = KNeighborsClassifier(n_neighbors = k, metric= "cosine")
knn.fit(X_train, y_train)
y_pred = knn.predict(X_test)
print("cosine K-Nearest Neighbours")
print(metrics.accuracy_score(y_test, y_pred))


"""
8. euclidean K-Nearest Neighbours..............................................................................
"""
from sklearn.neighbors import KNeighborsClassifier
knn = KNeighborsClassifier(n_neighbors = k, metric= "euclidean")
knn.fit(X_train, y_train)
y_pred = knn.predict(X_test)
print("euclidean K-Nearest Neighbours")
print(metrics.accuracy_score(y_test, y_pred))

        
"""
9. scipy.spatial.distance K-Nearest Neighbours..............................................................................
"""
from scipy.spatial.distance import *
knn = KNeighborsClassifier(n_neighbors = k, metric= "braycurtis")
knn.fit(X_train, y_train)
y_pred = knn.predict(X_test)
print("braycurtis K-Nearest Neighbours")
print(metrics.accuracy_score(y_test, y_pred))
       
        
knn = KNeighborsClassifier(n_neighbors = k, metric= "canberra")
knn.fit(X_train, y_train)
y_pred = knn.predict(X_test)
print("canberra K-Nearest Neighbours")
print(metrics.accuracy_score(y_test, y_pred))
  
        
knn = KNeighborsClassifier(n_neighbors = k, metric= "jaccard")
knn.fit(X_train, y_train)
y_pred = knn.predict(X_test)
print("jaccard K-Nearest Neighbours")
print(metrics.accuracy_score(y_test, y_pred))
        
knn = KNeighborsClassifier(n_neighbors = k, metric= "yule")
knn.fit(X_train, y_train)
y_pred = knn.predict(X_test)
print("yule K-Nearest Neighbours")
print(metrics.accuracy_score(y_test, y_pred))
print("************************************ ****************************")


txt_files of features used ['/Users/catherine/Desktop/NLP/PAN Datasets/PAN2014blog/2014.1-blog-txtfiles/PAN2014_blog_female_chosen_set.1.csv', '/Users/catherine/Desktop/NLP/PAN Datasets/PAN2014blog/2014.1-blog-txtfiles/PAN2014_blog_male_chosen_set.1.csv']
/Users/catherine/Desktop/NLP/PAN Datasets/PAN2014blog/2014.1-blog-txtfiles/PAN2014_blog_female_chosen_set.1.csv 1526
/Users/catherine/Desktop/NLP/PAN Datasets/PAN2014blog/2014.1-blog-txtfiles/PAN2014_blog_male_chosen_set.1.csv 2063
length of features used = 3589
len(all_training_text) 147
len(X_train)= 147 len(y_train)= 147
len(all_test_text) 78
len(X_test= 78 len(y_test)= 78
selector = SelectKBest(score_func=f_classif, k=200)
5
manhattan K-Nearest Neighbours
0.5512820512820513
cosine K-Nearest Neighbours
0.6410256410256411
euclidean K-Nearest Neighbours
0.6282051282051282
braycurtis K-Nearest Neighbours
0.5897435897435898
canberra K-Nearest Neighbours
0.5769230769230769
jaccard K-Nearest Neighbours
0.5897435897435898
yule K-Nearest N

/anaconda3/lib/python3.7/site-packages/sklearn/utils/validation.py:475: DataConversionWarning: Data with input dtype float64 was converted to bool by check_pairwise_arrays.
  warnings.warn(msg, DataConversionWarning)


In [36]:
import sys
sys.path.append("/anaconda3/lib/python3.7/site-packages")
import numpy as np
import numpy
import nltk
import pickle
import pandas as pd
import glob
import os
import re
import operator
from collections import Counter
#from read_xml_files import *
##from s_stemmer import *
from nltk.stem.porter import PorterStemmer
from nltk.stem import LancasterStemmer, WordNetLemmatizer
from itertools import groupby
from nltk.collocations import *
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.cluster import KMeans
from scipy.cluster.vq import whiten
from nltk.tokenize.toktok import ToktokTokenizer
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2
import xml.etree.ElementTree as ET
import pandas as pd
#Import scikit-learn metrics module for accuracy calculation
from sklearn import metrics
##from pattern.text.en import singularize
##nltk.download('vader_lexicon')
##nltk.download('punkt')
#import the relevant modules from the NLTK library
from nltk.sentiment.vader import SentimentIntensityAnalyzer
sentence_tokenizer = nltk.data.load('tokenizers/punkt/english.pickle')
word_tokenizer = nltk.tokenize.RegexpTokenizer(r'\w+')
##tt = ToktokTokenizer()#keeps the url as is...................................................
from nltk.tokenize import TweetTokenizer
tt = TweetTokenizer()
Porter_stemmer = PorterStemmer()
Lancaster_stemmer = LancasterStemmer()
WordNet_lemmatizer = WordNetLemmatizer()
#nltk.download('averaged_perceptron_tagger')
##Train_path="D:/Data/PAN2015-Profiling/pan15-author-profiling-training-dataset-english-2015-04-23"#path for train data
##Test_path="D:/Data/PAN2015-Profiling/pan-ap2015-test/en" #path for test data

##Train_path="D:/NLP/PAN2014/pan14-author-profiling-training-corpus-english-twitter-2014-04-16"#path for train data D:\Data\PAN2013-Profiling\Training\en  D:\NLP\PAN2014
##train_truth_path="D:/NLP/PAN2014/truth_train"
##
##test_truth_path="D:/NLP/PAN2014/pan14_test"
##Test_path="D:/NLP/PAN2014/pan14_test/en"
##Content = open('results_of_clssification_all.txt', 'w')

M_train_file= "/Users/catherine/Desktop/NLP/PAN Datasets/PAN2014blog/2014.1-blog-txtfiles/PAN2014_blog_male_train.1.txt"
F_train_file= "/Users/catherine/Desktop/NLP/PAN Datasets/PAN2014blog/2014.1-blog-txtfiles/PAN2014_blog_female_train.1.txt"

M_test_file= "/Users/catherine/Desktop/NLP/PAN Datasets/PAN2014blog/2014.1-blog-txtfiles/PAN2014_blog_male_test.1.txt"
F_test_file= "/Users/catherine/Desktop/NLP/PAN Datasets/PAN2014blog/2014.1-blog-txtfiles/PAN2014_blog_female_test.1.txt"

def all_txt(M_txt_file,  F_txt_file):
    with open(M_txt_file, mode="r", encoding="utf-8") as f:
            txt_list = [(line[:-1]).split("\t") for line in f]
            male_dict = dict()
            content=[]
            for line in txt_list:
                if line[1] in male_dict:
                # append the new number to the existing array at this slot
                    male_dict[line[1]].append(line[3])
                else:
                # create a new array in this slot
                    male_dict[line[1]] = [line[3]]
    male_content= [("M", (' '.join(a)).split()) for a in list(male_dict.values())]
  
    with open(F_txt_file, mode="r", encoding="utf-8") as f:
            txt_list = [(line[:-1]).split("\t") for line in f]
            female_dict = dict()
            content=[]
            for line in txt_list:
                if line[1] in female_dict:
                # append the new number to the existing array at this slot
                    female_dict[line[1]].append(line[3])
                else:
                # create a new array in this slot
                    female_dict[line[1]] = [line[3]]
    female_content= [("F", (' '.join(a)).split()) for a in list(female_dict.values())]
    all_txt_per_person = male_content + female_content
    return all_txt_per_person

##print( [((all_txt(M_txt_file,  F_txt_file))[i][0])  for i in range(len((all_txt(M_txt_file,  F_txt_file))))]) helps to determine the gender....

      
"""
create feature vector from the training set.....................................................................................
"""

def WordFeatures(word_list, all_training_text):

    fvs_words = np.array([[author.count(word) for word in word_list] for author in all_training_text]).astype(np.float64)

    # normalise by dividing each row by number of tokens for each author........
    fvs_words /= np.c_[np.array([len(author) for author in all_training_text])]

    return fvs_words



"""
From the saved csv file, recover the saved features to be used...............................................................
"""
import csv
word_list=[]
##txt_files =[ "/Users/catherine/Desktop/NLP/PAN Datasets/PAN_chosen_tokens/2014/output2014OR.1M.txt", "/Users/catherine/Desktop/NLP/PAN Datasets/PAN_chosen_tokens/2014/output2014OR.2F.txt"]
txt_files =[ "/Users/catherine/Desktop/NLP/PAN Datasets/PAN2014blog/2014.1-blog-txtfiles/PAN2014_blog_female_chosen_set.1.csv", "/Users/catherine/Desktop/NLP/PAN Datasets/PAN2014blog/2014.1-blog-txtfiles/PAN2014_blog_male_chosen_set.1.csv"]
print("txt_files of features used",txt_files)
##for txt_file in txt_files:
##    with open(txt_file, mode="r", encoding="utf-8") as f:
##        reader = csv.reader(f, delimiter=",")
##        next(reader) # skip header
##        word_list1 =  ([(r.split(":"))[1] for r in f])
##        print(txt_file, len(word_list1))
##        word_list = word_list + word_list1
##print("length of features used =", len(word_list))

for txt_file in txt_files:
    with open(txt_file, mode="r", encoding="utf-8") as f:
        reader = csv.reader(f, delimiter=",")
        next(reader) # skip header
        word_list1 =  [r[0] for r in reader]
        print(txt_file, len(word_list1))
        word_list = word_list + word_list1
print("length of features used =", len(word_list))


"""
Prepare the training and test sets to be parsed to the classifies............................................................
"""
A=(all_txt(M_train_file,  F_train_file))
all_training_text = ( [(A[i][1])  for i in range(len(A))])
print("len(all_training_text)",len(all_training_text))
X_train=WordFeatures(word_list, all_training_text)
y_train=np.array( [(A[i][0])  for i in range(len(A))])
print("len(X_train)=",len(X_train), "len(y_train)=", len(y_train))

B=(all_txt(M_test_file,  F_test_file))
all_test_text = ( [(B[i][1])  for i in range(len(B))])
print("len(all_test_text)",len(all_test_text))
X_test = WordFeatures(word_list, all_test_text)
y_test = np.array( [(B[i][0])  for i in range(len(B))])
print("len(X_test=",len(X_test), "len(y_test)=",len(y_test))


X_train = np.nan_to_num(X_train)
X_test = np.nan_to_num(X_test)

"""
Perform feature selection using chi2............................................................
"""
from sklearn.feature_selection import *
##
####def select_Bestfeatures(X, y):
print("selector = SelectKBest(score_func=f_classif, k=300)")
selector = SelectKBest(score_func=f_classif, k=300)
fit = selector.fit(X_train,y_train)
train_selected_features = fit.transform(X_train)
test_selected_features = fit.transform(X_test)



"""
x_train and x_test based on the new features using chi2............................................................
"""
##X_train = select_Bestfeatures(X_train, y_train)
##X_test = select_Bestfeatures(X_test, y_test)


"""
Perform feature selection using f_classif............................................................
"""
from sklearn.feature_selection import *
##def f_classif_select_Bestfeatures(X, y):
##selector=SelectKBest(score_func=f_classif,k=500)
##fit = selector.fit(X_train,y_train)
##train_selected_features = fit.transform(X_train)
##test_selected_features = fit.transform(X_test)
##    selected_features = fit.transform(X)
##    return selected_features

"""
x_train and x_test based on the new features using f_classif............................................................
"""
##X_train = f_classif_select_Bestfeatures(X_train, y_train)
##X_test = f_classif_select_Bestfeatures(X_test, y_test)



"""
Perform feature selection using mutual_info_classif............................................................
"""
from sklearn.feature_selection import *
##def mutual_info_classif_select_Bestfeatures(X_tr, X_te, y):
##selector=SelectKBest(score_func=mutual_info_classif,k=100)
##fit = selector.fit(X_train,y_train)
##train_selected_features = fit.transform(X_train)
##test_selected_features = fit.transform(X_test)
##    return train_selected_features, test_selected_features

"""
x_train and x_test based on the new features using f_classif............................................................
"""
X_train =train_selected_features 
X_test = test_selected_features


"""
Perform feature selection using pca............................................................
"""

##from sklearn.decomposition import PCA
##def PCA_select_Bestfeatures(X):
##    # feature extraction
##    pca = PCA(n_components=200)
##    fit = pca.fit(X, y)
##    Train_selected_features = fit.transform(X)
##    return selected_features

"""
x_train and x_test based on the new features using f_classif............................................................
"""
##X_train = PCA_select_Bestfeatures(X_train)
##X_test = PCA_select_Bestfeatures(X_test)

"""
Below are a list of Classification Algorithms with different distance measres ..............................................................................
"""
"""
K-Nearest Neighbours (average aa over all the nearest neighbour selectiopn)..............................................................................
"""

k=5

print(k)
"""
8. manhattan K-Nearest Neighbours..............................................................................
"""
from sklearn.neighbors import KNeighborsClassifier
knn = KNeighborsClassifier(n_neighbors = k, metric= "manhattan")
knn.fit(X_train, y_train)
y_pred = knn.predict(X_test)
print("manhattan K-Nearest Neighbours")
print(metrics.accuracy_score(y_test, y_pred))

    
"""
8. cosine K-Nearest Neighbours..............................................................................
"""
from sklearn.neighbors import KNeighborsClassifier
knn = KNeighborsClassifier(n_neighbors = k, metric= "cosine")
knn.fit(X_train, y_train)
y_pred = knn.predict(X_test)
print("cosine K-Nearest Neighbours")
print(metrics.accuracy_score(y_test, y_pred))


"""
8. euclidean K-Nearest Neighbours..............................................................................
"""
from sklearn.neighbors import KNeighborsClassifier
knn = KNeighborsClassifier(n_neighbors = k, metric= "euclidean")
knn.fit(X_train, y_train)
y_pred = knn.predict(X_test)
print("euclidean K-Nearest Neighbours")
print(metrics.accuracy_score(y_test, y_pred))

        
"""
9. scipy.spatial.distance K-Nearest Neighbours..............................................................................
"""
from scipy.spatial.distance import *
knn = KNeighborsClassifier(n_neighbors = k, metric= "braycurtis")
knn.fit(X_train, y_train)
y_pred = knn.predict(X_test)
print("braycurtis K-Nearest Neighbours")
print(metrics.accuracy_score(y_test, y_pred))
       
        
knn = KNeighborsClassifier(n_neighbors = k, metric= "canberra")
knn.fit(X_train, y_train)
y_pred = knn.predict(X_test)
print("canberra K-Nearest Neighbours")
print(metrics.accuracy_score(y_test, y_pred))
  
        
knn = KNeighborsClassifier(n_neighbors = k, metric= "jaccard")
knn.fit(X_train, y_train)
y_pred = knn.predict(X_test)
print("jaccard K-Nearest Neighbours")
print(metrics.accuracy_score(y_test, y_pred))
        
knn = KNeighborsClassifier(n_neighbors = k, metric= "yule")
knn.fit(X_train, y_train)
y_pred = knn.predict(X_test)
print("yule K-Nearest Neighbours")
print(metrics.accuracy_score(y_test, y_pred))
print("************************************ ****************************")



txt_files of features used ['/Users/catherine/Desktop/NLP/PAN Datasets/PAN2014blog/2014.1-blog-txtfiles/PAN2014_blog_female_chosen_set.1.csv', '/Users/catherine/Desktop/NLP/PAN Datasets/PAN2014blog/2014.1-blog-txtfiles/PAN2014_blog_male_chosen_set.1.csv']
/Users/catherine/Desktop/NLP/PAN Datasets/PAN2014blog/2014.1-blog-txtfiles/PAN2014_blog_female_chosen_set.1.csv 1526
/Users/catherine/Desktop/NLP/PAN Datasets/PAN2014blog/2014.1-blog-txtfiles/PAN2014_blog_male_chosen_set.1.csv 2063
length of features used = 3589
len(all_training_text) 147
len(X_train)= 147 len(y_train)= 147
len(all_test_text) 78
len(X_test= 78 len(y_test)= 78
selector = SelectKBest(score_func=f_classif, k=300)
5
manhattan K-Nearest Neighbours
0.6025641025641025
cosine K-Nearest Neighbours
0.6282051282051282
euclidean K-Nearest Neighbours
0.6410256410256411
braycurtis K-Nearest Neighbours
0.6282051282051282
canberra K-Nearest Neighbours
0.5897435897435898
jaccard K-Nearest Neighbours
0.6153846153846154
yule K-Nearest N

/anaconda3/lib/python3.7/site-packages/sklearn/utils/validation.py:475: DataConversionWarning: Data with input dtype float64 was converted to bool by check_pairwise_arrays.
  warnings.warn(msg, DataConversionWarning)


In [37]:
import sys
sys.path.append("/anaconda3/lib/python3.7/site-packages")
import numpy as np
import numpy
import nltk
import pickle
import pandas as pd
import glob
import os
import re
import operator
from collections import Counter
#from read_xml_files import *
##from s_stemmer import *
from nltk.stem.porter import PorterStemmer
from nltk.stem import LancasterStemmer, WordNetLemmatizer
from itertools import groupby
from nltk.collocations import *
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.cluster import KMeans
from scipy.cluster.vq import whiten
from nltk.tokenize.toktok import ToktokTokenizer
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2
import xml.etree.ElementTree as ET
import pandas as pd
#Import scikit-learn metrics module for accuracy calculation
from sklearn import metrics
##from pattern.text.en import singularize
##nltk.download('vader_lexicon')
##nltk.download('punkt')
#import the relevant modules from the NLTK library
from nltk.sentiment.vader import SentimentIntensityAnalyzer
sentence_tokenizer = nltk.data.load('tokenizers/punkt/english.pickle')
word_tokenizer = nltk.tokenize.RegexpTokenizer(r'\w+')
##tt = ToktokTokenizer()#keeps the url as is...................................................
from nltk.tokenize import TweetTokenizer
tt = TweetTokenizer()
Porter_stemmer = PorterStemmer()
Lancaster_stemmer = LancasterStemmer()
WordNet_lemmatizer = WordNetLemmatizer()
#nltk.download('averaged_perceptron_tagger')
##Train_path="D:/Data/PAN2015-Profiling/pan15-author-profiling-training-dataset-english-2015-04-23"#path for train data
##Test_path="D:/Data/PAN2015-Profiling/pan-ap2015-test/en" #path for test data

##Train_path="D:/NLP/PAN2014/pan14-author-profiling-training-corpus-english-twitter-2014-04-16"#path for train data D:\Data\PAN2013-Profiling\Training\en  D:\NLP\PAN2014
##train_truth_path="D:/NLP/PAN2014/truth_train"
##
##test_truth_path="D:/NLP/PAN2014/pan14_test"
##Test_path="D:/NLP/PAN2014/pan14_test/en"
##Content = open('results_of_clssification_all.txt', 'w')

M_train_file= "/Users/catherine/Desktop/NLP/PAN Datasets/PAN2014blog/2014.1-blog-txtfiles/PAN2014_blog_male_train.1.txt"
F_train_file= "/Users/catherine/Desktop/NLP/PAN Datasets/PAN2014blog/2014.1-blog-txtfiles/PAN2014_blog_female_train.1.txt"

M_test_file= "/Users/catherine/Desktop/NLP/PAN Datasets/PAN2014blog/2014.1-blog-txtfiles/PAN2014_blog_male_test.1.txt"
F_test_file= "/Users/catherine/Desktop/NLP/PAN Datasets/PAN2014blog/2014.1-blog-txtfiles/PAN2014_blog_female_test.1.txt"

def all_txt(M_txt_file,  F_txt_file):
    with open(M_txt_file, mode="r", encoding="utf-8") as f:
            txt_list = [(line[:-1]).split("\t") for line in f]
            male_dict = dict()
            content=[]
            for line in txt_list:
                if line[1] in male_dict:
                # append the new number to the existing array at this slot
                    male_dict[line[1]].append(line[3])
                else:
                # create a new array in this slot
                    male_dict[line[1]] = [line[3]]
    male_content= [("M", (' '.join(a)).split()) for a in list(male_dict.values())]
  
    with open(F_txt_file, mode="r", encoding="utf-8") as f:
            txt_list = [(line[:-1]).split("\t") for line in f]
            female_dict = dict()
            content=[]
            for line in txt_list:
                if line[1] in female_dict:
                # append the new number to the existing array at this slot
                    female_dict[line[1]].append(line[3])
                else:
                # create a new array in this slot
                    female_dict[line[1]] = [line[3]]
    female_content= [("F", (' '.join(a)).split()) for a in list(female_dict.values())]
    all_txt_per_person = male_content + female_content
    return all_txt_per_person

##print( [((all_txt(M_txt_file,  F_txt_file))[i][0])  for i in range(len((all_txt(M_txt_file,  F_txt_file))))]) helps to determine the gender....

      
"""
create feature vector from the training set.....................................................................................
"""

def WordFeatures(word_list, all_training_text):

    fvs_words = np.array([[author.count(word) for word in word_list] for author in all_training_text]).astype(np.float64)

    # normalise by dividing each row by number of tokens for each author........
    fvs_words /= np.c_[np.array([len(author) for author in all_training_text])]

    return fvs_words



"""
From the saved csv file, recover the saved features to be used...............................................................
"""
import csv
word_list=[]
##txt_files =[ "/Users/catherine/Desktop/NLP/PAN Datasets/PAN_chosen_tokens/2014/output2014OR.1M.txt", "/Users/catherine/Desktop/NLP/PAN Datasets/PAN_chosen_tokens/2014/output2014OR.2F.txt"]
txt_files =[ "/Users/catherine/Desktop/NLP/PAN Datasets/PAN2014blog/2014.1-blog-txtfiles/PAN2014_blog_female_chosen_set.1.csv", "/Users/catherine/Desktop/NLP/PAN Datasets/PAN2014blog/2014.1-blog-txtfiles/PAN2014_blog_male_chosen_set.1.csv"]
print("txt_files of features used",txt_files)
##for txt_file in txt_files:
##    with open(txt_file, mode="r", encoding="utf-8") as f:
##        reader = csv.reader(f, delimiter=",")
##        next(reader) # skip header
##        word_list1 =  ([(r.split(":"))[1] for r in f])
##        print(txt_file, len(word_list1))
##        word_list = word_list + word_list1
##print("length of features used =", len(word_list))

for txt_file in txt_files:
    with open(txt_file, mode="r", encoding="utf-8") as f:
        reader = csv.reader(f, delimiter=",")
        next(reader) # skip header
        word_list1 =  [r[0] for r in reader]
        print(txt_file, len(word_list1))
        word_list = word_list + word_list1
print("length of features used =", len(word_list))


"""
Prepare the training and test sets to be parsed to the classifies............................................................
"""
A=(all_txt(M_train_file,  F_train_file))
all_training_text = ( [(A[i][1])  for i in range(len(A))])
print("len(all_training_text)",len(all_training_text))
X_train=WordFeatures(word_list, all_training_text)
y_train=np.array( [(A[i][0])  for i in range(len(A))])
print("len(X_train)=",len(X_train), "len(y_train)=", len(y_train))

B=(all_txt(M_test_file,  F_test_file))
all_test_text = ( [(B[i][1])  for i in range(len(B))])
print("len(all_test_text)",len(all_test_text))
X_test = WordFeatures(word_list, all_test_text)
y_test = np.array( [(B[i][0])  for i in range(len(B))])
print("len(X_test=",len(X_test), "len(y_test)=",len(y_test))


X_train = np.nan_to_num(X_train)
X_test = np.nan_to_num(X_test)

"""
Perform feature selection using chi2............................................................
"""
from sklearn.feature_selection import *
##
####def select_Bestfeatures(X, y):
print("selector = SelectKBest(score_func=f_classif, k=500)")
selector = SelectKBest(score_func=f_classif, k=500)
fit = selector.fit(X_train,y_train)
train_selected_features = fit.transform(X_train)
test_selected_features = fit.transform(X_test)



"""
x_train and x_test based on the new features using chi2............................................................
"""
##X_train = select_Bestfeatures(X_train, y_train)
##X_test = select_Bestfeatures(X_test, y_test)


"""
Perform feature selection using f_classif............................................................
"""
from sklearn.feature_selection import *
##def f_classif_select_Bestfeatures(X, y):
##selector=SelectKBest(score_func=f_classif,k=500)
##fit = selector.fit(X_train,y_train)
##train_selected_features = fit.transform(X_train)
##test_selected_features = fit.transform(X_test)
##    selected_features = fit.transform(X)
##    return selected_features

"""
x_train and x_test based on the new features using f_classif............................................................
"""
##X_train = f_classif_select_Bestfeatures(X_train, y_train)
##X_test = f_classif_select_Bestfeatures(X_test, y_test)



"""
Perform feature selection using mutual_info_classif............................................................
"""
from sklearn.feature_selection import *
##def mutual_info_classif_select_Bestfeatures(X_tr, X_te, y):
##selector=SelectKBest(score_func=mutual_info_classif,k=100)
##fit = selector.fit(X_train,y_train)
##train_selected_features = fit.transform(X_train)
##test_selected_features = fit.transform(X_test)
##    return train_selected_features, test_selected_features

"""
x_train and x_test based on the new features using f_classif............................................................
"""
X_train =train_selected_features 
X_test = test_selected_features


"""
Perform feature selection using pca............................................................
"""

##from sklearn.decomposition import PCA
##def PCA_select_Bestfeatures(X):
##    # feature extraction
##    pca = PCA(n_components=200)
##    fit = pca.fit(X, y)
##    Train_selected_features = fit.transform(X)
##    return selected_features

"""
x_train and x_test based on the new features using f_classif............................................................
"""
##X_train = PCA_select_Bestfeatures(X_train)
##X_test = PCA_select_Bestfeatures(X_test)

"""
Below are a list of Classification Algorithms with different distance measres ..............................................................................
"""
"""
K-Nearest Neighbours (average aa over all the nearest neighbour selectiopn)..............................................................................
"""

k=5

print(k)
"""
8. manhattan K-Nearest Neighbours..............................................................................
"""
from sklearn.neighbors import KNeighborsClassifier
knn = KNeighborsClassifier(n_neighbors = k, metric= "manhattan")
knn.fit(X_train, y_train)
y_pred = knn.predict(X_test)
print("manhattan K-Nearest Neighbours")
print(metrics.accuracy_score(y_test, y_pred))

    
"""
8. cosine K-Nearest Neighbours..............................................................................
"""
from sklearn.neighbors import KNeighborsClassifier
knn = KNeighborsClassifier(n_neighbors = k, metric= "cosine")
knn.fit(X_train, y_train)
y_pred = knn.predict(X_test)
print("cosine K-Nearest Neighbours")
print(metrics.accuracy_score(y_test, y_pred))


"""
8. euclidean K-Nearest Neighbours..............................................................................
"""
from sklearn.neighbors import KNeighborsClassifier
knn = KNeighborsClassifier(n_neighbors = k, metric= "euclidean")
knn.fit(X_train, y_train)
y_pred = knn.predict(X_test)
print("euclidean K-Nearest Neighbours")
print(metrics.accuracy_score(y_test, y_pred))

        
"""
9. scipy.spatial.distance K-Nearest Neighbours..............................................................................
"""
from scipy.spatial.distance import *
knn = KNeighborsClassifier(n_neighbors = k, metric= "braycurtis")
knn.fit(X_train, y_train)
y_pred = knn.predict(X_test)
print("braycurtis K-Nearest Neighbours")
print(metrics.accuracy_score(y_test, y_pred))
       
        
knn = KNeighborsClassifier(n_neighbors = k, metric= "canberra")
knn.fit(X_train, y_train)
y_pred = knn.predict(X_test)
print("canberra K-Nearest Neighbours")
print(metrics.accuracy_score(y_test, y_pred))
  
        
knn = KNeighborsClassifier(n_neighbors = k, metric= "jaccard")
knn.fit(X_train, y_train)
y_pred = knn.predict(X_test)
print("jaccard K-Nearest Neighbours")
print(metrics.accuracy_score(y_test, y_pred))
        
knn = KNeighborsClassifier(n_neighbors = k, metric= "yule")
knn.fit(X_train, y_train)
y_pred = knn.predict(X_test)
print("yule K-Nearest Neighbours")
print(metrics.accuracy_score(y_test, y_pred))
print("************************************ ****************************")

txt_files of features used ['/Users/catherine/Desktop/NLP/PAN Datasets/PAN2014blog/2014.1-blog-txtfiles/PAN2014_blog_female_chosen_set.1.csv', '/Users/catherine/Desktop/NLP/PAN Datasets/PAN2014blog/2014.1-blog-txtfiles/PAN2014_blog_male_chosen_set.1.csv']
/Users/catherine/Desktop/NLP/PAN Datasets/PAN2014blog/2014.1-blog-txtfiles/PAN2014_blog_female_chosen_set.1.csv 1526
/Users/catherine/Desktop/NLP/PAN Datasets/PAN2014blog/2014.1-blog-txtfiles/PAN2014_blog_male_chosen_set.1.csv 2063
length of features used = 3589
len(all_training_text) 147
len(X_train)= 147 len(y_train)= 147
len(all_test_text) 78
len(X_test= 78 len(y_test)= 78
selector = SelectKBest(score_func=f_classif, k=500)
5
manhattan K-Nearest Neighbours
0.6538461538461539
cosine K-Nearest Neighbours
0.6538461538461539
euclidean K-Nearest Neighbours
0.6282051282051282
braycurtis K-Nearest Neighbours
0.6025641025641025
canberra K-Nearest Neighbours
0.5512820512820513
jaccard K-Nearest Neighbours
0.5897435897435898
yule K-Nearest N

/anaconda3/lib/python3.7/site-packages/sklearn/utils/validation.py:475: DataConversionWarning: Data with input dtype float64 was converted to bool by check_pairwise_arrays.
  warnings.warn(msg, DataConversionWarning)


In [38]:
import sys
sys.path.append("/anaconda3/lib/python3.7/site-packages")
import numpy as np
import numpy
import nltk
import pickle
import pandas as pd
import glob
import os
import re
import operator
from collections import Counter
#from read_xml_files import *
##from s_stemmer import *
from nltk.stem.porter import PorterStemmer
from nltk.stem import LancasterStemmer, WordNetLemmatizer
from itertools import groupby
from nltk.collocations import *
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.cluster import KMeans
from scipy.cluster.vq import whiten
from nltk.tokenize.toktok import ToktokTokenizer
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2
import xml.etree.ElementTree as ET
import pandas as pd
#Import scikit-learn metrics module for accuracy calculation
from sklearn import metrics
##from pattern.text.en import singularize
##nltk.download('vader_lexicon')
##nltk.download('punkt')
#import the relevant modules from the NLTK library
from nltk.sentiment.vader import SentimentIntensityAnalyzer
sentence_tokenizer = nltk.data.load('tokenizers/punkt/english.pickle')
word_tokenizer = nltk.tokenize.RegexpTokenizer(r'\w+')
##tt = ToktokTokenizer()#keeps the url as is...................................................
from nltk.tokenize import TweetTokenizer
tt = TweetTokenizer()
Porter_stemmer = PorterStemmer()
Lancaster_stemmer = LancasterStemmer()
WordNet_lemmatizer = WordNetLemmatizer()
#nltk.download('averaged_perceptron_tagger')
##Train_path="D:/Data/PAN2015-Profiling/pan15-author-profiling-training-dataset-english-2015-04-23"#path for train data
##Test_path="D:/Data/PAN2015-Profiling/pan-ap2015-test/en" #path for test data

##Train_path="D:/NLP/PAN2014/pan14-author-profiling-training-corpus-english-twitter-2014-04-16"#path for train data D:\Data\PAN2013-Profiling\Training\en  D:\NLP\PAN2014
##train_truth_path="D:/NLP/PAN2014/truth_train"
##
##test_truth_path="D:/NLP/PAN2014/pan14_test"
##Test_path="D:/NLP/PAN2014/pan14_test/en"
##Content = open('results_of_clssification_all.txt', 'w')

M_train_file= "/Users/catherine/Desktop/NLP/PAN Datasets/PAN2014blog/2014.1-blog-txtfiles/PAN2014_blog_male_train.1.txt"
F_train_file= "/Users/catherine/Desktop/NLP/PAN Datasets/PAN2014blog/2014.1-blog-txtfiles/PAN2014_blog_female_train.1.txt"

M_test_file= "/Users/catherine/Desktop/NLP/PAN Datasets/PAN2014blog/2014.1-blog-txtfiles/PAN2014_blog_male_test.1.txt"
F_test_file= "/Users/catherine/Desktop/NLP/PAN Datasets/PAN2014blog/2014.1-blog-txtfiles/PAN2014_blog_female_test.1.txt"

def all_txt(M_txt_file,  F_txt_file):
    with open(M_txt_file, mode="r", encoding="utf-8") as f:
            txt_list = [(line[:-1]).split("\t") for line in f]
            male_dict = dict()
            content=[]
            for line in txt_list:
                if line[1] in male_dict:
                # append the new number to the existing array at this slot
                    male_dict[line[1]].append(line[3])
                else:
                # create a new array in this slot
                    male_dict[line[1]] = [line[3]]
    male_content= [("M", (' '.join(a)).split()) for a in list(male_dict.values())]
  
    with open(F_txt_file, mode="r", encoding="utf-8") as f:
            txt_list = [(line[:-1]).split("\t") for line in f]
            female_dict = dict()
            content=[]
            for line in txt_list:
                if line[1] in female_dict:
                # append the new number to the existing array at this slot
                    female_dict[line[1]].append(line[3])
                else:
                # create a new array in this slot
                    female_dict[line[1]] = [line[3]]
    female_content= [("F", (' '.join(a)).split()) for a in list(female_dict.values())]
    all_txt_per_person = male_content + female_content
    return all_txt_per_person

##print( [((all_txt(M_txt_file,  F_txt_file))[i][0])  for i in range(len((all_txt(M_txt_file,  F_txt_file))))]) helps to determine the gender....

      
"""
create feature vector from the training set.....................................................................................
"""

def WordFeatures(word_list, all_training_text):

    fvs_words = np.array([[author.count(word) for word in word_list] for author in all_training_text]).astype(np.float64)

    # normalise by dividing each row by number of tokens for each author........
    fvs_words /= np.c_[np.array([len(author) for author in all_training_text])]

    return fvs_words



"""
From the saved csv file, recover the saved features to be used...............................................................
"""
import csv
word_list=[]
##txt_files =[ "/Users/catherine/Desktop/NLP/PAN Datasets/PAN_chosen_tokens/2014/output2014OR.1M.txt", "/Users/catherine/Desktop/NLP/PAN Datasets/PAN_chosen_tokens/2014/output2014OR.2F.txt"]
txt_files =[ "/Users/catherine/Desktop/NLP/PAN Datasets/PAN2014blog/2014.1-blog-txtfiles/PAN2014_blog_female_chosen_set.1.csv", "/Users/catherine/Desktop/NLP/PAN Datasets/PAN2014blog/2014.1-blog-txtfiles/PAN2014_blog_male_chosen_set.1.csv"]
print("txt_files of features used",txt_files)
##for txt_file in txt_files:
##    with open(txt_file, mode="r", encoding="utf-8") as f:
##        reader = csv.reader(f, delimiter=",")
##        next(reader) # skip header
##        word_list1 =  ([(r.split(":"))[1] for r in f])
##        print(txt_file, len(word_list1))
##        word_list = word_list + word_list1
##print("length of features used =", len(word_list))

for txt_file in txt_files:
    with open(txt_file, mode="r", encoding="utf-8") as f:
        reader = csv.reader(f, delimiter=",")
        next(reader) # skip header
        word_list1 =  [r[0] for r in reader]
        print(txt_file, len(word_list1))
        word_list = word_list + word_list1
print("length of features used =", len(word_list))


"""
Prepare the training and test sets to be parsed to the classifies............................................................
"""
A=(all_txt(M_train_file,  F_train_file))
all_training_text = ( [(A[i][1])  for i in range(len(A))])
print("len(all_training_text)",len(all_training_text))
X_train=WordFeatures(word_list, all_training_text)
y_train=np.array( [(A[i][0])  for i in range(len(A))])
print("len(X_train)=",len(X_train), "len(y_train)=", len(y_train))

B=(all_txt(M_test_file,  F_test_file))
all_test_text = ( [(B[i][1])  for i in range(len(B))])
print("len(all_test_text)",len(all_test_text))
X_test = WordFeatures(word_list, all_test_text)
y_test = np.array( [(B[i][0])  for i in range(len(B))])
print("len(X_test=",len(X_test), "len(y_test)=",len(y_test))


X_train = np.nan_to_num(X_train)
X_test = np.nan_to_num(X_test)

"""
Perform feature selection using chi2............................................................
"""
from sklearn.feature_selection import *
##
####def select_Bestfeatures(X, y):
print("selector = SelectKBest(score_func=f_classif, k=200)")
selector = SelectKBest(score_func=f_classif, k=200)
fit = selector.fit(X_train,y_train)
train_selected_features = fit.transform(X_train)
test_selected_features = fit.transform(X_test)



"""
x_train and x_test based on the new features using chi2............................................................
"""
##X_train = select_Bestfeatures(X_train, y_train)
##X_test = select_Bestfeatures(X_test, y_test)


"""
Perform feature selection using f_classif............................................................
"""
from sklearn.feature_selection import *
##def f_classif_select_Bestfeatures(X, y):
##selector=SelectKBest(score_func=f_classif,k=500)
##fit = selector.fit(X_train,y_train)
##train_selected_features = fit.transform(X_train)
##test_selected_features = fit.transform(X_test)
##    selected_features = fit.transform(X)
##    return selected_features

"""
x_train and x_test based on the new features using f_classif............................................................
"""
##X_train = f_classif_select_Bestfeatures(X_train, y_train)
##X_test = f_classif_select_Bestfeatures(X_test, y_test)



"""
Perform feature selection using mutual_info_classif............................................................
"""
from sklearn.feature_selection import *
##def mutual_info_classif_select_Bestfeatures(X_tr, X_te, y):
##selector=SelectKBest(score_func=mutual_info_classif,k=100)
##fit = selector.fit(X_train,y_train)
##train_selected_features = fit.transform(X_train)
##test_selected_features = fit.transform(X_test)
##    return train_selected_features, test_selected_features

"""
x_train and x_test based on the new features using f_classif............................................................
"""
X_train =train_selected_features 
X_test = test_selected_features


"""
Perform feature selection using pca............................................................
"""

##from sklearn.decomposition import PCA
##def PCA_select_Bestfeatures(X):
##    # feature extraction
##    pca = PCA(n_components=200)
##    fit = pca.fit(X, y)
##    Train_selected_features = fit.transform(X)
##    return selected_features

"""
x_train and x_test based on the new features using f_classif............................................................
"""
##X_train = PCA_select_Bestfeatures(X_train)
##X_test = PCA_select_Bestfeatures(X_test)

"""
Below are a list of Classification Algorithms with different distance measres ..............................................................................
"""
"""
K-Nearest Neighbours (average aa over all the nearest neighbour selectiopn)..............................................................................
"""

k=5

print(k)
"""
8. manhattan K-Nearest Neighbours..............................................................................
"""
from sklearn.neighbors import KNeighborsClassifier
knn = KNeighborsClassifier(n_neighbors = k, metric= "manhattan")
knn.fit(X_train, y_train)
y_pred = knn.predict(X_test)
print("manhattan K-Nearest Neighbours")
print(metrics.accuracy_score(y_test, y_pred))

    
"""
8. cosine K-Nearest Neighbours..............................................................................
"""
from sklearn.neighbors import KNeighborsClassifier
knn = KNeighborsClassifier(n_neighbors = k, metric= "cosine")
knn.fit(X_train, y_train)
y_pred = knn.predict(X_test)
print("cosine K-Nearest Neighbours")
print(metrics.accuracy_score(y_test, y_pred))


"""
8. euclidean K-Nearest Neighbours..............................................................................
"""
from sklearn.neighbors import KNeighborsClassifier
knn = KNeighborsClassifier(n_neighbors = k, metric= "euclidean")
knn.fit(X_train, y_train)
y_pred = knn.predict(X_test)
print("euclidean K-Nearest Neighbours")
print(metrics.accuracy_score(y_test, y_pred))

        
"""
9. scipy.spatial.distance K-Nearest Neighbours..............................................................................
"""
from scipy.spatial.distance import *
knn = KNeighborsClassifier(n_neighbors = k, metric= "braycurtis")
knn.fit(X_train, y_train)
y_pred = knn.predict(X_test)
print("braycurtis K-Nearest Neighbours")
print(metrics.accuracy_score(y_test, y_pred))
       
        
knn = KNeighborsClassifier(n_neighbors = k, metric= "canberra")
knn.fit(X_train, y_train)
y_pred = knn.predict(X_test)
print("canberra K-Nearest Neighbours")
print(metrics.accuracy_score(y_test, y_pred))
  
        
knn = KNeighborsClassifier(n_neighbors = k, metric= "jaccard")
knn.fit(X_train, y_train)
y_pred = knn.predict(X_test)
print("jaccard K-Nearest Neighbours")
print(metrics.accuracy_score(y_test, y_pred))
        
knn = KNeighborsClassifier(n_neighbors = k, metric= "yule")
knn.fit(X_train, y_train)
y_pred = knn.predict(X_test)
print("yule K-Nearest Neighbours")
print(metrics.accuracy_score(y_test, y_pred))

knn = KNeighborsClassifier(n_neighbors = k, metric= "rogerstanimoto")
knn.fit(X_train, y_train)
y_pred = knn.predict(X_test)
print("rogerstanimoto K-Nearest Neighbours")
print(metrics.accuracy_score(y_test, y_pred))

print("************************************ ****************************")

txt_files of features used ['/Users/catherine/Desktop/NLP/PAN Datasets/PAN2014blog/2014.1-blog-txtfiles/PAN2014_blog_female_chosen_set.1.csv', '/Users/catherine/Desktop/NLP/PAN Datasets/PAN2014blog/2014.1-blog-txtfiles/PAN2014_blog_male_chosen_set.1.csv']
/Users/catherine/Desktop/NLP/PAN Datasets/PAN2014blog/2014.1-blog-txtfiles/PAN2014_blog_female_chosen_set.1.csv 1526
/Users/catherine/Desktop/NLP/PAN Datasets/PAN2014blog/2014.1-blog-txtfiles/PAN2014_blog_male_chosen_set.1.csv 2063
length of features used = 3589
len(all_training_text) 147
len(X_train)= 147 len(y_train)= 147
len(all_test_text) 78
len(X_test= 78 len(y_test)= 78
selector = SelectKBest(score_func=f_classif, k=200)
5
manhattan K-Nearest Neighbours
0.5512820512820513
cosine K-Nearest Neighbours
0.6410256410256411
euclidean K-Nearest Neighbours
0.6282051282051282
braycurtis K-Nearest Neighbours
0.5897435897435898
canberra K-Nearest Neighbours
0.5769230769230769
jaccard K-Nearest Neighbours
0.5897435897435898
yule K-Nearest N

/anaconda3/lib/python3.7/site-packages/sklearn/utils/validation.py:475: DataConversionWarning: Data with input dtype float64 was converted to bool by check_pairwise_arrays.
  warnings.warn(msg, DataConversionWarning)


In [39]:
import sys
sys.path.append("/anaconda3/lib/python3.7/site-packages")
import numpy as np
import numpy
import nltk
import pickle
import pandas as pd
import glob
import os
import re
import operator
from collections import Counter
#from read_xml_files import *
##from s_stemmer import *
from nltk.stem.porter import PorterStemmer
from nltk.stem import LancasterStemmer, WordNetLemmatizer
from itertools import groupby
from nltk.collocations import *
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.cluster import KMeans
from scipy.cluster.vq import whiten
from nltk.tokenize.toktok import ToktokTokenizer
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2
import xml.etree.ElementTree as ET
import pandas as pd
#Import scikit-learn metrics module for accuracy calculation
from sklearn import metrics
##from pattern.text.en import singularize
##nltk.download('vader_lexicon')
##nltk.download('punkt')
#import the relevant modules from the NLTK library
from nltk.sentiment.vader import SentimentIntensityAnalyzer
sentence_tokenizer = nltk.data.load('tokenizers/punkt/english.pickle')
word_tokenizer = nltk.tokenize.RegexpTokenizer(r'\w+')
##tt = ToktokTokenizer()#keeps the url as is...................................................
from nltk.tokenize import TweetTokenizer
tt = TweetTokenizer()
Porter_stemmer = PorterStemmer()
Lancaster_stemmer = LancasterStemmer()
WordNet_lemmatizer = WordNetLemmatizer()
#nltk.download('averaged_perceptron_tagger')
##Train_path="D:/Data/PAN2015-Profiling/pan15-author-profiling-training-dataset-english-2015-04-23"#path for train data
##Test_path="D:/Data/PAN2015-Profiling/pan-ap2015-test/en" #path for test data

##Train_path="D:/NLP/PAN2014/pan14-author-profiling-training-corpus-english-twitter-2014-04-16"#path for train data D:\Data\PAN2013-Profiling\Training\en  D:\NLP\PAN2014
##train_truth_path="D:/NLP/PAN2014/truth_train"
##
##test_truth_path="D:/NLP/PAN2014/pan14_test"
##Test_path="D:/NLP/PAN2014/pan14_test/en"
##Content = open('results_of_clssification_all.txt', 'w')

M_train_file= "/Users/catherine/Desktop/NLP/PAN Datasets/PAN2014blog/2014.1-blog-txtfiles/PAN2014_blog_male_train.1.txt"
F_train_file= "/Users/catherine/Desktop/NLP/PAN Datasets/PAN2014blog/2014.1-blog-txtfiles/PAN2014_blog_female_train.1.txt"

M_test_file= "/Users/catherine/Desktop/NLP/PAN Datasets/PAN2014blog/2014.1-blog-txtfiles/PAN2014_blog_male_test.1.txt"
F_test_file= "/Users/catherine/Desktop/NLP/PAN Datasets/PAN2014blog/2014.1-blog-txtfiles/PAN2014_blog_female_test.1.txt"

def all_txt(M_txt_file,  F_txt_file):
    with open(M_txt_file, mode="r", encoding="utf-8") as f:
            txt_list = [(line[:-1]).split("\t") for line in f]
            male_dict = dict()
            content=[]
            for line in txt_list:
                if line[1] in male_dict:
                # append the new number to the existing array at this slot
                    male_dict[line[1]].append(line[3])
                else:
                # create a new array in this slot
                    male_dict[line[1]] = [line[3]]
    male_content= [("M", (' '.join(a)).split()) for a in list(male_dict.values())]
  
    with open(F_txt_file, mode="r", encoding="utf-8") as f:
            txt_list = [(line[:-1]).split("\t") for line in f]
            female_dict = dict()
            content=[]
            for line in txt_list:
                if line[1] in female_dict:
                # append the new number to the existing array at this slot
                    female_dict[line[1]].append(line[3])
                else:
                # create a new array in this slot
                    female_dict[line[1]] = [line[3]]
    female_content= [("F", (' '.join(a)).split()) for a in list(female_dict.values())]
    all_txt_per_person = male_content + female_content
    return all_txt_per_person

##print( [((all_txt(M_txt_file,  F_txt_file))[i][0])  for i in range(len((all_txt(M_txt_file,  F_txt_file))))]) helps to determine the gender....

      
"""
create feature vector from the training set.....................................................................................
"""

def WordFeatures(word_list, all_training_text):

    fvs_words = np.array([[author.count(word) for word in word_list] for author in all_training_text]).astype(np.float64)

    # normalise by dividing each row by number of tokens for each author........
    fvs_words /= np.c_[np.array([len(author) for author in all_training_text])]

    return fvs_words



"""
From the saved csv file, recover the saved features to be used...............................................................
"""
import csv
word_list=[]
##txt_files =[ "/Users/catherine/Desktop/NLP/PAN Datasets/PAN_chosen_tokens/2014/output2014OR.1M.txt", "/Users/catherine/Desktop/NLP/PAN Datasets/PAN_chosen_tokens/2014/output2014OR.2F.txt"]
txt_files =[ "/Users/catherine/Desktop/NLP/PAN Datasets/PAN2014blog/2014.1-blog-txtfiles/PAN2014_blog_female_vocubulary.1.csv", "/Users/catherine/Desktop/NLP/PAN Datasets/PAN2014blog/2014.1-blog-txtfiles/PAN2014_blog_male_ vocubulary_set.1.csv"]
print("txt_files of features used",txt_files)
##for txt_file in txt_files:
##    with open(txt_file, mode="r", encoding="utf-8") as f:
##        reader = csv.reader(f, delimiter=",")
##        next(reader) # skip header
##        word_list1 =  ([(r.split(":"))[1] for r in f])
##        print(txt_file, len(word_list1))
##        word_list = word_list + word_list1
##print("length of features used =", len(word_list))

for txt_file in txt_files:
    with open(txt_file, mode="r", encoding="utf-8") as f:
        reader = csv.reader(f, delimiter=",")
        next(reader) # skip header
        word_list1 =  [r[0] for r in reader]
        print(txt_file, len(word_list1))
        word_list = word_list + word_list1
print("length of features used =", len(word_list))


"""
Prepare the training and test sets to be parsed to the classifies............................................................
"""
A=(all_txt(M_train_file,  F_train_file))
all_training_text = ( [(A[i][1])  for i in range(len(A))])
print("len(all_training_text)",len(all_training_text))
X_train=WordFeatures(word_list, all_training_text)
y_train=np.array( [(A[i][0])  for i in range(len(A))])
print("len(X_train)=",len(X_train), "len(y_train)=", len(y_train))

B=(all_txt(M_test_file,  F_test_file))
all_test_text = ( [(B[i][1])  for i in range(len(B))])
print("len(all_test_text)",len(all_test_text))
X_test = WordFeatures(word_list, all_test_text)
y_test = np.array( [(B[i][0])  for i in range(len(B))])
print("len(X_test=",len(X_test), "len(y_test)=",len(y_test))


X_train = np.nan_to_num(X_train)
X_test = np.nan_to_num(X_test)

"""
Perform feature selection using chi2............................................................
"""
from sklearn.feature_selection import *
##
####def select_Bestfeatures(X, y):
#print("selector = SelectKBest(score_func=mutual_info_classif, k=500)")
#selector = SelectKBest(score_func=mutual_info_classif, k=500)
#fit = selector.fit(X_train,y_train)
#train_selected_features = fit.transform(X_train)
#test_selected_features = fit.transform(X_test)



"""
x_train and x_test based on the new features using chi2............................................................
"""
##X_train = select_Bestfeatures(X_train, y_train)
##X_test = select_Bestfeatures(X_test, y_test)


"""
Perform feature selection using f_classif............................................................
"""
from sklearn.feature_selection import *
##def f_classif_select_Bestfeatures(X, y):
##selector=SelectKBest(score_func=f_classif,k=500)
##fit = selector.fit(X_train,y_train)
##train_selected_features = fit.transform(X_train)
##test_selected_features = fit.transform(X_test)
##    selected_features = fit.transform(X)
##    return selected_features

"""
x_train and x_test based on the new features using f_classif............................................................
"""
##X_train = f_classif_select_Bestfeatures(X_train, y_train)
##X_test = f_classif_select_Bestfeatures(X_test, y_test)



"""
Perform feature selection using mutual_info_classif............................................................
"""
from sklearn.feature_selection import *
##def mutual_info_classif_select_Bestfeatures(X_tr, X_te, y):
##selector=SelectKBest(score_func=mutual_info_classif,k=100)
##fit = selector.fit(X_train,y_train)
##train_selected_features = fit.transform(X_train)
##test_selected_features = fit.transform(X_test)
##    return train_selected_features, test_selected_features

"""
x_train and x_test based on the new features using f_classif............................................................
"""
#X_train =train_selected_features 
#X_test = test_selected_features


"""
Perform feature selection using pca............................................................
"""

##from sklearn.decomposition import PCA
##def PCA_select_Bestfeatures(X):
##    # feature extraction
##    pca = PCA(n_components=200)
##    fit = pca.fit(X, y)
##    Train_selected_features = fit.transform(X)
##    return selected_features

"""
x_train and x_test based on the new features using f_classif............................................................
"""
##X_train = PCA_select_Bestfeatures(X_train)
##X_test = PCA_select_Bestfeatures(X_test)

"""
Below are a list of Classification Algorithms with different distance measres ..............................................................................
"""
"""
K-Nearest Neighbours (average aa over all the nearest neighbour selectiopn)..............................................................................
"""

k=5

print(k)
"""
8. manhattan K-Nearest Neighbours..............................................................................
"""
from sklearn.neighbors import KNeighborsClassifier
knn = KNeighborsClassifier(n_neighbors = k, metric= "manhattan")
knn.fit(X_train, y_train)
y_pred = knn.predict(X_test)
print("manhattan K-Nearest Neighbours")
print(metrics.accuracy_score(y_test, y_pred))

    
"""
8. cosine K-Nearest Neighbours..............................................................................
"""
from sklearn.neighbors import KNeighborsClassifier
knn = KNeighborsClassifier(n_neighbors = k, metric= "cosine")
knn.fit(X_train, y_train)
y_pred = knn.predict(X_test)
print("cosine K-Nearest Neighbours")
print(metrics.accuracy_score(y_test, y_pred))


"""
8. euclidean K-Nearest Neighbours..............................................................................
"""
from sklearn.neighbors import KNeighborsClassifier
knn = KNeighborsClassifier(n_neighbors = k, metric= "euclidean")
knn.fit(X_train, y_train)
y_pred = knn.predict(X_test)
print("euclidean K-Nearest Neighbours")
print(metrics.accuracy_score(y_test, y_pred))

        
"""
9. scipy.spatial.distance K-Nearest Neighbours..............................................................................
"""
from scipy.spatial.distance import *
knn = KNeighborsClassifier(n_neighbors = k, metric= "braycurtis")
knn.fit(X_train, y_train)
y_pred = knn.predict(X_test)
print("braycurtis K-Nearest Neighbours")
print(metrics.accuracy_score(y_test, y_pred))
       
        
knn = KNeighborsClassifier(n_neighbors = k, metric= "canberra")
knn.fit(X_train, y_train)
y_pred = knn.predict(X_test)
print("canberra K-Nearest Neighbours")
print(metrics.accuracy_score(y_test, y_pred))
  
        
knn = KNeighborsClassifier(n_neighbors = k, metric= "jaccard")
knn.fit(X_train, y_train)
y_pred = knn.predict(X_test)
print("jaccard K-Nearest Neighbours")
print(metrics.accuracy_score(y_test, y_pred))
        
knn = KNeighborsClassifier(n_neighbors = k, metric= "yule")
knn.fit(X_train, y_train)
y_pred = knn.predict(X_test)
print("yule K-Nearest Neighbours")
print(metrics.accuracy_score(y_test, y_pred))


knn = KNeighborsClassifier(n_neighbors = k, metric= "rogerstanimoto")
knn.fit(X_train, y_train)
y_pred = knn.predict(X_test)
print("rogerstanimoto K-Nearest Neighbours")
print(metrics.accuracy_score(y_test, y_pred))
print("************************************ ****************************")

txt_files of features used ['/Users/catherine/Desktop/NLP/PAN Datasets/PAN2014blog/2014.1-blog-txtfiles/PAN2014_blog_female_vocubulary.1.csv', '/Users/catherine/Desktop/NLP/PAN Datasets/PAN2014blog/2014.1-blog-txtfiles/PAN2014_blog_male_ vocubulary_set.1.csv']
/Users/catherine/Desktop/NLP/PAN Datasets/PAN2014blog/2014.1-blog-txtfiles/PAN2014_blog_female_vocubulary.1.csv 12375
/Users/catherine/Desktop/NLP/PAN Datasets/PAN2014blog/2014.1-blog-txtfiles/PAN2014_blog_male_ vocubulary_set.1.csv 18889
length of features used = 31264
len(all_training_text) 147
len(X_train)= 147 len(y_train)= 147
len(all_test_text) 78
len(X_test= 78 len(y_test)= 78
5
manhattan K-Nearest Neighbours
0.6666666666666666
cosine K-Nearest Neighbours
0.6538461538461539
euclidean K-Nearest Neighbours
0.6410256410256411
braycurtis K-Nearest Neighbours
0.6410256410256411
canberra K-Nearest Neighbours
0.5128205128205128
jaccard K-Nearest Neighbours
0.5256410256410257


/anaconda3/lib/python3.7/site-packages/sklearn/utils/validation.py:475: DataConversionWarning: Data with input dtype float64 was converted to bool by check_pairwise_arrays.
  warnings.warn(msg, DataConversionWarning)


yule K-Nearest Neighbours
0.5897435897435898
rogerstanimoto K-Nearest Neighbours
0.5897435897435898
************************************ ****************************


In [40]:
import sys
sys.path.append("/anaconda3/lib/python3.7/site-packages")
import numpy as np
import numpy
import nltk
import pickle
import pandas as pd
import glob
import os
import re
import operator
from collections import Counter
#from read_xml_files import *
##from s_stemmer import *
from nltk.stem.porter import PorterStemmer
from nltk.stem import LancasterStemmer, WordNetLemmatizer
from itertools import groupby
from nltk.collocations import *
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.cluster import KMeans
from scipy.cluster.vq import whiten
from nltk.tokenize.toktok import ToktokTokenizer
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2
import xml.etree.ElementTree as ET
import pandas as pd
#Import scikit-learn metrics module for accuracy calculation
from sklearn import metrics
##from pattern.text.en import singularize
##nltk.download('vader_lexicon')
##nltk.download('punkt')
#import the relevant modules from the NLTK library
from nltk.sentiment.vader import SentimentIntensityAnalyzer
sentence_tokenizer = nltk.data.load('tokenizers/punkt/english.pickle')
word_tokenizer = nltk.tokenize.RegexpTokenizer(r'\w+')
##tt = ToktokTokenizer()#keeps the url as is...................................................
from nltk.tokenize import TweetTokenizer
tt = TweetTokenizer()
Porter_stemmer = PorterStemmer()
Lancaster_stemmer = LancasterStemmer()
WordNet_lemmatizer = WordNetLemmatizer()
#nltk.download('averaged_perceptron_tagger')
##Train_path="D:/Data/PAN2015-Profiling/pan15-author-profiling-training-dataset-english-2015-04-23"#path for train data
##Test_path="D:/Data/PAN2015-Profiling/pan-ap2015-test/en" #path for test data

##Train_path="D:/NLP/PAN2014/pan14-author-profiling-training-corpus-english-twitter-2014-04-16"#path for train data D:\Data\PAN2013-Profiling\Training\en  D:\NLP\PAN2014
##train_truth_path="D:/NLP/PAN2014/truth_train"
##
##test_truth_path="D:/NLP/PAN2014/pan14_test"
##Test_path="D:/NLP/PAN2014/pan14_test/en"
##Content = open('results_of_clssification_all.txt', 'w')

M_train_file= "/Users/catherine/Desktop/NLP/PAN Datasets/PAN2014blog/2014.1-blog-txtfiles/PAN2014_blog_male_train.1.txt"
F_train_file= "/Users/catherine/Desktop/NLP/PAN Datasets/PAN2014blog/2014.1-blog-txtfiles/PAN2014_blog_female_train.1.txt"

M_test_file= "/Users/catherine/Desktop/NLP/PAN Datasets/PAN2014blog/2014.1-blog-txtfiles/PAN2014_blog_male_test.1.txt"
F_test_file= "/Users/catherine/Desktop/NLP/PAN Datasets/PAN2014blog/2014.1-blog-txtfiles/PAN2014_blog_female_test.1.txt"

def all_txt(M_txt_file,  F_txt_file):
    with open(M_txt_file, mode="r", encoding="utf-8") as f:
            txt_list = [(line[:-1]).split("\t") for line in f]
            male_dict = dict()
            content=[]
            for line in txt_list:
                if line[1] in male_dict:
                # append the new number to the existing array at this slot
                    male_dict[line[1]].append(line[3])
                else:
                # create a new array in this slot
                    male_dict[line[1]] = [line[3]]
    male_content= [("M", (' '.join(a)).split()) for a in list(male_dict.values())]
  
    with open(F_txt_file, mode="r", encoding="utf-8") as f:
            txt_list = [(line[:-1]).split("\t") for line in f]
            female_dict = dict()
            content=[]
            for line in txt_list:
                if line[1] in female_dict:
                # append the new number to the existing array at this slot
                    female_dict[line[1]].append(line[3])
                else:
                # create a new array in this slot
                    female_dict[line[1]] = [line[3]]
    female_content= [("F", (' '.join(a)).split()) for a in list(female_dict.values())]
    all_txt_per_person = male_content + female_content
    return all_txt_per_person

##print( [((all_txt(M_txt_file,  F_txt_file))[i][0])  for i in range(len((all_txt(M_txt_file,  F_txt_file))))]) helps to determine the gender....

      
"""
create feature vector from the training set.....................................................................................
"""

def WordFeatures(word_list, all_training_text):

    fvs_words = np.array([[author.count(word) for word in word_list] for author in all_training_text]).astype(np.float64)

    # normalise by dividing each row by number of tokens for each author........
    fvs_words /= np.c_[np.array([len(author) for author in all_training_text])]

    return fvs_words



"""
From the saved csv file, recover the saved features to be used...............................................................
"""
import csv
word_list=[]
##txt_files =[ "/Users/catherine/Desktop/NLP/PAN Datasets/PAN_chosen_tokens/2014/output2014OR.1M.txt", "/Users/catherine/Desktop/NLP/PAN Datasets/PAN_chosen_tokens/2014/output2014OR.2F.txt"]
txt_files =[ "/Users/catherine/Desktop/NLP/PAN Datasets/PAN2014blog/2014.1-blog-txtfiles/PAN2014_blog_female_chosen_set.1.csv", "/Users/catherine/Desktop/NLP/PAN Datasets/PAN2014blog/2014.1-blog-txtfiles/PAN2014_blog_male_chosen_set.1.csv"]
print("txt_files of features used",txt_files)
##for txt_file in txt_files:
##    with open(txt_file, mode="r", encoding="utf-8") as f:
##        reader = csv.reader(f, delimiter=",")
##        next(reader) # skip header
##        word_list1 =  ([(r.split(":"))[1] for r in f])
##        print(txt_file, len(word_list1))
##        word_list = word_list + word_list1
##print("length of features used =", len(word_list))

for txt_file in txt_files:
    with open(txt_file, mode="r", encoding="utf-8") as f:
        reader = csv.reader(f, delimiter=",")
        next(reader) # skip header
        word_list1 =  [r[0] for r in reader]
        print(txt_file, len(word_list1))
        word_list = word_list + word_list1
print("length of features used =", len(word_list))


"""
Prepare the training and test sets to be parsed to the classifies............................................................
"""
A=(all_txt(M_train_file,  F_train_file))
all_training_text = ( [(A[i][1])  for i in range(len(A))])
print("len(all_training_text)",len(all_training_text))
X_train=WordFeatures(word_list, all_training_text)
y_train=np.array( [(A[i][0])  for i in range(len(A))])
print("len(X_train)=",len(X_train), "len(y_train)=", len(y_train))

B=(all_txt(M_test_file,  F_test_file))
all_test_text = ( [(B[i][1])  for i in range(len(B))])
print("len(all_test_text)",len(all_test_text))
X_test = WordFeatures(word_list, all_test_text)
y_test = np.array( [(B[i][0])  for i in range(len(B))])
print("len(X_test=",len(X_test), "len(y_test)=",len(y_test))


X_train = np.nan_to_num(X_train)
X_test = np.nan_to_num(X_test)

"""
Perform feature selection using chi2............................................................
"""
from sklearn.feature_selection import *
##
####def select_Bestfeatures(X, y):
print("selector = SelectKBest(score_func=f_classif, k=500)")
#selector = SelectKBest(score_func=f_classif, k=500)
#fit = selector.fit(X_train,y_train)
#train_selected_features = fit.transform(X_train)
#test_selected_features = fit.transform(X_test)



"""
x_train and x_test based on the new features using chi2............................................................
"""
##X_train = select_Bestfeatures(X_train, y_train)
##X_test = select_Bestfeatures(X_test, y_test)


"""
Perform feature selection using f_classif............................................................
"""
from sklearn.feature_selection import *
##def f_classif_select_Bestfeatures(X, y):
##selector=SelectKBest(score_func=f_classif,k=500)
##fit = selector.fit(X_train,y_train)
##train_selected_features = fit.transform(X_train)
##test_selected_features = fit.transform(X_test)
##    selected_features = fit.transform(X)
##    return selected_features

"""
x_train and x_test based on the new features using f_classif............................................................
"""
##X_train = f_classif_select_Bestfeatures(X_train, y_train)
##X_test = f_classif_select_Bestfeatures(X_test, y_test)



"""
Perform feature selection using mutual_info_classif............................................................
"""
from sklearn.feature_selection import *
##def mutual_info_classif_select_Bestfeatures(X_tr, X_te, y):
##selector=SelectKBest(score_func=mutual_info_classif,k=100)
##fit = selector.fit(X_train,y_train)
##train_selected_features = fit.transform(X_train)
##test_selected_features = fit.transform(X_test)
##    return train_selected_features, test_selected_features

"""
x_train and x_test based on the new features using f_classif............................................................
"""
#X_train =train_selected_features 
#X_test = test_selected_features


"""
Perform feature selection using pca............................................................
"""

##from sklearn.decomposition import PCA
##def PCA_select_Bestfeatures(X):
##    # feature extraction
##    pca = PCA(n_components=200)
##    fit = pca.fit(X, y)
##    Train_selected_features = fit.transform(X)
##    return selected_features

"""
x_train and x_test based on the new features using f_classif............................................................
"""
##X_train = PCA_select_Bestfeatures(X_train)
##X_test = PCA_select_Bestfeatures(X_test)

"""
Below are a list of Classification Algorithms with different distance measres ..............................................................................
"""
"""
K-Nearest Neighbours (average aa over all the nearest neighbour selectiopn)..............................................................................
"""

k=5

print(k)
"""
8. manhattan K-Nearest Neighbours..............................................................................
"""
from sklearn.neighbors import KNeighborsClassifier
knn = KNeighborsClassifier(n_neighbors = k, metric= "manhattan")
knn.fit(X_train, y_train)
y_pred = knn.predict(X_test)
print("manhattan K-Nearest Neighbours")
truth_y_pred = [i for i, j in zip(y_pred, y_test) if  i == j]

print("len of truth_y_pre =", len(truth_y_pred))

false_y_pred = [i for i, j in zip(y_pred, y_test) if  i != j ]

print("len of false_y_pre =", len(false_y_pred))
print(metrics.accuracy_score(y_test, y_pred))

    
"""
8. cosine K-Nearest Neighbours..............................................................................
"""
from sklearn.neighbors import KNeighborsClassifier
knn = KNeighborsClassifier(n_neighbors = k, metric= "cosine")
knn.fit(X_train, y_train)
y_pred = knn.predict(X_test)
print("cosine K-Nearest Neighbours")
truth_y_pred = [i for i, j in zip(y_pred, y_test) if  i == j]

print("len of truth_y_pre =", len(truth_y_pred))

false_y_pred = [i for i, j in zip(y_pred, y_test) if  i != j ]

print("len of false_y_pre =", len(false_y_pred))
print(metrics.accuracy_score(y_test, y_pred))


"""
8. euclidean K-Nearest Neighbours..............................................................................
"""
from sklearn.neighbors import KNeighborsClassifier
knn = KNeighborsClassifier(n_neighbors = k, metric= "euclidean")
knn.fit(X_train, y_train)
y_pred = knn.predict(X_test)
print("euclidean K-Nearest Neighbours")
truth_y_pred = [i for i, j in zip(y_pred, y_test) if  i == j]

print("len of truth_y_pre =", len(truth_y_pred))

false_y_pred = [i for i, j in zip(y_pred, y_test) if  i != j ]

print("len of false_y_pre =", len(false_y_pred))
print(metrics.accuracy_score(y_test, y_pred))

        
"""
9. scipy.spatial.distance K-Nearest Neighbours..............................................................................
"""
from scipy.spatial.distance import *
knn = KNeighborsClassifier(n_neighbors = k, metric= "braycurtis")
knn.fit(X_train, y_train)
y_pred = knn.predict(X_test)
print("braycurtis K-Nearest Neighbours")
truth_y_pred = [i for i, j in zip(y_pred, y_test) if  i == j]

print("len of truth_y_pre =", len(truth_y_pred))

false_y_pred = [i for i, j in zip(y_pred, y_test) if  i != j ]

print("len of false_y_pre =", len(false_y_pred))
print(metrics.accuracy_score(y_test, y_pred))
       
        
knn = KNeighborsClassifier(n_neighbors = k, metric= "canberra")
knn.fit(X_train, y_train)
y_pred = knn.predict(X_test)
print("canberra K-Nearest Neighbours")
truth_y_pred = [i for i, j in zip(y_pred, y_test) if  i == j]

print("len of truth_y_pre =", len(truth_y_pred))

false_y_pred = [i for i, j in zip(y_pred, y_test) if  i != j ]

print("len of false_y_pre =", len(false_y_pred))
print(metrics.accuracy_score(y_test, y_pred))
  
        
knn = KNeighborsClassifier(n_neighbors = k, metric= "jaccard")
knn.fit(X_train, y_train)
y_pred = knn.predict(X_test)
print("jaccard K-Nearest Neighbours")
truth_y_pred = [i for i, j in zip(y_pred, y_test) if  i == j]

print("len of truth_y_pre =", len(truth_y_pred))

false_y_pred = [i for i, j in zip(y_pred, y_test) if  i != j ]

print("len of false_y_pre =", len(false_y_pred))
print(metrics.accuracy_score(y_test, y_pred))
        
knn = KNeighborsClassifier(n_neighbors = k, metric= "yule")
knn.fit(X_train, y_train)
y_pred = knn.predict(X_test)
print("yule K-Nearest Neighbours")
truth_y_pred = [i for i, j in zip(y_pred, y_test) if  i == j]

print("len of truth_y_pre =", len(truth_y_pred))

false_y_pred = [i for i, j in zip(y_pred, y_test) if  i != j ]

print("len of false_y_pre =", len(false_y_pred))
print(metrics.accuracy_score(y_test, y_pred))
print("************************************ ****************************")

txt_files of features used ['/Users/catherine/Desktop/NLP/PAN Datasets/PAN2014blog/2014.1-blog-txtfiles/PAN2014_blog_female_chosen_set.1.csv', '/Users/catherine/Desktop/NLP/PAN Datasets/PAN2014blog/2014.1-blog-txtfiles/PAN2014_blog_male_chosen_set.1.csv']
/Users/catherine/Desktop/NLP/PAN Datasets/PAN2014blog/2014.1-blog-txtfiles/PAN2014_blog_female_chosen_set.1.csv 1526
/Users/catherine/Desktop/NLP/PAN Datasets/PAN2014blog/2014.1-blog-txtfiles/PAN2014_blog_male_chosen_set.1.csv 2063
length of features used = 3589
len(all_training_text) 147
len(X_train)= 147 len(y_train)= 147
len(all_test_text) 78
len(X_test= 78 len(y_test)= 78
selector = SelectKBest(score_func=f_classif, k=500)
5
manhattan K-Nearest Neighbours
0.6794871794871795
cosine K-Nearest Neighbours
0.6538461538461539
euclidean K-Nearest Neighbours
0.6410256410256411
braycurtis K-Nearest Neighbours
0.6025641025641025
canberra K-Nearest Neighbours
0.5
jaccard K-Nearest Neighbours
0.5769230769230769
yule K-Nearest Neighbours
0.551

/anaconda3/lib/python3.7/site-packages/sklearn/utils/validation.py:475: DataConversionWarning: Data with input dtype float64 was converted to bool by check_pairwise_arrays.
  warnings.warn(msg, DataConversionWarning)


In [ ]:
product(78, 0.6794871794871795)


In [41]:
import sys
sys.path.append("/anaconda3/lib/python3.7/site-packages")
import numpy as np
import numpy
import nltk
import pickle
import pandas as pd
import glob
import os
import re
import operator
from collections import Counter
#from read_xml_files import *
##from s_stemmer import *
from nltk.stem.porter import PorterStemmer
from nltk.stem import LancasterStemmer, WordNetLemmatizer
from itertools import groupby
from nltk.collocations import *
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.cluster import KMeans
from scipy.cluster.vq import whiten
from nltk.tokenize.toktok import ToktokTokenizer
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2
import xml.etree.ElementTree as ET
import pandas as pd
#Import scikit-learn metrics module for accuracy calculation
from sklearn import metrics
##from pattern.text.en import singularize
##nltk.download('vader_lexicon')
##nltk.download('punkt')
#import the relevant modules from the NLTK library
from nltk.sentiment.vader import SentimentIntensityAnalyzer
sentence_tokenizer = nltk.data.load('tokenizers/punkt/english.pickle')
word_tokenizer = nltk.tokenize.RegexpTokenizer(r'\w+')
##tt = ToktokTokenizer()#keeps the url as is...................................................
from nltk.tokenize import TweetTokenizer
tt = TweetTokenizer()
Porter_stemmer = PorterStemmer()
Lancaster_stemmer = LancasterStemmer()
WordNet_lemmatizer = WordNetLemmatizer()
#nltk.download('averaged_perceptron_tagger')
##Train_path="D:/Data/PAN2015-Profiling/pan15-author-profiling-training-dataset-english-2015-04-23"#path for train data
##Test_path="D:/Data/PAN2015-Profiling/pan-ap2015-test/en" #path for test data

##Train_path="D:/NLP/PAN2014/pan14-author-profiling-training-corpus-english-twitter-2014-04-16"#path for train data D:\Data\PAN2013-Profiling\Training\en  D:\NLP\PAN2014
##train_truth_path="D:/NLP/PAN2014/truth_train"
##
##test_truth_path="D:/NLP/PAN2014/pan14_test"
##Test_path="D:/NLP/PAN2014/pan14_test/en"
##Content = open('results_of_clssification_all.txt', 'w')

M_train_file= "/Users/catherine/Desktop/NLP/PAN Datasets/PAN2014blog/2014.1-blog-txtfiles/PAN2014_blog_male_train.1.txt"
F_train_file= "/Users/catherine/Desktop/NLP/PAN Datasets/PAN2014blog/2014.1-blog-txtfiles/PAN2014_blog_female_train.1.txt"

M_test_file= "/Users/catherine/Desktop/NLP/PAN Datasets/PAN2014blog/2014.1-blog-txtfiles/PAN2014_blog_male_test.1.txt"
F_test_file= "/Users/catherine/Desktop/NLP/PAN Datasets/PAN2014blog/2014.1-blog-txtfiles/PAN2014_blog_female_test.1.txt"

def all_txt(M_txt_file,  F_txt_file):
    with open(M_txt_file, mode="r", encoding="utf-8") as f:
            txt_list = [(line[:-1]).split("\t") for line in f]
            male_dict = dict()
            content=[]
            for line in txt_list:
                if line[1] in male_dict:
                # append the new number to the existing array at this slot
                    male_dict[line[1]].append(line[3])
                else:
                # create a new array in this slot
                    male_dict[line[1]] = [line[3]]
    male_content= [("M", (' '.join(a)).split()) for a in list(male_dict.values())]
  
    with open(F_txt_file, mode="r", encoding="utf-8") as f:
            txt_list = [(line[:-1]).split("\t") for line in f]
            female_dict = dict()
            content=[]
            for line in txt_list:
                if line[1] in female_dict:
                # append the new number to the existing array at this slot
                    female_dict[line[1]].append(line[3])
                else:
                # create a new array in this slot
                    female_dict[line[1]] = [line[3]]
    female_content= [("F", (' '.join(a)).split()) for a in list(female_dict.values())]
    all_txt_per_person = male_content + female_content
    return all_txt_per_person

##print( [((all_txt(M_txt_file,  F_txt_file))[i][0])  for i in range(len((all_txt(M_txt_file,  F_txt_file))))]) helps to determine the gender....

      
"""
create feature vector from the training set.....................................................................................
"""

def WordFeatures(word_list, all_training_text):

    fvs_words = np.array([[author.count(word) for word in word_list] for author in all_training_text]).astype(np.float64)

    # normalise by dividing each row by number of tokens for each author........
    fvs_words /= np.c_[np.array([len(author) for author in all_training_text])]

    return fvs_words



"""
From the saved csv file, recover the saved features to be used...............................................................
"""
import csv
word_list=[]
##txt_files =[ "/Users/catherine/Desktop/NLP/PAN Datasets/PAN_chosen_tokens/2014/output2014OR.1M.txt", "/Users/catherine/Desktop/NLP/PAN Datasets/PAN_chosen_tokens/2014/output2014OR.2F.txt"]
txt_files =[ "/Users/catherine/Desktop/NLP/PAN Datasets/PAN2014blog/2014.1-blog-txtfiles/PAN2014_blog_female_vocubulary.1.csv", "/Users/catherine/Desktop/NLP/PAN Datasets/PAN2014blog/2014.1-blog-txtfiles/PAN2014_blog_male_ vocubulary_set.1.csv"]
print("txt_files of features used",txt_files)
##for txt_file in txt_files:
##    with open(txt_file, mode="r", encoding="utf-8") as f:
##        reader = csv.reader(f, delimiter=",")
##        next(reader) # skip header
##        word_list1 =  ([(r.split(":"))[1] for r in f])
##        print(txt_file, len(word_list1))
##        word_list = word_list + word_list1
##print("length of features used =", len(word_list))

for txt_file in txt_files:
    with open(txt_file, mode="r", encoding="utf-8") as f:
        reader = csv.reader(f, delimiter=",")
        next(reader) # skip header
        word_list1 =  [r[0] for r in reader]
        print(txt_file, len(word_list1))
        word_list = word_list + word_list1
print("length of features used =", len(word_list))


"""
Prepare the training and test sets to be parsed to the classifies............................................................
"""
A=(all_txt(M_train_file,  F_train_file))
all_training_text = ( [(A[i][1])  for i in range(len(A))])
print("len(all_training_text)",len(all_training_text))
X_train=WordFeatures(word_list, all_training_text)
y_train=np.array( [(A[i][0])  for i in range(len(A))])
print("len(X_train)=",len(X_train), "len(y_train)=", len(y_train))

B=(all_txt(M_test_file,  F_test_file))
all_test_text = ( [(B[i][1])  for i in range(len(B))])
print("len(all_test_text)",len(all_test_text))
X_test = WordFeatures(word_list, all_test_text)
y_test = np.array( [(B[i][0])  for i in range(len(B))])
print("len(X_test=",len(X_test), "len(y_test)=",len(y_test))


X_train = np.nan_to_num(X_train)
X_test = np.nan_to_num(X_test)

"""
Perform feature selection using chi2............................................................
"""
from sklearn.feature_selection import *
##
####def select_Bestfeatures(X, y):
#print("selector = SelectKBest(score_func=mutual_info_classif, k=500)")
#selector = SelectKBest(score_func=mutual_info_classif, k=500)
#fit = selector.fit(X_train,y_train)
#train_selected_features = fit.transform(X_train)
#test_selected_features = fit.transform(X_test)



"""
x_train and x_test based on the new features using chi2............................................................
"""
##X_train = select_Bestfeatures(X_train, y_train)
##X_test = select_Bestfeatures(X_test, y_test)


"""
Perform feature selection using f_classif............................................................
"""
from sklearn.feature_selection import *
##def f_classif_select_Bestfeatures(X, y):
##selector=SelectKBest(score_func=f_classif,k=500)
##fit = selector.fit(X_train,y_train)
##train_selected_features = fit.transform(X_train)
##test_selected_features = fit.transform(X_test)
##    selected_features = fit.transform(X)
##    return selected_features

"""
x_train and x_test based on the new features using f_classif............................................................
"""
##X_train = f_classif_select_Bestfeatures(X_train, y_train)
##X_test = f_classif_select_Bestfeatures(X_test, y_test)



"""
Perform feature selection using mutual_info_classif............................................................
"""
from sklearn.feature_selection import *
##def mutual_info_classif_select_Bestfeatures(X_tr, X_te, y):
##selector=SelectKBest(score_func=mutual_info_classif,k=100)
##fit = selector.fit(X_train,y_train)
##train_selected_features = fit.transform(X_train)
##test_selected_features = fit.transform(X_test)
##    return train_selected_features, test_selected_features

"""
x_train and x_test based on the new features using f_classif............................................................
"""
#X_train =train_selected_features 
#X_test = test_selected_features


"""
Perform feature selection using pca............................................................
"""

##from sklearn.decomposition import PCA
##def PCA_select_Bestfeatures(X):
##    # feature extraction
##    pca = PCA(n_components=200)
##    fit = pca.fit(X, y)
##    Train_selected_features = fit.transform(X)
##    return selected_features

"""
x_train and x_test based on the new features using f_classif............................................................
"""
##X_train = PCA_select_Bestfeatures(X_train)
##X_test = PCA_select_Bestfeatures(X_test)
def tanimoto_distance(x,y):
    """Calculates the tanimoto (normalised manhattan) distance between the vectors x and y
            Keyword arguments:
                x,y -- the vectors between which the distance is to be calculated
    """
    try:
        iter(x)
    except TypeError:
        logging.warning( 'Argument x is not iterable. None is returned')
        return None
    try:
        iter(y)
    except TypeError:
        logging.warning( 'Argument y is not iterable. None is returned')
        return None
    numerator = sum(abs(a - b) for a, b in zip(x, y))
    denominator = sum(max(a,b) for a,b in zip(x,y))
    return numerator/denominator
"""
Below are a list of Classification Algorithms with different distance measres ..............................................................................
"""
"""
K-Nearest Neighbours (average aa over all the nearest neighbour selectiopn)..............................................................................
"""

k=5

print(k)

"""
8.0. tanimoto_distance K-Nearest Neighbours..............................................................................
"""
from sklearn.neighbors import KNeighborsClassifier
knn = KNeighborsClassifier(n_neighbors = k, metric= tanimoto_distance)
knn.fit(X_train, y_train)
y_pred = knn.predict(X_test)
print("tanimoto_distance K-Nearest Neighbours")
truth_y_pred = [i for i, j in zip(y_pred, y_test) if  i == j]

print("len of truth_y_pre =", len(truth_y_pred))

false_y_pred = [i for i, j in zip(y_pred, y_test) if  i != j ]

print("len of false_y_pre =", len(false_y_pred))
print(metrics.accuracy_score(y_test, y_pred))

"""
8. manhattan K-Nearest Neighbours..............................................................................
"""
from sklearn.neighbors import KNeighborsClassifier
knn = KNeighborsClassifier(n_neighbors = k, metric= "manhattan")
knn.fit(X_train, y_train)
y_pred = knn.predict(X_test)
print("manhattan K-Nearest Neighbours")
truth_y_pred = [i for i, j in zip(y_pred, y_test) if  i == j]

print("len of truth_y_pre =", len(truth_y_pred))

false_y_pred = [i for i, j in zip(y_pred, y_test) if  i != j ]

print("len of false_y_pre =", len(false_y_pred))
print(metrics.accuracy_score(y_test, y_pred))

    
"""
8. cosine K-Nearest Neighbours..............................................................................
"""
from sklearn.neighbors import KNeighborsClassifier
knn = KNeighborsClassifier(n_neighbors = k, metric= "cosine")
knn.fit(X_train, y_train)
y_pred = knn.predict(X_test)
print("cosine K-Nearest Neighbours")
truth_y_pred = [i for i, j in zip(y_pred, y_test) if  i == j]

print("len of truth_y_pre =", len(truth_y_pred))

false_y_pred = [i for i, j in zip(y_pred, y_test) if  i != j ]

print("len of false_y_pre =", len(false_y_pred))
print(metrics.accuracy_score(y_test, y_pred))


"""
8. euclidean K-Nearest Neighbours..............................................................................
"""
from sklearn.neighbors import KNeighborsClassifier
knn = KNeighborsClassifier(n_neighbors = k, metric= "euclidean")
knn.fit(X_train, y_train)
y_pred = knn.predict(X_test)
print("euclidean K-Nearest Neighbours")
truth_y_pred = [i for i, j in zip(y_pred, y_test) if  i == j]

print("len of truth_y_pre =", len(truth_y_pred))

false_y_pred = [i for i, j in zip(y_pred, y_test) if  i != j ]

print("len of false_y_pre =", len(false_y_pred))
print(metrics.accuracy_score(y_test, y_pred))

        
"""
9. scipy.spatial.distance K-Nearest Neighbours..............................................................................
"""
from scipy.spatial.distance import *
knn = KNeighborsClassifier(n_neighbors = k, metric= "braycurtis")
knn.fit(X_train, y_train)
y_pred = knn.predict(X_test)
print("braycurtis K-Nearest Neighbours")
truth_y_pred = [i for i, j in zip(y_pred, y_test) if  i == j]

print("len of truth_y_pre =", len(truth_y_pred))

false_y_pred = [i for i, j in zip(y_pred, y_test) if  i != j ]

print("len of false_y_pre =", len(false_y_pred))
print(metrics.accuracy_score(y_test, y_pred))
       
        
knn = KNeighborsClassifier(n_neighbors = k, metric= "canberra")
knn.fit(X_train, y_train)
y_pred = knn.predict(X_test)
print("canberra K-Nearest Neighbours")
truth_y_pred = [i for i, j in zip(y_pred, y_test) if  i == j]

print("len of truth_y_pre =", len(truth_y_pred))

false_y_pred = [i for i, j in zip(y_pred, y_test) if  i != j ]

print("len of false_y_pre =", len(false_y_pred))
print(metrics.accuracy_score(y_test, y_pred))
  
        
knn = KNeighborsClassifier(n_neighbors = k, metric= "jaccard")
knn.fit(X_train, y_train)
y_pred = knn.predict(X_test)
print("jaccard K-Nearest Neighbours")
print(metrics.accuracy_score(y_test, y_pred))
        
knn = KNeighborsClassifier(n_neighbors = k, metric= "yule")
knn.fit(X_train, y_train)
y_pred = knn.predict(X_test)
print("yule K-Nearest Neighbours")
truth_y_pred = [i for i, j in zip(y_pred, y_test) if  i == j]

print("len of truth_y_pre =", len(truth_y_pred))

false_y_pred = [i for i, j in zip(y_pred, y_test) if  i != j ]

print("len of false_y_pre =", len(false_y_pred))
print(metrics.accuracy_score(y_test, y_pred))


knn = KNeighborsClassifier(n_neighbors = k, metric= "rogerstanimoto")
knn.fit(X_train, y_train)
y_pred = knn.predict(X_test)
print("rogerstanimoto K-Nearest Neighbours")
truth_y_pred = [i for i, j in zip(y_pred, y_test) if  i == j]

print("len of truth_y_pre =", len(truth_y_pred))

false_y_pred = [i for i, j in zip(y_pred, y_test) if  i != j ]

print("len of false_y_pre =", len(false_y_pred))
print(metrics.accuracy_score(y_test, y_pred))
print("************************************ ****************************")

txt_files of features used ['/Users/catherine/Desktop/NLP/PAN Datasets/PAN2014blog/2014.1-blog-txtfiles/PAN2014_blog_female_vocubulary.1.csv', '/Users/catherine/Desktop/NLP/PAN Datasets/PAN2014blog/2014.1-blog-txtfiles/PAN2014_blog_male_ vocubulary_set.1.csv']
/Users/catherine/Desktop/NLP/PAN Datasets/PAN2014blog/2014.1-blog-txtfiles/PAN2014_blog_female_vocubulary.1.csv 12375
/Users/catherine/Desktop/NLP/PAN Datasets/PAN2014blog/2014.1-blog-txtfiles/PAN2014_blog_male_ vocubulary_set.1.csv 18889
length of features used = 31264
len(all_training_text) 147
len(X_train)= 147 len(y_train)= 147
len(all_test_text) 78
len(X_test= 78 len(y_test)= 78
5
tanimoto_distance K-Nearest Neighbours
0.6410256410256411
manhattan K-Nearest Neighbours
0.6666666666666666
cosine K-Nearest Neighbours
0.6538461538461539
euclidean K-Nearest Neighbours
0.6410256410256411
braycurtis K-Nearest Neighbours
0.6410256410256411
canberra K-Nearest Neighbours
0.5128205128205128
jaccard K-Nearest Neighbours
0.52564102564102

/anaconda3/lib/python3.7/site-packages/sklearn/utils/validation.py:475: DataConversionWarning: Data with input dtype float64 was converted to bool by check_pairwise_arrays.
  warnings.warn(msg, DataConversionWarning)


yule K-Nearest Neighbours
0.5897435897435898
rogerstanimoto K-Nearest Neighbours
0.5897435897435898
************************************ ****************************


In [4]:
import sys
sys.path.append("/anaconda3/lib/python3.7/site-packages")
import numpy as np
import numpy
import nltk
import pickle
import pandas as pd
import glob
import os
import re
import operator
from collections import Counter
#from read_xml_files import *
##from s_stemmer import *
from nltk.stem.porter import PorterStemmer
from nltk.stem import LancasterStemmer, WordNetLemmatizer
from itertools import groupby
from nltk.collocations import *
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.cluster import KMeans
from scipy.cluster.vq import whiten
from nltk.tokenize.toktok import ToktokTokenizer
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2
import xml.etree.ElementTree as ET
import pandas as pd
#Import scikit-learn metrics module for accuracy calculation
from sklearn import metrics
##from pattern.text.en import singularize
##nltk.download('vader_lexicon')
##nltk.download('punkt')
#import the relevant modules from the NLTK library
from nltk.sentiment.vader import SentimentIntensityAnalyzer
sentence_tokenizer = nltk.data.load('tokenizers/punkt/english.pickle')
word_tokenizer = nltk.tokenize.RegexpTokenizer(r'\w+')
##tt = ToktokTokenizer()#keeps the url as is...................................................
from nltk.tokenize import TweetTokenizer
tt = TweetTokenizer()
Porter_stemmer = PorterStemmer()
Lancaster_stemmer = LancasterStemmer()
WordNet_lemmatizer = WordNetLemmatizer()
#nltk.download('averaged_perceptron_tagger')
##Train_path="D:/Data/PAN2015-Profiling/pan15-author-profiling-training-dataset-english-2015-04-23"#path for train data
##Test_path="D:/Data/PAN2015-Profiling/pan-ap2015-test/en" #path for test data

##Train_path="D:/NLP/PAN2014/pan14-author-profiling-training-corpus-english-twitter-2014-04-16"#path for train data D:\Data\PAN2013-Profiling\Training\en  D:\NLP\PAN2014
##train_truth_path="D:/NLP/PAN2014/truth_train"
##
##test_truth_path="D:/NLP/PAN2014/pan14_test"
##Test_path="D:/NLP/PAN2014/pan14_test/en"
##Content = open('results_of_clssification_all.txt', 'w')

M_train_file= "/Users/catherine/Desktop/NLP/PAN_Datasets/PAN2014blog/2014.1-blog-txtfiles/PAN2014_blog_male_train.1.txt"
F_train_file= "/Users/catherine/Desktop/NLP/PAN_Datasets/PAN2014blog/2014.1-blog-txtfiles/PAN2014_blog_female_train.1.txt"

M_test_file= "/Users/catherine/Desktop/NLP/PAN_Datasets/PAN2014blog/2014.1-blog-txtfiles/PAN2014_blog_male_test.1.txt"
F_test_file= "/Users/catherine/Desktop/NLP/PAN_Datasets/PAN2014blog/2014.1-blog-txtfiles/PAN2014_blog_female_test.1.txt"

def all_txt(M_txt_file,  F_txt_file):
    with open(M_txt_file, mode="r", encoding="utf-8") as f:
            txt_list = [(line[:-1]).split("\t") for line in f]
            male_dict = dict()
            content=[]
            for line in txt_list:
                if line[1] in male_dict:
                # append the new number to the existing array at this slot
                    male_dict[line[1]].append(line[3])
                else:
                # create a new array in this slot
                    male_dict[line[1]] = [line[3]]
    male_content= [("M", (' '.join(a)).split()) for a in list(male_dict.values())]
  
    with open(F_txt_file, mode="r", encoding="utf-8") as f:
            txt_list = [(line[:-1]).split("\t") for line in f]
            female_dict = dict()
            content=[]
            for line in txt_list:
                if line[1] in female_dict:
                # append the new number to the existing array at this slot
                    female_dict[line[1]].append(line[3])
                else:
                # create a new array in this slot
                    female_dict[line[1]] = [line[3]]
    female_content= [("F", (' '.join(a)).split()) for a in list(female_dict.values())]
    all_txt_per_person = male_content + female_content
    return all_txt_per_person

##print( [((all_txt(M_txt_file,  F_txt_file))[i][0])  for i in range(len((all_txt(M_txt_file,  F_txt_file))))]) helps to determine the gender....

      
"""
create feature vector from the training set.....................................................................................
"""

def WordFeatures(word_list, all_training_text):

    fvs_words = np.array([[author.count(word) for word in word_list] for author in all_training_text]).astype(np.float64)

    # normalise by dividing each row by number of tokens for each author........
    fvs_words /= np.c_[np.array([len(author) for author in all_training_text])]

    return fvs_words



"""
From the saved csv file, recover the saved features to be used...............................................................
"""
import csv
word_list=[]
##txt_files =[ "/Users/catherine/Desktop/NLP/PAN Datasets/PAN_chosen_tokens/2014/output2014OR.1M.txt", "/Users/catherine/Desktop/NLP/PAN Datasets/PAN_chosen_tokens/2014/output2014OR.2F.txt"]
txt_files =[ "/Users/catherine/Desktop/NLP/PAN_Datasets/PAN2014blog/2014.1-blog-txtfiles/PAN2014_blog_female_vocubulary.1.csv", "/Users/catherine/Desktop/NLP/PAN_Datasets/PAN2014blog/2014.1-blog-txtfiles/PAN2014_blog_male_ vocubulary_set.1.csv"]
print("txt_files of features used",txt_files)
##for txt_file in txt_files:
##    with open(txt_file, mode="r", encoding="utf-8") as f:
##        reader = csv.reader(f, delimiter=",")
##        next(reader) # skip header
##        word_list1 =  ([(r.split(":"))[1] for r in f])
##        print(txt_file, len(word_list1))
##        word_list = word_list + word_list1
##print("length of features used =", len(word_list))

for txt_file in txt_files:
    with open(txt_file, mode="r", encoding="utf-8") as f:
        reader = csv.reader(f, delimiter=",")
        next(reader) # skip header
        word_list1 =  [r[0] for r in reader]
        print(txt_file, len(word_list1))
        word_list = word_list + word_list1
print("length of features used =", len(word_list))


"""
Prepare the training and test sets to be parsed to the classifies............................................................
"""
A=(all_txt(M_train_file,  F_train_file))
all_training_text = ( [(A[i][1])  for i in range(len(A))])
print("len(all_training_text)",len(all_training_text))
X_train=WordFeatures(word_list, all_training_text)
y_train=np.array( [(A[i][0])  for i in range(len(A))])
print("len(X_train)=",len(X_train), "len(y_train)=", len(y_train))

B=(all_txt(M_test_file,  F_test_file))
all_test_text = ( [(B[i][1])  for i in range(len(B))])
print("len(all_test_text)",len(all_test_text))
X_test = WordFeatures(word_list, all_test_text)
y_test = np.array( [(B[i][0])  for i in range(len(B))])
print("len(X_test=",len(X_test), "len(y_test)=",len(y_test))


X_train = np.nan_to_num(X_train)
X_test = np.nan_to_num(X_test)

"""
Perform feature selection using chi2............................................................
"""
from sklearn.feature_selection import *
##
####def select_Bestfeatures(X, y):
#print("selector = SelectKBest(score_func=mutual_info_classif, k=500)")
#selector = SelectKBest(score_func=mutual_info_classif, k=500)
#fit = selector.fit(X_train,y_train)
#train_selected_features = fit.transform(X_train)
#test_selected_features = fit.transform(X_test)



"""
x_train and x_test based on the new features using chi2............................................................
"""
##X_train = select_Bestfeatures(X_train, y_train)
##X_test = select_Bestfeatures(X_test, y_test)


"""
Perform feature selection using f_classif............................................................
"""
from sklearn.feature_selection import *
##def f_classif_select_Bestfeatures(X, y):
##selector=SelectKBest(score_func=f_classif,k=500)
##fit = selector.fit(X_train,y_train)
##train_selected_features = fit.transform(X_train)
##test_selected_features = fit.transform(X_test)
##    selected_features = fit.transform(X)
##    return selected_features

"""
x_train and x_test based on the new features using f_classif............................................................
"""
##X_train = f_classif_select_Bestfeatures(X_train, y_train)
##X_test = f_classif_select_Bestfeatures(X_test, y_test)



"""
Perform feature selection using mutual_info_classif............................................................
"""
from sklearn.feature_selection import *
##def mutual_info_classif_select_Bestfeatures(X_tr, X_te, y):
##selector=SelectKBest(score_func=mutual_info_classif,k=100)
##fit = selector.fit(X_train,y_train)
##train_selected_features = fit.transform(X_train)
##test_selected_features = fit.transform(X_test)
##    return train_selected_features, test_selected_features

"""
x_train and x_test based on the new features using f_classif............................................................
"""
#X_train =train_selected_features 
#X_test = test_selected_features


"""
Perform feature selection using pca............................................................
"""

##from sklearn.decomposition import PCA
##def PCA_select_Bestfeatures(X):
##    # feature extraction
##    pca = PCA(n_components=200)
##    fit = pca.fit(X, y)
##    Train_selected_features = fit.transform(X)
##    return selected_features

"""
x_train and x_test based on the new features using f_classif............................................................
"""
##X_train = PCA_select_Bestfeatures(X_train)
##X_test = PCA_select_Bestfeatures(X_test)
def tanimoto_distance(x,y):
    """Calculates the tanimoto (normalised manhattan) distance between the vectors x and y
            Keyword arguments:
                x,y -- the vectors between which the distance is to be calculated
    """
    try:
        iter(x)
    except TypeError:
        logging.warning( 'Argument x is not iterable. None is returned')
        return None
    try:
        iter(y)
    except TypeError:
        logging.warning( 'Argument y is not iterable. None is returned')
        return None
    numerator = sum(abs(a - b) for a, b in zip(x, y))
    denominator = sum(max(a,b) for a,b in zip(x,y))
    return numerator/denominator
"""
Below are a list of Classification Algorithms with different distance measres ..............................................................................
"""
"""
K-Nearest Neighbours (average aa over all the nearest neighbour selectiopn)..............................................................................
"""

k=5

print(k)

"""
8.0. tanimoto_distance K-Nearest Neighbours..............................................................................
"""
from sklearn.neighbors import KNeighborsClassifier
knn = KNeighborsClassifier(n_neighbors = k, metric= tanimoto_distance)
knn.fit(X_train, y_train)
y_pred = knn.predict(X_test)
print("tanimoto_distance K-Nearest Neighbours")
truth_y_pred = [i for i, j in zip(y_pred, y_test) if  i == j]

print("len of truth_y_pre =", len(truth_y_pred))

false_y_pred = [i for i, j in zip(y_pred, y_test) if  i != j ]

print("len of false_y_pre =", len(false_y_pred))
print(metrics.accuracy_score(y_test, y_pred))

"""
8. manhattan K-Nearest Neighbours..............................................................................
"""
from sklearn.neighbors import KNeighborsClassifier
knn = KNeighborsClassifier(n_neighbors = k, metric= "manhattan")
knn.fit(X_train, y_train)
y_pred = knn.predict(X_test)
print("manhattan K-Nearest Neighbours")
truth_y_pred = [i for i, j in zip(y_pred, y_test) if  i == j]

print("len of truth_y_pre =", len(truth_y_pred))

false_y_pred = [i for i, j in zip(y_pred, y_test) if  i != j ]

print("len of false_y_pre =", len(false_y_pred))
print(metrics.accuracy_score(y_test, y_pred))

    
"""
8. cosine K-Nearest Neighbours..............................................................................
"""
from sklearn.neighbors import KNeighborsClassifier
knn = KNeighborsClassifier(n_neighbors = k, metric= "cosine")
knn.fit(X_train, y_train)
y_pred = knn.predict(X_test)
print("cosine K-Nearest Neighbours")
truth_y_pred = [i for i, j in zip(y_pred, y_test) if  i == j]

print("len of truth_y_pre =", len(truth_y_pred))

false_y_pred = [i for i, j in zip(y_pred, y_test) if  i != j ]

print("len of false_y_pre =", len(false_y_pred))
print(metrics.accuracy_score(y_test, y_pred))


"""
8. euclidean K-Nearest Neighbours..............................................................................
"""
from sklearn.neighbors import KNeighborsClassifier
knn = KNeighborsClassifier(n_neighbors = k, metric= "euclidean")
knn.fit(X_train, y_train)
y_pred = knn.predict(X_test)
print("euclidean K-Nearest Neighbours")
truth_y_pred = [i for i, j in zip(y_pred, y_test) if  i == j]

print("len of truth_y_pre =", len(truth_y_pred))

false_y_pred = [i for i, j in zip(y_pred, y_test) if  i != j ]

print("len of false_y_pre =", len(false_y_pred))
print(metrics.accuracy_score(y_test, y_pred))

        
"""
9. scipy.spatial.distance K-Nearest Neighbours..............................................................................
"""
from scipy.spatial.distance import *
knn = KNeighborsClassifier(n_neighbors = k, metric= "braycurtis")
knn.fit(X_train, y_train)
y_pred = knn.predict(X_test)
print("braycurtis K-Nearest Neighbours")
truth_y_pred = [i for i, j in zip(y_pred, y_test) if  i == j]

print("len of truth_y_pre =", len(truth_y_pred))

false_y_pred = [i for i, j in zip(y_pred, y_test) if  i != j ]

print("len of false_y_pre =", len(false_y_pred))
print(metrics.accuracy_score(y_test, y_pred))
       
        
knn = KNeighborsClassifier(n_neighbors = k, metric= "canberra")
knn.fit(X_train, y_train)
y_pred = knn.predict(X_test)
print("canberra K-Nearest Neighbours")
truth_y_pred = [i for i, j in zip(y_pred, y_test) if  i == j]

print("len of truth_y_pre =", len(truth_y_pred))

false_y_pred = [i for i, j in zip(y_pred, y_test) if  i != j ]

print("len of false_y_pre =", len(false_y_pred))
print(metrics.accuracy_score(y_test, y_pred))
  
        
knn = KNeighborsClassifier(n_neighbors = k, metric= "jaccard")
knn.fit(X_train, y_train)
y_pred = knn.predict(X_test)
print("jaccard K-Nearest Neighbours")
print(metrics.accuracy_score(y_test, y_pred))
        
knn = KNeighborsClassifier(n_neighbors = k, metric= "yule")
knn.fit(X_train, y_train)
y_pred = knn.predict(X_test)
print("yule K-Nearest Neighbours")
truth_y_pred = [i for i, j in zip(y_pred, y_test) if  i == j]

print("len of truth_y_pre =", len(truth_y_pred))

false_y_pred = [i for i, j in zip(y_pred, y_test) if  i != j ]

print("len of false_y_pre =", len(false_y_pred))
print(metrics.accuracy_score(y_test, y_pred))


knn = KNeighborsClassifier(n_neighbors = k, metric= "rogerstanimoto")
knn.fit(X_train, y_train)
y_pred = knn.predict(X_test)
print("rogerstanimoto K-Nearest Neighbours")
truth_y_pred = [i for i, j in zip(y_pred, y_test) if  i == j]

print("len of truth_y_pre =", len(truth_y_pred))

false_y_pred = [i for i, j in zip(y_pred, y_test) if  i != j ]

print("len of false_y_pre =", len(false_y_pred))
print(metrics.accuracy_score(y_test, y_pred))
print("************************************ ****************************")

txt_files of features used ['/Users/catherine/Desktop/NLP/PAN_Datasets/PAN2014blog/2014.1-blog-txtfiles/PAN2014_blog_female_vocubulary.1.csv', '/Users/catherine/Desktop/NLP/PAN_Datasets/PAN2014blog/2014.1-blog-txtfiles/PAN2014_blog_male_ vocubulary_set.1.csv']
/Users/catherine/Desktop/NLP/PAN_Datasets/PAN2014blog/2014.1-blog-txtfiles/PAN2014_blog_female_vocubulary.1.csv 12375
/Users/catherine/Desktop/NLP/PAN_Datasets/PAN2014blog/2014.1-blog-txtfiles/PAN2014_blog_male_ vocubulary_set.1.csv 18889
length of features used = 31264
len(all_training_text) 147
len(X_train)= 147 len(y_train)= 147
len(all_test_text) 78
len(X_test= 78 len(y_test)= 78
5
tanimoto_distance K-Nearest Neighbours
len of truth_y_pre = 50
len of false_y_pre = 28
0.6410256410256411
manhattan K-Nearest Neighbours
len of truth_y_pre = 52
len of false_y_pre = 26
0.6666666666666666
cosine K-Nearest Neighbours
len of truth_y_pre = 51
len of false_y_pre = 27
0.6538461538461539
euclidean K-Nearest Neighbours
len of truth_y_pre =

/anaconda3/lib/python3.7/site-packages/sklearn/utils/validation.py:475: DataConversionWarning: Data with input dtype float64 was converted to bool by check_pairwise_arrays.
  warnings.warn(msg, DataConversionWarning)


yule K-Nearest Neighbours
len of truth_y_pre = 46
len of false_y_pre = 32
0.5897435897435898
rogerstanimoto K-Nearest Neighbours
len of truth_y_pre = 46
len of false_y_pre = 32
0.5897435897435898
************************************ ****************************


In [42]:
import sys
sys.path.append("/anaconda3/lib/python3.7/site-packages")
import numpy as np
import numpy
import nltk
import pickle
import pandas as pd
import glob
import os
import re
import operator
from collections import Counter
#from read_xml_files import *
##from s_stemmer import *
from nltk.stem.porter import PorterStemmer
from nltk.stem import LancasterStemmer, WordNetLemmatizer
from itertools import groupby
from nltk.collocations import *
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.cluster import KMeans
from scipy.cluster.vq import whiten
from nltk.tokenize.toktok import ToktokTokenizer
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2
import xml.etree.ElementTree as ET
import pandas as pd
#Import scikit-learn metrics module for accuracy calculation
from sklearn import metrics
##from pattern.text.en import singularize
##nltk.download('vader_lexicon')
##nltk.download('punkt')
#import the relevant modules from the NLTK library
from nltk.sentiment.vader import SentimentIntensityAnalyzer
sentence_tokenizer = nltk.data.load('tokenizers/punkt/english.pickle')
word_tokenizer = nltk.tokenize.RegexpTokenizer(r'\w+')
##tt = ToktokTokenizer()#keeps the url as is...................................................
from nltk.tokenize import TweetTokenizer
tt = TweetTokenizer()
Porter_stemmer = PorterStemmer()
Lancaster_stemmer = LancasterStemmer()
WordNet_lemmatizer = WordNetLemmatizer()
#nltk.download('averaged_perceptron_tagger')
##Train_path="D:/Data/PAN2015-Profiling/pan15-author-profiling-training-dataset-english-2015-04-23"#path for train data
##Test_path="D:/Data/PAN2015-Profiling/pan-ap2015-test/en" #path for test data

##Train_path="D:/NLP/PAN2014/pan14-author-profiling-training-corpus-english-twitter-2014-04-16"#path for train data D:\Data\PAN2013-Profiling\Training\en  D:\NLP\PAN2014
##train_truth_path="D:/NLP/PAN2014/truth_train"
##
##test_truth_path="D:/NLP/PAN2014/pan14_test"
##Test_path="D:/NLP/PAN2014/pan14_test/en"
##Content = open('results_of_clssification_all.txt', 'w')

M_train_file= "/Users/catherine/Desktop/NLP/PAN Datasets/PAN2014blog/2014.1-blog-txtfiles/PAN2014_blog_male_train.1.txt"
F_train_file= "/Users/catherine/Desktop/NLP/PAN Datasets/PAN2014blog/2014.1-blog-txtfiles/PAN2014_blog_female_train.1.txt"

M_test_file= "/Users/catherine/Desktop/NLP/PAN Datasets/PAN2014blog/2014.1-blog-txtfiles/PAN2014_blog_male_test.1.txt"
F_test_file= "/Users/catherine/Desktop/NLP/PAN Datasets/PAN2014blog/2014.1-blog-txtfiles/PAN2014_blog_female_test.1.txt"

def all_txt(M_txt_file,  F_txt_file):
    with open(M_txt_file, mode="r", encoding="utf-8") as f:
            txt_list = [(line[:-1]).split("\t") for line in f]
            male_dict = dict()
            content=[]
            for line in txt_list:
                if line[1] in male_dict:
                # append the new number to the existing array at this slot
                    male_dict[line[1]].append(line[3])
                else:
                # create a new array in this slot
                    male_dict[line[1]] = [line[3]]
    male_content= [("M", (' '.join(a)).split()) for a in list(male_dict.values())]
  
    with open(F_txt_file, mode="r", encoding="utf-8") as f:
            txt_list = [(line[:-1]).split("\t") for line in f]
            female_dict = dict()
            content=[]
            for line in txt_list:
                if line[1] in female_dict:
                # append the new number to the existing array at this slot
                    female_dict[line[1]].append(line[3])
                else:
                # create a new array in this slot
                    female_dict[line[1]] = [line[3]]
    female_content= [("F", (' '.join(a)).split()) for a in list(female_dict.values())]
    all_txt_per_person = male_content + female_content
    return all_txt_per_person

##print( [((all_txt(M_txt_file,  F_txt_file))[i][0])  for i in range(len((all_txt(M_txt_file,  F_txt_file))))]) helps to determine the gender....

      
"""
create feature vector from the training set.....................................................................................
"""

def WordFeatures(word_list, all_training_text):

    fvs_words = np.array([[author.count(word) for word in word_list] for author in all_training_text]).astype(np.float64)

    # normalise by dividing each row by number of tokens for each author........
    fvs_words /= np.c_[np.array([len(author) for author in all_training_text])]

    return fvs_words



"""
From the saved csv file, recover the saved features to be used...............................................................
"""
import csv
word_list=[]
##txt_files =[ "/Users/catherine/Desktop/NLP/PAN Datasets/PAN_chosen_tokens/2014/output2014OR.1M.txt", "/Users/catherine/Desktop/NLP/PAN Datasets/PAN_chosen_tokens/2014/output2014OR.2F.txt"]
txt_files =[ "/Users/catherine/Desktop/NLP/PAN Datasets/PAN2014blog/2014.1-blog-txtfiles/PAN2014_blog_female_chosen_set.1.csv", "/Users/catherine/Desktop/NLP/PAN Datasets/PAN2014blog/2014.1-blog-txtfiles/PAN2014_blog_male_chosen_set.1.csv"]
print("txt_files of features used",txt_files)
##for txt_file in txt_files:
##    with open(txt_file, mode="r", encoding="utf-8") as f:
##        reader = csv.reader(f, delimiter=",")
##        next(reader) # skip header
##        word_list1 =  ([(r.split(":"))[1] for r in f])
##        print(txt_file, len(word_list1))
##        word_list = word_list + word_list1
##print("length of features used =", len(word_list))

for txt_file in txt_files:
    with open(txt_file, mode="r", encoding="utf-8") as f:
        reader = csv.reader(f, delimiter=",")
        next(reader) # skip header
        word_list1 =  [r[0] for r in reader]
        print(txt_file, len(word_list1))
        word_list = word_list + word_list1
print("length of features used =", len(word_list))


"""
Prepare the training and test sets to be parsed to the classifies............................................................
"""
A=(all_txt(M_train_file,  F_train_file))
all_training_text = ( [(A[i][1])  for i in range(len(A))])
print("len(all_training_text)",len(all_training_text))
X_train=WordFeatures(word_list, all_training_text)
y_train=np.array( [(A[i][0])  for i in range(len(A))])
print("len(X_train)=",len(X_train), "len(y_train)=", len(y_train))

B=(all_txt(M_test_file,  F_test_file))
all_test_text = ( [(B[i][1])  for i in range(len(B))])
print("len(all_test_text)",len(all_test_text))
X_test = WordFeatures(word_list, all_test_text)
y_test = np.array( [(B[i][0])  for i in range(len(B))])
print("len(X_test=",len(X_test), "len(y_test)=",len(y_test))


X_train = np.nan_to_num(X_train)
X_test = np.nan_to_num(X_test)

"""
Perform feature selection using chi2............................................................
"""
from sklearn.feature_selection import *
##
####def select_Bestfeatures(X, y):
print("selector = SelectKBest(score_func=f_classif, k=500)")
#selector = SelectKBest(score_func=f_classif, k=500)
#fit = selector.fit(X_train,y_train)
#train_selected_features = fit.transform(X_train)
#test_selected_features = fit.transform(X_test)



"""
x_train and x_test based on the new features using chi2............................................................
"""
##X_train = select_Bestfeatures(X_train, y_train)
##X_test = select_Bestfeatures(X_test, y_test)


"""
Perform feature selection using f_classif............................................................
"""
from sklearn.feature_selection import *
##def f_classif_select_Bestfeatures(X, y):
##selector=SelectKBest(score_func=f_classif,k=500)
##fit = selector.fit(X_train,y_train)
##train_selected_features = fit.transform(X_train)
##test_selected_features = fit.transform(X_test)
##    selected_features = fit.transform(X)
##    return selected_features

"""
x_train and x_test based on the new features using f_classif............................................................
"""
##X_train = f_classif_select_Bestfeatures(X_train, y_train)
##X_test = f_classif_select_Bestfeatures(X_test, y_test)



"""
Perform feature selection using mutual_info_classif............................................................
"""
from sklearn.feature_selection import *
##def mutual_info_classif_select_Bestfeatures(X_tr, X_te, y):
##selector=SelectKBest(score_func=mutual_info_classif,k=100)
##fit = selector.fit(X_train,y_train)
##train_selected_features = fit.transform(X_train)
##test_selected_features = fit.transform(X_test)
##    return train_selected_features, test_selected_features

"""
x_train and x_test based on the new features using f_classif............................................................
"""
#X_train =train_selected_features 
#X_test = test_selected_features


"""
Perform feature selection using pca............................................................
"""

##from sklearn.decomposition import PCA
##def PCA_select_Bestfeatures(X):
##    # feature extraction
##    pca = PCA(n_components=200)
##    fit = pca.fit(X, y)
##    Train_selected_features = fit.transform(X)
##    return selected_features

"""
x_train and x_test based on the new features using f_classif............................................................
"""
##X_train = PCA_select_Bestfeatures(X_train)
##X_test = PCA_select_Bestfeatures(X_test)
def tanimoto_distance(x,y):
    """Calculates the tanimoto (normalised manhattan) distance between the vectors x and y
            Keyword arguments:
                x,y -- the vectors between which the distance is to be calculated
    """
    try:
        iter(x)
    except TypeError:
        logging.warning( 'Argument x is not iterable. None is returned')
        return None
    try:
        iter(y)
    except TypeError:
        logging.warning( 'Argument y is not iterable. None is returned')
        return None
    numerator = sum(abs(a - b) for a, b in zip(x, y))
    denominator = sum(max(a,b) for a,b in zip(x,y))
    return numerator/denominator
"""
Below are a list of Classification Algorithms with different distance measres ..............................................................................
"""
"""
K-Nearest Neighbours (average aa over all the nearest neighbour selectiopn)..............................................................................
"""

k=5

print(k)

"""
8.0. tanimoto_distance K-Nearest Neighbours..............................................................................
"""
from sklearn.neighbors import KNeighborsClassifier
knn = KNeighborsClassifier(n_neighbors = k, metric= tanimoto_distance)
knn.fit(X_train, y_train)
y_pred = knn.predict(X_test)
print("tanimoto_distance K-Nearest Neighbours")
print(metrics.accuracy_score(y_test, y_pred))


"""
8. manhattan K-Nearest Neighbours..............................................................................
"""
from sklearn.neighbors import KNeighborsClassifier
knn = KNeighborsClassifier(n_neighbors = k, metric= "manhattan")
knn.fit(X_train, y_train)
y_pred = knn.predict(X_test)
print("manhattan K-Nearest Neighbours")
print(metrics.accuracy_score(y_test, y_pred))

    
"""
8. cosine K-Nearest Neighbours..............................................................................
"""
from sklearn.neighbors import KNeighborsClassifier
knn = KNeighborsClassifier(n_neighbors = k, metric= "cosine")
knn.fit(X_train, y_train)
y_pred = knn.predict(X_test)
print("cosine K-Nearest Neighbours")
print(metrics.accuracy_score(y_test, y_pred))


"""
8. euclidean K-Nearest Neighbours..............................................................................
"""
from sklearn.neighbors import KNeighborsClassifier
knn = KNeighborsClassifier(n_neighbors = k, metric= "euclidean")
knn.fit(X_train, y_train)
y_pred = knn.predict(X_test)
print("euclidean K-Nearest Neighbours")
print(metrics.accuracy_score(y_test, y_pred))

        
"""
9. scipy.spatial.distance K-Nearest Neighbours..............................................................................
"""
from scipy.spatial.distance import *
knn = KNeighborsClassifier(n_neighbors = k, metric= "braycurtis")
knn.fit(X_train, y_train)
y_pred = knn.predict(X_test)
print("braycurtis K-Nearest Neighbours")
print(metrics.accuracy_score(y_test, y_pred))
       
        
knn = KNeighborsClassifier(n_neighbors = k, metric= "canberra")
knn.fit(X_train, y_train)
y_pred = knn.predict(X_test)
print("canberra K-Nearest Neighbours")
print(metrics.accuracy_score(y_test, y_pred))
  
        
knn = KNeighborsClassifier(n_neighbors = k, metric= "jaccard")
knn.fit(X_train, y_train)
y_pred = knn.predict(X_test)
print("jaccard K-Nearest Neighbours")
print(metrics.accuracy_score(y_test, y_pred))
        
knn = KNeighborsClassifier(n_neighbors = k, metric= "yule")
knn.fit(X_train, y_train)
y_pred = knn.predict(X_test)
print("yule K-Nearest Neighbours")
print(metrics.accuracy_score(y_test, y_pred))
print("************************************ ****************************")

txt_files of features used ['/Users/catherine/Desktop/NLP/PAN Datasets/PAN2014blog/2014.1-blog-txtfiles/PAN2014_blog_female_chosen_set.1.csv', '/Users/catherine/Desktop/NLP/PAN Datasets/PAN2014blog/2014.1-blog-txtfiles/PAN2014_blog_male_chosen_set.1.csv']
/Users/catherine/Desktop/NLP/PAN Datasets/PAN2014blog/2014.1-blog-txtfiles/PAN2014_blog_female_chosen_set.1.csv 1526
/Users/catherine/Desktop/NLP/PAN Datasets/PAN2014blog/2014.1-blog-txtfiles/PAN2014_blog_male_chosen_set.1.csv 2063
length of features used = 3589
len(all_training_text) 147
len(X_train)= 147 len(y_train)= 147
len(all_test_text) 78
len(X_test= 78 len(y_test)= 78
selector = SelectKBest(score_func=f_classif, k=500)
5
tanimoto_distance K-Nearest Neighbours
0.6025641025641025
manhattan K-Nearest Neighbours
0.6794871794871795
cosine K-Nearest Neighbours
0.6538461538461539
euclidean K-Nearest Neighbours
0.6410256410256411
braycurtis K-Nearest Neighbours
0.6025641025641025
canberra K-Nearest Neighbours
0.5
jaccard K-Nearest Nei

/anaconda3/lib/python3.7/site-packages/sklearn/utils/validation.py:475: DataConversionWarning: Data with input dtype float64 was converted to bool by check_pairwise_arrays.
  warnings.warn(msg, DataConversionWarning)


In [43]:
import sys
sys.path.append("/anaconda3/lib/python3.7/site-packages")
import numpy as np
import numpy
import nltk
import pickle
import pandas as pd
import glob
import os
import re
import operator
from collections import Counter
#from read_xml_files import *
##from s_stemmer import *
from nltk.stem.porter import PorterStemmer
from nltk.stem import LancasterStemmer, WordNetLemmatizer
from itertools import groupby
from nltk.collocations import *
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.cluster import KMeans
from scipy.cluster.vq import whiten
from nltk.tokenize.toktok import ToktokTokenizer
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2
import xml.etree.ElementTree as ET
import pandas as pd
#Import scikit-learn metrics module for accuracy calculation
from sklearn import metrics
##from pattern.text.en import singularize
##nltk.download('vader_lexicon')
##nltk.download('punkt')
#import the relevant modules from the NLTK library
from nltk.sentiment.vader import SentimentIntensityAnalyzer
sentence_tokenizer = nltk.data.load('tokenizers/punkt/english.pickle')
word_tokenizer = nltk.tokenize.RegexpTokenizer(r'\w+')
##tt = ToktokTokenizer()#keeps the url as is...................................................
from nltk.tokenize import TweetTokenizer
tt = TweetTokenizer()
Porter_stemmer = PorterStemmer()
Lancaster_stemmer = LancasterStemmer()
WordNet_lemmatizer = WordNetLemmatizer()
#nltk.download('averaged_perceptron_tagger')
##Train_path="D:/Data/PAN2015-Profiling/pan15-author-profiling-training-dataset-english-2015-04-23"#path for train data
##Test_path="D:/Data/PAN2015-Profiling/pan-ap2015-test/en" #path for test data

##Train_path="D:/NLP/PAN2014/pan14-author-profiling-training-corpus-english-twitter-2014-04-16"#path for train data D:\Data\PAN2013-Profiling\Training\en  D:\NLP\PAN2014
##train_truth_path="D:/NLP/PAN2014/truth_train"
##
##test_truth_path="D:/NLP/PAN2014/pan14_test"
##Test_path="D:/NLP/PAN2014/pan14_test/en"
##Content = open('results_of_clssification_all.txt', 'w')

M_train_file= "/Users/catherine/Desktop/NLP/PAN Datasets/PAN2014blog/2014.1-blog-txtfiles/PAN2014_blog_male_train.1.txt"
F_train_file= "/Users/catherine/Desktop/NLP/PAN Datasets/PAN2014blog/2014.1-blog-txtfiles/PAN2014_blog_female_train.1.txt"

M_test_file= "/Users/catherine/Desktop/NLP/PAN Datasets/PAN2014blog/2014.1-blog-txtfiles/PAN2014_blog_male_test.1.txt"
F_test_file= "/Users/catherine/Desktop/NLP/PAN Datasets/PAN2014blog/2014.1-blog-txtfiles/PAN2014_blog_female_test.1.txt"

def all_txt(M_txt_file,  F_txt_file):
    with open(M_txt_file, mode="r", encoding="utf-8") as f:
            txt_list = [(line[:-1]).split("\t") for line in f]
            male_dict = dict()
            content=[]
            for line in txt_list:
                if line[1] in male_dict:
                # append the new number to the existing array at this slot
                    male_dict[line[1]].append(line[3])
                else:
                # create a new array in this slot
                    male_dict[line[1]] = [line[3]]
    male_content= [("M", (' '.join(a)).split()) for a in list(male_dict.values())]
  
    with open(F_txt_file, mode="r", encoding="utf-8") as f:
            txt_list = [(line[:-1]).split("\t") for line in f]
            female_dict = dict()
            content=[]
            for line in txt_list:
                if line[1] in female_dict:
                # append the new number to the existing array at this slot
                    female_dict[line[1]].append(line[3])
                else:
                # create a new array in this slot
                    female_dict[line[1]] = [line[3]]
    female_content= [("F", (' '.join(a)).split()) for a in list(female_dict.values())]
    all_txt_per_person = male_content + female_content
    return all_txt_per_person

##print( [((all_txt(M_txt_file,  F_txt_file))[i][0])  for i in range(len((all_txt(M_txt_file,  F_txt_file))))]) helps to determine the gender....

      
"""
create feature vector from the training set.....................................................................................
"""

def WordFeatures(word_list, all_training_text):

    fvs_words = np.array([[author.count(word) for word in word_list] for author in all_training_text]).astype(np.float64)

    # normalise by dividing each row by number of tokens for each author........
    fvs_words /= np.c_[np.array([len(author) for author in all_training_text])]

    return fvs_words



"""
From the saved csv file, recover the saved features to be used...............................................................
"""
import csv
word_list=[]
##txt_files =[ "/Users/catherine/Desktop/NLP/PAN Datasets/PAN_chosen_tokens/2014/output2014OR.1M.txt", "/Users/catherine/Desktop/NLP/PAN Datasets/PAN_chosen_tokens/2014/output2014OR.2F.txt"]
txt_files =[ "/Users/catherine/Desktop/NLP/PAN Datasets/PAN2014blog/2014.1-blog-txtfiles/PAN2014_blog_female_chosen_set.1.csv", "/Users/catherine/Desktop/NLP/PAN Datasets/PAN2014blog/2014.1-blog-txtfiles/PAN2014_blog_male_chosen_set.1.csv"]
print("txt_files of features used",txt_files)
##for txt_file in txt_files:
##    with open(txt_file, mode="r", encoding="utf-8") as f:
##        reader = csv.reader(f, delimiter=",")
##        next(reader) # skip header
##        word_list1 =  ([(r.split(":"))[1] for r in f])
##        print(txt_file, len(word_list1))
##        word_list = word_list + word_list1
##print("length of features used =", len(word_list))

for txt_file in txt_files:
    with open(txt_file, mode="r", encoding="utf-8") as f:
        reader = csv.reader(f, delimiter=",")
        next(reader) # skip header
        word_list1 =  [r[0] for r in reader]
        print(txt_file, len(word_list1))
        word_list = word_list + word_list1
print("length of features used =", len(word_list))


"""
Prepare the training and test sets to be parsed to the classifies............................................................
"""
A=(all_txt(M_train_file,  F_train_file))
all_training_text = ( [(A[i][1])  for i in range(len(A))])
print("len(all_training_text)",len(all_training_text))
X_train=WordFeatures(word_list, all_training_text)
y_train=np.array( [(A[i][0])  for i in range(len(A))])
print("len(X_train)=",len(X_train), "len(y_train)=", len(y_train))

B=(all_txt(M_test_file,  F_test_file))
all_test_text = ( [(B[i][1])  for i in range(len(B))])
print("len(all_test_text)",len(all_test_text))
X_test = WordFeatures(word_list, all_test_text)
y_test = np.array( [(B[i][0])  for i in range(len(B))])
print("len(X_test=",len(X_test), "len(y_test)=",len(y_test))


X_train = np.nan_to_num(X_train)
X_test = np.nan_to_num(X_test)

"""
Perform feature selection using chi2............................................................
"""
from sklearn.feature_selection import *
##
####def select_Bestfeatures(X, y):
print("selector = SelectKBest(score_func=f_classif, k=200)")
selector = SelectKBest(score_func=f_classif, k=200)
fit = selector.fit(X_train,y_train)
train_selected_features = fit.transform(X_train)
test_selected_features = fit.transform(X_test)



"""
x_train and x_test based on the new features using chi2............................................................
"""
##X_train = select_Bestfeatures(X_train, y_train)
##X_test = select_Bestfeatures(X_test, y_test)


"""
Perform feature selection using f_classif............................................................
"""
from sklearn.feature_selection import *
##def f_classif_select_Bestfeatures(X, y):
##selector=SelectKBest(score_func=f_classif,k=500)
##fit = selector.fit(X_train,y_train)
##train_selected_features = fit.transform(X_train)
##test_selected_features = fit.transform(X_test)
##    selected_features = fit.transform(X)
##    return selected_features

"""
x_train and x_test based on the new features using f_classif............................................................
"""
##X_train = f_classif_select_Bestfeatures(X_train, y_train)
##X_test = f_classif_select_Bestfeatures(X_test, y_test)



"""
Perform feature selection using mutual_info_classif............................................................
"""
from sklearn.feature_selection import *
##def mutual_info_classif_select_Bestfeatures(X_tr, X_te, y):
##selector=SelectKBest(score_func=mutual_info_classif,k=100)
##fit = selector.fit(X_train,y_train)
##train_selected_features = fit.transform(X_train)
##test_selected_features = fit.transform(X_test)
##    return train_selected_features, test_selected_features

"""
x_train and x_test based on the new features using f_classif............................................................
"""
X_train =train_selected_features 
X_test = test_selected_features


"""
Perform feature selection using pca............................................................
"""

##from sklearn.decomposition import PCA
##def PCA_select_Bestfeatures(X):
##    # feature extraction
##    pca = PCA(n_components=200)
##    fit = pca.fit(X, y)
##    Train_selected_features = fit.transform(X)
##    return selected_features

"""
x_train and x_test based on the new features using f_classif............................................................
"""
##X_train = PCA_select_Bestfeatures(X_train)
##X_test = PCA_select_Bestfeatures(X_test)

"""
Below are a list of Classification Algorithms with different distance measres ..............................................................................
"""
"""
K-Nearest Neighbours (average aa over all the nearest neighbour selectiopn)..............................................................................
"""

def tanimoto_distance(x,y):
    """Calculates the tanimoto (normalised manhattan) distance between the vectors x and y
            Keyword arguments:
                x,y -- the vectors between which the distance is to be calculated
    """
    try:
        iter(x)
    except TypeError:
        logging.warning( 'Argument x is not iterable. None is returned')
        return None
    try:
        iter(y)
    except TypeError:
        logging.warning( 'Argument y is not iterable. None is returned')
        return None
    numerator = sum(abs(a - b) for a, b in zip(x, y))
    denominator = sum(max(a,b) for a,b in zip(x,y))
    return numerator/denominator

k=5

print(k)

"""
8.0. tanimoto_distance K-Nearest Neighbours..............................................................................
"""
from sklearn.neighbors import KNeighborsClassifier
knn = KNeighborsClassifier(n_neighbors = k, metric= tanimoto_distance)
knn.fit(X_train, y_train)
y_pred = knn.predict(X_test)
print("tanimoto_distance K-Nearest Neighbours")
print(metrics.accuracy_score(y_test, y_pred))




txt_files of features used ['/Users/catherine/Desktop/NLP/PAN Datasets/PAN2014blog/2014.1-blog-txtfiles/PAN2014_blog_female_chosen_set.1.csv', '/Users/catherine/Desktop/NLP/PAN Datasets/PAN2014blog/2014.1-blog-txtfiles/PAN2014_blog_male_chosen_set.1.csv']
/Users/catherine/Desktop/NLP/PAN Datasets/PAN2014blog/2014.1-blog-txtfiles/PAN2014_blog_female_chosen_set.1.csv 1526
/Users/catherine/Desktop/NLP/PAN Datasets/PAN2014blog/2014.1-blog-txtfiles/PAN2014_blog_male_chosen_set.1.csv 2063
length of features used = 3589
len(all_training_text) 147
len(X_train)= 147 len(y_train)= 147
len(all_test_text) 78
len(X_test= 78 len(y_test)= 78
selector = SelectKBest(score_func=f_classif, k=200)
5
tanimoto_distance K-Nearest Neighbours
0.5897435897435898


In [45]:
import sys
sys.path.append("/anaconda3/lib/python3.7/site-packages")
import numpy as np
import numpy
import nltk
import pickle
import pandas as pd
import glob
import os
import re
import operator
from collections import Counter
#from read_xml_files import *
##from s_stemmer import *
from nltk.stem.porter import PorterStemmer
from nltk.stem import LancasterStemmer, WordNetLemmatizer
from itertools import groupby
from nltk.collocations import *
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.cluster import KMeans
from scipy.cluster.vq import whiten
from nltk.tokenize.toktok import ToktokTokenizer
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2
import xml.etree.ElementTree as ET
import pandas as pd
#Import scikit-learn metrics module for accuracy calculation
from sklearn import metrics
##from pattern.text.en import singularize
##nltk.download('vader_lexicon')
##nltk.download('punkt')
#import the relevant modules from the NLTK library
from nltk.sentiment.vader import SentimentIntensityAnalyzer
sentence_tokenizer = nltk.data.load('tokenizers/punkt/english.pickle')
word_tokenizer = nltk.tokenize.RegexpTokenizer(r'\w+')
##tt = ToktokTokenizer()#keeps the url as is...................................................
from nltk.tokenize import TweetTokenizer
tt = TweetTokenizer()
Porter_stemmer = PorterStemmer()
Lancaster_stemmer = LancasterStemmer()
WordNet_lemmatizer = WordNetLemmatizer()
#nltk.download('averaged_perceptron_tagger')
##Train_path="D:/Data/PAN2015-Profiling/pan15-author-profiling-training-dataset-english-2015-04-23"#path for train data
##Test_path="D:/Data/PAN2015-Profiling/pan-ap2015-test/en" #path for test data

##Train_path="D:/NLP/PAN2014/pan14-author-profiling-training-corpus-english-twitter-2014-04-16"#path for train data D:\Data\PAN2013-Profiling\Training\en  D:\NLP\PAN2014
##train_truth_path="D:/NLP/PAN2014/truth_train"
##
##test_truth_path="D:/NLP/PAN2014/pan14_test"
##Test_path="D:/NLP/PAN2014/pan14_test/en"
##Content = open('results_of_clssification_all.txt', 'w')

M_train_file= "/Users/catherine/Desktop/NLP/PAN Datasets/PAN2014blog/2014.1-blog-txtfiles/PAN2014_blog_male_train.1.txt"
F_train_file= "/Users/catherine/Desktop/NLP/PAN Datasets/PAN2014blog/2014.1-blog-txtfiles/PAN2014_blog_female_train.1.txt"

M_test_file= "/Users/catherine/Desktop/NLP/PAN Datasets/PAN2014blog/2014.1-blog-txtfiles/PAN2014_blog_male_test.1.txt"
F_test_file= "/Users/catherine/Desktop/NLP/PAN Datasets/PAN2014blog/2014.1-blog-txtfiles/PAN2014_blog_female_test.1.txt"

def all_txt(M_txt_file,  F_txt_file):
    with open(M_txt_file, mode="r", encoding="utf-8") as f:
            txt_list = [(line[:-1]).split("\t") for line in f]
            male_dict = dict()
            content=[]
            for line in txt_list:
                if line[1] in male_dict:
                # append the new number to the existing array at this slot
                    male_dict[line[1]].append(line[3])
                else:
                # create a new array in this slot
                    male_dict[line[1]] = [line[3]]
    male_content= [("M", (' '.join(a)).split()) for a in list(male_dict.values())]
  
    with open(F_txt_file, mode="r", encoding="utf-8") as f:
            txt_list = [(line[:-1]).split("\t") for line in f]
            female_dict = dict()
            content=[]
            for line in txt_list:
                if line[1] in female_dict:
                # append the new number to the existing array at this slot
                    female_dict[line[1]].append(line[3])
                else:
                # create a new array in this slot
                    female_dict[line[1]] = [line[3]]
    female_content= [("F", (' '.join(a)).split()) for a in list(female_dict.values())]
    all_txt_per_person = male_content + female_content
    return all_txt_per_person

##print( [((all_txt(M_txt_file,  F_txt_file))[i][0])  for i in range(len((all_txt(M_txt_file,  F_txt_file))))]) helps to determine the gender....

      
"""
create feature vector from the training set.....................................................................................
"""

def WordFeatures(word_list, all_training_text):

    fvs_words = np.array([[author.count(word) for word in word_list] for author in all_training_text]).astype(np.float64)

    # normalise by dividing each row by number of tokens for each author........
    fvs_words /= np.c_[np.array([len(author) for author in all_training_text])]

    return fvs_words



"""
From the saved csv file, recover the saved features to be used...............................................................
"""
import csv
word_list=[]
txt_files =[ "/Users/catherine/Desktop/NLP/PAN Datasets/PAN_TermSelection/2014TermSelection_Blog/outputNewVersion2014blogCHI.txt"]
#txt_files =[ "/Users/catherine/Desktop/NLP/PAN Datasets/PAN_TermSelection/2014TermSelection_Blog/outputNewVersion2014blogIG.txt"]
#txt_files =[ "/Users/catherine/Desktop/NLP/PAN Datasets/PAN_TermSelection/2014TermSelection_Blog/outputNewVersion2014blogGSS.txt"]
#txt_files =[ "/Users/catherine/Desktop/NLP/PAN Datasets/PAN_TermSelection/2014TermSelection_Blog/outputNewVersion2014blogOR.1M.txt", "/Users/catherine/Desktop/NLP/PAN Datasets/PAN_TermSelection/2014TermSelection_Blog/outputNewVersion2014blogOR.2F.txt"]
#txt_files =[ "/Users/catherine/Desktop/NLP/PAN Datasets/PAN_TermSelection/2014TermSelection_Blog/outputNewVersion2014blogPMI.1M.txt", "/outputNewVersion201blogPMI.2F.txt"]
#txt_files =[ "/Users/catherine/Desktop/NLP/PAN Datasets/PAN_TermSelection/2014TermSelection_Blog/outputNewVersion201blogRF.1M.txt", "/Users/catherine/Desktop/NLP/PAN Datasets/PAN_TermSelection/2014TermSelection_Blog/outputNewVersion2014blogRF.2F.txt"]
#txt_files =[ "/Users/catherine/Desktop/NLP/PAN Datasets/PAN_TermSelection/2014TermSelection_Blog/outputNewVersion201blogWLLR.1M.txt", "/Users/catherine/Desktop/NLP/PAN Datasets/PAN_TermSelection/2014TermSelection_Blog/outputNewVersion2014blogWLLR.2F.txt"]
print("txt_files of features used",txt_files)
for txt_file in txt_files:
    with open(txt_file, mode="r", encoding="utf-8") as f:
        word_list2 =  ([r.strip() for r in f])[0:200]
        word_list1 =  ([(r.split(":"))[1] for r in word_list2])
        print(txt_file, len(word_list1))
        word_list = word_list + word_list1
print("length of features used =", len(word_list))
#for txt_file in txt_files:
#    with open(txt_file, mode="r", encoding="utf-8") as f:
#        reader = csv.reader(f, delimiter=",")
#        next(reader) # skip header
#        word_list1 =  [r[0] for r in reader]
#        print(txt_file, len(word_list1))
#        word_list = word_list + word_list1
#print("length of features used =", len(word_list))


"""
Prepare the training and test sets to be parsed to the classifies............................................................
"""
A=(all_txt(M_train_file,  F_train_file))
all_training_text = ( [(A[i][1])  for i in range(len(A))])
print("len(all_training_text)",len(all_training_text))
X_train=WordFeatures(word_list, all_training_text)
y_train=np.array( [(A[i][0])  for i in range(len(A))])
print("len(X_train)=",len(X_train), "len(y_train)=", len(y_train))

B=(all_txt(M_test_file,  F_test_file))
all_test_text = ( [(B[i][1])  for i in range(len(B))])
print("len(all_test_text)",len(all_test_text))
X_test = WordFeatures(word_list, all_test_text)
y_test = np.array( [(B[i][0])  for i in range(len(B))])
print("len(X_test=",len(X_test), "len(y_test)=",len(y_test))


X_train = np.nan_to_num(X_train)
X_test = np.nan_to_num(X_test)

"""
Perform feature selection using chi2............................................................
"""
from sklearn.feature_selection import *
##
####def select_Bestfeatures(X, y):
print("selector = SelectKBest(score_func=f_classif, k=200)")
#selector = SelectKBest(score_func=f_classif, k=200)
#fit = selector.fit(X_train,y_train)
#train_selected_features = fit.transform(X_train)
#test_selected_features = fit.transform(X_test)



"""
x_train and x_test based on the new features using chi2............................................................
"""
##X_train = select_Bestfeatures(X_train, y_train)
##X_test = select_Bestfeatures(X_test, y_test)


"""
Perform feature selection using f_classif............................................................
"""
from sklearn.feature_selection import *
##def f_classif_select_Bestfeatures(X, y):
##selector=SelectKBest(score_func=f_classif,k=500)
##fit = selector.fit(X_train,y_train)
##train_selected_features = fit.transform(X_train)
##test_selected_features = fit.transform(X_test)
##    selected_features = fit.transform(X)
##    return selected_features

"""
x_train and x_test based on the new features using f_classif............................................................
"""
##X_train = f_classif_select_Bestfeatures(X_train, y_train)
##X_test = f_classif_select_Bestfeatures(X_test, y_test)



"""
Perform feature selection using mutual_info_classif............................................................
"""
from sklearn.feature_selection import *
##def mutual_info_classif_select_Bestfeatures(X_tr, X_te, y):
##selector=SelectKBest(score_func=mutual_info_classif,k=100)
##fit = selector.fit(X_train,y_train)
##train_selected_features = fit.transform(X_train)
##test_selected_features = fit.transform(X_test)
##    return train_selected_features, test_selected_features

"""
x_train and x_test based on the new features using f_classif............................................................
"""
#X_train =train_selected_features 
#X_test = test_selected_features


"""
Perform feature selection using pca............................................................
"""

##from sklearn.decomposition import PCA
##def PCA_select_Bestfeatures(X):
##    # feature extraction
##    pca = PCA(n_components=200)
##    fit = pca.fit(X, y)
##    Train_selected_features = fit.transform(X)
##    return selected_features

"""
x_train and x_test based on the new features using f_classif............................................................
"""
##X_train = PCA_select_Bestfeatures(X_train)
##X_test = PCA_select_Bestfeatures(X_test)

"""
Below are a list of Classification Algorithms with different distance measres ..............................................................................
"""
"""
K-Nearest Neighbours (average aa over all the nearest neighbour selectiopn)..............................................................................
"""

def tanimoto_distance(x,y):
    """Calculates the tanimoto (normalised manhattan) distance between the vectors x and y
            Keyword arguments:
                x,y -- the vectors between which the distance is to be calculated
    """
    try:
        iter(x)
    except TypeError:
        logging.warning( 'Argument x is not iterable. None is returned')
        return None
    try:
        iter(y)
    except TypeError:
        logging.warning( 'Argument y is not iterable. None is returned')
        return None
    numerator = sum(abs(a - b) for a, b in zip(x, y))
    denominator = sum(max(a,b) for a,b in zip(x,y))
    return numerator/denominator

k=5

print(k)

"""
8.0. tanimoto_distance K-Nearest Neighbours..............................................................................
"""
from sklearn.neighbors import KNeighborsClassifier
knn = KNeighborsClassifier(n_neighbors = k, metric= tanimoto_distance)
knn.fit(X_train, y_train)
y_pred = knn.predict(X_test)
print("tanimoto_distance K-Nearest Neighbours")
print(metrics.accuracy_score(y_test, y_pred))





txt_files of features used ['/Users/catherine/Desktop/NLP/PAN Datasets/PAN_TermSelection/2014TermSelection_Blog/outputNewVersion2014blogCHI.txt']
/Users/catherine/Desktop/NLP/PAN Datasets/PAN_TermSelection/2014TermSelection_Blog/outputNewVersion2014blogCHI.txt 200
length of features used = 200
len(all_training_text) 147
len(X_train)= 147 len(y_train)= 147
len(all_test_text) 78
len(X_test= 78 len(y_test)= 78
selector = SelectKBest(score_func=f_classif, k=200)
5
tanimoto_distance K-Nearest Neighbours
0.5897435897435898


In [46]:
import sys
sys.path.append("/anaconda3/lib/python3.7/site-packages")
import numpy as np
import numpy
import nltk
import pickle
import pandas as pd
import glob
import os
import re
import operator
from collections import Counter
#from read_xml_files import *
##from s_stemmer import *
from nltk.stem.porter import PorterStemmer
from nltk.stem import LancasterStemmer, WordNetLemmatizer
from itertools import groupby
from nltk.collocations import *
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.cluster import KMeans
from scipy.cluster.vq import whiten
from nltk.tokenize.toktok import ToktokTokenizer
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2
import xml.etree.ElementTree as ET
import pandas as pd
#Import scikit-learn metrics module for accuracy calculation
from sklearn import metrics
##from pattern.text.en import singularize
##nltk.download('vader_lexicon')
##nltk.download('punkt')
#import the relevant modules from the NLTK library
from nltk.sentiment.vader import SentimentIntensityAnalyzer
sentence_tokenizer = nltk.data.load('tokenizers/punkt/english.pickle')
word_tokenizer = nltk.tokenize.RegexpTokenizer(r'\w+')
##tt = ToktokTokenizer()#keeps the url as is...................................................
from nltk.tokenize import TweetTokenizer
tt = TweetTokenizer()
Porter_stemmer = PorterStemmer()
Lancaster_stemmer = LancasterStemmer()
WordNet_lemmatizer = WordNetLemmatizer()
#nltk.download('averaged_perceptron_tagger')
##Train_path="D:/Data/PAN2015-Profiling/pan15-author-profiling-training-dataset-english-2015-04-23"#path for train data
##Test_path="D:/Data/PAN2015-Profiling/pan-ap2015-test/en" #path for test data

##Train_path="D:/NLP/PAN2014/pan14-author-profiling-training-corpus-english-twitter-2014-04-16"#path for train data D:\Data\PAN2013-Profiling\Training\en  D:\NLP\PAN2014
##train_truth_path="D:/NLP/PAN2014/truth_train"
##
##test_truth_path="D:/NLP/PAN2014/pan14_test"
##Test_path="D:/NLP/PAN2014/pan14_test/en"
##Content = open('results_of_clssification_all.txt', 'w')

M_train_file= "/Users/catherine/Desktop/NLP/PAN Datasets/PAN2014blog/2014.1-blog-txtfiles/PAN2014_blog_male_train.1.txt"
F_train_file= "/Users/catherine/Desktop/NLP/PAN Datasets/PAN2014blog/2014.1-blog-txtfiles/PAN2014_blog_female_train.1.txt"

M_test_file= "/Users/catherine/Desktop/NLP/PAN Datasets/PAN2014blog/2014.1-blog-txtfiles/PAN2014_blog_male_test.1.txt"
F_test_file= "/Users/catherine/Desktop/NLP/PAN Datasets/PAN2014blog/2014.1-blog-txtfiles/PAN2014_blog_female_test.1.txt"

def all_txt(M_txt_file,  F_txt_file):
    with open(M_txt_file, mode="r", encoding="utf-8") as f:
            txt_list = [(line[:-1]).split("\t") for line in f]
            male_dict = dict()
            content=[]
            for line in txt_list:
                if line[1] in male_dict:
                # append the new number to the existing array at this slot
                    male_dict[line[1]].append(line[3])
                else:
                # create a new array in this slot
                    male_dict[line[1]] = [line[3]]
    male_content= [("M", (' '.join(a)).split()) for a in list(male_dict.values())]
  
    with open(F_txt_file, mode="r", encoding="utf-8") as f:
            txt_list = [(line[:-1]).split("\t") for line in f]
            female_dict = dict()
            content=[]
            for line in txt_list:
                if line[1] in female_dict:
                # append the new number to the existing array at this slot
                    female_dict[line[1]].append(line[3])
                else:
                # create a new array in this slot
                    female_dict[line[1]] = [line[3]]
    female_content= [("F", (' '.join(a)).split()) for a in list(female_dict.values())]
    all_txt_per_person = male_content + female_content
    return all_txt_per_person

##print( [((all_txt(M_txt_file,  F_txt_file))[i][0])  for i in range(len((all_txt(M_txt_file,  F_txt_file))))]) helps to determine the gender....

      
"""
create feature vector from the training set.....................................................................................
"""

def WordFeatures(word_list, all_training_text):

    fvs_words = np.array([[author.count(word) for word in word_list] for author in all_training_text]).astype(np.float64)

    # normalise by dividing each row by number of tokens for each author........
    fvs_words /= np.c_[np.array([len(author) for author in all_training_text])]

    return fvs_words



"""
From the saved csv file, recover the saved features to be used...............................................................
"""
import csv
word_list=[]
txt_files =[ "/Users/catherine/Desktop/NLP/PAN Datasets/PAN_TermSelection/2014TermSelection_Blog/outputNewVersion2014blogCHI.txt"]
txt_files =[ "/Users/catherine/Desktop/NLP/PAN Datasets/PAN_TermSelection/2014TermSelection_Blog/outputNewVersion2014blogIG.txt"]
#txt_files =[ "/Users/catherine/Desktop/NLP/PAN Datasets/PAN_TermSelection/2014TermSelection_Blog/outputNewVersion2014blogGSS.txt"]
#txt_files =[ "/Users/catherine/Desktop/NLP/PAN Datasets/PAN_TermSelection/2014TermSelection_Blog/outputNewVersion2014blogOR.1M.txt", "/Users/catherine/Desktop/NLP/PAN Datasets/PAN_TermSelection/2014TermSelection_Blog/outputNewVersion2014blogOR.2F.txt"]
#txt_files =[ "/Users/catherine/Desktop/NLP/PAN Datasets/PAN_TermSelection/2014TermSelection_Blog/outputNewVersion2014blogPMI.1M.txt", "/outputNewVersion201blogPMI.2F.txt"]
#txt_files =[ "/Users/catherine/Desktop/NLP/PAN Datasets/PAN_TermSelection/2014TermSelection_Blog/outputNewVersion201blogRF.1M.txt", "/Users/catherine/Desktop/NLP/PAN Datasets/PAN_TermSelection/2014TermSelection_Blog/outputNewVersion2014blogRF.2F.txt"]
#txt_files =[ "/Users/catherine/Desktop/NLP/PAN Datasets/PAN_TermSelection/2014TermSelection_Blog/outputNewVersion201blogWLLR.1M.txt", "/Users/catherine/Desktop/NLP/PAN Datasets/PAN_TermSelection/2014TermSelection_Blog/outputNewVersion2014blogWLLR.2F.txt"]
print("txt_files of features used",txt_files)
for txt_file in txt_files:
    with open(txt_file, mode="r", encoding="utf-8") as f:
        word_list2 =  ([r.strip() for r in f])[0:200]
        word_list1 =  ([(r.split(":"))[1] for r in word_list2])
        print(txt_file, len(word_list1))
        word_list = word_list + word_list1
print("length of features used =", len(word_list))
#for txt_file in txt_files:
#    with open(txt_file, mode="r", encoding="utf-8") as f:
#        reader = csv.reader(f, delimiter=",")
#        next(reader) # skip header
#        word_list1 =  [r[0] for r in reader]
#        print(txt_file, len(word_list1))
#        word_list = word_list + word_list1
#print("length of features used =", len(word_list))


"""
Prepare the training and test sets to be parsed to the classifies............................................................
"""
A=(all_txt(M_train_file,  F_train_file))
all_training_text = ( [(A[i][1])  for i in range(len(A))])
print("len(all_training_text)",len(all_training_text))
X_train=WordFeatures(word_list, all_training_text)
y_train=np.array( [(A[i][0])  for i in range(len(A))])
print("len(X_train)=",len(X_train), "len(y_train)=", len(y_train))

B=(all_txt(M_test_file,  F_test_file))
all_test_text = ( [(B[i][1])  for i in range(len(B))])
print("len(all_test_text)",len(all_test_text))
X_test = WordFeatures(word_list, all_test_text)
y_test = np.array( [(B[i][0])  for i in range(len(B))])
print("len(X_test=",len(X_test), "len(y_test)=",len(y_test))


X_train = np.nan_to_num(X_train)
X_test = np.nan_to_num(X_test)

"""
Perform feature selection using chi2............................................................
"""
from sklearn.feature_selection import *
##
####def select_Bestfeatures(X, y):
print("selector = SelectKBest(score_func=f_classif, k=200)")
#selector = SelectKBest(score_func=f_classif, k=200)
#fit = selector.fit(X_train,y_train)
#train_selected_features = fit.transform(X_train)
#test_selected_features = fit.transform(X_test)



"""
x_train and x_test based on the new features using chi2............................................................
"""
##X_train = select_Bestfeatures(X_train, y_train)
##X_test = select_Bestfeatures(X_test, y_test)


"""
Perform feature selection using f_classif............................................................
"""
from sklearn.feature_selection import *
##def f_classif_select_Bestfeatures(X, y):
##selector=SelectKBest(score_func=f_classif,k=500)
##fit = selector.fit(X_train,y_train)
##train_selected_features = fit.transform(X_train)
##test_selected_features = fit.transform(X_test)
##    selected_features = fit.transform(X)
##    return selected_features

"""
x_train and x_test based on the new features using f_classif............................................................
"""
##X_train = f_classif_select_Bestfeatures(X_train, y_train)
##X_test = f_classif_select_Bestfeatures(X_test, y_test)



"""
Perform feature selection using mutual_info_classif............................................................
"""
from sklearn.feature_selection import *
##def mutual_info_classif_select_Bestfeatures(X_tr, X_te, y):
##selector=SelectKBest(score_func=mutual_info_classif,k=100)
##fit = selector.fit(X_train,y_train)
##train_selected_features = fit.transform(X_train)
##test_selected_features = fit.transform(X_test)
##    return train_selected_features, test_selected_features

"""
x_train and x_test based on the new features using f_classif............................................................
"""
#X_train =train_selected_features 
#X_test = test_selected_features


"""
Perform feature selection using pca............................................................
"""

##from sklearn.decomposition import PCA
##def PCA_select_Bestfeatures(X):
##    # feature extraction
##    pca = PCA(n_components=200)
##    fit = pca.fit(X, y)
##    Train_selected_features = fit.transform(X)
##    return selected_features

"""
x_train and x_test based on the new features using f_classif............................................................
"""
##X_train = PCA_select_Bestfeatures(X_train)
##X_test = PCA_select_Bestfeatures(X_test)

"""
Below are a list of Classification Algorithms with different distance measres ..............................................................................
"""
"""
K-Nearest Neighbours (average aa over all the nearest neighbour selectiopn)..............................................................................
"""

def tanimoto_distance(x,y):
    """Calculates the tanimoto (normalised manhattan) distance between the vectors x and y
            Keyword arguments:
                x,y -- the vectors between which the distance is to be calculated
    """
    try:
        iter(x)
    except TypeError:
        logging.warning( 'Argument x is not iterable. None is returned')
        return None
    try:
        iter(y)
    except TypeError:
        logging.warning( 'Argument y is not iterable. None is returned')
        return None
    numerator = sum(abs(a - b) for a, b in zip(x, y))
    denominator = sum(max(a,b) for a,b in zip(x,y))
    return numerator/denominator

k=5

print(k)

"""
8.0. tanimoto_distance K-Nearest Neighbours..............................................................................
"""
from sklearn.neighbors import KNeighborsClassifier
knn = KNeighborsClassifier(n_neighbors = k, metric= tanimoto_distance)
knn.fit(X_train, y_train)
y_pred = knn.predict(X_test)
print("tanimoto_distance K-Nearest Neighbours")
print(metrics.accuracy_score(y_test, y_pred))





txt_files of features used ['/Users/catherine/Desktop/NLP/PAN Datasets/PAN_TermSelection/2014TermSelection_Blog/outputNewVersion2014blogIG.txt']
/Users/catherine/Desktop/NLP/PAN Datasets/PAN_TermSelection/2014TermSelection_Blog/outputNewVersion2014blogIG.txt 200
length of features used = 200
len(all_training_text) 147
len(X_train)= 147 len(y_train)= 147
len(all_test_text) 78
len(X_test= 78 len(y_test)= 78
selector = SelectKBest(score_func=f_classif, k=200)
5
tanimoto_distance K-Nearest Neighbours
0.5641025641025641


In [47]:
import sys
sys.path.append("/anaconda3/lib/python3.7/site-packages")
import numpy as np
import numpy
import nltk
import pickle
import pandas as pd
import glob
import os
import re
import operator
from collections import Counter
#from read_xml_files import *
##from s_stemmer import *
from nltk.stem.porter import PorterStemmer
from nltk.stem import LancasterStemmer, WordNetLemmatizer
from itertools import groupby
from nltk.collocations import *
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.cluster import KMeans
from scipy.cluster.vq import whiten
from nltk.tokenize.toktok import ToktokTokenizer
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2
import xml.etree.ElementTree as ET
import pandas as pd
#Import scikit-learn metrics module for accuracy calculation
from sklearn import metrics
##from pattern.text.en import singularize
##nltk.download('vader_lexicon')
##nltk.download('punkt')
#import the relevant modules from the NLTK library
from nltk.sentiment.vader import SentimentIntensityAnalyzer
sentence_tokenizer = nltk.data.load('tokenizers/punkt/english.pickle')
word_tokenizer = nltk.tokenize.RegexpTokenizer(r'\w+')
##tt = ToktokTokenizer()#keeps the url as is...................................................
from nltk.tokenize import TweetTokenizer
tt = TweetTokenizer()
Porter_stemmer = PorterStemmer()
Lancaster_stemmer = LancasterStemmer()
WordNet_lemmatizer = WordNetLemmatizer()
#nltk.download('averaged_perceptron_tagger')
##Train_path="D:/Data/PAN2015-Profiling/pan15-author-profiling-training-dataset-english-2015-04-23"#path for train data
##Test_path="D:/Data/PAN2015-Profiling/pan-ap2015-test/en" #path for test data

##Train_path="D:/NLP/PAN2014/pan14-author-profiling-training-corpus-english-twitter-2014-04-16"#path for train data D:\Data\PAN2013-Profiling\Training\en  D:\NLP\PAN2014
##train_truth_path="D:/NLP/PAN2014/truth_train"
##
##test_truth_path="D:/NLP/PAN2014/pan14_test"
##Test_path="D:/NLP/PAN2014/pan14_test/en"
##Content = open('results_of_clssification_all.txt', 'w')

M_train_file= "/Users/catherine/Desktop/NLP/PAN Datasets/PAN2014blog/2014.1-blog-txtfiles/PAN2014_blog_male_train.1.txt"
F_train_file= "/Users/catherine/Desktop/NLP/PAN Datasets/PAN2014blog/2014.1-blog-txtfiles/PAN2014_blog_female_train.1.txt"

M_test_file= "/Users/catherine/Desktop/NLP/PAN Datasets/PAN2014blog/2014.1-blog-txtfiles/PAN2014_blog_male_test.1.txt"
F_test_file= "/Users/catherine/Desktop/NLP/PAN Datasets/PAN2014blog/2014.1-blog-txtfiles/PAN2014_blog_female_test.1.txt"

def all_txt(M_txt_file,  F_txt_file):
    with open(M_txt_file, mode="r", encoding="utf-8") as f:
            txt_list = [(line[:-1]).split("\t") for line in f]
            male_dict = dict()
            content=[]
            for line in txt_list:
                if line[1] in male_dict:
                # append the new number to the existing array at this slot
                    male_dict[line[1]].append(line[3])
                else:
                # create a new array in this slot
                    male_dict[line[1]] = [line[3]]
    male_content= [("M", (' '.join(a)).split()) for a in list(male_dict.values())]
  
    with open(F_txt_file, mode="r", encoding="utf-8") as f:
            txt_list = [(line[:-1]).split("\t") for line in f]
            female_dict = dict()
            content=[]
            for line in txt_list:
                if line[1] in female_dict:
                # append the new number to the existing array at this slot
                    female_dict[line[1]].append(line[3])
                else:
                # create a new array in this slot
                    female_dict[line[1]] = [line[3]]
    female_content= [("F", (' '.join(a)).split()) for a in list(female_dict.values())]
    all_txt_per_person = male_content + female_content
    return all_txt_per_person

##print( [((all_txt(M_txt_file,  F_txt_file))[i][0])  for i in range(len((all_txt(M_txt_file,  F_txt_file))))]) helps to determine the gender....

      
"""
create feature vector from the training set.....................................................................................
"""

def WordFeatures(word_list, all_training_text):

    fvs_words = np.array([[author.count(word) for word in word_list] for author in all_training_text]).astype(np.float64)

    # normalise by dividing each row by number of tokens for each author........
    fvs_words /= np.c_[np.array([len(author) for author in all_training_text])]

    return fvs_words



"""
From the saved csv file, recover the saved features to be used...............................................................
"""
import csv
word_list=[]
txt_files =[ "/Users/catherine/Desktop/NLP/PAN Datasets/PAN_TermSelection/2014TermSelection_Blog/outputNewVersion2014blogCHI.txt"]
#txt_files =[ "/Users/catherine/Desktop/NLP/PAN Datasets/PAN_TermSelection/2014TermSelection_Blog/outputNewVersion2014blogIG.txt"]
txt_files =[ "/Users/catherine/Desktop/NLP/PAN Datasets/PAN_TermSelection/2014TermSelection_Blog/outputNewVersion2014blogGSS.txt"]
#txt_files =[ "/Users/catherine/Desktop/NLP/PAN Datasets/PAN_TermSelection/2014TermSelection_Blog/outputNewVersion2014blogOR.1M.txt", "/Users/catherine/Desktop/NLP/PAN Datasets/PAN_TermSelection/2014TermSelection_Blog/outputNewVersion2014blogOR.2F.txt"]
#txt_files =[ "/Users/catherine/Desktop/NLP/PAN Datasets/PAN_TermSelection/2014TermSelection_Blog/outputNewVersion2014blogPMI.1M.txt", "/outputNewVersion201blogPMI.2F.txt"]
#txt_files =[ "/Users/catherine/Desktop/NLP/PAN Datasets/PAN_TermSelection/2014TermSelection_Blog/outputNewVersion201blogRF.1M.txt", "/Users/catherine/Desktop/NLP/PAN Datasets/PAN_TermSelection/2014TermSelection_Blog/outputNewVersion2014blogRF.2F.txt"]
#txt_files =[ "/Users/catherine/Desktop/NLP/PAN Datasets/PAN_TermSelection/2014TermSelection_Blog/outputNewVersion201blogWLLR.1M.txt", "/Users/catherine/Desktop/NLP/PAN Datasets/PAN_TermSelection/2014TermSelection_Blog/outputNewVersion2014blogWLLR.2F.txt"]
print("txt_files of features used",txt_files)
for txt_file in txt_files:
    with open(txt_file, mode="r", encoding="utf-8") as f:
        word_list2 =  ([r.strip() for r in f])[0:200]
        word_list1 =  ([(r.split(":"))[1] for r in word_list2])
        print(txt_file, len(word_list1))
        word_list = word_list + word_list1
print("length of features used =", len(word_list))
#for txt_file in txt_files:
#    with open(txt_file, mode="r", encoding="utf-8") as f:
#        reader = csv.reader(f, delimiter=",")
#        next(reader) # skip header
#        word_list1 =  [r[0] for r in reader]
#        print(txt_file, len(word_list1))
#        word_list = word_list + word_list1
#print("length of features used =", len(word_list))


"""
Prepare the training and test sets to be parsed to the classifies............................................................
"""
A=(all_txt(M_train_file,  F_train_file))
all_training_text = ( [(A[i][1])  for i in range(len(A))])
print("len(all_training_text)",len(all_training_text))
X_train=WordFeatures(word_list, all_training_text)
y_train=np.array( [(A[i][0])  for i in range(len(A))])
print("len(X_train)=",len(X_train), "len(y_train)=", len(y_train))

B=(all_txt(M_test_file,  F_test_file))
all_test_text = ( [(B[i][1])  for i in range(len(B))])
print("len(all_test_text)",len(all_test_text))
X_test = WordFeatures(word_list, all_test_text)
y_test = np.array( [(B[i][0])  for i in range(len(B))])
print("len(X_test=",len(X_test), "len(y_test)=",len(y_test))


X_train = np.nan_to_num(X_train)
X_test = np.nan_to_num(X_test)

"""
Perform feature selection using chi2............................................................
"""
from sklearn.feature_selection import *
##
####def select_Bestfeatures(X, y):
print("selector = SelectKBest(score_func=f_classif, k=200)")
#selector = SelectKBest(score_func=f_classif, k=200)
#fit = selector.fit(X_train,y_train)
#train_selected_features = fit.transform(X_train)
#test_selected_features = fit.transform(X_test)



"""
x_train and x_test based on the new features using chi2............................................................
"""
##X_train = select_Bestfeatures(X_train, y_train)
##X_test = select_Bestfeatures(X_test, y_test)


"""
Perform feature selection using f_classif............................................................
"""
from sklearn.feature_selection import *
##def f_classif_select_Bestfeatures(X, y):
##selector=SelectKBest(score_func=f_classif,k=500)
##fit = selector.fit(X_train,y_train)
##train_selected_features = fit.transform(X_train)
##test_selected_features = fit.transform(X_test)
##    selected_features = fit.transform(X)
##    return selected_features

"""
x_train and x_test based on the new features using f_classif............................................................
"""
##X_train = f_classif_select_Bestfeatures(X_train, y_train)
##X_test = f_classif_select_Bestfeatures(X_test, y_test)



"""
Perform feature selection using mutual_info_classif............................................................
"""
from sklearn.feature_selection import *
##def mutual_info_classif_select_Bestfeatures(X_tr, X_te, y):
##selector=SelectKBest(score_func=mutual_info_classif,k=100)
##fit = selector.fit(X_train,y_train)
##train_selected_features = fit.transform(X_train)
##test_selected_features = fit.transform(X_test)
##    return train_selected_features, test_selected_features

"""
x_train and x_test based on the new features using f_classif............................................................
"""
#X_train =train_selected_features 
#X_test = test_selected_features


"""
Perform feature selection using pca............................................................
"""

##from sklearn.decomposition import PCA
##def PCA_select_Bestfeatures(X):
##    # feature extraction
##    pca = PCA(n_components=200)
##    fit = pca.fit(X, y)
##    Train_selected_features = fit.transform(X)
##    return selected_features

"""
x_train and x_test based on the new features using f_classif............................................................
"""
##X_train = PCA_select_Bestfeatures(X_train)
##X_test = PCA_select_Bestfeatures(X_test)

"""
Below are a list of Classification Algorithms with different distance measres ..............................................................................
"""
"""
K-Nearest Neighbours (average aa over all the nearest neighbour selectiopn)..............................................................................
"""

def tanimoto_distance(x,y):
    """Calculates the tanimoto (normalised manhattan) distance between the vectors x and y
            Keyword arguments:
                x,y -- the vectors between which the distance is to be calculated
    """
    try:
        iter(x)
    except TypeError:
        logging.warning( 'Argument x is not iterable. None is returned')
        return None
    try:
        iter(y)
    except TypeError:
        logging.warning( 'Argument y is not iterable. None is returned')
        return None
    numerator = sum(abs(a - b) for a, b in zip(x, y))
    denominator = sum(max(a,b) for a,b in zip(x,y))
    return numerator/denominator

k=5

print(k)

"""
8.0. tanimoto_distance K-Nearest Neighbours..............................................................................
"""
from sklearn.neighbors import KNeighborsClassifier
knn = KNeighborsClassifier(n_neighbors = k, metric= tanimoto_distance)
knn.fit(X_train, y_train)
y_pred = knn.predict(X_test)
print("tanimoto_distance K-Nearest Neighbours")
print(metrics.accuracy_score(y_test, y_pred))






txt_files of features used ['/Users/catherine/Desktop/NLP/PAN Datasets/PAN_TermSelection/2014TermSelection_Blog/outputNewVersion2014blogGSS.txt']
/Users/catherine/Desktop/NLP/PAN Datasets/PAN_TermSelection/2014TermSelection_Blog/outputNewVersion2014blogGSS.txt 200
length of features used = 200
len(all_training_text) 147
len(X_train)= 147 len(y_train)= 147
len(all_test_text) 78
len(X_test= 78 len(y_test)= 78
selector = SelectKBest(score_func=f_classif, k=200)
5
tanimoto_distance K-Nearest Neighbours
0.6923076923076923


In [48]:
import sys
sys.path.append("/anaconda3/lib/python3.7/site-packages")
import numpy as np
import numpy
import nltk
import pickle
import pandas as pd
import glob
import os
import re
import operator
from collections import Counter
#from read_xml_files import *
##from s_stemmer import *
from nltk.stem.porter import PorterStemmer
from nltk.stem import LancasterStemmer, WordNetLemmatizer
from itertools import groupby
from nltk.collocations import *
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.cluster import KMeans
from scipy.cluster.vq import whiten
from nltk.tokenize.toktok import ToktokTokenizer
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2
import xml.etree.ElementTree as ET
import pandas as pd
#Import scikit-learn metrics module for accuracy calculation
from sklearn import metrics
##from pattern.text.en import singularize
##nltk.download('vader_lexicon')
##nltk.download('punkt')
#import the relevant modules from the NLTK library
from nltk.sentiment.vader import SentimentIntensityAnalyzer
sentence_tokenizer = nltk.data.load('tokenizers/punkt/english.pickle')
word_tokenizer = nltk.tokenize.RegexpTokenizer(r'\w+')
##tt = ToktokTokenizer()#keeps the url as is...................................................
from nltk.tokenize import TweetTokenizer
tt = TweetTokenizer()
Porter_stemmer = PorterStemmer()
Lancaster_stemmer = LancasterStemmer()
WordNet_lemmatizer = WordNetLemmatizer()
#nltk.download('averaged_perceptron_tagger')
##Train_path="D:/Data/PAN2015-Profiling/pan15-author-profiling-training-dataset-english-2015-04-23"#path for train data
##Test_path="D:/Data/PAN2015-Profiling/pan-ap2015-test/en" #path for test data

##Train_path="D:/NLP/PAN2014/pan14-author-profiling-training-corpus-english-twitter-2014-04-16"#path for train data D:\Data\PAN2013-Profiling\Training\en  D:\NLP\PAN2014
##train_truth_path="D:/NLP/PAN2014/truth_train"
##
##test_truth_path="D:/NLP/PAN2014/pan14_test"
##Test_path="D:/NLP/PAN2014/pan14_test/en"
##Content = open('results_of_clssification_all.txt', 'w')

M_train_file= "/Users/catherine/Desktop/NLP/PAN Datasets/PAN2014blog/2014.1-blog-txtfiles/PAN2014_blog_male_train.1.txt"
F_train_file= "/Users/catherine/Desktop/NLP/PAN Datasets/PAN2014blog/2014.1-blog-txtfiles/PAN2014_blog_female_train.1.txt"

M_test_file= "/Users/catherine/Desktop/NLP/PAN Datasets/PAN2014blog/2014.1-blog-txtfiles/PAN2014_blog_male_test.1.txt"
F_test_file= "/Users/catherine/Desktop/NLP/PAN Datasets/PAN2014blog/2014.1-blog-txtfiles/PAN2014_blog_female_test.1.txt"

def all_txt(M_txt_file,  F_txt_file):
    with open(M_txt_file, mode="r", encoding="utf-8") as f:
            txt_list = [(line[:-1]).split("\t") for line in f]
            male_dict = dict()
            content=[]
            for line in txt_list:
                if line[1] in male_dict:
                # append the new number to the existing array at this slot
                    male_dict[line[1]].append(line[3])
                else:
                # create a new array in this slot
                    male_dict[line[1]] = [line[3]]
    male_content= [("M", (' '.join(a)).split()) for a in list(male_dict.values())]
  
    with open(F_txt_file, mode="r", encoding="utf-8") as f:
            txt_list = [(line[:-1]).split("\t") for line in f]
            female_dict = dict()
            content=[]
            for line in txt_list:
                if line[1] in female_dict:
                # append the new number to the existing array at this slot
                    female_dict[line[1]].append(line[3])
                else:
                # create a new array in this slot
                    female_dict[line[1]] = [line[3]]
    female_content= [("F", (' '.join(a)).split()) for a in list(female_dict.values())]
    all_txt_per_person = male_content + female_content
    return all_txt_per_person

##print( [((all_txt(M_txt_file,  F_txt_file))[i][0])  for i in range(len((all_txt(M_txt_file,  F_txt_file))))]) helps to determine the gender....

      
"""
create feature vector from the training set.....................................................................................
"""

def WordFeatures(word_list, all_training_text):

    fvs_words = np.array([[author.count(word) for word in word_list] for author in all_training_text]).astype(np.float64)

    # normalise by dividing each row by number of tokens for each author........
    fvs_words /= np.c_[np.array([len(author) for author in all_training_text])]

    return fvs_words



"""
From the saved csv file, recover the saved features to be used...............................................................
"""
import csv
word_list=[]
#txt_files =[ "/Users/catherine/Desktop/NLP/PAN Datasets/PAN_TermSelection/2014TermSelection_Blog/outputNewVersion2014blogCHI.txt"]
#txt_files =[ "/Users/catherine/Desktop/NLP/PAN Datasets/PAN_TermSelection/2014TermSelection_Blog/outputNewVersion2014blogIG.txt"]
#txt_files =[ "/Users/catherine/Desktop/NLP/PAN Datasets/PAN_TermSelection/2014TermSelection_Blog/outputNewVersion2014blogGSS.txt"]
txt_files =[ "/Users/catherine/Desktop/NLP/PAN Datasets/PAN_TermSelection/2014TermSelection_Blog/outputNewVersion2014blogOR.1M.txt", "/Users/catherine/Desktop/NLP/PAN Datasets/PAN_TermSelection/2014TermSelection_Blog/outputNewVersion2014blogOR.2F.txt"]
#txt_files =[ "/Users/catherine/Desktop/NLP/PAN Datasets/PAN_TermSelection/2014TermSelection_Blog/outputNewVersion2014blogPMI.1M.txt", "/outputNewVersion201blogPMI.2F.txt"]
#txt_files =[ "/Users/catherine/Desktop/NLP/PAN Datasets/PAN_TermSelection/2014TermSelection_Blog/outputNewVersion201blogRF.1M.txt", "/Users/catherine/Desktop/NLP/PAN Datasets/PAN_TermSelection/2014TermSelection_Blog/outputNewVersion2014blogRF.2F.txt"]
#txt_files =[ "/Users/catherine/Desktop/NLP/PAN Datasets/PAN_TermSelection/2014TermSelection_Blog/outputNewVersion201blogWLLR.1M.txt", "/Users/catherine/Desktop/NLP/PAN Datasets/PAN_TermSelection/2014TermSelection_Blog/outputNewVersion2014blogWLLR.2F.txt"]
print("txt_files of features used",txt_files)
for txt_file in txt_files:
    with open(txt_file, mode="r", encoding="utf-8") as f:
        word_list2 =  ([r.strip() for r in f])[0:100]
        word_list1 =  ([(r.split(":"))[1] for r in word_list2])
        print(txt_file, len(word_list1))
        word_list = word_list + word_list1
print("length of features used =", len(word_list))
#for txt_file in txt_files:
#    with open(txt_file, mode="r", encoding="utf-8") as f:
#        reader = csv.reader(f, delimiter=",")
#        next(reader) # skip header
#        word_list1 =  [r[0] for r in reader]
#        print(txt_file, len(word_list1))
#        word_list = word_list + word_list1
#print("length of features used =", len(word_list))


"""
Prepare the training and test sets to be parsed to the classifies............................................................
"""
A=(all_txt(M_train_file,  F_train_file))
all_training_text = ( [(A[i][1])  for i in range(len(A))])
print("len(all_training_text)",len(all_training_text))
X_train=WordFeatures(word_list, all_training_text)
y_train=np.array( [(A[i][0])  for i in range(len(A))])
print("len(X_train)=",len(X_train), "len(y_train)=", len(y_train))

B=(all_txt(M_test_file,  F_test_file))
all_test_text = ( [(B[i][1])  for i in range(len(B))])
print("len(all_test_text)",len(all_test_text))
X_test = WordFeatures(word_list, all_test_text)
y_test = np.array( [(B[i][0])  for i in range(len(B))])
print("len(X_test=",len(X_test), "len(y_test)=",len(y_test))


X_train = np.nan_to_num(X_train)
X_test = np.nan_to_num(X_test)

"""
Perform feature selection using chi2............................................................
"""
from sklearn.feature_selection import *
##
####def select_Bestfeatures(X, y):
print("selector = SelectKBest(score_func=f_classif, k=200)")
#selector = SelectKBest(score_func=f_classif, k=200)
#fit = selector.fit(X_train,y_train)
#train_selected_features = fit.transform(X_train)
#test_selected_features = fit.transform(X_test)



"""
x_train and x_test based on the new features using chi2............................................................
"""
##X_train = select_Bestfeatures(X_train, y_train)
##X_test = select_Bestfeatures(X_test, y_test)


"""
Perform feature selection using f_classif............................................................
"""
from sklearn.feature_selection import *
##def f_classif_select_Bestfeatures(X, y):
##selector=SelectKBest(score_func=f_classif,k=500)
##fit = selector.fit(X_train,y_train)
##train_selected_features = fit.transform(X_train)
##test_selected_features = fit.transform(X_test)
##    selected_features = fit.transform(X)
##    return selected_features

"""
x_train and x_test based on the new features using f_classif............................................................
"""
##X_train = f_classif_select_Bestfeatures(X_train, y_train)
##X_test = f_classif_select_Bestfeatures(X_test, y_test)



"""
Perform feature selection using mutual_info_classif............................................................
"""
from sklearn.feature_selection import *
##def mutual_info_classif_select_Bestfeatures(X_tr, X_te, y):
##selector=SelectKBest(score_func=mutual_info_classif,k=100)
##fit = selector.fit(X_train,y_train)
##train_selected_features = fit.transform(X_train)
##test_selected_features = fit.transform(X_test)
##    return train_selected_features, test_selected_features

"""
x_train and x_test based on the new features using f_classif............................................................
"""
#X_train =train_selected_features 
#X_test = test_selected_features


"""
Perform feature selection using pca............................................................
"""

##from sklearn.decomposition import PCA
##def PCA_select_Bestfeatures(X):
##    # feature extraction
##    pca = PCA(n_components=200)
##    fit = pca.fit(X, y)
##    Train_selected_features = fit.transform(X)
##    return selected_features

"""
x_train and x_test based on the new features using f_classif............................................................
"""
##X_train = PCA_select_Bestfeatures(X_train)
##X_test = PCA_select_Bestfeatures(X_test)

"""
Below are a list of Classification Algorithms with different distance measres ..............................................................................
"""
"""
K-Nearest Neighbours (average aa over all the nearest neighbour selectiopn)..............................................................................
"""

def tanimoto_distance(x,y):
    """Calculates the tanimoto (normalised manhattan) distance between the vectors x and y
            Keyword arguments:
                x,y -- the vectors between which the distance is to be calculated
    """
    try:
        iter(x)
    except TypeError:
        logging.warning( 'Argument x is not iterable. None is returned')
        return None
    try:
        iter(y)
    except TypeError:
        logging.warning( 'Argument y is not iterable. None is returned')
        return None
    numerator = sum(abs(a - b) for a, b in zip(x, y))
    denominator = sum(max(a,b) for a,b in zip(x,y))
    return numerator/denominator

k=5

print(k)

"""
8.0. tanimoto_distance K-Nearest Neighbours..............................................................................
"""
from sklearn.neighbors import KNeighborsClassifier
knn = KNeighborsClassifier(n_neighbors = k, metric= tanimoto_distance)
knn.fit(X_train, y_train)
y_pred = knn.predict(X_test)
print("tanimoto_distance K-Nearest Neighbours")
print(metrics.accuracy_score(y_test, y_pred))






txt_files of features used ['/Users/catherine/Desktop/NLP/PAN Datasets/PAN_TermSelection/2014TermSelection_Blog/outputNewVersion2014blogOR.1M.txt', '/Users/catherine/Desktop/NLP/PAN Datasets/PAN_TermSelection/2014TermSelection_Blog/outputNewVersion2014blogOR.2F.txt']
/Users/catherine/Desktop/NLP/PAN Datasets/PAN_TermSelection/2014TermSelection_Blog/outputNewVersion2014blogOR.1M.txt 100
/Users/catherine/Desktop/NLP/PAN Datasets/PAN_TermSelection/2014TermSelection_Blog/outputNewVersion2014blogOR.2F.txt 100
length of features used = 200
len(all_training_text) 147
len(X_train)= 147 len(y_train)= 147
len(all_test_text) 78
len(X_test= 78 len(y_test)= 78
selector = SelectKBest(score_func=f_classif, k=200)
5


/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:258: RuntimeWarning: invalid value encountered in double_scalars


tanimoto_distance K-Nearest Neighbours
0.6153846153846154


In [52]:
import sys
sys.path.append("/anaconda3/lib/python3.7/site-packages")
import numpy as np
import numpy
import nltk
import pickle
import pandas as pd
import glob
import os
import re
import operator
from collections import Counter
#from read_xml_files import *
##from s_stemmer import *
from nltk.stem.porter import PorterStemmer
from nltk.stem import LancasterStemmer, WordNetLemmatizer
from itertools import groupby
from nltk.collocations import *
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.cluster import KMeans
from scipy.cluster.vq import whiten
from nltk.tokenize.toktok import ToktokTokenizer
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2
import xml.etree.ElementTree as ET
import pandas as pd
#Import scikit-learn metrics module for accuracy calculation
from sklearn import metrics
##from pattern.text.en import singularize
##nltk.download('vader_lexicon')
##nltk.download('punkt')
#import the relevant modules from the NLTK library
from nltk.sentiment.vader import SentimentIntensityAnalyzer
sentence_tokenizer = nltk.data.load('tokenizers/punkt/english.pickle')
word_tokenizer = nltk.tokenize.RegexpTokenizer(r'\w+')
##tt = ToktokTokenizer()#keeps the url as is...................................................
from nltk.tokenize import TweetTokenizer
tt = TweetTokenizer()
Porter_stemmer = PorterStemmer()
Lancaster_stemmer = LancasterStemmer()
WordNet_lemmatizer = WordNetLemmatizer()
#nltk.download('averaged_perceptron_tagger')
##Train_path="D:/Data/PAN2015-Profiling/pan15-author-profiling-training-dataset-english-2015-04-23"#path for train data
##Test_path="D:/Data/PAN2015-Profiling/pan-ap2015-test/en" #path for test data

##Train_path="D:/NLP/PAN2014/pan14-author-profiling-training-corpus-english-twitter-2014-04-16"#path for train data D:\Data\PAN2013-Profiling\Training\en  D:\NLP\PAN2014
##train_truth_path="D:/NLP/PAN2014/truth_train"
##
##test_truth_path="D:/NLP/PAN2014/pan14_test"
##Test_path="D:/NLP/PAN2014/pan14_test/en"
##Content = open('results_of_clssification_all.txt', 'w')

M_train_file= "/Users/catherine/Desktop/NLP/PAN Datasets/PAN2014blog/2014.1-blog-txtfiles/PAN2014_blog_male_train.1.txt"
F_train_file= "/Users/catherine/Desktop/NLP/PAN Datasets/PAN2014blog/2014.1-blog-txtfiles/PAN2014_blog_female_train.1.txt"

M_test_file= "/Users/catherine/Desktop/NLP/PAN Datasets/PAN2014blog/2014.1-blog-txtfiles/PAN2014_blog_male_test.1.txt"
F_test_file= "/Users/catherine/Desktop/NLP/PAN Datasets/PAN2014blog/2014.1-blog-txtfiles/PAN2014_blog_female_test.1.txt"

def all_txt(M_txt_file,  F_txt_file):
    with open(M_txt_file, mode="r", encoding="utf-8") as f:
            txt_list = [(line[:-1]).split("\t") for line in f]
            male_dict = dict()
            content=[]
            for line in txt_list:
                if line[1] in male_dict:
                # append the new number to the existing array at this slot
                    male_dict[line[1]].append(line[3])
                else:
                # create a new array in this slot
                    male_dict[line[1]] = [line[3]]
    male_content= [("M", (' '.join(a)).split()) for a in list(male_dict.values())]
  
    with open(F_txt_file, mode="r", encoding="utf-8") as f:
            txt_list = [(line[:-1]).split("\t") for line in f]
            female_dict = dict()
            content=[]
            for line in txt_list:
                if line[1] in female_dict:
                # append the new number to the existing array at this slot
                    female_dict[line[1]].append(line[3])
                else:
                # create a new array in this slot
                    female_dict[line[1]] = [line[3]]
    female_content= [("F", (' '.join(a)).split()) for a in list(female_dict.values())]
    all_txt_per_person = male_content + female_content
    return all_txt_per_person

##print( [((all_txt(M_txt_file,  F_txt_file))[i][0])  for i in range(len((all_txt(M_txt_file,  F_txt_file))))]) helps to determine the gender....

      
"""
create feature vector from the training set.....................................................................................
"""

def WordFeatures(word_list, all_training_text):

    fvs_words = np.array([[author.count(word) for word in word_list] for author in all_training_text]).astype(np.float64)

    # normalise by dividing each row by number of tokens for each author........
    fvs_words /= np.c_[np.array([len(author) for author in all_training_text])]

    return fvs_words



"""
From the saved csv file, recover the saved features to be used...............................................................
"""
import csv
word_list=[]
#txt_files =[ "/Users/catherine/Desktop/NLP/PAN Datasets/PAN_TermSelection/2014TermSelection_Blog/outputNewVersion2014blogCHI.txt"]
#txt_files =[ "/Users/catherine/Desktop/NLP/PAN Datasets/PAN_TermSelection/2014TermSelection_Blog/outputNewVersion2014blogIG.txt"]
#txt_files =[ "/Users/catherine/Desktop/NLP/PAN Datasets/PAN_TermSelection/2014TermSelection_Blog/outputNewVersion2014blogGSS.txt"]
#txt_files =[ "/Users/catherine/Desktop/NLP/PAN Datasets/PAN_TermSelection/2014TermSelection_Blog/outputNewVersion2014blogOR.1M.txt", "/Users/catherine/Desktop/NLP/PAN Datasets/PAN_TermSelection/2014TermSelection_Blog/outputNewVersion2014blogOR.2F.txt"]
txt_files =[ "/Users/catherine/Desktop/NLP/PAN Datasets/PAN_TermSelection/2014TermSelection_Blog/outputNewVersion2014blogPMI.1M.txt", "/Users/catherine/Desktop/NLP/PAN Datasets/PAN_TermSelection/2014TermSelection_Blog/outputNewVersion2014blogPMI.2F.txt"]
#txt_files =[ "/Users/catherine/Desktop/NLP/PAN Datasets/PAN_TermSelection/2014TermSelection_Blog/outputNewVersion2014blogRF.1M.txt", "/Users/catherine/Desktop/NLP/PAN Datasets/PAN_TermSelection/2014TermSelection_Blog/outputNewVersion2014blogRF.2F.txt"]
#txt_files =[ "/Users/catherine/Desktop/NLP/PAN Datasets/PAN_TermSelection/2014TermSelection_Blog/outputNewVersion201blogWLLR.1M.txt", "/Users/catherine/Desktop/NLP/PAN Datasets/PAN_TermSelection/2014TermSelection_Blog/outputNewVersion2014blogWLLR.2F.txt"]
print("txt_files of features used",txt_files)
for txt_file in txt_files:
    with open(txt_file, mode="r", encoding="utf-8") as f:
        word_list2 =  ([r.strip() for r in f])[0:100]
        word_list1 =  ([(r.split(":"))[1] for r in word_list2])
        print(txt_file, len(word_list1))
        word_list = word_list + word_list1
print("length of features used =", len(word_list))
#for txt_file in txt_files:
#    with open(txt_file, mode="r", encoding="utf-8") as f:
#        reader = csv.reader(f, delimiter=",")
#        next(reader) # skip header
#        word_list1 =  [r[0] for r in reader]
#        print(txt_file, len(word_list1))
#        word_list = word_list + word_list1
#print("length of features used =", len(word_list))


"""
Prepare the training and test sets to be parsed to the classifies............................................................
"""
A=(all_txt(M_train_file,  F_train_file))
all_training_text = ( [(A[i][1])  for i in range(len(A))])
print("len(all_training_text)",len(all_training_text))
X_train=WordFeatures(word_list, all_training_text)
y_train=np.array( [(A[i][0])  for i in range(len(A))])
print("len(X_train)=",len(X_train), "len(y_train)=", len(y_train))

B=(all_txt(M_test_file,  F_test_file))
all_test_text = ( [(B[i][1])  for i in range(len(B))])
print("len(all_test_text)",len(all_test_text))
X_test = WordFeatures(word_list, all_test_text)
y_test = np.array( [(B[i][0])  for i in range(len(B))])
print("len(X_test=",len(X_test), "len(y_test)=",len(y_test))


X_train = np.nan_to_num(X_train)
X_test = np.nan_to_num(X_test)

"""
Perform feature selection using chi2............................................................
"""
from sklearn.feature_selection import *
##
####def select_Bestfeatures(X, y):
print("selector = SelectKBest(score_func=f_classif, k=200)")
#selector = SelectKBest(score_func=f_classif, k=200)
#fit = selector.fit(X_train,y_train)
#train_selected_features = fit.transform(X_train)
#test_selected_features = fit.transform(X_test)



"""
x_train and x_test based on the new features using chi2............................................................
"""
##X_train = select_Bestfeatures(X_train, y_train)
##X_test = select_Bestfeatures(X_test, y_test)


"""
Perform feature selection using f_classif............................................................
"""
from sklearn.feature_selection import *
##def f_classif_select_Bestfeatures(X, y):
##selector=SelectKBest(score_func=f_classif,k=500)
##fit = selector.fit(X_train,y_train)
##train_selected_features = fit.transform(X_train)
##test_selected_features = fit.transform(X_test)
##    selected_features = fit.transform(X)
##    return selected_features

"""
x_train and x_test based on the new features using f_classif............................................................
"""
##X_train = f_classif_select_Bestfeatures(X_train, y_train)
##X_test = f_classif_select_Bestfeatures(X_test, y_test)



"""
Perform feature selection using mutual_info_classif............................................................
"""
from sklearn.feature_selection import *
##def mutual_info_classif_select_Bestfeatures(X_tr, X_te, y):
##selector=SelectKBest(score_func=mutual_info_classif,k=100)
##fit = selector.fit(X_train,y_train)
##train_selected_features = fit.transform(X_train)
##test_selected_features = fit.transform(X_test)
##    return train_selected_features, test_selected_features

"""
x_train and x_test based on the new features using f_classif............................................................
"""
#X_train =train_selected_features 
#X_test = test_selected_features


"""
Perform feature selection using pca............................................................
"""

##from sklearn.decomposition import PCA
##def PCA_select_Bestfeatures(X):
##    # feature extraction
##    pca = PCA(n_components=200)
##    fit = pca.fit(X, y)
##    Train_selected_features = fit.transform(X)
##    return selected_features

"""
x_train and x_test based on the new features using f_classif............................................................
"""
##X_train = PCA_select_Bestfeatures(X_train)
##X_test = PCA_select_Bestfeatures(X_test)

"""
Below are a list of Classification Algorithms with different distance measres ..............................................................................
"""
"""
K-Nearest Neighbours (average aa over all the nearest neighbour selectiopn)..............................................................................
"""

def tanimoto_distance(x,y):
    """Calculates the tanimoto (normalised manhattan) distance between the vectors x and y
            Keyword arguments:
                x,y -- the vectors between which the distance is to be calculated
    """
    try:
        iter(x)
    except TypeError:
        logging.warning( 'Argument x is not iterable. None is returned')
        return None
    try:
        iter(y)
    except TypeError:
        logging.warning( 'Argument y is not iterable. None is returned')
        return None
    numerator = sum(abs(a - b) for a, b in zip(x, y))
    denominator = sum(max(a,b) for a,b in zip(x,y))
    return numerator/denominator

k=5

print(k)

"""
8.0. tanimoto_distance K-Nearest Neighbours..............................................................................
"""
from sklearn.neighbors import KNeighborsClassifier
knn = KNeighborsClassifier(n_neighbors = k, metric= tanimoto_distance)
knn.fit(X_train, y_train)
y_pred = knn.predict(X_test)
print("tanimoto_distance K-Nearest Neighbours")
print(metrics.accuracy_score(y_test, y_pred))







txt_files of features used ['/Users/catherine/Desktop/NLP/PAN Datasets/PAN_TermSelection/2014TermSelection_Blog/outputNewVersion2014blogPMI.1M.txt', '/Users/catherine/Desktop/NLP/PAN Datasets/PAN_TermSelection/2014TermSelection_Blog/outputNewVersion2014blogPMI.2F.txt']
/Users/catherine/Desktop/NLP/PAN Datasets/PAN_TermSelection/2014TermSelection_Blog/outputNewVersion2014blogPMI.1M.txt 100
/Users/catherine/Desktop/NLP/PAN Datasets/PAN_TermSelection/2014TermSelection_Blog/outputNewVersion2014blogPMI.2F.txt 100
length of features used = 200
len(all_training_text) 147
len(X_train)= 147 len(y_train)= 147
len(all_test_text) 78
len(X_test= 78 len(y_test)= 78
selector = SelectKBest(score_func=f_classif, k=200)
5


/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:258: RuntimeWarning: invalid value encountered in double_scalars


tanimoto_distance K-Nearest Neighbours
0.6538461538461539


In [53]:
import sys
sys.path.append("/anaconda3/lib/python3.7/site-packages")
import numpy as np
import numpy
import nltk
import pickle
import pandas as pd
import glob
import os
import re
import operator
from collections import Counter
#from read_xml_files import *
##from s_stemmer import *
from nltk.stem.porter import PorterStemmer
from nltk.stem import LancasterStemmer, WordNetLemmatizer
from itertools import groupby
from nltk.collocations import *
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.cluster import KMeans
from scipy.cluster.vq import whiten
from nltk.tokenize.toktok import ToktokTokenizer
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2
import xml.etree.ElementTree as ET
import pandas as pd
#Import scikit-learn metrics module for accuracy calculation
from sklearn import metrics
##from pattern.text.en import singularize
##nltk.download('vader_lexicon')
##nltk.download('punkt')
#import the relevant modules from the NLTK library
from nltk.sentiment.vader import SentimentIntensityAnalyzer
sentence_tokenizer = nltk.data.load('tokenizers/punkt/english.pickle')
word_tokenizer = nltk.tokenize.RegexpTokenizer(r'\w+')
##tt = ToktokTokenizer()#keeps the url as is...................................................
from nltk.tokenize import TweetTokenizer
tt = TweetTokenizer()
Porter_stemmer = PorterStemmer()
Lancaster_stemmer = LancasterStemmer()
WordNet_lemmatizer = WordNetLemmatizer()
#nltk.download('averaged_perceptron_tagger')
##Train_path="D:/Data/PAN2015-Profiling/pan15-author-profiling-training-dataset-english-2015-04-23"#path for train data
##Test_path="D:/Data/PAN2015-Profiling/pan-ap2015-test/en" #path for test data

##Train_path="D:/NLP/PAN2014/pan14-author-profiling-training-corpus-english-twitter-2014-04-16"#path for train data D:\Data\PAN2013-Profiling\Training\en  D:\NLP\PAN2014
##train_truth_path="D:/NLP/PAN2014/truth_train"
##
##test_truth_path="D:/NLP/PAN2014/pan14_test"
##Test_path="D:/NLP/PAN2014/pan14_test/en"
##Content = open('results_of_clssification_all.txt', 'w')

M_train_file= "/Users/catherine/Desktop/NLP/PAN Datasets/PAN2014blog/2014.1-blog-txtfiles/PAN2014_blog_male_train.1.txt"
F_train_file= "/Users/catherine/Desktop/NLP/PAN Datasets/PAN2014blog/2014.1-blog-txtfiles/PAN2014_blog_female_train.1.txt"

M_test_file= "/Users/catherine/Desktop/NLP/PAN Datasets/PAN2014blog/2014.1-blog-txtfiles/PAN2014_blog_male_test.1.txt"
F_test_file= "/Users/catherine/Desktop/NLP/PAN Datasets/PAN2014blog/2014.1-blog-txtfiles/PAN2014_blog_female_test.1.txt"

def all_txt(M_txt_file,  F_txt_file):
    with open(M_txt_file, mode="r", encoding="utf-8") as f:
            txt_list = [(line[:-1]).split("\t") for line in f]
            male_dict = dict()
            content=[]
            for line in txt_list:
                if line[1] in male_dict:
                # append the new number to the existing array at this slot
                    male_dict[line[1]].append(line[3])
                else:
                # create a new array in this slot
                    male_dict[line[1]] = [line[3]]
    male_content= [("M", (' '.join(a)).split()) for a in list(male_dict.values())]
  
    with open(F_txt_file, mode="r", encoding="utf-8") as f:
            txt_list = [(line[:-1]).split("\t") for line in f]
            female_dict = dict()
            content=[]
            for line in txt_list:
                if line[1] in female_dict:
                # append the new number to the existing array at this slot
                    female_dict[line[1]].append(line[3])
                else:
                # create a new array in this slot
                    female_dict[line[1]] = [line[3]]
    female_content= [("F", (' '.join(a)).split()) for a in list(female_dict.values())]
    all_txt_per_person = male_content + female_content
    return all_txt_per_person

##print( [((all_txt(M_txt_file,  F_txt_file))[i][0])  for i in range(len((all_txt(M_txt_file,  F_txt_file))))]) helps to determine the gender....

      
"""
create feature vector from the training set.....................................................................................
"""

def WordFeatures(word_list, all_training_text):

    fvs_words = np.array([[author.count(word) for word in word_list] for author in all_training_text]).astype(np.float64)

    # normalise by dividing each row by number of tokens for each author........
    fvs_words /= np.c_[np.array([len(author) for author in all_training_text])]

    return fvs_words



"""
From the saved csv file, recover the saved features to be used...............................................................
"""
import csv
word_list=[]
#txt_files =[ "/Users/catherine/Desktop/NLP/PAN Datasets/PAN_TermSelection/2014TermSelection_Blog/outputNewVersion2014blogCHI.txt"]
#txt_files =[ "/Users/catherine/Desktop/NLP/PAN Datasets/PAN_TermSelection/2014TermSelection_Blog/outputNewVersion2014blogIG.txt"]
#txt_files =[ "/Users/catherine/Desktop/NLP/PAN Datasets/PAN_TermSelection/2014TermSelection_Blog/outputNewVersion2014blogGSS.txt"]
#txt_files =[ "/Users/catherine/Desktop/NLP/PAN Datasets/PAN_TermSelection/2014TermSelection_Blog/outputNewVersion2014blogOR.1M.txt", "/Users/catherine/Desktop/NLP/PAN Datasets/PAN_TermSelection/2014TermSelection_Blog/outputNewVersion2014blogOR.2F.txt"]
#txt_files =[ "/Users/catherine/Desktop/NLP/PAN Datasets/PAN_TermSelection/2014TermSelection_Blog/outputNewVersion2014blogPMI.1M.txt", "/Users/catherine/Desktop/NLP/PAN Datasets/PAN_TermSelection/2014TermSelection_Blog/outputNewVersion2014blogPMI.2F.txt"]
txt_files =[ "/Users/catherine/Desktop/NLP/PAN Datasets/PAN_TermSelection/2014TermSelection_Blog/outputNewVersion2014blogRF.1M.txt", "/Users/catherine/Desktop/NLP/PAN Datasets/PAN_TermSelection/2014TermSelection_Blog/outputNewVersion2014blogRF.2F.txt"]
#txt_files =[ "/Users/catherine/Desktop/NLP/PAN Datasets/PAN_TermSelection/2014TermSelection_Blog/outputNewVersion201blogWLLR.1M.txt", "/Users/catherine/Desktop/NLP/PAN Datasets/PAN_TermSelection/2014TermSelection_Blog/outputNewVersion2014blogWLLR.2F.txt"]
print("txt_files of features used",txt_files)
for txt_file in txt_files:
    with open(txt_file, mode="r", encoding="utf-8") as f:
        word_list2 =  ([r.strip() for r in f])[0:100]
        word_list1 =  ([(r.split(":"))[1] for r in word_list2])
        print(txt_file, len(word_list1))
        word_list = word_list + word_list1
print("length of features used =", len(word_list))
#for txt_file in txt_files:
#    with open(txt_file, mode="r", encoding="utf-8") as f:
#        reader = csv.reader(f, delimiter=",")
#        next(reader) # skip header
#        word_list1 =  [r[0] for r in reader]
#        print(txt_file, len(word_list1))
#        word_list = word_list + word_list1
#print("length of features used =", len(word_list))


"""
Prepare the training and test sets to be parsed to the classifies............................................................
"""
A=(all_txt(M_train_file,  F_train_file))
all_training_text = ( [(A[i][1])  for i in range(len(A))])
print("len(all_training_text)",len(all_training_text))
X_train=WordFeatures(word_list, all_training_text)
y_train=np.array( [(A[i][0])  for i in range(len(A))])
print("len(X_train)=",len(X_train), "len(y_train)=", len(y_train))

B=(all_txt(M_test_file,  F_test_file))
all_test_text = ( [(B[i][1])  for i in range(len(B))])
print("len(all_test_text)",len(all_test_text))
X_test = WordFeatures(word_list, all_test_text)
y_test = np.array( [(B[i][0])  for i in range(len(B))])
print("len(X_test=",len(X_test), "len(y_test)=",len(y_test))


X_train = np.nan_to_num(X_train)
X_test = np.nan_to_num(X_test)

"""
Perform feature selection using chi2............................................................
"""
from sklearn.feature_selection import *
##
####def select_Bestfeatures(X, y):
print("selector = SelectKBest(score_func=f_classif, k=200)")
#selector = SelectKBest(score_func=f_classif, k=200)
#fit = selector.fit(X_train,y_train)
#train_selected_features = fit.transform(X_train)
#test_selected_features = fit.transform(X_test)



"""
x_train and x_test based on the new features using chi2............................................................
"""
##X_train = select_Bestfeatures(X_train, y_train)
##X_test = select_Bestfeatures(X_test, y_test)


"""
Perform feature selection using f_classif............................................................
"""
from sklearn.feature_selection import *
##def f_classif_select_Bestfeatures(X, y):
##selector=SelectKBest(score_func=f_classif,k=500)
##fit = selector.fit(X_train,y_train)
##train_selected_features = fit.transform(X_train)
##test_selected_features = fit.transform(X_test)
##    selected_features = fit.transform(X)
##    return selected_features

"""
x_train and x_test based on the new features using f_classif............................................................
"""
##X_train = f_classif_select_Bestfeatures(X_train, y_train)
##X_test = f_classif_select_Bestfeatures(X_test, y_test)



"""
Perform feature selection using mutual_info_classif............................................................
"""
from sklearn.feature_selection import *
##def mutual_info_classif_select_Bestfeatures(X_tr, X_te, y):
##selector=SelectKBest(score_func=mutual_info_classif,k=100)
##fit = selector.fit(X_train,y_train)
##train_selected_features = fit.transform(X_train)
##test_selected_features = fit.transform(X_test)
##    return train_selected_features, test_selected_features

"""
x_train and x_test based on the new features using f_classif............................................................
"""
#X_train =train_selected_features 
#X_test = test_selected_features


"""
Perform feature selection using pca............................................................
"""

##from sklearn.decomposition import PCA
##def PCA_select_Bestfeatures(X):
##    # feature extraction
##    pca = PCA(n_components=200)
##    fit = pca.fit(X, y)
##    Train_selected_features = fit.transform(X)
##    return selected_features

"""
x_train and x_test based on the new features using f_classif............................................................
"""
##X_train = PCA_select_Bestfeatures(X_train)
##X_test = PCA_select_Bestfeatures(X_test)

"""
Below are a list of Classification Algorithms with different distance measres ..............................................................................
"""
"""
K-Nearest Neighbours (average aa over all the nearest neighbour selectiopn)..............................................................................
"""

def tanimoto_distance(x,y):
    """Calculates the tanimoto (normalised manhattan) distance between the vectors x and y
            Keyword arguments:
                x,y -- the vectors between which the distance is to be calculated
    """
    try:
        iter(x)
    except TypeError:
        logging.warning( 'Argument x is not iterable. None is returned')
        return None
    try:
        iter(y)
    except TypeError:
        logging.warning( 'Argument y is not iterable. None is returned')
        return None
    numerator = sum(abs(a - b) for a, b in zip(x, y))
    denominator = sum(max(a,b) for a,b in zip(x,y))
    return numerator/denominator

k=5

print(k)

"""
8.0. tanimoto_distance K-Nearest Neighbours..............................................................................
"""
from sklearn.neighbors import KNeighborsClassifier
knn = KNeighborsClassifier(n_neighbors = k, metric= tanimoto_distance)
knn.fit(X_train, y_train)
y_pred = knn.predict(X_test)
print("tanimoto_distance K-Nearest Neighbours")
print(metrics.accuracy_score(y_test, y_pred))








txt_files of features used ['/Users/catherine/Desktop/NLP/PAN Datasets/PAN_TermSelection/2014TermSelection_Blog/outputNewVersion2014blogRF.1M.txt', '/Users/catherine/Desktop/NLP/PAN Datasets/PAN_TermSelection/2014TermSelection_Blog/outputNewVersion2014blogRF.2F.txt']
/Users/catherine/Desktop/NLP/PAN Datasets/PAN_TermSelection/2014TermSelection_Blog/outputNewVersion2014blogRF.1M.txt 100
/Users/catherine/Desktop/NLP/PAN Datasets/PAN_TermSelection/2014TermSelection_Blog/outputNewVersion2014blogRF.2F.txt 100
length of features used = 200
len(all_training_text) 147
len(X_train)= 147 len(y_train)= 147
len(all_test_text) 78
len(X_test= 78 len(y_test)= 78
selector = SelectKBest(score_func=f_classif, k=200)
5


/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:258: RuntimeWarning: invalid value encountered in double_scalars


tanimoto_distance K-Nearest Neighbours
0.5769230769230769


In [55]:
import sys
sys.path.append("/anaconda3/lib/python3.7/site-packages")
import numpy as np
import numpy
import nltk
import pickle
import pandas as pd
import glob
import os
import re
import operator
from collections import Counter
#from read_xml_files import *
##from s_stemmer import *
from nltk.stem.porter import PorterStemmer
from nltk.stem import LancasterStemmer, WordNetLemmatizer
from itertools import groupby
from nltk.collocations import *
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.cluster import KMeans
from scipy.cluster.vq import whiten
from nltk.tokenize.toktok import ToktokTokenizer
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2
import xml.etree.ElementTree as ET
import pandas as pd
#Import scikit-learn metrics module for accuracy calculation
from sklearn import metrics
##from pattern.text.en import singularize
##nltk.download('vader_lexicon')
##nltk.download('punkt')
#import the relevant modules from the NLTK library
from nltk.sentiment.vader import SentimentIntensityAnalyzer
sentence_tokenizer = nltk.data.load('tokenizers/punkt/english.pickle')
word_tokenizer = nltk.tokenize.RegexpTokenizer(r'\w+')
##tt = ToktokTokenizer()#keeps the url as is...................................................
from nltk.tokenize import TweetTokenizer
tt = TweetTokenizer()
Porter_stemmer = PorterStemmer()
Lancaster_stemmer = LancasterStemmer()
WordNet_lemmatizer = WordNetLemmatizer()
#nltk.download('averaged_perceptron_tagger')
##Train_path="D:/Data/PAN2015-Profiling/pan15-author-profiling-training-dataset-english-2015-04-23"#path for train data
##Test_path="D:/Data/PAN2015-Profiling/pan-ap2015-test/en" #path for test data

##Train_path="D:/NLP/PAN2014/pan14-author-profiling-training-corpus-english-twitter-2014-04-16"#path for train data D:\Data\PAN2013-Profiling\Training\en  D:\NLP\PAN2014
##train_truth_path="D:/NLP/PAN2014/truth_train"
##
##test_truth_path="D:/NLP/PAN2014/pan14_test"
##Test_path="D:/NLP/PAN2014/pan14_test/en"
##Content = open('results_of_clssification_all.txt', 'w')

M_train_file= "/Users/catherine/Desktop/NLP/PAN Datasets/PAN2014blog/2014.1-blog-txtfiles/PAN2014_blog_male_train.1.txt"
F_train_file= "/Users/catherine/Desktop/NLP/PAN Datasets/PAN2014blog/2014.1-blog-txtfiles/PAN2014_blog_female_train.1.txt"

M_test_file= "/Users/catherine/Desktop/NLP/PAN Datasets/PAN2014blog/2014.1-blog-txtfiles/PAN2014_blog_male_test.1.txt"
F_test_file= "/Users/catherine/Desktop/NLP/PAN Datasets/PAN2014blog/2014.1-blog-txtfiles/PAN2014_blog_female_test.1.txt"

def all_txt(M_txt_file,  F_txt_file):
    with open(M_txt_file, mode="r", encoding="utf-8") as f:
            txt_list = [(line[:-1]).split("\t") for line in f]
            male_dict = dict()
            content=[]
            for line in txt_list:
                if line[1] in male_dict:
                # append the new number to the existing array at this slot
                    male_dict[line[1]].append(line[3])
                else:
                # create a new array in this slot
                    male_dict[line[1]] = [line[3]]
    male_content= [("M", (' '.join(a)).split()) for a in list(male_dict.values())]
  
    with open(F_txt_file, mode="r", encoding="utf-8") as f:
            txt_list = [(line[:-1]).split("\t") for line in f]
            female_dict = dict()
            content=[]
            for line in txt_list:
                if line[1] in female_dict:
                # append the new number to the existing array at this slot
                    female_dict[line[1]].append(line[3])
                else:
                # create a new array in this slot
                    female_dict[line[1]] = [line[3]]
    female_content= [("F", (' '.join(a)).split()) for a in list(female_dict.values())]
    all_txt_per_person = male_content + female_content
    return all_txt_per_person

##print( [((all_txt(M_txt_file,  F_txt_file))[i][0])  for i in range(len((all_txt(M_txt_file,  F_txt_file))))]) helps to determine the gender....

      
"""
create feature vector from the training set.....................................................................................
"""

def WordFeatures(word_list, all_training_text):

    fvs_words = np.array([[author.count(word) for word in word_list] for author in all_training_text]).astype(np.float64)

    # normalise by dividing each row by number of tokens for each author........
    fvs_words /= np.c_[np.array([len(author) for author in all_training_text])]

    return fvs_words



"""
From the saved csv file, recover the saved features to be used...............................................................
"""
import csv
word_list=[]
#txt_files =[ "/Users/catherine/Desktop/NLP/PAN Datasets/PAN_TermSelection/2014TermSelection_Blog/outputNewVersion2014blogCHI.txt"]
#txt_files =[ "/Users/catherine/Desktop/NLP/PAN Datasets/PAN_TermSelection/2014TermSelection_Blog/outputNewVersion2014blogIG.txt"]
#txt_files =[ "/Users/catherine/Desktop/NLP/PAN Datasets/PAN_TermSelection/2014TermSelection_Blog/outputNewVersion2014blogGSS.txt"]
#txt_files =[ "/Users/catherine/Desktop/NLP/PAN Datasets/PAN_TermSelection/2014TermSelection_Blog/outputNewVersion2014blogOR.1M.txt", "/Users/catherine/Desktop/NLP/PAN Datasets/PAN_TermSelection/2014TermSelection_Blog/outputNewVersion2014blogOR.2F.txt"]
#txt_files =[ "/Users/catherine/Desktop/NLP/PAN Datasets/PAN_TermSelection/2014TermSelection_Blog/outputNewVersion2014blogPMI.1M.txt", "/Users/catherine/Desktop/NLP/PAN Datasets/PAN_TermSelection/2014TermSelection_Blog/outputNewVersion2014blogPMI.2F.txt"]
#txt_files =[ "/Users/catherine/Desktop/NLP/PAN Datasets/PAN_TermSelection/2014TermSelection_Blog/outputNewVersion2014blogRF.1M.txt", "/Users/catherine/Desktop/NLP/PAN Datasets/PAN_TermSelection/2014TermSelection_Blog/outputNewVersion2014blogRF.2F.txt"]
txt_files =[ "/Users/catherine/Desktop/NLP/PAN Datasets/PAN_TermSelection/2014TermSelection_Blog/outputNewVersion2014blogWLLR.1M.txt", "/Users/catherine/Desktop/NLP/PAN Datasets/PAN_TermSelection/2014TermSelection_Blog/outputNewVersion2014blogWLLR.2F.txt"]
print("txt_files of features used",txt_files)
for txt_file in txt_files:
    with open(txt_file, mode="r", encoding="utf-8") as f:
        word_list2 =  ([r.strip() for r in f])[0:100]
        word_list1 =  ([(r.split(":"))[1] for r in word_list2])
        print(txt_file, len(word_list1))
        word_list = word_list + word_list1
print("length of features used =", len(word_list))
#for txt_file in txt_files:
#    with open(txt_file, mode="r", encoding="utf-8") as f:
#        reader = csv.reader(f, delimiter=",")
#        next(reader) # skip header
#        word_list1 =  [r[0] for r in reader]
#        print(txt_file, len(word_list1))
#        word_list = word_list + word_list1
#print("length of features used =", len(word_list))


"""
Prepare the training and test sets to be parsed to the classifies............................................................
"""
A=(all_txt(M_train_file,  F_train_file))
all_training_text = ( [(A[i][1])  for i in range(len(A))])
print("len(all_training_text)",len(all_training_text))
X_train=WordFeatures(word_list, all_training_text)
y_train=np.array( [(A[i][0])  for i in range(len(A))])
print("len(X_train)=",len(X_train), "len(y_train)=", len(y_train))

B=(all_txt(M_test_file,  F_test_file))
all_test_text = ( [(B[i][1])  for i in range(len(B))])
print("len(all_test_text)",len(all_test_text))
X_test = WordFeatures(word_list, all_test_text)
y_test = np.array( [(B[i][0])  for i in range(len(B))])
print("len(X_test=",len(X_test), "len(y_test)=",len(y_test))


X_train = np.nan_to_num(X_train)
X_test = np.nan_to_num(X_test)

"""
Perform feature selection using chi2............................................................
"""
from sklearn.feature_selection import *
##
####def select_Bestfeatures(X, y):
print("selector = SelectKBest(score_func=f_classif, k=200)")
#selector = SelectKBest(score_func=f_classif, k=200)
#fit = selector.fit(X_train,y_train)
#train_selected_features = fit.transform(X_train)
#test_selected_features = fit.transform(X_test)



"""
x_train and x_test based on the new features using chi2............................................................
"""
##X_train = select_Bestfeatures(X_train, y_train)
##X_test = select_Bestfeatures(X_test, y_test)


"""
Perform feature selection using f_classif............................................................
"""
from sklearn.feature_selection import *
##def f_classif_select_Bestfeatures(X, y):
##selector=SelectKBest(score_func=f_classif,k=500)
##fit = selector.fit(X_train,y_train)
##train_selected_features = fit.transform(X_train)
##test_selected_features = fit.transform(X_test)
##    selected_features = fit.transform(X)
##    return selected_features

"""
x_train and x_test based on the new features using f_classif............................................................
"""
##X_train = f_classif_select_Bestfeatures(X_train, y_train)
##X_test = f_classif_select_Bestfeatures(X_test, y_test)



"""
Perform feature selection using mutual_info_classif............................................................
"""
from sklearn.feature_selection import *
##def mutual_info_classif_select_Bestfeatures(X_tr, X_te, y):
##selector=SelectKBest(score_func=mutual_info_classif,k=100)
##fit = selector.fit(X_train,y_train)
##train_selected_features = fit.transform(X_train)
##test_selected_features = fit.transform(X_test)
##    return train_selected_features, test_selected_features

"""
x_train and x_test based on the new features using f_classif............................................................
"""
#X_train =train_selected_features 
#X_test = test_selected_features


"""
Perform feature selection using pca............................................................
"""

##from sklearn.decomposition import PCA
##def PCA_select_Bestfeatures(X):
##    # feature extraction
##    pca = PCA(n_components=200)
##    fit = pca.fit(X, y)
##    Train_selected_features = fit.transform(X)
##    return selected_features

"""
x_train and x_test based on the new features using f_classif............................................................
"""
##X_train = PCA_select_Bestfeatures(X_train)
##X_test = PCA_select_Bestfeatures(X_test)

"""
Below are a list of Classification Algorithms with different distance measres ..............................................................................
"""
"""
K-Nearest Neighbours (average aa over all the nearest neighbour selectiopn)..............................................................................
"""

def tanimoto_distance(x,y):
    """Calculates the tanimoto (normalised manhattan) distance between the vectors x and y
            Keyword arguments:
                x,y -- the vectors between which the distance is to be calculated
    """
    try:
        iter(x)
    except TypeError:
        logging.warning( 'Argument x is not iterable. None is returned')
        return None
    try:
        iter(y)
    except TypeError:
        logging.warning( 'Argument y is not iterable. None is returned')
        return None
    numerator = sum(abs(a - b) for a, b in zip(x, y))
    denominator = sum(max(a,b) for a,b in zip(x,y))
    return numerator/denominator

k=5

print(k)

"""
8.0. tanimoto_distance K-Nearest Neighbours..............................................................................
"""
from sklearn.neighbors import KNeighborsClassifier
knn = KNeighborsClassifier(n_neighbors = k, metric= tanimoto_distance)
knn.fit(X_train, y_train)
y_pred = knn.predict(X_test)
print("tanimoto_distance K-Nearest Neighbours")
print(metrics.accuracy_score(y_test, y_pred))









txt_files of features used ['/Users/catherine/Desktop/NLP/PAN Datasets/PAN_TermSelection/2014TermSelection_Blog/outputNewVersion2014blogWLLR.1M.txt', '/Users/catherine/Desktop/NLP/PAN Datasets/PAN_TermSelection/2014TermSelection_Blog/outputNewVersion2014blogWLLR.2F.txt']
/Users/catherine/Desktop/NLP/PAN Datasets/PAN_TermSelection/2014TermSelection_Blog/outputNewVersion2014blogWLLR.1M.txt 100
/Users/catherine/Desktop/NLP/PAN Datasets/PAN_TermSelection/2014TermSelection_Blog/outputNewVersion2014blogWLLR.2F.txt 100
length of features used = 200
len(all_training_text) 147
len(X_train)= 147 len(y_train)= 147
len(all_test_text) 78
len(X_test= 78 len(y_test)= 78
selector = SelectKBest(score_func=f_classif, k=200)
5
tanimoto_distance K-Nearest Neighbours
0.6025641025641025


In [1]:
import sys
sys.path.append("/anaconda3/lib/python3.7/site-packages")
import numpy as np
import numpy
import nltk
import pickle
import pandas as pd
import glob
import os
import re
import operator
from collections import Counter
#from read_xml_files import *
##from s_stemmer import *
from nltk.stem.porter import PorterStemmer
from nltk.stem import LancasterStemmer, WordNetLemmatizer
from itertools import groupby
from nltk.collocations import *
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.cluster import KMeans
from scipy.cluster.vq import whiten
from nltk.tokenize.toktok import ToktokTokenizer
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2
import xml.etree.ElementTree as ET
import pandas as pd
#Import scikit-learn metrics module for accuracy calculation
from sklearn import metrics
##from pattern.text.en import singularize
##nltk.download('vader_lexicon')
##nltk.download('punkt')
#import the relevant modules from the NLTK library
from nltk.sentiment.vader import SentimentIntensityAnalyzer
sentence_tokenizer = nltk.data.load('tokenizers/punkt/english.pickle')
word_tokenizer = nltk.tokenize.RegexpTokenizer(r'\w+')
##tt = ToktokTokenizer()#keeps the url as is...................................................
from nltk.tokenize import TweetTokenizer
tt = TweetTokenizer()
Porter_stemmer = PorterStemmer()
Lancaster_stemmer = LancasterStemmer()
WordNet_lemmatizer = WordNetLemmatizer()
#nltk.download('averaged_perceptron_tagger')
##Train_path="D:/Data/PAN2015-Profiling/pan15-author-profiling-training-dataset-english-2015-04-23"#path for train data
##Test_path="D:/Data/PAN2015-Profiling/pan-ap2015-test/en" #path for test data

##Train_path="D:/NLP/PAN2014/pan14-author-profiling-training-corpus-english-twitter-2014-04-16"#path for train data D:\Data\PAN2013-Profiling\Training\en  D:\NLP\PAN2014
##train_truth_path="D:/NLP/PAN2014/truth_train"
##
##test_truth_path="D:/NLP/PAN2014/pan14_test"
##Test_path="D:/NLP/PAN2014/pan14_test/en"
##Content = open('results_of_clssification_all.txt', 'w')

M_train_file= "PAN2014_blog_male_train.1.txt"
F_train_file= "PAN2014_blog_female_train.1.txt"

M_test_file= "PAN2014_blog_male_test.1.txt"
F_test_file= "PAN2014_blog_female_test.1.txt"

def all_txt(M_txt_file,  F_txt_file):
    with open(M_txt_file, mode="r", encoding="utf-8") as f:
            txt_list = [(line[:-1]).split("\t") for line in f]
            male_dict = dict()
            content=[]
            for line in txt_list:
                if line[1] in male_dict:
                # append the new number to the existing array at this slot
                    male_dict[line[1]].append(line[3])
                else:
                # create a new array in this slot
                    male_dict[line[1]] = [line[3]]
    male_content= [("M", (' '.join(a)).split()) for a in list(male_dict.values())]
  
    with open(F_txt_file, mode="r", encoding="utf-8") as f:
            txt_list = [(line[:-1]).split("\t") for line in f]
            female_dict = dict()
            content=[]
            for line in txt_list:
                if line[1] in female_dict:
                # append the new number to the existing array at this slot
                    female_dict[line[1]].append(line[3])
                else:
                # create a new array in this slot
                    female_dict[line[1]] = [line[3]]
    female_content= [("F", (' '.join(a)).split()) for a in list(female_dict.values())]
    all_txt_per_person = male_content + female_content
    return all_txt_per_person

##print( [((all_txt(M_txt_file,  F_txt_file))[i][0])  for i in range(len((all_txt(M_txt_file,  F_txt_file))))]) helps to determine the gender....

      
"""
create feature vector from the training set.....................................................................................
"""

def WordFeatures(word_list, all_training_text):

    fvs_words = np.array([[author.count(word) for word in word_list] for author in all_training_text]).astype(np.float64)

    # normalise by dividing each row by number of tokens for each author........
    fvs_words /= np.c_[np.array([len(author) for author in all_training_text])]

    return fvs_words



"""
From the saved csv file, recover the saved features to be used...............................................................
"""
import csv
word_list=[]
##txt_files =[ "/Users/catherine/Desktop/NLP/PAN Datasets/PAN_chosen_tokens/2014/output2014OR.1M.txt", "/Users/catherine/Desktop/NLP/PAN Datasets/PAN_chosen_tokens/2014/output2014OR.2F.txt"]
txt_files =[ "/Users/catherine/Desktop/NLP/PAN Datasets/PAN2014blog/2014.1-blog-txtfiles/PAN2014_blog_female_chosen_set.1.csv", "/Users/catherine/Desktop/NLP/PAN Datasets/PAN2014blog/2014.1-blog-txtfiles/PAN2014_blog_male_chosen_set.1.csv"]
#txt_files =[ "D:/NLP/PAN_TermSelection/2014TermSelection_Blog/outputNewVersion2014blogOR.1M.txt", "D:/NLP/PAN_TermSelection/2014TermSelection_Blog/outputNewVersion2014blogOR.2F.txt"]
#txt_files =[ "D:/NLP/PAN_TermSelection/2014TermSelection_Blog/outputNewVersion2014blogRF.1M.txt", "D:/NLP/PAN_TermSelection/2014TermSelection_Blog/outputNewVersion2014blogRF.2F.txt"]
#txt_files =[ "D:/NLP/PAN_TermSelection/2014TermSelection_Blog/outputNewVersion2014blogGSS.txt"]
print("txt_files of features used",txt_files)
for txt_file in txt_files:

    with open(txt_file, mode="r", encoding="utf-8") as f:

        word_list2 =  ([r.strip() for r in f])[0:200]
        word_list1 =  ([(r.split(":"))[1] for r in word_list2])

        print(txt_file, len(word_list1))

        word_list = word_list + word_list1

print("length of features used =", len(word_list))

for txt_file in txt_files:
    with open(txt_file, mode="r", encoding="utf-8") as f:
        reader = csv.reader(f, delimiter=",")
        next(reader) # skip header
        word_list1 =  [r[0] for r in reader]
        print(txt_file, len(word_list1))
        word_list = word_list + word_list1
print("length of features used =", len(word_list))


"""
Prepare the training and test sets to be parsed to the classifies............................................................
"""
A=(all_txt(M_train_file,  F_train_file))
all_training_text = ( [(A[i][1])  for i in range(len(A))])
print("len(all_training_text)",len(all_training_text))
X_train=WordFeatures(word_list, all_training_text)
y_train=np.array( [(A[i][0])  for i in range(len(A))])
print("len(X_train)=",len(X_train), "len(y_train)=", len(y_train))

B=(all_txt(M_test_file,  F_test_file))
all_test_text = ( [(B[i][1])  for i in range(len(B))])
print("len(all_test_text)",len(all_test_text))
X_test = WordFeatures(word_list, all_test_text)
y_test = np.array( [(B[i][0])  for i in range(len(B))])
print("len(X_test=",len(X_test), "len(y_test)=",len(y_test))


X_train = np.nan_to_num(X_train)
X_test = np.nan_to_num(X_test)

"""
Perform feature selection using chi2............................................................
"""
from sklearn.feature_selection import *
##
####def select_Bestfeatures(X, y):
##print("selector = SelectKBest(score_func=chi2, k=1000)")
##selector = SelectKBest(score_func=chi2, k=1000)
##fit = selector.fit(X_train,y_train)
##train_selected_features = fit.transform(X_train)
##test_selected_features = fit.transform(X_test)



"""
x_train and x_test based on the new features using chi2............................................................
"""
##X_train = select_Bestfeatures(X_train, y_train)
##X_test = select_Bestfeatures(X_test, y_test)


"""
Perform feature selection using f_classif............................................................
"""
from sklearn.feature_selection import *
##def f_classif_select_Bestfeatures(X, y):
##selector=SelectKBest(score_func=f_classif,k=500)
##fit = selector.fit(X_train,y_train)
##train_selected_features = fit.transform(X_train)
##test_selected_features = fit.transform(X_test)
##    selected_features = fit.transform(X)
##    return selected_features

"""
x_train and x_test based on the new features using f_classif............................................................
"""
##X_train = f_classif_select_Bestfeatures(X_train, y_train)
##X_test = f_classif_select_Bestfeatures(X_test, y_test)



"""
Perform feature selection using mutual_info_classif............................................................
"""
from sklearn.feature_selection import *
##def mutual_info_classif_select_Bestfeatures(X_tr, X_te, y):
##selector=SelectKBest(score_func=mutual_info_classif,k=100)
##fit = selector.fit(X_train,y_train)
##train_selected_features = fit.transform(X_train)
##test_selected_features = fit.transform(X_test)
##    return train_selected_features, test_selected_features

"""
x_train and x_test based on the new features using f_classif............................................................
"""
##X_train =train_selected_features 
##X_test = test_selected_features


"""
Perform feature selection using pca............................................................
"""

##from sklearn.decomposition import PCA
##def PCA_select_Bestfeatures(X):
##    # feature extraction
##    pca = PCA(n_components=200)
##    fit = pca.fit(X, y)
##    Train_selected_features = fit.transform(X)
##    return selected_features

"""
x_train and x_test based on the new features using f_classif............................................................
"""
##X_train = PCA_select_Bestfeatures(X_train)
##X_test = PCA_select_Bestfeatures(X_test)

"""
Below are a list of Classification Algorithms with different distance measres ..............................................................................
"""
"""
K-Nearest Neighbours (average aa over all the nearest neighbour selectiopn)..............................................................................
"""

k=5

print(k)
"""
8. manhattan K-Nearest Neighbours..............................................................................
"""
from sklearn.neighbors import KNeighborsClassifier
knn = KNeighborsClassifier(n_neighbors = k, metric= "manhattan")
knn.fit(X_train, y_train)
y_pred = knn.predict(X_test)
print("manhattan K-Nearest Neighbours")
truth_y_pred = [i for i, j in zip(y_pred, y_test) if  i == j]

print("len of truth_y_pre =", len(truth_y_pred))

false_y_pred = [i for i, j in zip(y_pred, y_test) if  i != j ]

print("len of false_y_pre =", len(false_y_pred))
print(metrics.accuracy_score(y_test, y_pred))

    
"""
8. cosine K-Nearest Neighbours..............................................................................
"""
from sklearn.neighbors import KNeighborsClassifier
knn = KNeighborsClassifier(n_neighbors = k, metric= "cosine")
knn.fit(X_train, y_train)
y_pred = knn.predict(X_test)
print("cosine K-Nearest Neighbours")
truth_y_pred = [i for i, j in zip(y_pred, y_test) if  i == j]

print("len of truth_y_pre =", len(truth_y_pred))

false_y_pred = [i for i, j in zip(y_pred, y_test) if  i != j ]

print("len of false_y_pre =", len(false_y_pred))
print(metrics.accuracy_score(y_test, y_pred))


"""
8. euclidean K-Nearest Neighbours..............................................................................
"""
from sklearn.neighbors import KNeighborsClassifier
knn = KNeighborsClassifier(n_neighbors = k, metric= "euclidean")
knn.fit(X_train, y_train)
y_pred = knn.predict(X_test)
print("euclidean K-Nearest Neighbours")
truth_y_pred = [i for i, j in zip(y_pred, y_test) if  i == j]

print("len of truth_y_pre =", len(truth_y_pred))

false_y_pred = [i for i, j in zip(y_pred, y_test) if  i != j ]

print("len of false_y_pre =", len(false_y_pred))
print(metrics.accuracy_score(y_test, y_pred))

        
"""
9. scipy.spatial.distance K-Nearest Neighbours..............................................................................
"""
from scipy.spatial.distance import *
knn = KNeighborsClassifier(n_neighbors = k, metric= "braycurtis")
knn.fit(X_train, y_train)
y_pred = knn.predict(X_test)
print("braycurtis K-Nearest Neighbours")
truth_y_pred = [i for i, j in zip(y_pred, y_test) if  i == j]

print("len of truth_y_pre =", len(truth_y_pred))

false_y_pred = [i for i, j in zip(y_pred, y_test) if  i != j ]

print("len of false_y_pre =", len(false_y_pred))
print(metrics.accuracy_score(y_test, y_pred))
       
        
knn = KNeighborsClassifier(n_neighbors = k, metric= "canberra")
knn.fit(X_train, y_train)
y_pred = knn.predict(X_test)
print("canberra K-Nearest Neighbours")
truth_y_pred = [i for i, j in zip(y_pred, y_test) if  i == j]

print("len of truth_y_pre =", len(truth_y_pred))

false_y_pred = [i for i, j in zip(y_pred, y_test) if  i != j ]

print("len of false_y_pre =", len(false_y_pred))
print(metrics.accuracy_score(y_test, y_pred))
  
        
knn = KNeighborsClassifier(n_neighbors = k, metric= "jaccard")
knn.fit(X_train, y_train)
y_pred = knn.predict(X_test)
print("jaccard K-Nearest Neighbours")
truth_y_pred = [i for i, j in zip(y_pred, y_test) if  i == j]

print("len of truth_y_pre =", len(truth_y_pred))

false_y_pred = [i for i, j in zip(y_pred, y_test) if  i != j ]

print("len of false_y_pre =", len(false_y_pred))
print(metrics.accuracy_score(y_test, y_pred))
        
knn = KNeighborsClassifier(n_neighbors = k, metric= "yule")
knn.fit(X_train, y_train)
y_pred = knn.predict(X_test)
print("yule K-Nearest Neighbours")
truth_y_pred = [i for i, j in zip(y_pred, y_test) if  i == j]

print("len of truth_y_pre =", len(truth_y_pred))

false_y_pred = [i for i, j in zip(y_pred, y_test) if  i != j ]

print("len of false_y_pre =", len(false_y_pred))
print(metrics.accuracy_score(y_test, y_pred))
print("************************************ ****************************")


/anaconda3/lib/python3.7/site-packages/nltk/twitter/__init__.py:20: UserWarning: The twython library has not been installed. Some functionality from the twitter package will not be available.
  warnings.warn("The twython library has not been installed. "


txt_files of features used ['/Users/catherine/Desktop/NLP/PAN Datasets/PAN2014blog/2014.1-blog-txtfiles/PAN2014_blog_female_chosen_set.1.csv', '/Users/catherine/Desktop/NLP/PAN Datasets/PAN2014blog/2014.1-blog-txtfiles/PAN2014_blog_male_chosen_set.1.csv']


FileNotFoundError: [Errno 2] No such file or directory: '/Users/catherine/Desktop/NLP/PAN Datasets/PAN2014blog/2014.1-blog-txtfiles/PAN2014_blog_female_chosen_set.1.csv'

In [3]:
import sys
sys.path.append("/anaconda3/lib/python3.7/site-packages")
import numpy as np
import numpy
import nltk
import pickle
import pandas as pd
import glob
import os
import re
import operator
from collections import Counter
#from read_xml_files import *
##from s_stemmer import *
from nltk.stem.porter import PorterStemmer
from nltk.stem import LancasterStemmer, WordNetLemmatizer
from itertools import groupby
from nltk.collocations import *
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.cluster import KMeans
from scipy.cluster.vq import whiten
from nltk.tokenize.toktok import ToktokTokenizer
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2
import xml.etree.ElementTree as ET
import pandas as pd
#Import scikit-learn metrics module for accuracy calculation
from sklearn import metrics
##from pattern.text.en import singularize
##nltk.download('vader_lexicon')
##nltk.download('punkt')
#import the relevant modules from the NLTK library
from nltk.sentiment.vader import SentimentIntensityAnalyzer
sentence_tokenizer = nltk.data.load('tokenizers/punkt/english.pickle')
word_tokenizer = nltk.tokenize.RegexpTokenizer(r'\w+')
##tt = ToktokTokenizer()#keeps the url as is...................................................
from nltk.tokenize import TweetTokenizer
tt = TweetTokenizer()
Porter_stemmer = PorterStemmer()
Lancaster_stemmer = LancasterStemmer()
WordNet_lemmatizer = WordNetLemmatizer()
#nltk.download('averaged_perceptron_tagger')
##Train_path="D:/Data/PAN2015-Profiling/pan15-author-profiling-training-dataset-english-2015-04-23"#path for train data
##Test_path="D:/Data/PAN2015-Profiling/pan-ap2015-test/en" #path for test data

##Train_path="D:/NLP/PAN2014/pan14-author-profiling-training-corpus-english-twitter-2014-04-16"#path for train data D:\Data\PAN2013-Profiling\Training\en  D:\NLP\PAN2014
##train_truth_path="D:/NLP/PAN2014/truth_train"
##
##test_truth_path="D:/NLP/PAN2014/pan14_test"
##Test_path="D:/NLP/PAN2014/pan14_test/en"
##Content = open('results_of_clssification_all.txt', 'w')

M_train_file= "/Users/catherine/Desktop/NLP/PAN_Datasets/PAN2014blog/2014.1-blog-txtfiles/PAN2014_blog_male_train.1.txt"
F_train_file= "/Users/catherine/Desktop/NLP/PAN_Datasets/PAN2014blog/2014.1-blog-txtfiles/PAN2014_blog_female_train.1.txt"

M_test_file= "/Users/catherine/Desktop/NLP/PAN_Datasets/PAN2014blog/2014.1-blog-txtfiles/PAN2014_blog_male_test.1.txt"
F_test_file= "/Users/catherine/Desktop/NLP/PAN_Datasets/PAN2014blog/2014.1-blog-txtfiles/PAN2014_blog_female_test.1.txt"

def all_txt(M_txt_file,  F_txt_file):
    with open(M_txt_file, mode="r", encoding="utf-8") as f:
            txt_list = [(line[:-1]).split("\t") for line in f]
            male_dict = dict()
            content=[]
            for line in txt_list:
                if line[1] in male_dict:
                # append the new number to the existing array at this slot
                    male_dict[line[1]].append(line[3])
                else:
                # create a new array in this slot
                    male_dict[line[1]] = [line[3]]
    male_content= [("M", (' '.join(a)).split()) for a in list(male_dict.values())]
  
    with open(F_txt_file, mode="r", encoding="utf-8") as f:
            txt_list = [(line[:-1]).split("\t") for line in f]
            female_dict = dict()
            content=[]
            for line in txt_list:
                if line[1] in female_dict:
                # append the new number to the existing array at this slot
                    female_dict[line[1]].append(line[3])
                else:
                # create a new array in this slot
                    female_dict[line[1]] = [line[3]]
    female_content= [("F", (' '.join(a)).split()) for a in list(female_dict.values())]
    all_txt_per_person = male_content + female_content
    return all_txt_per_person

##print( [((all_txt(M_txt_file,  F_txt_file))[i][0])  for i in range(len((all_txt(M_txt_file,  F_txt_file))))]) helps to determine the gender....

      
"""
create feature vector from the training set.....................................................................................
"""

def WordFeatures(word_list, all_training_text):

    fvs_words = np.array([[author.count(word) for word in word_list] for author in all_training_text]).astype(np.float64)

    # normalise by dividing each row by number of tokens for each author........
    fvs_words /= np.c_[np.array([len(author) for author in all_training_text])]

    return fvs_words



"""
From the saved csv file, recover the saved features to be used...............................................................
"""
import csv
word_list=[]
##txt_files =[ "/Users/catherine/Desktop/NLP/PAN Datasets/PAN_chosen_tokens/2014/output2014OR.1M.txt", "/Users/catherine/Desktop/NLP/PAN Datasets/PAN_chosen_tokens/2014/output2014OR.2F.txt"]
txt_files =[ "/Users/catherine/Desktop/NLP/PAN_Datasets/PAN2014blog/2014.1-blog-txtfiles/PAN2014_blog_female_vocubulary.1.csv", "/Users/catherine/Desktop/NLP/PAN_Datasets/PAN2014blog/2014.1-blog-txtfiles/PAN2014_blog_male_ vocubulary_set.1.csv"]
#txt_files =[ "D:/NLP/PAN_TermSelection/2014TermSelection_Blog/outputNewVersion2014blogOR.1M.txt", "D:/NLP/PAN_TermSelection/2014TermSelection_Blog/outputNewVersion2014blogOR.2F.txt"]
#txt_files =[ "D:/NLP/PAN_TermSelection/2014TermSelection_Blog/outputNewVersion2014blogRF.1M.txt", "D:/NLP/PAN_TermSelection/2014TermSelection_Blog/outputNewVersion2014blogRF.2F.txt"]
#txt_files =[ "D:/NLP/PAN_TermSelection/2014TermSelection_Blog/outputNewVersion2014blogGSS.txt"]
print("txt_files of features used",txt_files)
#for txt_file in txt_files:

#    with open(txt_file, mode="r", encoding="utf-8") as f:

#        word_list2 =  ([r.strip() for r in f])[0:200]
#        word_list1 =  ([(r.split(":"))[1] for r in word_list2])

#        print(txt_file, len(word_list1))

#        word_list = word_list + word_list1

#print("length of features used =", len(word_list))

for txt_file in txt_files:
    with open(txt_file, mode="r", encoding="utf-8") as f:
        reader = csv.reader(f, delimiter=",")
        next(reader) # skip header
        word_list1 =  [r[0] for r in reader]
        print(txt_file, len(word_list1))
        word_list = word_list + word_list1
print("length of features used =", len(word_list))


"""
Prepare the training and test sets to be parsed to the classifies............................................................
"""
A=(all_txt(M_train_file,  F_train_file))
all_training_text = ( [(A[i][1])  for i in range(len(A))])
print("len(all_training_text)",len(all_training_text))
X_train=WordFeatures(word_list, all_training_text)
y_train=np.array( [(A[i][0])  for i in range(len(A))])
print("len(X_train)=",len(X_train), "len(y_train)=", len(y_train))

B=(all_txt(M_test_file,  F_test_file))
all_test_text = ( [(B[i][1])  for i in range(len(B))])
print("len(all_test_text)",len(all_test_text))
X_test = WordFeatures(word_list, all_test_text)
y_test = np.array( [(B[i][0])  for i in range(len(B))])
print("len(X_test=",len(X_test), "len(y_test)=",len(y_test))


X_train = np.nan_to_num(X_train)
X_test = np.nan_to_num(X_test)

"""
Perform feature selection using chi2............................................................
"""
from sklearn.feature_selection import *
##
####def select_Bestfeatures(X, y):
##print("selector = SelectKBest(score_func=chi2, k=1000)")
##selector = SelectKBest(score_func=chi2, k=1000)
##fit = selector.fit(X_train,y_train)
##train_selected_features = fit.transform(X_train)
##test_selected_features = fit.transform(X_test)



"""
x_train and x_test based on the new features using chi2............................................................
"""
##X_train = select_Bestfeatures(X_train, y_train)
##X_test = select_Bestfeatures(X_test, y_test)


"""
Perform feature selection using f_classif............................................................
"""
from sklearn.feature_selection import *
##def f_classif_select_Bestfeatures(X, y):
##selector=SelectKBest(score_func=f_classif,k=500)
##fit = selector.fit(X_train,y_train)
##train_selected_features = fit.transform(X_train)
##test_selected_features = fit.transform(X_test)
##    selected_features = fit.transform(X)
##    return selected_features

"""
x_train and x_test based on the new features using f_classif............................................................
"""
##X_train = f_classif_select_Bestfeatures(X_train, y_train)
##X_test = f_classif_select_Bestfeatures(X_test, y_test)



"""
Perform feature selection using mutual_info_classif............................................................
"""
from sklearn.feature_selection import *
##def mutual_info_classif_select_Bestfeatures(X_tr, X_te, y):
##selector=SelectKBest(score_func=mutual_info_classif,k=100)
##fit = selector.fit(X_train,y_train)
##train_selected_features = fit.transform(X_train)
##test_selected_features = fit.transform(X_test)
##    return train_selected_features, test_selected_features

"""
x_train and x_test based on the new features using f_classif............................................................
"""
##X_train =train_selected_features 
##X_test = test_selected_features


"""
Perform feature selection using pca............................................................
"""

##from sklearn.decomposition import PCA
##def PCA_select_Bestfeatures(X):
##    # feature extraction
##    pca = PCA(n_components=200)
##    fit = pca.fit(X, y)
##    Train_selected_features = fit.transform(X)
##    return selected_features

"""
x_train and x_test based on the new features using f_classif............................................................
"""
##X_train = PCA_select_Bestfeatures(X_train)
##X_test = PCA_select_Bestfeatures(X_test)

"""
Below are a list of Classification Algorithms with different distance measres ..............................................................................
"""
"""
K-Nearest Neighbours (average aa over all the nearest neighbour selectiopn)..............................................................................
"""

k=5

print(k)
"""
8. manhattan K-Nearest Neighbours..............................................................................
"""
from sklearn.neighbors import KNeighborsClassifier
knn = KNeighborsClassifier(n_neighbors = k, metric= "manhattan")
knn.fit(X_train, y_train)
y_pred = knn.predict(X_test)
print("manhattan K-Nearest Neighbours")
print([(i,j) for i, j in zip(y_pred, y_test)])
print(metrics.accuracy_score(y_test, y_pred))

    
"""
8. cosine K-Nearest Neighbours..............................................................................
"""
from sklearn.neighbors import KNeighborsClassifier
knn = KNeighborsClassifier(n_neighbors = k, metric= "cosine")
knn.fit(X_train, y_train)
y_pred = knn.predict(X_test)
print("cosine K-Nearest Neighbours")
print([(i,j) for i, j in zip(y_pred, y_test)])
print(metrics.accuracy_score(y_test, y_pred))


"""
8. euclidean K-Nearest Neighbours..............................................................................
"""
from sklearn.neighbors import KNeighborsClassifier
knn = KNeighborsClassifier(n_neighbors = k, metric= "euclidean")
knn.fit(X_train, y_train)
y_pred = knn.predict(X_test)
print("euclidean K-Nearest Neighbours")
print([(i,j) for i, j in zip(y_pred, y_test)])
print(metrics.accuracy_score(y_test, y_pred))

        
"""
9. scipy.spatial.distance K-Nearest Neighbours..............................................................................
"""
from scipy.spatial.distance import *
knn = KNeighborsClassifier(n_neighbors = k, metric= "braycurtis")
knn.fit(X_train, y_train)
y_pred = knn.predict(X_test)
print("braycurtis K-Nearest Neighbours")
print([(i,j) for i, j in zip(y_pred, y_test)])
print(metrics.accuracy_score(y_test, y_pred))
       
        
knn = KNeighborsClassifier(n_neighbors = k, metric= "canberra")
knn.fit(X_train, y_train)
y_pred = knn.predict(X_test)
print("canberra K-Nearest Neighbours")
print([(i,j) for i, j in zip(y_pred, y_test)])
print(metrics.accuracy_score(y_test, y_pred))
  
        
knn = KNeighborsClassifier(n_neighbors = k, metric= "jaccard")
knn.fit(X_train, y_train)
y_pred = knn.predict(X_test)
print("jaccard K-Nearest Neighbours")
print([(i,j) for i, j in zip(y_pred, y_test)])
print(metrics.accuracy_score(y_test, y_pred))
        
knn = KNeighborsClassifier(n_neighbors = k, metric= "yule")
knn.fit(X_train, y_train)
y_pred = knn.predict(X_test)
print("yule K-Nearest Neighbours")
print([(i,j) for i, j in zip(y_pred, y_test)])
print(metrics.accuracy_score(y_test, y_pred))
print("************************************ ****************************")



txt_files of features used ['/Users/catherine/Desktop/NLP/PAN_Datasets/PAN2014blog/2014.1-blog-txtfiles/PAN2014_blog_female_vocubulary.1.csv', '/Users/catherine/Desktop/NLP/PAN_Datasets/PAN2014blog/2014.1-blog-txtfiles/PAN2014_blog_male_ vocubulary_set.1.csv']
/Users/catherine/Desktop/NLP/PAN_Datasets/PAN2014blog/2014.1-blog-txtfiles/PAN2014_blog_female_vocubulary.1.csv 12375
/Users/catherine/Desktop/NLP/PAN_Datasets/PAN2014blog/2014.1-blog-txtfiles/PAN2014_blog_male_ vocubulary_set.1.csv 18889
length of features used = 31264
len(all_training_text) 147
len(X_train)= 147 len(y_train)= 147
len(all_test_text) 78
len(X_test= 78 len(y_test)= 78
5
manhattan K-Nearest Neighbours
[('F', 'M'), ('F', 'M'), ('M', 'M'), ('M', 'M'), ('M', 'M'), ('F', 'M'), ('F', 'M'), ('F', 'M'), ('M', 'M'), ('F', 'M'), ('F', 'M'), ('F', 'M'), ('M', 'M'), ('M', 'M'), ('M', 'M'), ('M', 'M'), ('M', 'M'), ('M', 'M'), ('M', 'M'), ('M', 'M'), ('F', 'M'), ('M', 'M'), ('M', 'M'), ('M', 'M'), ('M', 'M'), ('F', 'M'), ('M', 

/anaconda3/lib/python3.7/site-packages/sklearn/utils/validation.py:475: DataConversionWarning: Data with input dtype float64 was converted to bool by check_pairwise_arrays.
  warnings.warn(msg, DataConversionWarning)


yule K-Nearest Neighbours
[('F', 'M'), ('F', 'M'), ('M', 'M'), ('M', 'M'), ('M', 'M'), ('F', 'M'), ('M', 'M'), ('M', 'M'), ('M', 'M'), ('M', 'M'), ('M', 'M'), ('M', 'M'), ('F', 'M'), ('M', 'M'), ('M', 'M'), ('F', 'M'), ('M', 'M'), ('F', 'M'), ('F', 'M'), ('F', 'M'), ('F', 'M'), ('M', 'M'), ('F', 'M'), ('M', 'M'), ('M', 'M'), ('F', 'M'), ('M', 'M'), ('F', 'M'), ('M', 'M'), ('M', 'M'), ('F', 'M'), ('M', 'M'), ('F', 'M'), ('M', 'M'), ('F', 'M'), ('M', 'M'), ('F', 'M'), ('M', 'M'), ('M', 'M'), ('M', 'F'), ('F', 'F'), ('M', 'F'), ('F', 'F'), ('M', 'F'), ('F', 'F'), ('M', 'F'), ('M', 'F'), ('F', 'F'), ('F', 'F'), ('F', 'F'), ('M', 'F'), ('M', 'F'), ('F', 'F'), ('F', 'F'), ('F', 'F'), ('M', 'F'), ('F', 'F'), ('F', 'F'), ('F', 'F'), ('F', 'F'), ('F', 'F'), ('F', 'F'), ('F', 'F'), ('M', 'F'), ('M', 'F'), ('F', 'F'), ('F', 'F'), ('M', 'F'), ('M', 'F'), ('F', 'F'), ('F', 'F'), ('M', 'F'), ('M', 'F'), ('F', 'F'), ('M', 'F'), ('F', 'F'), ('F', 'F'), ('M', 'F')]
0.5897435897435898
******************

In [5]:
import sys
sys.path.append("/anaconda3/lib/python3.7/site-packages")
import numpy as np
import numpy
import nltk
import pickle
import pandas as pd
import glob
import os
import re
import operator
from collections import Counter
#from read_xml_files import *
##from s_stemmer import *
from nltk.stem.porter import PorterStemmer
from nltk.stem import LancasterStemmer, WordNetLemmatizer
from itertools import groupby
from nltk.collocations import *
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.cluster import KMeans
from scipy.cluster.vq import whiten
from nltk.tokenize.toktok import ToktokTokenizer
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2
import xml.etree.ElementTree as ET
import pandas as pd
#Import scikit-learn metrics module for accuracy calculation
from sklearn import metrics
##from pattern.text.en import singularize
##nltk.download('vader_lexicon')
##nltk.download('punkt')
#import the relevant modules from the NLTK library
from nltk.sentiment.vader import SentimentIntensityAnalyzer
sentence_tokenizer = nltk.data.load('tokenizers/punkt/english.pickle')
word_tokenizer = nltk.tokenize.RegexpTokenizer(r'\w+')
##tt = ToktokTokenizer()#keeps the url as is...................................................
from nltk.tokenize import TweetTokenizer
tt = TweetTokenizer()
Porter_stemmer = PorterStemmer()
Lancaster_stemmer = LancasterStemmer()
WordNet_lemmatizer = WordNetLemmatizer()
#nltk.download('averaged_perceptron_tagger')
##Train_path="D:/Data/PAN2015-Profiling/pan15-author-profiling-training-dataset-english-2015-04-23"#path for train data
##Test_path="D:/Data/PAN2015-Profiling/pan-ap2015-test/en" #path for test data

##Train_path="D:/NLP/PAN2014/pan14-author-profiling-training-corpus-english-twitter-2014-04-16"#path for train data D:\Data\PAN2013-Profiling\Training\en  D:\NLP\PAN2014
##train_truth_path="D:/NLP/PAN2014/truth_train"
##
##test_truth_path="D:/NLP/PAN2014/pan14_test"
##Test_path="D:/NLP/PAN2014/pan14_test/en"
##Content = open('results_of_clssification_all.txt', 'w')

M_train_file= "/Users/catherine/Desktop/NLP/PAN_Datasets/PAN2014blog/2014.1-blog-txtfiles/PAN2014_blog_male_train.1.txt"
F_train_file= "/Users/catherine/Desktop/NLP/PAN_Datasets/PAN2014blog/2014.1-blog-txtfiles/PAN2014_blog_female_train.1.txt"

M_test_file= "/Users/catherine/Desktop/NLP/PAN_Datasets/PAN2014blog/2014.1-blog-txtfiles/PAN2014_blog_male_test.1.txt"
F_test_file= "/Users/catherine/Desktop/NLP/PAN_Datasets/PAN2014blog/2014.1-blog-txtfiles/PAN2014_blog_female_test.1.txt"

def all_txt(M_txt_file,  F_txt_file):
    with open(M_txt_file, mode="r", encoding="utf-8") as f:
            txt_list = [(line[:-1]).split("\t") for line in f]
            male_dict = dict()
            content=[]
            for line in txt_list:
                if line[1] in male_dict:
                # append the new number to the existing array at this slot
                    male_dict[line[1]].append(line[3])
                else:
                # create a new array in this slot
                    male_dict[line[1]] = [line[3]]
    male_content= [("M", (' '.join(a)).split()) for a in list(male_dict.values())]
  
    with open(F_txt_file, mode="r", encoding="utf-8") as f:
            txt_list = [(line[:-1]).split("\t") for line in f]
            female_dict = dict()
            content=[]
            for line in txt_list:
                if line[1] in female_dict:
                # append the new number to the existing array at this slot
                    female_dict[line[1]].append(line[3])
                else:
                # create a new array in this slot
                    female_dict[line[1]] = [line[3]]
    female_content= [("F", (' '.join(a)).split()) for a in list(female_dict.values())]
    all_txt_per_person = male_content + female_content
    return all_txt_per_person

##print( [((all_txt(M_txt_file,  F_txt_file))[i][0])  for i in range(len((all_txt(M_txt_file,  F_txt_file))))]) helps to determine the gender....

      
"""
create feature vector from the training set.....................................................................................
"""

def WordFeatures(word_list, all_training_text):

    fvs_words = np.array([[author.count(word) for word in word_list] for author in all_training_text]).astype(np.float64)

    # normalise by dividing each row by number of tokens for each author........
    fvs_words /= np.c_[np.array([len(author) for author in all_training_text])]

    return fvs_words



"""
From the saved csv file, recover the saved features to be used...............................................................
"""
import csv
word_list=[]
##txt_files =[ "/Users/catherine/Desktop/NLP/PAN Datasets/PAN_chosen_tokens/2014/output2014OR.1M.txt", "/Users/catherine/Desktop/NLP/PAN Datasets/PAN_chosen_tokens/2014/output2014OR.2F.txt"]
txt_files =[ "//Users/catherine/Desktop/NLP/PAN_Datasets/PAN2014blog/2014.1-blog-txtfiles/PAN2014_blog_female_chosen_set.1.csv", "/Users/catherine/Desktop/NLP/PAN_Datasets/PAN2014blog/2014.1-blog-txtfiles/PAN2014_blog_male_chosen_set.1.csv"]
print("txt_files of features used",txt_files)
##for txt_file in txt_files:
##    with open(txt_file, mode="r", encoding="utf-8") as f:
##        reader = csv.reader(f, delimiter=",")
##        next(reader) # skip header
##        word_list1 =  ([(r.split(":"))[1] for r in f])
##        print(txt_file, len(word_list1))
##        word_list = word_list + word_list1
##print("length of features used =", len(word_list))

for txt_file in txt_files:
    with open(txt_file, mode="r", encoding="utf-8") as f:
        reader = csv.reader(f, delimiter=",")
        next(reader) # skip header
        word_list1 =  [r[0] for r in reader]
        print(txt_file, len(word_list1))
        word_list = word_list + word_list1
print("length of features used =", len(word_list))


"""
Prepare the training and test sets to be parsed to the classifies............................................................
"""
A=(all_txt(M_train_file,  F_train_file))
all_training_text = ( [(A[i][1])  for i in range(len(A))])
print("len(all_training_text)",len(all_training_text))
X_train=WordFeatures(word_list, all_training_text)
y_train=np.array( [(A[i][0])  for i in range(len(A))])
print("len(X_train)=",len(X_train), "len(y_train)=", len(y_train))

B=(all_txt(M_test_file,  F_test_file))
all_test_text = ( [(B[i][1])  for i in range(len(B))])
print("len(all_test_text)",len(all_test_text))
X_test = WordFeatures(word_list, all_test_text)
y_test = np.array( [(B[i][0])  for i in range(len(B))])
print("len(X_test=",len(X_test), "len(y_test)=",len(y_test))


X_train = np.nan_to_num(X_train)
X_test = np.nan_to_num(X_test)

"""
Perform feature selection using chi2,f_classif,mutual_info_classif............................................................
"""
from sklearn.feature_selection import *
##
####def select_Bestfeatures(X, y):
print("selector = SelectKBest(score_func=chi2, k=200)")
selector = SelectKBest(score_func=chi2, k=200)
fit = selector.fit(X_train,y_train)
train_selected_features = fit.transform(X_train)
test_selected_features = fit.transform(X_test)



"""
x_train and x_test based on the new features using chi2............................................................
"""
##X_train = select_Bestfeatures(X_train, y_train)
##X_test = select_Bestfeatures(X_test, y_test)


"""
Perform feature selection using f_classif,mutual_info_classif............................................................
"""
from sklearn.feature_selection import *
##def f_classif_select_Bestfeatures(X, y):
##selector=SelectKBest(score_func=f_classif,k=500)
##fit = selector.fit(X_train,y_train)
##train_selected_features = fit.transform(X_train)
##test_selected_features = fit.transform(X_test)
##    selected_features = fit.transform(X)
##    return selected_features

"""
x_train and x_test based on the new features using f_classif............................................................
"""
##X_train = f_classif_select_Bestfeatures(X_train, y_train)
##X_test = f_classif_select_Bestfeatures(X_test, y_test)



"""
Perform feature selection using mutual_info_classif............................................................
"""
from sklearn.feature_selection import *
##def mutual_info_classif_select_Bestfeatures(X_tr, X_te, y):
##selector=SelectKBest(score_func=mutual_info_classif,k=100)
##fit = selector.fit(X_train,y_train)
##train_selected_features = fit.transform(X_train)
##test_selected_features = fit.transform(X_test)
##    return train_selected_features, test_selected_features

"""
x_train and x_test based on the new features using f_classif............................................................
"""
X_train =train_selected_features 
X_test = test_selected_features


"""
Perform feature selection using pca............................................................
"""

##from sklearn.decomposition import PCA
##def PCA_select_Bestfeatures(X):
##    # feature extraction
##    pca = PCA(n_components=200)
##    fit = pca.fit(X, y)
##    Train_selected_features = fit.transform(X)
##    return selected_features

"""
x_train and x_test based on the new features using f_classif............................................................
"""
##X_train = PCA_select_Bestfeatures(X_train)
##X_test = PCA_select_Bestfeatures(X_test)

"""
Below are a list of Classification Algorithms with different distance measres ..............................................................................
"""
"""
K-Nearest Neighbours (average aa over all the nearest neighbour selectiopn)..............................................................................
"""

k=5

print(k)

from sklearn.neighbors import KNeighborsClassifier
      
"""
9. scipy.spatial.distance K-Nearest Neighbours..............................................................................
"""
from scipy.spatial.distance import *
knn = KNeighborsClassifier(n_neighbors = k, metric= "braycurtis")
knn.fit(X_train, y_train)
y_pred = knn.predict(X_test)
print("braycurtis K-Nearest Neighbours")
print([(i,j) for i, j in zip(y_pred, y_test)])
print(metrics.accuracy_score(y_test, y_pred))
       
        
print("************************************ ****************************")

txt_files of features used ['//Users/catherine/Desktop/NLP/PAN_Datasets/PAN2014blog/2014.1-blog-txtfiles/PAN2014_blog_female_chosen_set.1.csv', '/Users/catherine/Desktop/NLP/PAN_Datasets/PAN2014blog/2014.1-blog-txtfiles/PAN2014_blog_male_chosen_set.1.csv']
//Users/catherine/Desktop/NLP/PAN_Datasets/PAN2014blog/2014.1-blog-txtfiles/PAN2014_blog_female_chosen_set.1.csv 1526
/Users/catherine/Desktop/NLP/PAN_Datasets/PAN2014blog/2014.1-blog-txtfiles/PAN2014_blog_male_chosen_set.1.csv 2063
length of features used = 3589
len(all_training_text) 147
len(X_train)= 147 len(y_train)= 147
len(all_test_text) 78
len(X_test= 78 len(y_test)= 78
selector = SelectKBest(score_func=chi2, k=200)
5
braycurtis K-Nearest Neighbours
[('M', 'M'), ('F', 'M'), ('M', 'M'), ('M', 'M'), ('M', 'M'), ('F', 'M'), ('M', 'M'), ('F', 'M'), ('F', 'M'), ('M', 'M'), ('F', 'M'), ('F', 'M'), ('F', 'M'), ('M', 'M'), ('F', 'M'), ('F', 'M'), ('M', 'M'), ('F', 'M'), ('M', 'M'), ('M', 'M'), ('F', 'M'), ('M', 'M'), ('F', 'M'), ('M',

In [6]:
import sys
sys.path.append("/anaconda3/lib/python3.7/site-packages")
import numpy as np
import numpy
import nltk
import pickle
import pandas as pd
import glob
import os
import re
import operator
from collections import Counter
#from read_xml_files import *
##from s_stemmer import *
from nltk.stem.porter import PorterStemmer
from nltk.stem import LancasterStemmer, WordNetLemmatizer
from itertools import groupby
from nltk.collocations import *
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.cluster import KMeans
from scipy.cluster.vq import whiten
from nltk.tokenize.toktok import ToktokTokenizer
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2
import xml.etree.ElementTree as ET
import pandas as pd
#Import scikit-learn metrics module for accuracy calculation
from sklearn import metrics
##from pattern.text.en import singularize
##nltk.download('vader_lexicon')
##nltk.download('punkt')
#import the relevant modules from the NLTK library
from nltk.sentiment.vader import SentimentIntensityAnalyzer
sentence_tokenizer = nltk.data.load('tokenizers/punkt/english.pickle')
word_tokenizer = nltk.tokenize.RegexpTokenizer(r'\w+')
##tt = ToktokTokenizer()#keeps the url as is...................................................
from nltk.tokenize import TweetTokenizer
tt = TweetTokenizer()
Porter_stemmer = PorterStemmer()
Lancaster_stemmer = LancasterStemmer()
WordNet_lemmatizer = WordNetLemmatizer()
#nltk.download('averaged_perceptron_tagger')
##Train_path="D:/Data/PAN2015-Profiling/pan15-author-profiling-training-dataset-english-2015-04-23"#path for train data
##Test_path="D:/Data/PAN2015-Profiling/pan-ap2015-test/en" #path for test data

##Train_path="D:/NLP/PAN2014/pan14-author-profiling-training-corpus-english-twitter-2014-04-16"#path for train data D:\Data\PAN2013-Profiling\Training\en  D:\NLP\PAN2014
##train_truth_path="D:/NLP/PAN2014/truth_train"
##
##test_truth_path="D:/NLP/PAN2014/pan14_test"
##Test_path="D:/NLP/PAN2014/pan14_test/en"
##Content = open('results_of_clssification_all.txt', 'w')

M_train_file= "/Users/catherine/Desktop/NLP/PAN_Datasets/PAN2014blog/2014.1-blog-txtfiles/PAN2014_blog_male_train.1.txt"
F_train_file= "/Users/catherine/Desktop/NLP/PAN_Datasets/PAN2014blog/2014.1-blog-txtfiles/PAN2014_blog_female_train.1.txt"

M_test_file= "/Users/catherine/Desktop/NLP/PAN_Datasets/PAN2014blog/2014.1-blog-txtfiles/PAN2014_blog_male_test.1.txt"
F_test_file= "/Users/catherine/Desktop/NLP/PAN_Datasets/PAN2014blog/2014.1-blog-txtfiles/PAN2014_blog_female_test.1.txt"

def all_txt(M_txt_file,  F_txt_file):
    with open(M_txt_file, mode="r", encoding="utf-8") as f:
            txt_list = [(line[:-1]).split("\t") for line in f]
            male_dict = dict()
            content=[]
            for line in txt_list:
                if line[1] in male_dict:
                # append the new number to the existing array at this slot
                    male_dict[line[1]].append(line[3])
                else:
                # create a new array in this slot
                    male_dict[line[1]] = [line[3]]
    male_content= [("M", (' '.join(a)).split()) for a in list(male_dict.values())]
  
    with open(F_txt_file, mode="r", encoding="utf-8") as f:
            txt_list = [(line[:-1]).split("\t") for line in f]
            female_dict = dict()
            content=[]
            for line in txt_list:
                if line[1] in female_dict:
                # append the new number to the existing array at this slot
                    female_dict[line[1]].append(line[3])
                else:
                # create a new array in this slot
                    female_dict[line[1]] = [line[3]]
    female_content= [("F", (' '.join(a)).split()) for a in list(female_dict.values())]
    all_txt_per_person = male_content + female_content
    return all_txt_per_person

##print( [((all_txt(M_txt_file,  F_txt_file))[i][0])  for i in range(len((all_txt(M_txt_file,  F_txt_file))))]) helps to determine the gender....

      
"""
create feature vector from the training set.....................................................................................
"""

def WordFeatures(word_list, all_training_text):

    fvs_words = np.array([[author.count(word) for word in word_list] for author in all_training_text]).astype(np.float64)

    # normalise by dividing each row by number of tokens for each author........
    fvs_words /= np.c_[np.array([len(author) for author in all_training_text])]

    return fvs_words



"""
From the saved csv file, recover the saved features to be used...............................................................
"""
import csv
word_list=[]
##txt_files =[ "/Users/catherine/Desktop/NLP/PAN Datasets/PAN_chosen_tokens/2014/output2014OR.1M.txt", "/Users/catherine/Desktop/NLP/PAN Datasets/PAN_chosen_tokens/2014/output2014OR.2F.txt"]
txt_files =[ "//Users/catherine/Desktop/NLP/PAN_Datasets/PAN2014blog/2014.1-blog-txtfiles/PAN2014_blog_female_chosen_set.1.csv", "/Users/catherine/Desktop/NLP/PAN_Datasets/PAN2014blog/2014.1-blog-txtfiles/PAN2014_blog_male_chosen_set.1.csv"]
print("txt_files of features used",txt_files)
##for txt_file in txt_files:
##    with open(txt_file, mode="r", encoding="utf-8") as f:
##        reader = csv.reader(f, delimiter=",")
##        next(reader) # skip header
##        word_list1 =  ([(r.split(":"))[1] for r in f])
##        print(txt_file, len(word_list1))
##        word_list = word_list + word_list1
##print("length of features used =", len(word_list))

for txt_file in txt_files:
    with open(txt_file, mode="r", encoding="utf-8") as f:
        reader = csv.reader(f, delimiter=",")
        next(reader) # skip header
        word_list1 =  [r[0] for r in reader]
        print(txt_file, len(word_list1))
        word_list = word_list + word_list1
print("length of features used =", len(word_list))


"""
Prepare the training and test sets to be parsed to the classifies............................................................
"""
A=(all_txt(M_train_file,  F_train_file))
all_training_text = ( [(A[i][1])  for i in range(len(A))])
print("len(all_training_text)",len(all_training_text))
X_train=WordFeatures(word_list, all_training_text)
y_train=np.array( [(A[i][0])  for i in range(len(A))])
print("len(X_train)=",len(X_train), "len(y_train)=", len(y_train))

B=(all_txt(M_test_file,  F_test_file))
all_test_text = ( [(B[i][1])  for i in range(len(B))])
print("len(all_test_text)",len(all_test_text))
X_test = WordFeatures(word_list, all_test_text)
y_test = np.array( [(B[i][0])  for i in range(len(B))])
print("len(X_test=",len(X_test), "len(y_test)=",len(y_test))


X_train = np.nan_to_num(X_train)
X_test = np.nan_to_num(X_test)

"""
Perform feature selection using chi2,f_classif,mutual_info_classif............................................................
"""
from sklearn.feature_selection import *
##
####def select_Bestfeatures(X, y):
print("selector = SelectKBest(score_func=f_classif, k=200)")
selector = SelectKBest(score_func=f_classif, k=200)
fit = selector.fit(X_train,y_train)
train_selected_features = fit.transform(X_train)
test_selected_features = fit.transform(X_test)



"""
x_train and x_test based on the new features using chi2............................................................
"""
##X_train = select_Bestfeatures(X_train, y_train)
##X_test = select_Bestfeatures(X_test, y_test)


"""
Perform feature selection using f_classif,mutual_info_classif............................................................
"""
from sklearn.feature_selection import *
##def f_classif_select_Bestfeatures(X, y):
##selector=SelectKBest(score_func=f_classif,k=500)
##fit = selector.fit(X_train,y_train)
##train_selected_features = fit.transform(X_train)
##test_selected_features = fit.transform(X_test)
##    selected_features = fit.transform(X)
##    return selected_features

"""
x_train and x_test based on the new features using f_classif............................................................
"""
##X_train = f_classif_select_Bestfeatures(X_train, y_train)
##X_test = f_classif_select_Bestfeatures(X_test, y_test)



"""
Perform feature selection using mutual_info_classif............................................................
"""
from sklearn.feature_selection import *
##def mutual_info_classif_select_Bestfeatures(X_tr, X_te, y):
##selector=SelectKBest(score_func=mutual_info_classif,k=100)
##fit = selector.fit(X_train,y_train)
##train_selected_features = fit.transform(X_train)
##test_selected_features = fit.transform(X_test)
##    return train_selected_features, test_selected_features

"""
x_train and x_test based on the new features using f_classif............................................................
"""
X_train =train_selected_features 
X_test = test_selected_features


"""
Perform feature selection using pca............................................................
"""

##from sklearn.decomposition import PCA
##def PCA_select_Bestfeatures(X):
##    # feature extraction
##    pca = PCA(n_components=200)
##    fit = pca.fit(X, y)
##    Train_selected_features = fit.transform(X)
##    return selected_features

"""
x_train and x_test based on the new features using f_classif............................................................
"""
##X_train = PCA_select_Bestfeatures(X_train)
##X_test = PCA_select_Bestfeatures(X_test)

"""
Below are a list of Classification Algorithms with different distance measres ..............................................................................
"""
"""
K-Nearest Neighbours (average aa over all the nearest neighbour selectiopn)..............................................................................
"""

k=5

print(k)

from sklearn.neighbors import KNeighborsClassifier
      
"""
9. scipy.spatial.distance K-Nearest Neighbours..............................................................................
"""
from scipy.spatial.distance import *
knn = KNeighborsClassifier(n_neighbors = k, metric= "braycurtis")
knn.fit(X_train, y_train)
y_pred = knn.predict(X_test)
print("braycurtis K-Nearest Neighbours")
print([(i,j) for i, j in zip(y_pred, y_test)])
print(metrics.accuracy_score(y_test, y_pred))
       
        
print("************************************ ****************************")

txt_files of features used ['//Users/catherine/Desktop/NLP/PAN_Datasets/PAN2014blog/2014.1-blog-txtfiles/PAN2014_blog_female_chosen_set.1.csv', '/Users/catherine/Desktop/NLP/PAN_Datasets/PAN2014blog/2014.1-blog-txtfiles/PAN2014_blog_male_chosen_set.1.csv']
//Users/catherine/Desktop/NLP/PAN_Datasets/PAN2014blog/2014.1-blog-txtfiles/PAN2014_blog_female_chosen_set.1.csv 1526
/Users/catherine/Desktop/NLP/PAN_Datasets/PAN2014blog/2014.1-blog-txtfiles/PAN2014_blog_male_chosen_set.1.csv 2063
length of features used = 3589
len(all_training_text) 147
len(X_train)= 147 len(y_train)= 147
len(all_test_text) 78
len(X_test= 78 len(y_test)= 78
selector = SelectKBest(score_func=f_classif, k=200)
5
braycurtis K-Nearest Neighbours
[('F', 'M'), ('F', 'M'), ('M', 'M'), ('F', 'M'), ('M', 'M'), ('F', 'M'), ('F', 'M'), ('F', 'M'), ('F', 'M'), ('M', 'M'), ('F', 'M'), ('F', 'M'), ('F', 'M'), ('M', 'M'), ('F', 'M'), ('F', 'M'), ('M', 'M'), ('F', 'M'), ('M', 'M'), ('F', 'M'), ('F', 'M'), ('M', 'M'), ('F', 'M'), 

In [7]:
import sys
sys.path.append("/anaconda3/lib/python3.7/site-packages")
import numpy as np
import numpy
import nltk
import pickle
import pandas as pd
import glob
import os
import re
import operator
from collections import Counter
#from read_xml_files import *
##from s_stemmer import *
from nltk.stem.porter import PorterStemmer
from nltk.stem import LancasterStemmer, WordNetLemmatizer
from itertools import groupby
from nltk.collocations import *
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.cluster import KMeans
from scipy.cluster.vq import whiten
from nltk.tokenize.toktok import ToktokTokenizer
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2
import xml.etree.ElementTree as ET
import pandas as pd
#Import scikit-learn metrics module for accuracy calculation
from sklearn import metrics
##from pattern.text.en import singularize
##nltk.download('vader_lexicon')
##nltk.download('punkt')
#import the relevant modules from the NLTK library
from nltk.sentiment.vader import SentimentIntensityAnalyzer
sentence_tokenizer = nltk.data.load('tokenizers/punkt/english.pickle')
word_tokenizer = nltk.tokenize.RegexpTokenizer(r'\w+')
##tt = ToktokTokenizer()#keeps the url as is...................................................
from nltk.tokenize import TweetTokenizer
tt = TweetTokenizer()
Porter_stemmer = PorterStemmer()
Lancaster_stemmer = LancasterStemmer()
WordNet_lemmatizer = WordNetLemmatizer()
#nltk.download('averaged_perceptron_tagger')
##Train_path="D:/Data/PAN2015-Profiling/pan15-author-profiling-training-dataset-english-2015-04-23"#path for train data
##Test_path="D:/Data/PAN2015-Profiling/pan-ap2015-test/en" #path for test data

##Train_path="D:/NLP/PAN2014/pan14-author-profiling-training-corpus-english-twitter-2014-04-16"#path for train data D:\Data\PAN2013-Profiling\Training\en  D:\NLP\PAN2014
##train_truth_path="D:/NLP/PAN2014/truth_train"
##
##test_truth_path="D:/NLP/PAN2014/pan14_test"
##Test_path="D:/NLP/PAN2014/pan14_test/en"
##Content = open('results_of_clssification_all.txt', 'w')

M_train_file= "/Users/catherine/Desktop/NLP/PAN_Datasets/PAN2014blog/2014.1-blog-txtfiles/PAN2014_blog_male_train.1.txt"
F_train_file= "/Users/catherine/Desktop/NLP/PAN_Datasets/PAN2014blog/2014.1-blog-txtfiles/PAN2014_blog_female_train.1.txt"

M_test_file= "/Users/catherine/Desktop/NLP/PAN_Datasets/PAN2014blog/2014.1-blog-txtfiles/PAN2014_blog_male_test.1.txt"
F_test_file= "/Users/catherine/Desktop/NLP/PAN_Datasets/PAN2014blog/2014.1-blog-txtfiles/PAN2014_blog_female_test.1.txt"

def all_txt(M_txt_file,  F_txt_file):
    with open(M_txt_file, mode="r", encoding="utf-8") as f:
            txt_list = [(line[:-1]).split("\t") for line in f]
            male_dict = dict()
            content=[]
            for line in txt_list:
                if line[1] in male_dict:
                # append the new number to the existing array at this slot
                    male_dict[line[1]].append(line[3])
                else:
                # create a new array in this slot
                    male_dict[line[1]] = [line[3]]
    male_content= [("M", (' '.join(a)).split()) for a in list(male_dict.values())]
  
    with open(F_txt_file, mode="r", encoding="utf-8") as f:
            txt_list = [(line[:-1]).split("\t") for line in f]
            female_dict = dict()
            content=[]
            for line in txt_list:
                if line[1] in female_dict:
                # append the new number to the existing array at this slot
                    female_dict[line[1]].append(line[3])
                else:
                # create a new array in this slot
                    female_dict[line[1]] = [line[3]]
    female_content= [("F", (' '.join(a)).split()) for a in list(female_dict.values())]
    all_txt_per_person = male_content + female_content
    return all_txt_per_person

##print( [((all_txt(M_txt_file,  F_txt_file))[i][0])  for i in range(len((all_txt(M_txt_file,  F_txt_file))))]) helps to determine the gender....

      
"""
create feature vector from the training set.....................................................................................
"""

def WordFeatures(word_list, all_training_text):

    fvs_words = np.array([[author.count(word) for word in word_list] for author in all_training_text]).astype(np.float64)

    # normalise by dividing each row by number of tokens for each author........
    fvs_words /= np.c_[np.array([len(author) for author in all_training_text])]

    return fvs_words



"""
From the saved csv file, recover the saved features to be used...............................................................
"""
import csv
word_list=[]
##txt_files =[ "/Users/catherine/Desktop/NLP/PAN Datasets/PAN_chosen_tokens/2014/output2014OR.1M.txt", "/Users/catherine/Desktop/NLP/PAN Datasets/PAN_chosen_tokens/2014/output2014OR.2F.txt"]
txt_files =[ "//Users/catherine/Desktop/NLP/PAN_Datasets/PAN2014blog/2014.1-blog-txtfiles/PAN2014_blog_female_chosen_set.1.csv", "/Users/catherine/Desktop/NLP/PAN_Datasets/PAN2014blog/2014.1-blog-txtfiles/PAN2014_blog_male_chosen_set.1.csv"]
print("txt_files of features used",txt_files)
##for txt_file in txt_files:
##    with open(txt_file, mode="r", encoding="utf-8") as f:
##        reader = csv.reader(f, delimiter=",")
##        next(reader) # skip header
##        word_list1 =  ([(r.split(":"))[1] for r in f])
##        print(txt_file, len(word_list1))
##        word_list = word_list + word_list1
##print("length of features used =", len(word_list))

for txt_file in txt_files:
    with open(txt_file, mode="r", encoding="utf-8") as f:
        reader = csv.reader(f, delimiter=",")
        next(reader) # skip header
        word_list1 =  [r[0] for r in reader]
        print(txt_file, len(word_list1))
        word_list = word_list + word_list1
print("length of features used =", len(word_list))


"""
Prepare the training and test sets to be parsed to the classifies............................................................
"""
A=(all_txt(M_train_file,  F_train_file))
all_training_text = ( [(A[i][1])  for i in range(len(A))])
print("len(all_training_text)",len(all_training_text))
X_train=WordFeatures(word_list, all_training_text)
y_train=np.array( [(A[i][0])  for i in range(len(A))])
print("len(X_train)=",len(X_train), "len(y_train)=", len(y_train))

B=(all_txt(M_test_file,  F_test_file))
all_test_text = ( [(B[i][1])  for i in range(len(B))])
print("len(all_test_text)",len(all_test_text))
X_test = WordFeatures(word_list, all_test_text)
y_test = np.array( [(B[i][0])  for i in range(len(B))])
print("len(X_test=",len(X_test), "len(y_test)=",len(y_test))


X_train = np.nan_to_num(X_train)
X_test = np.nan_to_num(X_test)

"""
Perform feature selection using chi2,f_classif,mutual_info_classif............................................................
"""
from sklearn.feature_selection import *
##
####def select_Bestfeatures(X, y):
print("selector = SelectKBest(score_func=mutual_info_classif, k=200)")
selector = SelectKBest(score_func=mutual_info_classif, k=200)
fit = selector.fit(X_train,y_train)
train_selected_features = fit.transform(X_train)
test_selected_features = fit.transform(X_test)



"""
x_train and x_test based on the new features using chi2............................................................
"""
##X_train = select_Bestfeatures(X_train, y_train)
##X_test = select_Bestfeatures(X_test, y_test)


"""
Perform feature selection using f_classif,mutual_info_classif............................................................
"""
from sklearn.feature_selection import *
##def f_classif_select_Bestfeatures(X, y):
##selector=SelectKBest(score_func=f_classif,k=500)
##fit = selector.fit(X_train,y_train)
##train_selected_features = fit.transform(X_train)
##test_selected_features = fit.transform(X_test)
##    selected_features = fit.transform(X)
##    return selected_features

"""
x_train and x_test based on the new features using f_classif............................................................
"""
##X_train = f_classif_select_Bestfeatures(X_train, y_train)
##X_test = f_classif_select_Bestfeatures(X_test, y_test)



"""
Perform feature selection using mutual_info_classif............................................................
"""
from sklearn.feature_selection import *
##def mutual_info_classif_select_Bestfeatures(X_tr, X_te, y):
##selector=SelectKBest(score_func=mutual_info_classif,k=100)
##fit = selector.fit(X_train,y_train)
##train_selected_features = fit.transform(X_train)
##test_selected_features = fit.transform(X_test)
##    return train_selected_features, test_selected_features

"""
x_train and x_test based on the new features using f_classif............................................................
"""
X_train =train_selected_features 
X_test = test_selected_features


"""
Perform feature selection using pca............................................................
"""

##from sklearn.decomposition import PCA
##def PCA_select_Bestfeatures(X):
##    # feature extraction
##    pca = PCA(n_components=200)
##    fit = pca.fit(X, y)
##    Train_selected_features = fit.transform(X)
##    return selected_features

"""
x_train and x_test based on the new features using f_classif............................................................
"""
##X_train = PCA_select_Bestfeatures(X_train)
##X_test = PCA_select_Bestfeatures(X_test)

"""
Below are a list of Classification Algorithms with different distance measres ..............................................................................
"""
"""
K-Nearest Neighbours (average aa over all the nearest neighbour selectiopn)..............................................................................
"""

k=5

print(k)

from sklearn.neighbors import KNeighborsClassifier
      
"""
9. scipy.spatial.distance K-Nearest Neighbours..............................................................................
"""
from scipy.spatial.distance import *
knn = KNeighborsClassifier(n_neighbors = k, metric= "braycurtis")
knn.fit(X_train, y_train)
y_pred = knn.predict(X_test)
print("braycurtis K-Nearest Neighbours")
print([(i,j) for i, j in zip(y_pred, y_test)])
print(metrics.accuracy_score(y_test, y_pred))
       
        
print("************************************ ****************************")

txt_files of features used ['//Users/catherine/Desktop/NLP/PAN_Datasets/PAN2014blog/2014.1-blog-txtfiles/PAN2014_blog_female_chosen_set.1.csv', '/Users/catherine/Desktop/NLP/PAN_Datasets/PAN2014blog/2014.1-blog-txtfiles/PAN2014_blog_male_chosen_set.1.csv']
//Users/catherine/Desktop/NLP/PAN_Datasets/PAN2014blog/2014.1-blog-txtfiles/PAN2014_blog_female_chosen_set.1.csv 1526
/Users/catherine/Desktop/NLP/PAN_Datasets/PAN2014blog/2014.1-blog-txtfiles/PAN2014_blog_male_chosen_set.1.csv 2063
length of features used = 3589
len(all_training_text) 147
len(X_train)= 147 len(y_train)= 147
len(all_test_text) 78
len(X_test= 78 len(y_test)= 78
selector = SelectKBest(score_func=mutual_info_classif, k=200)
5
braycurtis K-Nearest Neighbours
[('F', 'M'), ('F', 'M'), ('M', 'M'), ('F', 'M'), ('M', 'M'), ('F', 'M'), ('F', 'M'), ('F', 'M'), ('F', 'M'), ('M', 'M'), ('F', 'M'), ('F', 'M'), ('F', 'M'), ('M', 'M'), ('M', 'M'), ('M', 'M'), ('M', 'M'), ('M', 'M'), ('M', 'M'), ('F', 'M'), ('F', 'M'), ('M', 'M'), ('

In [1]:
import sys
sys.path.append("/anaconda3/lib/python3.7/site-packages")
import numpy as np
import numpy
import nltk
import pickle
import pandas as pd
import glob
import os
import re
import operator
from collections import Counter
#from read_xml_files import *
##from s_stemmer import *
from nltk.stem.porter import PorterStemmer
from nltk.stem import LancasterStemmer, WordNetLemmatizer
from itertools import groupby
from nltk.collocations import *
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.cluster import KMeans
from scipy.cluster.vq import whiten
from nltk.tokenize.toktok import ToktokTokenizer
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2
import xml.etree.ElementTree as ET
import pandas as pd
#Import scikit-learn metrics module for accuracy calculation
from sklearn import metrics
##from pattern.text.en import singularize
##nltk.download('vader_lexicon')
##nltk.download('punkt')
#import the relevant modules from the NLTK library
from nltk.sentiment.vader import SentimentIntensityAnalyzer
sentence_tokenizer = nltk.data.load('tokenizers/punkt/english.pickle')
word_tokenizer = nltk.tokenize.RegexpTokenizer(r'\w+')
##tt = ToktokTokenizer()#keeps the url as is...................................................
from nltk.tokenize import TweetTokenizer
tt = TweetTokenizer()
Porter_stemmer = PorterStemmer()
Lancaster_stemmer = LancasterStemmer()
WordNet_lemmatizer = WordNetLemmatizer()
#nltk.download('averaged_perceptron_tagger')
##Train_path="D:/Data/PAN2015-Profiling/pan15-author-profiling-training-dataset-english-2015-04-23"#path for train data
##Test_path="D:/Data/PAN2015-Profiling/pan-ap2015-test/en" #path for test data

##Train_path="D:/NLP/PAN2014/pan14-author-profiling-training-corpus-english-twitter-2014-04-16"#path for train data D:\Data\PAN2013-Profiling\Training\en  D:\NLP\PAN2014
##train_truth_path="D:/NLP/PAN2014/truth_train"
##
##test_truth_path="D:/NLP/PAN2014/pan14_test"
##Test_path="D:/NLP/PAN2014/pan14_test/en"
##Content = open('results_of_clssification_all.txt', 'w')

M_train_file= "/Users/catherine/Desktop/NLP/PAN_Datasets/PAN2014blog/2014.1-blog-txtfiles/PAN2014_blog_male_train.1.txt"
F_train_file= "/Users/catherine/Desktop/NLP/PAN_Datasets/PAN2014blog/2014.1-blog-txtfiles/PAN2014_blog_female_train.1.txt"

M_test_file= "/Users/catherine/Desktop/NLP/PAN_Datasets/PAN2014blog/2014.1-blog-txtfiles/PAN2014_blog_male_test.1.txt"
F_test_file= "/Users/catherine/Desktop/NLP/PAN_Datasets/PAN2014blog/2014.1-blog-txtfiles/PAN2014_blog_female_test.1.txt"

def all_txt(M_txt_file,  F_txt_file):
    with open(M_txt_file, mode="r", encoding="utf-8") as f:
            txt_list = [(line[:-1]).split("\t") for line in f]
            male_dict = dict()
            content=[]
            for line in txt_list:
                if line[1] in male_dict:
                # append the new number to the existing array at this slot
                    male_dict[line[1]].append(line[3])
                else:
                # create a new array in this slot
                    male_dict[line[1]] = [line[3]]
    male_content= [("M", (' '.join(a)).split()) for a in list(male_dict.values())]
  
    with open(F_txt_file, mode="r", encoding="utf-8") as f:
            txt_list = [(line[:-1]).split("\t") for line in f]
            female_dict = dict()
            content=[]
            for line in txt_list:
                if line[1] in female_dict:
                # append the new number to the existing array at this slot
                    female_dict[line[1]].append(line[3])
                else:
                # create a new array in this slot
                    female_dict[line[1]] = [line[3]]
    female_content= [("F", (' '.join(a)).split()) for a in list(female_dict.values())]
    all_txt_per_person = male_content + female_content
    return all_txt_per_person

##print( [((all_txt(M_txt_file,  F_txt_file))[i][0])  for i in range(len((all_txt(M_txt_file,  F_txt_file))))]) helps to determine the gender....

      
"""
create feature vector from the training set.....................................................................................
"""

def WordFeatures(word_list, all_training_text):

    fvs_words = np.array([[author.count(word) for word in word_list] for author in all_training_text]).astype(np.float64)

    # normalise by dividing each row by number of tokens for each author........
    fvs_words /= np.c_[np.array([len(author) for author in all_training_text])]

    return fvs_words



"""
From the saved csv file, recover the saved features to be used...............................................................
"""
import csv
word_list=[]
##txt_files =[ "/Users/catherine/Desktop/NLP/PAN Datasets/PAN_chosen_tokens/2014/output2014OR.1M.txt", "/Users/catherine/Desktop/NLP/PAN Datasets/PAN_chosen_tokens/2014/output2014OR.2F.txt"]
txt_files =[ "//Users/catherine/Desktop/NLP/PAN_Datasets/PAN2014blog/2014.1-blog-txtfiles/PAN2014_blog_female_chosen_set.1.csv", "/Users/catherine/Desktop/NLP/PAN_Datasets/PAN2014blog/2014.1-blog-txtfiles/PAN2014_blog_male_chosen_set.1.csv"]
print("txt_files of features used",txt_files)
##for txt_file in txt_files:
##    with open(txt_file, mode="r", encoding="utf-8") as f:
##        reader = csv.reader(f, delimiter=",")
##        next(reader) # skip header
##        word_list1 =  ([(r.split(":"))[1] for r in f])
##        print(txt_file, len(word_list1))
##        word_list = word_list + word_list1
##print("length of features used =", len(word_list))

for txt_file in txt_files:
    with open(txt_file, mode="r", encoding="utf-8") as f:
        reader = csv.reader(f, delimiter=",")
        next(reader) # skip header
        word_list1 =  [r[0] for r in reader]
        print(txt_file, len(word_list1))
        word_list = word_list + word_list1
print("length of features used =", len(word_list))


"""
Prepare the training and test sets to be parsed to the classifies............................................................
"""
A=(all_txt(M_train_file,  F_train_file))
all_training_text = ( [(A[i][1])  for i in range(len(A))])
print("len(all_training_text)",len(all_training_text))
X_train=WordFeatures(word_list, all_training_text)
y_train=np.array( [(A[i][0])  for i in range(len(A))])
print("len(X_train)=",len(X_train), "len(y_train)=", len(y_train))

B=(all_txt(M_test_file,  F_test_file))
all_test_text = ( [(B[i][1])  for i in range(len(B))])
print("len(all_test_text)",len(all_test_text))
X_test = WordFeatures(word_list, all_test_text)
y_test = np.array( [(B[i][0])  for i in range(len(B))])
print("len(X_test=",len(X_test), "len(y_test)=",len(y_test))


X_train = np.nan_to_num(X_train)
X_test = np.nan_to_num(X_test)


k=5

print(k)
"""
8. manhattan K-Nearest Neighbours..............................................................................
"""
print("manhattan_k=5************************************ ****************************")
from sklearn.neighbors import KNeighborsClassifier
knn = KNeighborsClassifier(n_neighbors = k, metric= "manhattan")
knn.fit(X_train, y_train)
y_pred = knn.predict(X_test)
print("manhattan K-Nearest Neighbours")
print(metrics.accuracy_score(y_test, y_pred))
print("************************************ ****************************")

k=13

print(k)
"""
8. manhattan K-Nearest Neighbours..............................................................................
"""
print("manhattan_k=13************************************ ****************************")
from sklearn.neighbors import KNeighborsClassifier
knn = KNeighborsClassifier(n_neighbors = k, metric= "manhattan")
knn.fit(X_train, y_train)
y_pred = knn.predict(X_test)
print("manhattan K-Nearest Neighbours")
print(metrics.accuracy_score(y_test, y_pred))
print("****************************************************************")


from sklearn import model_selection, naive_bayes, svm
from sklearn.linear_model import SGDClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import LinearSVC



print("Naive****************************************************************")
Naive = naive_bayes.MultinomialNB()
Naive.fit(X_train, y_train)
y_pred = Naive.predict(X_test)
print(metrics.accuracy_score(y_test, y_pred))


print("SVM****************************************************************")
SVM = svm.SVC(C=1.0, kernel='linear', degree=3, gamma='auto')
SVM.fit(X_train, y_train)
y_pred = SVM.predict(X_test)
print(metrics.accuracy_score(y_test, y_pred))


print("RandomForest_100****************************************************************")
clf=RandomForestClassifier(n_estimators=100)
#Train the model using the training sets y_pred=clf.predict(X_test)
clf.fit(X_train,y_train)
y_pred=clf.predict(X_test)
print(metrics.accuracy_score(y_test, y_pred))

print("RandomForest_200****************************************************************")
clf=RandomForestClassifier(n_estimators=200)
#Train the model using the training sets y_pred=clf.predict(X_test)
clf.fit(X_train,y_train)
y_pred=clf.predict(X_test)
print(metrics.accuracy_score(y_test, y_pred))



/anaconda3/lib/python3.7/site-packages/nltk/twitter/__init__.py:20: UserWarning: The twython library has not been installed. Some functionality from the twitter package will not be available.
  warnings.warn("The twython library has not been installed. "


txt_files of features used ['//Users/catherine/Desktop/NLP/PAN_Datasets/PAN2014blog/2014.1-blog-txtfiles/PAN2014_blog_female_chosen_set.1.csv', '/Users/catherine/Desktop/NLP/PAN_Datasets/PAN2014blog/2014.1-blog-txtfiles/PAN2014_blog_male_chosen_set.1.csv']
//Users/catherine/Desktop/NLP/PAN_Datasets/PAN2014blog/2014.1-blog-txtfiles/PAN2014_blog_female_chosen_set.1.csv 1526
/Users/catherine/Desktop/NLP/PAN_Datasets/PAN2014blog/2014.1-blog-txtfiles/PAN2014_blog_male_chosen_set.1.csv 2063
length of features used = 3589
len(all_training_text) 147
len(X_train)= 147 len(y_train)= 147
len(all_test_text) 78
len(X_test= 78 len(y_test)= 78
5
manhattan_k=5************************************ ****************************
manhattan K-Nearest Neighbours
0.6794871794871795
************************************ ****************************
13
manhattan_k=13************************************ ****************************
manhattan K-Nearest Neighbours
0.7051282051282052
*********************************

/anaconda3/lib/python3.7/site-packages/sklearn/ensemble/weight_boosting.py:29: DeprecationWarning: numpy.core.umath_tests is an internal NumPy module and should not be imported. It will be removed in a future NumPy release.
  from numpy.core.umath_tests import inner1d


0.5
RandomForest_100****************************************************************
0.6025641025641025
RandomForest_200****************************************************************
0.5769230769230769


In [2]:
import sys
sys.path.append("/anaconda3/lib/python3.7/site-packages")
import numpy as np
import numpy
import nltk
import pickle
import pandas as pd
import glob
import os
import re
import operator
from collections import Counter
#from read_xml_files import *
##from s_stemmer import *
from nltk.stem.porter import PorterStemmer
from nltk.stem import LancasterStemmer, WordNetLemmatizer
from itertools import groupby
from nltk.collocations import *
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.cluster import KMeans
from scipy.cluster.vq import whiten
from nltk.tokenize.toktok import ToktokTokenizer
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2
import xml.etree.ElementTree as ET
import pandas as pd
#Import scikit-learn metrics module for accuracy calculation
from sklearn import metrics
##from pattern.text.en import singularize
##nltk.download('vader_lexicon')
##nltk.download('punkt')
#import the relevant modules from the NLTK library
from nltk.sentiment.vader import SentimentIntensityAnalyzer
sentence_tokenizer = nltk.data.load('tokenizers/punkt/english.pickle')
word_tokenizer = nltk.tokenize.RegexpTokenizer(r'\w+')
##tt = ToktokTokenizer()#keeps the url as is...................................................
from nltk.tokenize import TweetTokenizer
tt = TweetTokenizer()
Porter_stemmer = PorterStemmer()
Lancaster_stemmer = LancasterStemmer()
WordNet_lemmatizer = WordNetLemmatizer()
#nltk.download('averaged_perceptron_tagger')
##Train_path="D:/Data/PAN2015-Profiling/pan15-author-profiling-training-dataset-english-2015-04-23"#path for train data
##Test_path="D:/Data/PAN2015-Profiling/pan-ap2015-test/en" #path for test data

##Train_path="D:/NLP/PAN2014/pan14-author-profiling-training-corpus-english-twitter-2014-04-16"#path for train data D:\Data\PAN2013-Profiling\Training\en  D:\NLP\PAN2014
##train_truth_path="D:/NLP/PAN2014/truth_train"
##
##test_truth_path="D:/NLP/PAN2014/pan14_test"
##Test_path="D:/NLP/PAN2014/pan14_test/en"
##Content = open('results_of_clssification_all.txt', 'w')

M_train_file= "/Users/catherine/Desktop/NLP/PAN_Datasets/PAN2014blog/2014.1-blog-txtfiles/PAN2014_blog_male_train.1.txt"
F_train_file= "/Users/catherine/Desktop/NLP/PAN_Datasets/PAN2014blog/2014.1-blog-txtfiles/PAN2014_blog_female_train.1.txt"

M_test_file= "/Users/catherine/Desktop/NLP/PAN_Datasets/PAN2014blog/2014.1-blog-txtfiles/PAN2014_blog_male_test.1.txt"
F_test_file= "/Users/catherine/Desktop/NLP/PAN_Datasets/PAN2014blog/2014.1-blog-txtfiles/PAN2014_blog_female_test.1.txt"

def all_txt(M_txt_file,  F_txt_file):
    with open(M_txt_file, mode="r", encoding="utf-8") as f:
            txt_list = [(line[:-1]).split("\t") for line in f]
            male_dict = dict()
            content=[]
            for line in txt_list:
                if line[1] in male_dict:
                # append the new number to the existing array at this slot
                    male_dict[line[1]].append(line[3])
                else:
                # create a new array in this slot
                    male_dict[line[1]] = [line[3]]
    male_content= [("M", (' '.join(a)).split()) for a in list(male_dict.values())]
  
    with open(F_txt_file, mode="r", encoding="utf-8") as f:
            txt_list = [(line[:-1]).split("\t") for line in f]
            female_dict = dict()
            content=[]
            for line in txt_list:
                if line[1] in female_dict:
                # append the new number to the existing array at this slot
                    female_dict[line[1]].append(line[3])
                else:
                # create a new array in this slot
                    female_dict[line[1]] = [line[3]]
    female_content= [("F", (' '.join(a)).split()) for a in list(female_dict.values())]
    all_txt_per_person = male_content + female_content
    return all_txt_per_person

##print( [((all_txt(M_txt_file,  F_txt_file))[i][0])  for i in range(len((all_txt(M_txt_file,  F_txt_file))))]) helps to determine the gender....

      
"""
create feature vector from the training set.....................................................................................
"""

def WordFeatures(word_list, all_training_text):

    fvs_words = np.array([[author.count(word) for word in word_list] for author in all_training_text]).astype(np.float64)

    # normalise by dividing each row by number of tokens for each author........
    fvs_words /= np.c_[np.array([len(author) for author in all_training_text])]

    return fvs_words



"""
From the saved csv file, recover the saved features to be used...............................................................
"""
import csv
word_list=[]
##txt_files =[ "/Users/catherine/Desktop/NLP/PAN Datasets/PAN_chosen_tokens/2014/output2014OR.1M.txt", "/Users/catherine/Desktop/NLP/PAN Datasets/PAN_chosen_tokens/2014/output2014OR.2F.txt"]
txt_files =[ "//Users/catherine/Desktop/NLP/PAN_Datasets/PAN2014blog/2014.1-blog-txtfiles/PAN2014_blog_female_vocubulary.1.csv", "/Users/catherine/Desktop/NLP/PAN_Datasets/PAN2014blog/2014.1-blog-txtfiles/PAN2014_blog_male_ vocubulary_set.1.csv"]
print("txt_files of features used",txt_files)
##for txt_file in txt_files:
##    with open(txt_file, mode="r", encoding="utf-8") as f:
##        reader = csv.reader(f, delimiter=",")
##        next(reader) # skip header
##        word_list1 =  ([(r.split(":"))[1] for r in f])
##        print(txt_file, len(word_list1))
##        word_list = word_list + word_list1
##print("length of features used =", len(word_list))

for txt_file in txt_files:
    with open(txt_file, mode="r", encoding="utf-8") as f:
        reader = csv.reader(f, delimiter=",")
        next(reader) # skip header
        word_list1 =  [r[0] for r in reader]
        print(txt_file, len(word_list1))
        word_list = word_list + word_list1
print("length of features used =", len(word_list))


"""
Prepare the training and test sets to be parsed to the classifies............................................................
"""
A=(all_txt(M_train_file,  F_train_file))
all_training_text = ( [(A[i][1])  for i in range(len(A))])
print("len(all_training_text)",len(all_training_text))
X_train=WordFeatures(word_list, all_training_text)
y_train=np.array( [(A[i][0])  for i in range(len(A))])
print("len(X_train)=",len(X_train), "len(y_train)=", len(y_train))

B=(all_txt(M_test_file,  F_test_file))
all_test_text = ( [(B[i][1])  for i in range(len(B))])
print("len(all_test_text)",len(all_test_text))
X_test = WordFeatures(word_list, all_test_text)
y_test = np.array( [(B[i][0])  for i in range(len(B))])
print("len(X_test=",len(X_test), "len(y_test)=",len(y_test))


X_train = np.nan_to_num(X_train)
X_test = np.nan_to_num(X_test)


k=5

print(k)
"""
8. manhattan K-Nearest Neighbours..............................................................................
"""
print("manhattan_k=5************************************ ****************************")
from sklearn.neighbors import KNeighborsClassifier
knn = KNeighborsClassifier(n_neighbors = k, metric= "manhattan")
knn.fit(X_train, y_train)
y_pred = knn.predict(X_test)
print("manhattan K-Nearest Neighbours")
print(metrics.accuracy_score(y_test, y_pred))
print("************************************ ****************************")

k=13

print(k)
"""
8. manhattan K-Nearest Neighbours..............................................................................
"""
print("manhattan_k=13************************************ ****************************")
from sklearn.neighbors import KNeighborsClassifier
knn = KNeighborsClassifier(n_neighbors = k, metric= "manhattan")
knn.fit(X_train, y_train)
y_pred = knn.predict(X_test)
print("manhattan K-Nearest Neighbours")
print(metrics.accuracy_score(y_test, y_pred))
print("****************************************************************")


from sklearn import model_selection, naive_bayes, svm
from sklearn.linear_model import SGDClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import LinearSVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.neural_network import MLPClassifier


print("MLPClassifier****************************************************************")
clf = MLPClassifier()
clf = clf.fit(X_train, y_train)
#Predict the response for test dataset
y_pred = clf.predict(X_test)
print(metrics.accuracy_score(y_test, y_pred))


print("LogisticRegression****************************************************************")
clf = LogisticRegression()
clf = clf.fit(X_train, y_train)
#Predict the response for test dataset
y_pred = clf.predict(X_test)
print(metrics.accuracy_score(y_test, y_pred))


print("GDClassifier****************************************************************")
clf = GDClassifier()
clf = clf.fit(X_train, y_train)
#Predict the response for test dataset
y_pred = clf.predict(X_test)
print(metrics.accuracy_score(y_test, y_pred))


print("DecisionTreeClassifier****************************************************************")
# Create Decision Tree classifer object
clf = DecisionTreeClassifier()
# Train Decision Tree Classifer
clf = clf.fit(X_train, y_train)
#Predict the response for test dataset
y_pred = clf.predict(X_test)
print(metrics.accuracy_score(y_test, y_pred))


print("Naive****************************************************************")
Naive = naive_bayes.MultinomialNB()
Naive.fit(X_train, y_train)
y_pred = Naive.predict(X_test)
print(metrics.accuracy_score(y_test, y_pred))


print("SVM****************************************************************")
SVM = svm.SVC(C=1.0, kernel='linear', degree=3, gamma='auto')
SVM.fit(X_train, y_train)
y_pred = SVM.predict(X_test)
print(metrics.accuracy_score(y_test, y_pred))


print("RandomForest_100****************************************************************")
clf=RandomForestClassifier(n_estimators=100)
#Train the model using the training sets y_pred=clf.predict(X_test)
clf.fit(X_train,y_train)
y_pred=clf.predict(X_test)
print(metrics.accuracy_score(y_test, y_pred))

print("RandomForest_200****************************************************************")
clf=RandomForestClassifier(n_estimators=200)
#Train the model using the training sets y_pred=clf.predict(X_test)
clf.fit(X_train,y_train)
y_pred=clf.predict(X_test)
print(metrics.accuracy_score(y_test, y_pred))


txt_files of features used ['//Users/catherine/Desktop/NLP/PAN_Datasets/PAN2014blog/2014.1-blog-txtfiles/PAN2014_blog_female_vocubulary.1.csv', '/Users/catherine/Desktop/NLP/PAN_Datasets/PAN2014blog/2014.1-blog-txtfiles/PAN2014_blog_male_ vocubulary_set.1.csv']
//Users/catherine/Desktop/NLP/PAN_Datasets/PAN2014blog/2014.1-blog-txtfiles/PAN2014_blog_female_vocubulary.1.csv 12375
/Users/catherine/Desktop/NLP/PAN_Datasets/PAN2014blog/2014.1-blog-txtfiles/PAN2014_blog_male_ vocubulary_set.1.csv 18889
length of features used = 31264
len(all_training_text) 147
len(X_train)= 147 len(y_train)= 147
len(all_test_text) 78
len(X_test= 78 len(y_test)= 78
5
manhattan_k=5************************************ ****************************
manhattan K-Nearest Neighbours
0.6666666666666666
************************************ ****************************
13
manhattan_k=13************************************ ****************************
manhattan K-Nearest Neighbours
0.6282051282051282
********************

In [9]:
import sys
sys.path.append("/anaconda3/lib/python3.7/site-packages")
import numpy as np
import numpy
import nltk
import pickle
import pandas as pd
import glob
import os
import re
import operator
from collections import Counter
#from read_xml_files import *
##from s_stemmer import *
from nltk.stem.porter import PorterStemmer
from nltk.stem import LancasterStemmer, WordNetLemmatizer
from itertools import groupby
from nltk.collocations import *
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.cluster import KMeans
from scipy.cluster.vq import whiten
from nltk.tokenize.toktok import ToktokTokenizer
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2
import xml.etree.ElementTree as ET
import pandas as pd
#Import scikit-learn metrics module for accuracy calculation
from sklearn import metrics
##from pattern.text.en import singularize
##nltk.download('vader_lexicon')
##nltk.download('punkt')
#import the relevant modules from the NLTK library
from nltk.sentiment.vader import SentimentIntensityAnalyzer
sentence_tokenizer = nltk.data.load('tokenizers/punkt/english.pickle')
word_tokenizer = nltk.tokenize.RegexpTokenizer(r'\w+')
##tt = ToktokTokenizer()#keeps the url as is...................................................
from nltk.tokenize import TweetTokenizer
tt = TweetTokenizer()
Porter_stemmer = PorterStemmer()
Lancaster_stemmer = LancasterStemmer()
WordNet_lemmatizer = WordNetLemmatizer()
#nltk.download('averaged_perceptron_tagger')
##Train_path="D:/Data/PAN2015-Profiling/pan15-author-profiling-training-dataset-english-2015-04-23"#path for train data
##Test_path="D:/Data/PAN2015-Profiling/pan-ap2015-test/en" #path for test data

##Train_path="D:/NLP/PAN2014/pan14-author-profiling-training-corpus-english-twitter-2014-04-16"#path for train data D:\Data\PAN2013-Profiling\Training\en  D:\NLP\PAN2014
##train_truth_path="D:/NLP/PAN2014/truth_train"
##
##test_truth_path="D:/NLP/PAN2014/pan14_test"
##Test_path="D:/NLP/PAN2014/pan14_test/en"
##Content = open('results_of_clssification_all.txt', 'w')

M_train_file= "/Users/catherine/Desktop/NLP/PAN_Datasets/PAN2014blog/2014.1-blog-txtfiles/PAN2014_blog_male_train.1.txt"
F_train_file= "/Users/catherine/Desktop/NLP/PAN_Datasets/PAN2014blog/2014.1-blog-txtfiles/PAN2014_blog_female_train.1.txt"

M_test_file= "/Users/catherine/Desktop/NLP/PAN_Datasets/PAN2014blog/2014.1-blog-txtfiles/PAN2014_blog_male_test.1.txt"
F_test_file= "/Users/catherine/Desktop/NLP/PAN_Datasets/PAN2014blog/2014.1-blog-txtfiles/PAN2014_blog_female_test.1.txt"

def all_txt(M_txt_file,  F_txt_file):
    with open(M_txt_file, mode="r", encoding="utf-8") as f:
            txt_list = [(line[:-1]).split("\t") for line in f]
            male_dict = dict()
            content=[]
            for line in txt_list:
                if line[1] in male_dict:
                # append the new number to the existing array at this slot
                    male_dict[line[1]].append(line[3])
                else:
                # create a new array in this slot
                    male_dict[line[1]] = [line[3]]
    male_content= [("M", (' '.join(a)).split()) for a in list(male_dict.values())]
  
    with open(F_txt_file, mode="r", encoding="utf-8") as f:
            txt_list = [(line[:-1]).split("\t") for line in f]
            female_dict = dict()
            content=[]
            for line in txt_list:
                if line[1] in female_dict:
                # append the new number to the existing array at this slot
                    female_dict[line[1]].append(line[3])
                else:
                # create a new array in this slot
                    female_dict[line[1]] = [line[3]]
    female_content= [("F", (' '.join(a)).split()) for a in list(female_dict.values())]
    all_txt_per_person = male_content + female_content
    return all_txt_per_person

##print( [((all_txt(M_txt_file,  F_txt_file))[i][0])  for i in range(len((all_txt(M_txt_file,  F_txt_file))))]) helps to determine the gender....

      
"""
create feature vector from the training set.....................................................................................
"""

def WordFeatures(word_list, all_training_text):

    fvs_words = np.array([[author.count(word) for word in word_list] for author in all_training_text]).astype(np.float64)

    # normalise by dividing each row by number of tokens for each author........
    fvs_words /= np.c_[np.array([len(author) for author in all_training_text])]

    return fvs_words



"""
From the saved csv file, recover the saved features to be used...............................................................
"""
import csv
word_list=[]
##txt_files =[ "/Users/catherine/Desktop/NLP/PAN Datasets/PAN_chosen_tokens/2014/output2014OR.1M.txt", "/Users/catherine/Desktop/NLP/PAN Datasets/PAN_chosen_tokens/2014/output2014OR.2F.txt"]
txt_files =[ "/Users/catherine/Desktop/NLP/PAN_Datasets/PAN2014blog/2014.1-blog-txtfiles/PAN2014_blog_female_vocubulary.1.csv", "/Users/catherine/Desktop/NLP/PAN_Datasets/PAN2014blog/2014.1-blog-txtfiles/PAN2014_blog_male_ vocubulary_set.1.csv"]
print("txt_files of features used",txt_files)
##for txt_file in txt_files:
##    with open(txt_file, mode="r", encoding="utf-8") as f:
##        reader = csv.reader(f, delimiter=",")
##        next(reader) # skip header
##        word_list1 =  ([(r.split(":"))[1] for r in f])
##        print(txt_file, len(word_list1))
##        word_list = word_list + word_list1
##print("length of features used =", len(word_list))

for txt_file in txt_files:
    with open(txt_file, mode="r", encoding="utf-8") as f:
        reader = csv.reader(f, delimiter=",")
        next(reader) # skip header
        word_list1 =  [r[0] for r in reader]
        print(txt_file, len(word_list1))
        word_list = word_list + word_list1
print("length of features used =", len(word_list))


"""
Prepare the training and test sets to be parsed to the classifies............................................................
"""
A=(all_txt(M_train_file,  F_train_file))
all_training_text = ( [(A[i][1])  for i in range(len(A))])
print("len(all_training_text)",len(all_training_text))
X_train=WordFeatures(word_list, all_training_text)
y_train=np.array( [(A[i][0])  for i in range(len(A))])
print("len(X_train)=",len(X_train), "len(y_train)=", len(y_train))

B=(all_txt(M_test_file,  F_test_file))
all_test_text = ( [(B[i][1])  for i in range(len(B))])
print("len(all_test_text)",len(all_test_text))
X_test = WordFeatures(word_list, all_test_text)
y_test = np.array( [(B[i][0])  for i in range(len(B))])
print("len(X_test=",len(X_test), "len(y_test)=",len(y_test))


X_train = np.nan_to_num(X_train)
X_test = np.nan_to_num(X_test)


k=5

print(k)
"""
8. manhattan K-Nearest Neighbours..............................................................................
"""
print("manhattan_k=5************************************ ****************************")
from sklearn.neighbors import KNeighborsClassifier
knn = KNeighborsClassifier(n_neighbors = k, metric= "manhattan")
knn.fit(X_train, y_train)
y_pred = knn.predict(X_test)
print("manhattan K-Nearest Neighbours")
manhattan_k5voc=([(i,j) for i, j in zip(y_pred, y_test)])
print(metrics.accuracy_score(y_test, y_pred))
print("************************************ ****************************")

k=13

print(k)
"""
8. manhattan K-Nearest Neighbours..............................................................................
"""
print("manhattan_k=13************************************ ****************************")
from sklearn.neighbors import KNeighborsClassifier
knn = KNeighborsClassifier(n_neighbors = k, metric= "manhattan")
knn.fit(X_train, y_train)
y_pred = knn.predict(X_test)
print("manhattan K-Nearest Neighbours")
manhattan_k13voc=([(i,j) for i, j in zip(y_pred, y_test)])
print(metrics.accuracy_score(y_test, y_pred))
print("****************************************************************")


from sklearn import model_selection, naive_bayes, svm
from sklearn.linear_model import SGDClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import LinearSVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.neural_network import MLPClassifier


print("MLPClassifier****************************************************************")
clf = MLPClassifier()
clf = clf.fit(X_train, y_train)
#Predict the response for test dataset
y_pred = clf.predict(X_test)
MLPClassifiervoc=([(i,j) for i, j in zip(y_pred, y_test)])
print(metrics.accuracy_score(y_test, y_pred))


print("LogisticRegression****************************************************************")
clf = LogisticRegression()
clf = clf.fit(X_train, y_train)
#Predict the response for test dataset
y_pred = clf.predict(X_test)
LogisticRegressionvoc=([(i,j) for i, j in zip(y_pred, y_test)])
print(metrics.accuracy_score(y_test, y_pred))


print("SGDClassifier****************************************************************")
clf = SGDClassifier()
clf = clf.fit(X_train, y_train)
#Predict the response for test dataset
y_pred = clf.predict(X_test)
SGDClassifiervoc=([(i,j) for i, j in zip(y_pred, y_test)])
print(metrics.accuracy_score(y_test, y_pred))


print("DecisionTreeClassifier****************************************************************")
# Create Decision Tree classifer object
clf = DecisionTreeClassifier()
# Train Decision Tree Classifer
clf = clf.fit(X_train, y_train)
#Predict the response for test dataset
y_pred = clf.predict(X_test)
DecisionTreeClassifiervoc=([(i,j) for i, j in zip(y_pred, y_test)])
print(metrics.accuracy_score(y_test, y_pred))


print("Naive****************************************************************")
Naive = naive_bayes.MultinomialNB()
Naive.fit(X_train, y_train)
y_pred = Naive.predict(X_test)
Naivevoc=([(i,j) for i, j in zip(y_pred, y_test)])
print(metrics.accuracy_score(y_test, y_pred))


print("SVM****************************************************************")
SVM = svm.SVC(C=1.0, kernel='linear', degree=3, gamma='auto')
SVM.fit(X_train, y_train)
y_pred = SVM.predict(X_test)
SVMvoc=([(i,j) for i, j in zip(y_pred, y_test)])
print(metrics.accuracy_score(y_test, y_pred))


print("RandomForest_100****************************************************************")
clf=RandomForestClassifier(n_estimators=100)
#Train the model using the training sets y_pred=clf.predict(X_test)
clf.fit(X_train,y_train)
y_pred=clf.predict(X_test)
RandomForest_100voc=([(i,j) for i, j in zip(y_pred, y_test)])
print(metrics.accuracy_score(y_test, y_pred))

print("RandomForest_200****************************************************************")
clf=RandomForestClassifier(n_estimators=200)
#Train the model using the training sets y_pred=clf.predict(X_test)
clf.fit(X_train,y_train)
y_pred=clf.predict(X_test)
RandomForest_200voc=([(i,j) for i, j in zip(y_pred, y_test)])
print(metrics.accuracy_score(y_test, y_pred))

from xgboost.sklearn import XGBClassifier 
print("XGBClassifier****************************************************************")
clf=XGBClassifier()
#Train the model using the training sets y_pred=clf.predict(X_test)
clf.fit(X_train, y_train)
y_pred=clf.predict(X_test)
XGBClassifiervoc=([(i,j) for i, j in zip(y_pred, y_test)])
print(metrics.accuracy_score(y_test, y_pred))



print("BaggingClassifier****************************************************************")
from sklearn.ensemble.bagging import BaggingClassifier
clf=BaggingClassifier()
#Train the model using the training sets y_pred=clf.predict(X_test)
clf.fit(X_train, y_train)
y_pred=clf.predict(X_test)
BaggingClassifiervoc=([(i,j) for i, j in zip(y_pred, y_test)])
print(metrics.accuracy_score(y_test, y_pred))


print("AdaBoostClassifier****************************************************************")
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
clf=AdaBoostClassifier()
#Train the model using the training sets y_pred=clf.predict(X_test)
clf.fit(X_train, y_train)
y_pred=clf.predict(X_test)
AdaBoostClassifiervoc=([(i,j) for i, j in zip(y_pred, y_test)])
print(metrics.accuracy_score(y_test, y_pred))








txt_files of features used ['/Users/catherine/Desktop/NLP/PAN_Datasets/PAN2014blog/2014.1-blog-txtfiles/PAN2014_blog_female_vocubulary.1.csv', '/Users/catherine/Desktop/NLP/PAN_Datasets/PAN2014blog/2014.1-blog-txtfiles/PAN2014_blog_male_ vocubulary_set.1.csv']
/Users/catherine/Desktop/NLP/PAN_Datasets/PAN2014blog/2014.1-blog-txtfiles/PAN2014_blog_female_vocubulary.1.csv 12375
/Users/catherine/Desktop/NLP/PAN_Datasets/PAN2014blog/2014.1-blog-txtfiles/PAN2014_blog_male_ vocubulary_set.1.csv 18889
length of features used = 31264
len(all_training_text) 147
len(X_train)= 147 len(y_train)= 147
len(all_test_text) 78
len(X_test= 78 len(y_test)= 78
5
manhattan_k=5************************************ ****************************
manhattan K-Nearest Neighbours
0.6666666666666666
************************************ ****************************
13
manhattan_k=13************************************ ****************************
manhattan K-Nearest Neighbours
0.6282051282051282
**********************

/anaconda3/lib/python3.7/site-packages/sklearn/neural_network/multilayer_perceptron.py:564: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)


0.717948717948718
LogisticRegression****************************************************************
0.5769230769230769
SGDClassifier****************************************************************
0.5
DecisionTreeClassifier****************************************************************


/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/stochastic_gradient.py:128: FutureWarning: max_iter and tol parameters have been added in <class 'sklearn.linear_model.stochastic_gradient.SGDClassifier'> in 0.19. If both are left unset, they default to max_iter=5 and tol=None. If tol is not None, max_iter defaults to max_iter=1000. From 0.21, default max_iter will be 1000, and default tol will be 1e-3.
  "and default tol will be 1e-3." % type(self), FutureWarning)


0.6153846153846154
Naive****************************************************************
0.5641025641025641
SVM****************************************************************
0.5
RandomForest_100****************************************************************
0.5897435897435898
RandomForest_200****************************************************************
0.5384615384615384
XGBClassifier****************************************************************


/anaconda3/lib/python3.7/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


0.6025641025641025
BaggingClassifier****************************************************************
0.5512820512820513
AdaBoostClassifier****************************************************************
0.5256410256410257


In [11]:
print("The sign test...****************************************************************")
print("manhattan_k=5************************************ ****************************")
x= MLPClassifiervoc 
y= manhattan_k5voc

positive=0
negative=0
for i in range(len(x)):
    if (x[i]!=y[i]):
        if (x[i][0]==x[i][1] and y[i][0]!=y[i][1]):
            positive=positive+1
        else: 
            negative=negative+1
print("positive=",positive, "negative =" , negative)
        
print("manhattan_k=13************************************ ****************************")
x= MLPClassifiervoc 
y= manhattan_k13voc

positive=0
negative=0
for i in range(len(x)):
    if (x[i]!=y[i]):
        if (x[i][0]==x[i][1] and y[i][0]!=y[i][1]):
            positive=positive+1
        else: 
            negative=negative+1
print("positive=",positive, "negative =" , negative)

print("MLPClassifier****************************************************************")
x= MLPClassifiervoc 
y= MLPClassifiervoc

positive=0
negative=0
for i in range(len(x)):
    if (x[i]!=y[i]):
        if (x[i][0]==x[i][1] and y[i][0]!=y[i][1]):
            positive=positive+1
        else: 
            negative=negative+1
print("positive=",positive, "negative =" , negative)




print("LogisticRegression****************************************************************")
x= MLPClassifiervoc 
y= LogisticRegressionvoc

positive=0
negative=0
for i in range(len(x)):
    if (x[i]!=y[i]):
        if (x[i][0]==x[i][1] and y[i][0]!=y[i][1]):
            positive=positive+1
        else: 
            negative=negative+1
print("positive=",positive, "negative =" , negative)




print("SGDClassifier****************************************************************")
x= MLPClassifiervoc 
y= SGDClassifiervoc

positive=0
negative=0
for i in range(len(x)):
    if (x[i]!=y[i]):
        if (x[i][0]==x[i][1] and y[i][0]!=y[i][1]):
            positive=positive+1
        else: 
            negative=negative+1
print("positive=",positive, "negative =" , negative)



print("DecisionTreeClassifier****************************************************************")
x= MLPClassifiervoc 
y= DecisionTreeClassifiervoc

positive=0
negative=0
for i in range(len(x)):
    if (x[i]!=y[i]):
        if (x[i][0]==x[i][1] and y[i][0]!=y[i][1]):
            positive=positive+1
        else: 
            negative=negative+1
print("positive=",positive, "negative =" , negative)



print("Naive****************************************************************")
x= MLPClassifiervoc 
y= Naivevoc

positive=0
negative=0
for i in range(len(x)):
    if (x[i]!=y[i]):
        if (x[i][0]==x[i][1] and y[i][0]!=y[i][1]):
            positive=positive+1
        else: 
            negative=negative+1
print("positive=",positive, "negative =" , negative)



print("SVM****************************************************************")
x= MLPClassifiervoc 
y= SVMvoc

positive=0
negative=0
for i in range(len(x)):
    if (x[i]!=y[i]):
        if (x[i][0]==x[i][1] and y[i][0]!=y[i][1]):
            positive=positive+1
        else: 
            negative=negative+1
print("positive=",positive, "negative =" , negative)



print("RandomForest_100****************************************************************")
x= MLPClassifiervoc 
y= RandomForest_100voc

positive=0
negative=0
for i in range(len(x)):
    if (x[i]!=y[i]):
        if (x[i][0]==x[i][1] and y[i][0]!=y[i][1]):
            positive=positive+1
        else: 
            negative=negative+1
print("positive=",positive, "negative =" , negative)



print("RandomForest_200****************************************************************")
x= MLPClassifiervoc 
y= RandomForest_200voc

positive=0
negative=0
for i in range(len(x)):
    if (x[i]!=y[i]):
        if (x[i][0]==x[i][1] and y[i][0]!=y[i][1]):
            positive=positive+1
        else: 
            negative=negative+1
print("positive=",positive, "negative =" , negative)




print("XGBClassifier****************************************************************")
x= MLPClassifiervoc 
y= XGBClassifiervoc

positive=0
negative=0
for i in range(len(x)):
    if (x[i]!=y[i]):
        if (x[i][0]==x[i][1] and y[i][0]!=y[i][1]):
            positive=positive+1
        else: 
            negative=negative+1
print("positive=",positive, "negative =" , negative)




print("BaggingClassifier****************************************************************")
x= MLPClassifiervoc 
y= BaggingClassifiervoc

positive=0
negative=0
for i in range(len(x)):
    if (x[i]!=y[i]):
        if (x[i][0]==x[i][1] and y[i][0]!=y[i][1]):
            positive=positive+1
        else: 
            negative=negative+1
print("positive=",positive, "negative =" , negative)



print("AdaBoostClassifier****************************************************************")
x= MLPClassifiervoc
y= AdaBoostClassifiervoc

positive=0
negative=0
for i in range(len(x)):
    if (x[i]!=y[i]):
        if (x[i][0]==x[i][1] and y[i][0]!=y[i][1]):
            positive=positive+1
        else: 
            negative=negative+1
print("positive=",positive, "negative =" , negative)











The sign test...****************************************************************
manhattan_k=5************************************ ****************************
positive= 8 negative = 4
manhattan_k=13************************************ ****************************
positive= 11 negative = 4
MLPClassifier****************************************************************
positive= 0 negative = 0
LogisticRegression****************************************************************
positive= 16 negative = 5
SGDClassifier****************************************************************
positive= 30 negative = 13
DecisionTreeClassifier****************************************************************
positive= 18 negative = 10
Naive****************************************************************
positive= 15 negative = 3
SVM****************************************************************
positive= 30 negative = 13
RandomForest_100****************************************************************
positi

In [57]:
print("BaggingClassifier****************************************************************")
from sklearn.ensemble.bagging import BaggingClassifier
clf=BaggingClassifier()
#Train the model using the training sets y_pred=clf.predict(X_test)
clf.fit(X_train, y_train)
y_pred=clf.predict(X_test)
print(metrics.accuracy_score(y_test, y_pred))


print("AdaBoostClassifier****************************************************************")
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
clf=AdaBoostClassifier()
#Train the model using the training sets y_pred=clf.predict(X_test)
clf.fit(X_train, y_train)
y_pred=clf.predict(X_test)
print(metrics.accuracy_score(y_test, y_pred))



BaggingClassifier****************************************************************
0.5897435897435898
AdaBoostClassifier****************************************************************
0.5256410256410257


In [12]:
import sys
sys.path.append("/anaconda3/lib/python3.7/site-packages")
import numpy as np
import numpy
import nltk
import pickle
import pandas as pd
import glob
import os
import re
import operator
from collections import Counter
#from read_xml_files import *
##from s_stemmer import *
from nltk.stem.porter import PorterStemmer
from nltk.stem import LancasterStemmer, WordNetLemmatizer
from itertools import groupby
from nltk.collocations import *
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.cluster import KMeans
from scipy.cluster.vq import whiten
from nltk.tokenize.toktok import ToktokTokenizer
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2
import xml.etree.ElementTree as ET
import pandas as pd
#Import scikit-learn metrics module for accuracy calculation
from sklearn import metrics
##from pattern.text.en import singularize
##nltk.download('vader_lexicon')
##nltk.download('punkt')
#import the relevant modules from the NLTK library
from nltk.sentiment.vader import SentimentIntensityAnalyzer
sentence_tokenizer = nltk.data.load('tokenizers/punkt/english.pickle')
word_tokenizer = nltk.tokenize.RegexpTokenizer(r'\w+')
##tt = ToktokTokenizer()#keeps the url as is...................................................
from nltk.tokenize import TweetTokenizer
tt = TweetTokenizer()
Porter_stemmer = PorterStemmer()
Lancaster_stemmer = LancasterStemmer()
WordNet_lemmatizer = WordNetLemmatizer()
#nltk.download('averaged_perceptron_tagger')
##Train_path="D:/Data/PAN2015-Profiling/pan15-author-profiling-training-dataset-english-2015-04-23"#path for train data
##Test_path="D:/Data/PAN2015-Profiling/pan-ap2015-test/en" #path for test data

##Train_path="D:/NLP/PAN2014/pan14-author-profiling-training-corpus-english-twitter-2014-04-16"#path for train data D:\Data\PAN2013-Profiling\Training\en  D:\NLP\PAN2014
##train_truth_path="D:/NLP/PAN2014/truth_train"
##
##test_truth_path="D:/NLP/PAN2014/pan14_test"
##Test_path="D:/NLP/PAN2014/pan14_test/en"
##Content = open('results_of_clssification_all.txt', 'w')

M_train_file= "/Users/catherine/Desktop/NLP/PAN_Datasets/PAN2014blog/2014.1-blog-txtfiles/PAN2014_blog_male_train.1.txt"
F_train_file= "/Users/catherine/Desktop/NLP/PAN_Datasets/PAN2014blog/2014.1-blog-txtfiles/PAN2014_blog_female_train.1.txt"

M_test_file= "/Users/catherine/Desktop/NLP/PAN_Datasets/PAN2014blog/2014.1-blog-txtfiles/PAN2014_blog_male_test.1.txt"
F_test_file= "/Users/catherine/Desktop/NLP/PAN_Datasets/PAN2014blog/2014.1-blog-txtfiles/PAN2014_blog_female_test.1.txt"

def all_txt(M_txt_file,  F_txt_file):
    with open(M_txt_file, mode="r", encoding="utf-8") as f:
            txt_list = [(line[:-1]).split("\t") for line in f]
            male_dict = dict()
            content=[]
            for line in txt_list:
                if line[1] in male_dict:
                # append the new number to the existing array at this slot
                    male_dict[line[1]].append(line[3])
                else:
                # create a new array in this slot
                    male_dict[line[1]] = [line[3]]
    male_content= [("M", (' '.join(a)).split()) for a in list(male_dict.values())]
  
    with open(F_txt_file, mode="r", encoding="utf-8") as f:
            txt_list = [(line[:-1]).split("\t") for line in f]
            female_dict = dict()
            content=[]
            for line in txt_list:
                if line[1] in female_dict:
                # append the new number to the existing array at this slot
                    female_dict[line[1]].append(line[3])
                else:
                # create a new array in this slot
                    female_dict[line[1]] = [line[3]]
    female_content= [("F", (' '.join(a)).split()) for a in list(female_dict.values())]
    all_txt_per_person = male_content + female_content
    return all_txt_per_person

##print( [((all_txt(M_txt_file,  F_txt_file))[i][0])  for i in range(len((all_txt(M_txt_file,  F_txt_file))))]) helps to determine the gender....

      
"""
create feature vector from the training set.....................................................................................
"""

def WordFeatures(word_list, all_training_text):

    fvs_words = np.array([[author.count(word) for word in word_list] for author in all_training_text]).astype(np.float64)

    # normalise by dividing each row by number of tokens for each author........
    fvs_words /= np.c_[np.array([len(author) for author in all_training_text])]

    return fvs_words



"""
From the saved csv file, recover the saved features to be used...............................................................
"""
import csv
word_list=[]
##txt_files =[ "/Users/catherine/Desktop/NLP/PAN Datasets/PAN_chosen_tokens/2014/output2014OR.1M.txt", "/Users/catherine/Desktop/NLP/PAN Datasets/PAN_chosen_tokens/2014/output2014OR.2F.txt"]
txt_files =[ "//Users/catherine/Desktop/NLP/PAN_Datasets/PAN2014blog/2014.1-blog-txtfiles/PAN2014_blog_female_chosen_set.1.csv", "/Users/catherine/Desktop/NLP/PAN_Datasets/PAN2014blog/2014.1-blog-txtfiles/PAN2014_blog_male_chosen_set.1.csv"]
print("txt_files of features used",txt_files)
##for txt_file in txt_files:
##    with open(txt_file, mode="r", encoding="utf-8") as f:
##        reader = csv.reader(f, delimiter=",")
##        next(reader) # skip header
##        word_list1 =  ([(r.split(":"))[1] for r in f])
##        print(txt_file, len(word_list1))
##        word_list = word_list + word_list1
##print("length of features used =", len(word_list))

for txt_file in txt_files:
    with open(txt_file, mode="r", encoding="utf-8") as f:
        reader = csv.reader(f, delimiter=",")
        next(reader) # skip header
        word_list1 =  [r[0] for r in reader]
        print(txt_file, len(word_list1))
        word_list = word_list + word_list1
print("length of features used =", len(word_list))


"""
Prepare the training and test sets to be parsed to the classifies............................................................
"""
A=(all_txt(M_train_file,  F_train_file))
all_training_text = ( [(A[i][1])  for i in range(len(A))])
print("len(all_training_text)",len(all_training_text))
X_train=WordFeatures(word_list, all_training_text)
y_train=np.array( [(A[i][0])  for i in range(len(A))])
print("len(X_train)=",len(X_train), "len(y_train)=", len(y_train))

B=(all_txt(M_test_file,  F_test_file))
all_test_text = ( [(B[i][1])  for i in range(len(B))])
print("len(all_test_text)",len(all_test_text))
X_test = WordFeatures(word_list, all_test_text)
y_test = np.array( [(B[i][0])  for i in range(len(B))])
print("len(X_test=",len(X_test), "len(y_test)=",len(y_test))


X_train = np.nan_to_num(X_train)
X_test = np.nan_to_num(X_test)


k=5

print(k)
"""
8. manhattan K-Nearest Neighbours..............................................................................
"""
print("manhattan_k=5************************************ ****************************")
from sklearn.neighbors import KNeighborsClassifier
knn = KNeighborsClassifier(n_neighbors = k, metric= "manhattan")
knn.fit(X_train, y_train)
y_pred = knn.predict(X_test)
print("manhattan K-Nearest Neighbours")
manhattan_k5fs=([(i,j) for i, j in zip(y_pred, y_test)])
print(metrics.accuracy_score(y_test, y_pred))
print("************************************ ****************************")

k=13

print(k)
"""
8. manhattan K-Nearest Neighbours..............................................................................
"""
print("manhattan_k=13************************************ ****************************")
from sklearn.neighbors import KNeighborsClassifier
knn = KNeighborsClassifier(n_neighbors = k, metric= "manhattan")
knn.fit(X_train, y_train)
y_pred = knn.predict(X_test)
print("manhattan K-Nearest Neighbours")
manhattan_k13fs=([(i,j) for i, j in zip(y_pred, y_test)])
print(metrics.accuracy_score(y_test, y_pred))
print("****************************************************************")


from sklearn import model_selection, naive_bayes, svm
from sklearn.linear_model import SGDClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import LinearSVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.neural_network import MLPClassifier


print("MLPClassifier****************************************************************")
clf = MLPClassifier()
clf = clf.fit(X_train, y_train)
#Predict the response for test dataset
y_pred = clf.predict(X_test)
MLPClassifierfs=([(i,j) for i, j in zip(y_pred, y_test)])
print(metrics.accuracy_score(y_test, y_pred))


print("LogisticRegression****************************************************************")
clf = LogisticRegression()
clf = clf.fit(X_train, y_train)
#Predict the response for test dataset
y_pred = clf.predict(X_test)
LogisticRegressionfs=([(i,j) for i, j in zip(y_pred, y_test)])
print(metrics.accuracy_score(y_test, y_pred))


print("SGDClassifier****************************************************************")
clf = SGDClassifier()
clf = clf.fit(X_train, y_train)
#Predict the response for test dataset
y_pred = clf.predict(X_test)
SGDClassifierfs=([(i,j) for i, j in zip(y_pred, y_test)])
print(metrics.accuracy_score(y_test, y_pred))


print("DecisionTreeClassifier****************************************************************")
# Create Decision Tree classifer object
clf = DecisionTreeClassifier()
# Train Decision Tree Classifer
clf = clf.fit(X_train, y_train)
#Predict the response for test dataset
y_pred = clf.predict(X_test)
DecisionTreeClassifierfs=([(i,j) for i, j in zip(y_pred, y_test)])
print(metrics.accuracy_score(y_test, y_pred))


print("Naive****************************************************************")
Naive = naive_bayes.MultinomialNB()
Naive.fit(X_train, y_train)
y_pred = Naive.predict(X_test)
Naivefs=([(i,j) for i, j in zip(y_pred, y_test)])
print(metrics.accuracy_score(y_test, y_pred))


print("SVM****************************************************************")
SVM = svm.SVC(C=1.0, kernel='linear', degree=3, gamma='auto')
SVM.fit(X_train, y_train)
y_pred = SVM.predict(X_test)
SVMfs=([(i,j) for i, j in zip(y_pred, y_test)])
print(metrics.accuracy_score(y_test, y_pred))


print("RandomForest_100****************************************************************")
clf=RandomForestClassifier(n_estimators=100)
#Train the model using the training sets y_pred=clf.predict(X_test)
clf.fit(X_train,y_train)
y_pred=clf.predict(X_test)
RandomForest_100fs=([(i,j) for i, j in zip(y_pred, y_test)])
print(metrics.accuracy_score(y_test, y_pred))

print("RandomForest_200****************************************************************")
clf=RandomForestClassifier(n_estimators=200)
#Train the model using the training sets y_pred=clf.predict(X_test)
clf.fit(X_train,y_train)
y_pred=clf.predict(X_test)
RandomForest_200fs=([(i,j) for i, j in zip(y_pred, y_test)])
print(metrics.accuracy_score(y_test, y_pred))

from xgboost.sklearn import XGBClassifier 
print("XGBClassifier****************************************************************")
clf=XGBClassifier()
#Train the model using the training sets y_pred=clf.predict(X_test)
clf.fit(X_train, y_train)
y_pred=clf.predict(X_test)
XGBClassifierfs=([(i,j) for i, j in zip(y_pred, y_test)])
print(metrics.accuracy_score(y_test, y_pred))



print("BaggingClassifier****************************************************************")
from sklearn.ensemble.bagging import BaggingClassifier
clf=BaggingClassifier()
#Train the model using the training sets y_pred=clf.predict(X_test)
clf.fit(X_train, y_train)
y_pred=clf.predict(X_test)
BaggingClassifierfs=([(i,j) for i, j in zip(y_pred, y_test)])
print(metrics.accuracy_score(y_test, y_pred))


print("AdaBoostClassifier****************************************************************")
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
clf=AdaBoostClassifier()
#Train the model using the training sets y_pred=clf.predict(X_test)
clf.fit(X_train, y_train)
y_pred=clf.predict(X_test)
AdaBoostClassifierfs=([(i,j) for i, j in zip(y_pred, y_test)])
print(metrics.accuracy_score(y_test, y_pred))




txt_files of features used ['//Users/catherine/Desktop/NLP/PAN_Datasets/PAN2014blog/2014.1-blog-txtfiles/PAN2014_blog_female_chosen_set.1.csv', '/Users/catherine/Desktop/NLP/PAN_Datasets/PAN2014blog/2014.1-blog-txtfiles/PAN2014_blog_male_chosen_set.1.csv']
//Users/catherine/Desktop/NLP/PAN_Datasets/PAN2014blog/2014.1-blog-txtfiles/PAN2014_blog_female_chosen_set.1.csv 1526
/Users/catherine/Desktop/NLP/PAN_Datasets/PAN2014blog/2014.1-blog-txtfiles/PAN2014_blog_male_chosen_set.1.csv 2063
length of features used = 3589
len(all_training_text) 147
len(X_train)= 147 len(y_train)= 147
len(all_test_text) 78
len(X_test= 78 len(y_test)= 78
5
manhattan_k=5************************************ ****************************
manhattan K-Nearest Neighbours
0.6794871794871795
************************************ ****************************
13
manhattan_k=13************************************ ****************************
manhattan K-Nearest Neighbours
0.7051282051282052
*********************************

/anaconda3/lib/python3.7/site-packages/sklearn/neural_network/multilayer_perceptron.py:564: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)
/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/stochastic_gradient.py:128: FutureWarning: max_iter and tol parameters have been added in <class 'sklearn.linear_model.stochastic_gradient.SGDClassifier'> in 0.19. If both are left unset, they default to max_iter=5 and tol=None. If tol is not None, max_iter defaults to max_iter=1000. From 0.21, default max_iter will be 1000, and default tol will be 1e-3.
  "and default tol will be 1e-3." % type(self), FutureWarning)


0.717948717948718
LogisticRegression****************************************************************
0.5384615384615384
SGDClassifier****************************************************************
0.5
DecisionTreeClassifier****************************************************************
0.5897435897435898
Naive****************************************************************
0.5512820512820513
SVM****************************************************************
0.5
RandomForest_100****************************************************************
0.5897435897435898
RandomForest_200****************************************************************
0.6025641025641025
XGBClassifier****************************************************************


/anaconda3/lib/python3.7/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


0.5897435897435898
BaggingClassifier****************************************************************
0.6025641025641025
AdaBoostClassifier****************************************************************
0.5256410256410257


In [13]:
print("The sign test...****************************************************************")
print("manhattan_k=5************************************ ****************************")
x= MLPClassifierfs 
y= manhattan_k5fs

positive=0
negative=0
for i in range(len(x)):
    if (x[i]!=y[i]):
        if (x[i][0]==x[i][1] and y[i][0]!=y[i][1]):
            positive=positive+1
        else: 
            negative=negative+1
print("positive=",positive, "negative =" , negative)
        
print("manhattan_k=13************************************ ****************************")
x= MLPClassifierfs 
y= manhattan_k13fs

positive=0
negative=0
for i in range(len(x)):
    if (x[i]!=y[i]):
        if (x[i][0]==x[i][1] and y[i][0]!=y[i][1]):
            positive=positive+1
        else: 
            negative=negative+1
print("positive=",positive, "negative =" , negative)

print("MLPClassifier****************************************************************")
x= MLPClassifierfs 
y= MLPClassifierfs

positive=0
negative=0
for i in range(len(x)):
    if (x[i]!=y[i]):
        if (x[i][0]==x[i][1] and y[i][0]!=y[i][1]):
            positive=positive+1
        else: 
            negative=negative+1
print("positive=",positive, "negative =" , negative)




print("LogisticRegression****************************************************************")
x= MLPClassifierfs 
y= LogisticRegressionfs

positive=0
negative=0
for i in range(len(x)):
    if (x[i]!=y[i]):
        if (x[i][0]==x[i][1] and y[i][0]!=y[i][1]):
            positive=positive+1
        else: 
            negative=negative+1
print("positive=",positive, "negative =" , negative)




print("SGDClassifier****************************************************************")
x= MLPClassifierfs 
y= SGDClassifierfs

positive=0
negative=0
for i in range(len(x)):
    if (x[i]!=y[i]):
        if (x[i][0]==x[i][1] and y[i][0]!=y[i][1]):
            positive=positive+1
        else: 
            negative=negative+1
print("positive=",positive, "negative =" , negative)



print("DecisionTreeClassifier****************************************************************")
x= MLPClassifierfs 
y= DecisionTreeClassifierfs

positive=0
negative=0
for i in range(len(x)):
    if (x[i]!=y[i]):
        if (x[i][0]==x[i][1] and y[i][0]!=y[i][1]):
            positive=positive+1
        else: 
            negative=negative+1
print("positive=",positive, "negative =" , negative)



print("Naive****************************************************************")
x= MLPClassifierfs 
y= Naivefs

positive=0
negative=0
for i in range(len(x)):
    if (x[i]!=y[i]):
        if (x[i][0]==x[i][1] and y[i][0]!=y[i][1]):
            positive=positive+1
        else: 
            negative=negative+1
print("positive=",positive, "negative =" , negative)



print("SVM****************************************************************")
x= MLPClassifierfs 
y= SVMfs

positive=0
negative=0
for i in range(len(x)):
    if (x[i]!=y[i]):
        if (x[i][0]==x[i][1] and y[i][0]!=y[i][1]):
            positive=positive+1
        else: 
            negative=negative+1
print("positive=",positive, "negative =" , negative)



print("RandomForest_100****************************************************************")
x= MLPClassifierfs 
y= RandomForest_100fs

positive=0
negative=0
for i in range(len(x)):
    if (x[i]!=y[i]):
        if (x[i][0]==x[i][1] and y[i][0]!=y[i][1]):
            positive=positive+1
        else: 
            negative=negative+1
print("positive=",positive, "negative =" , negative)



print("RandomForest_200****************************************************************")
x= MLPClassifierfs 
y= RandomForest_200fs

positive=0
negative=0
for i in range(len(x)):
    if (x[i]!=y[i]):
        if (x[i][0]==x[i][1] and y[i][0]!=y[i][1]):
            positive=positive+1
        else: 
            negative=negative+1
print("positive=",positive, "negative =" , negative)




print("XGBClassifier****************************************************************")
x= MLPClassifierfs 
y= XGBClassifierfs

positive=0
negative=0
for i in range(len(x)):
    if (x[i]!=y[i]):
        if (x[i][0]==x[i][1] and y[i][0]!=y[i][1]):
            positive=positive+1
        else: 
            negative=negative+1
print("positive=",positive, "negative =" , negative)




print("BaggingClassifier****************************************************************")
x= MLPClassifierfs 
y= BaggingClassifierfs

positive=0
negative=0
for i in range(len(x)):
    if (x[i]!=y[i]):
        if (x[i][0]==x[i][1] and y[i][0]!=y[i][1]):
            positive=positive+1
        else: 
            negative=negative+1
print("positive=",positive, "negative =" , negative)



print("AdaBoostClassifier****************************************************************")
x= MLPClassifierfs 
y= AdaBoostClassifierfs

positive=0
negative=0
for i in range(len(x)):
    if (x[i]!=y[i]):
        if (x[i][0]==x[i][1] and y[i][0]!=y[i][1]):
            positive=positive+1
        else: 
            negative=negative+1
print("positive=",positive, "negative =" , negative)


The sign test...****************************************************************
manhattan_k=5************************************ ****************************
positive= 10 negative = 7
manhattan_k=13************************************ ****************************
positive= 5 negative = 4
MLPClassifier****************************************************************
positive= 0 negative = 0
LogisticRegression****************************************************************
positive= 22 negative = 8
SGDClassifier****************************************************************
positive= 28 negative = 11
DecisionTreeClassifier****************************************************************
positive= 20 negative = 10
Naive****************************************************************
positive= 21 negative = 8
SVM****************************************************************
positive= 28 negative = 11
RandomForest_100****************************************************************
positi

In [56]:
print("BaggingClassifier****************************************************************")
from sklearn.ensemble.bagging import BaggingClassifier
clf=BaggingClassifier()
#Train the model using the training sets y_pred=clf.predict(X_test)
clf.fit(X_train, y_train)
y_pred=clf.predict(X_test)
print(metrics.accuracy_score(y_test, y_pred))


print("AdaBoostClassifier****************************************************************")
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
clf=AdaBoostClassifier()
#Train the model using the training sets y_pred=clf.predict(X_test)
clf.fit(X_train, y_train)
y_pred=clf.predict(X_test)
print(metrics.accuracy_score(y_test, y_pred))



BaggingClassifier****************************************************************
0.6025641025641025
AdaBoostClassifier****************************************************************
0.5256410256410257


In [6]:
#..................................................................................

data_tags = ["type","gender","tweet"]
train_data_list = []
i=0
for f in y_train:
   
    train_data_list.append((f, y_train[i],all_training_text[i]))
    i += 1
# We have training data available as dictionary filename, category, data
train_data = pd.DataFrame.from_records(train_data_list, columns=data_tags)




#..................................................................................

test_data_list = []
ii=0
for f in y_test:
   
    test_data_list.append((f, y_test[ii], all_test_text[ii]))
    ii += 1
# We have training data available as dictionary filename, category, data
test_data = pd.DataFrame.from_records(test_data_list, columns=data_tags)

#..................................................................................


import pandas as pd
import numpy as np
import pickle
from keras.preprocessing.text import Tokenizer
from keras.models import Sequential
from keras.layers import Activation, Dense, Dropout
from sklearn.preprocessing import LabelBinarizer
import sklearn.datasets as skds
from pathlib import Path


train_posts = train_data['tweet']
train_tags = train_data['type']

test_posts = test_data['tweet']
test_tags = test_data['type']




# :Tokenize and Prepare Vocabulary
num_labels = 2
#vocab_size = 15000#len(word_list)#most common number of words will be then kept for use in the vector
vocab_size = 1000#by changing the vocubulary size... acc count_type =  0.932258064516129#acc count_gender = 0.8298387096774194
#vocab_size = 100#acc count_type =0.9274193548387096 acc count_gender = 0.792741935483871
batch_size = 100
 
# define Tokenizer with Vocab Size
tokenizer = Tokenizer(num_words=vocab_size)#IF I USE OTHER TOKENIZER...
tokenizer.fit_on_texts(train_posts)
#print(tokenizer.word_index)
#print((tokenizer.word_counts))#provides a dictionary of the words and the count......................
#sorted_x = sorted((tokenizer.word_counts).items(), key=operator.itemgetter(1),reverse=True)
#print(sorted_x)
#print((tokenizer.document_count))#number of documents..............
#print((tokenizer.word_docs))#provides a dictionary of the words and the number of documents they appear in......................
print('Found %d unique words.' % len(tokenizer.word_index))#shows total vocubulary of the text dataset
x_train = tokenizer.texts_to_matrix(train_posts, mode='tfidf')#WHAT HAPPENS WHEN I GIVE IT THE HAND CRAFTED TOKENS??
x_test = tokenizer.texts_to_matrix(test_posts, mode='tfidf')

##--------what about applying knn at this point-------------------------------
##--------what about changing the vocubulary size-------------------------------works well with reduced sized
##----------One popular method for hyperparameter optimization is grid search.-----------
##-----determine the best set of parameters with the highest accuracy..........this is for the keras model------


    
    
encoder = LabelBinarizer()


encoder.fit(train_tags)

y_train = encoder.transform(train_tags)#same as the y train generated with my model
y_train = np.hstack((y_train, 1 - y_train))#used for two label cases........

y_test = encoder.transform(test_tags)
y_test = np.hstack((y_test, 1 - y_test))

"""
fit_on_texts Updates internal vocabulary based on a list of texts. This method creates the vocabulary index based on word frequency. So if you give it something like, "The cat sat on the mat." It will create a dictionary s.t. word_index["the"] = 0; word_index["cat"] = 1 it is word -> index dictionary so every word gets a unique integer value. So lower integer means more frequent word (often the first few are punctuation because they appear a lot).
texts_to_sequences Transforms each text in texts to a sequence of integers. So it basically takes each word in the text and replaces it with its corresponding integer value from the word_index dictionary. Nothing more, nothing less, certainly no magic involved.
CLEAN TEXT TO THE DESIRED LEVEL AND USE KERAS INBUILF TFIDF TO GENERATE A MATRIX....
"""

from keras.models import *
from keras.layers import *

model = Sequential()
model.add(Dense(512, input_shape=(vocab_size,)))#512 neurons in the first hidden layer
model.add(Activation('relu'))
#model.add(Dropout(0.3))
model.add(Dense(512))
model.add(Activation('relu'))
#model.add(Dropout(0.3))
model.add(Dense(num_labels))
model.add(Activation('softmax'))
model.summary()
 
model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

history = model.fit(x_train, y_train,
                    batch_size=batch_size,
                    epochs=30,
                    verbose=1,
                    validation_split=0.1)



#Evaluate model.............................
score = model.evaluate(x_test, y_test,
                       batch_size=batch_size, verbose=1)
 
print('Test accuracy:', score[1])

Found 24093 unique words.


ValueError: Unknown label type: (0      [1, 0]
1      [1, 0]
2      [1, 0]
3      [1, 0]
4      [1, 0]
5      [1, 0]
6      [1, 0]
7      [1, 0]
8      [1, 0]
9      [1, 0]
10     [1, 0]
11     [1, 0]
12     [1, 0]
13     [1, 0]
14     [1, 0]
15     [1, 0]
16     [1, 0]
17     [1, 0]
18     [1, 0]
19     [1, 0]
20     [1, 0]
21     [1, 0]
22     [1, 0]
23     [1, 0]
24     [1, 0]
25     [1, 0]
26     [1, 0]
27     [1, 0]
28     [1, 0]
29     [1, 0]
        ...  
117    [0, 1]
118    [0, 1]
119    [0, 1]
120    [0, 1]
121    [0, 1]
122    [0, 1]
123    [0, 1]
124    [0, 1]
125    [0, 1]
126    [0, 1]
127    [0, 1]
128    [0, 1]
129    [0, 1]
130    [0, 1]
131    [0, 1]
132    [0, 1]
133    [0, 1]
134    [0, 1]
135    [0, 1]
136    [0, 1]
137    [0, 1]
138    [0, 1]
139    [0, 1]
140    [0, 1]
141    [0, 1]
142    [0, 1]
143    [0, 1]
144    [0, 1]
145    [0, 1]
146    [0, 1]
Name: type, Length: 147, dtype: object,)

In [16]:
import sys
sys.path.append("/anaconda3/lib/python3.7/site-packages")
import numpy as np
import numpy
import nltk
import pickle
import pandas as pd
import glob
import os
import re
import operator
from collections import Counter
#from read_xml_files import *
##from s_stemmer import *
from nltk.stem.porter import PorterStemmer
from nltk.stem import LancasterStemmer, WordNetLemmatizer
from itertools import groupby
from nltk.collocations import *
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.cluster import KMeans
from scipy.cluster.vq import whiten
from nltk.tokenize.toktok import ToktokTokenizer
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2
import xml.etree.ElementTree as ET
import pandas as pd
#Import scikit-learn metrics module for accuracy calculation
from sklearn import metrics
##from pattern.text.en import singularize
##nltk.download('vader_lexicon')
##nltk.download('punkt')
#import the relevant modules from the NLTK library
from nltk.sentiment.vader import SentimentIntensityAnalyzer
sentence_tokenizer = nltk.data.load('tokenizers/punkt/english.pickle')
word_tokenizer = nltk.tokenize.RegexpTokenizer(r'\w+')
##tt = ToktokTokenizer()#keeps the url as is...................................................
from nltk.tokenize import TweetTokenizer
tt = TweetTokenizer()
Porter_stemmer = PorterStemmer()
Lancaster_stemmer = LancasterStemmer()
WordNet_lemmatizer = WordNetLemmatizer()
#nltk.download('averaged_perceptron_tagger')
##Train_path="D:/Data/PAN2015-Profiling/pan15-author-profiling-training-dataset-english-2015-04-23"#path for train data
##Test_path="D:/Data/PAN2015-Profiling/pan-ap2015-test/en" #path for test data

##Train_path="D:/NLP/PAN2014/pan14-author-profiling-training-corpus-english-twitter-2014-04-16"#path for train data D:\Data\PAN2013-Profiling\Training\en  D:\NLP\PAN2014
##train_truth_path="D:/NLP/PAN2014/truth_train"
##
##test_truth_path="D:/NLP/PAN2014/pan14_test"
##Test_path="D:/NLP/PAN2014/pan14_test/en"
##Content = open('results_of_clssification_all.txt', 'w')

M_train_file= "/Users/catherine/Desktop/NLP/PAN_Datasets/PAN2014blog/2014.1-blog-txtfiles/PAN2014_blog_male_train.1.txt"
F_train_file= "/Users/catherine/Desktop/NLP/PAN_Datasets/PAN2014blog/2014.1-blog-txtfiles/PAN2014_blog_female_train.1.txt"

M_test_file= "/Users/catherine/Desktop/NLP/PAN_Datasets/PAN2014blog/2014.1-blog-txtfiles/PAN2014_blog_male_test.1.txt"
F_test_file= "/Users/catherine/Desktop/NLP/PAN_Datasets/PAN2014blog/2014.1-blog-txtfiles/PAN2014_blog_female_test.1.txt"

def all_txt(M_txt_file,  F_txt_file):
    with open(M_txt_file, mode="r", encoding="utf-8") as f:
            txt_list = [(line[:-1]).split("\t") for line in f]
            male_dict = dict()
            content=[]
            for line in txt_list:
                if line[1] in male_dict:
                # append the new number to the existing array at this slot
                    male_dict[line[1]].append(line[3])
                else:
                # create a new array in this slot
                    male_dict[line[1]] = [line[3]]
    male_content= [("M", (' '.join(a)).split()) for a in list(male_dict.values())]
  
    with open(F_txt_file, mode="r", encoding="utf-8") as f:
            txt_list = [(line[:-1]).split("\t") for line in f]
            female_dict = dict()
            content=[]
            for line in txt_list:
                if line[1] in female_dict:
                # append the new number to the existing array at this slot
                    female_dict[line[1]].append(line[3])
                else:
                # create a new array in this slot
                    female_dict[line[1]] = [line[3]]
    female_content= [("F", (' '.join(a)).split()) for a in list(female_dict.values())]
    all_txt_per_person = male_content + female_content
    return all_txt_per_person

##print( [((all_txt(M_txt_file,  F_txt_file))[i][0])  for i in range(len((all_txt(M_txt_file,  F_txt_file))))]) helps to determine the gender....

      
"""
create feature vector from the training set.....................................................................................
"""

def WordFeatures(word_list, all_training_text):

    fvs_words = np.array([[author.count(word) for word in word_list] for author in all_training_text]).astype(np.float64)

    # normalise by dividing each row by number of tokens for each author........
    fvs_words /= np.c_[np.array([len(author) for author in all_training_text])]

    return fvs_words



"""
From the saved csv file, recover the saved features to be used...............................................................
"""
import csv
word_list=[]
##txt_files =[ "/Users/catherine/Desktop/NLP/PAN Datasets/PAN_chosen_tokens/2014/output2014OR.1M.txt", "/Users/catherine/Desktop/NLP/PAN Datasets/PAN_chosen_tokens/2014/output2014OR.2F.txt"]
txt_files =[ "//Users/catherine/Desktop/NLP/PAN_Datasets/PAN2014blog/2014.1-blog-txtfiles/PAN2014_blog_female_chosen_set.1.csv", "/Users/catherine/Desktop/NLP/PAN_Datasets/PAN2014blog/2014.1-blog-txtfiles/PAN2014_blog_male_chosen_set.1.csv"]
print("txt_files of features used",txt_files)
##for txt_file in txt_files:
##    with open(txt_file, mode="r", encoding="utf-8") as f:
##        reader = csv.reader(f, delimiter=",")
##        next(reader) # skip header
##        word_list1 =  ([(r.split(":"))[1] for r in f])
##        print(txt_file, len(word_list1))
##        word_list = word_list + word_list1
##print("length of features used =", len(word_list))

for txt_file in txt_files:
    with open(txt_file, mode="r", encoding="utf-8") as f:
        reader = csv.reader(f, delimiter=",")
        next(reader) # skip header
        word_list1 =  [r[0] for r in reader]
        print(txt_file, len(word_list1))
        word_list = word_list + word_list1
print("length of features used =", len(word_list))


"""
Prepare the training and test sets to be parsed to the classifies............................................................
"""
A=(all_txt(M_train_file,  F_train_file))
all_training_text = ( [(A[i][1])  for i in range(len(A))])
print("len(all_training_text)",len(all_training_text))
#X_train=WordFeatures(word_list, all_training_text)
y_train=np.array( [(A[i][0])  for i in range(len(A))])
#print("len(X_train)=",len(X_train), "len(y_train)=", len(y_train))

B=(all_txt(M_test_file,  F_test_file))
all_test_text = ( [(B[i][1])  for i in range(len(B))])
print("len(all_test_text)",len(all_test_text))
#X_test = WordFeatures(word_list, all_test_text)
y_test = np.array( [(B[i][0])  for i in range(len(B))])
#print("len(X_test=",len(X_test), "len(y_test)=",len(y_test))


X_train = np.nan_to_num(X_train)
X_test = np.nan_to_num(X_test)




#..................................................................................

data_tags = ["type","gender","tweet"]
train_data_list = []
i=0
for f in y_train:
   
    train_data_list.append((f, y_train[i],all_training_text[i]))
    i += 1
# We have training data available as dictionary filename, category, data
train_data = pd.DataFrame.from_records(train_data_list, columns=data_tags)




#..................................................................................

test_data_list = []
ii=0
for f in y_test:
   
    test_data_list.append((f, y_test[ii], all_test_text[ii]))
    ii += 1
# We have training data available as dictionary filename, category, data
test_data = pd.DataFrame.from_records(test_data_list, columns=data_tags)

#..................................................................................


import pandas as pd
import numpy as np
import pickle
from keras.preprocessing.text import Tokenizer
from keras.models import Sequential
from keras.layers import Activation, Dense, Dropout
from sklearn.preprocessing import LabelBinarizer
import sklearn.datasets as skds
from pathlib import Path

# :Tokenize and Prepare Vocabulary

train_posts = train_data['tweet']
train_tags = train_data['type']

test_posts = test_data['tweet']
test_tags = test_data['type']


num_labels = 2
vocab_size = 1000#len(word_list)#most common number of words will be then kept for use in the vector
#vocab_size = 1000#by changing the vocubulary size... acc count_type =  0.932258064516129#acc count_gender = 0.8298387096774194
#vocab_size = 100#acc count_type =0.9274193548387096 acc count_gender = 0.792741935483871
batch_size = 100
 
# define Tokenizer with Vocab Size
tokenizer = Tokenizer(num_words=vocab_size)#IF I USE OTHER TOKENIZER...
tokenizer.fit_on_texts(train_posts)
#print(tokenizer.word_index)
#print((tokenizer.word_counts))#provides a dictionary of the words and the count......................
#sorted_x = sorted((tokenizer.word_counts).items(), key=operator.itemgetter(1),reverse=True)
#print(sorted_x)
#print((tokenizer.document_count))#number of documents..............
#print((tokenizer.word_docs))#provides a dictionary of the words and the number of documents they appear in......................
print('Found %d unique words.' % len(tokenizer.word_index))#shows total vocubulary of the text dataset
x_train = tokenizer.texts_to_matrix(train_posts, mode='tfidf')#WHAT HAPPENS WHEN I GIVE IT THE HAND CRAFTED TOKENS??
x_test = tokenizer.texts_to_matrix(test_posts, mode='tfidf')

##--------what about applying knn at this point-------------------------------
##--------what about changing the vocubulary size-------------------------------works well with reduced sized
##----------One popular method for hyperparameter optimization is grid search.-----------
##-----determine the best set of parameters with the highest accuracy..........this is for the keras model------


      
encoder = LabelBinarizer()


encoder.fit(train_tags)

y_train = encoder.transform(train_tags)#same as the y train generated with my model
y_train = np.hstack((y_train, 1 - y_train))#used for two label cases........

y_test = encoder.transform(test_tags)
y_test = np.hstack((y_test, 1 - y_test))


"""
fit_on_texts Updates internal vocabulary based on a list of texts. This method creates the vocabulary index based on word frequency. So if you give it something like, "The cat sat on the mat." It will create a dictionary s.t. word_index["the"] = 0; word_index["cat"] = 1 it is word -> index dictionary so every word gets a unique integer value. So lower integer means more frequent word (often the first few are punctuation because they appear a lot).
texts_to_sequences Transforms each text in texts to a sequence of integers. So it basically takes each word in the text and replaces it with its corresponding integer value from the word_index dictionary. Nothing more, nothing less, certainly no magic involved.
CLEAN TEXT TO THE DESIRED LEVEL AND USE KERAS INBUILF TFIDF TO GENERATE A MATRIX....
"""
from sklearn import model_selection, naive_bayes, svm
from sklearn.linear_model import SGDClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import LinearSVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.neural_network import MLPClassifier


#8. manhattan K-Nearest Neighbours..............................................................................
k=5

print(k)
"""
8. manhattan K-Nearest Neighbours..............................................................................
"""
print("manhattan_k=5************************************ ****************************")
from sklearn.neighbors import KNeighborsClassifier
knn = KNeighborsClassifier(n_neighbors = k, metric= "manhattan")
knn.fit(x_train, train_tags)
y_pred = knn.predict(x_test)
manhattan_k5tfidf=([(i,j) for i, j in zip(y_pred, test_tags)])
print("manhattan K-Nearest Neighbours")
print(metrics.accuracy_score(test_tags, y_pred))
print("************************************ ****************************")

k=13

print(k)
"""
8. manhattan K-Nearest Neighbours..............................................................................
"""
print("manhattan_k=13************************************ ****************************")
from sklearn.neighbors import KNeighborsClassifier
knn = KNeighborsClassifier(n_neighbors = k, metric= "manhattan")
knn.fit(x_train, train_tags)
y_pred = knn.predict(x_test)
manhattan_k13tfidf=([(i,j) for i, j in zip(y_pred, test_tags)])
print("manhattan K-Nearest Neighbours")
print(metrics.accuracy_score(test_tags, y_pred))
print("****************************************************************")


from sklearn import model_selection, naive_bayes, svm
from sklearn.linear_model import SGDClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import LinearSVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.neural_network import MLPClassifier


print("MLPClassifier****************************************************************")
clf = MLPClassifier()
clf = clf.fit(x_train, train_tags)
#Predict the response for test dataset
y_pred = clf.predict(x_test)
MLPClassifiertfidf=([(i,j) for i, j in zip(y_pred, test_tags)])
print(metrics.accuracy_score(test_tags, y_pred))


print("LogisticRegression****************************************************************")
clf = LogisticRegression()
clf = clf.fit(x_train, train_tags)
#Predict the response for test dataset
y_pred = clf.predict(x_test)
LogisticRegressiontfidf=([(i,j) for i, j in zip(y_pred, test_tags)])
print(metrics.accuracy_score(test_tags, y_pred))


print("SGDClassifier****************************************************************")
clf = SGDClassifier()
clf = clf.fit(x_train, train_tags)
#Predict the response for test dataset
y_pred = clf.predict(x_test)
SGDClassifiertfidf=([(i,j) for i, j in zip(y_pred, test_tags)])
print(metrics.accuracy_score(test_tags, y_pred))


print("DecisionTreeClassifier****************************************************************")
# Create Decision Tree classifer object
clf = DecisionTreeClassifier()
# Train Decision Tree Classifer
clf = clf.fit(x_train, train_tags)
#Predict the response for test dataset
y_pred = clf.predict(x_test)
DecisionTreeClassifiertfidf=([(i,j) for i, j in zip(y_pred, test_tags)])
print(metrics.accuracy_score(test_tags, y_pred))


print("Naive****************************************************************")
Naive = naive_bayes.MultinomialNB()
Naive.fit(x_train, train_tags)
y_pred = Naive.predict(x_test)
Naivetfidf=([(i,j) for i, j in zip(y_pred, test_tags)])
print(metrics.accuracy_score(test_tags, y_pred))


print("SVM****************************************************************")
SVM = svm.SVC(C=1.0, kernel='linear', degree=3, gamma='auto')
SVM.fit(x_train, train_tags)
y_pred = SVM.predict(x_test)
SVMtfidf=([(i,j) for i, j in zip(y_pred, test_tags)])
print(metrics.accuracy_score(test_tags, y_pred))


print("RandomForest_100****************************************************************")
clf=RandomForestClassifier(n_estimators=100)
#Train the model using the training sets y_pred=clf.predict(X_test)
clf.fit(x_train,train_tags)
y_pred=clf.predict(x_test)
RandomForest_100tfidf=([(i,j) for i, j in zip(y_pred, test_tags)])
print(metrics.accuracy_score(test_tags, y_pred))

print("RandomForest_200****************************************************************")
clf=RandomForestClassifier(n_estimators=200)
#Train the model using the training sets y_pred=clf.predict(X_test)
clf.fit(x_train,train_tags)
y_pred=clf.predict(x_test)
RandomForest_200tfidf=([(i,j) for i, j in zip(y_pred, test_tags)])
print(metrics.accuracy_score(test_tags, y_pred))

from xgboost.sklearn import XGBClassifier 
print("XGBClassifier****************************************************************")
clf=XGBClassifier()
#Train the model using the training sets y_pred=clf.predict(X_test)
clf.fit(x_train, train_tags)
y_pred=clf.predict(x_test)
XGBClassifiertfidf=([(i,j) for i, j in zip(y_pred, test_tags)])
print(metrics.accuracy_score(test_tags, y_pred))



print("BaggingClassifier****************************************************************")
from sklearn.ensemble.bagging import BaggingClassifier
clf=BaggingClassifier()
#Train the model using the training sets y_pred=clf.predict(X_test)
clf.fit(x_train, train_tags)
y_pred=clf.predict(x_test)
BaggingClassifiertfidf=([(i,j) for i, j in zip(y_pred, test_tags)])
print(metrics.accuracy_score(test_tags, y_pred))


print("AdaBoostClassifier****************************************************************")
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
clf=AdaBoostClassifier()
#Train the model using the training sets y_pred=clf.predict(X_test)
clf.fit(x_train, train_tags)
y_pred=clf.predict(x_test)
AdaBoostClassifiertfidf=([(i,j) for i, j in zip(y_pred, test_tags)])
print(metrics.accuracy_score(test_tags, y_pred))







from keras.models import *
from keras.layers import *

model = Sequential()
model.add(Dense(512, input_shape=(vocab_size,)))#512 neurons in the first hidden layer
model.add(Activation('relu'))
#model.add(Dropout(0.3))
model.add(Dense(512))
model.add(Activation('relu'))
#model.add(Dropout(0.3))
model.add(Dense(num_labels))
model.add(Activation('softmax'))
model.summary()
 
model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

history = model.fit(x_train, y_train,
                    batch_size=batch_size,
                    epochs=30,
                    verbose=1,
                    validation_split=0.1)



#Evaluate model.............................
score = model.evaluate(x_test, y_test,
                       batch_size=batch_size, verbose=1)
 
print('Test accuracy:', score[1])







txt_files of features used ['//Users/catherine/Desktop/NLP/PAN_Datasets/PAN2014blog/2014.1-blog-txtfiles/PAN2014_blog_female_chosen_set.1.csv', '/Users/catherine/Desktop/NLP/PAN_Datasets/PAN2014blog/2014.1-blog-txtfiles/PAN2014_blog_male_chosen_set.1.csv']
//Users/catherine/Desktop/NLP/PAN_Datasets/PAN2014blog/2014.1-blog-txtfiles/PAN2014_blog_female_chosen_set.1.csv 1526
/Users/catherine/Desktop/NLP/PAN_Datasets/PAN2014blog/2014.1-blog-txtfiles/PAN2014_blog_male_chosen_set.1.csv 2063
length of features used = 3589
len(all_training_text) 147
len(all_test_text) 78
Found 24093 unique words.
5
manhattan_k=5************************************ ****************************
manhattan K-Nearest Neighbours
0.5384615384615384
************************************ ****************************
13
manhattan_k=13************************************ ****************************
manhattan K-Nearest Neighbours
0.5256410256410257
****************************************************************
MLPClassi

/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/stochastic_gradient.py:128: FutureWarning: max_iter and tol parameters have been added in <class 'sklearn.linear_model.stochastic_gradient.SGDClassifier'> in 0.19. If both are left unset, they default to max_iter=5 and tol=None. If tol is not None, max_iter defaults to max_iter=1000. From 0.21, default max_iter will be 1000, and default tol will be 1e-3.
  "and default tol will be 1e-3." % type(self), FutureWarning)


0.5641025641025641
RandomForest_200****************************************************************
0.5512820512820513
XGBClassifier****************************************************************


/anaconda3/lib/python3.7/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


0.6282051282051282
BaggingClassifier****************************************************************
0.5641025641025641
AdaBoostClassifier****************************************************************
0.6153846153846154
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_7 (Dense)              (None, 512)               512512    
_________________________________________________________________
activation_7 (Activation)    (None, 512)               0         
_________________________________________________________________
dense_8 (Dense)              (None, 512)               262656    
_________________________________________________________________
activation_8 (Activation)    (None, 512)               0         
_________________________________________________________________
dense_9 (Dense)              (None, 2)                 1026      
____________________________________________________

In [17]:
print("The sign test...****************************************************************")
print("manhattan_k=5************************************ ****************************")
x= SGDClassifiertfidf 
y= manhattan_k5tfidf

positive=0
negative=0
for i in range(len(x)):
    if (x[i]!=y[i]):
        if (x[i][0]==x[i][1] and y[i][0]!=y[i][1]):
            positive=positive+1
        else: 
            negative=negative+1
print("positive=",positive, "negative =" , negative)
        
print("manhattan_k=13************************************ ****************************")
x= SGDClassifiertfidf 
y= manhattan_k13tfidf

positive=0
negative=0
for i in range(len(x)):
    if (x[i]!=y[i]):
        if (x[i][0]==x[i][1] and y[i][0]!=y[i][1]):
            positive=positive+1
        else: 
            negative=negative+1
print("positive=",positive, "negative =" , negative)

print("MLPClassifier****************************************************************")
x= SGDClassifiertfidf 
y= MLPClassifiertfidf

positive=0
negative=0
for i in range(len(x)):
    if (x[i]!=y[i]):
        if (x[i][0]==x[i][1] and y[i][0]!=y[i][1]):
            positive=positive+1
        else: 
            negative=negative+1
print("positive=",positive, "negative =" , negative)




print("LogisticRegression****************************************************************")
x= SGDClassifiertfidf 
y= LogisticRegressiontfidf

positive=0
negative=0
for i in range(len(x)):
    if (x[i]!=y[i]):
        if (x[i][0]==x[i][1] and y[i][0]!=y[i][1]):
            positive=positive+1
        else: 
            negative=negative+1
print("positive=",positive, "negative =" , negative)




print("SGDClassifier****************************************************************")
x= SGDClassifiertfidf 
y= SGDClassifiertfidf

positive=0
negative=0
for i in range(len(x)):
    if (x[i]!=y[i]):
        if (x[i][0]==x[i][1] and y[i][0]!=y[i][1]):
            positive=positive+1
        else: 
            negative=negative+1
print("positive=",positive, "negative =" , negative)



print("DecisionTreeClassifier****************************************************************")
x= SGDClassifiertfidf 
y= DecisionTreeClassifiertfidf

positive=0
negative=0
for i in range(len(x)):
    if (x[i]!=y[i]):
        if (x[i][0]==x[i][1] and y[i][0]!=y[i][1]):
            positive=positive+1
        else: 
            negative=negative+1
print("positive=",positive, "negative =" , negative)



print("Naive****************************************************************")
x= SGDClassifiertfidf 
y= Naivetfidf

positive=0
negative=0
for i in range(len(x)):
    if (x[i]!=y[i]):
        if (x[i][0]==x[i][1] and y[i][0]!=y[i][1]):
            positive=positive+1
        else: 
            negative=negative+1
print("positive=",positive, "negative =" , negative)



print("SVM****************************************************************")
x= SGDClassifiertfidf 
y= SVMtfidf

positive=0
negative=0
for i in range(len(x)):
    if (x[i]!=y[i]):
        if (x[i][0]==x[i][1] and y[i][0]!=y[i][1]):
            positive=positive+1
        else: 
            negative=negative+1
print("positive=",positive, "negative =" , negative)



print("RandomForest_100****************************************************************")
x= SGDClassifiertfidf 
y= RandomForest_100tfidf

positive=0
negative=0
for i in range(len(x)):
    if (x[i]!=y[i]):
        if (x[i][0]==x[i][1] and y[i][0]!=y[i][1]):
            positive=positive+1
        else: 
            negative=negative+1
print("positive=",positive, "negative =" , negative)



print("RandomForest_200****************************************************************")
x= SGDClassifiertfidf
y= RandomForest_200tfidf

positive=0
negative=0
for i in range(len(x)):
    if (x[i]!=y[i]):
        if (x[i][0]==x[i][1] and y[i][0]!=y[i][1]):
            positive=positive+1
        else: 
            negative=negative+1
print("positive=",positive, "negative =" , negative)




print("XGBClassifier****************************************************************")
x= SGDClassifiertfidf 
y= XGBClassifiertfidf

positive=0
negative=0
for i in range(len(x)):
    if (x[i]!=y[i]):
        if (x[i][0]==x[i][1] and y[i][0]!=y[i][1]):
            positive=positive+1
        else: 
            negative=negative+1
print("positive=",positive, "negative =" , negative)




print("BaggingClassifier****************************************************************")
x= SGDClassifiertfidf 
y= BaggingClassifiertfidf

positive=0
negative=0
for i in range(len(x)):
    if (x[i]!=y[i]):
        if (x[i][0]==x[i][1] and y[i][0]!=y[i][1]):
            positive=positive+1
        else: 
            negative=negative+1
print("positive=",positive, "negative =" , negative)



print("AdaBoostClassifier****************************************************************")
x= SGDClassifiertfidf
y= AdaBoostClassifiertfidf

positive=0
negative=0
for i in range(len(x)):
    if (x[i]!=y[i]):
        if (x[i][0]==x[i][1] and y[i][0]!=y[i][1]):
            positive=positive+1
        else: 
            negative=negative+1
print("positive=",positive, "negative =" , negative)








The sign test...****************************************************************
manhattan_k=5************************************ ****************************
positive= 21 negative = 14
manhattan_k=13************************************ ****************************
positive= 20 negative = 12
MLPClassifier****************************************************************
positive= 14 negative = 12
LogisticRegression****************************************************************
positive= 7 negative = 7
SGDClassifier****************************************************************
positive= 0 negative = 0
DecisionTreeClassifier****************************************************************
positive= 16 negative = 11
Naive****************************************************************
positive= 11 negative = 9
SVM****************************************************************
positive= 10 negative = 9
RandomForest_100****************************************************************
posit

In [13]:
import sys
sys.path.append("/anaconda3/lib/python3.7/site-packages")
import numpy as np
import numpy
import nltk
import pickle
import pandas as pd
import glob
import os
import re
import operator
from collections import Counter
#from read_xml_files import *
##from s_stemmer import *
from nltk.stem.porter import PorterStemmer
from nltk.stem import LancasterStemmer, WordNetLemmatizer
from itertools import groupby
from nltk.collocations import *
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.cluster import KMeans
from scipy.cluster.vq import whiten
from nltk.tokenize.toktok import ToktokTokenizer
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2
import xml.etree.ElementTree as ET
import pandas as pd
#Import scikit-learn metrics module for accuracy calculation
from sklearn import metrics
##from pattern.text.en import singularize
##nltk.download('vader_lexicon')
##nltk.download('punkt')
#import the relevant modules from the NLTK library
from nltk.sentiment.vader import SentimentIntensityAnalyzer
sentence_tokenizer = nltk.data.load('tokenizers/punkt/english.pickle')
word_tokenizer = nltk.tokenize.RegexpTokenizer(r'\w+')
##tt = ToktokTokenizer()#keeps the url as is...................................................
from nltk.tokenize import TweetTokenizer
tt = TweetTokenizer()
Porter_stemmer = PorterStemmer()
Lancaster_stemmer = LancasterStemmer()
WordNet_lemmatizer = WordNetLemmatizer()
#nltk.download('averaged_perceptron_tagger')
##Train_path="D:/Data/PAN2015-Profiling/pan15-author-profiling-training-dataset-english-2015-04-23"#path for train data
##Test_path="D:/Data/PAN2015-Profiling/pan-ap2015-test/en" #path for test data

##Train_path="D:/NLP/PAN2014/pan14-author-profiling-training-corpus-english-twitter-2014-04-16"#path for train data D:\Data\PAN2013-Profiling\Training\en  D:\NLP\PAN2014
##train_truth_path="D:/NLP/PAN2014/truth_train"
##
##test_truth_path="D:/NLP/PAN2014/pan14_test"
##Test_path="D:/NLP/PAN2014/pan14_test/en"
##Content = open('results_of_clssification_all.txt', 'w')

M_train_file= "/Users/catherine/Desktop/NLP/PAN_Datasets/PAN2014blog/2014.1-blog-txtfiles/PAN2014_blog_male_train.1.txt"
F_train_file= "/Users/catherine/Desktop/NLP/PAN_Datasets/PAN2014blog/2014.1-blog-txtfiles/PAN2014_blog_female_train.1.txt"

M_test_file= "/Users/catherine/Desktop/NLP/PAN_Datasets/PAN2014blog/2014.1-blog-txtfiles/PAN2014_blog_male_test.1.txt"
F_test_file= "/Users/catherine/Desktop/NLP/PAN_Datasets/PAN2014blog/2014.1-blog-txtfiles/PAN2014_blog_female_test.1.txt"

def all_txt(M_txt_file,  F_txt_file):
    with open(M_txt_file, mode="r", encoding="utf-8") as f:
            txt_list = [(line[:-1]).split("\t") for line in f]
            male_dict = dict()
            content=[]
            for line in txt_list:
                if line[1] in male_dict:
                # append the new number to the existing array at this slot
                    male_dict[line[1]].append(line[3])
                else:
                # create a new array in this slot
                    male_dict[line[1]] = [line[3]]
    male_content= [("M", (' '.join(a)).split()) for a in list(male_dict.values())]
  
    with open(F_txt_file, mode="r", encoding="utf-8") as f:
            txt_list = [(line[:-1]).split("\t") for line in f]
            female_dict = dict()
            content=[]
            for line in txt_list:
                if line[1] in female_dict:
                # append the new number to the existing array at this slot
                    female_dict[line[1]].append(line[3])
                else:
                # create a new array in this slot
                    female_dict[line[1]] = [line[3]]
    female_content= [("F", (' '.join(a)).split()) for a in list(female_dict.values())]
    all_txt_per_person = male_content + female_content
    return all_txt_per_person

##print( [((all_txt(M_txt_file,  F_txt_file))[i][0])  for i in range(len((all_txt(M_txt_file,  F_txt_file))))]) helps to determine the gender....

      
"""
create feature vector from the training set.....................................................................................
"""

def WordFeatures(word_list, all_training_text):

    fvs_words = np.array([[author.count(word) for word in word_list] for author in all_training_text]).astype(np.float64)

    # normalise by dividing each row by number of tokens for each author........
    fvs_words /= np.c_[np.array([len(author) for author in all_training_text])]

    return fvs_words



"""
From the saved csv file, recover the saved features to be used...............................................................
"""
import csv
word_list=[]
##txt_files =[ "/Users/catherine/Desktop/NLP/PAN Datasets/PAN_chosen_tokens/2014/output2014OR.1M.txt", "/Users/catherine/Desktop/NLP/PAN Datasets/PAN_chosen_tokens/2014/output2014OR.2F.txt"]
txt_files =[ "//Users/catherine/Desktop/NLP/PAN_Datasets/PAN2014blog/2014.1-blog-txtfiles/PAN2014_blog_female_chosen_set.1.csv", "/Users/catherine/Desktop/NLP/PAN_Datasets/PAN2014blog/2014.1-blog-txtfiles/PAN2014_blog_male_chosen_set.1.csv"]
print("txt_files of features used",txt_files)
##for txt_file in txt_files:
##    with open(txt_file, mode="r", encoding="utf-8") as f:
##        reader = csv.reader(f, delimiter=",")
##        next(reader) # skip header
##        word_list1 =  ([(r.split(":"))[1] for r in f])
##        print(txt_file, len(word_list1))
##        word_list = word_list + word_list1
##print("length of features used =", len(word_list))

for txt_file in txt_files:
    with open(txt_file, mode="r", encoding="utf-8") as f:
        reader = csv.reader(f, delimiter=",")
        next(reader) # skip header
        word_list1 =  [r[0] for r in reader]
        print(txt_file, len(word_list1))
        word_list = word_list + word_list1
print("length of features used =", len(word_list))


"""
Prepare the training and test sets to be parsed to the classifies............................................................
"""
A=(all_txt(M_train_file,  F_train_file))
all_training_text = ( [(A[i][1])  for i in range(len(A))])
print("len(all_training_text)",len(all_training_text))
X_train=WordFeatures(word_list, all_training_text)
y_train=np.array( [(A[i][0])  for i in range(len(A))])
print("len(X_train)=",len(X_train), "len(y_train)=", len(y_train))

B=(all_txt(M_test_file,  F_test_file))
all_test_text = ( [(B[i][1])  for i in range(len(B))])
print("len(all_test_text)",len(all_test_text))
X_test = WordFeatures(word_list, all_test_text)
y_test = np.array( [(B[i][0])  for i in range(len(B))])
print("len(X_test=",len(X_test), "len(y_test)=",len(y_test))


X_train = np.nan_to_num(X_train)
X_test = np.nan_to_num(X_test)




#..................................................................................

data_tags = ["type","gender","tweet"]
train_data_list = []
i=0
for f in y_train:
   
    train_data_list.append((f, y_train[i],all_training_text[i]))
    i += 1
# We have training data available as dictionary filename, category, data
train_data = pd.DataFrame.from_records(train_data_list, columns=data_tags)




#..................................................................................

test_data_list = []
ii=0
for f in y_test:
   
    test_data_list.append((f, y_test[ii], all_test_text[ii]))
    ii += 1
# We have training data available as dictionary filename, category, data
test_data = pd.DataFrame.from_records(test_data_list, columns=data_tags)

#..................................................................................


import pandas as pd
import numpy as np
import pickle
from keras.preprocessing.text import Tokenizer
from keras.models import Sequential
from keras.layers import Activation, Dense, Dropout
from sklearn.preprocessing import LabelBinarizer
import sklearn.datasets as skds
from pathlib import Path

# :Tokenize and Prepare Vocabulary

train_posts = train_data['tweet']
train_tags = train_data['type']
train_tags=[1 if x=="M" else 0 for x in train_tags]
print(train_tags)

test_posts = test_data['tweet']
test_tags = test_data['type']
test_tags=[1 if x=="M" else 0 for x in test_tags]


from numpy import array
from numpy import asarray
from numpy import zeros
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers import Embedding
from keras.layers import Dense, Conv1D, Flatten


# define train documents
docs = train_posts

# define train class labels
labels = train_tags

# prepare tokenizer
t = Tokenizer()
t.fit_on_texts(docs)
vocab_size = len(t.word_index) + 1

# integer encode the documents
encoded_docs = t.texts_to_sequences(docs)
#print(encoded_docs)

# pad documents to a max length of 4 words
max_length = 100
padded_docs = pad_sequences(encoded_docs, maxlen=max_length, padding='post')
#print(padded_docs)



# define test documents
test_docs = test_posts

# define train class labels
test_labels = test_tags

# prepare tokenizer
test_t = Tokenizer()
test_t.fit_on_texts(test_docs)
test_vocab_size = len(test_t.word_index) + 1

# integer encode the documents
test_encoded_docs = test_t.texts_to_sequences(test_docs)
#print(encoded_docs)

# pad documents to a max length of 4 words

test_padded_docs = pad_sequences(test_encoded_docs, maxlen=max_length, padding='post')
#print(padded_docs)



# load the whole embedding into memory
embeddings_index = dict()
f = open('/Users/catherine/Downloads/glove.6B.100d.txt')
for line in f:
    values = line.split()
    word = values[0]
    coefs = asarray(values[1:], dtype='float32')
    embeddings_index[word] = coefs
f.close()
print('Loaded %s word vectors.' % len(embeddings_index))

# create a weight matrix for words in training docs
embedding_matrix = zeros((vocab_size, 100))
for word, i in t.word_index.items():
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        embedding_matrix[i] = embedding_vector
        
# define model
model = Sequential()
e = Embedding(vocab_size, 100, weights=[embedding_matrix], input_length=max_length, trainable= False)
model.add(e)
model.add(Flatten())
model.add(Dense(1, activation='sigmoid'))

# compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['acc'])

# summarize the model
print(model.summary())

# fit the model
model.fit(padded_docs, labels, epochs=100, verbose=0)

# evaluate the model
loss, accuracy = model.evaluate(test_padded_docs, test_labels, verbose=0)
print('Accuracy: %f' % (accuracy*100))


txt_files of features used ['//Users/catherine/Desktop/NLP/PAN_Datasets/PAN2014blog/2014.1-blog-txtfiles/PAN2014_blog_female_chosen_set.1.csv', '/Users/catherine/Desktop/NLP/PAN_Datasets/PAN2014blog/2014.1-blog-txtfiles/PAN2014_blog_male_chosen_set.1.csv']
//Users/catherine/Desktop/NLP/PAN_Datasets/PAN2014blog/2014.1-blog-txtfiles/PAN2014_blog_female_chosen_set.1.csv 1526
/Users/catherine/Desktop/NLP/PAN_Datasets/PAN2014blog/2014.1-blog-txtfiles/PAN2014_blog_male_chosen_set.1.csv 2063
length of features used = 3589
len(all_training_text) 147
len(X_train)= 147 len(y_train)= 147
len(all_test_text) 78
len(X_test= 78 len(y_test)= 78
[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,

In [88]:
import sys
sys.path.append("/anaconda3/lib/python3.7/site-packages")
import numpy as np
import numpy
import nltk
import pickle
import pandas as pd
import glob
import os
import re
import operator
from collections import Counter
#from read_xml_files import *
##from s_stemmer import *
from nltk.stem.porter import PorterStemmer
from nltk.stem import LancasterStemmer, WordNetLemmatizer
from itertools import groupby
from nltk.collocations import *
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.cluster import KMeans
from scipy.cluster.vq import whiten
from nltk.tokenize.toktok import ToktokTokenizer
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2
import xml.etree.ElementTree as ET
import pandas as pd
#Import scikit-learn metrics module for accuracy calculation
from sklearn import metrics
##from pattern.text.en import singularize
##nltk.download('vader_lexicon')
##nltk.download('punkt')
#import the relevant modules from the NLTK library
from nltk.sentiment.vader import SentimentIntensityAnalyzer
sentence_tokenizer = nltk.data.load('tokenizers/punkt/english.pickle')
word_tokenizer = nltk.tokenize.RegexpTokenizer(r'\w+')
##tt = ToktokTokenizer()#keeps the url as is...................................................
from nltk.tokenize import TweetTokenizer
tt = TweetTokenizer()
Porter_stemmer = PorterStemmer()
Lancaster_stemmer = LancasterStemmer()
WordNet_lemmatizer = WordNetLemmatizer()
#nltk.download('averaged_perceptron_tagger')
##Train_path="D:/Data/PAN2015-Profiling/pan15-author-profiling-training-dataset-english-2015-04-23"#path for train data
##Test_path="D:/Data/PAN2015-Profiling/pan-ap2015-test/en" #path for test data

##Train_path="D:/NLP/PAN2014/pan14-author-profiling-training-corpus-english-twitter-2014-04-16"#path for train data D:\Data\PAN2013-Profiling\Training\en  D:\NLP\PAN2014
##train_truth_path="D:/NLP/PAN2014/truth_train"
##
##test_truth_path="D:/NLP/PAN2014/pan14_test"
##Test_path="D:/NLP/PAN2014/pan14_test/en"
##Content = open('results_of_clssification_all.txt', 'w')

M_train_file= "/Users/catherine/Desktop/NLP/PAN_Datasets/PAN2014blog/2014.1-blog-txtfiles/PAN2014_blog_male_train.1.txt"
F_train_file= "/Users/catherine/Desktop/NLP/PAN_Datasets/PAN2014blog/2014.1-blog-txtfiles/PAN2014_blog_female_train.1.txt"

M_test_file= "/Users/catherine/Desktop/NLP/PAN_Datasets/PAN2014blog/2014.1-blog-txtfiles/PAN2014_blog_male_test.1.txt"
F_test_file= "/Users/catherine/Desktop/NLP/PAN_Datasets/PAN2014blog/2014.1-blog-txtfiles/PAN2014_blog_female_test.1.txt"

def all_txt(M_txt_file,  F_txt_file):
    with open(M_txt_file, mode="r", encoding="utf-8") as f:
            txt_list = [(line[:-1]).split("\t") for line in f]
            male_dict = dict()
            content=[]
            for line in txt_list:
                if line[1] in male_dict:
                # append the new number to the existing array at this slot
                    male_dict[line[1]].append(line[3])
                else:
                # create a new array in this slot
                    male_dict[line[1]] = [line[3]]
    male_content= [("M", (' '.join(a)).split()) for a in list(male_dict.values())]
  
    with open(F_txt_file, mode="r", encoding="utf-8") as f:
            txt_list = [(line[:-1]).split("\t") for line in f]
            female_dict = dict()
            content=[]
            for line in txt_list:
                if line[1] in female_dict:
                # append the new number to the existing array at this slot
                    female_dict[line[1]].append(line[3])
                else:
                # create a new array in this slot
                    female_dict[line[1]] = [line[3]]
    female_content= [("F", (' '.join(a)).split()) for a in list(female_dict.values())]
    all_txt_per_person = male_content + female_content
    return all_txt_per_person

##print( [((all_txt(M_txt_file,  F_txt_file))[i][0])  for i in range(len((all_txt(M_txt_file,  F_txt_file))))]) helps to determine the gender....

      
"""
create feature vector from the training set.....................................................................................
"""

def WordFeatures(word_list, all_training_text):

    fvs_words = np.array([[author.count(word) for word in word_list] for author in all_training_text]).astype(np.float64)

    # normalise by dividing each row by number of tokens for each author........
    fvs_words /= np.c_[np.array([len(author) for author in all_training_text])]

    return fvs_words



"""
From the saved csv file, recover the saved features to be used...............................................................
"""
import csv
word_list=[]
##txt_files =[ "/Users/catherine/Desktop/NLP/PAN Datasets/PAN_chosen_tokens/2014/output2014OR.1M.txt", "/Users/catherine/Desktop/NLP/PAN Datasets/PAN_chosen_tokens/2014/output2014OR.2F.txt"]
txt_files =[ "//Users/catherine/Desktop/NLP/PAN_Datasets/PAN2014blog/2014.1-blog-txtfiles/PAN2014_blog_female_chosen_set.1.csv", "/Users/catherine/Desktop/NLP/PAN_Datasets/PAN2014blog/2014.1-blog-txtfiles/PAN2014_blog_male_chosen_set.1.csv"]
print("txt_files of features used",txt_files)
##for txt_file in txt_files:
##    with open(txt_file, mode="r", encoding="utf-8") as f:
##        reader = csv.reader(f, delimiter=",")
##        next(reader) # skip header
##        word_list1 =  ([(r.split(":"))[1] for r in f])
##        print(txt_file, len(word_list1))
##        word_list = word_list + word_list1
##print("length of features used =", len(word_list))

for txt_file in txt_files:
    with open(txt_file, mode="r", encoding="utf-8") as f:
        reader = csv.reader(f, delimiter=",")
        next(reader) # skip header
        word_list1 =  [r[0] for r in reader]
        print(txt_file, len(word_list1))
        word_list = word_list + word_list1
print("length of features used =", len(word_list))


"""
Prepare the training and test sets to be parsed to the classifies............................................................
"""
A=(all_txt(M_train_file,  F_train_file))
all_training_text = ( [(A[i][1])  for i in range(len(A))])
#print("len(all_training_text)",len(all_training_text))
#X_train=WordFeatures(word_list, all_training_text)
y_train=np.array( [(A[i][0])  for i in range(len(A))])
#print("len(X_train)=",len(X_train), "len(y_train)=", len(y_train))

B=(all_txt(M_test_file,  F_test_file))
all_test_text = ( [(B[i][1])  for i in range(len(B))])
#print("len(all_test_text)",len(all_test_text))
#X_test = WordFeatures(word_list, all_test_text)
y_test = np.array( [(B[i][0])  for i in range(len(B))])
#print("len(X_test=",len(X_test), "len(y_test)=",len(y_test))


#X_train = np.nan_to_num(X_train)
#X_test = np.nan_to_num(X_test)




#..................................................................................

data_tags = ["type","gender","tweet"]
train_data_list = []
i=0
for f in y_train:
   
    train_data_list.append((f, y_train[i],all_training_text[i]))
    i += 1
# We have training data available as dictionary filename, category, data
train_data = pd.DataFrame.from_records(train_data_list, columns=data_tags)




#..................................................................................

test_data_list = []
ii=0
for f in y_test:
   
    test_data_list.append((f, y_test[ii], all_test_text[ii]))
    ii += 1
# We have training data available as dictionary filename, category, data
test_data = pd.DataFrame.from_records(test_data_list, columns=data_tags)

#..................................................................................


import pandas as pd
import numpy as np
from numpy import array
import pickle
from keras.preprocessing.text import Tokenizer
from keras.models import Sequential
from keras.layers import Activation, Dense, Dropout
from sklearn.preprocessing import LabelBinarizer
import sklearn.datasets as skds
from pathlib import Path
import pandas as pd
import numpy as np
import pickle
from keras.preprocessing.text import Tokenizer
from keras.models import Sequential
from keras.layers import Activation, Dense, Dropout
from sklearn.preprocessing import LabelBinarizer
import sklearn.datasets as skds
from pathlib import Path
from numpy import array
from numpy import asarray
from numpy import zeros
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers import Embedding
from keras.layers import Dense, Conv1D, Flatten
from keras.layers import LSTM
from keras.layers.embeddings import Embedding
from keras.preprocessing import sequence
from keras.layers import Activation, Conv1D, Dense, Embedding, Flatten, Input, MaxPooling1D, Bidirectional



# :Tokenize and Prepare Vocabulary

train_posts = train_data['tweet']
train_tags = train_data['type']
train_tags=array([1 if x=="M" else 0 for x in train_tags])
#train_tags=train_tags.reshape(-1, 1)
#print("train_tags", print(train_tags.shape), (train_tags))

test_posts = test_data['tweet']
test_tags = test_data['type']
test_tags=[1 if x=="M" else 0 for x in test_tags]




# define train documents
docs = train_posts

# define train class labels
labels = train_tags

# prepare tokenizer
t = Tokenizer()
t.fit_on_texts(docs)
vocab_size = len(t.word_index) + 1

# integer encode the documents
encoded_docs = t.texts_to_sequences(docs)
#print(encoded_docs)

# pad documents to a max length of 4 words
max_length = 1000

padded_docs = pad_sequences(encoded_docs, maxlen=max_length, padding='post')

#print(padded_docs)



# define test documents
test_docs = test_posts

# define train class labels
test_labels = test_tags

# prepare tokenizer
test_t = Tokenizer()
test_t.fit_on_texts(test_docs)
test_vocab_size = len(test_t.word_index) + 1

# integer encode the documents
test_encoded_docs = test_t.texts_to_sequences(test_docs)
#print(encoded_docs)

# pad documents to a max length of 4 words

test_padded_docs = pad_sequences(test_encoded_docs, maxlen=max_length, padding='post')
#print(padded_docs)



# load the whole embedding into memory
embeddings_index = dict()
f = open('/Users/catherine/Downloads/glove.6B.100d.txt')
for line in f:
    values = line.split()
    word = values[0]
    coefs = asarray(values[1:], dtype='float32')
    embeddings_index[word] = coefs
f.close()
print('Loaded %s word vectors.' % len(embeddings_index))

# create a weight matrix for words in training docs
embedding_matrix = zeros((vocab_size, 100))
for word, i in t.word_index.items():
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        embedding_matrix[i] = embedding_vector
        
# define model_1
model = Sequential()
model.add(Embedding(vocab_size, 100, input_length=max_length, trainable= True))
model.add(Conv1D(filters=32, kernel_size=8, activation='relu'))
model.add(MaxPooling1D(pool_size=2))
model.add(Flatten())
model.add(Dense(10, activation='relu'))
model.add(Dense(1, activation='sigmoid'))
print(model.summary())
# compile network
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
# fit network
model.fit(padded_docs, labels, epochs=10, verbose=2)
# evaluate
loss, acc = model.evaluate(test_padded_docs, test_labels, verbose=0)
print('model_1 Accuracy: %f' % (acc*100))
        
# define model_2
model = Sequential()
model.add(Embedding(vocab_size, 100, weights=[embedding_matrix], input_length=max_length, trainable= False))
#model.add(Embedding(vocab_size, 100, input_length=max_length, trainable= False))
model.add(Conv1D(50, 5))
model.add(Flatten())
model.add(Dense(100, activation='relu'))
model.add(Dense(1, activation='softmax'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
print(padded_docs.shape)
model.fit(padded_docs, labels, epochs=10, batch_size=64)

loss, accuracy = model.evaluate(test_padded_docs, test_labels, verbose=0)
print('model_2 Accuracy: %f' % (accuracy*100))

# create the model_3
top_words = 5000
# create the model
embedding_vecor_length = 32
model = Sequential()
#model.add(Embedding(top_words, len(embedding_matrix), input_length=max_length))
model.add(Embedding(vocab_size, 100, weights=[embedding_matrix], input_length=max_length, trainable= False))
#model.add(Dropout(0.2))
model.add(LSTM(100))
#model.add(Dropout(0.2))
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
print(model.summary())
model.fit(padded_docs, labels, epochs=30, batch_size=64)
# Final evaluation of the model
scores = model.evaluate(test_padded_docs, test_labels, verbose=0)
print("model_3 Accuracy: %.2f%%" % (scores[1]*100))



# create the model_4
model = Sequential()
model.add(Embedding(len(embedding_matrix), 100, weights=[embedding_matrix], input_length=max_length, trainable=False))
model.add(Bidirectional(LSTM(128)))
#model.add(Bidirectional(LSTM(128, dropout=0.2, recurrent_dropout=0.2)))
model.add(Dense(512, activation='relu'))
#model.add(Dropout(0.50))
model.add(Dense(1, activation='softmax'))
# Adam Optimiser
model.compile(loss='binary_crossentropy',optimizer='adam', metrics=['accuracy'])
print(model.summary())
model.fit(padded_docs, labels, epochs=10, batch_size=64)
# Final evaluation of the model
scores = model.evaluate(test_padded_docs, test_labels, verbose=0)
print('model_4 Accuracy: %f' % (scores[1]*100))

# create the model_4
model = Sequential()
model.add(Embedding(len(embedding_matrix), 100, weights=[embedding_matrix], input_length=max_length, trainable=False))
model.add(Conv1D(128, 5, activation='relu'))
model.add(MaxPooling1D(5))
model.add(Conv1D(128, 5, activation='relu'))
model.add(MaxPooling1D(5))
model.add(Conv1D(128, 5, activation='relu'))
model.add(MaxPooling1D(35))  # global max pooling
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.compile(loss='binary_crossentropy',optimizer='adam', metrics=['accuracy'])
print(model.summary())
model.fit(padded_docs, labels, epochs=10, batch_size=64)
# Final evaluation of the model
scores = model.evaluate(test_padded_docs, test_labels, verbose=0)
print('model_5 Accuracy: %f' % (scores[1]*100))



txt_files of features used ['//Users/catherine/Desktop/NLP/PAN_Datasets/PAN2014blog/2014.1-blog-txtfiles/PAN2014_blog_female_chosen_set.1.csv', '/Users/catherine/Desktop/NLP/PAN_Datasets/PAN2014blog/2014.1-blog-txtfiles/PAN2014_blog_male_chosen_set.1.csv']
//Users/catherine/Desktop/NLP/PAN_Datasets/PAN2014blog/2014.1-blog-txtfiles/PAN2014_blog_female_chosen_set.1.csv 1526
/Users/catherine/Desktop/NLP/PAN_Datasets/PAN2014blog/2014.1-blog-txtfiles/PAN2014_blog_male_chosen_set.1.csv 2063
length of features used = 3589
Loaded 400000 word vectors.
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_148 (Embedding)    (None, 1000, 100)         2409400   
_________________________________________________________________
conv1d_105 (Conv1D)          (None, 993, 32)           25632     
_________________________________________________________________
max_pooling1d_45 (MaxPooling (None, 496, 32)           

ValueError: Error when checking target: expected dense_227 to have shape (128,) but got array with shape (1,)

In [ ]:
from string import punctuation
from os import listdir
from gensim.models import Word2Vec
# train word2vec model
#model = Word2Vec(padded_docs, size=100, window=5, workers=8, min_count=1)
# summarize vocabulary size in model
words = list(model.wv.vocab)
print('Vocabulary size: %d' % len(words))

In [18]:
#changing the classifiers..........


import sys
sys.path.append("/anaconda3/lib/python3.7/site-packages")
import numpy as np
import numpy
import nltk
import pickle
import pandas as pd
import glob
import os
import re
import operator
from collections import Counter
#from read_xml_files import *
##from s_stemmer import *
from nltk.stem.porter import PorterStemmer
from nltk.stem import LancasterStemmer, WordNetLemmatizer
from itertools import groupby
from nltk.collocations import *
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.cluster import KMeans
from scipy.cluster.vq import whiten
from nltk.tokenize.toktok import ToktokTokenizer
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2
import xml.etree.ElementTree as ET
import pandas as pd
#Import scikit-learn metrics module for accuracy calculation
from sklearn import metrics
##from pattern.text.en import singularize
##nltk.download('vader_lexicon')
##nltk.download('punkt')
#import the relevant modules from the NLTK library
from nltk.sentiment.vader import SentimentIntensityAnalyzer
sentence_tokenizer = nltk.data.load('tokenizers/punkt/english.pickle')
word_tokenizer = nltk.tokenize.RegexpTokenizer(r'\w+')
##tt = ToktokTokenizer()#keeps the url as is...................................................
from nltk.tokenize import TweetTokenizer
tt = TweetTokenizer()
Porter_stemmer = PorterStemmer()
Lancaster_stemmer = LancasterStemmer()
WordNet_lemmatizer = WordNetLemmatizer()
#nltk.download('averaged_perceptron_tagger')
##Train_path="D:/Data/PAN2015-Profiling/pan15-author-profiling-training-dataset-english-2015-04-23"#path for train data
##Test_path="D:/Data/PAN2015-Profiling/pan-ap2015-test/en" #path for test data

##Train_path="D:/NLP/PAN2014/pan14-author-profiling-training-corpus-english-twitter-2014-04-16"#path for train data D:\Data\PAN2013-Profiling\Training\en  D:\NLP\PAN2014
##train_truth_path="D:/NLP/PAN2014/truth_train"
##
##test_truth_path="D:/NLP/PAN2014/pan14_test"
##Test_path="D:/NLP/PAN2014/pan14_test/en"
##Content = open('results_of_clssification_all.txt', 'w')

M_train_file= "/Users/catherine/Desktop/NLP/PAN_Datasets/PAN2016/2016.1_textfiles/PAN2016_tweet_male_train.1.txt"
F_train_file= "/Users/catherine/Desktop/NLP/PAN_Datasets/PAN2016/2016.1_textfiles/PAN2016_tweet_female_train.1.txt"

M_test_file= "/Users/catherine/Desktop/NLP/PAN_Datasets/PAN2016/2016.1_textfiles/PAN2016_tweet_male_test.1.txt"
F_test_file= "/Users/catherine/Desktop/NLP/PAN_Datasets/PAN2016/2016.1_textfiles/PAN2016_tweet_female_test.1.txt"

def all_txt(M_txt_file,  F_txt_file):
    with open(M_txt_file, mode="r", encoding="utf-8") as f:
            txt_list = [(line[:-1]).split("\t") for line in f]
            male_dict = dict()
            content=[]
            for line in txt_list:
                if line[1] in male_dict:
                # append the new number to the existing array at this slot
                    male_dict[line[1]].append(line[3])
                else:
                # create a new array in this slot
                    male_dict[line[1]] = [line[3]]
    male_content= [("M", (' '.join(a)).split()) for a in list(male_dict.values())]
  
    with open(F_txt_file, mode="r", encoding="utf-8") as f:
            txt_list = [(line[:-1]).split("\t") for line in f]
            female_dict = dict()
            content=[]
            for line in txt_list:
                if line[1] in female_dict:
                # append the new number to the existing array at this slot
                    female_dict[line[1]].append(line[3])
                else:
                # create a new array in this slot
                    female_dict[line[1]] = [line[3]]
    female_content= [("F", (' '.join(a)).split()) for a in list(female_dict.values())]
    all_txt_per_person = male_content + female_content
    return all_txt_per_person

##print( [((all_txt(M_txt_file,  F_txt_file))[i][0])  for i in range(len((all_txt(M_txt_file,  F_txt_file))))]) helps to determine the gender....

      
"""
create feature vector from the training set.....................................................................................
"""

def WordFeatures(word_list, all_training_text):

    fvs_words = np.array([[author.count(word) for word in word_list] for author in all_training_text]).astype(np.float64)

    # normalise by dividing each row by number of tokens for each author........
    fvs_words /= np.c_[np.array([len(author) for author in all_training_text])]

    return fvs_words



"""
From the saved csv file, recover the saved features to be used...............................................................
"""
import csv
word_list=[]
##txt_files =[ "/Users/catherine/Desktop/NLP/PAN Datasets/PAN_chosen_tokens/2016/output2016OR.1M.txt", "/Users/catherine/Desktop/NLP/PAN Datasets/PAN_chosen_tokens/2016/output2016OR.2F.txt"]
txt_files =[ "/Users/catherine/Desktop/NLP/PAN_Datasets/PAN2016/2016.1_textfiles/PAN2016_tweet_female_chosen_set.1.csv", "/Users/catherine/Desktop/NLP/PAN_Datasets/PAN2016/2016.1_textfiles/PAN2016_tweet_male_chosen_set.1.csv"]
print("txt_files of features used",txt_files)
##for txt_file in txt_files:
##    with open(txt_file, mode="r", encoding="utf-8") as f:
##        reader = csv.reader(f, delimiter=",")
##        next(reader) # skip header
##        word_list1 =  ([(r.split(":"))[1] for r in f])
##        print(txt_file, len(word_list1))
##        word_list = word_list + word_list1
##print("length of features used =", len(word_list))

for txt_file in txt_files:
    with open(txt_file, mode="r", encoding="utf-8") as f:
        reader = csv.reader(f, delimiter=",")
        next(reader) # skip header
        word_list1 =  [r[0] for r in reader]
        print(txt_file, len(word_list1))
        word_list = word_list + word_list1
print("length of features used =", len(word_list))


"""
Prepare the training and test sets to be parsed to the classifies............................................................
"""
A=(all_txt(M_train_file,  F_train_file))
all_training_text = ( [(A[i][1])  for i in range(len(A))])
print("len(all_training_text)",len(all_training_text))
#X_train=WordFeatures(word_list, all_training_text)
y_train=np.array( [(A[i][0])  for i in range(len(A))])
#print("len(X_train)=",len(X_train), "len(y_train)=", len(y_train))

B=(all_txt(M_test_file,  F_test_file))
all_test_text = ( [(B[i][1])  for i in range(len(B))])
print("len(all_test_text)",len(all_test_text))
#X_test = WordFeatures(word_list, all_test_text)
y_test = np.array( [(B[i][0])  for i in range(len(B))])
#print("len(X_test=",len(X_test), "len(y_test)=",len(y_test))


X_train = np.nan_to_num(X_train)
X_test = np.nan_to_num(X_test)



#..................................................................................

data_tags = ["type","gender","tweet"]
train_data_list = []
i=0
for f in y_train:
   
    train_data_list.append((f, y_train[i],all_training_text[i]))
    i += 1
# We have training data available as dictionary filename, category, data
train_data = pd.DataFrame.from_records(train_data_list, columns=data_tags)




#..................................................................................

test_data_list = []
ii=0
for f in y_test:
   
    test_data_list.append((f, y_test[ii], all_test_text[ii]))
    ii += 1
# We have training data available as dictionary filename, category, data
test_data = pd.DataFrame.from_records(test_data_list, columns=data_tags)

#..................................................................................


import pandas as pd
import numpy as np
import pickle
from keras.preprocessing.text import Tokenizer
from keras.models import Sequential
from keras.layers import Activation, Dense, Dropout
from sklearn.preprocessing import LabelBinarizer
import sklearn.datasets as skds
from pathlib import Path

# :Tokenize and Prepare Vocabulary

train_posts = train_data['tweet']
train_tags = train_data['type']

test_posts = test_data['tweet']
test_tags = test_data['type']


num_labels = 2
vocab_size = 1000#len(word_list)#most common number of words will be then kept for use in the vector
#vocab_size = 1000#by changing the vocubulary size... acc count_type =  0.932258064516129#acc count_gender = 0.8298387096774194
#vocab_size = 100#acc count_type =0.9274193548387096 acc count_gender = 0.792741935483871
batch_size = 100
 
# define Tokenizer with Vocab Size
tokenizer = Tokenizer(num_words=vocab_size)#IF I USE OTHER TOKENIZER...
tokenizer.fit_on_texts(train_posts)
#print(tokenizer.word_index)
#print((tokenizer.word_counts))#provides a dictionary of the words and the count......................
#sorted_x = sorted((tokenizer.word_counts).items(), key=operator.itemgetter(1),reverse=True)
#print(sorted_x)
#print((tokenizer.document_count))#number of documents..............
#print((tokenizer.word_docs))#provides a dictionary of the words and the number of documents they appear in......................
print('Found %d unique words.' % len(tokenizer.word_index))#shows total vocubulary of the text dataset
x_train = tokenizer.texts_to_matrix(train_posts, mode='tfidf')#WHAT HAPPENS WHEN I GIVE IT THE HAND CRAFTED TOKENS??
x_test = tokenizer.texts_to_matrix(test_posts, mode='tfidf')

##--------what about applying knn at this point-------------------------------
##--------what about changing the vocubulary size-------------------------------works well with reduced sized
##----------One popular method for hyperparameter optimization is grid search.-----------
##-----determine the best set of parameters with the highest accuracy..........this is for the keras model------


      
encoder = LabelBinarizer()


encoder.fit(train_tags)

y_train = encoder.transform(train_tags)#same as the y train generated with my model
y_train = np.hstack((y_train, 1 - y_train))#used for two label cases........

y_test = encoder.transform(test_tags)
y_test = np.hstack((y_test, 1 - y_test))


"""
fit_on_texts Updates internal vocabulary based on a list of texts. This method creates the vocabulary index based on word frequency. So if you give it something like, "The cat sat on the mat." It will create a dictionary s.t. word_index["the"] = 0; word_index["cat"] = 1 it is word -> index dictionary so every word gets a unique integer value. So lower integer means more frequent word (often the first few are punctuation because they appear a lot).
texts_to_sequences Transforms each text in texts to a sequence of integers. So it basically takes each word in the text and replaces it with its corresponding integer value from the word_index dictionary. Nothing more, nothing less, certainly no magic involved.
CLEAN TEXT TO THE DESIRED LEVEL AND USE KERAS INBUILF TFIDF TO GENERATE A MATRIX....
"""
from sklearn import model_selection, naive_bayes, svm
from sklearn.linear_model import SGDClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import LinearSVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.neural_network import MLPClassifier
#8. manhattan K-Nearest Neighbours..............................................................................

k=5

print(k)
"""
8. manhattan K-Nearest Neighbours..............................................................................
"""
print("manhattan_k=5************************************ ****************************")
from sklearn.neighbors import KNeighborsClassifier
knn = KNeighborsClassifier(n_neighbors = k, metric= "manhattan")
knn.fit(x_train, train_tags)
y_pred = knn.predict(x_test)
manhattan_k5tfidf=([(i,j) for i, j in zip(y_pred, test_tags)])
print("manhattan K-Nearest Neighbours")
print(metrics.accuracy_score(test_tags, y_pred))
print("************************************ ****************************")

k=13

print(k)
"""
8. manhattan K-Nearest Neighbours..............................................................................
"""
print("manhattan_k=13************************************ ****************************")
from sklearn.neighbors import KNeighborsClassifier
knn = KNeighborsClassifier(n_neighbors = k, metric= "manhattan")
knn.fit(x_train, train_tags)
y_pred = knn.predict(x_test)
manhattan_k13tfidf=([(i,j) for i, j in zip(y_pred, test_tags)])
print("manhattan K-Nearest Neighbours")
print(metrics.accuracy_score(test_tags, y_pred))
print("****************************************************************")


from sklearn import model_selection, naive_bayes, svm
from sklearn.linear_model import SGDClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import LinearSVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.neural_network import MLPClassifier


print("MLPClassifier****************************************************************")
clf = MLPClassifier()
clf = clf.fit(x_train, train_tags)
#Predict the response for test dataset
y_pred = clf.predict(x_test)
MLPClassifiertfidf=([(i,j) for i, j in zip(y_pred, test_tags)])
print(metrics.accuracy_score(test_tags, y_pred))


print("LogisticRegression****************************************************************")
clf = LogisticRegression()
clf = clf.fit(x_train, train_tags)
#Predict the response for test dataset
y_pred = clf.predict(x_test)
LogisticRegressiontfidf=([(i,j) for i, j in zip(y_pred, test_tags)])
print(metrics.accuracy_score(test_tags, y_pred))


print("SGDClassifier****************************************************************")
clf = SGDClassifier()
clf = clf.fit(x_train, train_tags)
#Predict the response for test dataset
y_pred = clf.predict(x_test)
SGDClassifiertfidf=([(i,j) for i, j in zip(y_pred, test_tags)])
print(metrics.accuracy_score(test_tags, y_pred))


print("DecisionTreeClassifier****************************************************************")
# Create Decision Tree classifer object
clf = DecisionTreeClassifier()
# Train Decision Tree Classifer
clf = clf.fit(x_train, train_tags)
#Predict the response for test dataset
y_pred = clf.predict(x_test)
DecisionTreeClassifiertfidf=([(i,j) for i, j in zip(y_pred, test_tags)])
print(metrics.accuracy_score(test_tags, y_pred))


print("Naive****************************************************************")
Naive = naive_bayes.MultinomialNB()
Naive.fit(x_train, train_tags)
y_pred = Naive.predict(x_test)
Naivetfidf=([(i,j) for i, j in zip(y_pred, test_tags)])
print(metrics.accuracy_score(test_tags, y_pred))


print("SVM****************************************************************")
SVM = svm.SVC(C=1.0, kernel='linear', degree=3, gamma='auto')
SVM.fit(x_train, train_tags)
y_pred = SVM.predict(x_test)
SVMtfidf=([(i,j) for i, j in zip(y_pred, test_tags)])
print(metrics.accuracy_score(test_tags, y_pred))


print("RandomForest_100****************************************************************")
clf=RandomForestClassifier(n_estimators=100)
#Train the model using the training sets y_pred=clf.predict(X_test)
clf.fit(x_train,train_tags)
y_pred=clf.predict(x_test)
RandomForest_100tfidf=([(i,j) for i, j in zip(y_pred, test_tags)])
print(metrics.accuracy_score(test_tags, y_pred))

print("RandomForest_200****************************************************************")
clf=RandomForestClassifier(n_estimators=200)
#Train the model using the training sets y_pred=clf.predict(X_test)
clf.fit(x_train,train_tags)
y_pred=clf.predict(x_test)
RandomForest_200tfidf=([(i,j) for i, j in zip(y_pred, test_tags)])
print(metrics.accuracy_score(test_tags, y_pred))

from xgboost.sklearn import XGBClassifier 
print("XGBClassifier****************************************************************")
clf=XGBClassifier()
#Train the model using the training sets y_pred=clf.predict(X_test)
clf.fit(x_train, train_tags)
y_pred=clf.predict(x_test)
XGBClassifiertfidf=([(i,j) for i, j in zip(y_pred, test_tags)])
print(metrics.accuracy_score(test_tags, y_pred))



print("BaggingClassifier****************************************************************")
from sklearn.ensemble.bagging import BaggingClassifier
clf=BaggingClassifier()
#Train the model using the training sets y_pred=clf.predict(X_test)
clf.fit(x_train, train_tags)
y_pred=clf.predict(x_test)
BaggingClassifiertfidf=([(i,j) for i, j in zip(y_pred, test_tags)])
print(metrics.accuracy_score(test_tags, y_pred))


print("AdaBoostClassifier****************************************************************")
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
clf=AdaBoostClassifier()
#Train the model using the training sets y_pred=clf.predict(X_test)
clf.fit(x_train, train_tags)
y_pred=clf.predict(x_test)
AdaBoostClassifiertfidf=([(i,j) for i, j in zip(y_pred, test_tags)])
print(metrics.accuracy_score(test_tags, y_pred))





from keras.models import *
from keras.layers import *

model = Sequential()
model.add(Dense(512, input_shape=(vocab_size,)))#512 neurons in the first hidden layer
model.add(Activation('relu'))
#model.add(Dropout(0.3))
model.add(Dense(512))
model.add(Activation('relu'))
#model.add(Dropout(0.3))
model.add(Dense(num_labels))
model.add(Activation('softmax'))
model.summary()
 
model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

history = model.fit(x_train, y_train,
                    batch_size=batch_size,
                    epochs=30,
                    verbose=1,
                    validation_split=0.1)



#Evaluate model.............................
score = model.evaluate(x_test, y_test,
                       batch_size=batch_size, verbose=1)
 
print('Test accuracy:', score[1])





txt_files of features used ['/Users/catherine/Desktop/NLP/PAN_Datasets/PAN2016/2016.1_textfiles/PAN2016_tweet_female_chosen_set.1.csv', '/Users/catherine/Desktop/NLP/PAN_Datasets/PAN2016/2016.1_textfiles/PAN2016_tweet_male_chosen_set.1.csv']
/Users/catherine/Desktop/NLP/PAN_Datasets/PAN2016/2016.1_textfiles/PAN2016_tweet_female_chosen_set.1.csv 6328
/Users/catherine/Desktop/NLP/PAN_Datasets/PAN2016/2016.1_textfiles/PAN2016_tweet_male_chosen_set.1.csv 8793
length of features used = 15121
len(all_training_text) 383
len(all_test_text) 78
Found 221453 unique words.
5
manhattan_k=5************************************ ****************************
manhattan K-Nearest Neighbours
0.5641025641025641
************************************ ****************************
13
manhattan_k=13************************************ ****************************
manhattan K-Nearest Neighbours
0.5256410256410257
****************************************************************
MLPClassifier************************

/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/stochastic_gradient.py:128: FutureWarning: max_iter and tol parameters have been added in <class 'sklearn.linear_model.stochastic_gradient.SGDClassifier'> in 0.19. If both are left unset, they default to max_iter=5 and tol=None. If tol is not None, max_iter defaults to max_iter=1000. From 0.21, default max_iter will be 1000, and default tol will be 1e-3.
  "and default tol will be 1e-3." % type(self), FutureWarning)


0.6666666666666666
RandomForest_100****************************************************************
0.6410256410256411
RandomForest_200****************************************************************
0.5512820512820513
XGBClassifier****************************************************************


/anaconda3/lib/python3.7/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


0.6282051282051282
BaggingClassifier****************************************************************
0.6410256410256411
AdaBoostClassifier****************************************************************
0.5769230769230769
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_10 (Dense)             (None, 512)               512512    
_________________________________________________________________
activation_10 (Activation)   (None, 512)               0         
_________________________________________________________________
dense_11 (Dense)             (None, 512)               262656    
_________________________________________________________________
activation_11 (Activation)   (None, 512)               0         
_________________________________________________________________
dense_12 (Dense)             (None, 2)                 1026      
____________________________________________________

In [19]:
print("The sign test...****************************************************************")
print("manhattan_k=5************************************ ****************************")
x= LogisticRegressiontfidf 
y= manhattan_k5tfidf

positive=0
negative=0
for i in range(len(x)):
    if (x[i]!=y[i]):
        if (x[i][0]==x[i][1] and y[i][0]!=y[i][1]):
            positive=positive+1
        else: 
            negative=negative+1
print("positive=",positive, "negative =" , negative)
        
print("manhattan_k=13************************************ ****************************")
x= LogisticRegressiontfidf 
y= manhattan_k13tfidf

positive=0
negative=0
for i in range(len(x)):
    if (x[i]!=y[i]):
        if (x[i][0]==x[i][1] and y[i][0]!=y[i][1]):
            positive=positive+1
        else: 
            negative=negative+1
print("positive=",positive, "negative =" , negative)

print("MLPClassifier****************************************************************")
x= LogisticRegressiontfidf 
y= MLPClassifiertfidf

positive=0
negative=0
for i in range(len(x)):
    if (x[i]!=y[i]):
        if (x[i][0]==x[i][1] and y[i][0]!=y[i][1]):
            positive=positive+1
        else: 
            negative=negative+1
print("positive=",positive, "negative =" , negative)




print("LogisticRegression****************************************************************")
x= LogisticRegressiontfidf 
y= LogisticRegressiontfidf

positive=0
negative=0
for i in range(len(x)):
    if (x[i]!=y[i]):
        if (x[i][0]==x[i][1] and y[i][0]!=y[i][1]):
            positive=positive+1
        else: 
            negative=negative+1
print("positive=",positive, "negative =" , negative)




print("SGDClassifier****************************************************************")
x= LogisticRegressiontfidf 
y= SGDClassifiertfidf

positive=0
negative=0
for i in range(len(x)):
    if (x[i]!=y[i]):
        if (x[i][0]==x[i][1] and y[i][0]!=y[i][1]):
            positive=positive+1
        else: 
            negative=negative+1
print("positive=",positive, "negative =" , negative)



print("DecisionTreeClassifier****************************************************************")
x= LogisticRegressiontfidf 
y= DecisionTreeClassifiertfidf

positive=0
negative=0
for i in range(len(x)):
    if (x[i]!=y[i]):
        if (x[i][0]==x[i][1] and y[i][0]!=y[i][1]):
            positive=positive+1
        else: 
            negative=negative+1
print("positive=",positive, "negative =" , negative)



print("Naive****************************************************************")
x= LogisticRegressiontfidf 
y= Naivetfidf

positive=0
negative=0
for i in range(len(x)):
    if (x[i]!=y[i]):
        if (x[i][0]==x[i][1] and y[i][0]!=y[i][1]):
            positive=positive+1
        else: 
            negative=negative+1
print("positive=",positive, "negative =" , negative)



print("SVM****************************************************************")
x= LogisticRegressiontfidf
y= SVMtfidf

positive=0
negative=0
for i in range(len(x)):
    if (x[i]!=y[i]):
        if (x[i][0]==x[i][1] and y[i][0]!=y[i][1]):
            positive=positive+1
        else: 
            negative=negative+1
print("positive=",positive, "negative =" , negative)



print("RandomForest_100****************************************************************")
x= LogisticRegressiontfidf
y= RandomForest_100tfidf

positive=0
negative=0
for i in range(len(x)):
    if (x[i]!=y[i]):
        if (x[i][0]==x[i][1] and y[i][0]!=y[i][1]):
            positive=positive+1
        else: 
            negative=negative+1
print("positive=",positive, "negative =" , negative)



print("RandomForest_200****************************************************************")
x= LogisticRegressiontfidf
y= RandomForest_200tfidf

positive=0
negative=0
for i in range(len(x)):
    if (x[i]!=y[i]):
        if (x[i][0]==x[i][1] and y[i][0]!=y[i][1]):
            positive=positive+1
        else: 
            negative=negative+1
print("positive=",positive, "negative =" , negative)




print("XGBClassifier****************************************************************")
x= LogisticRegressiontfidf 
y= XGBClassifiertfidf

positive=0
negative=0
for i in range(len(x)):
    if (x[i]!=y[i]):
        if (x[i][0]==x[i][1] and y[i][0]!=y[i][1]):
            positive=positive+1
        else: 
            negative=negative+1
print("positive=",positive, "negative =" , negative)




print("BaggingClassifier****************************************************************")
x= LogisticRegressiontfidf 
y= BaggingClassifiertfidf

positive=0
negative=0
for i in range(len(x)):
    if (x[i]!=y[i]):
        if (x[i][0]==x[i][1] and y[i][0]!=y[i][1]):
            positive=positive+1
        else: 
            negative=negative+1
print("positive=",positive, "negative =" , negative)



print("AdaBoostClassifier****************************************************************")
x= LogisticRegressiontfidf
y= AdaBoostClassifiertfidf

positive=0
negative=0
for i in range(len(x)):
    if (x[i]!=y[i]):
        if (x[i][0]==x[i][1] and y[i][0]!=y[i][1]):
            positive=positive+1
        else: 
            negative=negative+1
print("positive=",positive, "negative =" , negative)





The sign test...****************************************************************
manhattan_k=5************************************ ****************************
positive= 21 negative = 11
manhattan_k=13************************************ ****************************
positive= 20 negative = 7
MLPClassifier****************************************************************
positive= 10 negative = 10
LogisticRegression****************************************************************
positive= 0 negative = 0
SGDClassifier****************************************************************
positive= 17 negative = 9
DecisionTreeClassifier****************************************************************
positive= 22 negative = 12
Naive****************************************************************
positive= 15 negative = 10
SVM****************************************************************
positive= 13 negative = 11
RandomForest_100****************************************************************
pos